In [1]:
import numpy as np
import pandas as pd
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession


In [2]:
conftfos = pyspark.SparkConf().setAll([
                                       ('spark.executor.memory','30G'),
                                       ('spark.driver.memory','30G'),
                                             
                                      ])

In [3]:
sc = SparkContext(conf=conftfos)
spark = SparkSession(sc)

## 从hdfs上下载数据

In [4]:
path = '/benchmark_data/fanxiqian/basic_data/fanxiqian_real_OriginData/'
ori_str = spark.read.parquet(path + 'ori_str') #可疑案宗表
ori_trx_new = spark.read.parquet(path + 'ori_trx_new') #交易详情表
#ori_retail_new = spark.read.parquet(path + 'ori_retail_new') #对私客户表
#ori_coporate = spark.read.parquet(path + 'ori_coporate') #对公客户表
#ori_account = spark.read.parquet(path + 'ori_account') #账户(卡)表

In [5]:
# 注册临时表
ori_str.registerTempTable('ori_str_table')
ori_trx_new.registerTempTable('ori_trx_new')

#### 添加label_flag标志，表示该trx_id有label

In [6]:
#获取label标签
#177603
str_label = spark.sql(
"""
select
    trx_id,
    max(label) as label
from ori_str_table
group by trx_id
"""
)

#### 从ori_trx_new中剔除确定label的trx_id行

In [7]:
str_label.registerTempTable('str_label') # 注册临时表
#获取不含label的交易详情数据
ori_trx_new_part = spark.sql(
"""
select
    ori.*
from ori_trx_new as ori left join str_label as str
on ori.trx_id=str.trx_id
where str.trx_id is null

"""
)

## 从ori_trx_new_part中提取用户历史交易信息

In [8]:
#sql  历史交易信息
ori_trx_new_part.registerTempTable('ori_trx_new_part') #注册临时表
ori_trx_new_history = spark.sql(
"""
select
    cust_id,
    collect_list(concat_ws('*',trx_amt, u_trx_time)) as amt_all,
    collect_list(concat_ws('*',balance_amt, u_trx_time)) as balance_amt_all,
    collect_list(u_trx_time) as u_time_all

from (
        select
            trx_id,
            cust_id,
            target_cust_id,
            flag_crdr,
            trx_amt,
            balance_amt,
            trx_date,
            trx_time,
            unix_timestamp(concat(trx_date, ' ', trx_time )) as u_trx_time
        from ori_trx_new_part


)
group by cust_id


"""

)

In [9]:
ori_trx_new_history.columns

['cust_id', 'amt_all', 'balance_amt_all', 'u_time_all']

In [10]:
#数据示例
from pyspark.sql import Row
ax = [Row(
    cust_id='cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272', 
    amt_all=['51685516.3413*1462383970', '37278805.7334*1478136108', '24521539.314*1465695904', '71990010.1128*1476955551', '74426782.4814*1470647917', '98407679.6488*1468048947', '22767199.3886*1452816239', '73542152.7001*1468547747', '42814201.0101*1472411160', '34944071.7785*1458539976', '8259153.47854*1455522621', '12820040.5433*1463771383', '12084991.6214*1462344575', '47713171.3304*1483302924', '33066934.1372*1464335852', '26870743.0974*1462083228', '22227311.7943*1467527448', '64698069.8271*1474273158', '58786177.1944*1455745939', '83526454.5286*1464452091', '67434275.4235*1482822662', '46846758.3424*1477208853', '23187348.0539*1459915257', '52819905.9342*1476343777', '94725881.2706*1476662094', '9902517.47562*1462173032', '52117986.662*1453290881', '35501044.6523*1477200953', '67937780.0158*1453689175', '38464729.5197*1455751592', '32221806.1489*1470651624', '48720587.8572*1460266258', '21780503.7954*1456249594', '84412169.3246*1466904670', '4332732.18458*1459201442', '22757413.6641*1452119971', '95229613.1116*1459259673', '43094339.8752*1474879917', '27735941.8206*1468711626', '47926725.6149*1463929366', '90273974.6591*1454002554', '1287597.54509*1478584127', '35213012.0741*1470628379', '8872697.78701*1479980717', '89392462.8639*1481557857', '21349930.4052*1474658277', '12453670.545*1468954483', '44884380.6695*1452405394', '42676013.565*1481209893', '53986218.9146*1487022197', '39372147.1568*1470537888', '34571288.8098*1462149244', '13523125.9881*1476364017', '82576397.3*1487734413', '11416235.53*1472006245', '37640513.4467*1487965733', '73988313.0295*1472426879', '31881977.8964*1459198975', '36198207.1111*1478562172', '42814201.0101*1472411160', '1673069.67023*1460750221', '6196055.5884*1482730117', '31074926.1086*1464057941', '9019859.93605*1470684073', '77940133.7905*1470406523', '22066357.0917*1477079202', '89876350.9556*1477563699', '38984712.0692*1452125095', '79437144.6464*1468773873', '84166038.8513*1486264947', '81091749.123*1481595789', '95646273.3507*1483211638', '74885386.3984*1469955065', '43114449.5081*1477504480', '55021375.2617*1462607979', '127687.382861*1472174390', '73277349.6209*1463312173', '22285775.9194*1474999838', '79449292.9874*1486363777', '60745194.6279*1461249959', '66221747.0626*1453098110', '99071328.5089*1453709033', '64047092.9596*1472444422', '58979612.0787*1486154663', '20061124.9501*1475378684', '35942685.5485*1482295243', '64000272.6781*1466408899', '85819059.5923*1482465613', '92926778.5266*1455765648', '30785508.2772*1452968455', '9764168.31372*1472280453', '76484937.5127*1479492587', '56285965.6793*1469295631', '16769520.2536*1456880895', '53362447.9035*1462111884', '63135721.313*1479560259', '92633328.4409*1461990798', '81311903.292*1459968159', '62185852.969*1472385965', '8271264.15601*1456986556', '81809215.3285*1474812541', '14873894.8426*1482056186', '96707698.2741*1481535169', '20519970.09*1485200541', '45874721.8437*1456472516', '53951994.7897*1483926341', '52695329.0583*1483307602', '12088625.1911*1462908723', '19012608.5773*1463980128', '83659543.8625*1469055418', '13895314.4421*1484472887', '17299824.005*1474702980', '93942436.6333*1474940736', '46227114.1312*1464563772', '9058460.63137*1456548289', '70386632.8369*1472987703', '35023850.9633*1483791040', '92183586.3414*1453845989', '23369111.8971*1483999357', '48216348.6326*1470958687', '2201331.6615*1473803238', '1547930.87258*1457140502', '32700961.2391*1455318216', '94988377.7147*1458271141', '70785099.01*1455171344', '24517597.5127*1475047002', '64847558.3943*1458025711', '3448294.82341*1459854002', '33256036.4577*1467003579', '68921012.4999*1472645803', '14157231.6025*1479433223', '67666635.5828*1454159939', '49465041.5145*1463998790', '37651742.3431*1468229492', '81731734.7363*1464301288', '65811699.1054*1458906241', '53305484.4619*1483199984', '21009385.7307*1480462336', '86818016.8289*1471065449', '68156549.0328*1483966148', '33373032.6089*1483544160', '44884380.6695*1452405394', '48769693.7632*1465390354', '79323044.6381*1455023121', '58959371.9219*1487759915', '45297106.6001*1474737666', '98850116.9372*1480002856', '45374060.7508*1452413435', '80307828.3775*1475234127', '77535920.4035*1464742244', '53121217.1737*1475286661', '50608814.8572*1483903896', '77662279.6552*1459862094', '2580293.08926*1461519905', '57354055.2629*1467613149', '39008493.7872*1484893770', '56532055.1486*1461313763', '42186386.6609*1470337438', '33572406.612*1468120460', '81818245.4203*1477752060', '17640572.4224*1483593350', '19352269.3244*1481796763', '78243942.2459*1483317858', '69605403.7824*1458362067', '26573458.3952*1483662118', '16824489.0814*1484394873', '90146570.3472*1474240582', '32543114.1309*1464208480', '27795840.2522*1456193229', '67289183.7481*1486953312', '83303419.2295*1486478769', '54912152.0331*1478561078', '57116141.117*1456103885', '14898286.6167*1472651110', '70669649.6657*1471886048', '38061238.4207*1462258520', '84391126.0283*1456274073', '42542169.8532*1457744397', '32553120.5028*1473144227', '12827962.7149*1467610671', '54351614.0534*1453621894', '76980067.5876*1472659350', '38885978.2684*1484414747', '18797122.9326*1455881593', '84668082.5701*1465029725', '98915480.0409*1484624891', '36236312.4212*1484082868', '5065451.5111*1482904557', '2933511.74059*1485334410', '86981549.2079*1467896655', '49822237.1391*1486697570', '54798049.4908*1474544535', '18922023.3249*1468522511', '52241997.7043*1460200360', '16664020.7691*1478830092', '70383142.36*1476829288', '12937174.1672*1473963389', '38622441.0914*1453890445', '18889096.4618*1481728579', '84726620.765*1461145974', '44569561.2167*1456510217', '26349539.3661*1473397746', '89883417.7765*1483767063', '52749870.6402*1478831086', '94701604.6009*1470390268', '89621375.6199*1485643255', '69351919.8257*1464520726', '12938394.5478*1451881008', '80579713.1659*1453457120', '7524221.03308*1464182203', '56330035.1562*1472489099', '72115746.9895*1455376451', '94809303.4559*1472050715', '47625193.9601*1481597023', '5493132.45745*1459517925', '56809730.0101*1472029624', '58058001.0788*1481036592', '33298335.2431*1453968432', '22898237.7601*1488077301', '95164413.8165*1488146854', '70947004.9759*1468251028', '72646623.4494*1471228354', '12914082.6762*1479672155', '28276139.9785*1476172029', '54666396.1775*1459173798', '22680215.3979*1487405204', '92493526.5539*1483602117', '68797564.0381*1470275318', '6567622.48453*1465449790', '96002062.2015*1470025692', '54442582.8543*1452604122', '7060522.08945*1451704165', '29935984.6231*1478674633', '16219797.2139*1482044336', '66440433.617*1455047319', '28636517.1334*1463493959', '72406971.1645*1478640603', '16755018.1592*1479031178', '988800.61217*1470402332', '66859479.93*1478794927', '46073055.1026*1464699702', '26193261.2779*1481112283', '36536951.0244*1474676527', '84174920.2457*1473130248', '85275065.279*1463958332', '19506765.0346*1483749508', '51261367.5187*1463651530', '5098452.87266*1472264628', '81507937.8899*1457273482', '74449250.0478*1460107953', '18104718.977*1464072950', '74243926.6345*1475051931', '60346073.7406*1478992492', '78781184.8652*1466215521', '26201848.3871*1457712502', '99996700.5028*1482521867', '80906210.0772*1475204260', '4452970.37851*1475300352', '77102768.8499*1486380807', '66989510.7975*1482337598', '89983261.6022*1457453427', '68855217.3995*1485403125', '10302089.0174*1487402938', '83195425.651*1471385858', '8885627.10489*1469532439', '99152798.9221*1464494966', '23427250.146*1473783403', '9732719.52478*1453912917', '52837855.4203*1476154144', '35058655.4936*1475764983', '11966447.1765*1465377155', '25743007.4215*1486845540', '24584530.7895*1467265828', '30834381.6672*1457736654', '68159630.156*1467261920', '83035399.4458*1471218839', '13792455.691*1467546952', '78396316.7779*1483586745', '63135721.313*1479560259', '64120121.0165*1460756076', '98592316.5386*1460305469', '10879824.5204*1466345863', '38179832.255*1470296530', '64257901.1901*1453513833', '66672372.1332*1472552067', '43975775.0978*1483794305', '65746146.0345*1464931071', '11416235.53*1472006245', '52959243.0948*1466590713', '69178946.6534*1461206402', '52362667.2616*1485162481', '70404525.5811*1468086059', '10123189.2319*1453343936', '66221747.0626*1453098110', '94250126.3883*1453103934', '96152548.5405*1465683117', '72879583.9435*1452578263', '4577070.20791*1480201150', '43535221.2725*1474411864', '90830777.6058*1466283123', '83525202.562*1467318996', '24485901.4663*1472858414', '45042294.2298*1469044780', '538959.635452*1485985018', '62562243.7516*1486457504', '16351061.11*1467237713', '59424955.9654*1469463781', '92369682.752*1464611701', '87078009.4944*1475684092', '98539036.7989*1480065724', '90325405.2223*1455444657', '13692128.0996*1472867294', '80747321.8054*1488293852', '6092128.34925*1456164248', '24138499.5385*1484143884', '93217619.7092*1484595730', '33210288.4316*1487233955', '58230611.2501*1476187553', '58044373.4257*1474689388', '70999518.2083*1484562265', '12237989.9971*1479182828', '22535045.9585*1477468057', '91875709.5173*1483435555', '61258337.4399*1458093093', '28758331.9539*1458080011', '65029230.1158*1469522184', '25556180.7008*1465886531', '26437305.4177*1479939874', '42278755.9806*1453299753', '37716168.7503*1456921458', '34584679.1582*1461079234', '84996984.3155*1459959727', '46020444.1945*1468422952', '69351363.8915*1474587624', '62807936.9262*1483310665', '83598740.4709*1452333329', '16068249.5953*1486202283', '32095186.135*1462665453', '1876533.92391*1454551879', '59045021.7904*1471586467', '9467039.99937*1453523098', '55997616.104*1467283468', '17738940.7876*1456535235', '56648735.5096*1472712923', '38187474.7034*1468055758', '11414596.1893*1485095913', '7787640.94809*1482013737', '50581815.2703*1468808003', '43642009.4077*1486889022', '2091402.41406*1487353203', '53411964.7782*1453201571', '79118365.6654*1452288867', '18861357.1363*1456359902', '81921715.5755*1460728454', '3013524.9544*1482274753', '33068554.5326*1481876143', '88735224.264*1452581752', '65014295.1033*1474692794', '1102868.19373*1487668202', '81779266.584*1479149677', '19687299.1482*1454917435', '85902096.7847*1459953327', '69446847.5124*1481896082', '59869502.1631*1486687096', '62999897.9102*1454650397', '44844525.4141*1483462180', '35939775.9612*1466674779', '38557045.5662*1452008988', '2100825.19281*1464753577', '88851446.6026*1488021081', '88668079.9684*1472041655', '11610614.1195*1487910366', '69652475.2933*1480659475', '99114259.5957*1453496513', '60626551.5737*1455007512', '80476562.0969*1460327808', '84597029.8061*1481291978', '98379507.5254*1487662579', '44188434.75*1484666960', '92958618.3508*1477549371', '47700536.8643*1455808865', '87914360.012*1478091117', '43293564.3053*1455921753', '93252791.8676*1459222319', '52598749.6512*1485078416', '5435525.17522*1463746160', '29728043.5953*1480983820', '34992543.6867*1463923947', '8271264.15601*1456986556', '46537994.2658*1471231013', '43888719.4553*1474586612', '45170747.5018*1473676997', '3908894.72214*1466886648', '12278854.668*1473556146', '94301014.8516*1453372768', '6132003.27616*1469883062', '47790930.7361*1462436536', '58811538.4502*1470486524', '98655135.3181*1481510794', '93306381.0205*1479738009', '82229211.0427*1474368068', '14836960.9201*1460730540', '84253882.9671*1478754471', '14773442.0928*1463838931', '53070746.7973*1481025861', '49378028.4254*1477684899', '84916466.7936*1466234067', '13571772.7939*1467780176', '76999818.143*1465621384', '29207311.8346*1472232133', '70460570.0486*1458987833', '55648937.2709*1475491203', '10604546.5787*1487877064', '50654848.4794*1477604257', '30200348.3437*1470265753', '85711097.1656*1474310976', '36058612.4287*1465050879', '70208059.5517*1468380414', '87185939.885*1467039912', '20088887.5757*1471552539', '40582605.9598*1454577825', '16422891.6353*1485836882', '39643224.4471*1471155157', '639864.139323*1458159718', '36570158.9815*1468442606', '12006871.1519*1467658286', '82145225.8064*1488294907', '77952598.7276*1484088064', '94627879.0581*1487550522', '11450713.2226*1454946135', '67274601.8158*1477327809', '96014907.1285*1469174490', '80507761.3124*1461895726', '87337125.6523*1461198205', '39920633.614*1463356807', '70999518.2083*1484562265', '1400086.44944*1457294484', '56809730.0101*1472029624', '84521655.4415*1487431318', '32863073.0628*1456252683', '60754855.4114*1453944332', '23124051.9048*1452655684', '55544549.0627*1467629313', '31440595.5336*1461773549', '73873823.6026*1468720741', '29019103.1885*1484364602', '53802954.4629*1453771251', '92740502.8344*1453945396', '68210153.2768*1469063080', '45517665.9779*1484077876', '51378541.2966*1481053585', '7702411.04282*1486974712', '13861991.2884*1467144343', '34598976.3456*1461464892', '53094624.2901*1476931897', '85358823.9106*1475687538', '57936855.0851*1456885760', '7275242.28019*1454456460', '51187450.9288*1467236668', '22614622.6204*1474733576', '54404215.7026*1473923367', '4005098.07482*1472016535', '21271880.9127*1455241006', '3114095.34428*1464957168', '63154842.9907*1481800321', '43217879.6595*1454627808', '18947357.2999*1475348880', '10697542.8259*1484736473', '46063158.1781*1456608749', '43841408.4885*1457199486', '87534606.6155*1454479886', '27135923.4568*1462979862', '18488840.28*1459896522', '54908669.859*1484182748', '52214611.1551*1469405937', '68991183.7462*1452113567', '62093151.3611*1452327342', '93454429.1207*1473432942', '18473952.3205*1478842685', '1950932.25957*1472299272', '92600939.1266*1471248360', '65919428.2646*1452975707', '30459158.7373*1452831852', '4483966.53161*1459025780', '23774179.3978*1462526398', '82332046.6694*1467988992', '40436612.9747*1465251598', '55022013.1623*1486925563', '75626862.3825*1486817398', '13386742.3384*1452346423', '48956507.8992*1470974157', '93709691.6155*1460796736', '11606952.3216*1475487386', '42171860.5684*1470830277', '57529448.393*1466124316', '99655977.4567*1462246953', '67465401.6127*1464687961', '27546081.5424*1454904515', '50799448.514*1464782630', '78977513.8639*1476158446', '99964206.3962*1464900508', '27751726.5672*1455376717', '45785640.1169*1481240542', '93959705.2037*1454902806', '66191808.7569*1470821942', '71051253.8672*1478339900', '12338365.1318*1484071939', '98649291.2477*1457732124', '21858468.2566*1461129070', '62317738.5458*1481965506', '29973122.9672*1468251877', '19954418.4449*1473624237', '8652509.79943*1455697338', '83665714.547*1480877578', '45170747.5018*1473676997', '88024762.0325*1467560493', '7430841.69868*1467310155', '40591525.1025*1473117622', '94988804.5531*1470723659', '69438869.617*1473769762', '41015720.2145*1471962101', '41277530.3176*1478287047', '40848737.1414*1474388912', '4883122.08788*1464320331', '48803933.6431*1484291921', '58807194.8961*1483606251', '77660394.6022*1471431857', '16856200.1998*1463180841', '38290100.7862*1475819131', '58058001.0788*1481036592', '47992239.7251*1476916676', '88689749.0542*1485403942', '47498430.6995*1465957423', '35725224.0345*1474553993', '21215135.0887*1462469782', '25263804.3423*1471805603', '57474678.3005*1467907854', '96983119.6445*1466636956', '53610803.7528*1468580839', '94900103.1286*1462885248', '49440869.4862*1451744398', '81811802.1502*1468325414', '87841314.4803*1460338235', '48076472.7858*1487510444', '12189698.7496*1475846389', '38564451.4254*1462627441', '2201331.6615*1473803238', '71667216.0055*1457275266', '25212007.4276*1468686693', '80916575.5304*1465230292', '89748888.555*1474147915', '28612670.8276*1477064596', '15693010.3935*1455856975', '30641498.6021*1485425870', '83311933.0628*1481334081', '8149794.38766*1480705117', '42943417.6086*1463309367', '91015745.2053*1477776660', '94090074.8855*1469040783', '58411336.0472*1477327645', '35410641.3221*1462720927', '83789762.4425*1487788525', '96826713.2759*1473030221', '66488838.0112*1472554597', '50177472.8671*1452672902', '1887921.60464*1461644791', '35776269.3599*1454331975', '33885714.1627*1476802829', '87904879.3353*1459270738', '54952333.2627*1458415505', '68030336.2248*1469111040', '33945259.1186*1455161152', '65747821.1216*1479850639', '13577786.3492*1462524869', '25102724.3822*1458478120', '65378817.6059*1479836450', '10711546.0681*1469017087', '85358823.9106*1475687538', '71933107.199*1472378164', '94970289.7693*1480337951', '88051578.9454*1467527703', '9748392.73234*1461735974', '68120.3281803*1478121095', '91326387.3053*1454252796', '69698830.0375*1469964065', '63978972.1652*1470717408', '25961614.7306*1476659308', '56199345.3918*1484887812', '41462849.4288*1469170262', '6172694.38973*1461945735', '47000600.8462*1455028846', '96489882.8482*1465353493', '99889810.6519*1471633395', '52843422.8436*1459417091', '14963066.8022*1473962513', '65001503.9453*1475947766', '81892887.4051*1486924490', '31809672.0417*1466852996', '33744387.8518*1455496864', '89721345.276*1472665022', '51295362.1996*1481349785', '54419512.5434*1475612679', '57072175.5975*1452990876', '68368923.3858*1457302840', '14323981.0923*1469831449', '49326686.8044*1457375605', '33579628.6105*1482700847', '71654727.8695*1474485651', '30259908.6*1458958989', '39823355.0431*1485267186', '29525509.6146*1485610113', '37509718.219*1480271031', '32088645.7624*1465848897', '83157211.5725*1467375041', '81158220.7453*1461019168', '66303724.5924*1464337755', '76701139.0206*1467929039', '17417485.5484*1465518865', '77934772.8426*1484242512', '46470372.5141*1460069670', '91356684.2584*1461265529', '63139029.7208*1486931588', '23722661.2212*1471365913', '74059620.6391*1478970880', '71394722.2071*1457784165', '16834855.7317*1469529922', '77456470.6405*1471555412', '95365216.5803*1458992428', '25401963.535*1477350296', '35192235.4648*1473216006', '57361206.5148*1482055414', '59647998.4178*1484327382', '17456659.9936*1466469303', '39754446.9507*1456074324', '55685747.3976*1473511477', '71482297.9363*1474791425', '54173515.0299*1454863159', '79497320.174*1468341568', '18312696.9728*1473705521', '58572592.1394*1470439810', '380239.136078*1459705100', '34455407.6843*1486470340', '13699396.8396*1487448705', '15155343.2548*1460138611', '92600939.1266*1471248360', '16918059.098*1473045439', '14794618.9342*1480665171', '18580846.9283*1477119995', '34671093.4347*1485169111', '16351191.5286*1473742453', '43530720.1303*1465654668', '50612840.5796*1454323619', '37195335.608*1478685848', '86804342.8251*1459730023', '54086589.0467*1467301884', '43711046.1809*1464590990', '74822160.4546*1482943079', '56343248.3519*1480660251', '81974798.4459*1452174336', '79038023.1869*1469714839', '57485842.2427*1457068593', '42524040.0244*1480877490', '77866870.9226*1479970552', '48751332.437*1458628866', '16453896.8318*1465713640', '88565999.0223*1472530594', '83339527.1927*1462723885', '24454543.3659*1456547588', '12088625.1911*1462908723', '21465626.2361*1474218104', '69258024.3626*1454409671', '83196434.5434*1458295554', '56430800.3671*1460436482', '96980005.3167*1457392699', '74059620.6391*1478970880', '11597429.8116*1461698419', '5784506.89201*1469506346', '87058307.0628*1469213474', '67163268.6084*1487110425', '51685516.3413*1462383970', '4369186.30225*1473996898', '49836376.3488*1486553274', '76258893.8072*1452356249', '64606568.6624*1477041119', '18889096.4618*1481728579', '35465117.1233*1464580461', '77133785.4574*1483059504', '54081837.4455*1457791328', '68913234.9132*1454029420', '25027265.9519*1463647086', '74384913.72*1455974855', '34493537.2426*1472000961', '80512439.3035*1456303924', '98878998.5586*1482706455', '95988837.468*1461180807', '65094945.2966*1472111541', '40531629.8674*1479956926', '27100679.5091*1461822377', '68063097.3402*1464214374', '91242543.9393*1474676805', '60235066.0245*1488304300', '62418517.316*1483656720', '84977563.7086*1476711830', '28497.3534258*1470360410', '5529249.78185*1456851509', '75009030.5327*1467987090', '47184400.6766*1461863665', '63409637.5047*1479497295', '33080614.1083*1466552489', '48526630.7402*1477700433', '76711329.7216*1454664482', '96503352.7831*1471812860', '68106044.0317*1471885404', '4013832.67062*1481804226', '37448047.5502*1457902945', '47133235.211*1459957054', '90048114.9864*1486584484', '83560443.7547*1459326817', '61917007.1842*1465876215', '56648735.5096*1472712923', '9296695.46453*1478674087', '3017221.46187*1456686177', '84876364.703*1452768022', '36790708.4651*1476613838', '24567182.2439*1474758670', '1280000.97539*1471445443', '72790546.4378*1457668470', '32164381.7637*1466400231', '79523832.1802*1459386771', '60355542.6556*1477633906', '44639511.4194*1452341096', '68384081.7518*1453062175', '40024808.4487*1464321870', '14247936.7799*1486623148', '26298767.3111*1453698824', '59066340.2896*1465212852', '17491383.429*1455171614', '74384913.72*1455974855', '50003784.7063*1468809304', '76899290.0157*1461531377', '78225642.8457*1479064931', '77618936.4936*1460842041', '34142562.4016*1458001110', '62645675.2022*1487851146', '40064945.4452*1478065412', '88759494.6445*1471749640', '73333279.3768*1461641937', '2774698.58305*1465257194', '79041872.4924*1458177046', '3007350.01229*1471850906', '47108062.4232*1454927567', '24502420.2242*1474059510', '98853795.5334*1461195000', '84962829.6208*1466302737', '49006731.2521*1484444914', '83067643.5096*1463240347', '22590534.09*1471172968', '1781345.14046*1453931943', '21448868.3991*1470573781', '10889775.7119*1472529656', '54422617.1288*1476901382', '69918709.5168*1459756566', '72720602.3836*1457313615', '34037977.7021*1485278432', '71652674.0599*1453932043', '6235834.58968*1456080555', '36831146.8481*1476026075', '61639111.4576*1457507542', '41799477.3027*1461263414', '1665302.13195*1484546797', '67081582.8975*1459616539', '18977827.7612*1474932788', '50612840.5796*1454323619', '33364019.0747*1484149008', '88380146.0104*1457120256', '51565722.3134*1487940164', '2901765.72506*1479964789', '35183757.0781*1475090917', '74179146.8428*1484627238', '97252738.0237*1454234791', '39812546.5182*1457331811', '97515048.2293*1472012412', '46186449.8945*1477031974', '47667191.5237*1482353102', '61939197.677*1460566109', '57270015.7757*1474898079', '21205402.4665*1474728050', '61476743.527*1484109302', '12653268.4265*1464015844', '95472586.6088*1461515316', '193139.021739*1472259903', '94725881.2706*1476662094', '25752356.5155*1455815009', '19509954.2841*1485484873', '90269909.2833*1481467631', '18889926.5731*1463594975', '62801301.87*1466918112', '10481244.1029*1477224971', '78524862.2468*1478295834', '25975500.9628*1482163658', '80567687.5546*1476595150', '65177558.2274*1461063265', '88468313.4491*1470060954', '52945427.4414*1456405136', '70220878.3538*1454399632', '93374682.427*1487884387', '20982065.0716*1460469174', '26840867.7039*1465365864', '62034363.3496*1487612106', '89889638.6682*1475390338', '45386078.5378*1488195719', '57485842.2427*1457068593', '96555164.9503*1469357843', '20792828.6589*1485790278', '35322620.4192*1477442797', '89889638.6682*1475390338', '74171797.5049*1464577672', '19822245.0838*1460126155', '26304814.5706*1477555341', '36234857.1187*1474694573', '63384544.8254*1471925648', '68052924.5281*1478748976', '85694919.7078*1454930246', '10848927.1398*1478466464', '48463896.9329*1463690516', '72781926.7227*1455349984', '13065268.0585*1455568641', '54388897.75*1465986582', '94012121.977*1479616945', '47403624.733*1486343234', '22845570.8157*1467241182', '96461109.7426*1483295487', '64939060.1841*1456123988', '68426487.5302*1483147494', '80045629.3178*1469075370', '45378467.3355*1461669026', '94204751.8592*1465897572', '95695842.3596*1471426928', '72316701.9698*1454975992', '95402826.6127*1468612967', '46838793.664*1483696071', '10647315.8024*1477776120', '34626346.9806*1479253117', '47932952.7607*1456468100', '44626475.4738*1487352159', '86824545.8863*1458053458', '98547508.9827*1474261815', '25671688.4062*1465407137', '25184576.0823*1452714322', '12173728.6025*1483431557', '50627568.3427*1477743182', '26866226.8055*1455475665', '55512375.9063*1480532726', '48346894.2106*1468916983', '84060862.915*1459066998', '22916458.6178*1478954251', '60381813.3285*1487841885', '4773448.59461*1462830407', '26866226.8055*1455475665', '79159876.798*1473050157', '10676263.9359*1483518120', '34648240.8914*1469794798', '46128230.3335*1454930090', '83010853.7464*1466936553', '93892116.2176*1452648070', '31615944.7365*1481241860', '93585664.6025*1457867513', '68125072.8879*1460918494', '73873823.6026*1468720741', '61260145.0777*1473189371', '75718215.8022*1456364297', '10446844.301*1459930249', '99421903.3723*1465486158', '41285270.023*1468644483', '77502907.3591*1468765411', '14059453.7137*1474323603', '83912092.2271*1454506025', '27759079.1342*1459876146', '95860862.1867*1463564700', '51547628.3994*1472163178', '54357595.0813*1482659850', '72419372.4649*1481321700', '92078809.2697*1484840397', '97812579.9035*1481800198', '95404505.1682*1482442799', '88602665.39*1451996117', '73103910.0751*1486732074', '67286274.8838*1478997024', '70726776.713*1470280524', '93004864.2644*1480658632', '46082740.3659*1486166613', '20231312.9804*1455388310', '2859760.54087*1487230867', '92564358.7065*1452652212', '58617448.3046*1459550471', '16056823.2382*1483648659', '65095133.9368*1485018370', '31719316.671*1455099366', '26623771.9576*1465835684', '14684465.5513*1458611875', '73589602.4739*1476770151', '7101401.95618*1463200192', '41959903.9304*1455237667', '7821583.17423*1462186275', '34433601.5827*1467808323', '66714505.9454*1463895517', '61690298.7124*1478668217', '12339374.3313*1474288205', '4169687.71922*1466907845', '50125846.2902*1485220778', '35287109.0096*1480562505', '37353248.4247*1460353341', '39177628.8805*1476767279', '41989896.2369*1452611559', '98192306.3248*1480468252', '69831512.1778*1465123048', '33277842.2887*1477374676', '29784677.0798*1475139007', '18608056.4327*1479683895', '95295090.6439*1481041422', '19636323.0812*1466239651', '52051558.6886*1487548766', '35524366.2528*1486297640', '80533879.2547*1473656406', '86952600.7389*1466666691', '77464128.5944*1473414876', '21009385.7307*1480462336', '77430075.5718*1455299230', '56966816.643*1480241858', '84676770.5928*1465086932', '38887644.1068*1451866109', '71565586.0711*1475276782', '66733373.4073*1468060836', '17524616.9566*1483167464', '66314131.6851*1458976500', '36065748.5458*1483307417', '78186354.9724*1462775721', '12320194.6217*1478032243', '292739.90253*1471917573', '75202861.0116*1459132726', '44449478.7873*1466639260', '64414445.8949*1485128005', '17004864.0015*1479465592', '65527950.9565*1457714920', '98474294.4806*1488186396', '39510901.5206*1452879203', '75297784.3061*1474706236', '82703477.6916*1487997212', '542649.872153*1456836602', '71650048.5034*1452650689', '83613115.8926*1471331774', '94284861.8769*1456680773', '64803815.5559*1467879372', '24263805.1712*1487199104', '40459776.4877*1453758649', '46043573.9764*1456882567', '44638426.8483*1456685834', '9094888.5806*1451990236', '28513414.3312*1467199181', '7094246.65713*1481124440', '35128811.8967*1460302455', '34918676.4457*1451739437', '86682279.8285*1486092109', '16704810.6932*1454779263', '67991152.0919*1473395411', '53117043.4685*1481643167', '53730283.8654*1456143816', '89253002.9162*1476901325', '58823277.2077*1462267954', '79832188.7915*1453332112', '46287826.7188*1468614918', '33578633.9998*1470155264', '73884334.436*1479407549', '58389438.0534*1477024568', '81961161.6118*1475605932', '24932501.4326*1471329387', '31833276.384*1488315093', '42913254.878*1464738437', '9901719.46042*1457675085', '50480306.9704*1453420059', '82544860.6579*1453701543', '20243782.5944*1466270950', '87314511.8732*1456811901', '34560668.6119*1479203847', '38453695.3375*1471781801', '52911539.9901*1457984559', '52819905.9342*1476343777', '59481592.2912*1460145199', '53332591.8915*1452150718', '40478313.7729*1458750578', '88099872.5826*1467208813', '99052560.3344*1456976059', '76483913.8649*1460194342', '81087443.729*1454426778', '66358000.5532*1453522238', '2604859.21856*1481560952', '88169153.3829*1463269853', '52840725.2127*1469191167', '44738134.3218*1472004874', '4315895.01239*1479743465', '11352211.0916*1464560100', '28253837.8328*1466034349', '68364746.6034*1453860255', '19960411.9745*1471473377', '60101650.3446*1486561877', '45042294.2298*1469044780', '83525259.0324*1466760845', '78202838.778*1473067590', '21542668.6178*1472881426', '10945676.2713*1466691986', '41000845.758*1487661706', '36967157.4199*1465743242', '64396165.894*1476584144', '166243.047044*1464332609', '56927769.084*1454140970', '82608474.1572*1486381413', '98547508.9827*1474261815', '54328640.3973*1480364782', '31668796.997*1466825018', '49593757.2645*1454164927', '9780636.49418*1468156206', '42174728.0488*1475182171', '27014709.8923*1454925128', '38212906.556*1461189132', '34530648.0625*1487836481', '76727371.3981*1454337334', '54710021.363*1477320523', '93122141.5462*1480809608', '84640876.7094*1470693697', '65493758.767*1480217559', '9415616.10694*1454346239', '26524647.5948*1473017955', '45472836.2517*1483097630', '1784056.51043*1451754280', '95727411.7995*1488062362', '28655461.9306*1461577921', '59735949.3315*1477469262', '2153292.90906*1465827219', '96114677.4895*1455378963', '39084444.1446*1459449075', '50837566.5866*1454014755', '79160062.1378*1475773137', '20938012.7037*1456985398', '46918399.0686*1469777097', '83087171.8644*1467007344', '92808013.6868*1472149357', '13968865.7456*1467218820', '50572018.1947*1476295065', '87970882.2185*1465616515', '62852892.5268*1470074388', '61136361.0035*1476387894', '39459540.7511*1459193502', '83637347.2206*1473241025', '55916876.072*1487974617', '74269973.5136*1476883859', '51462275.9642*1468126059', '4592220.20735*1453587326', '93913342.2395*1473506286', '74961200.2386*1466239281', '17131509.457*1464301967', '96784463.8612*1481257591', '20662392.1862*1453398547', '53114393.748*1456003191', '47586954.4082*1452334132', '55463491.5061*1475282117', '63265610.0496*1473753803', '32041049.8312*1458547049', '66848813.7005*1453661080', '44348469.0876*1465455754', '38885978.2684*1484414747', '2934523.73143*1456089984', '95848767.0183*1471539718', '41807938.3511*1484316641', '1511086.98115*1479298914', '374273.526822*1464002139', '47595057.1104*1468174720', '51985424.6788*1461221108', '62910530.5375*1463667095', '72215878.7465*1451656253', '24466325.9116*1478661741', '41381964.0117*1469269255', '87256448.157*1473437387', '47744431.6117*1455905915', '42113684.7671*1471730203', '80615046.9165*1459932043', '3980360.779*1458858064', '66356518.7711*1485350387', '74122304.1579*1479097701', '13065268.0585*1455568641', '97252738.0237*1454234791', '95945693.0363*1461452536', '99504522.9394*1462106212', '64778783.9365*1480850955', '27356376.5132*1481362743', '66842942.21*1458356486', '77900905.2077*1485879471', '29080785.7616*1472355578', '73685439.2077*1474871092', '10891560.5334*1454774360', '57033374.7588*1460315268', '73228691.9383*1470246741', '50572018.1947*1476295065', '2639718.3441*1462770053', '74786901.375*1487199223', '56896224.1122*1476792498', '36944818.3768*1464506639', '45034889.9868*1472508021', '25648056.9011*1458245776', '36787421.2173*1470298861', '85139241.299*1467760761', '68084868.1721*1465505988', '37287480.9354*1472349962', '87757946.8006*1474460213', '97473413.2739*1463334054', '70537951.2663*1462769221', '58482484.382*1464004286', '32265318.2082*1468255288', '68053221.3194*1486317191', '87670785.0181*1460389559', '26156341.0787*1460966037', '74442056.6778*1487670782', '12282478.7767*1469345557', '5448428.60566*1487114895', '41195778.998*1473105204', '3013524.9544*1482274753', '77704864.182*1484079692', '61309924.4192*1460925128', '96446287.6575*1472532804', '55157861.7095*1477720255', '87745232.4426*1455019422', '22965636.5828*1465087911', '75009030.5327*1467987090', '82808910.6338*1472591222', '18068341.8824*1468766751', '77738051.2996*1467443136', '19430975.1729*1486033074', '79192910.8061*1454229565', '3093022.47791*1459327391', '57579091.5663*1461986187', '23993436.552*1463540745', '53082033.8724*1462338390', '86981549.2079*1467896655', '10254587.691*1479176697', '55187008.057*1462276744', '51046807.6471*1474459921', '23309843.1089*1467799415', '90458298.2103*1453486849', '75939834.3654*1466332912', '49363032.9974*1479844593', '88205255.6225*1453848928', '51503696.9889*1472878317', '83800784.7501*1460851111', '4090433.3328*1456413285', '2814080.7726*1455852426', '45383040.4792*1454270358', '23956366.6788*1465723274', '60346073.7406*1478992492', '47193401.7947*1455832036', '45867509.6769*1457412365', '36031993.5043*1465718342', '40643100.6558*1470360943', '31258770.3312*1487939074', '39709094.0095*1479004749', '54130463.2744*1471771346', '99426768.8965*1458295437', '57010375.123*1471016537', '24440888.8225*1462294273', '98477127.3723*1461541886', '59553196.589*1467595107', '39165211.848*1466892793', '81010241.0091*1478138994', '17738940.7876*1456535235', '42612996.024*1481074236', '80615046.9165*1459932043', '58032188.9836*1453895459', '87661077.7611*1465315212', '94674558.6491*1466027738', '37711345.8437*1484205548', '16925105.2203*1484258313', '39586604.392*1463068973', '95018573.4784*1483874687', '25266281.5666*1470747662', '73937586.8223*1474637712', '57015951.4785*1467536479', '14994009.4493*1452635710', '96539570.9476*1462458360', '6348884.62408*1457197692', '45204396.7725*1461581835', '65370246.2963*1453605272', '18965568.7387*1471333041', '60670699.371*1475043285', '82431799.7823*1484860562', '83828336.4107*1466860466', '93402632.1275*1477512614', '166243.047044*1464332609', '39088691.4271*1461294559', '74767089.4098*1455776452', '47900370.8628*1468110372', '15416874.7064*1458405589', '25472105.4405*1484547516', '49605867.8351*1487365740', '98254222.3689*1458727899', '9333788.09613*1483430299', '88571169.7909*1460796420', '11268821.7371*1462943699', '92086637.6359*1474431780', '85667463.5086*1480053656', '14247936.7799*1486623148', '94043059.1527*1479098448', '54912152.0331*1478561078', '77353288.1388*1465921496', '5535810.16893*1487125638', '42458346.7464*1471021662', '78164230.241*1473299427', '32798939.2584*1453080014', '62509739.5207*1466301333', '86902988.0756*1458254212', '7946415.00326*1477893073', '93788086.491*1466878400', '15154756.6307*1467179948', '95087632.0865*1464219144', '60980782.3658*1455738843', '32255993.4226*1478362297', '28285901.3783*1458167732', '41353394.1607*1462303496', '1960482.6931*1454728612', '71565182.5402*1473061673', '8055073.58881*1468584001', '79137300.2995*1481857884', '15786563.2292*1471852533', '12751636.9395*1486957106', '68447760.9715*1466366160', '55379296.7031*1472225346', '11986612.1554*1479007929', '61970695.7975*1473950752', '33077499.035*1469363628', '21657488.859*1469738793', '3293808.95655*1464698367', '41249204.1466*1466569099', '10549607.7069*1468448794', '72882116.9555*1467842258', '71752723.8042*1475425901', '25986704.0953*1485071639', '38605224.8702*1464705876', '62185143.5998*1452968117', '32393272.5846*1458903709', '7523719.10614*1471942265', '80430104.6485*1458119550', '10120280.4424*1480951462', '72849245.1955*1459264890', '81818245.4203*1477752060', '66725586.312*1476087676', '12946175.7714*1478428834', '92129351.3508*1485215515', '78880100.0102*1482329788', '52765605.9716*1480315999', '20455618.2622*1453022075', '25359626.5483*1452937982', '57220075.4936*1485923174', '49751287.9131*1465909695', '73641272.176*1467244784', '53310403.0459*1478197467', '33094223.1643*1479292764', '73300322.7442*1485370767', '64350517.9108*1452536984', '16090701.9584*1468030250', '25239262.0903*1460864711', '59769214.8871*1475776898', '59746131.1138*1468378950', '35227053.4327*1487122962', '43184193.88*1472034401', '99557747.5824*1457956050', '74701564.4693*1475529729', '53906830.4562*1465423958', '98486812.2808*1458154904', '56496349.4684*1485310854', '78846176.0576*1472906706', '34517301.585*1477201748', '58734523.8282*1461860066', '35444580.2944*1482567187', '76676087.3165*1466412159', '75504962.7953*1487441646', '35880591.4282*1457525051', '15571499.1819*1476422520', '63186033.7549*1465614206', '84679747.6904*1472110936', '42997286.7679*1480719578', '35230297.5699*1452539800', '72023131.4994*1478591291', '43563373.8631*1484332344', '52927231.7901*1481484187', '37186934.0652*1485487496', '29400107.2473*1454824673', '40507076.0078*1454083118', '28452914.6865*1467415031', '62852892.5268*1470074388', '15388362.5331*1469157612', '43582977.2218*1476680831', '40367378.923*1469889307', '18815006.6069*1473681219', '14530476.9102*1457348733', '83661406.9311*1475790764', '62317605.6725*1462266365', '53117552.5337*1482409439', '47020962.3996*1487527668', '49492100.0394*1461552428', '23161687.9446*1456991938', '24624568.8824*1478133262', '25219674.1962*1473959224', '812188.184585*1474162606', '93740288.3243*1452779418', '43084217.3427*1475708832', '48932536.9851*1456754811', '2173066.91941*1457089218', '62806039.4796*1487236019', '25026603.2223*1459467502', '91687276.0389*1482830973', '98785792.988*1485377691', '23831938.2282*1453782354', '88483782.0825*1467401969', '46941047.1969*1486050359', '52749870.6402*1478831086', '32228261.1169*1477406016', '43753990.6709*1472029839', '45673971.6571*1454547014', '58732358.1422*1486534436', '83361740.2642*1465686239', '63699235.689*1476974214', '17141837.8463*1454307713', '93637430.3378*1466954696', '37763628.1433*1468228401', '69775713.8671*1471545829', '94575134.5836*1468863158', '19904898.243*1469838746', '25961614.7306*1476659308', '37422088.173*1458086796', '57388968.468*1484253457', '60484182.5679*1484414591', '88205255.6225*1453848928', '44213845.0904*1477207043', '15219405.4553*1467408338', '19748331.5024*1478178972', '48969168.4103*1472639441', '61435180.6036*1462634565', '14404936.3013*1456817576', '56035372.092*1455091782', '53354012.843*1487976639', '32431662.2421*1469363659', '87740103.7541*1459972787', '43782333.1684*1485328621', '90758827.3044*1485031171', '77940133.7905*1470406523', '67614671.0443*1471166606', '59879946.6769*1478900236', '7596344.20945*1478570492', '92606099.4332*1486700138', '35743551.8138*1452528419', '31368214.5838*1461105104', '85797316.1214*1471492142', '93773830.068*1459207091', '70867917.2915*1460940707', '36944818.3768*1464506639', '93721897.0204*1482757364', '18450107.1695*1455473336', '27222967.6116*1473974358', '69355127.7035*1473260875', '10402653.3965*1482404482', '21363913.5141*1472640946', '31173705.9005*1466966210', '11450713.2226*1454946135', '55913483.1595*1484805216', '73104832.7631*1461386614', '97658297.1788*1474638958', '16825367.2606*1483933539', '31594903.6724*1462508946', '74612902.8948*1467570445', '41025019.5384*1472291782', '40795753.4957*1479425416', '70557620.311*1484387383', '13634094.6124*1461826961', '46423891.9383*1452807355', '38387266.4398*1456148146', '47474531.5224*1473037616', '53325744.3952*1461986917', '17018454.0619*1469537154', '4209132.90087*1485643310', '40868238.7197*1456709554', '69744118.3656*1451860532', '47732414.0443*1452353837', '21681334.7592*1478678842', '92511685.3046*1462925012', '33804689.7399*1483273017', '57415590.9796*1473571235', '83146552.0452*1478441481', '78698795.06*1483090219', '7625556.70056*1465659914', '36231343.7046*1454360848', '33697965.0631*1457012287', '60419800.0257*1460692516', '5067413.60199*1476359917', '81974861.0191*1465631420', '79638285.138*1488279350', '50076973.7773*1487650200', '46730101.4*1466536991', '86804342.8251*1459730023', '49207985.0055*1457584969', '83222957.0971*1472578636', '52057294.838*1458333715', '61820440.5636*1452167835', '42211982.3416*1471818290', '11589035.0349*1485910750', '17424030.0231*1463067300', '74397667.5374*1470266996', '46439992.1694*1461530870', '63009793.5858*1478386938', '94241274.3873*1483955727', '30620519.2164*1475528811', '33216869.0204*1479128305', '37786691.5657*1486842856', '24555482.2938*1470763368', '57119838.7446*1476093110', '77422600.0821*1483791354', '91354231.2897*1460363487', '54119027.3049*1475886827', '73652993.2372*1451995791', '33791516.0603*1455292421', '60177092.4569*1478098317', '16126692.4936*1484724222', '13015825.6997*1471087427', '67057170.2638*1455768519', '38513513.5213*1481561918', '94081613.628*1461729029', '65582494.9719*1462048714', '30444311.9834*1475934588', '58733031.2969*1467049089', '61373357.057*1466270247', '79567811.3653*1452960650', '51114437.5*1472829855', '88099357.2244*1468119841', '91633272.358*1460267142', '58014440.1536*1479075772', '67275172.0499*1456607527', '49730264.5299*1467851627', '42186386.6609*1470337438', '58776514.8457*1483685771', '29025117.3717*1486420347', '66917921.3499*1461356627', '77816904.7767*1480581133', '52363458.6197*1453932661', '61639992.8444*1469423465', '16331169.4513*1463492998', '66285217.5183*1465480589', '61258337.4399*1458093093', '27968412.935*1486153663', '15380162.8348*1481681342', '39459540.7511*1459193502', '10506612.5201*1461225587', '99997313.9013*1474202301', '29540715.818*1471872151', '80482743.0303*1455937730', '75081279.8567*1468409114', '37353248.4247*1460353341', '81904457.4241*1470718154', '1784056.51043*1451754280', '94988377.7147*1458271141', '66474823.2898*1453876393', '31167721.8031*1459056940', '96081217.802*1461632921', '25891630.0799*1469764471', '71912838.4769*1471775205', '14090643.2868*1462424576', '37037827.7902*1471536483', '42312512.9045*1462750728', '44295452.6927*1459609059', '23414881.962*1488031693', '12875229.7745*1479126434', '67794895.0712*1457313892', '12088625.1911*1462908723', '57681612.7843*1486630847', '1303491.02451*1453098407', '53397747.3005*1458246166', '22085237.583*1482210470', '77431058.5994*1462896748', '46287826.7188*1468614918', '57034333.5016*1459176457', '99255992.3621*1479176236', '32058395.3621*1481759297', '1434734.59619*1458565064', '70894225.6992*1460954706', '69534657.2455*1464232257', '17386893.9957*1469023605', '86823303.8357*1454316695', '72485098.0483*1454308703', '12916987.5373*1475886884', '40531629.8674*1479956926', '84239421.5313*1485343124', '2901765.72506*1479964789', '57284706.4449*1478888218', '47035257.797*1481696067', '40974220.4788*1462930618', '93687250.4045*1469438275', '21061324.0631*1452356280', '42868796.4507*1456126595', '59460800.9351*1486094949', '14850888.0651*1471725152', '94684506.9072*1472376802', '61780067.0193*1483183042', '36224245.6699*1484860932', '82292145.1222*1453789740', '47472078.6711*1463788343', '36323672.9995*1461280113', '62587513.5801*1465479931', '73099700.6119*1456658943', '23408131.9716*1454007716', '47663962.2652*1474961540', '96707698.2741*1481535169', '94561594.9915*1456422910', '86485490.5681*1486979018', '20263017.9249*1455369581', '1406697.50858*1465708857', '96289611.3456*1467287499', '23121371.4109*1455082313', '47719123.1996*1455735272', '6365864.74411*1483074134', '16441210.2448*1456968823', '16697982.7739*1483116199', '72577433.0246*1484368300', '69153526.8297*1462987107', '45007217.027*1485893379', '98288588.824*1458739337', '18519857.2603*1464773046', '10885085.3918*1454178197', '55540540.0544*1475367226', '29297945.3538*1483962560', '41249204.1466*1466569099', '54420066.8136*1473520051', '55767529.8139*1458183851', '26585896.4574*1453278609', '57159699.63*1463485782', '53434948.3105*1454444150', '4013832.67062*1481804226', '83196434.5434*1458295554', '17504681.112*1483199238', '76906895.7798*1469476107', '97876443.9447*1479731545', '44771720.5966*1465141344', '95772711.2791*1476344014', '38782425.8148*1486736608', '27006370.5137*1486111540', '78728983.0399*1487837423', '67210601.6122*1473275925', '84100585.7904*1455277245', '43854102.7011*1464186923', '28406190.8614*1456902769', '28792797.8917*1462093134', '38264204.7066*1483088016', '18573798.453*1461426005', '52107349.1538*1459830891', '98883778.2248*1477424812', '25545817.1107*1461456397', '41234011.1116*1473514790', '51051718.234*1467770969', '73449452.789*1487100226', '6709763.61126*1465916333', '50220282.6313*1480734606', '6799538.76232*1462497767', '51111765.7015*1484503600', '86632399.2676*1452921598', '44626475.4738*1487352159', '55219294.3304*1472968298', '38597885.2249*1482997531', '20507925.7334*1485099835', '83749935.2959*1476115826', '70669649.6657*1471886048', '55221977.2488*1483277921', '41057533.2175*1477269834', '84582218.6942*1455357736', '57007.9584623*1485128752', '92954866.0069*1462815518', '26334905.6593*1462602522', '97113591.573*1461083748', '66233912.0816*1478893766', '91483252.6015*1464099685', '21221624.7179*1464643565', '59523223.33*1487539117', '33225264.0199*1482174872', '26201848.3871*1457712502', '13662429.917*1454244594', '28585043.6219*1461881529', '56150598.9243*1465549270', '9689650.27474*1462041061', '33998331.2989*1454918800', '17598736.7327*1474604198', '25524624.2114*1469234459', '75853818.1663*1482983563', '24753078.6439*1478897191', '15041907.5405*1470644537', '83551149.1643*1458836939', '64348290.5883*1466181868', '75589119.5252*1470809179', '12857186.0577*1461235999', '93069890.629*1460922116', '88117001.1358*1467278017', '24693612.4425*1452865700', '57529448.393*1466124316', '98825701.2758*1482804587', '47582568.5568*1469796800', '53349203.6338*1470497477', '41131692.6393*1473921386', '1271940.09203*1482655853', '29165806.4934*1461559689', '31802292.1331*1479795956', '82387408.2841*1467329584', '4097880.27118*1475979761', '23716893.1946*1459507985', '79849440.6445*1468574675', '28603392.642*1476664006', '15210230.2911*1460430595', '54380847.671*1472172747', '65245311.4403*1481320989', '77081850.4656*1483690926', '65737792.4915*1484094640', '70941622.7181*1459422386', '57472365.8117*1484345865', '82158631.454*1469507347', '2063218.14956*1457432565', '16722801.6335*1484299508', '2720743.73021*1472732054', '42676013.565*1481209893', '23424416.8327*1454130410', '85070530.4653*1476526588', '35822016.983*1473952196', '37735313.901*1482206548', '63989856.4601*1475010101', '39678532.4247*1463339472', '11480411.2247*1473778084', '26197890.4985*1466181736', '43357861.011*1452668562', '52533312.8403*1477153835', '33741880.1558*1485389662', '48216348.6326*1470958687', '2544360.97909*1487138107', '76369942.0101*1480835911', '50572018.1947*1476295065', '15965226.553*1476052789', '60374210.1634*1477015693', '29303232.0376*1478514041', '45113290.8542*1478749685', '53775179.152*1460140610', '94343455.7478*1487471205', '81158220.7453*1461019168', '95781605.7673*1459377896', '17865216.1694*1486550089', '23522496.5237*1459509883', '9353433.41788*1456539510', '71152511.1972*1478272515', '16887712.0036*1467395133', '69082701.5615*1455303231', '70147043.3536*1452732126', '94090074.8855*1469040783', '93958021.6108*1469839942', '44123737.3285*1457455848', '98818859.2497*1487893453', '39261003.5545*1485332353', '97087264.5925*1477788331', '67744366.8935*1453425660', '50585167.4498*1456884796', '11069864.739*1458760420', '16587148.6344*1462617888', '52840725.2127*1469191167', '78592557.0408*1471160839', '48928721.0814*1472252821', '95610387.9748*1453558558', '76275837.4266*1475518882', '98085171.9733*1456104541', '6111930.10592*1466235461', '95000641.5339*1451948169', '8188387.36424*1467832352', '47133235.211*1459957054', '77504593.6646*1476567111', '72125373.4605*1452379508', '39165436.2457*1454216853', '9103765.28874*1459515893', '74286820.4051*1466886639', '28221241.4753*1467699949', '79058186.1472*1476655837', '15321435.0967*1467643383', '46287384.9437*1461969360', '23238658.3875*1461283377', '3875426.16101*1484205875', '68368923.3858*1457302840', '49058012.3976*1457080501', '70756413.1898*1462317529', '16732431.7705*1487328431', '62249244.1568*1471461074', '32220780.0109*1456189018', '62172250.0041*1458326321', '55443396.9638*1465945073', '41842366.8493*1481104645', '18758288.5731*1457882295', '79449292.9874*1486363777', '93623698.2126*1473182148', '18317919.3522*1476030783', '18398300.9467*1466398587', '79356504.5686*1460503168', '65359810.7706*1471476882', '70324357.0652*1481871110', '68052924.5281*1478748976', '44037712.7113*1456907775', '18305178.3993*1474043245', '92819258.9376*1479915356', '80417111.9278*1479099409', '43673468.8115*1463367411', '40506124.9729*1458670547', '43459573.0016*1455784536', '59739137.6267*1486022269', '65119152.9909*1470868847', '18131967.4788*1486903320', '27896884.558*1458859746', '52162089.6102*1473831707', '96110899.5466*1470817648', '29650160.374*1479075545', '48089313.2521*1463963368', '71650048.5034*1452650689', '74122314.2945*1472574723', '52577543.3193*1465479522', '79460812.7782*1464268076', '73042882.0856*1464888282', '37389324.9343*1471441015', '78319392.016*1470868989', '29816836.3016*1487223382', '50092943.5338*1470655978', '40642960.8642*1488104805', '39993774.7617*1476095630', '46063158.1781*1456608749', '14829904.7031*1471473775', '4846808.57284*1470148718', '91968134.168*1453559946', '1785326.45646*1451687279', '57472365.8117*1484345865', '91708008.1531*1484380110', '72396771.825*1466174885', '64399117.4507*1456450203', '27220510.8491*1485117525', '72543996.4424*1458282622', '69815271.4807*1480527782', '35815993.2623*1454447518', '20265662.6637*1454253916', '33651115.6216*1467723817', '33804689.7399*1483273017', '87885272.3448*1487395937', '80827230.6503*1485398022', '32289260.5295*1476179137', '51914580.2363*1472482217', '56528643.1939*1456730665', '94476606.9243*1457930957', '57004655.2947*1475809824', '29681235.5714*1481440909', '42979989.5554*1477756260', '22949348.3739*1478660350', '49325129.0341*1484147419', '37324204.148*1452757700', '56527905.2039*1464957443', '70285139.2875*1475845026', '17704521.602*1461864636', '34774296.9394*1469883820', '39212085.4815*1461955488', '64497984.5913*1467614920', '62383773.8191*1478290849', '3157895.87085*1479182317', '48680424.2221*1456672387', '61953671.812*1465220792', '20887381.8214*1461128473', '94615405.7843*1479925704', '47595057.1104*1468174720', '84764316.9749*1485461790', '67666635.5828*1454159939', '91704220.1393*1461452865', '72949881.4229*1469362258', '65445341.2391*1457094160', '16925105.2203*1484258313', '61553662.2841*1460324653', '63510061.4099*1472089377', '7804986.47629*1469986624', '93942436.6333*1474940736', '57024467.5561*1462128933', '24910639.1158*1456089156', '74636029.3464*1482402046', '34900636.8967*1452218280', '88189597.1391*1483018139', '7552205.11129*1466694059', '71152511.1972*1478272515', '97496047.126*1469914449', '21829382.9834*1453534129', '14997111.4785*1458187487', '90681555.9483*1487330687', '11265868.683*1467766007', '99725884.6142*1461617708', '28910002.6492*1475263897', '64684716.6927*1480093079', '85189386.5137*1465248810', '76125267.5858*1485351985', '94331425.4959*1476356027', '82202952.1408*1484836565', '83385791.9095*1478002644', '54984517.3425*1454643414', '60598301.0073*1474831215', '66285217.5183*1465480589', '62937414.6107*1468155888', '63567828.5531*1471298424', '12100252.702*1484992764', '19430975.1729*1486033074', '32417901.1231*1465221086', '14268697.9253*1471670989', '37478948.3783*1487874625', '93165958.6984*1451949746', '33074117.0321*1469243596', '44870503.3099*1486347903', '59495640.9972*1475429256', '6634418.20662*1463498939', '5858963.88948*1452903894', '24083403.4882*1484660868', '47941687.1584*1479734139', '70756413.1898*1462317529', '81731734.7363*1464301288', '14205964.298*1459361883', '5159988.72267*1460348989', '64991113.7098*1485232490', '74122304.1579*1479097701', '10898422.9427*1455786985', '9685906.53547*1480670841', '43972082.7161*1474805438', '93747818.3317*1484134890', '27798447.2371*1479905531', '55507894.7629*1476477456', '50355147.2582*1468487480', '15636164.433*1485450857', '81148945.0648*1466480743', '25212007.4276*1468686693', '70151638.141*1476670552', '79709693.3935*1472329739', '1294074.29904*1468845716', '69652475.2933*1480659475', '82139867.9197*1483362949', '43854102.7011*1464186923', '72627384.6896*1454463929', '32091212.1892*1453053092', '9353433.41788*1456539510', '69931094.2813*1455204975', '98874581.5531*1469212867', '17621942.4269*1473500874', '86811463.8431*1465732568', '72529399.701*1482177480', '67111462.2576*1470611685', '68398455.581*1487595488', '98477127.3723*1461541886', '61821052.8537*1472806883', '62564140.8899*1464604014', '12909863.2328*1480284809', '52820166.3703*1473055115', '82094852.575*1457965722', '84301660.9037*1474800592', '94128214.8793*1482723552', '35046731.0191*1487649036', '83642962.0042*1473027213', '37195335.608*1478685848', '24437857.621*1480914093', '29381739.5187*1459419476', '76631974.0975*1472298379', '376484.06604*1456075049', '23689588.118*1479684322', '29034949.826*1479253511', '74153025.3951*1482889629', '61731093.0914*1482146934', '7932744.64129*1481087894', '25045816.8588*1466949406', '97515048.2293*1472012412', '18895235.187*1452531371', '8090088.39602*1472710128', '34630140.2902*1487465120', '16130431.6993*1480024461', '74595826.4361*1484624920', '27397597.5564*1468544171', '23689588.118*1479684322', '33301941.7802*1468793128', '33460609.0855*1487794550', '65114137.1238*1487848490', '75606499.036*1454053805', '27960813.6962*1456706558', '85564503.0131*1483969235', '24753078.6439*1478897191', '90343733.6696*1487226052', '12820040.5433*1463771383', '72949881.4229*1469362258', '5232309.31364*1459530014', '82334956.2892*1467708359', '30515789.4046*1458780732', '98302277.6216*1454927400', '95778534.1141*1482789027', '37278805.7334*1478136108', '50008324.7711*1487871526', '75931334.9031*1465491845', '28627138.878*1454519860', '5559825.92341*1458081844', '4520426.58401*1487490041', '41923565.1272*1457347854', '94177325.7819*1461462576', '53536184.0598*1462305196', '45046301.4495*1481220288', '28144105.9321*1463225983', '43114449.5081*1477504480', '10366027.8765*1461564862', '17386785.993*1472943217', '27599359.6395*1452280846', '48858319.3498*1471889806', '60866155.5475*1458220650', '97645126.8398*1454309437', '3586914.30921*1470014427', '83761257.0233*1455634429', '99933102.2099*1480363317', '88957215.9567*1474110631', '31698118.7637*1465484059', '485485.584361*1487595935', '20558805.9063*1481556993', '88355118.838*1460021205', '79883274.672*1456446767', '66782087.4614*1471244459', '96646083.6183*1479427490', '84385853.4245*1475314173', '97911537.3058*1465757742', '565052.683988*1455730097', '4686147.77692*1458990609', '89595728.0082*1468251621', '44453347.9476*1464413990', '51928152.316*1483113273', '90134563.5919*1453502539', '13002026.1254*1477001630', '59449905.5034*1482732848', '64309297.1269*1474224523', '5570621.33935*1487003839', '1757897.10575*1473575380', '26626505.9439*1453770702', '40860340.2411*1468684395', '91426881.4328*1470924750', '46260891.6845*1475481366', '40908576.1418*1467265045', '28221241.4753*1467699949', '60706426.7367*1455400140', '45020975.6395*1465516606', '83136987.2649*1474173392', '51947652.2609*1457357840', '97473413.2739*1463334054', '47744431.6117*1455905915', '32222515.3173*1471117663', '67055192.0593*1487032827', '71284127.9191*1470151211', '44093279.1403*1486106261', '82994410.0337*1457511120', '69540574.9179*1464162696', '92251112.0389*1474047085', '33838579.5647*1454282198', '170764.890583*1453106140', '17299824.005*1474702980', '62172250.0041*1458326321', '20630065.8588*1472905888', '56330035.1562*1472489099', '72315854.4695*1476007562', '44904523.5241*1455210973', '90123561.663*1482238749', '52632725.7413*1469541912', '31926519.0842*1474533788', '38808003.7417*1460496018', '42072929.1597*1457526215', '3013524.9544*1482274753', '60802887.8696*1483558510', '9685906.53547*1480670841', '207003.604663*1485984906', '9982426.09459*1470987518', '62545297.5214*1455862076', '74649606.1451*1486810617', '66962815.0115*1463573925', '49495291.2847*1463671986', '32863073.0628*1456252683', '32019019.782*1461344593', '15172774.4562*1478440209', '43744826.7829*1462822684', '13002026.1254*1477001630', '20052506.4654*1464382943', '88602665.39*1451996117', '77400322.5107*1454296396', '51438240.6874*1466491136', '49096983.9572*1469790518', '46105294.4611*1482100215', '29043129.1584*1479086887', '75931304.2189*1458026078', '65290852.8245*1469067541', '98219830.5049*1452420153', '85055884.1133*1465062924', '9021758.89829*1461462941', '43888719.4553*1474586612', '45970849.8201*1453224654', '89446457.1859*1466954989', '63510061.4099*1472089377', '8160560.52548*1452056349', '66250013.5987*1458205908', '85694919.7078*1454930246', '56147031.2492*1483742146', '30640819.4442*1487020342', '34665001.5475*1483954045', '97473413.2739*1463334054', '67237827.0319*1486548023', '33462505.3812*1465718696', '18899738.8345*1465992022', '40974220.4788*1462930618', '7220885.1006*1468900202', '96991967.6049*1472682691', '85944448.2941*1471793924', '25065557.914*1451910519', '35766031.5448*1459289045', '74412069.1667*1476375260', '3506591.7582*1469200422', '80647595.7813*1484106767', '44295452.6927*1459609059', '62545297.5214*1455862076', '40841735.5868*1454133166', '63059856.9764*1483090886', '94256528.4933*1456847109', '7998842.94827*1467136294', '34037977.7021*1485278432', '89554661.1754*1468128169', '13185990.2018*1474173098', '43705985.1595*1462766811', '59131204.8689*1466751331', '83849578.0189*1458277912', '45272131.1324*1479986056', '61907254.4027*1452959901', '31552151.1078*1484399374', '87018000.9549*1484460505', '2262428.99704*1457831748', '74357556.5313*1473110984', '85689094.3612*1461057913', '15636164.433*1485450857', '48463896.9329*1463690516', '92728949.336*1463951887', '4155022.64652*1475692040', '52842114.1717*1482607609', '53079535.3346*1481851997', '59242529.7667*1485974856', '66173573.6899*1454651265', '77135558.2715*1472541934', '61833295.3525*1473346941', '18370638.1937*1485241416', '95805685.6325*1486083885', '12937174.1672*1473963389', '22944728.7945*1479233299', '27714733.3137*1464380430', '47463140.5104*1471295099', '4067435.53614*1460980860', '26723861.8288*1487457638', '10745512.9295*1464225850', '81771350.0081*1475872178', '76115251.6649*1479737281', '37999953.8672*1483472562', '50939588.4691*1461740019', '501455.849764*1467076670', '22233836.7402*1473542592', '87058307.0628*1469213474', '42952132.1187*1473081961', '96014907.1285*1469174490', '73139764.7085*1464406959', '25320570.3643*1485866634', '63467327.2218*1458732780', '50526779.0658*1456185892', '47161955.764*1468888480', '96545417.1131*1476041800', '15390066.5901*1462122974', '7578576.25484*1467270962', '16093839.3541*1475498899', '66083677.4668*1476254037', '12039936.3763*1475066418', '73542152.7001*1468547747', '18663134.829*1473816335', '92817723.6389*1455717073', '27650952.439*1484848259', '15842310.2638*1476928390', '24666206.6139*1458171227', '10366027.8765*1461564862', '77298328.7422*1470608278', '95387506.8005*1469313933', '89502816.4258*1480977436', '13577786.3492*1462524869', '32067033.0322*1486262568', '63327059.7133*1470713621', '40786149.0992*1461008724', '25239262.0903*1460864711', '16733108.2028*1472323839', '7857840.47799*1453615656', '77716598.2137*1470193247', '25609866.4749*1468906794', '85536855.7402*1477456253', '42847881.4247*1457044104', '99853746.0722*1462995356', '44671317.4353*1460051329', '64002006.1799*1479780984', '30673237.0756*1459161153', '13871694.9517*1481678013', '34061892.9849*1485186803', '38574126.9816*1452174023', '78754435.5165*1486017830', '59803180.2976*1465288957', '76724620.2439*1465146307', '17681998.333*1470808377', '5520862.00177*1471795249', '80897645.8983*1465761699', '1437847.06962*1479938216', '65029230.1158*1469522184', '59663804.5575*1460749565', '62708978.4247*1465730240', '41220723.4897*1478631477', '55928371.1229*1478312591', '30303971.4507*1466794692', '53914011.7579*1488224292', '59402391.3714*1486599838', '50903384.7807*1481932993', '88051770.8975*1461424531', '81245433.9415*1455298313', '41757516.546*1463315638', '58620215.1877*1456019812', '98161903.4685*1454523984', '89007729.5606*1466681406', '32441001.8613*1463520565', '10202581.0343*1460081347', '2344304.90603*1487728355', '19663264.06*1470107660', '12264306.7092*1477460886', '50968373.1*1473159829', '32778755.1945*1482409181', '44093279.1403*1486106261', '11710526.0941*1460694180', '45046301.4495*1481220288', '71908586.3342*1469670429', '10443948.2859*1461581220', '54086589.0467*1467301884', '70979985.4783*1457782817', '55588894.2716*1486418193', '93992810.4611*1463690303', '46908303.2589*1461661691', '86207313.7284*1475319816', '24839462.7425*1455169524', '30620519.2164*1475528811', '80920210.3492*1459850388', '43046983.9284*1481388056', '8375813.11949*1460508515', '7654378.67705*1462085778', '54684866.3368*1452908307', '15323275.6307*1475062797', '14949848.9108*1479892182', '84916466.7936*1466234067', '47030293.3045*1452076829', '63139029.7208*1486931588', '63040662.9195*1465105342', '17643739.3463*1462649574', '30133397.8987*1486420690', '83800784.7501*1460851111', '11945382.9246*1485076246', '62587513.5801*1465479931', '538959.635452*1485985018', '45383040.4792*1454270358', '127687.382861*1472174390', '98152400.0264*1478441926', '7891463.54445*1481535092', '19855341.2896*1468460513', '35939775.9612*1466674779', '34056905.6237*1456247725', '34009924.8148*1456373404', '4783086.29175*1460999664', '6274813.43582*1460254622', '37390704.6577*1478155657', '88946474.9912*1482719681', '13578280.2674*1471253184', '83196434.5434*1458295554', '30524648.249*1480030204', '77585210.4914*1460274366', '47161955.764*1468888480', '87885272.3448*1487395937', '68060456.3805*1485494697', '90436188.5604*1484200087', '15116310.466*1469025060', '2253776.42771*1459130015', '42258170.0519*1467410827', '83462786.7214*1468973229', '41097907.2915*1482791532', '13276228.3782*1466431645', '266114.627659*1468479109', '41596983.439*1464397334', '83618121.5979*1470057435', '28083058.2513*1473030771', '59769214.8871*1475776898', '59100423.9179*1464720462', '3767588.23969*1487708133', '62357978.206*1454793536', '20979908.4863*1481507568', '83642962.0042*1473027213', '67759622.6224*1472508232', '43479010.1489*1487242306', '17168634.5622*1475978739', '46416801.2615*1464404644', '5307134.99828*1465493018', '74449250.0478*1460107953', '43275289.3182*1453146650', '71880594.8913*1469788511', '57005076.5993*1476825866', '46011777.507*1453679448', '30016058.6888*1469362970', '64644337.5329*1469162550', '25897165.8952*1460131809', '71042921.5879*1460798849', '75867009.5709*1456731099', '90377973.3565*1470740235', '45688607.0221*1460655865', '18520663.8363*1475712072', '11828298.3128*1455240825', '78537572.9837*1475456620', '86719826.1886*1472141310', '78604149.7842*1469374501', '46063158.1781*1456608749', '14210741.9643*1454518723', '13115591.4468*1479064809', '19352269.3244*1481796763', '85827361.7266*1454966454', '20105715.0856*1458465335', '93295907.3327*1474697295', '53804986.1216*1451940275', '75910998.2067*1480474495', '76806677.1158*1457384524', '54388897.75*1465986582', '2091402.41406*1487353203', '62830688.8084*1471333910', '21079758.1734*1462614711', '47961870.7636*1461646085', '7094904.86084*1487484256', '890790.067644*1477348152', '68888825.6841*1471067687', '84466759.6415*1462626187', '7521921.73251*1464962331', '92047290.1645*1471400650', '87185366.7938*1461418648', '28393066.1622*1488077237', '84352451.4878*1471629386', '54830722.6299*1474103560', '76153817.1581*1467755429', '18801350.5429*1487668068', '41583871.2003*1483906388', '1921405.58982*1472832641', '77464128.5944*1473414876', '55728310.4339*1455510595', '19582882.9853*1467907258', '86358958.5804*1453965348', '88950506.9977*1469084072', '11730366.5222*1462329812', '83509340.3151*1479676058', '1818505.73452*1474638207', '72616520.8854*1481395026', '58092363.2376*1453465523', '41003523.1376*1463195840', '17832039.7842*1470598239', '47054711.9801*1456729774', '88483782.0825*1467401969', '84781188.9216*1465534156', '78430616.9969*1474926477', '23722661.2212*1471365913', '76779669.1322*1488310882', '97626373.7479*1482367075', '25139387.4861*1486387308', '71498286.953*1468740968', '93829272.4246*1460310409', '23575287.6772*1462544052', '82198921.401*1469548175', '2514271.96496*1472325742', '76153817.1581*1467755429', '15455621.698*1486226049', '519251.250268*1462367103', '56045803.6089*1487588543', '84027817.7083*1460437660', '66216073.7798*1482778011', '33128899.7535*1465020548', '62748048.9967*1483742448', '76063709.1625*1465548677', '47754937.9256*1483792275', '6050407.27058*1473942706', '15945517.3687*1458052666', '17995783.241*1485485325', '39117114.0936*1473913617', '79159876.798*1473050157', '61313241.5685*1467843555', '10443948.2859*1461581220', '85024449.8561*1484765740', '59246781.8693*1466850404', '23667021.9377*1463591804', '59460800.9351*1486094949', '60936179.9818*1474421352', '7787640.94809*1482013737', '60110613.1051*1487736761', '31659127.691*1456669739', '86156950.5339*1469417431', '27001775.7996*1452125623', '90333863.4001*1453899702', '58720378.6994*1464548406', '91462114.7188*1486586442', '47963109.1009*1481894637', '1927619.16497*1472279708', '81464114.0168*1454591444', '31146582.9386*1455180106', '494012.891686*1452144735', '485485.584361*1487595935', '78592557.0408*1471160839', '61410035.3833*1457818565', '12349885.6255*1480139027', '82486534.1257*1485039954', '88117001.1358*1467278017', '87337331.7404*1468039325', '94331425.4959*1476356027', '5543281.54666*1479866485', '73982095.6148*1465828323', '41263236.7614*1483902395', '46643984.6583*1482770468', '33198065.2402*1458599033', '84407327.2408*1455428264', '75867009.5709*1456731099', '39643224.4471*1471155157', '69321600.8896*1474130843', '6778164.99582*1463939659', '64032829.3087*1484170061', '40420364.4229*1465724824', '27084278.9289*1484131680', '83106658.2271*1455443280', '59101672.7786*1460599904', '32563176.9548*1456041293', '61456857.1313*1455628630', '66333931.2259*1466363717', '95153652.9705*1452099865', '99874909.437*1474419553', '91003090.6114*1478010438', '42694988.0956*1485747626', '75789555.5137*1467160550', '85573375.2671*1487033733', '75399847.7348*1473876784', '1759411.8348*1487073712', '77866870.9226*1479970552', '37784273.9867*1475268455', '67218155.6231*1475459910', '30032763.7218*1483415949', '63822368.7671*1471333161', '33761845.3375*1451783625', '12937174.1672*1473963389', '30086606.8834*1482867859', '48790165.6763*1459795085', '88668079.9684*1472041655', '60906049.6891*1484225562', '95387506.8005*1469313933', '45047037.8887*1486206672', '50741330.7806*1486764228', '32778755.1945*1482409181', '67241130.3826*1474675300', '97134258.9163*1484652622', '5352001.79264*1475821231', '53883583.7558*1478242655', '84189361.6613*1461140170', '93725404.3491*1463884577', '18895235.187*1452531371', '90757243.8988*1462473378', '26248905.3488*1474491307', '5963391.35903*1473935098', '24785604.42*1482606347', '91223032.9074*1482854470', '17530344.9539*1487849816', '70033600.1509*1467129558', '5021689.34882*1482320511', '68053164.2*1470744941', '40551744.265*1469810362', '79316290.7014*1480616093', '39880499.7334*1454707744', '49928799.8897*1462768700', '68784082.809*1462064570', '82192030.3937*1487469419', '69901027.2489*1456998924', '20647363.714*1486717521', '59895345.4418*1472360507', '51059784.852*1464607294', '98876047.1185*1459002087', '391010.425267*1478047346', '92190843.3383*1470202138', '53023884.7692*1461118595', '90913349.947*1456204236', '62005349.7451*1474305259', '42603274.6346*1470088190', '60944446.8792*1487496057', '62509739.5207*1466301333', '66075786.9071*1459163721', '3506591.7582*1469200422', '65378817.6059*1479836450', '69490764.7747*1471232617', '13832549.3555*1467611946', '55237141.7774*1460887245', '67289183.7481*1486953312', '68459487.1115*1460349200', '81621744.3093*1453066114', '85119475.7299*1481008727', '34736475.1851*1461536951', '19904898.243*1469838746', '51255218.1165*1462568556', '21765462.5597*1476384923', '2191423.64814*1468968537', '88099743.3971*1466258497', '70979985.4783*1457782817', '71650555.0339*1477793564', '43203382.4554*1478449422', '49992325.156*1461164437', '60288992.254*1465770805', '69599113.4104*1473884628', '2580293.08926*1461519905', '27874121.9831*1481778551', '15837235.1744*1464613506', '62683313.6423*1479134768', '53349203.6338*1470497477', '4090433.3328*1456413285', '84085799.1631*1463266795', '30823165.6428*1453227295', '2953791.22336*1478397725', '26272361.1593*1483559726', '6963063.34692*1463135868', '19886578.289*1459768328', '43036822.9487*1483279463', '62141513.1136*1479286568', '96555164.9503*1469357843', '70557620.311*1484387383', '16621480.8842*1464945142', '16309079.8087*1456848431', '48737945.9612*1453335819', '91656827.8511*1476485876', '64824550.6985*1459249246', '7769047.05538*1456179693', '33744387.8518*1455496864', '71021164.7721*1462864455', '37418951.728*1484459834', '39433551.4937*1463139242', '80391303.5486*1473023788', '90458298.2103*1453486849', '41039630.9026*1478884407', '30757071.7211*1480039622', '55793631.0506*1466466544', '26480862.4608*1452615816', '2874677.63036*1457506357', '94090505.8544*1477841138', '27832758.8378*1466000617', '70162003.6514*1472546298', '38629783.8674*1456715365', '28782779.7733*1481641632', '41677532.281*1454090337', '33476270.3096*1483743149', '46207264.6073*1475312218', '2723596.25911*1459715329', '16180447.8429*1483773577', '82994410.0337*1457511120', '43609163.8926*1458827572', '92958618.3508*1477549371', '80958529.5628*1462676593', '96461109.7426*1483295487', '74391406.3312*1482688625', '48903183.346*1483976968', '86021413.1202*1475531931', '74008221.7088*1454018760', '13608328.8786*1454706140', '98169438.7543*1457427533', '79838757.8832*1462217082', '27167748.9295*1452782503', '71997782.8005*1473322569', '62367983.0876*1481010261', '61232760.2607*1487962485', '19879637.7676*1465908232', '81681775.3084*1464176629', '32447604.6158*1470421977', '99725884.6142*1461617708', '20590199.3005*1472951005', '12006871.1519*1467658286', '24083403.4882*1484660868', '52603717.7102*1463036487', '2604859.21856*1481560952', '742490.359921*1453047910', '89190616.9462*1481755730', '28305732.1842*1468722351', '39165436.2457*1454216853', '46941047.1969*1486050359', '39323094.9643*1456610511', '96338883.127*1467115887', '2248485.43819*1451982880', '75012552.0581*1456290271', '87414836.9789*1461056746', '40795753.4957*1479425416', '5853538.70996*1471150778', '54454704.8059*1473887962', '98063091.9571*1477244386', '5842393.62391*1481913382', '44823037.5073*1456495496', '60897717.7712*1483457939', '83918328.1696*1464141290', '24555482.2938*1470763368', '76502167.9157*1486799053', '28612670.8276*1477064596', '45025815.135*1468692367', '22906076.9417*1465625350', '48522517.4732*1475853267', '62386846.2126*1470815802', '30772628.0186*1471966273', '96332050.0521*1465531163', '99927464.3731*1467226839', '2257394.49903*1464744879', '8069548.26203*1453996456', '14035422.4445*1479225434', '79381643.6109*1466070618', '44771720.5966*1465141344', '95359735.4862*1474693058', '30270166.9112*1468218769', '54358870.2995*1484116638', '66806854.3786*1473922064', '76986399.5217*1452909693', '47622363.0947*1474588080', '49752964.9622*1476904047', '81087443.729*1454426778', '22555464.0175*1473273269', '10013694.0752*1452119398', '1355262.29473*1456531422', '16251288.3722*1472349289', '24981576.2328*1476496162', '63204420.3248*1461770562', '38557045.5662*1452008988', '37476950.1057*1455894400', '29123752.254*1471313561', '72969964.7622*1465603853', '93327997.6751*1481503407', '35717184.8556*1467629328', '32442407.3501*1483951020', '68996780.3373*1468592156', '39045148.2414*1465487172', '6945997.71239*1465786328', '31870928.6431*1473680142', '66733373.4073*1468060836', '61612416.5513*1458991244', '98336418.8765*1456508222', '94373571.569*1487933267', '25184181.5483*1463386161', '67317155.4321*1451614183', '91338997.9051*1459610208', '16697982.7739*1483116199', '86698033.7042*1472698013', '30054315.9582*1485472522', '24377538.6028*1483232867', '69534657.2455*1464232257', '7447659.82367*1480033002', '66051209.6705*1484490925', '24309279.1264*1484691231', '51437852.5062*1453353213', '56518473.2633*1473131133', '71912838.4769*1471775205', '59451566.7979*1457674441', '23758729.1309*1478586864', '84177463.9731*1487287296', '31342305.8646*1477519058', '93640043.8228*1469214671', '684937.359844*1456789816', '30331074.0239*1453025083', '29108457.3109*1474049418', '11150501.1912*1460623735', '16351191.5286*1473742453', '94166551.3776*1466279075', '69637397.7469*1464378903', '29868183.1073*1471195280', '69954145.2162*1470312134', '56196907.8259*1462554557', '84874805.1202*1466883479', '9474523.21624*1480823081', '30438060.2629*1464450373', '42946909.5316*1461348377', '5304917.93672*1455898140', '40218039.1444*1458360720', '92653158.913*1461191450', '28455076.7912*1457666502', '79793635.291*1452505965', '48988049.4896*1459024273', '8401090.87211*1454529324', '44046154.6848*1459779761', '49752964.9622*1476904047', '81974798.4459*1452174336', '88815875.0339*1485823067', '37174026.8509*1464377667', '27928366.2241*1484472696', '43888719.4553*1474586612', '66494422.2743*1455954995', '6481236.52573*1468793021', '31792595.9277*1454601221', '90238787.6671*1468764182', '70996399.5259*1484344849', '72528175.7496*1466140425', '47923054.2314*1485950894', '62852892.5268*1470074388', '61821052.8537*1472806883', '4785821.25449*1476939820', '60946649.5206*1461009531', '11719913.1057*1481954643', '18756738.2523*1470725539', '30315209.9978*1459674313', '82576917.7842*1459453516', '74245522.5864*1468974940', '6167160.95437*1472062498', '52123650.8393*1479013966', '25239262.0903*1460864711', '48792286.6547*1481844506', '1400086.44944*1457294484', '67810086.1013*1455696589', '99996831.0841*1488213267', '52694927.0558*1460698353', '58733031.2969*1467049089', '98146815.2421*1481905788', '99482765.5438*1475366665', '96433867.4683*1481494075', '98042638.3594*1485501803', '40314283.5756*1477797862', '71506347.7751*1456475407', '20337210.5008*1461236377', '39709094.0095*1479004749', '75820738.9417*1483681601', '48264791.2794*1474290322', '24577662.0889*1452324397', '14032745.4031*1479368891', '89513489.2677*1475426734', '907892.493004*1482503104', '25943660.7181*1455922358', '81927554.506*1479161467', '71459095.3171*1477148147', '25208910.4839*1462935809', '15154756.6307*1467179948', '99557747.5824*1457956050', '92606099.4332*1486700138', '81904457.4241*1470718154', '73962097.923*1487011118', '18656128.3102*1473689258', '98519381.6029*1467005506', '62807936.9262*1483310665', '9058460.63137*1456548289', '75561253.0436*1465111452', '31142232.8154*1485026134', '51261974.942*1485108841', '55588894.2716*1486418193', '95018573.4784*1483874687', '6486738.9165*1478028237', '94345265.2684*1469277324', '56196153.5365*1470726619', '59100423.9179*1464720462', '85802389.9602*1458710304', '46716820.8222*1477424364', '33690930.0111*1456450030', '71735660.8973*1476138972', '38864454.1403*1480594653', '44227775.6209*1487663110', '40944404.7957*1480064490', '77464128.5944*1473414876', '86485490.5681*1486979018', '19782992.4588*1458308175', '87426316.3732*1452054902', '97484538.324*1481996556', '60790043.0177*1467691249', '96696547.0994*1486260979', '20426712.2314*1474029799', '52019351.3587*1487816688', '25359956.5632*1458770552', '99184163.4955*1475568365', '32059747.2747*1460578418', '66221747.0626*1453098110', '1764485.37125*1452067003', '7900189.11372*1474220655', '3793669.13941*1464965095', '84174920.2457*1473130248', '74203636.5064*1479957041', '70581437.3037*1458752337', '82591246.7083*1457545541', '39281687.8425*1476187058', '10745512.9295*1464225850', '12944761.8248*1465355977', '84775416.5394*1475774471', '94769933.8552*1468010270', '77877921.1771*1484665770', '26272957.7525*1479358881', '49508264.9829*1472670872', '72931667.9978*1468525781', '50837566.5866*1454014755', '18551006.6092*1479539996', '65640844.9928*1484229982', '20051182.6978*1480885334', '19954418.4449*1473624237', '22949348.3739*1478660350', '44109377.7166*1453894016', '32665776.7241*1453332005', '38023266.6749*1469618846', '22227311.7943*1467527448', '43790325.904*1458783171', '4324215.89677*1452337219', '79258722.4237*1479937899', '58543111.698*1468951769', '18398300.9467*1466398587', '42450208.4339*1462357123', '69012917.5192*1454063448', '7361991.03856*1483716932', '16678175.9508*1477178866', '57579091.5663*1461986187', '23326124.9182*1484645183', '37874515.9233*1485920279', '23785438.2282*1460313931', '71229124.6774*1464105310', '96730476.8493*1486455540', '66720262.1326*1457316671', '82409273.1872*1476387933', '24630613.8571*1457452741', '17574246.6151*1478454523', '59186620.7815*1451803874', '92020905.0777*1460046738', '19230967.9982*1457313946', '65857544.5929*1458135603', '58807194.8961*1483606251', '10627699.2733*1480645408', '59300335.0514*1474434530', '72577433.0246*1484368300', '41492343.3026*1488191778', '793416.040312*1476192675', '62562243.7516*1486457504', '4209132.90087*1485643310', '38087753.3899*1477111612', '7218659.0564*1472405189', '19004579.1422*1471427864', '57344283.8655*1456677100', '52837855.4203*1476154144', '85295379.4408*1475830358', '27408985.8838*1470343699', '74391406.3312*1482688625', '19675714.4613*1477146254', '66166970.4343*1481487910', '25980622.4785*1477656371', '11076388.4648*1453274408', '38290100.7862*1475819131', '89721345.276*1472665022', '67812910.1976*1485826983', '61770511.9473*1459218006', '88995056.3617*1473225322', '77515313.9702*1459138392', '58498591.4505*1476841879', '3114095.34428*1464957168', '89525055.1045*1451714043', '83160817.1649*1479448593', '16478948.6941*1462025349', '67827774.12*1457975156', '41701590.8219*1479168238', '1712252.06702*1469510886', '91293315.5874*1479570140', '97877201.8796*1463768912', '37509718.219*1480271031', '21534912.6998*1468105565', '27895075.2892*1463434328', '76239959.1917*1486072421', '44136912.8966*1482523656', '12951493.6122*1477782362', '74836008.0319*1456864957', '28812622.6799*1464627852', '24651449.9565*1486788917', '78186082.4681*1455086878', '19498817.5148*1479133517', '53843107.0824*1454467269', '19600961.0072*1468648745', '26431450.5603*1453023061', '38852500.6947*1475868644', '14586404.1275*1480156204', '40647622.2441*1473754904', '13811651.1916*1479265488', '65103379.7835*1464141833', '4325894.53106*1483508149', '81479577.2583*1457569913', '1355262.29473*1456531422', '52936881.5592*1466984643', '21007188.6817*1465809987', '21344804.6672*1466877143', '48751332.437*1458628866', '3585621.1081*1453752395', '60884478.8719*1457147809', '87581678.8654*1458608623', '25346652.8968*1462783774', '95618016.2019*1472528626', '95988837.468*1461180807', '76522303.1255*1476069830', '93025076.2407*1478036525', '80507761.3124*1461895726', '93662609.6971*1482958826', '79548973.0369*1463970024', '21007188.6817*1465809987', '5534847.38834*1474290553', '47397505.7338*1472011806', '32035135.4213*1468526672', '20907960.9733*1485342474', '40606219.9549*1485339204', '80798066.3313*1451970969', '13501217.9997*1470699057', '39232718.1228*1456225890', '18068341.8824*1468766751', '70583260.1614*1466588750', '16163434.7567*1481533335', '36728383.3348*1483814052', '31742363.2095*1485731467', '14304427.0418*1462930748', '73413934.4895*1473269183', '84994143.5091*1481863215', '95807529.0008*1477035880', '43691178.3186*1467382183', '29635738.4535*1463817777', '431650.545298*1479012195', '57812397.5794*1482037167', '70252004.796*1479767061', '9827703.68798*1458444845', '81904457.4241*1470718154', '90792088.223*1473137080', '52816535.2059*1458121995', '98649291.2477*1457732124', '92571732.2666*1463799339', '85295379.4408*1475830358', '85014194.4721*1480649043', '44730139.764*1476600536', '71410163.6115*1453064412', '96473008.8501*1471025980', '8024136.99218*1478973572', '54478823.7366*1464862498', '56528643.1939*1456730665', '7626398.37981*1458631016', '12282478.7767*1469345557', '71910773.8536*1452634324', '40067876.687*1481483829', '41630741.5814*1483838940', '75784146.4299*1485847265', '79631388.7298*1478245803', '13328818.1112*1463046480', '38574126.9816*1452174023', '33613233.1275*1467921969', '75978285.7923*1485195012', '58815403.3993*1474554935', '75504962.7953*1487441646', '70241173.3751*1464149745', '64851585.4811*1466638115', '4461267.24732*1468194198', '25969156.6798*1487267005', '36033587.7984*1480889112', '33476270.3096*1483743149', '91571837.4276*1461896518', '53688836.5169*1476847928', '69300751.3295*1474988445', '16093839.3541*1475498899', '98339795.5973*1452566959', '74665103.1277*1484812665', '4132816.39279*1486752990', '39567583.5846*1476976337', '82367672.0902*1463249168', '32048710.712*1487543037', '21271880.9127*1455241006', '2214587.89712*1470400025', '41000845.758*1487661706', '43735917.5527*1481788597', '49404599.0455*1483706962', '31074926.1086*1464057941', '30874790.3283*1462244120', '68063652.6229*1453134142', '4829411.65399*1481661548', '13901789.949*1479881016', '87056129.8234*1467040701', '80441424.0899*1466754823', '87325089.1103*1478848078', '37691796.209*1455019389', '77118317.1065*1478941079', '10455102.2205*1471753149', '83434580.4461*1479710579', '96377117.747*1471504989', '7253033.97889*1471701643', '15209265.8675*1470921242', '60462710.9216*1472306731', '66754486.7806*1458898099', '69919049.0783*1457541566', '72486989.1729*1476973394', '78659674.5903*1464168815', '86556196.8779*1488324413', '64155426.7971*1485290854', '21220009.1953*1460177099', '50261333.0886*1480906358', '72295941.2344*1461326112', '63846269.2245*1472389759', '24957950.2853*1485567817', '8295521.53069*1485300062', '93338088.9616*1487413184', '17284584.198*1461688585', '54344825.0706*1452438952', '3157895.87085*1479182317', '23199250.6719*1478615438', '28692715.3816*1484859946', '4021755.2448*1483648227', '21452843.8223*1461369451', '79162679.201*1458049270', '72228170.8755*1481483763', '51112131.2218*1485959066', '63383817.9947*1452618393', '36634855.504*1469442492', '61922271.2489*1476121429', '61249683.4714*1472097977', '43664557.1494*1477212115', '14055873.7052*1459676512', '61598722.3986*1474913077', '43519498.0845*1468913113', '44055213.6029*1477705407', '20337210.5008*1461236377', '84088857.673*1468598043', '28612670.8276*1477064596', '19864385.7165*1487922432', '86436996.7443*1451893789', '53238066.2164*1461516477', '36471402.8199*1462824726', '82569565.4954*1487567794', '69019048.0268*1460464582', '76581417.7705*1482556639', '54358870.2995*1484116638', '19276717.0787*1484497032', '44094180.1082*1480212333', '46730101.4*1466536991', '1547930.87258*1457140502', '14837044.1845*1463196663', '89586651.9395*1453539206', '33614670.884*1462847337', '84383484.0676*1486469301', '93829272.4246*1460310409', '37479108.5803*1485858690', '4805552.26854*1460628879', '13860762.594*1478338616', '79059417.6997*1474044948', '47867049.8549*1458855814', '85272264.3126*1471640949', '69874087.0092*1455152563', '19554039.7653*1476658818', '55443396.9638*1465945073', '2458364.77027*1464606992', '62154177.5383*1472386394', '49135251.2447*1484868100', '80070574.5594*1478310663', '20630039.8755*1479872717', '92588249.926*1461321727', '54253623.4244*1478586617', '50003784.7063*1468809304', '96331830.2376*1483666591', '41829184.3193*1465657265', '39433551.4937*1463139242', '24852564.628*1480489668', '67666635.5828*1454159939', '15111425.2182*1467205953', '16163434.7567*1481533335', '20343620.4501*1487476702', '31812017.6355*1478397318', '39088691.4271*1461294559', '18782820.0417*1475007140', '61690298.7124*1478668217', '78368167.333*1483616470', '41355257.4249*1476122333', '59205428.0041*1469196956', '63840324.5325*1468539516', '80121512.5888*1486338503', '96784463.8612*1481257591', '52603717.7102*1463036487', '12426554.8656*1470474509', '73483905.8968*1455878574', '13462461.5566*1479774613', '24370385.627*1457187172', '60239779.3655*1485812245', '66725586.312*1476087676', '19230967.9982*1457313946', '57015951.4785*1467536479', '37544451.8543*1462923231', '75441017.4104*1484385450', '44844525.4141*1483462180', '67684288.8396*1454743217', '91665833.3577*1473385120', '24437857.621*1480914093', '67146699.3385*1452451397', '98589730.013*1484427757', '89047446.8489*1477060238', '65142446.256*1485219612', '29008839.1511*1451800725', '7667799.31543*1478322518', '4565429.52615*1486455275', '67744366.8935*1453425660', '30394273.6363*1480391619', '12773377.7698*1466819026', '70911654.1825*1453466660', '62061133.1132*1477804679', '4871771.66458*1475712185', '51639300.9086*1465823914', '31964337.6972*1463519243', '25387852.8614*1458466293', '40146701.1817*1483792918', '49897737.7673*1462740361', '17229648.272*1467800837', '71364109.9454*1478472189', '31482812.1293*1473925079', '10409885.2691*1478573972', '16621480.8842*1464945142', '846123.440704*1480372228', '53594289.8827*1462276549', '54146890.4139*1487806079', '27899250.422*1470613958', '77018492.5565*1482770888', '76449878.0104*1452525036', '61256713.6453*1485594597', '9914614.88123*1462438357', '92863477.2075*1481684635', '14899334.0461*1468720152', '68345464.6986*1480693228', '42518355.9369*1464391526', '9785231.36335*1486400140', '48769693.7632*1465390354', '56285965.6793*1469295631', '42582524.8049*1463910293', '83462786.7214*1468973229', '71229124.6774*1464105310', '69489026.585*1469547432', '57900795.1932*1473058837', '11589035.0349*1485910750', '35076858.9862*1465451603', '36040290.9965*1474144919', '71543673.4533*1471697833', '1876533.92391*1454551879', '66076216.799*1474400644', '8530512.89094*1478586028', '13861991.2884*1467144343', '89754383.987*1459748022', '98547508.9827*1474261815', '37145871.8236*1452660679', '556288.438267*1460668165', '11855231.9459*1464107365', '43496679.5034*1483012879', '13144837.9587*1487990857', '52107349.1538*1459830891', '56199000.3035*1476546185', '353528.405902*1482687678', '34634363.0855*1479605011', '57004655.2947*1475809824', '21020393.7951*1459590149', '79222660.8861*1457872106', '18709426.3591*1481694187', '57913239.6583*1470868307', '13057746.8499*1454955349', '12159296.4928*1475369898', '88735224.264*1452581752', '82139867.9197*1483362949', '13898208.6027*1483835663', '98170067.8122*1456520059', '34428688.5526*1451874436', '12189698.7496*1475846389', '91244913.1955*1454454649', '79038809.2999*1466617345', '13322785.4872*1485254285', '66678253.2383*1476549412', '28603392.642*1476664006', '91082623.6226*1467257908', '13256178.5435*1472761952', '16488414.1686*1481790426', '16180447.8429*1483773577', '47472100.6359*1487645551', '25633697.3953*1475558321', '20647363.714*1486717521', '64387313.3394*1467888068', '292739.90253*1471917573', '43046983.9284*1481388056', '66529159.7651*1482742588', '5560785.98604*1462832393', '74449854.5779*1455989104', '53710862.0087*1477232637', '56336875.0601*1485601049', '57388527.3024*1469564089', '58997509.2209*1487695972', '89483321.362*1487070185', '96163293.1723*1454255517', '29043129.1584*1479086887', '63938940.8456*1460746045', '93649137.9696*1459001263', '55211566.7535*1469625789', '23136171.7761*1485064244', '96802481.3421*1476028603', '18078565.9097*1453745718', '99919148.7531*1460235288', '54644092.7358*1465824617', '91624062.6638*1472983704', '27239089.2535*1466574574', '29169852.2549*1458699184', '38248665.5371*1482305492', '49307781.3142*1461515444', '21665721.65*1475763799', '33741880.1558*1485389662', '67759622.6224*1472508232', '16873843.1182*1457075383', '80473373.2385*1463436025', '746308.657928*1483990167', '46918399.0686*1469777097', '14408149.9255*1454777253', '35042001.8971*1462668333', '99037540.7753*1471750379', '71025045.6437*1487841447', '16441210.2448*1456968823', '33927585.3372*1471828669', '40420364.4229*1465724824', '69505262.3876*1488134169', '80512439.3035*1456303924', '51813883.666*1471569908', '879621.209467*1461215940', '66451221.2487*1476103846', '63327059.7133*1470713621', '92970067.961*1462402759', '71317207.7142*1462889313', '21820679.7111*1469364936', '80605630.2495*1471418985', '37652356.6136*1474871413', '28654495.1478*1474638831', '63027505.9844*1479069308', '88184050.168*1481448496', '15115416.5125*1455938890', '95255692.3149*1462579693', '88864850.2312*1453912891', '38770101.6225*1488069600', '95708578.2146*1476006192', '47822614.3545*1466665566', '10409885.2691*1478573972', '78707464.0029*1470828917', '28843350.7838*1487397357', '92504328.4382*1460988013', '14081197.29*1465201706', '93158986.804*1463118212', '89097922.2428*1463463180', '75357890.7976*1456288436', '53648942.0174*1483230759', '11937817.9374*1462316929', '46822176.3596*1458366638', '26870743.0974*1462083228', '47822614.3545*1466665566', '85964531.6554*1454888909', '36702292.0995*1467716594', '54308323.4235*1463900541', '78021971.1233*1461663548', '18852484.0632*1451735657', '96158560.1525*1473468473', '15770024.4229*1482539191', '26807253.1427*1468230199', '55731686.7835*1483700101', '80378205.0409*1457975195', '64140630.788*1486649895', '88655978.2096*1468120668', '56839124.9667*1470440858', '76153817.1581*1467755429', '80393477.5514*1460976527', '24887432.617*1459444108', '58786177.1944*1455745939', '49132426.7593*1487357829', '21499402.7734*1466054384', '36496899.8235*1458527483', '8394995.70862*1462937136', '25911839.7069*1480472652', '63057882.2181*1469580816', '97533316.8912*1473822106', '34793088.5413*1482523026', '71615142.8496*1458664283', '23831938.2282*1453782354', '62317738.5458*1481965506', '69093931.2629*1477025632', '24785604.42*1482606347', '96991967.6049*1472682691', '93614277.5331*1461293353', '20131070.9587*1484717867', '34127403.5307*1476915300', '36266580.0342*1480548587', '3811016.02964*1464462524', '36323672.9995*1461280113', '51187450.9288*1467236668', '35392302.9879*1486168809', '81816802.9958*1486474655', '29005610.7401*1477574218', '71066150.2911*1487308841', '46555903.039*1463111804', '4219511.75136*1483209771', '78922767.2587*1485667155', '77076182.1365*1488199042', '51794356.9478*1472531241', '76667859.5686*1486213428', '48528954.9606*1486819922', '46822176.3596*1458366638', '98085171.9733*1456104541', '91014446.7666*1463769945', '57433280.3225*1468402935', '29792950.6034*1457162563', '43518400.3362*1484595567', '6518282.7682*1481065419', '20373535.1657*1467747443', '37578854.5209*1455964958', '18889926.5731*1463594975', '70395310.4894*1459633073', '49822745.9455*1463553054', '59882507.6671*1468677163', '92511685.3046*1462925012', '51378541.2966*1481053585', '52435842.2306*1478548413', '64456534.7223*1456387692', '59044845.2584*1469882471', '62947334.1126*1486481907', '25065557.914*1451910519', '11110763.4738*1468685011', '43686545.0916*1475438545', '7683915.86438*1454160043', '41591658.8818*1453576375', '73449452.789*1487100226', '47520153.2827*1466870854', '60233551.7856*1451842707', '57172144.811*1454520979', '66140689.2807*1474082914', '82991595.283*1457472289', '60629312.0959*1481489311', '39999620.5*1482469284', '28562070.0888*1459221090', '13276228.3782*1466431645', '78078697.7742*1476612282', '13260548.7335*1470994137', '48000820.3826*1487362371', '20995108.2587*1477963871', '7098238.89263*1465856667', '69351363.8915*1474587624', '58617448.3046*1459550471', '71846689.3575*1466112620', '92487926.5276*1477072856', '16358828.0927*1464783859', '72485098.0483*1454308703', '97918534.2271*1459558573', '91875709.5173*1483435555', '4829085.66228*1453687587', '37970388.2558*1469984605', '13848929.0341*1472986135', '71400571.3189*1480675318', '5560699.52261*1473198293', '74486802.2976*1451905921', '28424730.1976*1477641464', '64096800.1336*1483471781', '71042921.5879*1460798849', '89883417.7765*1483767063', '79655618.361*1457755035', '35480659.1054*1473127905', '19240400.5518*1456112756', '5731817.78427*1459018458', '3105519.23952*1473095166', '62645675.2022*1487851146', '802247.861826*1458983580', '38083500.2509*1488183475', '38174045.6296*1472736532', '46235293.2842*1456846084', '38395984.6584*1452741984', '36077199.7731*1457815336', '15763665.3223*1456892618', '96128463.9571*1452287217', '50089804.6828*1456153760', '29764697.3003*1457446853', '26270751.324*1470908639', '19090910.7189*1479366260', '11845288.7671*1473725544', '40583276.3721*1474848672', '54119027.3049*1475886827', '80457037.2396*1485228569', '42791351.3882*1484077297', '94500462.6052*1461630737', '61922271.2489*1476121429', '98026745.1573*1476395285', '40140679.7399*1462784866', '75729909.1092*1477639868', '95793028.1235*1470435965', '95229613.1116*1459259673', '9103765.28874*1459515893', '18226853.6629*1460937470', '63234941.2813*1463261678', '69314376.4055*1452945101', '56821311.7823*1469278271', '13809480.0186*1465798762', '19658150.1642*1484524702', '56841625.1852*1483592994', '68264649.1769*1458674135', '56103291.1772*1474714536', '7505874.96232*1484115841', '18543535.3848*1454059035', '99360999.3226*1459451772', '18226853.6629*1460937470', '80499560.687*1481950366', '45285013.4123*1472454861', '39586604.392*1463068973', '15932960.687*1477186673', '11391018.97*1478735326', '6634418.20662*1463498939', '74860439.7957*1474100847', '42999753.1455*1476968773', '64713715.5689*1464085772', '1100981.62472*1476063173', '85975276.2964*1468715550', '50833564.6885*1457760321', '13168459.644*1472820449', '41388063.5249*1470195597', '1398104.61407*1462847179', '14850697.1996*1470552236', '47325015.1493*1468877045', '78759957.8396*1453260668', '96331830.2376*1483666591', '59879946.6769*1478900236', '80818708.2571*1483283112', '22343067.3354*1459463363', '72228170.8755*1481483763', '75933896.5245*1475883524', '5065989.47875*1478281513', '43778658.8611*1487774199', '42783804.6238*1454805916', '93111655.2507*1454787953', '35811865.3026*1466793084', '65360512.271*1476540306', '24083403.4882*1484660868', '86874042.1862*1474003112', '27599359.6395*1452280846', '99578794.8656*1460893998', '50654848.4794*1477604257', '14210741.9643*1454518723', '79714105.0091*1478056851', '56463250.8344*1484660064', '41543590.3855*1471951187', '54422617.1288*1476901382', '16692222.1752*1464850969', '54269222.6225*1469451065', '7994716.35401*1452507859', '67794088.4634*1460325884', '68991183.7462*1452113567', '75528832.8663*1457278468', '26521879.785*1478275253', '93438440.475*1458878665', '23789951.2416*1470414049', '31274630.9343*1465472691', '42126421.8802*1483553720', '65464061.5569*1461837730', '28062657.9935*1455077902', '43887510.3363*1476328295', '50109793.0804*1454116165', '58608970.6599*1479533685', '77632260.7721*1455608978', '35697254.9312*1476258070', '64250912.2784*1462993430', '44844525.4141*1483462180', '25219207.4749*1460197501', '52347749.1586*1488225550', '49494382.4575*1463731487', '37324204.148*1452757700', '78133866.0753*1453917717', '15028044.4587*1465160327', '30854169.5889*1487270433', '41746964.1804*1483694077', '61348978.847*1473231598', '14032745.4031*1479368891', '45286478.4652*1478242046', '64653117.5492*1470270544', '20692179.0102*1459298982', '94701998.632*1467749667', '90706441.8902*1475535991', '71566885.8567*1458417780', '87480876.8148*1477638493', '47926843.0544*1484756668', '10582203.5258*1460088088', '57813969.3221*1462598541', '1716172.8979*1483280629', '96447057.0814*1460449728', '78290672.2877*1453560825', '90983439.7793*1459049036', '64726925.4378*1475860032', '3506591.7582*1469200422', '744240.689642*1467871746', '63955750.1406*1474939345', '26004182.4353*1472897154', '6017121.06207*1463751606', '38395984.6584*1452741984', '7069364.33886*1465822242', '4019610.52728*1454875861', '88568980.8244*1468928139', '24596195.8833*1476950986', '64248642.9178*1468954112', '67332416.9577*1485381356', '85977140.7472*1468036345', '78762924.7879*1481258310', '71714522.3957*1479346611', '99207657.7944*1454706673', '99150597.5808*1475444036', '10131397.2978*1473139082', '92740502.8344*1453945396', '98746948.6687*1465297971', '14812228.1478*1471386487', '63917403.3095*1455712092', '52688396.6452*1454955816', '45674713.775*1481122971', '51987541.052*1479245744', '53198231.4477*1461272340', '94892841.2005*1484705919', '81952303.2046*1463392107', '47986835.9358*1478396539', '60169629.676*1458382239', '76143608.7713*1453507345', '69795224.344*1464679990', '95526773.1241*1468144898', '9901719.46042*1457675085', '41151399.3234*1487567795', '30698916.6255*1466485105', '65537669.1803*1487890514', '13926276.6749*1465333929', '71603937.3363*1458764822', '53721252.5506*1471154090', '31695175.3193*1481081215', '55700809.3233*1472742490', '51271689.6717*1478979896', '82170803.157*1452804298', '17939596.6643*1464087228', '78846176.0576*1472906706', '97441657.7969*1462531839', '33225494.9648*1484091996', '26505052.1266*1477995168', '88762839.2633*1461707118', '90420199.3478*1456252402', '50821854.6413*1472019055', '50355147.2582*1468487480', '82410655.5795*1486972113', '33578633.9998*1470155264', '6070044.8611*1476538054', '21866094.4626*1472457984', '39993774.7617*1476095630', '32801218.2848*1487668949', '80419186.2209*1487271551', '51526925.1274*1457562150', '82387408.2841*1467329584', '37786691.5657*1486842856', '29310391.5795*1462834297', '79709693.3935*1472329739', '45599355.9552*1467804760', '49923233.3895*1472504020', '81721308.4674*1463169313', '49644665.2486*1459585568', '1532849.85046*1477305295', '24319820.5297*1468232872', '99919148.7531*1460235288', '83849578.0189*1458277912', '63533.0879987*1477733736', '48723282.0569*1465190897', '89908485.0136*1465110753', '90947545.1984*1455068143', '96660095.1832*1467573961', '97786160.9476*1482062953', '56370476.5509*1463712632', '59194990.8616*1487568097', '12378857.1214*1475797134', '95164413.8165*1488146854', '90202742.394*1485192466', '46287826.7188*1468614918', '69505262.3876*1488134169', '34052047.712*1464257722', '84876364.703*1452768022', '51985424.6788*1461221108', '42907332.5655*1468223932', '43363610.5017*1451762811', '58230611.2501*1476187553', '33614670.884*1462847337', '41674645.1497*1466184151', '82540242.401*1465536516', '22624244.6696*1466580876', '37059646.5162*1470607571', '7741349.81068*1475708899', '1268818.26254*1470293286', '63546703.8362*1468040427', '78785969.436*1452929465', '76032488.9444*1464616225', '76852312.1055*1483458864', '34771655.2568*1456272370', '27220432.2075*1463625875', '29477562.36*1456569587', '89921449.0799*1458579599', '96634082.4245*1468721049', '68823037.675*1481127920', '38458215.2299*1474024282', '92094882.7712*1467480272', '67563362.2056*1482010826', '54380847.671*1472172747', '93606424.2766*1476237351', '51092006.6712*1461960789', '71687699.038*1478842663', '10440294.1531*1473337341', '27222967.6116*1473974358', '84352451.4878*1471629386', '80944531.4765*1470691069', '88625466.6533*1455045975', '41999515.4055*1465533840', '73208226.692*1470673729', '13505045.0633*1480574561', '7879223.26966*1472142361', '34094349.7492*1488142338', '36739961.4861*1469463735', '96053007.1311*1461110843', '98930289.4108*1474672445', '82253699.5251*1482716886', '86844484.4445*1462164203', '41433562.0202*1453690171', '14797228.3711*1453676312', '86271163.9406*1474420426', '22590741.2468*1484891021', '49345624.1275*1462780120', '84799509.8703*1483983032', '62806039.4796*1487236019', '27541354.082*1484636963', '63257713.3861*1462573466', '86941290.3878*1451858456', '3212708.19704*1468631478', '75399847.7348*1473876784', '85517241.1638*1467121831', '43728704.9148*1453517484', '30228562.179*1455685882', '15210230.2911*1460430595', '52275064.9052*1455659233', '40619725.1578*1486317003', '27135095.8465*1464623470', '1759411.8348*1487073712', '44135592.3045*1484945175', '35598990.573*1472817387', '95330342.0981*1478147432', '32870410.1805*1487599699', '61777224.5434*1466812943', '11867264.5719*1479304008', '70658281.6382*1459613912', '20030410.5984*1456510080', '22555464.0175*1473273269', '88051578.9454*1467527703', '44615817.8495*1472837555', '48945435.212*1485720392', '31719316.671*1455099366', '76245142.5076*1455384453', '84383484.0676*1486469301', '51502956.5587*1456196338', '28607382.8313*1474791853', '17129821.3593*1466053432', '24378304.3267*1485370228', '49923233.3895*1472504020', '23835259.9241*1467692082', '31044974.8462*1481478070', '91365928.4374*1472335071', '45114421.9998*1465173985', '28367780.7906*1453084155', '45603647.9129*1475322717', '88483782.0825*1467401969', '64120121.0165*1460756076', '84582218.6942*1455357736', '89075614.0427*1460440296', '93418266.0823*1454189142', '55945548.7386*1484150578', '35598990.573*1472817387', '17763547.7774*1465890197', '3415742.54978*1457131177', '88054133.9195*1452273534', '26107859.9758*1480988378', '51125324.7342*1451997801', '13348469.1917*1476175856', '76063709.1625*1465548677', '77431058.5994*1462896748', '47932952.7607*1456468100', '9439599.56143*1452753019', '93160763.9415*1453441705', '88744566.5977*1465233237', '19489050.4317*1460382868', '74185707.8621*1471273805', '43094339.8752*1474879917', '84887158.2245*1456081366', '48022253.9107*1455276167', '81593941.2938*1472161357', '77425028.2301*1456347059', '8159296.17321*1483015754', '33280590.3023*1482244068', '38724811.1578*1479708247', '66457742.7195*1474614756', '18084850.9933*1483841652', '83659543.8625*1469055418', '47678782.5555*1456486196', '22892222.3629*1475206295', '66525305.7049*1463851768', '88913695.8922*1484935659', '63836428.2831*1468168005', '44904523.5241*1455210973', '40700568.8619*1473717103', '12909863.2328*1480284809', '65251854.9104*1485688176', '70147043.3536*1452732126', '251059.021859*1464953573', '90425806.3499*1485661134', '67023196.4003*1465346303', '78659674.5903*1464168815', '78168762.7045*1465967414', '44052591.0667*1466653759', '52695329.0583*1483307602', '44215815.05*1452830255', '70348898.6148*1454140557', '98650616.9274*1473845147', '29222333.0249*1461960956', '69744491.8368*1466579245', '76154642.8273*1487818374', '54312721.0133*1483641070', '34773171.6984*1459409801', '31049284.8646*1454836268', '59402391.3714*1486599838', '1547930.87258*1457140502', '87626440.4947*1454603259', '60883536.0808*1463958097', '13094614.2534*1487709102', '84506729.125*1481567877', '76659164.4521*1464757153', '55944885.2546*1464288667', '88433168.8398*1454967030', '32801218.2848*1487668949', '59882507.6671*1468677163', '5919649.47767*1459557175', '30618608.7369*1480824195', '14672659.4835*1482570962', '67494111.4113*1484252915', '75130263.5757*1463231993', '60020155.6608*1474425409', '44348469.0876*1465455754', '93317224.4214*1459090944', '25180039.0099*1462573393', '10949794.5833*1453738708', '81262696.0682*1454594623', '49593757.2645*1454164927', '88548479.6067*1483395846', '72181688.4798*1453822041', '88556942.6139*1458221033', '53445771.3525*1469858062', '16093839.3541*1475498899', '75040669.4381*1456786777', '61166103.3074*1474438429', '84004582.9048*1484754101', '50307438.4342*1458484304', '94664806.0332*1483233028', '99360999.3226*1459451772', '33209952.8525*1484216773', '61927303.9094*1473089822', '14798632.4394*1478613090', '82486733.3197*1465842707', '26272957.7525*1479358881', '59082869.842*1475369025', '61528400.6415*1482472677', '29914473.4879*1453610104', '48079325.7769*1470591744', '51156357.9857*1487079590', '90098367.1549*1483750295', '70320243.9815*1478834027', '42847881.4247*1457044104', '55945548.7386*1484150578', '85977140.7472*1468036345', '12474115.3099*1461221489', '68753334.5513*1462563432', '34679800.2046*1481469737', '91092044.9186*1485667426', '25481179.5287*1471855341', '18895235.187*1452531371', '84639957.0473*1478279161', '70863821.2466*1485296699', '16847055.3516*1465136534', '69080660.3791*1464344767', '40482061.0311*1480203763', '87569590.6083*1472232329', '41026384.3511*1464934763', '82448346.6613*1470231906', '44149425.3391*1457166548', '68185662.3577*1488195323', '10275515.5466*1455886292', '72821846.3687*1473425727', '41388063.5249*1470195597', '73988313.0295*1472426879', '59663804.5575*1460749565', '47058951.2527*1463168740', '94977146.578*1476231115', '61780067.0193*1483183042', '69455864.6119*1455758327', '82280051.203*1484397073', '30068371.9069*1481869012', '8596353.6921*1464277982', '36451791.7306*1479378400', '56918067.5109*1463825009', '66525305.7049*1463851768', '57913239.6583*1470868307', '77746472.3185*1470530088', '49505749.2612*1456559381', '26555529.4625*1466513720', '41833177.276*1483384043', '34865281.6999*1460053950', '67098419.6788*1486285764', '98675355.1842*1456305443', '72642053.5227*1475841735', '13636390.295*1485393084', '64759636.4461*1470239313', '68666092.2455*1476380636', '76363739.9439*1456684415', '66209319.6528*1462917020', '17644708.4033*1466750935', '95618016.2019*1472528626', '50601968.6887*1457808504', '67428560.4604*1479420626', '31590076.4251*1460350385', '98499845.2113*1476678185', '112917.323591*1463914113', '97175388.4172*1465171671', '26521879.785*1478275253', '6674593.16088*1454131563', '55100492.9424*1475877035', '86824545.8863*1458053458', '9019859.93605*1470684073', '43585120.8481*1465293941', '58434054.5128*1459399708', '89097922.2428*1463463180', '10947684.457*1473652085', '51114437.5*1472829855', '99145770.4823*1475746064', '75939834.3654*1466332912', '46790.7866806*1480045530', '47730652.1818*1485291631', '62056909.8021*1469734319', '48228385.3959*1451924664', '63226702.759*1468763523', '44853731.4818*1478504455', '67856013.0292*1455825453', '57901373.2937*1477826210', '3059130.55321*1477389423', '73252468.997*1472595128', '15670968.3853*1470234999', '63528996.9782*1476787498', '94988804.5531*1470723659', '82583852.0655*1466112214', '16170211.5471*1463930382', '36225362.0369*1484775074', '21215135.0887*1462469782', '95945693.0363*1461452536', '55472097.2799*1481642413', '69012917.5192*1454063448', '76728970.6535*1478132573', '76710257.9317*1456057928', '38424873.41*1484451054', '38977686.1417*1459416969', '14205964.298*1459361883', '93040954.3405*1482245788', '5332660.52006*1474818816', '23745790.3635*1480940841', '87337125.6523*1461198205', '41398978.3155*1482823977', '44638426.8483*1456685834', '66752832.7113*1486759164', '92099009.3327*1458904883', '684937.359844*1456789816', '39110743.0619*1473117614', '46064236.2942*1468596472', '34679800.2046*1481469737', '53397747.3005*1458246166', '56046243.1899*1487450336', '81401220.5995*1483313037', '9086929.30645*1480986308', '20792828.6589*1485790278', '85042216.5697*1471737456', '53573937.7379*1454194033', '83385791.9095*1478002644', '33509829.5008*1480723382', '64559775.0273*1479258101', '22088098.7264*1462888686', '96371320.1911*1479637413', '3937027.19135*1454707721', '96296866.4035*1464357264', '6887819.81127*1453482876', '47463140.5104*1471295099', '42053605.9571*1481336532', '86597378.1827*1464844011', '49692152.6412*1463417701', '10183919.6273*1454307056', '79373502.4825*1470030793', '70996399.5259*1484344849', '39969385.4006*1478568563', '28570179.7281*1470799684', '25184576.0823*1452714322', '30855320.3532*1485877274', '45215089.3753*1461308718', '96110899.5466*1470817648', '19194452.9873*1453709300', '6218299.05988*1480857510', '58135160.0093*1456093211', '95267182.3294*1460813044', '56668179.5014*1464506671', '60919126.3495*1482876227', '55021375.2617*1462607979', '80609487.8879*1487972829', '21866094.4626*1472457984', '14872488.1535*1460948113', '67794895.0712*1457313892', '61888769.5445*1482110701', '27874934.8568*1481569469', '87778597.9743*1472689910', '22071756.4796*1470645299', '20480509.2918*1468231242', '51114437.5*1472829855', '99792941.635*1475050191', '78846176.0576*1472906706', '76449878.0104*1452525036', '91263606.5386*1474403822', '4155022.64652*1475692040', '95315762.6604*1453899785', '91900880.0579*1483357755', '9822371.21094*1487059955', '20507925.7334*1485099835', '6905548.52172*1451871549', '4398737.50649*1463757050', '72396564.9124*1474570798', '33697965.0631*1457012287', '56532055.1486*1461313763', '28792797.8917*1462093134', '86067736.4486*1468376643', '96159734.1342*1468049263', '83068592.8104*1465902329', '71483512.7962*1468054396', '17029278.0919*1480527352', '71021895.9396*1484830064', '58675099.9726*1479416187', '28758331.9539*1458080011', '31350561.8864*1457386756', '71749897.3978*1486796862', '39281205.5729*1477153134', '85274970.6388*1453008152', '97281699.5543*1467279034', '181277.615938*1484787732', '44155936.2615*1473020118', '95772711.2791*1476344014', '99037540.7753*1471750379', '23001034.883*1458328457', '60968426.8273*1474737585', '56880867.6539*1455473023', '11898532.9539*1459507836', '85889493.9169*1484156958', '78431824.7141*1462697943', '64092973.437*1475247509', '36323672.9995*1461280113', '53121438.5395*1473782346', '63948962.261*1461119237', '70101863.3828*1455954519', '47941127.2632*1476074447', '32764877.3682*1465308903', '26000744.2419*1483538932', '50610010.3833*1486267283', '29207311.8346*1472232133', '79183528.8396*1454253293', '87256448.157*1473437387', '93687250.4045*1469438275', '82808910.6338*1472591222', '38875505.6026*1470719916', '77102768.8499*1486380807', '35329579.7095*1456713211', '80121512.5888*1486338503', '66382840.9166*1454029730', '29484276.0641*1481718567', '49651347.6619*1456704274', '19904898.243*1469838746', '38684455.712*1471624305', '57010375.123*1471016537', '60459376.4863*1459501059', '83375029.0883*1481299079', '28163068.086*1461056062', '34384697.6359*1453935267', '60502403.2895*1471512419', '48784280.0488*1487764170', '67997052.3513*1464398460', '27222967.6116*1473974358', '16417900.6199*1465268587', '82070993.1936*1470973030', '39725008.6526*1479379036', '79199565.134*1478825015', '88548038.6548*1485139760', '89201839.4406*1454896006', '62112387.3591*1478970530', '68745442.1838*1484666506', '19956821.7656*1480851082', '70934910.121*1481722944', '14684465.5513*1458611875', '59700383.7715*1471320556', '34771655.2568*1456272370', '43104517.0195*1481034516', '30557299.3895*1465674298', '80476562.0969*1460327808', '55453818.6348*1455945430', '98042638.3594*1485501803', '90720473.1561*1484335927', '20350011.4469*1458297813', '80867877.5995*1457866259', '86751306.049*1488057763', '70879620.707*1484431565', '3324350.4493*1452202758', '57672419.0415*1485021850', '86336928.3097*1466012588', '42487558.0839*1471628037', '41630741.5814*1483838940', '24881713.9912*1486535133', '53573937.7379*1454194033', '7500825.85954*1456496268', '94301014.8516*1453372768', '32771315.8464*1470711975', '64554414.6023*1481614607', '63009793.5858*1478386938', '10471521.9873*1480666059', '29320345.6979*1458101547', '62317605.6725*1462266365', '28612670.8276*1477064596', '78604149.7842*1469374501', '72068911.655*1460698066', '92740502.8344*1453945396', '49800164.2324*1458463034', '35815993.2623*1454447518', '56966816.643*1480241858', '37019451.3128*1479130129', '36663874.586*1477435190', '12023059.4*1482402510', '18311510.5631*1458666607', '16841909.688*1460522647', '7702882.81919*1475331741', '80648854.352*1479995668', '21221624.7179*1464643565', '34347729.3319*1459125454', '96965739.6331*1479687734', '13302899.658*1475584589', '38252484.3048*1485014311', '820029.17813*1476372193', '59956306.3611*1477788135', '66530456.9526*1460996884', '75901441.2274*1477502384', '83925414.05*1486414770', '75202861.0116*1459132726', '48904724.0021*1476986933', '31572322.3294*1485251334', '75286154.7372*1459867139', '85936025.8004*1465735775', '67321762.1391*1475631812', '60491602.5134*1488070752', '10549607.7069*1468448794', '51529923.5801*1488155202', '27421263.6122*1466467820', '31353195.5415*1481539497', '43902157.9446*1477079136', '63334256.3119*1473360715', '86356511.4002*1459556763', '83699747.0656*1459890269', '73374947.283*1462270770', '48932536.9851*1456754811', '74426170.6358*1461194102', '43888173.3352*1469980972', '1197406.69281*1486780010', '74792124.1155*1462542960', '29169852.2549*1458699184', '95453838.3567*1454980051', '51261974.942*1485108841', '15995589.4489*1457390686', '63840324.5325*1468539516', '29657536.0118*1456403249', '27076725.827*1477627246', '80562384.7385*1465035053', '53986218.9146*1487022197', '65101798.4698*1458595326', '37165041.0414*1455458503', '53522605.7816*1472028589', '94425330.0828*1459779776', '11520506.4185*1458938574', '60841142.9259*1465241988', '29363060.623*1457956584', '84317837.6191*1474288100', '27202107.6778*1473060700', '24371687.9041*1480396421', '4764975.80551*1453056246', '24799942.1454*1486244729', '82203308.8411*1475580821', '93771542.757*1477499632', '97639160.3722*1456297436', '16420945.0035*1460253323', '16453896.8318*1465713640', '36266580.0342*1480548587', '91633272.358*1460267142', '71009202.8773*1457755659', '42666717.5521*1468797530', '55579297.3975*1460389055', '3382031.07593*1471356039', '33998331.2989*1454918800', '21244385.2357*1466015085', '74841885.4333*1461649645', '39893834.1453*1485773530', '54738631.3399*1478363323', '79567811.3653*1452960650', '77102768.8499*1486380807', '2819459.21409*1470276920', '47108062.4232*1454927567', '95616218.122*1463150244', '18721749.0222*1451906236', '7664211.2874*1487480761', '56918854.2522*1475264050', '26396436.8041*1480114729', '65094945.2966*1472111541', '2874677.63036*1457506357', '84777697.9589*1469990335', '6887819.81127*1453482876', '61739053.9721*1475455035', '46678352.2899*1482282603', '53311750.6415*1463165993', '32697188.4341*1479185862', '87343505.9612*1454901281', '6117331.93398*1464037966', '72228170.8755*1481483763', '16732431.7705*1487328431', '86211542.1722*1476802368', '20286956.4*1479808616', '53305484.4619*1483199984', '68875165.4151*1461312458', '38869401.6999*1453563835', '36260464.4731*1486341583', '37446327.208*1472923272', '97014833.7181*1486413684', '87059482.2615*1487816751', '63160083.5015*1482784364', '64235315.4555*1486347717', '30182473.6313*1481161428', '5065451.5111*1482904557', '68753334.5513*1462563432', '3267858.47962*1461227126', '60583872.1436*1462184742', '59496564.2171*1486978079', '13617588.1601*1481407912', '25210517.2994*1478961978', '69811188.4986*1455389623', '13125999.4211*1482744908', '81839460.6066*1455162773', '17339177.4432*1479807719', '57033374.7588*1460315268', '29697226.9722*1475686709', '16453896.8318*1465713640', '75136732.2871*1468763211', '2085412.71571*1473276222', '89921449.0799*1458579599', '35944553.3003*1452425880', '21713615.5965*1462794732', '34377186.4645*1454686405', '59749174.7746*1484165866', '74153025.3951*1482889629', '90221720.3513*1476642234', '23563531.5275*1487467153', '10797198.2881*1455023074', '22556987.5859*1472665606', '63103128.3241*1456800228', '19144382.6417*1459309912', '29973122.9672*1468251877', '86514653.2632*1459418462', '6906105.79269*1469845449', '83432386.0664*1475595684', '96058035.8909*1483866387', '21448868.3991*1470573781', '51054270.8063*1471567235', '28636517.1334*1463493959', '21750446.566*1474481563', '30460876.538*1464502578', '6457996.34224*1470275677', '26525178.7386*1476384391', '41220723.4897*1478631477', '52251300.9418*1453236054', '72142056.8793*1477592990', '49135251.2447*1484868100', '44891265.4578*1473590470', '99783722.8699*1456855183', '99571460.6094*1464965835', '23380370.4928*1486766298', '83368744.379*1453033290', '91705396.6637*1454724059', '28321735.0634*1463795217', '60834275.1773*1460657625', '71663256.751*1473985724', '76348332.5797*1479578760', '57672419.0415*1485021850', '88985379.9847*1460576438', '47040366.4852*1486274327', '43184193.88*1472034401', '76287097.2576*1465861067', '78173755.2701*1462174702', '31508988.2219*1474111642', '35899556.4609*1463755986', '99778335.8588*1456203896', '72300034.6613*1480279596', '72316701.9698*1454975992', '96761987.4673*1486398683', '21830071.7802*1486908153', '24953269.6234*1478347652', '37476950.1057*1455894400', '44639511.4194*1452341096', '4872449.2913*1454271200', '64653117.5492*1470270544', '75630058.9799*1459878681', '68436519.9238*1467691490', '88668079.9684*1472041655', '7040095.01095*1475412378', '71538951.7014*1455879439', '94832227.0552*1457583482', '24247993.574*1487239540', '42357017.5574*1473107471', '72793803.7246*1457286431', '56741493.5092*1461228085', '70889563.9737*1478533697', '22949348.3739*1478660350', '31833276.384*1488315093', '10196165.0911*1461121135', '59465541.6683*1472018269', '45679507.6365*1466430471', '10388420.8402*1468136765', '87857597.0579*1453684671', '77515246.6984*1472061894', '8160560.52548*1452056349', '6531964.79029*1454369701', '68106044.0317*1471885404', '39519338.2479*1452214123', '94981400.394*1480291351', '78074004.4838*1457379724', '65014295.1033*1474692794', '73676058.7133*1476580612', '16581010.4171*1459751488', '46103709.7755*1480486581', '11596598.2719*1486710201', '14323981.0923*1469831449', '34944071.7785*1458539976', '18596923.0965*1472341708', '71623757.6544*1468525293', '61820352.4753*1460016726', '87626440.4947*1454603259', '17818176.0347*1465795825', '94769676.0497*1473647611', '13042106.3482*1465640673', '44853731.4818*1478504455', '57059991.5152*1479689821', '30996161.2447*1475451843', '11398923.2728*1463097005', '8218060.6293*1466236147', '69033978.4088*1479323131', '58000902.8023*1473884505', '56830928.0425*1476254133', '44830187.1691*1452909057', '78181840.0377*1468418437', '78214805.1995*1481624122', '56680811.0127*1479869612', '42025973.4373*1481585673', '58777872.0445*1483129201', '11418880.3204*1464688808', '73767115.3625*1485092355', '9901719.46042*1457675085', '27003854.3766*1455760440', '95526773.1241*1468144898', '64830700.3591*1488034140', '4315895.01239*1479743465', '92130831.5711*1460831397', '37512454.2086*1476611724', '49761794.8845*1478493217', '60648689.3658*1456148988', '21910233.1326*1454520818', '41309729.0324*1463037592', '68689281.9707*1469479559', '69879407.4007*1463664305', '97911537.3058*1465757742', '9776554.09446*1480110826', '48089313.2521*1463963368', '98390809.0318*1466159902', '59739137.6267*1486022269', '94416000.4633*1456397856', '28257615.6079*1467680355', '39090224.9103*1487885193', '43311826.1085*1476258862', '112917.323591*1463914113', '12857186.0577*1461235999', '93045116.7387*1453798480', '9429888.02674*1455382735', '9021758.89829*1461462941', '54183738.9551*1487809573', '841712.733182*1482420649', '98655135.3181*1481510794', '5856067.19242*1488213421', '82431799.7823*1484860562', '78754435.5165*1486017830', '65447941.3326*1473042251', '33754694.9688*1474591701', '74076834.3303*1479902363', '52978758.0208*1486641047', '17818176.0347*1465795825', '17470856.6814*1457270809', '67148043.2978*1459089710', '82268085.6979*1479170600', '15813634.8484*1455375933', '73451775.1024*1483460047', '53063687.4688*1474207246', '43744826.7829*1462822684', '40678951.4941*1468380256', '72037410.6519*1474608246', '35501044.6523*1477200953', '43895866.8295*1476023782', '11443402.5601*1484110503', '81723451.8895*1483354292', '78196631.2853*1476640353', '18361281.0226*1455060771', '35342300.2395*1484169512', '95997987.1225*1452371127', '90694173.13*1462255132', '15994519.9159*1482982472', '52162089.6102*1473831707', '85536855.7402*1477456253', '92954866.0069*1462815518', '37479108.5803*1485858690', '15311940.5424*1464667310', '69540574.9179*1464162696', '61780067.0193*1483183042', '12474115.3099*1461221489', '70285139.2875*1475845026', '2529094.46846*1473790435', '51604198.4956*1461060775', '4063840.71265*1456701689', '22767199.3886*1452816239', '95995726.716*1462520751', '54060711.2933*1464485697', '11719879.7015*1467594620', '15455621.698*1486226049', '48900297.4244*1486411825', '50365743.6901*1477120469', '50751527.9608*1477490798', '12550044.834*1480183336', '34773171.6984*1459409801', '31836345.3719*1485913028', '92515879.6615*1470583781', '59746131.1138*1468378950', '27006370.5137*1486111540', '90757243.8988*1462473378', '60918953.514*1483027802', '72832769.3126*1455735573', '74880211.2603*1456132712', '22088098.7264*1462888686', '50433930.9987*1455065233', '35944553.3003*1452425880', '15805111.6006*1452438361', '51009055.7208*1478791482', '43412139.7737*1462253767', '80412116.4726*1457454385', '14530813.801*1476119462', '91687610.5167*1452692188', '22823221.6884*1464238448', '38782425.8148*1486736608', '45998675.7323*1483824528', '23295493.7702*1453601341', '34107085.8849*1459457952', '95011020.8046*1456242225', '7911961.77505*1462340013', '18797122.9326*1455881593', '25640807.9629*1457612920', '58373382.294*1484370088', '7125770.16847*1474128879', '78880100.0102*1482329788', '32228635.7859*1461727509', '19553381.0598*1464638879', '58498591.4505*1476841879', '13505808.6555*1471424875', '90367390.9428*1455960887', '54428405.7218*1465076240', '35822016.983*1473952196', '78151770.4458*1461415710', '89535291.3279*1487965559', '63080236.6055*1465008499', '55022013.1623*1486925563', '25219207.4749*1460197501', '58929952.9012*1465720083', '7125770.16847*1474128879', '4432651.64894*1457405839', '97991306.9034*1475433038', '22330745.6666*1458884024', '54260838.0452*1481583223', '31835795.5611*1471120354', '5742884.87975*1479524402', '92504328.4382*1460988013', '68612687.3145*1468890786', '18398300.9467*1466398587', '25104990.4892*1482165395', '96617915.8809*1466803142', '22697052.5836*1485255898', '29241740.5356*1477370494', '46849622.1947*1470006970', '75718215.8022*1456364297', '57172144.811*1454520979', '52357914.2009*1454142069', '62457960.6735*1456124267', '3482641.64279*1487328753', '21466748.3625*1459473239', '43412139.7737*1462253767', '78235485.5649*1461941291', '76790469.5403*1470102373', '10361133.5346*1487212319', '58087569.0054*1479605005', '96240049.5061*1472138806', '36370556.4387*1453553770', '35743551.8138*1452528419', '75715663.5508*1458637096', '47700536.8643*1455808865', '82764840.5924*1484867871', '88677148.6882*1459143162', '84220370.4646*1481536478', '48089313.2521*1463963368', '99150597.5808*1475444036', '36125635.617*1480180836', '2308851.81705*1482341096', '15770024.4229*1482539191', '86772908.6459*1482467270', '67103423.6782*1473207817', '68120.3281803*1478121095', '220317.432271*1468010718', '83598740.4709*1452333329', '12442411.533*1473836500', '27425207.5374*1457153808', '33228285.4139*1457064565', '64193975.3086*1461203057', '81961161.6118*1475605932', '34435746.0297*1476671498', '18941872.5801*1487309954', '68125072.8879*1460918494', '24839462.7425*1455169524', '72434792.6248*1469464766', '99114259.5957*1453496513', '33898874.0687*1480493557', '11986612.1554*1479007929', '50519374.0429*1477364457', '62689710.6071*1473748830', '22941214.5712*1454646644', '37172288.442*1459405354', '94250126.3883*1453103934', '98181413.9278*1478896645', '25306897.071*1481580115', '85667463.5086*1480053656', '50573584.7172*1465018226', '54251474.4989*1475111063', '50158531.9242*1465128238', '6370144.45765*1455609672', '15087028.8003*1459392381', '66297341.2722*1481958211', '61639111.4576*1457507542', '5507843.02019*1461628512', '68345464.6986*1480693228', '84995797.454*1458983663', '74967886.8895*1484576700', '90380831.7364*1456449595', '43011523.5319*1481187027', '5731817.78427*1459018458', '87059482.2615*1487816751', '98407679.6488*1468048947', '47518801.5644*1454009796', '18876710.5212*1480856511', '75620811.6264*1487604020', '35254161.9929*1464791108', '5621140.14041*1463006957', '34792828.4224*1487140210', '968451.415474*1453294928', '88950506.9977*1469084072', '27014709.8923*1454925128', '60228291.0066*1464785291', '45025815.135*1468692367', '93506364.1729*1469116954', '60626551.5737*1455007512', '97648241.9914*1461838350', '68913234.9132*1454029420', '80886378.8812*1469159535', '63546703.8362*1468040427', '61276594.9789*1478833704', '72181688.4798*1453822041', '26926110.4897*1464057461', '44901645.0394*1458484651', '69703262.9526*1454870450', '44342202.5256*1467667562', '25306897.071*1481580115', '2449102.60525*1473200307', '36663874.586*1477435190', '71700345.8797*1453920355', '28257615.6079*1467680355', '16732431.7705*1487328431', '74767089.4098*1455776452', '57982742.5497*1452141507', '97915204.4093*1454482656', '35171109.2715*1473153365', '6146207.50372*1483068021', '2639718.3441*1462770053', '38117810.8726*1485166278', '96070558.8044*1484221767', '35853187.1554*1451744337', '61614157.3549*1475329699', '36094362.0786*1456271288', '37905095.7574*1459836976', '69440788.2679*1457832779', '4326495.54298*1468891591', '49752964.9622*1476904047', '33159143.2258*1458743001', '2027479.23991*1461577668', '29034977.9841*1473902251', '64992360.7121*1463100685', '82150455.7587*1459385987', '48124602.4038*1467055202', '53536184.0598*1462305196', '96617915.8809*1466803142', '83914144.7895*1458531196', '11237342.7884*1465053447', '29195970.3285*1475937481', '51224575.569*1470136068', '85014194.4721*1480649043', '41219826.1543*1484377113', '40602820.3719*1478001578', '36554059.491*1480679026', '76701139.0206*1467929039', '21657488.859*1469738793', '92853110.8467*1463607385', '81586667.8818*1469107211', '15965226.553*1476052789', '32059747.2747*1460578418', '54929201.671*1466258198', '48932536.9851*1456754811', '35743551.8138*1452528419', '90978290.8804*1468924910', '97558401.6756*1484332585', '18129281.7949*1481027512', '70156456.1713*1467555347', '29548077.2486*1476072894', '44765174.1046*1485391258', '97301569.1608*1485204516', '56966816.643*1480241858', '78267060.4882*1467889982', '35232454.3123*1473520625', '31428583.1239*1457151062', '76496233.9472*1462183024', '48803933.6431*1484291921', '66752832.7113*1486759164', '4846808.57284*1470148718', '98815181.6567*1470121486', '24094616.6615*1480313917', '22393652.1875*1487822169', '94055374.1168*1480395017', '1037692.51148*1473859005', '50849320.452*1458708645', '37235660.0237*1477136771', '57362317.9681*1454128720', '16130431.6993*1480024461', '28228057.966*1487437414', '92829024.3924*1459713634', '80070574.5594*1478310663', '27491523.4034*1465341119', '49071708.4084*1456547343', '15887556.5598*1458456102', '35880591.4282*1457525051', '26802529.3696*1476927392', '54081837.4455*1457791328', '44149043.3328*1474944730', '61820352.4753*1460016726', '54435236.0251*1485492231', '51251603.8892*1463625173', '43664557.1494*1477212115', '30823165.6428*1453227295', '56067042.2541*1452383896', '96744067.7824*1462932035', '57208944.5788*1482186580', '65898444.7963*1461971752', '57220075.4936*1485923174', '65527950.9565*1457714920', '68364746.6034*1453860255', '45765351.9657*1485220498', '37726966.5453*1465585702', '56529833.4681*1481838125', '33945259.1186*1455161152', '56196907.8259*1462554557', '52694927.0558*1460698353', '30778535.3319*1475389646', '96546437.6836*1487966528', '66624985.4234*1476799369', '50601968.6887*1457808504', '60802887.8696*1483558510', '40447558.9709*1458507479', '55660237.637*1470966514', '15837235.1744*1464613506', '53951994.7897*1483926341', '86465135.4894*1452659005', '40896050.4711*1462479999', '90091832.5668*1463471606', '43412139.7737*1462253767', '10444225.9937*1484706978', '90250156.0706*1453680882', '73383401.641*1457228199', '89900780.1339*1485138228', '83325409.0158*1464476496', '87907830.2335*1482908661', '76492653.4711*1465816892', '68084868.1721*1465505988', '22286824.6505*1488012521', '17663272.4768*1468967489', '66975323.9029*1477108456', '35702005.5204*1451816310', '28762463.253*1484855712', '95828115.1999*1451863272', '27798447.2371*1479905531', '27798447.2371*1479905531', '21282234.9276*1484410589', '65265157.4531*1466297501', '16715367.2565*1465121376', '54765591.5953*1452010591', '69534657.2455*1464232257', '16214027.2829*1486998789', '12453670.545*1468954483', '87078009.4944*1475684092', '65998005.1491*1478678871', '44236064.908*1477911144', '40619725.1578*1486317003', '83264643.0137*1457022973', '60174255.226*1459330864', '31127727.7308*1475164673', '18039600.893*1483965834', '17342809.3475*1472549228', '5928771.41704*1483346964', '8967160.50601*1464992533', '21133726.7216*1480403018', '29568185.56*1454889017', '18199827.2185*1457513470', '5427654.00078*1452665649', '18922023.3249*1468522511', '89389328.5047*1482859523', '86091249.2558*1487765777', '33344263.7673*1474642563', '63508610.5467*1453482148', '58362209.1652*1453265382', '5118700.52666*1476486022', '83311933.0628*1481334081', '55326966.4936*1478512813', '56069054.5211*1480745986', '43535069.9732*1470557720', '54522397.0191*1453672945', '67856013.0292*1455825453', '31354239.6172*1470566260', '90269909.2833*1481467631', '979989.676716*1457370855', '81308435.4143*1479813586', '96336784.7916*1472519446', '42652986.2085*1466570370', '21657488.859*1469738793', '13290646.1916*1457129165', '42850893.407*1453918507', '11967809.8954*1481862316', '44214897.4001*1463616646', '36539717.8984*1482335200', '42419414.7487*1477291496', '15830746.1157*1483921069', '92728949.336*1463951887', '68415895.9183*1474973566', '98042638.3594*1485501803', '820029.17813*1476372193', '60754855.4114*1453944332', '11899428.1476*1457331301', '82090463.5672*1483646778', '64551554.856*1478402633', '76287097.2576*1465861067', '82846140.7861*1459535669', '44788546.661*1485962144', '165417.448246*1469200695', '49512713.5388*1486175590', '31836345.3719*1485913028', '4755532.02896*1467035631', '2376255.49556*1475612155', '5958499.6807*1454398140', '94575134.5836*1468863158', '2772265.92349*1460258550', '26501688.4295*1487632452', '61953671.812*1465220792', '68582520.764*1485300656', '67004209.7623*1485318177', '43573912.0617*1485037650', '84828397.4447*1485525753', '86356511.4002*1459556763', '52900498.472*1458927017', '33618502.1118*1481899694', '18991161.0225*1485273781', '21020393.7951*1459590149', '78833900.7855*1452690354', '80647595.7813*1484106767', '82188897.8325*1464110344', '50078814.9236*1486945042', '60992216.3415*1454373576', '29597596.9235*1454517590', '68918565.732*1459696369', '54374473.0026*1469077451', '64606568.6624*1477041119', '37919081.5225*1452546192', '86103842.2842*1473278585', '81443190.5825*1468721554', '64309297.1269*1474224523', '59131204.8689*1466751331', '40700568.8619*1473717103', '87223981.0522*1477872838', '62457960.6735*1456124267', '21914819.9309*1460586997', '79470912.1661*1483832780', '35007505.1382*1468630970', '83245982.621*1483944219', '22467995.6204*1454605135', '61485581.4267*1473088653', '75979014.0188*1458933198', '35697254.9312*1476258070', '41901810.5389*1482286786', '33681922.9052*1456520305', '6811355.05701*1460232511', '19458625.885*1454694213', '19404825.1573*1458268178', '48440507.0386*1467438272', '88950506.9977*1469084072', '27647229.7156*1468965367', '49449338.3774*1465931088', '40237604.0583*1458152117', '31617131.2055*1475004112', '5746176.92257*1457938254', '85401893.4065*1481057052', '10925902.0758*1487705685', '56830928.0425*1476254133', '44093279.1403*1486106261', '29227104.865*1463183731', '92330288.7781*1475075470', '38091236.8358*1469516222', '25521397.1072*1479012511', '23689588.118*1479684322', '25496518.6392*1463968490', '90698041.895*1478419802', '64730750.9661*1456296943', '73236587.8728*1484877603', '9496836.5617*1461637924', '91405273.0369*1472512901', '8938978.04022*1474654355', '54323520.5362*1456507272', '78168762.7045*1465967414', '24108288.8822*1458794400', '14850888.0651*1471725152', '11237567.7042*1454632631', '70889563.9737*1478533697', '78987139.9249*1475438878', '22575334.4314*1477515946', '81342533.0468*1466757409', '16222401.755*1454565609', '59460800.9351*1486094949', '64218945.7397*1467298131', '40662359.0731*1481590094', '59425855.3046*1475009896', '80919994.4953*1461686946', '35248585.5087*1457689257', '6802434.49021*1487229534', '44980986.654*1471573760', '12836302.4461*1482546993', '46128230.3335*1454930090', '92786660.923*1475582135', '1920990.02867*1459229398', '30855320.3532*1485877274', '49651347.6619*1456704274', '31439617.4584*1473795277', '8303512.22548*1480945619', '97672665.4793*1451616403', '93225754.9394*1486953073', '3089797.34461*1474106069', '11005298.0542*1485652407', '41797875.9788*1454843655', '70049443.7616*1473459715', '65559741.1502*1455096542', '98181413.9278*1478896645', '89770429.3481*1476563436', '64606568.6624*1477041119', '27617616.2021*1478761358', '73542152.7001*1468547747', '24415865.094*1487006262', '36412250.3489*1482791977', '2191423.64814*1468968537', '44312579.4894*1485562885', '11443402.5601*1484110503', '57388523.0304*1479778867', '55602836.3351*1473029774', '54455212.8147*1467847757', '84985477.2858*1481370664', '64874494.809*1467129052', '5331163.89834*1458196268', '66209319.6528*1462917020', '43573912.0617*1485037650', '66032014.8299*1476698091', '18291694.6931*1461741292', '13828462.1604*1482550800', '1764485.37125*1452067003', '51313909.1932*1466071243', '11453216.7665*1467346110', '43535221.2725*1474411864', '38513513.5213*1481561918', '62792425.26*1463351602', '25387852.8614*1458466293', '5304917.93672*1455898140', '98824228.098*1485941326', '67993871.4732*1461161818', '56741493.5092*1461228085', '83246378.2568*1486673920', '23993436.552*1463540745', '28062657.9935*1455077902', '24326814.7141*1481350606', '12658576.4681*1469703201', '97915204.4093*1454482656', '38023752.6069*1458517683', '46281731.5368*1484454672', '94166551.3776*1466279075', '53843107.0824*1454467269', '21363913.5141*1472640946', '34732830.1295*1455186495', '74397667.5374*1470266996', '5118700.52666*1476486022', '75599154.1254*1463895930', '15259800.6778*1461244582', '85319479.7175*1477437329', '28125868.9043*1457749015', '19370967.774*1455586074', '15830746.1157*1483921069', '59012817.3237*1453194521', '28564515.8398*1468003801', '71700345.8797*1453920355', '38265066.1285*1475405514', '41791311.4333*1458624241', '22944728.7945*1479233299', '22035437.1678*1460132211', '57119838.7446*1476093110', '40633945.9266*1461385652', '10506612.5201*1461225587', '38355271.8298*1483289369', '70701628.4737*1458564745', '62955844.9563*1472544959', '82253699.5251*1482716886', '8809420.10856*1459878665', '55170058.0149*1456867842', '155834.820877*1470096015', '28627138.878*1454519860', '69183420.8898*1478163114', '77866870.9226*1479970552', '70701628.4737*1458564745', '31274630.9343*1465472691', '85916200.9179*1463600634', '51928152.316*1483113273', '12490278.3945*1471094379', '99255992.3621*1479176236', '88957215.9567*1474110631', '33543898.1486*1457652779', '27772556.5779*1485109216', '71284127.9191*1470151211', '17865216.1694*1486550089', '42092369.0479*1475272838', '14032745.4031*1479368891', '2934908.4705*1471635422', '86156950.5339*1469417431', '13836589.5658*1462310046', '13627635.0612*1456180387', '40252856.7967*1460596579', '59975001.8711*1479528715', '69555734.8507*1478625744', '60988920.812*1454965703', '58333454.0888*1477954559', '12039936.3763*1475066418', '93367341.5904*1466219810', '60918953.514*1483027802', '22312973.7896*1456269040', '33204127.975*1472756654', '28676206.5399*1473097187', '75313739.9216*1475616399', '71791442.4553*1473930327', '92966955.0763*1479676474', '97596509.7648*1475514221', '16393464.3139*1463963800', '75084346.177*1465708813', '62597582.444*1485552220', '61598938.9123*1464969440', '12946175.7714*1478428834', '57827464.6084*1463566591', '3839703.61967*1476193741', '98063091.9571*1477244386', '75915085.1778*1458112248', '75865767.0759*1485722833', '79493918.9596*1476970348', '41220493.0401*1462677105', '17540724.4216*1478090690', '72383020.1772*1472440877', '28389383.6517*1474421746', '89706954.4799*1478165663', '18581840.0153*1475941106', '96068322.9147*1484230814', '78157554.5068*1480203357', '85272264.3126*1471640949', '67134250.4963*1475732858', '73149660.7888*1474531823', '20008290.8822*1487828972', '79041872.4924*1458177046', '99207657.7944*1454706673', '85022200.0119*1452718442', '91386529.4935*1481730250', '21395302.0003*1485344728', '70726776.713*1470280524', '2173066.91941*1457089218', '59375089.4272*1480252283', '71804676.3659*1470231484', '85210122.7581*1458243835', '86612686.6095*1459344359', '77746472.3185*1470530088', '23093085.4567*1488261866', '77445929.9208*1457171686', '24455568.8636*1483051161', '30241912.5549*1470787429', '60685680.2358*1468818428', '42216971.932*1481233819', '15879543.6683*1473114680', '5534847.38834*1474290553', '19388324.4763*1457509152', '33225494.9648*1484091996', '56809875.6679*1478805173', '11637233.327*1461318124', '49631360.6637*1452387967', '42694988.0956*1485747626', '53710862.0087*1477232637', '67163268.6084*1487110425', '78788984.4787*1476649684', '83370487.6525*1470928754', '49404681.3007*1478358243', '14780176.564*1463446635', '1357944.09872*1451966880', '4452970.37851*1475300352', '93796248.1857*1465304436', '85797316.1214*1471492142', '62983360.1888*1478924651', '10557922.4026*1473861890', '66124975.6477*1459659469', '32228261.1169*1477406016', '25758269.0775*1487851016', '65359810.7706*1471476882', '3619799.73295*1470541877', '21319893.7833*1476836470', '88398991.6948*1487908413', '99007296.1658*1467596452', '40196881.213*1464643137', '17621178.0598*1456513761', '84317837.6191*1474288100', '85516523.1456*1475229412', '78163037.5016*1486534463', '43686545.0916*1475438545', '7872218.3848*1481573474', '968451.415474*1453294928', '94968120.1605*1475897545', '42850893.407*1453918507', '13227495.0924*1457330292', '21985536.5005*1477771397', '91544417.3719*1485535033', '47058951.2527*1463168740', '76363739.9439*1456684415', '93040954.3405*1482245788', '9314930.25711*1459859709', '1079659.09516*1459917887', '47625248.2571*1483694361', '74646059.7523*1483869538', '3382031.07593*1471356039', '58032188.9836*1453895459', '14268697.9253*1471670989', '83157211.5725*1467375041', '57362473.0997*1462734014', '42676013.565*1481209893', '44788546.661*1485962144', '61777224.5434*1466812943', '43881462.0366*1453007746', '67993871.4732*1461161818', '72577433.0246*1484368300', '34037590.4615*1463685542', '15286135.0854*1463278510', '46590956.3279*1455261256', '93771542.757*1477499632', '6218299.05988*1480857510', '98063091.9571*1477244386', '71021164.7721*1462864455', '49203002.3064*1466612287', '6941859.13498*1463488980', '19213256.9279*1484934410', '40868806.8656*1477027811', '13515965.9202*1458425740', '83937438.6577*1476593550', '79460812.7782*1464268076', '23791642.028*1464379353', '87698594.9042*1463240248', '45318309.4681*1474696066', '14456254.4465*1458430291', '74426170.6358*1461194102', '83160817.1649*1479448593', '48817693.3698*1453519353', '53397111.6093*1468808807', '24596195.8833*1476950986', '93642828.1815*1487608188', '1837528.37603*1467761894', '48817282.175*1462111958', '66811509.2134*1453379610', '81167766.5072*1481982265', '52975907.153*1472790670', '80368631.1095*1470492011', '41045320.5002*1478256538', '80512439.3035*1456303924', '58333454.0888*1477954559', '22611778.8778*1472502559', '55660698.621*1469517287', '9361940.69287*1486133938', '20122295.6365*1473164291', '25796796.7907*1482608153', '88658154.7927*1458035784', '6925544.09305*1453236124', '18720423.6164*1483524934', '82721087.1882*1474385047', '57926341.8509*1461434777', '94659087.4629*1475355050', '57588615.2054*1468375584', '98694259.8143*1470387302', '77320165.2262*1459740711', '62683313.6423*1479134768', '87275011.8797*1465292812', '21866147.4363*1466680080', '1437847.06962*1479938216', '1105309.23974*1478728206', '73980545.9143*1477082991', '15203629.0254*1455058460', '20968885.0138*1464639901', '13608328.8786*1454706140', '36820137.6137*1469236994', '51659158.1397*1478387923', '61702967.6411*1481148755', '47245230.6948*1484845132', '65335467.6906*1456997074', '43814865.7449*1462829855', '20420063.7434*1485838837', '5856067.19242*1488213421', '21858468.2566*1461129070', '71115799.7225*1467438290', '42032887.2154*1488020609', '9973999.90787*1465981242', '1788655.89892*1466622705', '2199518.77127*1476005244', '70413611.236*1457158951', '50008324.7711*1487871526', '12815703.2658*1477051689', '68551986.7249*1472617449', '55242016.1367*1468235816', '42611999.0923*1470345793', '70879620.707*1484431565', '65311960.2904*1463143424', '19637647.1914*1486433351', '22122056.1815*1484507581', '7950073.68168*1456588753', '52117986.662*1453290881', '49992325.156*1461164437', '42603274.6346*1470088190', '95626325.5317*1474264284', '27176923.9061*1485346430', '23451405.9625*1471166830', '19370967.774*1455586074', '69927817.7002*1477296283', '72125373.4605*1452379508', '2974087.26289*1473082848', '19509954.2841*1485484873', '77070217.6112*1467713068', '24567182.2439*1474758670', '17478168.9775*1479465450', '40896050.4711*1462479999', '76986399.5217*1452909693', '86936740.3473*1475088290', '35076858.9862*1465451603', '35073898.3249*1473124166', '15155343.2548*1460138611', '40534182.6324*1486269468', '39709094.0095*1479004749', '71021895.9396*1484830064', '35493712.0033*1484937982', '76258728.4206*1476540977', '72873937.307*1479504306', '14836170.3074*1463714610', '33867883.0622*1480536134', '8532755.35631*1468713614', '2431251.58087*1478067840', '36532542.5138*1459558505', '90053568.4662*1462129302', '46192868.8411*1462505345', '81023132.8694*1485301993', '93617725.0284*1471979887', '48346894.2106*1468916983', '39750799.4593*1485796285', '78129344.9701*1476546775', '95616218.122*1463150244', '48339916.3426*1480391713', '82561665.1424*1469938054', '31905725.2591*1452073276', '78028375.8487*1456440893', '82779066.4405*1454811064', '14604425.2269*1455165246', '47370556.2841*1462241128', '54908669.859*1484182748', '21822018.0213*1456846869', '41462849.4288*1469170262', '98379507.5254*1487662579', '6567622.48453*1465449790', '46287384.9437*1461969360', '49673655.2652*1487998416', '93676170.9729*1456028278', '52306005.5467*1459151022', '62357978.206*1454793536', '47964909.9165*1470740287', '31370482.671*1479735380', '5304917.93672*1455898140', '12946175.7714*1478428834', '10196165.0911*1461121135', '17339177.4432*1479807719', '83665714.547*1480877578', '16874843.5849*1483876875', '91218906.3393*1487539191', '81904457.4241*1470718154', '76022931.6977*1477796187', '28163068.086*1461056062', '42943417.6086*1463309367', '53399283.2075*1479316267', '21061324.0631*1452356280', '62399434.5143*1463348123', '88039156.0493*1476646213', '21208393.84*1486809600', '42086775.4655*1452458152', '84770964.1432*1463672854', '78042038.5312*1472236510', '13692128.0996*1472867294', '47122927.3425*1483337555', '1357944.09872*1451966880', '81419881.6467*1452821166', '2236692.50435*1452532683', '68113475.2561*1459826717', '32991494.459*1487943834', '94197222.4364*1475488313', '54351614.0534*1453621894', '70450779.63*1481200652', '83789762.4425*1487788525', '35727707.9929*1469520908', '7430841.69868*1467310155', '20880575.4063*1455285940', '485485.584361*1487595935', '81847124.432*1458185961', '43942642.6273*1455745719', '89434043.9695*1453801809', '12715684.9904*1485050726', '16829651.0422*1485659102', '46774866.0261*1481732023', '41926626.985*1480758732', '87059482.2615*1487816751', '1559263.90663*1485493046', '83068653.5222*1467807964', '27677447.2996*1462039515', '73041600.5957*1472929429', '18656128.3102*1473689258', '39956955.8524*1484608583', '879621.209467*1461215940', '58908706.7838*1483193298', '23326124.9182*1484645183', '39388379.1781*1452736674', '12582150.8686*1470228900', '4710180.765*1467236730', '32994178.6678*1462044144', '80430104.6485*1458119550', '93952970.3693*1478534160', '68195449.2675*1468015079', '15351173.8791*1479990617', '50089804.6828*1456153760', '73209960.0136*1453466342', '66656941.0445*1463037390', '70460570.0486*1458987833', '67044116.7341*1477121720', '75137472.1181*1470326622', '78181840.0377*1468418437', '82882589.6362*1466638717', '80832108.4643*1477705740', '84639957.0473*1478279161', '94011358.454*1460998935', '76283855.3999*1452480396', '85711097.1656*1474310976', '10123189.2319*1453343936', '20420063.7434*1485838837', '25764760.977*1475258614', '95844182.3311*1472643211', '40207945.3762*1464279022', '61746732.9001*1464021651', '78691891.638*1481032514', '26349539.3661*1473397746', '53790071.908*1470236522', '98379507.5254*1487662579', '55237141.7774*1460887245', '57982742.5497*1452141507', '25219207.4749*1460197501', '93160763.9415*1453441705', '69622062.1172*1466152258', '58066342.9881*1484949597', '22757413.6641*1452119971', '34734947.9443*1471301221', '75910998.2067*1480474495', '43798587.9052*1482348101', '67260685.8834*1481573609', '83478962.8475*1486347347', '50939588.4691*1461740019', '11719879.7015*1467594620', '11912370.9266*1465086517', '72383020.1772*1472440877', '34056905.6237*1456247725', '97064221.1002*1483269854', '78604149.7842*1469374501', '84962829.6208*1466302737', '21842953.5728*1454494659', '31570858.0608*1479888076', '39049745.5076*1461211648', '55072943.6791*1461276740', '43136660.3242*1466426832', '29182591.4551*1475513729', '25566107.9825*1471843981', '49556627.6518*1455739985', '57813969.3221*1462598541', '74815526.7621*1462639826', '80747321.8054*1488293852', '15813634.8484*1455375933', '38453695.3375*1471781801', '72925150.6402*1472431196', '53609918.766*1476565559', '81792012.7411*1460257727', '28328204.1099*1464771403', '80392992.7717*1487189322', '54420063.2392*1464814809', '71330729.4932*1480184542', '74612902.8948*1467570445', '28711634.1971*1458680665', '45557991.6568*1470811068', '76321009.2216*1478611816', '29140188.6074*1452763603', '92447890.0698*1458211674', '5682129.99934*1486591054', '59347247.8176*1483570528', '73115155.7005*1471601601', '95398559.609*1478149175', '77660394.6022*1471431857', '21775224.9834*1473224753', '13132259.2355*1451816738', '50939588.4691*1461740019', '87325089.1103*1478848078', '96970698.8575*1478552636', '78319392.016*1470868989', '54070428.1128*1464969486', '66656941.0445*1463037390', '94758138.9348*1486067985', '48928224.7186*1461081903', '64765253.3093*1460840850', '98878998.5586*1482706455', '79714105.0091*1478056851', '68880977.3425*1454957526', '17120495.3061*1467222370', '64008174.2024*1485963476', '18264469.5325*1488162434', '16581010.4171*1459751488', '78833890.7308*1480947885', '34997954.3854*1471812457', '68063097.3402*1464214374', '71924855.8052*1478658431', '92834790.5542*1458903372', '19960411.9745*1471473377', '49250004.3231*1454063311', '96887711.6003*1467931486', '31719316.671*1455099366', '89502570.5795*1470711336', '42132725.8889*1483164533', '13971493.6766*1475706825', '28676206.5399*1473097187', '31167721.8031*1459056940', '85083089.3115*1464039617', '41258740.3499*1462726677', '5159988.72267*1460348989', '88483782.0825*1467401969', '27033665.0626*1466576989', '339617.426628*1458847692', '49615969.5658*1468878988', '67866487.4928*1481784424', '46146025.6838*1478858355', '95711044.8506*1455607149', '62643598.72*1468592242', '53397111.6093*1468808807', '56508482.644*1483936276', '85684725.4726*1474158991', '87400700.6384*1464297084', '84391126.0283*1456274073', '93506364.1729*1469116954', '95814030.424*1473211379', '85789875.8034*1479509416', '23570174.4826*1485717124', '10361133.5346*1487212319', '77399890.5084*1464482588', '17475790.8939*1451955575', '52654260.2207*1461931681', '73722467.0999*1474989093', '64551554.856*1478402633', '7520300.84858*1480160797', '92384025.4056*1465534076', '24252822.244*1467982083', '7094246.65713*1481124440', '80708894.578*1467597322', '98930289.4108*1474672445', '40459776.4877*1453758649', '95807529.0008*1477035880', '26738579.2821*1475978780', '88106481.8634*1474744511', '58232085.2129*1470314550', '15043651.3192*1487869596', '12526178.369*1469960952', '99630370.4403*1483286264', '10366027.8765*1461564862', '46180268.3167*1483811839', '67547503.6616*1465695267', '93617725.0284*1471979887', '59941656.0672*1464431852', '68626660.1933*1476512007', '1837528.37603*1467761894', '83746441.3799*1468588006', '58611965.4048*1472155449', '51251603.8892*1463625173', '49078975.5392*1453029252', '70774426.2471*1477450908', '44460934.789*1459775304', '76999818.143*1465621384', '57220075.4936*1485923174', '73413934.4895*1473269183', '70981189.4251*1476340708', '24019464.4962*1460258679', '73568922.2668*1484696663', '61458791.0962*1452796793', '91604045.8698*1476434768', '14898286.6167*1472651110', '14322905.4068*1477001077', '33198065.2402*1458599033', '88984710.0592*1456395729', '78759957.8396*1453260668', '32217828.9786*1457817411', '18393630.9454*1482111604', '17018454.0619*1469537154', '43705985.1595*1462766811', '50766115.5329*1464877077', '10567953.1188*1487800363', '4779412.12442*1487751382', '1639867.73041*1477905530', '45113708.3929*1475795717', '61922271.2489*1476121429', '21665721.65*1475763799', '57320824.8858*1458149525', '44577364.5395*1456120658', '5535810.16893*1487125638', '79059417.6997*1474044948', '58135634.7278*1468939174', '7554434.97837*1480840485', '38622441.0914*1453890445', '75084136.0517*1471329238', '71606281.7699*1487124402', '34137509.5491*1486700437', '28497.3534258*1470360410', '94977146.578*1476231115', '41600885.0391*1461490874', '59124357.4048*1487110075', '98724208.8605*1470169004', '50667993.2085*1483476481', '38179832.255*1470296530', '90792088.223*1473137080', '93662609.6971*1482958826', '1342097.76991*1482420265', '51060977.7494*1461608246', '34672661.3781*1459270081', '1788655.89892*1466622705', '27202107.6778*1473060700', '16214027.2829*1486998789', '58798266.9919*1480717405', '71947018.4324*1484887029', '43225516.1729*1461473853', '19748331.5024*1478178972', '85289247.4942*1474432557', '52715147.3254*1453590343', '36031993.5043*1465718342', '49152610.2643*1466222415', '56778980.3892*1452753785', '55417838.0396*1466696903', '52228485.5102*1466359853', '5128290.51614*1469789894', '52410727.9384*1479847079', '22581854.8433*1462413880', '31146582.9386*1455180106', '88352795.6703*1452231411', '11379084.5094*1455393975', '39750799.4593*1485796285', '67866596.6463*1476574272', '32890983.7297*1474518263', '85575829.0101*1458525246', '79272370.3095*1462751110', '92783700.8835*1485393127', '77456470.6405*1471555412', '11717808.8507*1475098794', '95590.8272685*1477421526', '80148201.7161*1476470616', '2173066.91941*1457089218', '79373502.4825*1470030793', '81177543.7789*1468037109', '10444225.9937*1484706978', '7118490.73038*1459415239', '50048645.5882*1455745179', '57388968.468*1484253457', '91163823.9932*1470461529', '21765462.5597*1476384923', '50307389.2958*1466306977', '50799448.514*1464782630', '80586195.4781*1452917783', '74944003.6708*1471375318', '17348690.8851*1461132657', '99691115.2471*1468787750', '36629874.8752*1475691034', '36482461.1022*1466385810', '19960411.9745*1471473377', '67180278.4511*1486629509', '5420323.27817*1456732913', '62039401.2846*1470636126', '40064945.4452*1478065412', '87517927.9789*1475050380', '78838404.1439*1469482085', '20985669.8412*1458471098', '71683749.566*1469956457', '58092363.2376*1453465523', '1770646.0452*1471870712', '25481179.5287*1471855341', '11391018.97*1478735326', '79838284.1033*1463715663', '89314985.5418*1467451083', '35230297.5699*1452539800', '5420738.54883*1454163022', '81507937.8899*1457273482', '82820219.8065*1469559259', '53986218.9146*1487022197', '74865096.0563*1457147675', '78922767.2587*1485667155', '39385777.6532*1462589092', '76986399.5217*1452909693', '42086775.4655*1452458152', '135437.229714*1468423999', '14586539.9766*1456987665', '70934910.121*1481722944', '79684486.3212*1487520614', '2500118.28603*1461958485', '19954418.4449*1473624237', '69052826.2204*1460592484', '90367390.9428*1455960887', '85531613.0976*1485486929', '98304063.2632*1459471192', '31173652.12*1483123984', '11803193.0619*1482863660', '99207657.7944*1454706673', '83937438.6577*1476593550', '22429749.0675*1467446567', '57284706.4449*1478888218', '36370556.4387*1453553770', '16700393.5807*1454618548', '37729346.5683*1487053978', '4829085.66228*1453687587', '64264394.1194*1463634424', '87904879.3353*1459270738', '4355268.09536*1477860631', '89201839.4406*1454896006', '84391126.0283*1456274073', '39920204.8798*1476441041', '23793323.4501*1469054271', '19923235.4583*1460286271', '46135867.1287*1476182556', '81721772.6363*1485086031', '38507810.0532*1473479639', '86228734.6275*1470056208', '4759938.11742*1468464454', '36225362.0369*1484775074', '67083038.1249*1482925961', '76056090.777*1481839659', '58506913.4915*1460106655', '91231678.8416*1480526842', '13042106.3482*1465640673', '841712.733182*1482420649', '86597378.1827*1464844011', '71715095.6447*1460811252', '71671342.2923*1477708328', '20299808.9434*1458491540', '64653117.5492*1470270544', '85835893.4*1462630358', '46416801.2615*1464404644', '7640045.34904*1487127625', '93385131.7526*1475631836', '59700383.7715*1471320556', '53777935.0386*1467995205', '8864214.30951*1468570097', '97265273.2892*1473948922', '11966447.1765*1465377155', '97639160.3722*1456297436', '890790.067644*1477348152', '89867123.0896*1453382695', '4759938.11742*1468464454', '4142545.25758*1463973604', '42086775.4655*1452458152', '89065714.6456*1480547687', '8303512.22548*1480945619', '78673383.2531*1485398218', '6866426.48311*1462118289', '85019525.1119*1459111910', '7630042.76616*1468169953', '37578854.5209*1455964958', '93617725.0284*1471979887', '96171301.5066*1469558592', '45170747.5018*1473676997', '77326019.8557*1472665296', '84598101.6074*1464915423', '59553196.589*1467595107', '82268085.6979*1479170600', '47621722.6207*1459471185', '80133163.0426*1455584371', '25219199.805*1467657326', '53604371.9374*1454523030', '52448750.9469*1464309159', '5995672.96466*1452707474', '84010591.624*1455068032', '96163293.1723*1454255517', '95087632.0865*1464219144', '18964404.0364*1470076623', '98561661.7242*1482766112', '36335977.8326*1477111298', '6111930.10592*1466235461', '60381813.3285*1487841885', '61598938.9123*1464969440', '20913907.1978*1468933377', '66062981.4659*1474781026', '58484088.2076*1487641471', '52362667.2616*1485162481', '28382005.638*1487675569', '93325422.1372*1469477901', '60583872.1436*1462184742', '38508965.7583*1460037450', '18632085.2642*1458314037', '96016962.5912*1466984819', '34688926.1258*1485665134', '56321913.0056*1458847215', '81197487.7946*1486110660', '91470988.0362*1487505178', '90061115.3364*1466699291', '14157231.6025*1479433223', '30100652.683*1466912530', '41681438.9084*1454817075', '57167312.52*1465834843', '44360910.9506*1466846112', '77627055.6708*1473626957', '58786177.1944*1455745939', '71735660.8973*1476138972', '31836345.3719*1485913028', '62404826.346*1452837255', '11937817.9374*1462316929', '57273512.2567*1462664917', '3374289.69795*1451700694', '68436519.9238*1467691490', '19374549.7634*1457167098', '72522406.1086*1461679226', '68384847.8843*1474856814', '8418037.73131*1452195076', '68600773.1611*1478125349', '22910599.9561*1479682670', '99350293.2939*1469499135', '66926516.258*1452841216', '79929490.7053*1467491963', '85180.2567351*1453000153', '1391922.82416*1485224317', '79821341.9956*1472958374', '98964884.6679*1484233421', '17857273.5314*1481953760', '23932513.0705*1477162450', '46755437.251*1486936926', '59246781.8693*1466850404', '17029278.0919*1480527352', '98853795.5334*1461195000', '84458886.1074*1457944584', '28723863.2692*1456051300', '44884380.6695*1452405394', '54673307.0998*1481190864', '77070101.1854*1451849291', '56147031.2492*1483742146', '51313909.1932*1466071243', '81624022.7994*1452853320', '99006872.4439*1471410637', '30804969.9008*1463826925', '26802529.3696*1476927392', '84962829.6208*1466302737', '52825214.9843*1459052977', '60267123.2032*1461364883', '63384544.8254*1471925648', '61249683.4714*1472097977', '67431116.3109*1453576992', '55588098.1674*1451954552', '59400112.7312*1452501428', '29207311.8346*1472232133', '10252087.8402*1486052064', '84177463.9731*1487287296', '80487472.7527*1463949299', '27040151.5001*1484201923', '36077199.7731*1457815336', '19967814.821*1462056694', '58230611.2501*1476187553', '86954440.6126*1465201015', '25986704.0953*1485071639', '90720473.1561*1484335927', '18858695.8667*1469860370', '67057206.5969*1468871484', '43975775.0978*1483794305', '48761354.5192*1457951708', '42484997.6375*1456565956', '92252088.2137*1486896886', '54886050.5813*1454376407', '67991152.0919*1473395411', '9169947.76002*1482257507', '28659289.7971*1459661799', '31782455.9788*1473103298', '70147043.3536*1452732126', '64558682.2323*1476407389', '10848927.1398*1478466464', '18552535.481*1478734158', '8681936.88172*1475882421', '61595329.0818*1488238959', '43299700.0651*1471782750', '73937586.8223*1474637712', '41151399.3234*1487567795', '75012552.0581*1456290271', '83699747.0656*1459890269', '24793625.3021*1484781711', '6051319.00684*1467088075', '23990221.6126*1455021748', '16357123.6705*1480311859', '76899290.0157*1461531377', '19950991.0608*1469327045', '2454584.38031*1487863655', '65365374.2892*1478789621', '11043820.9365*1455125230', '84656814.0071*1471272023', '57103793.1003*1461930554', '43898546.6832*1475614162', '11335848.8555*1483067824', '4671853.48144*1461988604', '93959705.2037*1454902806', '46758094.3895*1470711621', '43147354.099*1455203368', '27251648.7792*1452567830', '18229418.7859*1462843455', '4929805.01294*1464534028', '19864385.7165*1487922432', '93045116.7387*1453798480', '93726608.6838*1454036981', '64606568.6624*1477041119', '18312696.9728*1473705521', '73937586.8223*1474637712', '38458215.2299*1474024282', '68812758.5199*1470727243', '29973122.9672*1468251877', '91483252.6015*1464099685', '16825367.2606*1483933539', '41833177.276*1483384043', '27334884.2683*1487792633', '32623877.6452*1487889371', '81731734.7363*1464301288', '70386632.8369*1472987703', '32543314.9025*1473749391', '5333569.38327*1481009427', '54308323.4235*1463900541', '20870479.4729*1463654391', '13400376.7725*1454402474', '59424955.9654*1469463781', '78408857.2314*1476799235', '46918399.0686*1469777097', '81731734.7363*1464301288', '7493565.79335*1467093514', '24415865.094*1487006262', '97175388.4172*1465171671', '30643593.585*1466070416', '74867309.7038*1466531690', '73483905.8968*1455878574', '4135130.47418*1466473272', '98225767.3481*1458165617', '19037622.8825*1461692075', '8945988.96078*1452694779', '91669892.441*1475790181', '14480565.3234*1464642869', '92323689.7424*1452156026', '65923087.311*1462872288', '83769095.3299*1475516454', '28659289.7971*1459661799', '84471876.2786*1457811006', '82813439.1712*1458655546', '79684486.3212*1487520614', '85275065.279*1463958332', '99007296.1658*1467596452', '16068249.5953*1486202283', '45866826.2412*1463291330', '83769095.3299*1475516454', '67614671.0443*1471166606', '75159517.8445*1474528207', '65857544.5929*1458135603', '55023582.6886*1453390947', '6495482.00177*1479341294', '33696783.2092*1478078169', '39317300.5824*1462627383', '62643598.72*1468592242', '28323648.8827*1488107117', '73173154.2012*1480835854', '73831864.7902*1471340943', '39232718.1228*1456225890', '2253776.42771*1459130015', '82576917.7842*1459453516', '99210304.0448*1477719554', '34974285.102*1453585041', '64388344.7855*1463308927', '99243137.0613*1466541114', '59882507.6671*1468677163', '20421341.6699*1458789930', '22209332.2318*1471018794', '76502167.9157*1486799053', '41757818.2661*1468868348', '24839110.363*1475501647', '13144837.9587*1487990857', '38442017.3459*1453411981', '8956135.10962*1475599318', '74497961.5735*1477660978', '34188900.1903*1487003549', '27531621.8894*1458546378', '77118246.0467*1482389874', '69316547.7215*1462986228', '83828336.4107*1466860466', '13083812.1573*1464965038', '29164972.44*1485990532', '4452970.37851*1475300352', '18229418.7859*1462843455', '69975711.8576*1480705609', '91327408.7351*1452360188', '57354055.2629*1467613149', '79764960.4903*1474593366', '34148297.1015*1483173641', '88995056.3617*1473225322', '83921616.8822*1474598635', '56699039.5413*1455327637', '97265273.2892*1473948922', '1375624.36641*1487290568', '73075211.2465*1480344418', '73865976.2416*1472330926', '26597649.8594*1458963183', '90229052.1688*1464123771', '5802566.10622*1461624399', '15571499.1819*1476422520', '62361511.9558*1466161005', '90289300.8544*1470626814', '16979503.9473*1471837585', '54146890.4139*1487806079', '47940645.8661*1487004524', '72729184.1616*1482449696', '40000471.4451*1486939537', '46737190.7154*1469388993', '5307134.99828*1465493018', '46985677.9552*1476212865', '69226165.3182*1463748030', '25735890.1644*1484324149', '98607147.1729*1484325248', '92095785.819*1458819366', '33228285.4139*1457064565', '3489991.20182*1457382562', '6125450.09593*1466055479', '21906421.9543*1464586818', '56187614.7715*1474284781', '14516067.4311*1456581054', '20938012.7037*1456985398', '42666717.5521*1468797530', '71197187.0875*1476499562', '56187614.7715*1474284781', '15028044.4587*1465160327', '68711931.948*1487629195', '22624619.4708*1471543142', '89169380.4482*1481429636', '60907946.1626*1474497803', '28638773.1436*1459108205', '6593489.76955*1484777702', '66482309.424*1478677211', '70252004.796*1479767061', '10221861.445*1479339382', '99214637.4154*1475270215', '13836589.5658*1462310046', '81087443.729*1454426778', '40700108.304*1476043298', '14393515.9834*1472989654', '95916916.6478*1466558353', '30852178.9514*1459729631', '33387559.5508*1452336906', '71378477.3428*1471218352', '11425033.8461*1480655997', '64250912.2784*1462993430', '41442227.2487*1479515354', '33898874.0687*1480493557', '20832217.5587*1477413640', '15115859.3463*1465889433', '40276526.998*1467655254', '53325744.3952*1461986917', '10795384.7832*1454208886', '77322720.3212*1475994622', '78195574.0715*1458925292', '61305783.1683*1462114650', '47396633.4028*1467450808', '64710535.4204*1476441671', '66356518.7711*1485350387', '77504593.6646*1476567111', '13154417.3002*1484060540', '52843422.8436*1459417091', '99449842.0648*1474632127', '29657536.0118*1456403249', '41892082.5761*1458601233', '37995287.8489*1466690116', '41867632.1235*1473996827', '41066663.3508*1478344463', '72627384.6896*1454463929', '59818894.3568*1467232120', '82203308.8411*1475580821', '67794088.4634*1460325884', '26525178.7386*1476384391', '32114583.8802*1486589395', '87399034.6457*1473383446', '49615969.5658*1468878988', '57526023.8577*1473222016', '32380027.9287*1472820251', '74122314.2945*1472574723', '23378307.2678*1479733692', '76954229.1658*1482057663', '93004864.2644*1480658632', '24575773.7998*1481531001', '56043532.5065*1477733893', '47499280.9788*1456109496', '31868296.5872*1481700002', '48461278.5436*1461859835', '29919680.753*1461788187', '46073055.1026*1464699702', '36287713.7198*1460428156', '6580155.44484*1457634565', '34529722.1813*1476907500', '81721308.4674*1463169313', '45034889.9868*1472508021', '22349744.7373*1476241454', '99429056.5677*1462263164', '82263584.8306*1478429696', '36231343.7046*1454360848', '4683223.90245*1466137640', '2967207.92883*1454371001', '42885657.4896*1468744139', '10221861.445*1479339382', '19590085.5009*1480976906', '22536331.8076*1471868208', '4880370.1887*1459570625', '95087632.0865*1464219144', '57936855.0851*1456885760', '9775767.07697*1471116401', '51985424.6788*1461221108', '2816425.31362*1465447351', '11352211.0916*1464560100', '16465196.6072*1463271681', '62400038.6095*1472909726', '98336418.8765*1456508222', '80417111.9278*1479099409', '86461889.9613*1473031363', '24466325.9116*1478661741', '98265031.1566*1480677908', '56285965.6793*1469295631', '31354239.6172*1470566260', '50773989.9197*1483851321', '26495821.0462*1476846883', '33247302.0789*1456869262', '44135592.3045*1484945175', '80031107.754*1463262636', '5511381.61096*1487326543', '29802813.1828*1476134834', '94710845.8924*1453455358', '85836754.2241*1481467632', '73882149.753*1478693180', '13077743.665*1468038655', '10538839.5111*1474665914', '52504773.6681*1463352785', '99052560.3344*1456976059', '92220225.1543*1462369262', '55610935.2018*1453002236', '68575992.1145*1465849725', '68304743.8198*1477297468', '55685747.3976*1473511477', '88975843.2321*1453683097', '54119027.3049*1475886827', '24741146.3619*1472814889', '28214821.9394*1480805551', '37369674.2454*1471374875', '63978972.1652*1470717408', '84776834.7014*1487743916', '43652207.3481*1483233191', '64092973.437*1475247509', '83010853.7464*1466936553', '6925544.09305*1453236124', '54665131.3935*1463801183', '13125999.4211*1482744908', '29140188.6074*1452763603', '65745374.1394*1479776262', '8677844.20075*1480383971', '29489479.8716*1466244182', '75209591.9972*1474867915', '6092128.34925*1456164248', '82251196.7276*1459102358', '67188244.4359*1486604593', '556288.438267*1460668165', '95724566.3576*1488012633', '12216955.4226*1473524627', '98867346.4305*1459291712', '28948892.1527*1480713090', '51547628.3994*1472163178', '29182591.4551*1475513729', '44284321.2034*1470641321', '47540589.7512*1459894383', '80647595.7813*1484106767', '44870503.3099*1486347903', '41045320.5002*1478256538', '11297504.9837*1477782793', '63956772.3283*1480868474', '98607147.1729*1484325248', '11998829.9479*1469322219', '63723676.739*1473042571', '43042265.7317*1474708238', '6833000.89482*1471636115', '75225210.1845*1476429624', '86721455.447*1480605385', '5793274.14365*1485839802', '23947051.1027*1465478038', '92086637.6359*1474431780', '23840363.1276*1457726663', '90985928.5528*1465946023', '69316547.7215*1462986228', '71430507.6758*1469119047', '15697450.4834*1474605856', '53600811.9198*1484478543', '96493530.0062*1475639499', '84994143.5091*1481863215', '26977534.9511*1466718775', '62371601.1608*1456493772', '29611281.4145*1470572004', '92277728.1948*1469948456', '64575864.7724*1452288890', '94743601.8766*1460690889', '12351787.527*1468111749', '10527979.1996*1471778439', '77766075.891*1463762284', '94433488.6669*1452606334', '35296499.3443*1467156992', '37523258.2092*1484026040', '68286350.0245*1470049483', '12944761.8248*1465355977', '30620519.2164*1475528811', '36103017.4909*1480223872', '42419414.7487*1477291496', '20206687.3819*1487376769', '51271689.6717*1478979896', '91134253.8294*1466454370', '61490160.7678*1476433083', '11297504.9837*1477782793', '21319893.7833*1476836470', '60106493.5834*1477159288', '90132976.9744*1484090218', '25263496.655*1467756158', '51156357.9857*1487079590', '66806854.3786*1473922064', '40896050.4711*1462479999', '93597303.2157*1453951117', '54070428.1128*1464969486', '90586587.6673*1462177576', '40290464.3775*1457416827', '32543314.9025*1473749391', '80800840.306*1477203103', '98507877.3218*1484378707', '40126890.0271*1472623623', '27686860.4245*1458683935', '62404826.346*1452837255', '30133397.8987*1486420690', '85832465.6594*1461051878', '66750224.0262*1466050266', '35064970.1753*1485056015', '43839508.0717*1468483537', '24738043.531*1468284308', '72529399.701*1482177480', '67283179.7976*1452853235', '11325939.4693*1451906285', '33462505.3812*1465718696', '4358888.90628*1477881447', '73509230.5583*1454131152', '67180278.4511*1486629509', '3267858.47962*1461227126', '10582203.5258*1460088088', '71683749.566*1469956457', '55610935.2018*1453002236', '69555734.8507*1478625744', '72373366.5065*1487685234', '91704220.1393*1461452865', '80187477.0184*1455061213', '94758138.9348*1486067985', '84464851.6444*1467609609', '89377802.526*1456533353', '80775986.097*1484031625', '64096800.1336*1483471781', '15720008.1002*1459025940', '47663962.2652*1474961540', '97301569.1608*1485204516', '59966198.9413*1469246439', '73512593.1274*1466850127', '55602836.3351*1473029774', '32171073.4419*1479553147', '26977534.9511*1466718775', '18965568.7387*1471333041', '2199518.77127*1476005244', '11880225.0999*1479879394', '46260891.6845*1475481366', '86154445.336*1480964691', '12118328.0841*1480780471', '69911776.9596*1477411388', '44891265.4578*1473590470', '89466011.3455*1481502401', '68397010.8994*1479170028', '10388420.8402*1468136765', '220317.432271*1468010718', '13654303.7882*1463207368', '76860438.2091*1485050184', '75630058.9799*1459878681', '85902096.7847*1459953327', '1534019.01662*1476094970', '79168754.8446*1485871297', '41057533.2175*1477269834', '41833177.276*1483384043', '32997245.5529*1465987452', '48124602.4038*1467055202', '40124155.7253*1486535478', '44816658.4346*1460670046', '93191713.3559*1481221163', '67846794.524*1463181398', '8232259.42314*1476182472', '35465117.1233*1464580461', '32697188.4341*1479185862', '99252661.1645*1485164700', '5742884.87975*1479524402', '99258203.5976*1484907457', '14836170.3074*1463714610', '46086550.6728*1476404894', '16734864.6512*1482777237', '92869684.5138*1478001714', '54665131.3935*1463801183', '61714640.0137*1455171455', '32457429.596*1478723870', '18305178.3993*1474043245', '44647164.9101*1454016298', '28645591.8693*1465788655', '20049031.4061*1466576738', '3106247.41421*1483922856', '32095186.135*1462665453', '38557045.5662*1452008988', '84264081.8453*1469649533', '23207606.0343*1469580247', '50975518.3943*1464368083', '29456781.3491*1485124025', '78459154.1167*1467280400', '89505873.1232*1487343334', '14694723.9217*1467761541', '25473523.0579*1481399430', '49885296.876*1456153415', '7248774.25741*1466814754', '87937701.8234*1468513288', '40619725.1578*1486317003', '23207606.0343*1469580247', '48540084.7388*1484936589', '27890532.733*1466610816', '55306905.7671*1471657370', '17307628.8095*1472712186', '47566286.565*1453215470', '39705514.1059*1478337309', '38938029.1249*1475054438', '96929470.9911*1472523561', '50084943.1988*1481305708', '11450713.2226*1454946135', '4005098.07482*1472016535', '62112984.077*1458854066', '65264045.5562*1459557827', '39281205.5729*1477153134', '4440050.29981*1470026319', '62646043.1751*1472330447', '1663489.91542*1464047091', '53721252.5506*1471154090', '83361740.2642*1465686239', '52787520.6045*1463858754', '27666455.9235*1479929818', '82448346.6613*1470231906', '8866162.65083*1477937376', '12736139.8817*1479155057', '97204310.791*1452449327', '2639718.3441*1462770053', '98219830.5049*1452420153', '60980782.3658*1455738843', '79478723.6322*1468229011', '63333376.0754*1473946158', '14604425.2269*1455165246', '94086914.5475*1458517692', '14100490.7776*1474625975', '61079103.7698*1466968473', '23206323.8654*1479565308', '98834498.0871*1453700935', '9828057.65158*1462850728', '12120637.492*1480454724', '1920007.81007*1462314666', '47498168.5107*1453573999', '28285901.3783*1458167732', '15167102.6746*1454198086', '6050407.27058*1473942706', '42804853.0884*1456713286', '9374264.94096*1482601220', '57418741.5018*1452586837', '89839370.5857*1467682053', '87171611.198*1481448883', '11480411.2247*1473778084', '78411890.0271*1486342561', '85180.2567351*1453000153', '2953791.22336*1478397725', '76834800.4869*1477562953', '92740502.8344*1453945396', '64551554.856*1478402633', '68134549.462*1456410631', '76980067.5876*1472659350', '4115108.42349*1473616313', '5148243.02744*1466712092', '5586490.01955*1487942154', '78437955.8907*1453304242', '37410487.6352*1458872690', '99927464.3731*1467226839', '77315983.3078*1480015271', '52895857.5865*1474195043', '73208226.692*1470673729', '94701998.632*1467749667', '75931304.2189*1458026078', '57362473.0997*1462734014', '74076834.3303*1479902363', '43139528.9681*1488008597', '86811463.8431*1465732568', '51547628.3994*1472163178', '26231875.0501*1457758290', '63152161.653*1463770557', '14710558.3403*1481592312', '26291062.7562*1465985516', '35803080.4686*1460079901', '31515294.125*1468290124', '34524106.7207*1483521030', '38341204.1313*1472117190', '85042216.5697*1471737456', '81507937.8899*1457273482', '34419864.759*1467380086', '87362805.3135*1457602368', '81721772.6363*1485086031', '39587118.8869*1474928443', '90864456.6328*1469287006', '86654215.1016*1467589743', '73818556.5234*1485141450', '44093279.1403*1486106261', '27066649.3053*1459515879', '19944187.1282*1468796505', '29709093.628*1454505804', '18106710.0798*1475943151', '78537572.9837*1475456620', '51529923.5801*1488155202', '84521655.4415*1487431318', '79521593.4092*1464203426', '80273107.9661*1465088686', '61305783.1683*1462114650', '25219199.805*1467657326', '53546940.6137*1477167047', '43108455.8371*1476453778', '75297784.3061*1474706236', '60944446.8792*1487496057', '64258157.1211*1480780106', '31080687.9442*1465416045', '2068311.41343*1470641834', '95330342.0981*1478147432', '70590184.561*1460916890', '95141174.8853*1486692921', '19192603.9292*1473823162', '15674515.3499*1464806469', '78596930.3223*1458792211', '68914705.8469*1460114409', '92869684.5138*1478001714', '68834045.4818*1465019770', '2814080.7726*1455852426', '48649360.1472*1480155325', '6778164.99582*1463939659', '54929201.671*1466258198', '36638946.9381*1463108993', '74269973.5136*1476883859', '77660394.6022*1471431857', '59124357.4048*1487110075', '64503997.5731*1463581062', '94043059.1527*1479098448', '56435843.2432*1487427072', '89457946.6185*1463577233', '45981448.0917*1456700762', '51012042.5194*1456676710', '91705396.6637*1454724059', '92172066.7185*1466401646', '46401257.6934*1479714519', '87337125.6523*1461198205', '94139301.7206*1482760525', '56026847.7974*1469940189', '4324215.89677*1452337219', '45713029.352*1466320214', '89535291.3279*1487965559', '34493537.2426*1472000961', '21007188.6817*1465809987', '69975711.8576*1480705609', '35606394.8194*1476935974', '65119152.9909*1470868847', '92804134.8135*1477256023', '14059453.7137*1474323603', '76779786.1498*1456846126', '26495821.0462*1476846883', '65773123.7855*1460362291', '12808077.9442*1487188201', '2477555.35602*1467397899', '37172288.442*1459405354', '78598705.4877*1457952375', '36539717.8984*1482335200', '34278966.5394*1480907963', '60751951.9734*1475635485', '53334819.7629*1486815018', '38087753.3899*1477111612', '43854102.7011*1464186923', '56330663.3521*1472231192', '62239943.3946*1460143893', '1677374.67953*1459644830', '86719826.1886*1472141310', '92140745.4837*1471901714', '31032328.54*1467725770', '69652475.2933*1480659475', '49111565.5968*1461583775', '30793454.9986*1469394641', '12039936.3763*1475066418', '30894801.0039*1478976360', '75681620.218*1478645164', '55997616.104*1467283468', '94284861.8769*1456680773', '84874805.1202*1466883479', '32205234.0881*1470004346', '35262571.8645*1472323220', '35691143.6449*1453397681', '49073349.1797*1475446595', '44696867.7785*1470831718', '36437890.0243*1481944496', '88205255.6225*1453848928', '68880230.209*1464444579', '52837855.4203*1476154144', '31146538.0298*1462621846', '87399034.6457*1473383446', '46797129.2792*1470278187', '27896884.558*1458859746', '99655977.4567*1462246953', '52019351.3587*1487816688', '75562377.8197*1473488788', '25607107.9041*1458957020', '38935126.1364*1482230672', '58959371.9219*1487759915', '40991556.2132*1463851726', '75853818.1663*1482983563', '66750224.0262*1466050266', '63800018.2696*1467146097', '19063645.0266*1488171614', '5159988.72267*1460348989', '40436815.4885*1483357089', '86824635.4319*1470361215', '44861637.4593*1470764082', '33931720.6595*1476687057', '60402995.8543*1456178265', '75986501.9083*1477795932', '71695324.2886*1476804954', '10582203.5258*1460088088', '55007284.1564*1467401481', '92530775.9095*1462722056', '95255692.3149*1462579693', '4417705.07427*1488056604', '62077681.1885*1487817593', '35224802.204*1473360307', '73785719.5793*1486102683', '8506447.09146*1453307354', '43837264.8905*1487958755', '27541354.082*1484636963', '77322720.3212*1475994622', '80509499.9615*1464407285', '46035247.9983*1470161072', '55379296.7031*1472225346', '20209151.9862*1483802642', '65228978.9419*1487240469', '15095854.8806*1475129801', '92640201.7263*1474547633', '69711464.7626*1466065296', '779942.342627*1452819141', '28197754.8472*1465646059', '50349644.0821*1464283684', '1805272.31758*1484477840', '54081837.4455*1457791328', '58997509.2209*1487695972', '81711696.6059*1479453346', '60751951.9734*1475635485', '15076929.8894*1473809092', '89792887.3879*1471832856', '11583635.898*1477553481', '93983532.5335*1466520500', '62748048.9967*1483742448', '50012335.2855*1475081707', '8977745.98309*1471669561', '10205353.3599*1471840130', '22572116.1556*1469433453', '76125267.5858*1485351985', '97250026.4924*1457194169', '29477562.36*1456569587', '96983119.6445*1466636956', '25184181.5483*1463386161', '32164381.7637*1466400231', '73252468.997*1472595128', '17120495.3061*1467222370', '89969910.2699*1478966501', '69012917.5192*1454063448', '32697188.4341*1479185862', '54122223.4568*1483159493', '67715432.8614*1457208464', '70894131.2033*1483004848', '66173987.1969*1481247727', '11668233.2883*1462306541', '41925154.6304*1458991429', '63936683.2082*1482609802', '33681922.9052*1456520305', '62414752.3914*1474635681', '54313817.2284*1472986195', '56866394.2789*1484187219', '46184142.2427*1487862065', '353528.405902*1482687678', '82346493.9102*1476297236', '10036070.0533*1467332183', '70749969.9146*1463404971', '99354137.1763*1486387956', '83454375.681*1459542366', '5128290.51614*1469789894', '32784732.5492*1461613469', '77756172.6834*1478092129', '93687250.4045*1469438275', '56245228.4508*1452260276', '68864635.4072*1474808515', '32012644.6602*1456325461', '65876726.6047*1470869224', '19063645.0266*1488171614', '37816426.8107*1457831573', '15076929.8894*1473809092', '71973942.9518*1481053123', '30241912.5549*1470787429', '78157554.5068*1480203357', '34089189.7757*1456791998', '64103303.6942*1480633811', '63257713.3861*1462573466', '65919126.1535*1459071808', '56839124.9667*1470440858', '20168799.4427*1472701619', '49512713.5388*1486175590', '94090505.8544*1477841138', '32336984.0155*1460955969', '22106903.9431*1461519033', '7872218.3848*1481573474', '64264394.1194*1463634424', '39212085.4815*1461955488', '43299700.0651*1471782750', '43606137.9515*1486322072', '94197222.4364*1475488313', '54200020.3489*1467778802', '71051253.8672*1478339900', '46824110.3968*1451950245', '46813455.9896*1477333863', '90380831.7364*1456449595', '72851325.179*1478241964', '70726776.713*1470280524', '49556627.6518*1455739985', '31155393.107*1462383572', '24986921.8428*1454991381', '93195568.5676*1481700303', '25180039.0099*1462573393', '38507810.0532*1473479639', '27135095.8465*1464623470', '85802389.9602*1458710304', '11589035.0349*1485910750', '68435846.9411*1477120469', '83914144.7895*1458531196', '29427269.0851*1453594313', '68745442.1838*1484666506', '30791124.0407*1458394097', '38083500.2509*1488183475', '39651925.0377*1455748894', '5220534.27577*1487208431', '1342097.76991*1482420265', '76153817.1581*1467755429', '49058012.3976*1457080501', '82715773.6869*1455707008', '24371687.9041*1480396421', '4221430.05194*1456653600', '35717184.8556*1467629328', '59593585.0208*1484950393', '55340828.4761*1485080461', '31590076.4251*1460350385', '59044845.2584*1469882471', '64399117.4507*1456450203', '21628884.5853*1475740862', '95493059.0738*1462123823', '13980317.1331*1483895948', '27001775.7996*1452125623', '70106409.1301*1473873639', '93476057.2514*1456379135', '61888769.5445*1482110701', '79059417.6997*1474044948', '87227186.0264*1485263422', '18239195.0579*1483684868', '19582882.9853*1467907258', '12100252.702*1484992764', '41713194.9278*1485447971', '6999423.0901*1453547526', '13811651.1916*1479265488', '73038389.1455*1451657744', '31673452.1173*1475507466', '39920633.614*1463356807', '15576025.6484*1481176933', '61054352.869*1452845969', '24094616.6615*1480313917', '62400038.6095*1472909726', '67614671.0443*1471166606', '79682359.021*1472551975', '460080.167241*1464298350', '64271531.3747*1480748380', '41300922.6663*1477386816', '80827230.6503*1485398022', '70981189.4251*1476340708', '46439992.1694*1461530870', '44730139.764*1476600536', '33225707.7061*1453789032', '67246312.5897*1487071005', '6274813.43582*1460254622', '8967160.50601*1464992533', '76483913.8649*1460194342', '17321466.1412*1463951459', '41097907.2915*1482791532', '70450779.63*1481200652', '20640286.4265*1475088315', '72731595.4355*1468994757', '87171611.198*1481448883', '83277350.6149*1482647254', '4398737.50649*1463757050', '27425207.5374*1457153808', '72889125.9558*1478097905', '2604859.21856*1481560952', '60264944.2104*1462310139', '2189894.49325*1474889695', '43060495.1994*1482437151', '74431095.8182*1468435324', '51794356.9478*1472531241', '68817665.5134*1456251757', '26248826.4176*1461118420', '68639308.7473*1476111977', '3017221.46187*1456686177', '99235085.0878*1465986443', '43895866.8295*1476023782', '7361991.03856*1483716932', '59449905.5034*1482732848', '87337331.7404*1468039325', '84916466.7936*1466234067', '27571351.6141*1463125313', '35409468.574*1476412872', '70894225.6992*1460954706', '51261367.5187*1463651530', '80333980.0907*1480184662', '50518745.8366*1453701652', '89691935.7348*1485220349', '77125102.6797*1479233177', '59375089.4272*1480252283', '30124521.4044*1463779071', '27939326.7162*1470256393', '1764485.37125*1452067003', '64512136.2009*1473360884', '73686409.5463*1482183284', '31611513.9211*1456782576', '12605832.4815*1479663924', '20120187.8479*1456397849', '18104718.977*1464072950', '52440980.5175*1461544958', '80337072.2906*1484936478', '5536456.97009*1466781718', '78598259.5236*1477207580', '97654056.0183*1468772906', '96058035.8909*1483866387', '7521921.73251*1464962331', '84298255.5629*1465850078', '39718835.6775*1486894760', '80554669.7637*1470518612', '39177628.8805*1476767279', '27308686.204*1456099742', '843613.193685*1464341059', '23722661.2212*1471365913', '22906076.9417*1465625350', '32873039.0699*1459992879', '82608814.3287*1473096725', '56150598.9243*1465549270', '21088702.7842*1452142954', '47396633.4028*1467450808', '85679119.5605*1458950731', '985439.44678*1469975651', '82049206.0086*1462381279', '55702038.6737*1455142281', '75341429.3085*1478665076', '5221264.79735*1468793612', '15644054.4182*1467670121', '77048501.2198*1463178130', '17475790.8939*1451955575', '39096361.2136*1455081992', '24440888.8225*1462294273', '40628876.9285*1483951242', '68895658.4403*1481425255', '90420199.3478*1456252402', '76954229.1658*1482057663', '73882149.753*1478693180', '76032488.9444*1464616225', '55262929.8142*1488323462', '37461066.605*1455775162', '30805514.8644*1466603936', '36225362.0369*1484775074', '47908047.1572*1479251031', '56464791.4593*1464783149', '31074926.1086*1464057941', '15391361.0048*1462955185', '60898783.512*1476590353', '51135758.6801*1475397103', '7199065.50371*1471254138', '67327157.338*1488171301', '47621722.6207*1459471185', '93915272.9258*1472626770', '65898444.7963*1461971752', '98146815.2421*1481905788', '42245881.2255*1485469991', '59809445.6029*1462377381', '58444559.68*1480589617', '30794685.6034*1473101824', '10344110.7984*1475286153', '50287846.0351*1467693671', '77934772.8426*1484242512', '19626273.2887*1483992010', '78193436.7757*1475807191', '68125072.8879*1460918494', '76363739.9439*1456684415', '82280051.203*1484397073', '50078814.9236*1486945042', '91748152.8092*1465626316', '63145192.7716*1463461833', '15388362.5331*1469157612', '41583871.2003*1483906388', '83081430.3429*1480386090', '63073189.1827*1452028034', '81721772.6363*1485086031', '37425779.1246*1474395138', '31117538.6747*1459478937', '29260607.9135*1486210501', '61567848.0273*1470660796', '59449905.5034*1482732848', '77445929.9208*1457171686', '96433867.4683*1481494075', '76806677.1158*1457384524', '33865478.5208*1483833860', '34493537.2426*1472000961', '56378479.8906*1471425942', '29568185.56*1454889017', '72396564.9124*1474570798', '19967814.821*1462056694', '84687178.391*1459888414', '63956772.3283*1480868474', '58625.2191816*1468479485', '93338402.0003*1468503308', '13399430.9646*1462932435', '46128230.3335*1454930090', '14773442.0928*1463838931', '37209886.3552*1454040583', '90697721.6388*1453701607', '83700933.2769*1472177399', '48322904.1992*1458246490', '96730476.8493*1486455540', '66334673.1951*1457404416', '17807842.7967*1487057733', '91471055.9128*1456648121', '28222504.2195*1454710290', '53349203.6338*1470497477', '94900103.1286*1462885248', '82170803.157*1452804298', '71149545.7552*1482284385', '82820219.8065*1469559259', '84231687.8851*1472932169', '93614277.5331*1461293353', '26351816.9577*1469688357', '23254171.774*1475918849', '15751307.8532*1451844588', '94343455.7478*1487471205', '14798632.4394*1478613090', '21344804.6672*1466877143', '67434275.4235*1482822662', '33867883.0622*1480536134', '97258716.1443*1488007092', '7231948.66792*1481622876', '34793088.5413*1482523026', '56012474.7686*1460714566', '55443396.9638*1465945073', '86336928.3097*1466012588', '23009250.2923*1478718238', '22611778.8778*1472502559', '43611455.2654*1484733007', '98652146.2255*1455026840', '22941214.5712*1454646644', '39610214.4729*1452013132', '54886050.5813*1454376407', '49975039.0057*1456605827', '60822412.8541*1477711399', '37553000.8926*1463196691', '59735877.6041*1478265450', '87282078.7692*1456681014', '44698326.4939*1471665052', '41807938.3511*1484316641', '60281188.5817*1455459294', '36859847.6577*1462492012', '48829729.5054*1483855729', '1338706.89571*1473585990', '15115416.5125*1455938890', '10120280.4424*1480951462', '97127350.9496*1473113331', '52228485.5102*1466359853', '28199718.4235*1461750749', '37401149.0392*1478666824', '58267928.6779*1459808162', '63081929.4937*1454194737', '54537568.4498*1481132549', '62240447.7065*1463786047', '74171474.3751*1481998156', '76667463.1729*1455928065', '39750799.4593*1485796285', '35745456.5672*1487171107', '82263584.8306*1478429696', '45201289.0721*1471219031', '46849622.1947*1470006970', '42907844.5083*1484799927', '33415277.5134*1456505011', '89739718.7541*1485376608', '84019810.0032*1469030662', '91196122.7906*1483538677', '32623877.6452*1487889371', '75620811.6264*1487604020', '26802529.3696*1476927392', '75978285.7923*1485195012', '90647435.9827*1468101724', '23830110.9722*1474074638', '62545297.5214*1455862076', '9019859.93605*1470684073', '22471458.6299*1461273469', '45144433.6515*1461880187', '53354378.2278*1457793913', '73036561.3695*1480614256', '89549049.1706*1485426796', '99964206.3962*1464900508', '84775416.5394*1475774471', '13832549.3555*1467611946', '10183919.6273*1454307056', '19090781.1177*1477304455', '92642884.1427*1460962339', '49975039.0057*1456605827', '10795384.7832*1454208886', '88310863.0798*1475820702', '13577786.3492*1462524869', '80897645.8983*1465761699', '44870683.1762*1487681453', '32217828.9786*1457817411', '24415169.0651*1453284558', '39643224.4471*1471155157', '19580969.7453*1472171430', '33415277.5134*1456505011', '76964429.492*1482916477', '61276594.9789*1478833704', '99997313.9013*1474202301', '86994351.5305*1475686178', '18663134.829*1473816335', '30618608.7369*1480824195', '44963012.8462*1452536032', '72136355.4719*1464723600', '45205482.4501*1476871808', '89586651.9395*1453539206', '33094676.3746*1464879813', '35865833.5033*1485155682', '54224794.3764*1465479172', '44764720.2968*1478826553', '44862211.1838*1462965918', '79272370.3095*1462751110', '70725581.838*1483491558', '94036158.0646*1475106915', '8853248.2442*1477732308', '51794356.9478*1472531241', '4355268.09536*1477860631', '6753814.72977*1459446726', '39192502.1879*1464442076', '93829272.4246*1460310409', '27440916.9417*1466561029', '56866320.6699*1470767315', '64657142.258*1451897630', '34932672.8854*1478253589', '69203479.6092*1456025353', '75875654.531*1476867895', '92863477.2075*1481684635', '51664427.9471*1465317468', '3103863.92781*1479176789', '46452622.1958*1458678223', '89372586.3317*1487605614', '92086637.6359*1474431780', '13222256.1308*1485370971', '98161271.8781*1454941641', '70863821.2466*1485296699', '15139463.7353*1479303671', '21215135.0887*1462469782', '88241050.4491*1469168643', '67784690.3491*1463887612', '18204166.6328*1472365954', '71399618.1518*1468053449', '61731093.0914*1482146934', '93201976.3774*1488072749', '79497320.174*1468341568', '90671907.462*1460908428', '4067435.53614*1460980860', '95961486.7419*1457504351', '18922023.3249*1468522511', '73638275.7084*1460834613', '13828462.1604*1482550800', '87426316.3732*1452054902', '4208667.82593*1454927056', '66191808.7569*1470821942', '85673562.7355*1465280701', '93454429.1207*1473432942', '87236730.7157*1486460437', '52900498.472*1458927017', '13194185.7717*1461840772', '49761794.8845*1478493217', '13094614.2534*1487709102', '45881944.7411*1472317457', '21572695.6082*1478481644', '23302558.0908*1464163660', '37509718.219*1480271031', '32538057.3426*1456719327', '21072254.2864*1454352689', '14993949.105*1472046389', '25796796.7907*1482608153', '84697626.4619*1484010588', '78988677.9692*1486637387', '59065017.7166*1467296318', '23005968.8485*1488315513', '72136355.4719*1464723600', '25911839.7069*1480472652', '18084850.9933*1483841652', '42694988.0956*1485747626', '99725576.9721*1468437660', '77399827.4227*1478988568', '51495273.1634*1478483780', '69711464.7626*1466065296', '70749969.9146*1463404971', '79359627.4994*1457035780', '13172646.1935*1453331761', '19379401.7494*1457388348', '42428678.1146*1470615575', '45300493.1767*1459946451', '33761845.3375*1451783625', '76754350.4787*1466468349', '10148126.7865*1479969451', '39281687.8425*1476187058', '1764485.37125*1452067003', '48838735.9788*1451937070', '86818016.8289*1471065449', '5915847.97598*1484801445', '22000138.6382*1479870420', '55569422.5529*1473904506', '89440139.4876*1460090385', '77174563.6494*1467989657', '78392171.1577*1473301889', '59180167.1258*1458015127', '96748154.6527*1456366084', '48803933.6431*1484291921', '47978994.0217*1464213827', '3875426.16101*1484205875', '9248691.63313*1485695042', '66191808.7569*1470821942', '44468070.4985*1473208162', '54276697.0743*1485717540', '37461066.605*1455775162', '28761085.5652*1464263098', '16056823.2382*1483648659', '43367178.1179*1486127798', '16687802.7987*1457798277', '84385853.4245*1475314173', '72316701.9698*1454975992', '42025696.9398*1452140012', '15421996.7016*1458452523', '76661004.0074*1456509414', '34930776.5862*1477964708', '75729909.1092*1477639868', '55168300.136*1479629063', '52706895.4968*1471344918', '74412069.1667*1476375260', '31668796.997*1466825018', '91924629.9449*1457555348', '47498430.6995*1465957423', '23207606.0343*1469580247', '56862328.4081*1477985534', '1458833.65634*1453233195', '56530076.2422*1484527166', '22824437.3406*1456011274', '84010591.624*1455068032', '8961024.67501*1467960464', '22824437.3406*1456011274', '79363330.6898*1483136324', '34252242.0411*1466546424', '251059.021859*1464953573', '93689382.7157*1481899534', '86743575.7795*1466753950', '92504328.4382*1460988013', '44637332.6062*1486631176', '38212906.556*1461189132', '43060495.1994*1482437151', '61821052.8537*1472806883', '61528400.6415*1482472677', '20426712.2314*1474029799', '23646581.4931*1456310033', '70258762.2646*1474347822', '8994401.06241*1457553896', '68052924.5281*1478748976', '98650616.9274*1473845147', '42216971.932*1481233819', '77515246.6984*1472061894', '89178229.0705*1471390238', '40874099.9101*1465229444', '59402391.3714*1486599838', '17529285.6082*1455208784', '41462987.7791*1466323308', '32999505.0109*1455479540', '1780085.97522*1456421920', '8677844.20075*1480383971', '11335848.8555*1483067824', '54323520.5362*1456507272', '93972012.3533*1484030988', '79478723.6322*1468229011', '5268617.75501*1471445654', '47671505.5771*1485458082', '23093085.4567*1488261866', '7231948.66792*1481622876', '63040662.9195*1465105342', '44094180.1082*1480212333', '7250387.88043*1456189648', '42574884.9977*1452635199', '22209332.2318*1471018794', '33696783.2092*1478078169', '23233890.1011*1454264344', '11855231.9459*1464107365', '80562384.7385*1465035053', '85274970.6388*1453008152', '96489882.8482*1465353493', '16155271.6937*1473626948', '54388897.75*1465986582', '35725224.0345*1474553993', '67142349.4772*1454902631', '95384660.2171*1485717623', '31353195.5415*1481539497', '41867632.1235*1473996827', '66989510.7975*1482337598', '94273102.9855*1486366916', '99682618.2568*1457931355', '44870503.3099*1486347903', '70220878.3538*1454399632', '13056182.7906*1486441248', '48526630.7402*1477700433', '29123752.254*1471313561', '1639867.73041*1477905530', '98528135.6044*1486036523', '7361991.03856*1483716932', '13571772.7939*1467780176', '61888769.5445*1482110701', '23831938.2282*1453782354', '98219830.5049*1452420153', '56187614.7715*1474284781', '83509340.3151*1479676058', '87569590.6083*1472232329', '72591693.3998*1479590085', '3889088.92455*1465860113', '78963182.8532*1454831361', '47768337.5099*1486072473', '85274360.994*1468588807', '22321592.7839*1451654044', '97974603.5916*1452595587', '26555529.4625*1466513720', '60948428.8322*1465516699', '58806934.9056*1473171133', '65119080.4569*1462030231', '65951605.1197*1473527218', '45906414.5094*1463994609', '61872294.1564*1482867817', '55597262.6294*1464901352', '3450540.23222*1453999266', '73104832.7631*1461386614', '18856178.426*1466679066', '98302026.8736*1478683718', '77298328.7422*1470608278', '18029961.7564*1483786875', '56862328.4081*1477985534', '41219826.1543*1484377113', '23556742.9937*1467893360', '84004582.9048*1484754101', '43781467.3993*1481024609', '25235919.7262*1452581508', '64414445.8949*1485128005', '70605136.6764*1478634406', '18603102.761*1472170929', '18889096.4618*1481728579', '8627256.4594*1474458345', '14740536.3759*1479234164', '78592557.0408*1471160839', '376484.06604*1456075049', '37476950.1057*1455894400', '32538057.3426*1456719327', '73236587.8728*1484877603', '88602665.39*1451996117', '18524821.8855*1480608659', '99997313.9013*1474202301', '14837044.1845*1463196663', '91259969.7358*1479782158', '81729749.6191*1470355663', '64638110.142*1473655601', '16829651.0422*1485659102', '32994178.6678*1462044144', '76210336.8919*1483234339', '62618888.7134*1472781417', '84891733.3326*1483537792', '94674558.6491*1466027738', '56821311.7823*1469278271', '2915991.37432*1477148559', '99071328.5089*1453709033', '33140854.2246*1467008442', '39510901.5206*1452879203', '71714522.3957*1479346611', '61237906.3278*1472570853', '26524647.5948*1473017955', '73759775.6769*1460847350', '84401576.7286*1481506293', '3563459.35301*1451778692', '87971575.3734*1476987326', '48948388.7366*1465449276', '93418266.0823*1454189142', '23233890.1011*1454264344', '64987326.6544*1487175601', '15219405.4553*1467408338', '6941859.13498*1463488980', '70605136.6764*1478634406', '32798939.2584*1453080014', '14913600.8328*1480967940', '89585306.1458*1487662683', '5731817.78427*1459018458', '64465062.49*1454656442', '65794125.5879*1475312323', '62910530.5375*1463667095', '77133785.4574*1483059504', '64854062.3459*1473050078', '3793669.13941*1464965095', '48986862.37*1471678032', '8834798.40397*1484181511', '61729563.0419*1467247719', '90343733.6696*1487226052', '96237406.7083*1487161948', '2256168.45051*1474523662', '50243045.4688*1458873282', '44156743.7549*1453463349', '76363739.9439*1456684415', '47911649.8354*1483923040', '4517798.04964*1476034988', '44109377.7166*1453894016', '15201665.4545*1465032833', '23440450.8276*1458302673', '49207985.0055*1457584969', '18947357.2999*1475348880', '14138453.3769*1478681652', '87778597.9743*1472689910', '56823126.0969*1461277210', '65311960.2904*1463143424', '55168300.136*1479629063', '84298220.1862*1471964984', '77481050.221*1469284557', '46481601.6725*1451965797', '27610179.1975*1461152456', '18858520.3046*1481051483', '50968373.1*1473159829', '71482297.9363*1474791425', '8909684.93561*1479918695', '29303232.0376*1478514041', '95539075.4215*1452175467', '35052325.1513*1453206175', '31080687.9442*1465416045', '83246378.2568*1486673920', '7640045.34904*1487127625', '76006518.4865*1483042546', '18311510.5631*1458666607', '33038667.823*1459708155', '75903850.2768*1480193422', '31617131.2055*1475004112', '70658281.6382*1459613912', '86632399.2676*1452921598', '58438292.1359*1480162967', '60462710.9216*1472306731', '5352755.79068*1459329538', '86844484.4445*1462164203', '41909166.1548*1464416839', '19654662.8403*1468182616', '51012042.5194*1456676710', '39676320.7373*1479561722', '41234011.1116*1473514790', '1847108.91397*1458828641', '98874581.5531*1469212867', '30658017.5225*1456963654', '11580070.643*1452794792', '99150597.5808*1475444036', '39871145.3849*1459374470', '93703811.6542*1473110223', '15670968.3853*1470234999', '65998005.1491*1478678871', '64456534.7223*1456387692', '39668231.3634*1478294588', '38464729.5197*1455751592', '79160062.1378*1475773137', '2874677.63036*1457506357', '71467898.0859*1464793236', '17287026.2407*1469743919', '75176483.4925*1485166235', '77877921.1771*1484665770', '27190193.2494*1470639632', '82146649.5574*1452301670', '74841885.4333*1461649645', '39323094.9643*1456610511', '38891453.6938*1464094071', '69019048.0268*1460464582', '45025815.135*1468692367', '75249381.7892*1485508286', '20331316.1004*1474899318', '49326686.8044*1457375605', '18444014.6456*1484064607', '12875229.7745*1479126434', '71623757.6544*1468525293', '34204253.3876*1455626153', '42126421.8802*1483553720', '52098535.1173*1469584712', '41622006.4807*1474446745', '5638702.29338*1475033693', '793615.597012*1472868286', '69959039.0878*1459145455', '18963092.377*1460085681', '34838439.7135*1479818801', '99864340.94*1486324101', '56466742.4858*1460439844', '54260838.0452*1481583223', '48606620.7997*1471533810', '7474959.33302*1479238018', '20132269.271*1485500376', '22330375.9761*1487219415', '49556650.2074*1480952789', '78762924.7879*1481258310', '34918676.4457*1451739437', '64730750.9661*1456296943', '21344804.6672*1466877143', '75221162.9921*1453631436', '60936179.9818*1474421352', '59553196.589*1467595107', '7191412.57471*1458165031', '759354.085838*1466050966', '88087526.3642*1483410876', '19462380.7309*1472717024', '69554687.4649*1455771738', '12193393.1902*1458371850', '71459095.3171*1477148147', '25629339.0504*1457446930', '77594087.079*1477499076', '62654321.4975*1461616667', '49555908.2145*1475492217', '35743551.8138*1452528419', '15787575.9162*1465952443', '21628884.5853*1475740862', '40498637.1995*1452955306', '62419437.6214*1453016507', '83334658.8308*1476625337', '72333786.4265*1461517870', '28369019.9223*1474803041', '27814127.062*1470884033', '9744944.23327*1452495175', '64713715.5689*1464085772', '88438112.5281*1469150441', '32784352.2038*1458670847', '77178912.6422*1461096926', '63836428.2831*1468168005', '76518099.1885*1454299175', '49605867.8351*1487365740', '44231089.9529*1470523142', '32070546.7382*1485498360', '88459682.4312*1464067601', '35457101.4896*1483105769', '11475341.5491*1466229638', '9868674.44574*1477136236', '98275009.9601*1466257500', '47030293.3045*1452076829', '32764877.3682*1465308903', '87426316.3732*1452054902', '16918059.098*1473045439', '48347267.9693*1479595970', '45520395.0755*1466003047', '44798080.3672*1460881925', '98187455.6721*1473748610', '44753412.3612*1477421456', '80690969.1524*1481267100', '89719460.1986*1462792975', '43519498.0845*1468913113', '6862858.52187*1478356003', '8671152.70018*1472627836', '32122553.1783*1485498424', '32067033.0322*1486262568', '79821341.9956*1472958374', '51222884.8239*1485529957', '14157231.6025*1479433223', '73640698.1917*1471930975', '63832906.0624*1486817904', '70725581.838*1483491558', '83699747.0656*1459890269', '67234907.2184*1458365301', '14004714.6733*1456179726', '13470415.363*1455620175', '19444859.537*1479567425', '90671975.7072*1463649108', '13351441.7798*1460019039', '49555908.2145*1475492217', '33791506.0903*1477648528', '76676634.0751*1474972546', '69376706.4977*1464684937', '48956507.8992*1470974157', '37776655.1287*1467300775', '47296378.3303*1480141589', '33697965.0631*1457012287', '49395328.1539*1461862978', '18861357.1363*1456359902', '52598529.9347*1455632296', '24521539.314*1465695904', '8570928.16702*1462182029', '40647622.2441*1473754904', '65744175.4451*1461580669', '22965636.5828*1465087911', '15009840.7659*1484614091', '92493924.1806*1472123477', '26165744.5674*1486732184', '8308690.64912*1481645388', '44279464.8049*1454370320', '8418037.73131*1452195076', '9116538.83489*1466534660', '49250004.3231*1454063311', '15887556.5598*1458456102', '44383012.8642*1482380648', '36415986.6365*1469679775', '10711546.0681*1469017087', '44615817.8495*1472837555', '74185249.8506*1478866558', '76623615.285*1476538184', '73290024.8763*1456529310', '18519857.2603*1464773046', '3908894.72214*1466886648', '39939271.5865*1487733734', '60841142.9259*1465241988', '960806.454337*1455575812', '49404599.0455*1483706962', '97811158.7984*1479849263', '27610179.1975*1461152456', '42113684.7671*1471730203', '25663529.175*1462797082', '20640286.4265*1475088315', '47336401.5827*1478830077', '84027817.7083*1460437660', '21628884.5853*1475740862', '61384721.7253*1474172102', '77550073.2566*1468568480', '81812674.9786*1475966864', '2799676.55313*1476353255', '70049199.3519*1477437780', '55007284.1564*1467401481', '31515294.125*1468290124', '16886001.5462*1475196297', '55157861.7095*1477720255', '39090224.9103*1487885193', '76313506.1349*1485304508', '85055884.1133*1465062924', '12779728.7423*1478906970', '64140074.5292*1453042834', '2814080.7726*1455852426', '52273651.8134*1470501176', '96716746.9464*1469745895', '70782388.9378*1474727582', '56862274.1052*1477288093', '88913695.8922*1484935659', '1228329.75932*1455635498', '35524366.2528*1486297640', '41000845.758*1487661706', '29649556.8379*1481274453', '41867632.1235*1473996827', '86067736.4486*1468376643', '63629000.4682*1467507927', '16251288.3722*1472349289', '2495877.12106*1474643154', '85081928.0081*1471986269', '29935984.6231*1478674633', '20943273.2077*1476899774', '34252242.0411*1466546424', '50433930.9987*1455065233', '11986612.1554*1479007929', '6503222.10632*1462779102', '79350136.6694*1453953288', '18891475.4167*1465273758', '49606550.3411*1456223773', '62154177.5383*1472386394', '41656657.8809*1456756967', '71330729.4932*1480184542', '93409788.6115*1465654334', '95646273.3507*1483211638', '33829395.8089*1475640018', '65265157.4531*1466297501', '82049206.0086*1462381279', '42134105.9586*1478705433', '90578690.3162*1460303860', '87107530.8022*1455080942', '24138705.3756*1469342830', '57982742.5497*1452141507', '53174746.3855*1471480174', '69815271.4807*1480527782', '73639210.9731*1467635508', '86021413.1202*1475531931', '68913234.9132*1454029420', '667917.43775*1455011208', '92414842.7975*1466487602', '64624629.1017*1455611975', '62748048.9967*1483742448', '64124746.4165*1471029509', '46182526.7082*1470813734', '83719480.0845*1477341785', '82139867.9197*1483362949', '91604045.8698*1476434768', '2150217.29141*1461589171', '38682793.9031*1452492711', '63327059.7133*1470713621', '8501594.64097*1467001731', '56262583.4617*1476230469', '73062702.5905*1482816355', '74352589.8549*1482936025', '24370385.627*1457187172', '21026473.0625*1452923510', '35457101.4896*1483105769', '17869130.5628*1478578796', '68864635.4072*1474808515', '33743105.4877*1458860918', '74140714.3052*1487114692', '85022200.0119*1452718442', '10252087.8402*1486052064', '24839110.363*1475501647', '8431901.93849*1455679068', '6048637.06475*1483484926', '73712073.5959*1464868998', '95228731.3307*1454549075', '44816658.4346*1460670046', '55298308.912*1455290987', '89921449.0799*1458579599', '7858082.24031*1483052846', '20765244.7947*1479171682', '6365864.74411*1483074134', '65626625.3484*1465312576', '28758331.9539*1458080011', '89200633.7529*1462727976', '3924411.03703*1462469031', '65014295.1033*1474692794', '54522397.0191*1453672945', '43081768.9162*1475416352', '98930802.1121*1472071201', '26909516.0784*1465166019', '3901751.50581*1469220902', '77292347.2756*1471606212', '22246456.923*1475276820', '22536331.8076*1471868208', '59246781.8693*1466850404', '86753260.286*1471303970', '69440790.6041*1470679262', '48748583.29*1482235774', '48499185.6454*1465050908', '82765839.498*1474602919', '8994401.06241*1457553896', '32006603.0844*1484272794', '62801301.87*1466918112', '51522661.6873*1485592474', '99133867.1033*1464044953', '3288188.3094*1458567682', '13299106.7607*1481317148', '15185032.5597*1478219245', '24370385.627*1457187172', '25473523.0579*1481399430', '84287453.2342*1465329895', '6048637.06475*1483484926', '77383577.0024*1464122694', '53766357.7378*1455653691', '77898462.2261*1486405211', '77231319.0386*1485428471', '71138287.4902*1477910013', '6927106.50435*1453521824', '51289101.2173*1465647999', '14850697.1996*1470552236', '15115416.5125*1455938890', '53362447.9035*1462111884', '16747969.7718*1483163953', '4252680.92443*1468077395', '11984482.2681*1477957353', '17287026.2407*1469743919', '60882892.6282*1461128664', '84832908.6902*1454443781', '41713719.8656*1480427548', '61770511.9473*1459218006', '70252004.796*1479767061', '29775851.5166*1479999731', '10576320.4497*1463047209', '14292659.4042*1477861943', '44040479.1712*1473584003', '72642053.5227*1475841735', '23221730.605*1476343022', '11730366.5222*1462329812', '71245171.4418*1465097444', '63027505.9844*1479069308', '16227519.3201*1479648048', '14417261.9795*1476935042', '91669892.441*1475790181', '74286820.4051*1466886639', '34665001.5475*1483954045', '31809905.7405*1467215637', '37418951.728*1484459834', '29802813.1828*1476134834', '89007729.5606*1466681406', '23990221.6126*1455021748', '5352755.79068*1459329538', '48476105.6787*1477033093', '3952540.73304*1482227442', '82851575.1553*1484877558', '45319254.99*1459658436', '44671317.4353*1460051329', '92844190.0874*1470012338', '94373571.569*1487933267', '70558547.5969*1465057205', '65723673.8248*1467149436', '1434734.59619*1458565064', '10783361.4164*1458532652', '68898669.2456*1455945213', '370758.810013*1467851147', '70983704.1805*1487733265', '72396564.9124*1474570798', '36787421.2173*1470298861', '60604702.408*1478652828', '65248526.7447*1483658318', '48928224.7186*1461081903', '65810724.7802*1459955613', '21985536.5005*1477771397', '78279643.2051*1457323764', '49975039.0057*1456605827', '90085340.1668*1469268298', '16655919.6917*1459861690', '61435180.6036*1462634565', '15751307.8532*1451844588', '81650528.4213*1486163854', '48336899.2536*1467541141', '9169947.76002*1482257507', '71378477.3428*1471218352', '68030336.2248*1469111040', '75542131.1136*1460200359', '57963182.9013*1488235718', '44823037.5073*1456495496', '30040375.3765*1463874747', '41120317.1972*1477140014', '27220432.2075*1463625875', '96077795.4482*1461726416', '67246312.5897*1487071005', '12237989.9971*1479182828', '57707520.4562*1464747496', '48927033.7108*1469859783', '2013383.96344*1470230851', '5164299.67087*1469777413', '5118700.52666*1476486022', '79381643.6109*1466070618', '74815526.7621*1462639826', '3345287.71662*1461034780', '75684035.2785*1485151447', '6111930.10592*1466235461', '9689650.27474*1462041061', '35944553.3003*1452425880', '38068418.9334*1470764886', '27939326.7162*1470256393', '96784463.8612*1481257591', '73358193.7546*1468098533', '2214587.89712*1470400025', '92157440.6298*1455063908', '75081279.8567*1468409114', '7610355.92556*1455221730', '35183757.0781*1475090917', '82090075.6029*1483285594', '47089972.8464*1463888131', '60149197.0436*1474444277', '67327055.2179*1454791437', '63836428.2831*1468168005', '44912599.3669*1472643398', '42536628.6442*1461465251', '86461889.9613*1473031363', '95848767.0183*1471539718', '66715410.9943*1477060798', '83146552.0452*1478441481', '46506049.0241*1456794960', '24192859.7757*1479819729', '25284989.0042*1468340148', '33371437.8536*1478092551', '42909314.0897*1470945080', '78391235.5155*1486257097', '50985622.4763*1456716664', '80071769.2233*1482812553', '72193525.6286*1465359192', '60049175.8908*1483794668', '95229613.1116*1459259673', '87431555.3674*1487873701', '60802887.8696*1483558510', '2819927.3801*1478654287', '65163341.9653*1487181430', '29824502.8904*1462210325', '38442017.3459*1453411981', '13067977.4341*1458796114', '32665776.7241*1453332005', '40582605.9598*1454577825', '16918059.098*1473045439', '69052826.2204*1460592484', '20595231.494*1467264447', '70785099.01*1455171344', '60267123.2032*1461364883', '12320194.6217*1478032243', '27795840.2522*1456193229', '43781467.3993*1481024609', '19352269.3244*1481796763', '69505262.3876*1488134169', '85689094.3612*1461057913', '47217432.0883*1469776948', '5971348.41758*1473411852', '22130675.3231*1488306067', '15697450.4834*1474605856', '52507236.7254*1481896176', '6410110.70358*1459875124', '67004209.7623*1485318177', '72486989.1729*1476973394', '7432345.23148*1480145612', '27306061.1007*1471366486', '95041086.5025*1482204854', '47142579.6302*1470725646', '90132976.9744*1484090218', '51945472.6763*1457806389', '40499653.2491*1483450158', '93165958.6984*1451949746', '6752644.13003*1469317311', '48761354.5192*1457951708', '62654321.4975*1461616667', '3487722.36978*1465611937', '35353772.1256*1460590908', '25757882.1442*1453865218', '43496679.5034*1483012879', '31350561.8864*1457386756', '62457960.6735*1456124267', '55955577.2964*1470171697', '8149794.38766*1480705117', '13501217.9997*1470699057', '15904145.8236*1466926640', '7150146.41317*1452441177', '37479108.5803*1485858690', '49827225.1656*1487332484', '11794016.7285*1455154618', '24377538.6028*1483232867', '90681584.8874*1487350889', '31902355.5813*1465071140', '58806934.9056*1473171133', '58980905.2962*1474651070', '28685839.1997*1463716705', '44626475.4738*1487352159', '40118510.7403*1462426982', '12419044.4908*1481905700', '42025973.4373*1481585673', '54014079.8809*1483934461', '14997111.4785*1458187487', '73767115.3625*1485092355', '16788396.0455*1485128077', '70867917.2915*1460940707', '28382005.638*1487675569', '21850095.4484*1459776343', '10467025.8503*1454531087', '51261974.942*1485108841', '3791890.9688*1472875589', '32019019.782*1461344593', '29187652.6009*1470785443', '53399283.2075*1479316267', '73333279.3768*1461641937', '74001168.0221*1481707161', '73383401.641*1457228199', '47413695.8166*1487740810', '4332732.18458*1459201442', '37174026.8509*1464377667', '13652648.9018*1484472068', '44714173.4402*1469572151', '370758.810013*1467851147', '82715773.6869*1455707008', '27824798.4143*1467172699', '97189023.5359*1474045474', '42315805.586*1468473766', '32778755.1945*1482409181', '53243763.5485*1485507315', '46423891.9383*1452807355', '25640807.9629*1457612920', '36663874.586*1477435190', '58874336.0544*1453561734', '73568922.2668*1484696663', '78728983.0399*1487837423', '85838074.7451*1479703237', '5065989.47875*1478281513', '36762614.7899*1468361049', '86357890.9523*1452463430', '78987139.9249*1475438878', '42245881.2255*1485469991', '70383142.36*1476829288', '82781264.5321*1460910220', '53710862.0087*1477232637', '93716648.6438*1487581696', '71063262.782*1455910201', '28673667.0094*1478702623', '84253882.9671*1478754471', '66036610.0131*1459934967', '34648240.8914*1469794798', '13828462.1604*1482550800', '86436996.7443*1451893789', '6410110.70358*1459875124', '63956772.3283*1480868474', '82647572.3591*1479217649', '71842768.6835*1487100107', '25671688.4062*1465407137', '56220358.2882*1486153211', '9098161.55501*1486171952', '96965739.6331*1479687734', '35706964.5721*1461251785', '7520300.84858*1480160797', '24741146.3619*1472814889', '15052537.241*1452345021', '84777697.9589*1469990335', '54774931.17*1454308486', '57861321.2114*1486642093', '23326124.9182*1484645183', '62207621.4091*1480289842', '30106894.4575*1476068210', '40030225.2612*1462840610', '69744118.3656*1451860532', '68646853.7625*1478178183', '49532057.1409*1473315398', '55660237.637*1470966514', '81443190.5825*1468721554', '53451629.2621*1485259402', '46180268.3167*1483811839', '13172646.1935*1453331761', '76784609.0776*1471194511', '82013465.1276*1479094259', '31146582.9386*1455180106', '44279464.8049*1454370320', '21795883.3331*1476243208', '31402406.7422*1458180319', '69186916.9338*1460475838', '29071977.7829*1487343906', '10159965.6991*1465419075', '74412069.1667*1476375260', '4783086.29175*1460999664', '56575394.9394*1460718388', '52251300.9418*1453236054', '22746847.93*1483987206', '86712772.3651*1472694661', '18361281.0226*1455060771', '67116770.525*1459950627', '67846794.524*1463181398', '92157440.6298*1455063908', '39419717.9403*1457863269', '99428882.4826*1482072074', '92741986.4473*1477053068', '82334956.2892*1467708359', '14996264.4875*1462576618', '83239218.9486*1484651043', '85516523.1456*1475229412', '51246774.1791*1455436764', '84995797.454*1458983663', '92966857.1922*1454530986', '43781467.3993*1481024609', '13692128.0996*1472867294', '48964886.4671*1458214196', '67709531.552*1471640420', '42092014.2641*1487960824', '79378254.6055*1481890568', '77783328.4306*1453175254', '99996831.0841*1488213267', '31252141.9292*1467914568', '44912046.7457*1469893396', '47054711.9801*1456729774', '32356293.9024*1452946274', '92662244.4601*1485829485', '86465135.4894*1452659005', '64865651.7616*1485778001', '62640237.3128*1483811634', '46128030.7212*1464997374', '8019312.86798*1466353797', '48449051.6977*1469670356', '32974260.8019*1484821041', '68364746.6034*1453860255', '69826732.2741*1463859525', '86719826.1886*1472141310', '97882197.3289*1466260849', '67226782.0109*1459403517', '23203775.5085*1470525446', '73638275.7084*1460834613', '14480565.3234*1464642869', '79629106.1086*1452801801', '329912.61587*1472090281', '380239.136078*1459705100', '76500172.7073*1465834382', '40944404.7957*1480064490', '45090618.5572*1468151560', '41995991.2916*1459207032', '83955724.443*1476981819', '78368167.333*1483616470', '46963375.3065*1458486364', '66951686.004*1461887466', '5226595.70546*1452693028', '11483816.9926*1460728584', '27167748.9295*1452782503', '45189063.1195*1485386064', '77695337.7596*1473968722', '79398088.255*1481996033', '5373412.21483*1462509571', '83354671.9542*1485113412', '56884936.6152*1461187550', '75881205.9038*1459974594', '23990221.6126*1455021748', '99534064.3341*1483565251', '63139029.7208*1486931588', '88725301.4553*1486126701', '22227311.7943*1467527448', '17832039.7842*1470598239', '89474613.9485*1479370894', '40662359.0731*1481590094', '76972716.5361*1467153028', '38927366.5023*1468200913', '82508873.9498*1474879575', '39823355.0431*1485267186', '5928771.41704*1483346964', '21176069.0545*1459222993', '75651645.0523*1487044041', '41151399.3234*1487567795', '635715.144042*1482709512', '40045531.5135*1460128805', '92387860.6067*1479835293', '23118525.6833*1462446757', '58543111.698*1468951769', '66346746.2436*1472390407', '6560980.27615*1485115941', '21822018.0213*1456846869', '49992325.156*1461164437', '92410073.1037*1468615461', '67218155.6231*1475459910', '25839776.85*1460463590', '4764975.80551*1453056246', '80349643.359*1473681260', '58777872.0445*1483129201', '70229156.2209*1462339312', '80378205.0409*1457975195', '75113323.5794*1457689085', '20613865.6331*1485243537', '64558682.2323*1476407389', '91244913.1955*1454454649', '63989856.4601*1475010101', '91483252.6015*1464099685', '92410073.1037*1468615461', '15720008.1002*1459025940', '14994009.4493*1452635710', '90409138.2177*1484329445', '32095186.135*1462665453', '43011523.5319*1481187027', '35232454.3123*1473520625', '49502137.4235*1476316890', '92829024.3924*1459713634', '59739137.6267*1486022269', '89806305.1802*1476299959', '19944187.1282*1468796505', '85797316.1214*1471492142', '72702557.7642*1473339304', '40734097.4445*1460952109', '71660948.2801*1479671230', '23203775.5085*1470525446', '24044070.0732*1481982416', '72383020.1772*1472440877', '77302896.8165*1458543585', '42276129.0462*1458249037', '36065449.6453*1480025182', '82583852.0655*1466112214', '50076973.7773*1487650200', '66655757.9616*1468734655', '1738834.32658*1458495391', '22286824.6505*1488012521', '21574984.9638*1459093907', '37999953.8672*1483472562', '7879223.26966*1472142361', '43042265.7317*1474708238', '59326251.5508*1460139735', '46281731.5368*1484454672', '63779210.291*1474819687', '22330745.6666*1458884024', '91968795.2498*1480811690', '91833097.4955*1477928033', '52306005.5467*1459151022', '48342737.1243*1453122069', '80051106.5146*1457698520', '78603828.8223*1454419884', '35256173.0366*1486096172', '39579252.1888*1469208281', '24278153.9388*1481895579', '35739232.893*1479401753', '2214587.89712*1470400025', '29780275.8617*1461591469', '24108288.8822*1458794400', '14323981.0923*1469831449', '96327015.7411*1478166202', '35822016.983*1473952196', '51262946.597*1460975253', '88201181.763*1485290437', '21133726.7216*1480403018', '84867697.0917*1465559814', '87201295.0264*1469950543', '71229124.6774*1464105310', '20374163.1335*1485361372', '5342563.90814*1461538441', '56529833.4681*1481838125', '35719453.9382*1469686278', '59521744.6955*1484253484', '70386632.8369*1472987703', '78155827.4959*1466075974', '59479241.4665*1484433617', '80161913.702*1485074093', '63532211.141*1486969601', '50859133.0687*1463431525', '45218392.6435*1473747696', '33444025.0997*1466347846', '59586121.6481*1483621568', '53309445.4195*1467043289', '91397403.4764*1488319813', '47209079.5618*1459485137', '67057206.5969*1468871484', '3820424.65999*1455693764', '82135409.0478*1460674661', '56103291.1772*1474714536', '20637673.0882*1469697234', '87585149.3396*1466562365', '83509340.3151*1479676058', '14899334.0461*1468720152', '30772628.0186*1471966273', '81262696.0682*1454594623', '34838439.7135*1479818801', '26272361.1593*1483559726', '73146839.1291*1476092722', '55944885.2546*1464288667', '26284134.9319*1453386744', '76790469.5403*1470102373', '28592237.7637*1470819182', '28285901.3783*1458167732', '64232241.3226*1457765861', '72616520.8854*1481395026', '14435698.1195*1482173925', '36231343.7046*1454360848', '29256487.4402*1476043976', '1338706.89571*1473585990', '40583276.3721*1474848672', '49266640.9164*1480428575', '49475025.0694*1466093864', '67327055.2179*1454791437', '26871653.7299*1460636111', '29816836.3016*1487223382', '1770646.0452*1471870712', '48499185.6454*1465050908', '96696547.0994*1486260979', '37387421.1976*1465400207', '32079966.0194*1476158920', '17204781.4569*1458929008', '74185707.8621*1471273805', '78043857.5945*1473506952', '52019351.3587*1487816688', '26482404.2587*1481806246', '74517592.4077*1487500062', '95207302.3007*1483269589', '33791506.0903*1477648528', '45319254.99*1459658436', '88851446.6026*1488021081', '27805571.5451*1473241391', '73767115.3625*1485092355', '30013490.3424*1484740867', '21459340.3251*1459384011', '79458061.9486*1484563639', '64362369.9534*1487812740', '14251620.589*1476255624', '19585046.9099*1461843055', '273920.33823*1453361398', '28305732.1842*1468722351', '93402632.1275*1477512614', '74640284.8075*1465146265', '69941559.6209*1482982241', '84317837.6191*1474288100', '91292552.2863*1472778773', '45653467.7467*1476210899', '70666631.0342*1459250533', '58331337.9145*1470420940', '55544549.0627*1467629313', '78977513.8639*1476158446', '91480949.463*1470402205', '83450751.9655*1463202675', '91196122.7906*1483538677', '28301833.3274*1455864599', '11163625.1241*1452287620', '53023884.7692*1461118595', '91592685.916*1466206275', '4440050.29981*1470026319', '90872003.9911*1488222010', '28378127.2649*1485637529', '17363664.0752*1479283005', '22887137.3841*1461169513', '11880225.0999*1479879394', '10440294.1531*1473337341', '85530373.1922*1475837084', '74665103.1277*1484812665', '31757596.5298*1461607890', '1271940.09203*1482655853', '89983261.6022*1457453427', '85375910.4934*1463214331', '44312579.4894*1485562885', '87412614.039*1468803396', '28933663.5341*1472277075', '73249072.643*1484684377', '76485387.5309*1470194613', '42582524.8049*1463910293', '61560264.4338*1485341337', '96460939.2406*1481000161', '10288511.237*1466193825', '41847105.7698*1461500043', '22263531.7273*1462747521', '87826052.0435*1479310813', '34728054.3895*1471471745', '51437852.5062*1453353213', '13205458.531*1475420105', '72352730.1221*1476572881', '85964531.6554*1454888909', '70162003.6514*1472546298', '18312406.8571*1478618704', '165417.448246*1469200695', '57939626.0296*1456269238', '44800059.3528*1486506037', '34598976.3456*1461464892', '33094223.1643*1479292764', '44980986.654*1471573760', '13132259.2355*1451816738', '83068653.5222*1467807964', '11891597.486*1472127565', '21985536.5005*1477771397', '41959903.9304*1455237667', '19196263.37*1451752723', '12861620.7441*1467878220', '11425033.8461*1480655997', '44213845.0904*1477207043', '29991811.8976*1460774926', '52347749.1586*1488225550', '45866826.2412*1463291330', '38935126.1364*1482230672', '82663448.5507*1473272789', '57002267.069*1476224878', '42791351.3882*1484077297', '30804969.9008*1463826925', '19275912.9618*1458326204', '231955.550766*1471213973', '99247891.3756*1473248832', '53451629.2621*1485259402', '96744067.7824*1462932035', '55472097.2799*1481642413', '847048.698649*1478321187', '62053316.91*1452468868', '20348868.3957*1452918657', '32262072.0499*1470340240', '14163642.4176*1453424792', '69744491.8368*1466579245', '96284902.892*1454864257', '40908576.1418*1467265045', '96730476.8493*1486455540', '37428781.1717*1484687389', '37691796.209*1455019389', '82663448.5507*1473272789', '35409468.574*1476412872', '79329274.487*1467276548', '31881977.8964*1459198975', '52306005.5467*1459151022', '18797122.9326*1455881593', '62656719.0588*1461122065', '16899558.9599*1460557653', '79137300.2995*1481857884', '73906637.4862*1470767268', '71025045.6437*1487841447', '45867509.6769*1457412365', '56602318.2948*1469411837', '70680959.0388*1455047051', '24437857.621*1480914093', '7250387.88043*1456189648', '27272805.9208*1459224722', '37512454.2086*1476611724', '82486733.3197*1465842707', '49465041.5145*1463998790', '89440344.2603*1480568552', '33076574.3956*1481224529', '41055296.7787*1457359735', '98815181.6567*1470121486', '39592997.6959*1455512405', '26313411.3831*1473799084', '13848929.0341*1472986135', '73883335.269*1477346578', '6593489.76955*1484777702', '22246456.923*1475276820', '68916332.786*1484815997', '9433432.06035*1486162158', '43011523.5319*1481187027', '24741146.3619*1472814889', '80151125.3303*1465671524', '59043802.4288*1487156974', '38597702.5971*1483941970', '59451566.7979*1457674441', '70983704.1805*1487733265', '22908643.725*1481091632', '54200020.3489*1467778802', '93004864.2644*1480658632', '62838167.2168*1482225555', '79436402.4559*1481896308', '72132289.8662*1485439095', '59400112.7312*1452501428', '44637332.6062*1486631176', '43895641.3114*1481891459', '76967678.8149*1485801525', '73856215.4022*1455601031', '13488080.6319*1486827063', '60343217.7049*1479388024', '74028207.8818*1482418666', '46342516.0055*1481847867', '37578854.5209*1455964958', '43640357.817*1452134995', '2529800.43418*1458841635', '77373727.3486*1459142317', '1979133.33152*1456639374', '11606952.3216*1475487386', '60810565.637*1457750519', '96367838.5497*1470361031', '86760512.3427*1462440719', '14582603.4313*1467044431', '51685516.3413*1462383970', '52051558.6886*1487548766', '15732608.0131*1463157418', '96332050.0521*1465531163', '64257901.1901*1453513833', '25457689.5071*1488299995', '34365192.441*1469318659', '83403156.9708*1468862006', '21009385.7307*1480462336', '76210336.8919*1483234339', '86556196.8779*1488324413', '8451361.39012*1481182888', '33690930.0111*1456450030', '15104771.2997*1481469932', '73103910.0751*1486732074', '47752183.6409*1456812678', '83613115.8926*1471331774', '87382350.0823*1485236479', '43837264.8905*1487958755', '63107978.6301*1455819831', '66285217.5183*1465480589', '84977563.7086*1476711830', '50538266.745*1480598836', '65315502.3478*1464239137', '250249.378079*1470247804', '84407327.2408*1455428264', '43081768.9162*1475416352', '24138499.5385*1484143884', '14480565.3234*1464642869', '67656570.4102*1470904721', '94257958.5498*1470293632', '53782174.0919*1468993430', '72642053.5227*1475841735', '14292659.4042*1477861943', '48877567.6026*1475010812', '9018894.3366*1459309032', '57895310.936*1456334038', '98964884.6679*1484233421', '83642962.0042*1473027213', '24263805.1712*1487199104', '32722085.7537*1482056736', '34156907.4423*1454413032', '19959621.5413*1479407133', '74636029.3464*1482402046', '9512734.50698*1467780174', '25891630.0799*1469764471', '24776896.4985*1477836558', '47362088.8185*1483048387', '76282219.5044*1468119498', '41557549.8871*1468311941', '65248526.7447*1483658318', '83005385.3472*1463337679', '23254586.1626*1461352506', '49266640.9164*1480428575', '85944448.2941*1471793924', '19388324.4763*1457509152', '18155572.174*1482967176', '84010591.624*1455068032', '52873155.9516*1459259390', '60491602.5134*1488070752', '70048855.0856*1470889240', '89307721.198*1452336993', '25560681.4078*1458415790', '98394015.782*1475125319', '77430075.5718*1455299230', '45603647.9129*1475322717', '69419159.975*1479570522', '41277530.3176*1478287047', '52695329.0583*1483307602', '92817723.6389*1455717073', '31926519.0842*1474533788', '21205402.4665*1474728050', '50939588.4691*1461740019', '42132725.8889*1483164533', '47382494.905*1485473614', '8767592.95345*1475137192', '7262623.87706*1484531362', '54745003.7415*1470679703', '66108349.0964*1472802889', '9402688.34257*1467736192', '81909613.092*1467829609', '74486802.2976*1451905921', '42113684.7671*1471730203', '31809905.7405*1467215637', '8977745.98309*1471669561', '65245311.4403*1481320989', '93225754.9394*1486953073', '33126479.3581*1461641602', '6517139.2162*1470172937', '61934401.8584*1481166968', '94565390.4055*1457800140', '88975843.2321*1453683097', '79548973.0369*1463970024', '21208393.84*1486809600', '86748228.8524*1465672569', '53642467.6157*1483394208', '24899504.1657*1471856824', '72132289.8662*1485439095', '74874340.4655*1487597165', '93721897.0204*1482757364', '9248691.63313*1485695042', '55731686.7835*1483700101', '96767876.6536*1486528418', '24319820.5297*1468232872', '96748154.6527*1456366084', '50557211.1975*1476077154', '56898659.1988*1461014012', '58344171.1161*1477887330', '4324215.89677*1452337219', '18168186.4379*1456370992', '61253893.7027*1465837350', '42123010.8573*1460502363', '41543343.6721*1474225832', '56021485.3025*1455091667', '82921146.4861*1454438240', '67241130.3826*1474675300', '64900687.7119*1473286352', '81744532.6937*1484043244', '78403119.0534*1484111886', '80337072.2906*1484936478', '16214027.2829*1486998789', '59523223.33*1487539117', '45567029.1774*1454292029', '25521397.1072*1479012511', '39125299.1995*1454385701', '88265223.2986*1464748965', '62290711.0101*1462910206', '11992897.6116*1458901000', '65335467.6906*1456997074', '99718150.1354*1463354901', '12471505.3357*1451607860', '4135130.47418*1466473272', '52023835.8786*1487124250', '6013976.21339*1477307527', '8378926.78948*1474969260', '28424730.1976*1477641464', '71947018.4324*1484887029', '18889096.4618*1481728579', '95511981.3704*1470294589', '65737792.4915*1484094640', '79329274.487*1467276548', '19878684.7384*1472067351', '21316181.3804*1479078418', '5221264.79735*1468793612', '74978729.049*1488062828', '89007729.5606*1466681406', '13002026.1254*1477001630', '1398104.61407*1462847179', '66297341.2722*1481958211', '6558924.30821*1452348638', '79584710.1292*1453249205', '93162673.1761*1486383847', '28782779.7733*1481641632', '10254587.691*1479176697', '6274575.70941*1475817668', '76787761.878*1483112731', '39993774.7617*1476095630', '32212947.3627*1459334181', '98968556.1138*1482726380', '59313071.1265*1478248924', '73228691.9383*1470246741', '9058460.63137*1456548289', '77594087.079*1477499076', '75286154.7372*1459867139', '34428688.5526*1451874436', '72719420.1618*1487874190', '14785188.0475*1475289657', '98133563.0961*1459113523', '90852175.8342*1487519801', '33256036.4577*1467003579', '70981189.4251*1476340708', '63147452.837*1478106825', '57729329.3108*1456556262', '12320194.6217*1478032243', '60898783.512*1476590353', '97290038.1374*1485036424', '20088887.5757*1471552539', '60149197.0436*1474444277', '6172694.38973*1461945735', '40124155.7253*1486535478', '82745461.5439*1459268827', '48713803.4326*1461929325', '14404936.3013*1456817576', '50903384.7807*1481932993', '86682279.8285*1486092109', '49050398.3611*1474347203', '1287597.54509*1478584127', '33641626.8735*1482918980', '89691935.7348*1485220349', '19008724.5674*1454683445', '41609039.4234*1478892578', '51246774.1791*1455436764', '87745232.4426*1455019422', '63917403.3095*1455712092', '83106658.2271*1455443280', '42542169.8532*1457744397', '51231832.2641*1459320441', '38787515.8599*1467463553', '1100981.62472*1476063173', '91592685.916*1466206275', '71566885.8567*1458417780', '81893092.7723*1462021189', '77481050.221*1469284557', '74967886.8895*1484576700', '54482515.5787*1468637073', '81047253.1406*1480918299', '33373032.6089*1483544160', '67188244.4359*1486604593', '49508264.9829*1472670872', '39492087.0161*1460026288', '69893489.2281*1462195074', '36125635.617*1480180836', '49314612.1762*1462989872', '39903528.7201*1463474390', '82203308.8411*1475580821', '8030548.06573*1478931653', '5388959.56857*1452939253', '97077686.3551*1467111099', '57321780.8066*1458185526', '98275009.9601*1466257500', '29935984.6231*1478674633', '87517576.6712*1474261121', '7741349.81068*1475708899', '40000471.4451*1486939537', '28305732.1842*1468722351', '22910599.9561*1479682670', '50140561.2575*1455148304', '31802292.1331*1479795956', '35248585.5087*1457689257', '37493606.0136*1474245760', '82298884.8731*1461748617', '31258770.3312*1487939074', '44057612.2512*1487608894', '34980176.9627*1468195072', '89690215.5025*1482498588', '3625856.28175*1466775364', '28026783.6473*1461091814', '30163792.71*1472741733', '28030217.2814*1466454739', '66303724.5924*1464337755', '45158146.0973*1486199789', '21650272.1907*1457224525', '2477555.35602*1467397899', '70320243.9815*1478834027', '88169153.3829*1463269853', '71824798.4828*1457227828', '77557880.5944*1459114165', '8284707.66498*1484436275', '38497611.5518*1487075326', '14497750.1459*1455011479', '60919126.3495*1482876227', '65289566.1524*1460764269', '83828866.215*1460883378', '84220370.4646*1481536478', '22044494.095*1483972350', '89457716.346*1478342313', '58721120.133*1475589274', '26075908.7961*1486826980', '36206435.5095*1478357746', '31809672.0417*1466852996', '30635505.8298*1472921840', '99028093.5418*1472370502', '29858863.1518*1471933236', '35211020.0814*1484720867', '19499181.0287*1477951817', '54929201.671*1466258198', '74595826.4361*1484624920', '89867123.0896*1453382695', '21910233.1326*1454520818', '69811188.4986*1455389623', '38574126.9816*1452174023', '62811412.5581*1458218979', '61054352.869*1452845969', '12653268.4265*1464015844', '46287826.7188*1468614918', '61711702.8476*1466381495', '43372611.5102*1484012550', '24899504.1657*1471856824', '9169947.76002*1482257507', '98152400.0264*1478441926', '70583260.1614*1466588750', '71009202.8773*1457755659', '68447760.9715*1466366160', '54745003.7415*1470679703', '26874709.115*1472976974', '53610803.7528*1468580839', '5742884.87975*1479524402', '63103128.3241*1456800228', '22949348.3739*1478660350', '25535872.1072*1454607964', '9476195.27712*1476172765', '16834855.7317*1469529922', '69125575.0711*1462816163', '37716168.7503*1456921458', '18889926.5731*1463594975', '78067560.4004*1472631049', '89009508.7255*1484800332', '28321735.0634*1463795217', '60269200.0105*1478574106', '77949398.375*1470074785', '95616218.122*1463150244', '43888173.3352*1469980972', '40436815.4885*1483357089', '87749474.8611*1471526206', '13348469.1917*1476175856', '48945435.212*1485720392', '30384598.8361*1464386012', '83525259.0324*1466760845', '89806305.1802*1476299959', '25585589.734*1469750859', '51378541.2966*1481053585', '34419864.759*1467380086', '75820738.9417*1483681601', '52895857.5865*1474195043', '50012335.2855*1475081707', '68313058.5106*1457989802', '43963084.9804*1479491878', '20968885.0138*1464639901', '43609163.8926*1458827572', '65318761.6917*1475346075', '64702891.2354*1453082412', '45599355.9552*1467804760', '92150506.5995*1457334689', '74426782.4814*1470647917', '81401220.5995*1483313037', '74886918.7435*1488217898', '3670169.1275*1459134195', '63204420.3248*1461770562', '35725224.0345*1474553993', '21856877.7528*1457518045', '38341204.1313*1472117190', '48331072.5504*1470581623', '802247.861826*1458983580', '44912599.3669*1472643398', '37111295.4891*1453883399', '63027505.9844*1479069308', '48170496.008*1468201731', '11335848.8555*1483067824', '21091536.3087*1456476630', '79567811.3653*1452960650', '98328834.1997*1455222928', '99214637.4154*1475270215', '4533053.19448*1463608559', '98265031.1566*1480677908', '72300034.6613*1480279596', '17348690.8851*1461132657', '14079855.0189*1453299440', '41335447.9078*1469247377', '88012817.2521*1474260325', '32265318.2082*1468255288', '59481592.2912*1460145199', '96237406.7083*1487161948', '94525928.7603*1466887139', '11513255.6968*1472365161', '71571630.1143*1478804442', '28033923.1818*1485928235', '31760400.8755*1477374547', '33144333.9198*1468416721', '47382494.905*1485473614', '50697348.1517*1462003045', '273818.318591*1451710949', '94118672.934*1474186327', '96338883.127*1467115887', '90697721.6388*1453701607', '14338690.6485*1461544927', '77591132.7557*1476507451', '90538007.1347*1455022964', '62221011.0555*1468507731', '64108949.5697*1460321599', '80391303.5486*1473023788', '93367341.5904*1466219810', '94629240.6951*1474287812', '65747821.1216*1479850639', '72373366.5065*1487685234', '66525305.7049*1463851768', '72854505.9221*1481820941', '38938029.1249*1475054438', '72873937.307*1479504306', '81477926.7867*1473237412', '54422617.1288*1476901382', '76650627.0153*1469009767', '4388441.87844*1455702216', '16068249.5953*1486202283', '75846506.2676*1481462670', '19855495.1118*1480041139', '78445983.6878*1461486147', '71752078.8069*1456741560', '4181609.38041*1475023015', '40868806.8656*1477027811', '11719913.1057*1481954643', '48652933.4526*1482954598', '4785821.25449*1476939820', '31529887.4948*1482639854', '88678961.7317*1486616823', '59586121.6481*1483621568', '60445280.9408*1455168260', '24415169.0651*1453284558', '6348884.62408*1457197692', '21572695.6082*1478481644', '99421903.3723*1465486158', '4781492.01121*1453268867', '90436188.5604*1484200087', '38174045.6296*1472736532', '3374289.69795*1451700694', '76635791.5994*1487851132', '76785083.92*1482284703', '39519338.2479*1452214123', '36787421.2173*1470298861', '67004209.7623*1485318177', '31199411.6392*1479929991', '70666631.0342*1459250533', '82034533.7748*1478611757', '24024841.0081*1468313518', '43549025.1252*1472780692', '54308323.4235*1463900541', '84764316.9749*1485461790', '56875541.7541*1466305406', '20318163.847*1477635642', '11193342.3165*1475481616', '38746521.7231*1486952358', '79460812.7782*1464268076', '13254790.9513*1484241922', '31342305.8646*1477519058', '4799494.18133*1471088026', '48103401.983*1463685208', '51947652.2609*1457357840', '62339936.5439*1479806662', '32994178.6678*1462044144', '82571823.1174*1475683713', '62564140.8899*1464604014', '22898237.7601*1488077301', '70947420.7291*1456243234', '86443825.3497*1474211922', '9439599.56143*1452753019', '79191184.0824*1469331324', '86826370.786*1475073509', '30013490.3424*1484740867', '4351457.50618*1460384437', '37176351.9806*1460797185', '98192306.3248*1480468252', '72272398.8191*1475803015', '11109073.0402*1475555239', '41936216.3037*1479224683', '53876089.5162*1476500452', '33225494.9648*1484091996', '74442056.6778*1487670782', '1363059.93803*1473097922', '80031107.754*1463262636', '81408940.2247*1452398529', '93973354.2478*1475736321', '6132003.27616*1469883062', '37493606.0136*1474245760', '57996990.5525*1478107705', '82561665.1424*1469938054', '69064093.1649*1453908840', '15536325.0887*1463520999', '96617915.8809*1466803142', '48029243.251*1471837505', '75709822.1641*1483581909', '89706954.4799*1478165663', '4530145.07635*1486126266', '50015928.3644*1467573356', '75009030.5327*1467987090', '63204420.3248*1461770562', '27531621.8894*1458546378', '56422332.498*1475397378', '74873105.4572*1466666337', '31698118.7637*1465484059', '31258770.3312*1487939074', '4221430.05194*1456653600', '84726620.765*1461145974', '41617385.5858*1457576443', '14393515.9834*1472989654', '89775204.3783*1462241475', '64305559.8419*1454343810', '42868796.4507*1456126595', '52823764.3389*1479645046', '61917007.1842*1465876215', '17284584.198*1461688585', '44738134.3218*1472004874', '76245142.5076*1455384453', '74352589.8549*1482936025', '61005337.9686*1472216669', '97496047.126*1469914449', '71683749.566*1469956457', '90001937.5563*1479646396', '19509954.2841*1485484873', '73865976.2416*1472330926', '4005098.07482*1472016535', '97204033.6938*1457849154', '3669109.37105*1476151447', '17996916.1169*1459547401', '22581854.8433*1462413880', '70123714.151*1464506276', '51612594.0466*1461969114', '18638617.2918*1483171159', '64199948.0943*1461512761', '64657142.258*1451897630', '89490745.01*1468293875', '6334544.86328*1483037476', '55309537.0102*1478882378', '7891463.54445*1481535092', '7800323.30109*1471421594', '72784209.6268*1453631592', '55394527.3187*1474229929', '79812867.3068*1452747674', '28369019.9223*1474803041', '80351088.0077*1469925795', '96433867.4683*1481494075', '7807296.44345*1472961569', '29308088.2806*1473009096', '24056536.4449*1484402335', '78086984.21*1454221896', '50581815.2703*1468808003', '20874693.9768*1482038156', '43459573.0016*1455784536', '73568922.2668*1484696663', '73038389.1455*1451657744', '19474973.1269*1452823522', '97558401.6756*1484332585', '68368923.3858*1457302840', '67770798.3664*1452888817', '90146570.3472*1474240582', '3728533.28245*1452351161', '59879946.6769*1478900236', '17282678.9077*1470138225', '70863821.2466*1485296699', '91979737.406*1461820557', '38091236.8358*1469516222', '18450107.1695*1455473336', '68567804.913*1458805679', '88381992.5026*1469096465', '38759554.6776*1470903523', '54881675.8136*1484862598', '65251854.9104*1485688176', '64558682.2323*1476407389', '39525383.4323*1471551543', '90077304.2444*1474614461', '72528175.7496*1466140425', '72215878.7465*1451656253', '1919177.12206*1452687642', '48748583.29*1482235774', '12120637.492*1480454724', '3267858.47962*1461227126', '23840363.1276*1457726663', '79938529.666*1474523233', '2709043.95279*1468905991', '69652475.2933*1480659475', '20870479.4729*1463654391', '64155426.7971*1485290854', '10205353.3599*1471840130', '25265059.4523*1473847925', '25895085.1746*1461518432', '70229156.2209*1462339312', '21820679.7111*1469364936', '34871035.5973*1465582716', '12768550.4582*1468464524', '76999109.5824*1469231310', '49785297.7047*1467410742', '86196197.7327*1476391589', '17631890.4268*1459576981', '83613827.602*1481720490', '37195928.5741*1465985955', '29222333.0249*1461960956', '16887712.0036*1467395133', '32609587.706*1481010706', '1434531.04786*1457885869', '53536184.0598*1462305196', '44765174.1046*1485391258', '52131443.1869*1465398458', '64854062.3459*1473050078', '58058001.0788*1481036592', '82158631.454*1469507347', '60175526.7865*1456599986', '42245881.2255*1485469991', '24370385.627*1457187172', '78623744.7975*1458984517', '29164972.44*1485990532', '44141392.6929*1466538268', '53243763.5485*1485507315', '60491602.5134*1488070752', '88438112.5281*1469150441', '57672419.0415*1485021850', '49438732.1062*1458406724', '94627879.0581*1487550522', '9596975.98811*1459875603', '41847105.7698*1461500043', '65463169.4301*1469315738', '40357161.5219*1462232505', '62770725.06*1464019431', '39007802.693*1470642767', '44558086.1381*1487033737', '94823261.686*1476190897', '26185178.2001*1479553311', '82122827.4007*1486346153', '42678397.402*1451969401', '36570158.9815*1468442606', '25342703.5476*1478847591', '8103010.79228*1455860802', '34428688.5526*1451874436', '17168634.5622*1475978739', '99421903.3723*1465486158', '43518400.3362*1484595567', '61181422.46*1456916919', '87282078.7692*1456681014', '27128068.3009*1483153840', '80605630.2495*1471418985', '85190231.8493*1453787516', '80629841.4359*1485553702', '76115251.6649*1479737281', '55637476.2341*1487015107', '88042220.6685*1459853692', '6166770.7518*1472520772', '31881977.8964*1459198975', '77716598.2137*1470193247', '87314511.8732*1456811901', '44348469.0876*1465455754', '67142349.4772*1454902631', '48829729.5054*1483855729', '60968426.8273*1474737585', '4883487.86866*1452080674', '68411920.0867*1487147736', '51169894.425*1482622775', '88058464.808*1472627582', '1228329.75932*1455635498', '49250004.3231*1454063311', '8072315.50501*1460109789', '29164972.44*1485990532', '31920448.9131*1462693370', '78408857.2314*1476799235', '29649556.8379*1481274453', '91263606.5386*1474403822', '99184163.4955*1475568365', '76342362.3267*1487143203', '1314505.85528*1467883843', '62086919.5671*1471010145', '28284618.8795*1460069714', '63081929.4937*1454194737', '76874232.2294*1453844453', '33609098.2583*1472692457', '19144382.6417*1459309912', '95074505.1116*1484771671', '94988377.7147*1458271141', '71383066.2194*1454581911', '87335184.7038*1453045271', '72440179.6744*1477887823', '30874790.3283*1462244120', '31905021.8157*1464497189', '83001336.7853*1483943235', '2745407.14434*1461665673', '79584710.1292*1453249205', '72784209.6268*1453631592', '19532950.4831*1462052228', '18581840.0153*1475941106', '97672665.4793*1451616403', '75566152.828*1477160606', '64305559.8419*1454343810', '61959626.0243*1482029622', '60079550.8512*1472222532', '89839370.5857*1467682053', '17417485.5484*1465518865', '76342362.3267*1487143203', '4466616.77463*1456209859', '91669892.441*1475790181', '86356511.4002*1459556763', '37716168.7503*1456921458', '16740906.9912*1486739613', '57270015.7757*1474898079', '84741313.7902*1462017352', '98007155.1887*1482867695', '16491295.5242*1485400073', '95402826.6127*1468612967', '25065557.914*1451910519', '64327045.3839*1481836171', '76449878.0104*1452525036', '51910807.2755*1474503693', '93207725.6265*1470152106', '19700181.2521*1452975753', '67069777.2004*1456343442', '442255.39612*1460119211', '20590199.3005*1472951005', '53243763.5485*1485507315', '55065941.5287*1480071845', '80070574.5594*1478310663', '98318638.2808*1467580385', '30874790.3283*1462244120', '61690298.7124*1478668217', '37807284.4531*1463791369', '27060117.8978*1486570213', '34106307.8046*1451921618', '23446420.1704*1486954823', '9655296.05747*1458157813', '56680811.0127*1479869612', '34626346.9806*1479253117', '11958471.5074*1483457246', '59496564.2171*1486978079', '90864456.6328*1469287006', '95298796.7663*1457778563', '86121920.5803*1475584075', '43744826.7829*1462822684', '30303971.4507*1466794692', '27690696.4763*1467618767', '56839124.9667*1470440858', '21061628.5002*1485803845', '28022879.9599*1484897390', '9230707.63813*1474718536', '26516774.7306*1454650427', '85185329.9169*1487748501', '69598598.3715*1468727201', '62645675.2022*1487851146', '66317635.1244*1483785312', '1391922.82416*1485224317', '66108349.0964*1472802889', '87480876.8148*1477638493', '21829382.9834*1453534129', '7218589.89123*1488193847', '10190143.3769*1467787430', '49556650.2074*1480952789', '6201044.00008*1485470316', '20602691.3868*1484340376', '10148126.7865*1479969451', '41324990.075*1469886051', '3157895.87085*1479182317', '56529833.4681*1481838125', '89258489.795*1471398462', '27036847.9969*1472375680', '75949835.8603*1485833584', '28228057.966*1487437414', '14575626.5153*1482988793', '85832465.6594*1461051878', '69605403.7824*1458362067', '61435180.6036*1462634565', '52107349.1538*1459830891', '88548479.6067*1483395846', '43496679.5034*1483012879', '78411890.0271*1486342561', '37327895.8873*1454461342', '3793669.13941*1464965095', '84277571.6352*1481186324', '33301941.7802*1468793128', '17721124.1586*1478068955', '36328188.2432*1479438751', '18603102.761*1472170929', '51955173.4503*1455071711', '48945435.212*1485720392', '38433168.3826*1453842931', '31006812.775*1454396814', '99534064.3341*1483565251', '52161817.9414*1456286971', '77040207.3105*1474242440', '27272805.9208*1459224722', '37323392.0554*1485918475', '12839077.0767*1461710950', '85809851.0905*1474132289', '46975494.5064*1451947798', '93250674.0404*1474110948', '14694723.9217*1467761541', '35793494.2762*1454024678', '33485793.9602*1453629550', '14922588.6158*1484517730', '843613.193685*1464341059', '41713719.8656*1480427548', '2244802.76081*1467359365', '24624568.8824*1478133262', '75498681.3101*1459028555', '71990010.1128*1476955551', '45280083.3374*1458033159', '71022491.2529*1485088010', '23463873.3936*1451798070', '65463169.4301*1469315738', '20767673.9435*1456238833', '72484561.231*1478467686', '75341429.3085*1478665076', '71430507.6758*1469119047', '27268809.4003*1461274066', '40436612.9747*1465251598', '29234592.496*1482635936', '66002645.6082*1460617269', '83659543.8625*1469055418', '66285317.3413*1457655373', '70348898.6148*1454140557', '8843073.9451*1461385606', '79603827.3644*1481368397', '49468477.1872*1465130277', '70867917.2915*1460940707', '81779266.584*1479149677', '90602516.2993*1481860350', '73358193.7546*1468098533', '74076834.3303*1479902363', '76514862.2937*1466147348', '79038809.2999*1466617345', '79470912.1661*1483832780', '80626066.0444*1465941286', '82607583.3436*1466960468', '40499653.2491*1483450158', '30854169.5889*1487270433', '92094882.7712*1467480272', '44765174.1046*1485391258', '63989856.4601*1475010101', '27534271.8782*1452058953', '56530076.2422*1484527166', '53642467.6157*1483394208', '57200904.7077*1458682987', '81839460.6066*1455162773', '18899738.8345*1465992022', '71042740.0511*1468645484', '25484060.4533*1464504647', '70233741.163*1475977995', '88278350.9532*1481723140', '47472078.6711*1463788343', '99865389.1886*1461459138', '25969156.6798*1487267005', '76860438.2091*1485050184', '28221241.4753*1467699949', '29303232.0376*1478514041', '32529832.3253*1473999329', '14090643.2868*1462424576', '67166825.8986*1485149042', '35392302.9879*1486168809', '49708890.6881*1486785116', '37195928.5741*1465985955', '32999505.0109*1455479540', '68551986.7249*1472617449', '1887921.60464*1461644791', '4577070.20791*1480201150', '45204396.7725*1461581835', '4937758.87166*1455285711', '5468424.44986*1477778155', '99726497.8061*1457946056', '21054807.4518*1458176678', '43797371.5834*1452164189', '75651645.0523*1487044041', '98561661.7242*1482766112', '1784056.51043*1451754280', '76210336.8919*1483234339', '4493397.30999*1472152533', '15484735.2509*1463998015', '61384721.7253*1474172102', '74051512.2879*1471484227', '33375104.3899*1464916946', '3014001.09012*1471257654', '86819000.9443*1460866578', '85916200.9179*1463600634', '65303954.7103*1479630183', '11912370.9266*1465086517', '46755437.251*1486936926', '54176189.2322*1461612083', '17053774.9279*1484555386', '78122934.4525*1467918347', '5874078.13159*1463374327', '58754835.8185*1458479561', '85667463.5086*1480053656', '33741880.1558*1485389662', '56821311.7823*1469278271', '75915084.2885*1481424220', '78574651.7136*1473329581', '30804969.9008*1463826925', '21072254.2864*1454352689', '9221372.7207*1479292033', '55139321.7789*1468054118', '45881944.7411*1472317457', '12480282.0233*1464169904', '66474823.2898*1453876393', '61260610.6916*1484572816', '37019451.3128*1479130129', '3297984.49726*1478126286', '23751978.9839*1476204977', '43799373.1838*1472831721', '66140689.2807*1474082914', '41781499.7678*1451940407', '61639992.8444*1469423465', '5811372.39504*1484847491', '94373571.569*1487933267', '64812961.3244*1483004273', '70836345.6032*1457377052', '12023059.4*1482402510', '21300733.9251*1461017459', '37690600.499*1468939217', '43217879.6595*1454627808', '28518176.5414*1459896428', '96747410.8275*1461368605', '84116139.7856*1484644105', '14694723.9217*1467761541', '47325015.1493*1468877045', '26834779.8741*1482354538', '8501594.64097*1467001731', '81091749.123*1481595789', '22904138.627*1466181976', '68278609.0276*1478537498', '29760062.761*1473247816', '85863078.9335*1469281735', '79707555.1686*1488135261', '55540540.0544*1475367226', '89009508.7255*1484800332', '31809905.7405*1467215637', '16769520.2536*1456880895', '54952333.2627*1458415505', '28144105.9321*1463225983', '42582524.8049*1463910293', '30854169.5889*1487270433', '67072941.9909*1468423853', '76134154.2129*1482240083', '93796248.1857*1465304436', '45114119.4935*1468249418', '31757596.5298*1461607890', '27541354.082*1484636963', '70383142.36*1476829288', '75718215.8022*1456364297', '94161532.9042*1471944919', '90586587.6673*1462177576', '90961050.1636*1487071312', '96991967.6049*1472682691', '56422332.498*1475397378', '42952132.1187*1473081961', '92874161.0028*1480391564', '88352795.6703*1452231411', '44460934.789*1459775304', '65796453.9248*1452430786', '57915784.0915*1471247811', '79764960.4903*1474593366', '22908643.725*1481091632', '12946017.9946*1469881217', '9922787.19227*1460152703', '86465135.4894*1452659005', '37271892.4194*1469319129', '95661815.2736*1482199372', '30460876.538*1464502578', '92330288.7781*1475075470', '72483476.1331*1487982842', '57901373.2937*1477826210', '58806934.9056*1473171133', '43606137.9515*1486322072', '22800277.9497*1464455530', '79648123.6167*1453949206', '28159049.3259*1477173709', '16740906.9912*1486739613', '56530076.2422*1484527166', '6017121.06207*1463751606', '64396165.894*1476584144', '13505808.6555*1471424875', '41583871.2003*1483906388', '44830187.1691*1452909057', '25969074.1107*1484052112', '91708008.1531*1484380110', '36753369.721*1478912655', '97876443.9447*1479731545', '8451361.39012*1481182888', '40315992.0457*1453035181', '66509925.0025*1472151466', '95207302.3007*1483269589', '60239779.3655*1485812245', '21582605.0331*1476946268', '34142562.4016*1458001110', '1833349.01376*1475367357', '61528400.6415*1482472677', '29542659.0446*1478234591', '89643474.5631*1480398681', '71258907.4586*1474518718', '1887921.60464*1461644791', '98192306.3248*1480468252', '81345584.1302*1453501265', '10947684.457*1473652085', '9706474.02066*1486065155', '80337072.2906*1484936478', '98486812.2808*1458154904', '57862969.6218*1452958646', '24199240.5354*1472556968', '14988113.0891*1466791321', '40026706.5584*1465217609', '49409159.2346*1481980143', '80721427.3729*1463189920', '12042455.2764*1457637849', '23136171.7761*1485064244', '41492343.3026*1488191778', '31911392.9563*1460393734', '80800840.306*1477203103', '3625856.28175*1466775364', '46833661.4329*1481231970', '5333569.38327*1481009427', '21363097.3871*1483090011', '39920633.614*1463356807', '44606646.022*1457699479', '69430464.6731*1482306040', '7218589.89123*1488193847', '47106484.3822*1468156331', '5947705.40325*1483601565', '74283069.8986*1458849935', '87390480.2736*1459409036', '76274089.9945*1478731971', '51653534.9229*1467964170', '11577944.8804*1454438746', '2189894.49325*1474889695', '37437956.418*1478407078', '86446992.1619*1467393368', '83502911.1644*1478446966', '76676634.0751*1474972546', '64503997.5731*1463581062', '53459300.6132*1488279177', '28325938.4191*1482362326', '43519498.0845*1468913113', '41362180.0126*1468579894', '64843761.3739*1482676569', '37555406.1123*1454525754', '21889050.9091*1482834564', '15391361.0048*1462955185', '61809159.9811*1486182940', '67275172.0499*1456607527', '25065557.914*1451910519', '27251648.7792*1452567830', '88992443.3272*1461268143', '47994518.5985*1463607766', '34838409.5639*1478373191', '90983439.7793*1459049036', '66715410.9943*1477060798', '35807673.7883*1454508604', '92220225.1543*1462369262', '83661406.9311*1475790764', '85451274.7309*1486195088', '28619008.7871*1460941272', '56541485.7603*1456373893', '97009803.32*1471930726', '44673068.0625*1477778116', '55106907.3085*1456017027', '28847589.2687*1460074150', '48769693.7632*1465390354', '56117387.251*1456856988', '61774549.0542*1485119692', '29363060.623*1457956584', '79715027.6925*1453447625', '63583820.5452*1465570252', '65910598.6864*1463668413', '73441912.6905*1483561078', '84352451.4878*1471629386', '44465940.9731*1454636170', '58996884.884*1462897353', '30845656.1622*1465454270', '21271880.9127*1455241006', '64482358.5954*1453402630', '93032449.1748*1475463748', '68210153.2768*1469063080', '75931334.9031*1465491845', '95303837.5082*1471939629', '55586632.213*1471855092', '10541992.8365*1454913596', '52746459.8754*1482365664', '61882712.8176*1476847329', '15932960.687*1477186673', '23791642.028*1464379353', '74020466.3508*1452703682', '82721087.1882*1474385047', '44730139.764*1476600536', '27060117.8978*1486570213', '70826581.8485*1478200158', '7268095.7213*1452240664', '35315321.6897*1484592819', '62419437.6214*1453016507', '37685100.6149*1480961108', '57915784.0915*1471247811', '26043941.9343*1467007714', '98026745.1573*1476395285', '14837044.1845*1463196663', '99006872.4439*1471410637', '40774233.6309*1471265968', '66032014.8299*1476698091', '68874884.787*1488213606', '13658.6609269*1473294012', '34645548.5156*1460681485', '17424030.0231*1463067300', '18007004.9258*1452680665', '31558302.7706*1468433077', '19354285.5973*1473604704', '83136987.2649*1474173392', '95754565.0439*1463368105', '13227495.0924*1457330292', '42315805.586*1468473766', '64812961.3244*1483004273', '43885455.0026*1462782607', '66456370.0361*1460044058', '32221806.1489*1470651624', '99778335.8588*1456203896', '15674515.3499*1464806469', '86336928.3097*1466012588', '24596195.8833*1476950986', '11998829.9479*1469322219', '68921012.4999*1472645803', '82079851.8336*1478762136', '18264469.5325*1488162434', '4580320.53691*1485237508', '53393350.8498*1486363750', '565052.683988*1455730097', '69080660.3791*1464344767', '431650.545298*1479012195', '63753830.4633*1478664775', '20509356.6189*1459415736', '11580070.643*1452794792', '90976621.1076*1457985996', '36675128.3177*1456592331', '8652509.79943*1455697338', '96826713.2759*1473030221', '17192470.3344*1485042915', '50024559.4356*1487540505', '18543535.3848*1454059035', '11905392.7464*1479865440', '20341560.5576*1453505090', '68809029.802*1461311365', '59869502.1631*1486687096', '23369111.8971*1483999357', '16180447.8429*1483773577', '7275242.28019*1454456460', '57775580.9777*1474906926', '74836008.0319*1456864957', '13351632.9103*1462973305', '29991811.8976*1460774926', '13809480.0186*1465798762', '51354297.5016*1457912143', '69876828.9594*1477212101', '37501232.6044*1472253643', '35171109.2715*1473153365', '743855.123076*1456500081', '31920177.5982*1484288549', '61136361.0035*1476387894', '43459573.0016*1455784536', '54665131.3935*1463801183', '9914577.62528*1462372141', '82576397.3*1487734413', '75407502.1472*1455228531', '81308435.4143*1479813586', '2479258.61308*1466339030', '78122934.4525*1467918347', '58522412.0407*1469008550', '58054639.1861*1484662929', '28144105.9321*1463225983', '30039559.2397*1453902404', '13185990.2018*1474173098', '85487307.595*1464777994', '8534328.78767*1456356935', '66002645.6082*1460617269', '12917351.682*1477866483', '68435846.9411*1477120469', '32697188.4341*1479185862', '36052175.9218*1483149902', '84832908.6902*1454443781', '3924411.03703*1462469031', '5995672.96466*1452707474', '53305484.4619*1483199984', '82409273.1872*1476387933', '38354588.8407*1479642513', '94500462.6052*1461630737', '21822018.0213*1456846869', '66696938.5107*1484948272', '38746521.7231*1486952358', '33375104.3899*1464916946', '35248585.5087*1457689257', '84220370.4646*1481536478', '43363610.5017*1451762811', '80605630.2495*1471418985', '73333221.2148*1482891028', '52873155.9516*1459259390', '12653268.4265*1464015844', '39165436.2457*1454216853', '95400841.0198*1472439515', '59394431.4675*1478050148', '12042455.2764*1457637849', '81726782.4626*1482724671', '33898874.0687*1480493557', '19585046.9099*1461843055', '75726255.947*1472472376', '47911649.8354*1483923040', '99480449.9587*1473353578', '76967678.8149*1485801525', '49473904.6224*1483036668', '78713642.0183*1456408037', '80777542.539*1452848232', '57320824.8858*1458149525', '76728970.6535*1478132573', '22744873.4124*1474512902', '95995726.716*1462520751', '76258728.4206*1476540977', '36224245.6699*1484860932', '19812357.2162*1457022868', '68435846.9411*1477120469', '9767031.54063*1461504281', '61539630.8801*1457786518', '7683915.86438*1454160043', '43653135.5523*1451798752', '261373.920703*1470009401', '45827602.0262*1486086317', '27093137.1365*1476248684', '93402192.1956*1481019725', '3267858.47962*1461227126', '47020962.3996*1487527668', '91326387.3053*1454252796', '95036757.9133*1461114030', '23993436.552*1463540745', '2085396.9253*1455650525', '42419414.7487*1477291496', '61389656.6033*1486634234', '31570858.0608*1479888076', '88731249.6049*1462810156', '47466922.3946*1476603362', '17865216.1694*1486550089', '12264306.7092*1477460886', '45874721.8437*1456472516', '4013832.67062*1481804226', '43624718.6296*1456471085', '62811412.5581*1458218979', '94128214.8793*1482723552', '30855320.3532*1485877274', '76785083.92*1482284703', '26496100.615*1464837393', '83339527.1927*1462723885', '61774549.0542*1485119692', '22035437.1678*1460132211', '91121476.8486*1475567758', '27268809.4003*1461274066', '44336724.4919*1458815844', '77178912.6422*1461096926', '3374289.69795*1451700694', '62056909.8021*1469734319', '26431450.5603*1453023061', '60988920.812*1454965703', '4531119.00074*1485124965', '44684500.3125*1471464304', '56739159.6665*1465972388', '79655618.361*1457755035', '48904724.0021*1476986933', '98085171.9733*1456104541', '5961540.61565*1461002487', '53094624.2901*1476931897', '2253776.42771*1459130015', '14582603.4313*1467044431', '84458886.1074*1457944584', '96058035.8909*1483866387', '31611513.9211*1456782576', '66334673.1951*1457404416', '76737130.5401*1465468862', '49438732.1062*1458406724', '68716232.3178*1470821232', '58720378.6994*1464548406', '24425929.1809*1474762838', '17475790.8939*1451955575', '34665001.5475*1483954045', '48592571.2124*1476472938', '20919623.2382*1482153232', '15598373.1316*1477818115', '49230455.0963*1479832836', '45472836.2517*1483097630', '60343217.7049*1479388024', '62652348.0122*1482101449', '72324847.0352*1484689768', '54096099.0344*1462519758', '62792425.26*1463351602', '17869130.5628*1478578796', '67084591.4004*1462502256', '71660948.2801*1479671230', '70348898.6148*1454140557', '54419512.5434*1475612679', '38938029.1249*1475054438', '41936216.3037*1479224683', '1950932.25957*1472299272', '98528135.6044*1486036523', '32322101.1744*1475248005', '36312635.4955*1481957841', '49326686.8044*1457375605', '14684465.5513*1458611875', '54549532.3029*1463268521', '22062601.9118*1472155065', '31205102.6122*1482572494', '13515965.9202*1458425740', '43046983.9284*1481388056', '31271316.5709*1462162002', '86240678.82*1473024542', '19626273.2887*1483992010', '36320472.7989*1479736899', '684937.359844*1456789816', '5731817.78427*1459018458', '37919081.5225*1452546192', '70285139.2875*1475845026', '13971493.6766*1475706825', '21830071.7802*1486908153', '99665858.7784*1453652357', '84676770.5928*1465086932', '92970067.961*1462402759', '18637608.1939*1453996241', '51118025.2697*1451937986', '77704864.182*1484079692', '96110899.5466*1470817648', '71842768.6835*1487100107', '52364079.8982*1473743159', '6963063.34692*1463135868', '38129501.4768*1475304347', '60341099.6239*1474643349', '81401220.5995*1483313037', '31577632.4408*1467480441', '47362088.8185*1483048387', '42615754.0084*1454450413', '15533563.9434*1473892509', '793615.597012*1472868286', '26626505.9439*1453770702', '28159049.3259*1477173709', '35211020.0814*1484720867', '43114449.5081*1477504480', '71009202.8773*1457755659', '41757516.546*1463315638', '58733031.2969*1467049089', '56866320.6699*1470767315', '7581895.34698*1473532595', '29290151.3724*1482199656', '95814030.424*1473211379', '47926725.6149*1463929366', '45777368.7736*1460310156', '90719018.5612*1484050924', '9581494.31804*1485940642', '37919081.5225*1452546192', '19590085.5009*1480976906', '38508965.7583*1460037450', '27146122.1463*1473485460', '59776028.1827*1487511155', '34773171.6984*1459409801', '32442407.3501*1483951020', '39192502.1879*1464442076', '27491523.4034*1465341119', '56861141.9848*1465011945', '93972012.3533*1484030988', '802247.861826*1458983580', '12152979.5652*1487257164', '9221372.7207*1479292033', '90976621.1076*1457985996', '534316.664734*1486302472', '25958626.5953*1482444172', '90671975.7072*1463649108', '74786901.375*1487199223', '55597612.4873*1452538235', '54558575.6365*1472010232', '36834669.5776*1453424996', '14082376.684*1468299180', '915336.332176*1473421779', '50349644.0821*1464283684', '84919924.2031*1473809325', '16157111.68*1469408240', '4005098.07482*1472016535', '75225210.1845*1476429624', '85210122.7581*1458243835', '2076963.82531*1470156867', '36847643.0572*1486086586', '48053686.932*1480123338', '90852175.8342*1487519801', '15991375.5656*1477429021', '96660095.1832*1467573961', '46039395.7594*1462917917', '85531613.0976*1485486929', '70048855.0856*1470889240', '57393355.1602*1481374365', '6448220.39189*1469994470', '36536145.468*1473493246', '86585280.7992*1470916768', '27772556.5779*1485109216', '67176311.629*1453064064', '29602662.0221*1471140707', '9780636.49418*1468156206', '14163642.4176*1453424792', '98683221.4651*1484208985', '75422249.7977*1486695404', '89620147.7584*1478195272', '99504522.9394*1462106212', '71842768.6835*1487100107', '48833916.6971*1464417250', '81581491.9814*1484711284', '23160377.8657*1468662447', '80506496.279*1453298606', '34037590.4615*1463685542', '43640357.817*1452134995', '14059453.7137*1474323603', '57913239.6583*1470868307', '22941214.5712*1454646644', '59962718.2946*1488257607', '28325938.4191*1482362326', '73883335.269*1477346578', '92220225.1543*1462369262', '9833257.25202*1487731859', '72522406.1086*1461679226', '37195335.608*1478685848', '13966356.791*1456121138', '8945988.96078*1452694779', '89748888.555*1474147915', '53642467.6157*1483394208', '55417838.0396*1466696903', '71715095.6447*1460811252', '7990321.96606*1461490009', '36412250.3489*1482791977', '11414596.1893*1485095913', '71197187.0875*1476499562', '81811802.1502*1468325414', '74059620.6391*1478970880', '5036926.7768*1480984052', '78617814.3777*1478839517', '6481236.52573*1468793021', '41489034.7463*1476575878', '93358315.7953*1468437640', '73686409.5463*1482183284', '746308.657928*1483990167', '23751978.9839*1476204977', '37418951.728*1484459834', '76790469.5403*1470102373', '98555019.4661*1467028530', '40863915.0537*1469724080', '57723559.8384*1471659335', '22680215.3979*1487405204', '19748331.5024*1478178972', '56570100.4696*1485201743', '11411746.6072*1460590162', '32828024.8871*1452208898', '88320507.9593*1458518143', '99927464.3731*1467226839', '29381739.5187*1459419476', '25595210.0066*1468690977', '52561012.6799*1459392886', '71947018.4324*1484887029', '15209265.8675*1470921242', '32554143.1483*1487981619', '27707826.7565*1458777812', '69064093.1649*1453908840', '99214637.4154*1475270215', '73385472.8138*1481992312', '41231352.5288*1479298660', '76000644.4074*1483551180', '85362032.03*1484356660', '35422351.2159*1472967024', '26396436.8041*1480114729', '84569155.1174*1471926279', '83727875.4866*1456867979', '98582719.9004*1465637699', '94565390.4055*1457800140', '51761026.4712*1488190955', '82070993.1936*1470973030', '93217619.7092*1484595730', '78762924.7879*1481258310', '13479963.6046*1465763961', '24326814.7141*1481350606', '24881713.9912*1486535133', '67116770.525*1459950627', '37387421.1976*1465400207', '59012817.3237*1453194521', '22234487.7406*1454188631', '64812961.3244*1483004273', '1686429.754*1461885482', '6743543.15071*1459825417', '15878243.1326*1476350423', '17763533.2503*1462441787', '40868238.7197*1456709554', '30996161.2447*1475451843', '24425929.1809*1474762838', '19600108.3349*1475274548', '30226852.0835*1457005981', '41442227.2487*1479515354', '76484937.5127*1479492587', '64212953.1749*1480475199', '39090224.9103*1487885193', '76539491.7671*1477740219', '33066794.9221*1473766683', '49345624.1275*1462780120', '24910639.1158*1456089156', '3952540.73304*1482227442', '48478504.221*1463887434', '91126784.2278*1460954913', '11998829.9479*1469322219', '50084943.1988*1481305708', '21582605.0331*1476946268', '75910998.2067*1480474495', '91002182.7297*1475835472', '74153025.3951*1482889629', '20283991.2883*1463586311', '47582568.5568*1469796800', '94674558.6491*1466027738', '64200252.5463*1478134375', '14530476.9102*1457348733', '59055111.3002*1468693801', '76711329.7216*1454664482', '38887644.1068*1451866109', '82781264.5321*1460910220', '84687178.391*1459888414', '20640286.4265*1475088315', '4785821.25449*1476939820', '50219093.3084*1472225333', '31461927.551*1465569561', '10872661.0767*1481654430', '80393477.5514*1460976527', '21208393.84*1486809600', '63948962.261*1461119237', '73252468.997*1472595128', '69226165.3182*1463748030', '30660139.4708*1471102838', '54709694.0904*1456519900', '76852312.1055*1483458864', '84668082.5701*1465029725', '88039156.0493*1476646213', '63533.0879987*1477733736', '23500699.74*1479331967', '60071122.3083*1468229917', '70581437.3037*1458752337', '66690115.8968*1452543678', '41097907.2915*1482791532', '62801301.87*1466918112', '26463914.864*1477010364', '32048710.712*1487543037', '98844795.0233*1476244453', '46445489.5315*1479273324', '34679800.2046*1481469737', '85390899.6502*1476545182', '35817984.2016*1468278403', '84385853.4245*1475314173', '21349930.4052*1474658277', '42783804.6238*1454805916', '59300335.0514*1474434530', '4615658.16065*1460469708', '49004400.6158*1474289147', '20032424.0671*1454618861', '66140689.2807*1474082914', '34252242.0411*1466546424', '32983528.6844*1482526329', '90436188.5604*1484200087', '66690115.8968*1452543678', '77320165.2262*1459740711', '89645500.6094*1485700498', '474263.060471*1452303377', '85137596.5527*1458008687', '66842942.21*1458356486', '32016772.6368*1457307473', '34688926.1258*1485665134', '22286824.6505*1488012521', '20421341.6699*1458789930', '1738834.32658*1458495391', '1585303.8512*1461410549', '53915422.5363*1479063454', '47058951.2527*1463168740', '19090910.7189*1479366260', '89307721.198*1452336993', '77395740.0745*1472090731', '23925935.9157*1477498838', '14456254.4465*1458430291', '99504522.9394*1462106212', '56378479.8906*1471425942', '42139155.8137*1467931672', '59100423.9179*1464720462', '58498591.4505*1476841879', '3586914.30921*1470014427', '80042563.4538*1485573477', '6017121.06207*1463751606', '52816535.2059*1458121995', '92082672.6792*1473807209', '87890659.2612*1475411525', '99725576.9721*1468437660', '59837190.455*1465227224', '20919623.2382*1482153232', '88039156.0493*1476646213', '94036158.0646*1475106915', '95961486.7419*1457504351', '82229211.0427*1474368068', '43847893.3665*1481459490', '18261221.5107*1473088762', '45272131.1324*1479986056', '59194990.8616*1487568097', '31637527.2553*1454392588', '18131967.4788*1486903320', '12487718.4318*1486365265', '24138499.5385*1484143884', '33074117.0321*1469243596', '14814244.1324*1470493409', '28847589.2687*1460074150', '34792828.4224*1487140210', '58918758.4056*1468955277', '43705985.1595*1462766811', '1095478.23807*1480615067', '85358823.9106*1475687538', '67275172.0499*1456607527', '73333279.3768*1461641937', '64124746.4165*1471029509', '56186144.227*1481765992', '28393066.1622*1488077237', '69747546.9044*1456093808', '63846269.2245*1472389759', '74020466.3508*1452703682', '5507843.02019*1461628512', '40662359.0731*1481590094', '91471055.9128*1456648121', '61702967.6411*1481148755', '41492343.3026*1488191778', '9922787.19227*1460152703', '81507937.8899*1457273482', '63410553.1018*1477664883', '28910002.6492*1475263897', '23758729.1309*1478586864', '48101423.3036*1473370854', '15967018.1207*1464939125', '91833097.4955*1477928033', '45563448.9283*1460081395', '3083292.41079*1485055465', '66209247.325*1456084286', '71317207.7142*1462889313', '49512713.5388*1486175590', '69959039.0878*1459145455', '59713275.063*1469329592', '91391357.7442*1465977967', '43887510.3363*1476328295', '70782388.9378*1474727582', '56803089.4001*1487016525', '31617131.2055*1475004112', '5402308.57833*1468168585', '21176069.0545*1459222993', '90044321.2635*1457701573', '8608742.46241*1475538902', '61410035.3833*1457818565', '75415478.4253*1469080309', '70537951.2663*1462769221', '17299824.005*1474702980', '43609163.8926*1458827572', '34061892.9849*1485186803', '52823459.3627*1476464887', '69598598.3715*1468727201', '15448860.8845*1464446202', '31907000.2262*1481726644', '10092862.8132*1480196725', '474263.060471*1452303377', '33364019.0747*1484149008', '82486534.1257*1485039954', '41541455.8469*1488122110', '89748888.555*1474147915', '59400112.7312*1452501428', '77435620.6553*1461686168', '16056823.2382*1483648659', '50008745.4493*1479782378', '91206986.3615*1481342026', '61492846.657*1474340104', '83130828.1537*1487764370', '82052437.1806*1486059405', '79471614.0005*1455740267', '78434079.534*1473961379', '5435525.17522*1463746160', '57862969.6218*1452958646', '97288906.1426*1454435643', '12916987.5373*1475886884', '89192707.2975*1454972021', '71749897.3978*1486796862', '49441705.774*1478403373', '20455618.2622*1453022075', '87337125.6523*1461198205', '45567029.1774*1454292029', '62188081.667*1461938679', '12378857.1214*1475797134', '60968426.8273*1474737585', '47617617.9301*1454991125', '38746521.7231*1486952358', '62383773.8191*1478290849', '27308686.204*1456099742', '32417901.1231*1465221086', '90913349.947*1456204236', '6862858.52187*1478356003', '86653833.5772*1457733955', '21750446.566*1474481563', '52120970.3834*1454628770', '55417838.0396*1466696903', '66720262.1326*1457316671', '7136963.12315*1461117910', '40786149.0992*1461008724', '55695544.5575*1468653925', '27889427.8968*1457424474', '88478032.6804*1462373810', '341053.353945*1464036484', '15636164.433*1485450857', '94086914.5475*1458517692', '33460609.0855*1487794550', '19240400.5518*1456112756', '17629879.6884*1459050851', '60288992.254*1465770805', '7787640.94809*1482013737', '52674473.0055*1474339654', '99482926.212*1463440328', '31205102.6122*1482572494', '58469906.8431*1486949606', '92150506.5995*1457334689', '84244241.1358*1459447294', '58469906.8431*1486949606', '88995056.3617*1473225322', '24665544.0282*1462446261', '13302899.658*1475584589', '79275078.3014*1451869294', '6927106.50435*1453521824', '28657301.0838*1483027976', '67831061.1544*1487790718', '64497984.5913*1467614920', '38869401.6999*1453563835', '8130202.63962*1483347894', '24138705.3756*1469342830', '67142349.4772*1454902631', '47940645.8661*1487004524', '24693612.4425*1452865700', '21582605.0331*1476946268', '18751106.2129*1464443405', '83683638.2111*1452223485', '2024355.17918*1478084307', '12773377.7698*1466819026', '48103401.983*1463685208', '12743536.5173*1464995913', '95769985.9009*1466083353', '34037977.7021*1485278432', '47355856.3041*1479515537', '22085237.583*1482210470', '83036127.7905*1478355520', '92697767.0136*1465969458', '75500169.2571*1457634862', '59131204.8689*1466751331', '57485842.2427*1457068593', '47142579.6302*1470725646', '60401333.6248*1462954797', '10362311.2873*1471942452', '44764720.2968*1478826553', '97876443.9447*1479731545', '32801218.2848*1487668949', '13972024.5088*1454427713', '57474678.3005*1467907854', '17631890.4268*1459576981', '23820178.4182*1458720091', '8501594.64097*1467001731', '96315571.9684*1458718615', '370758.810013*1467851147', '16829651.0422*1485659102', '6634418.20662*1463498939', '44751791.8417*1460605060', '99845807.67*1475337637', '15259800.6778*1461244582', '374273.526822*1464002139', '68002799.7352*1463639500', '60502403.2895*1471512419', '82263584.8306*1478429696', '80586195.4781*1452917783', '13214866.0893*1479601646', '75145863.994*1464038622', '94561594.9915*1456422910', '19004579.1422*1471427864', '73632060.0051*1461889582', '6963063.34692*1463135868', '944510.34403*1481759630', '33690930.0111*1456450030', '70782388.9378*1474727582', '52275064.9052*1455659233', '89857050.2459*1463794107', '53573937.7379*1454194033', '92179543.9713*1464014376', '69170779.1374*1467567992', '41285270.023*1468644483', '42932229.6468*1480428263', '73882149.753*1478693180', '71795964.6713*1460893540', '53328486.5056*1455769501', '8671152.70018*1472627836', '8401066.94195*1477304547', '84506729.125*1481567877', '11110763.4738*1468685011', '35903921.2266*1471973173', '83873882.1942*1480618639', '7052074.31428*1467328374', '29080785.7616*1472355578', '96152548.5405*1465683117', '2477555.35602*1467397899', '22746847.93*1483987206', '64140398.4437*1454500577', '94225390.9868*1474762014', '73501167.1195*1457182935', '21535146.0106*1467761617', '86375221.2549*1467383462', '52161817.9414*1456286971', '76820760.7924*1453299731', '77453012.2957*1486872565', '23696798.1093*1474300962', '67321762.1391*1475631812', '37225687.2791*1475880721', '34674064.261*1487078319', '24839110.363*1475501647', '29477562.36*1456569587', '44558086.1381*1487033737', '19954418.4449*1473624237', '29764697.3003*1457446853', '71565182.5402*1473061673', '77632260.7721*1455608978', '75852243.0565*1461544515', '13481349.73*1484979399', '59481592.2912*1460145199', '7858120.37273*1463195413', '29598944.5255*1479703393', '38865586.4483*1455542569', '49050398.3611*1474347203', '33225707.7061*1453789032', '88320507.9593*1458518143', '77320165.2262*1459740711', '23938032.3937*1465033246', '70756413.1898*1462317529', '999939.013158*1459188221', '22191472.7501*1484129214', '98658379.0782*1481842323', '10957384.5896*1478295247', '14525461.3497*1461249060', '48076472.7858*1487510444', '74286820.4051*1466886639', '83757738.6415*1459287627', '61553662.2841*1460324653', '33885714.1627*1476802829', '64246149.5219*1465649384', '67997052.3513*1464398460', '8512104.74712*1454291835', '86666437.1327*1467121344', '1434531.04786*1457885869', '98161271.8781*1454941641', '71456974.4857*1468040505', '43408026.7722*1468259753', '62008052.1122*1459084434', '80648854.352*1479995668', '94627879.0581*1487550522', '66656941.0445*1463037390', '47822614.3545*1466665566', '42244204.8498*1463600024', '10226931.9686*1459148090', '83769095.3299*1475516454', '97265273.2892*1473948922', '20088887.5757*1471552539', '64097988.5513*1473464940', '37501232.6044*1472253643', '17936695.8628*1462262940', '56108410.8833*1460236160', '32731913.4355*1454981874', '97648241.9914*1461838350', '65919428.2646*1452975707', '47513483.8545*1455092972', '39007802.693*1470642767', '24455568.8636*1483051161', '97807799.9021*1482572220', '14090643.2868*1462424576', '88293323.3151*1454703257', '73879840.3799*1482590307', '2777550.09469*1487812677', '36012220.106*1458110481', '94345628.2824*1483261347', '15351173.8791*1479990617', '36728383.3348*1483814052', '5160428.89015*1468906460', '69918709.5168*1459756566', '95265104.068*1461088810', '10196165.0911*1461121135', '63082583.9696*1485170291', '40064945.4452*1478065412', '76737130.5401*1465468862', '34127403.5307*1476915300', '26083885.3505*1480455767', '24839462.7425*1455169524', '58908706.7838*1483193298', '40031842.7185*1463660299', '20792828.6589*1485790278', '17704521.602*1461864636', '14694723.9217*1467761541', '5303110.48826*1453186277', '64559775.0273*1479258101', '56464791.4593*1464783149', '11880225.0999*1479879394', '78617814.3777*1478839517', '54984517.3425*1454643414', '44656105.9987*1455961709', '49449636.6934*1467552611', '65936764.5518*1459234811', '30068371.9069*1481869012', '37757582.9125*1459303488', '34424968.4066*1468943966', '80278066.6771*1463948617', '65577516.6791*1479644469', '45218392.6435*1473747696', '32828024.8871*1452208898', '72749100.1724*1453169901', '14882382.4888*1463575710', '97487549.1364*1470697032', '55037024.3214*1476568422', '97288906.1426*1454435643', '48790165.6763*1459795085', '44771720.5966*1465141344', '41242516.3427*1473342484', '44149043.3328*1474944730', '12743536.5173*1464995913', '92997180.7183*1465529263', '67834526.2523*1451684342', '36415986.6365*1469679775', '72087560.8674*1476804597', '33068554.5326*1481876143', '61729563.0419*1467247719', '80211991.8731*1453165799', '90309757.0232*1473912584', '3288188.3094*1458567682', '63878838.0383*1463631463', '29290151.3724*1482199656', '62594579.6516*1471238864', '74203636.5064*1479957041', '75313708.0318*1484241929', '53408945.6242*1487385662', '46481601.6725*1451965797', '9476195.27712*1476172765', '68145455.5388*1460040844', '92926778.5266*1455765648', '41362180.0126*1468579894', '66437448.6977*1488256441', '42113684.7671*1471730203', '18977827.7612*1474932788', '97441657.7969*1462531839', '3340441.41332*1462699158', '13658.6609269*1473294012', '49615969.5658*1468878988', '67866596.6463*1476574272', '13926276.6749*1465333929', '25180039.0099*1462573393', '25757882.1442*1453865218', '81443190.5825*1468721554', '58608970.6599*1479533685', '47237771.0555*1484237254', '43902157.9446*1477079136', '28494885.9153*1478307038', '70558547.5969*1465057205', '98977039.6314*1487755781', '86653833.5772*1457733955', '19374549.7634*1457167098', '28268632.6396*1464014039', '12768550.4582*1468464524', '68973182.4001*1472180522', '12360372.6298*1477758772', '11731599.5981*1462381923', '68999689.4512*1477441153', '68874884.787*1488213606', '52820166.3703*1473055115', '75086849.987*1465345653', '91941843.1958*1475324712', '53305484.4619*1483199984', '99557747.5824*1457956050', '61476743.527*1484109302', '46020444.1945*1468422952', '47768337.5099*1486072473', '41876734.6574*1487782094', '73464666.1995*1479660451', '75270239.0486*1466916711', '35052325.1513*1453206175', '27513502.862*1478459378', '8837242.92071*1463289862', '87201295.0264*1469950543', '59713275.063*1469329592', '99482926.212*1463440328', '43717291.6398*1465845888', '9333780.07148*1483822821', '80567687.5546*1476595150', '29099143.218*1476677847', '37805908.415*1462955656', '62424450.4589*1459397440', '12836302.4461*1482546993', '18264598.6195*1455148066', '99428882.4826*1482072074', '32220780.0109*1456189018', '54122223.4568*1483159493', '68541325.4715*1472696548', '28103947.051*1481923550', '89047446.8489*1477060238', '59586121.6481*1483621568', '10440294.1531*1473337341', '39826723.6851*1470702326', '70911654.1825*1453466660', '33829395.8089*1475640018', '12759592.7134*1483499781', '27546081.5424*1454904515', '85684725.4726*1474158991', '1809956.91858*1465946051', '59065017.7166*1467296318', '82881264.4604*1452190462', '73041600.5957*1472929429', '67444597.1178*1479686359', '9292458.70839*1475810753', '1833349.01376*1475367357', '15209265.8675*1470921242', '30793454.9986*1469394641', '61225230.9877*1467728921', '77511569.9675*1455786136', '92078809.2697*1484840397', '41442227.2487*1479515354', '52057294.838*1458333715', '17725183.3495*1464053021', '88744566.5977*1465233237', '15462252.6514*1463377894', '23872365.8768*1483827184', '5306424.86127*1471215929', '71565182.5402*1473061673', '11911790.6903*1476657577', '43691178.3186*1467382183', '97970218.491*1484193383', '73146839.1291*1476092722', '77368777.9457*1454336252', '98834498.0871*1453700935', '43814865.7449*1462829855', '6051319.00684*1467088075', '2351247.78214*1479300991', '88058464.808*1472627582', '1532849.85046*1477305295', '83771148.0285*1473371206', '56321913.0056*1458847215', '91618486.2284*1464336822', '30124521.4044*1463779071', '32016772.6368*1457307473', '62792425.26*1463351602', '15388362.5331*1469157612', '66811509.2134*1453379610', '20051182.6978*1480885334', '80482743.0303*1455937730', '15391361.0048*1462955185', '424717.01403*1471579623', '52889443.076*1452180614', '17640572.4224*1483593350', '16157111.68*1469408240', '48216007.5601*1480935704', '85401893.4065*1481057052', '52145438.4738*1463623343', '65682604.0789*1477844230', '61746732.9001*1464021651', '53775179.152*1460140610', '15533563.9434*1473892509', '12588230.2319*1480110594', '29140188.6074*1452763603', '31920177.5982*1484288549', '28636517.1334*1463493959', '69033978.4088*1479323131', '6567622.48453*1465449790', '99853746.0722*1462995356', '98110716.9312*1465225661', '39886610.354*1472037677', '79812867.3068*1452747674', '7889356.0429*1458460270', '69033978.4088*1479323131', '54419512.5434*1475612679', '30694921.6906*1488190209', '71018985.376*1481846439', '89841270.2565*1488029395', '97911537.3058*1465757742', '92047290.1645*1471400650', '69553267.5327*1477122241', '27238772.4662*1483152282', '63701555.1166*1483510199', '64938310.1659*1484483746', '47936536.5419*1477128329', '56370476.5509*1463712632', '21367014.0498*1480098400', '79381643.6109*1466070618', '95216559.8867*1467501319', '82695176.9888*1464506829', '60962186.2937*1464613402', '97777108.8629*1471919757', '10452745.9081*1464481357', '6342851.34717*1456231595', '86823303.8357*1454316695', '81701248.2517*1483114299', '48364410.4944*1463210302', '86280694.3017*1458663545', '36234857.1187*1474694573', '92874161.0028*1480391564', '83828866.215*1460883378', '73632060.0051*1461889582', '89595728.0082*1468251621', '53107075.7121*1477897061', '28951598.4262*1465914736', '5874078.13159*1463374327', '45520395.0755*1466003047', '45302225.058*1459276227', '43535069.9732*1470557720', '8138727.98216*1472012625', '44638426.8483*1456685834', '51576059.2352*1479534600', '37278805.7334*1478136108', '72731595.4355*1468994757', '57514846.1927*1455801023', '18573798.453*1461426005', '61600056.883*1459069264', '96482696.0901*1487364725', '23121371.4109*1455082313', '44923134.0054*1451994509', '7879223.26966*1472142361', '91971336.9584*1482270067', '18991161.0225*1485273781', '35234106.9248*1477404999', '99853746.0722*1462995356', '48720587.8572*1460266258', '11430066.4164*1482067052', '76852312.1055*1483458864', '34932672.8854*1478253589', '40494500.2038*1465309456', '68999689.4512*1477441153', '1298617.9735*1488153831', '72315854.4695*1476007562', '64640486.8327*1462702259', '71777492.7483*1480071237', '59326251.5508*1460139735', '92223054.588*1487502960', '82292145.1222*1453789740', '94659087.4629*1475355050', '34745383.9068*1470185855', '42814201.0101*1472411160', '2458364.77027*1464606992', '94177325.7819*1461462576', '6048637.06475*1483484926', '9381825.4506*1480259083', '86597378.1827*1464844011', '21822018.0213*1456846869', '41936216.3037*1479224683', '87185366.7938*1461418648', '41609039.4234*1478892578', '53332028.9617*1455417919', '97014833.7181*1486413684', '80161913.702*1485074093', '60445280.9408*1455168260', '33144333.9198*1468416721', '12042455.2764*1457637849', '99186745.8902*1479536435', '73363444.2597*1458138671', '11520506.4185*1458938574', '26253450.5093*1456709083', '69079039.8155*1482558086', '10409885.2691*1478573972', '66787150.2349*1467363654', '91396314.2104*1476396602', '93842068.4585*1456673272', '86748228.8524*1465672569', '57065354.3668*1484707058', '32380027.9287*1472820251', '86207313.7284*1475319816', '41627170.4985*1486077881', '23839263.5193*1462583101', '35322620.4192*1477442797', '43217879.6595*1454627808', '33387559.5508*1452336906', '40859007.5134*1459873358', '69125575.0711*1462816163', '70339498.392*1474192493', '68145455.5388*1460040844', '91279381.8413*1462439363', '71317207.7142*1462889313', '25671688.4062*1465407137', '64501135.5428*1478281211', '40030225.2612*1462840610', '62056909.8021*1469734319', '12909863.2328*1480284809', '85530373.1922*1475837084', '27471359.0784*1472397477', '42611999.0923*1470345793', '48307388.7811*1458019989', '10123189.2319*1453343936', '23938917.6502*1457843191', '98094972.5473*1472011052', '26872436.575*1457359893', '85855217.8617*1487483572', '58389438.0534*1477024568', '77575731.1964*1483750216', '93796248.1857*1465304436', '341053.353945*1464036484', '53979762.4164*1457214410', '63135721.313*1479560259', '93338088.9616*1487413184', '10094407.2102*1482888299', '48554926.3569*1454354848', '33110903.4397*1488262567', '80211991.8731*1453165799', '40447558.9709*1458507479', '67744366.8935*1453425660', '44081395.728*1472665609', '61871352.9245*1484199365', '63159751.5586*1457853068', '15484735.2509*1463998015', '32303443.1235*1464789777', '95503784.3901*1474660904', '28030217.2814*1466454739', '19782992.4588*1458308175', '11597429.8116*1461698419', '6166770.7518*1472520772', '659702.818746*1456614442', '82820219.8065*1469559259', '20350011.4469*1458297813', '51396438.7815*1463063248', '87534606.6155*1454479886', '88468313.4491*1470060954', '82387408.2841*1467329584', '89491295.3275*1460121610', '95000641.5339*1451948169', '67237827.0319*1486548023', '21454620.9336*1487107995', '17832039.7842*1470598239', '36539717.8984*1482335200', '341053.353945*1464036484', '98225767.3481*1458165617', '1631794.21225*1470576284', '44136912.8966*1482523656', '93040954.3405*1482245788', '10481244.1029*1477224971', '96433867.4683*1481494075', '57143582.0876*1481821682', '81287346.0181*1484995791', '55291691.0153*1478111425', '2337549.97784*1477757845', '54253623.4244*1478586617', '37784273.9867*1475268455', '43582977.2218*1476680831', '23059816.2139*1477279945', '6053629.27477*1456308665', '87670785.0181*1460389559', '65666814.5639*1465363844', '84383484.0676*1486469301', '83882478.9205*1458546762', '99691115.2471*1468787750', '22538789.9938*1479793265', '80560215.7683*1463127617', '61809159.9811*1486182940', '59496564.2171*1486978079', '43363610.5017*1451762811', '40248709.8606*1459307565', '35717184.8556*1467629328', '12487718.4318*1486365265', '3340441.41332*1462699158', '64096800.1336*1483471781', '78987139.9249*1475438878', '68355241.1866*1465384724', '93380340.7097*1462855376', '21820679.7111*1469364936', '13189839.0356*1461917734', '43374455.6477*1483276899', '15043651.3192*1487869596', '92734854.5376*1459417799', '78713642.0183*1456408037', '36254239.5431*1462767474', '93207725.6265*1470152106', '15091508.7937*1460087243', '13622329.4637*1453460459', '25556180.7008*1465886531', '92356130.9194*1475590579', '70947004.9759*1468251028', '18751106.2129*1464443405', '89474613.9485*1479370894', '97441657.7969*1462531839', '61835825.0204*1458521031', '46506049.0241*1456794960', '33485488.4029*1483213211', '58000902.8023*1473884505', '26871653.7299*1460636111', '13352234.4522*1458434775', '11794016.7285*1455154618', '71042740.0511*1468645484', '48242150.2286*1458333930', '89258489.795*1471398462', '57342173.4842*1463687230', '7625556.70056*1465659914', '57415590.9796*1473571235', '56046243.1899*1487450336', '21093237.5127*1461356140', '87165388.9789*1479104485', '20887381.8214*1461128473', '52117986.662*1453290881', '77511569.9675*1455786136', '78267060.4882*1467889982', '5557888.52282*1454097399', '83699747.0656*1459890269', '47586954.4082*1452334132', '97218623.593*1472493755', '36231343.7046*1454360848', '84004582.9048*1484754101', '75337902.5617*1480717110', '78992028.9547*1477131992', '15270900.3639*1462016512', '9902517.47562*1462173032', '66908951.6802*1461176895', '64461175.3921*1457863946', '17185311.2848*1453246994', '54070428.1128*1464969486', '29775851.5166*1479999731', '73718741.6662*1458477070', '97281699.5543*1467279034', '88099743.3971*1466258497', '63779210.291*1474819687', '8401090.87211*1454529324', '47581037.951*1470245638', '6593489.76955*1484777702', '95528783.9349*1485248401', '29681235.5714*1481440909', '60626970.398*1451954918', '54344825.0706*1452438952', '55881394.1751*1461896003', '70666631.0342*1459250533', '85069593.4684*1469463928', '7231948.66792*1481622876', '4759938.11742*1468464454', '60181310.851*1474695758', '44456678.7839*1458383038', '13015825.6997*1471087427', '81262696.0682*1454594623', '27895075.2892*1463434328', '35413427.4289*1487201308', '50780543.6088*1470066219', '10697542.8259*1484736473', '2907769.46368*1468928939', '28494885.9153*1478307038', '13506552.53*1472902690', '44449478.7873*1466639260', '83781720.474*1479514944', '12189698.7496*1475846389', '70901377.7097*1472743619', '50084943.1988*1481305708', '98336418.8765*1456508222', '43288555.1488*1473276409', '24309279.1264*1484691231', '32222515.3173*1471117663', '55850476.2046*1481727623', '30427395.1217*1481224573', '17721124.1586*1478068955', '29501168.434*1483421217', '32151500.4414*1477626085', '52221405.7908*1459197191', '25414910.2174*1483976663', '63631894.5312*1458570263', '82034533.7748*1478611757', '26071696.6809*1454808224', '16842651.5937*1482556846', '63978972.1652*1470717408', '77746472.3185*1470530088', '31563494.6344*1462893269', '4517798.04964*1476034988', '10574124.8003*1463642550', '7218659.0564*1472405189', '24021395.1635*1482695076', '66083677.4668*1476254037', '8977745.98309*1471669561', '92220225.1543*1462369262', '59647998.4178*1484327382', '69665780.4364*1476997890', '14516067.4311*1456581054', '2934908.4705*1471635422', '17754800.1515*1458061761', '30086606.8834*1482867859', '94043059.1527*1479098448', '64678891.8791*1454357108', '28367780.7906*1453084155', '43748863.4913*1479284857', '91572502.4326*1465138645', '4493397.30999*1472152533', '25515147.4014*1455543976', '75166501.4613*1473565140', '77344771.252*1454009509', '35527599.3739*1466093500', '13349489.8335*1469071986', '41925154.6304*1458991429', '24738043.531*1468284308', '9019925.82208*1464668906', '17268769.4781*1456752051', '59082869.842*1475369025', '77002802.0319*1472250069', '75040669.4381*1456786777', '72529399.701*1482177480', '64024962.4967*1453398592', '46643984.6583*1482770468', '17721124.1586*1478068955', '82591246.7083*1457545541', '7524221.03308*1464182203', '99243137.0613*1466541114', '89688434.1239*1473220985', '69386058.7551*1465696691', '51367285.34*1477336682', '14718959.4609*1475938746', '16687802.7987*1457798277', '35581676.9641*1487455431', '26525178.7386*1476384391', '37651742.3431*1468229492', '17631890.4268*1459576981', '23121371.4109*1455082313', '1878387.22283*1454213960', '67777702.2774*1471513716', '52980130.1963*1465477760', '74360122.5874*1464704731', '37425779.1246*1474395138', '48018255.085*1486496149', '11937817.9374*1462316929', '51046807.6471*1474459921', '30660139.4708*1471102838', '972206.51156*1454414086', '69178946.6534*1461206402', '82455232.8098*1482959404', '56570100.4696*1485201743', '42002653.3014*1460145720', '74146962.3701*1472952230', '99354137.1763*1486387956', '3547869.0042*1476845374', '30785508.2772*1452968455', '67176311.629*1453064064', '55544549.0627*1467629313', '59735949.3315*1477469262', '78838404.1439*1469482085', '24932501.4326*1471329387', '73856215.4022*1455601031', '58066342.9881*1484949597', '66482309.424*1478677211', '43958197.1602*1485306756', '61833295.3525*1473346941', '67286274.8838*1478997024', '7419718.43974*1464186351', '71399618.1518*1468053449', '26075908.7961*1486826980', '97087264.5925*1477788331', '2186306.55341*1453677354', '35666677.2796*1460464725', '87234736.4759*1465080690', '68880230.209*1464444579', '58087569.0054*1479605005', '73441912.6905*1483561078', '72452649.6444*1469317664', '1847108.91397*1458828641', '39871145.3849*1459374470', '72181688.4798*1453822041', '82170803.157*1452804298', '35230297.5699*1452539800', '64461175.3921*1457863946', '5495034.35884*1483251800', '64103303.6942*1480633811', '67286274.8838*1478997024', '71258907.4586*1474518718', '56160709.1619*1454153688', '49438732.1062*1458406724', '16842651.5937*1482556846', '47681069.2668*1468885152', '14718959.4609*1475938746', '77070217.6112*1467713068', '76787761.878*1483112731', '36003283.1291*1478212106', '69027469.4122*1486682218', '39317300.5824*1462627383', '61312631.9337*1461806783', '56823126.0969*1461277210', '87325089.1103*1478848078', '56809730.0101*1472029624', '47730652.1818*1485291631', '50610514.5298*1482402335', '63508610.5467*1453482148', '38149089.7314*1482286619', '26516774.7306*1454650427', '12628596.0151*1478179698', '69257232.5258*1483478683', '45383040.4792*1454270358', '71695324.2886*1476804954', '58373382.294*1484370088', '31032328.54*1467725770', '44639511.4194*1452341096', '63562682.727*1457502458', '17129821.3593*1466053432', '56585545.1571*1459745673', '13571772.7939*1467780176', '75784146.4299*1485847265', '14542122.4351*1470062246', '50362035.4185*1455163576', '95539075.4215*1452175467', '68185662.3577*1488195323', '35739280.5049*1487181702', '13523125.9881*1476364017', '63997417.8999*1452781794', '54064699.3959*1470641121', '96110899.5466*1470817648', '91941843.1958*1475324712', '76702254.7838*1480894774', '59424955.9654*1469463781', '4577070.20791*1480201150', '50158531.9242*1465128238', '47030293.3045*1452076829', '42666717.5521*1468797530', '27286596.0551*1480129051', '26069511.1083*1478202980', '25414910.2174*1483976663', '84088857.673*1468598043', '30460876.538*1464502578', '3297984.49726*1478126286', '27238772.4662*1483152282', '27798447.2371*1479905531', '77591132.7557*1476507451', '65315502.3478*1464239137', '7950073.68168*1456588753', '87030304.8501*1475412214', '28376478.9099*1475979664', '97424325.7662*1468883686', '97105471.5657*1485789546', '18632085.2642*1458314037', '84828397.4447*1485525753', '36289813.5505*1472807386', '2967207.92883*1454371001', '22581854.8433*1462413880', '72620360.8586*1455822710', '44569561.2167*1456510217', '22349744.7373*1476241454', '38932647.824*1486688572', '47192423.4201*1463458117', '94036158.0646*1475106915', '42132725.8889*1483164533', '54478823.7366*1464862498', '45805902.1461*1463610146', '57002267.069*1476224878', '25796796.7907*1482608153', '90409138.2177*1484329445', '43563373.8631*1484332344', '17470856.6814*1457270809', '21694077.713*1452492151', '62093151.3611*1452327342', '95961486.7419*1457504351', '49651347.6619*1456704274', '38172025.2729*1472469250', '46774866.0261*1481732023', '6838134.11951*1461250783', '78177818.8547*1473516770', '28257615.6079*1467680355', '18856737.4486*1472454736', '83757738.6415*1459287627', '68325686.3575*1480947190', '17494498.7696*1463172567', '31869056.4283*1467909650', '87400700.6384*1464297084', '1585303.8512*1461410549', '3839703.61967*1476193741', '79842537.8573*1477629062', '4522016.98972*1454344105', '75627447.911*1467238650', '23774179.3978*1462526398', '40549416.3033*1461314490', '69351363.8915*1474587624', '71022491.2529*1485088010', '8652509.79943*1455697338', '54358870.2995*1484116638', '53070746.7973*1481025861', '83361740.2642*1465686239', '54673307.0998*1481190864', '80763536.2819*1454973312', '87185939.885*1467039912', '46985677.9552*1476212865', '33209952.8525*1484216773', '24655926.4967*1475400641', '17307628.8095*1472712186', '98507877.3218*1484378707', '86598135.3755*1463822303', '79373502.4825*1470030793', '99052560.3344*1456976059', '75881205.9038*1459974594', '30858323.5209*1487146781', '17529285.6082*1455208784', '80802273.744*1464768990', '33532047.0646*1467473257', '36491182.9909*1461085274', '41047245.3314*1474806579', '54771642.9232*1453605592', '70386924.3965*1459656656', '76258893.8072*1452356249', '39008493.7872*1484893770', '56067042.2541*1452383896', '17129914.9854*1480666341', '71738164.1995*1456233402', '71387503.9726*1471261666', '40377838.7901*1482980090', '38117810.8726*1485166278', '43958197.1602*1485306756', '18450107.1695*1455473336', '40024808.4487*1464321870', '65810724.7802*1459955613', '28039402.7602*1458290707', '26744747.8475*1474859071', '62317738.5458*1481965506', '7499635.58374*1465435488', '89491295.3275*1460121610', '25263804.3423*1471805603', '45280083.3374*1458033159', '83307989.6938*1475222113', '4651514.61774*1455695958', '46774866.0261*1481732023', '77044100.0759*1486746610', '94197222.4364*1475488313', '55702038.6737*1455142281', '8513952.84621*1487819551', '74100494.7218*1487751573', '11268821.7371*1462943699', '84919924.2031*1473809325', '742490.359921*1453047910', '29627164.3441*1472832392', '49473904.6224*1483036668', '84391126.0283*1456274073', '16678175.9508*1477178866', '23036125.5345*1479238686', '77711561.3475*1484977719', '64954334.0532*1459215478', '50125846.2902*1485220778', '8909140.60055*1474840664', '21628884.5853*1475740862', '40533338.5811*1460612549', '54952333.2627*1458415505', '40057354.986*1472675098', '45673971.6571*1454547014', '81782590.2431*1487499866', '93166929.6122*1471765911', '30572796.118*1475433545', '59015306.7698*1478235619', '58107610.3667*1460718848', '12006871.1519*1467658286', '83450751.9655*1463202675', '94084809.4092*1474606661', '56648735.5096*1472712923', '93461862.1516*1455962879', '67072941.9909*1468423853', '41707225.5556*1476682943', '52663294.7276*1477969038', '69555734.8507*1478625744', '90698041.895*1478419802', '38128064.9634*1472651365', '7367430.35045*1454565042', '68895658.4403*1481425255', '1228329.75932*1455635498', '12938394.5478*1451881008', '55579297.3975*1460389055', '18667889.307*1474028292', '54565070.4088*1484297659', '89012091.52*1457813906', '83478962.8475*1486347347', '353528.405902*1482687678', '3965662.76881*1473719687', '49599860.9224*1488295605', '6006228.01139*1476116162', '78781184.8652*1466215521', '19037622.8825*1461692075', '15091508.7937*1460087243', '21557691.1172*1478251826', '56343248.3519*1480660251', '84449102.2042*1466683703', '87132345.434*1457899637', '78193436.7757*1475807191', '31167721.8031*1459056940', '83550294.72*1479076852', '79684486.3212*1487520614', '19444859.537*1479567425', '32220780.0109*1456189018', '83660339.2731*1454684450', '24985281.9277*1464455933', '8001989.54511*1462222169', '13094614.2534*1487709102', '12378857.1214*1475797134', '93473467.9295*1464125545', '11352106.2089*1482011366', '14814244.1324*1470493409', '56830928.0425*1476254133', '23302558.0908*1464163660', '85401893.4065*1481057052', '70879620.707*1484431565', '40034273.4086*1454410566', '77187072.0082*1483478801', '12779728.7423*1478906970', '77035153.3101*1454375737', '93160763.9415*1453441705', '542649.872153*1456836602', '82243880.0182*1477620544', '843613.193685*1464341059', '59523223.33*1487539117', '15380162.8348*1481681342', '61260145.0777*1473189371', '23751978.9839*1476204977', '7262623.87706*1484531362', '292739.90253*1471917573', '81091087.6598*1477745796', '26807253.1427*1468230199', '98746948.6687*1465297971', '73641272.176*1467244784', '29922620.5123*1456997263', '21850095.4484*1459776343', '79144210.8316*1453922682', '87480025.3804*1462550422', '40818575.4607*1453657764', '61410035.3833*1457818565', '41857405.1455*1456928572', '50701622.3633*1484811344', '88204257.2318*1475157590', '89457946.6185*1463577233', '93716648.6438*1487581696', '8853248.2442*1477732308', '47956566.2671*1461268775', '40494500.2038*1465309456', '1532849.85046*1477305295', '47986835.9358*1478396539', '7191412.57471*1458165031', '22891347.208*1467209211', '15992584.2673*1463245366', '70101863.3828*1455954519', '32059747.2747*1460578418', '40140679.7399*1462784866', '23570174.4826*1485717124', '71245171.4418*1465097444', '54171314.2117*1477450368', '76779786.1498*1456846126', '25609866.4749*1468906794', '26505052.1266*1477995168', '41309729.0324*1463037592', '2537186.60596*1468391673', '42216971.932*1481233819', '47397505.7338*1472011806', '87036484.8466*1468306571', '85777067.6394*1469363311', '70776062.4919*1468186982', '44005093.7748*1487696660', '84458886.1074*1457944584', '17268769.4781*1456752051', '55955577.2964*1470171697', '61422745.4302*1458198293', '60694064.0887*1460051261', '92362578.8193*1485505588', '37763628.1433*1468228401', '39519338.2479*1452214123', '68307936.5557*1483791841', '25102724.3822*1458478120', '58092363.2376*1453465523', '26392102.9109*1470844872', '54892590.9119*1486307058', '58623055.7417*1471527523', '88678961.7317*1486616823', '74815446.4893*1454912304', '21452843.8223*1461369451', '81108211.2849*1459932328', '81080757.265*1465082482', '98419262.8629*1481665386', '31570858.0608*1479888076', '7956106.1456*1465444017', '35413427.4289*1487201308', '10650919.1113*1457940440', '8952980.63814*1478513778', '22744873.4124*1474512902', '84506729.125*1481567877', '42741697.9957*1470089165', '98063024.6896*1476056659', '90458298.2103*1453486849', '92414842.7975*1466487602', '10145514.7238*1487382267', '15620205.5013*1461478153', '18078565.9097*1453745718', '21829382.9834*1453534129', '33614670.884*1462847337', '44094180.1082*1480212333', '27632751.9439*1480343474', '93380340.7097*1462855376', '34918676.4457*1451739437', '74815526.7621*1462639826', '48022189.7656*1487318377', '15991375.5656*1477429021', '33417739.4272*1472264379', '18311510.5631*1458666607', '4549108.28304*1470316482', '50753184.2297*1487708618', '49895310.7845*1457386437', '45603647.9129*1475322717', '57996990.5525*1478107705', '6638636.71362*1465246239', '65289566.1524*1460764269', '19592175.1259*1455395665', '69954145.2162*1470312134', '98214229.0173*1456269742', '30172801.1096*1472387967', '78904798.7771*1483792510', '90671975.7072*1463649108', '7578576.25484*1467270962', '82094852.575*1457965722', '87234736.4759*1465080690', '55702038.6737*1455142281', '52241505.6222*1456062402', '18834740.2042*1466989230', '41075217.13*1472966553', '54054412.9018*1456744814', '85916200.9179*1463600634', '73103910.0751*1486732074', '77081850.4656*1483690926', '63873821.7154*1457386138', '19687299.1482*1454917435', '49639964.255*1476531097', '74649606.1451*1486810617', '37299843.2478*1467904070', '69440788.2679*1457832779', '85083089.3115*1464039617', '65568844.7879*1470408450', '68577759.3482*1485770304', '71210368.9796*1465590923', '98683283.1367*1474036168', '48264791.2794*1474290322', '62654321.4975*1461616667', '2777550.09469*1487812677', '21750446.566*1474481563', '15932960.687*1477186673', '67507438.176*1467917283', '69554197.9954*1480660524', '71023458.5251*1456853934', '44336724.4919*1458815844', '18673684.724*1453008418', '78066945.4928*1477036772', '74885386.3984*1469955065', '54253623.4244*1478586617', '15390066.5901*1462122974', '50145628.3051*1459341832', '74640284.8075*1465146265', '55811517.3482*1459820984', '63878838.0383*1463631463', '18551006.6092*1479539996', '61917007.1842*1465876215', '93045116.7387*1453798480', '27332203.5305*1482751982', '54238066.3296*1467466736', '63057882.2181*1469580816', '96825767.8004*1458965370', '28654495.1478*1474638831', '51906608.3926*1469950586', '70072858.1647*1476466147', '47403624.733*1486343234', '47654361.9811*1461674031', '25825240.2253*1487096714', '46011777.507*1453679448', '29877750.7501*1466445364', '10557922.4026*1473861890', '79239892.0462*1473466690', '62852892.5268*1470074388', '97626373.7479*1482367075', '87572312.7005*1466593343', '28564515.8398*1468003801', '76125267.5858*1485351985', '99285447.7541*1456147936', '22563343.8721*1476189644', '5511381.61096*1487326543', '87833759.5462*1466555419', '23221730.605*1476343022', '654247.030339*1469390872', '83303419.2295*1486478769', '67434275.4235*1482822662', '14989448.3162*1482843802', '31350561.8864*1457386756', '55579297.3975*1460389055', '55340132.7088*1465800087', '61260145.0777*1473189371', '43664557.1494*1477212115', '36475569.9321*1471872498', '12975133.7502*1466259281', '45765351.9657*1485220498', '37505724.5672*1485555987', '38395984.6584*1452741984', '25545817.1107*1461456397', '72811418.8602*1458030184', '81401220.5995*1483313037', '96707698.2741*1481535169', '80836991.927*1470012664', '56430800.3671*1460436482', '40000471.4451*1486939537', '32974260.8019*1484821041', '95764587.2706*1486093644', '94055374.1168*1480395017', '38036983.372*1484718852', '77900905.2077*1485879471', '97654056.0183*1468772906', '52160349.6746*1470390062', '8182620.68458*1466084483', '43735917.5527*1481788597', '26043941.9343*1467007714', '99580278.1693*1462715815', '94717376.1226*1465056190', '89794309.9256*1452069224', '15576025.6484*1481176933', '76667463.1729*1455928065', '39446053.2816*1468699196', '49822237.1391*1486697570', '49492100.0394*1461552428', '51112131.2218*1485959066', '9021758.89829*1461462941', '52820166.3703*1473055115', '26688060.6618*1478833143', '73105641.8232*1454791106', '17560480.6289*1485601875', '33076574.3956*1481224529', '81158683.9869*1461366495', '83898762.7133*1460300122', '58611965.4048*1472155449', '4571640.15565*1464479817', '95472586.6088*1461515316', '86212701.8507*1468827770', '80777542.539*1452848232', '54507473.4977*1472787089', '32876055.6444*1474721033', '57769570.7792*1454947258', '61348978.847*1473231598', '84979321.4301*1476253102', '69322514.8656*1480703742', '93025076.2407*1478036525', '44798080.3672*1460881925', '15770024.4229*1482539191', '5160765.14155*1468046867', '24485901.4663*1472858414', '84687178.391*1459888414', '35817984.2016*1468278403', '69509699.645*1475124450', '67794895.0712*1457313892', '46633112.8381*1456649712', '40786149.0992*1461008724', '35527599.3739*1466093500', '49172347.1817*1487502516', '74224319.6432*1472522802', '37766864.4992*1459763762', '8403533.55919*1465277326', '55187008.057*1462276744', '74786901.375*1487199223', '3159028.50785*1470250437', '58563699.6004*1477136944', '64847558.3943*1458025711', '8289365.87234*1474015189', '9689650.27474*1462041061', '55238464.6865*1478945537', '64235315.4555*1486347717', '4219511.75136*1483209771', '64690112.7234*1452575051', '80210373.3073*1457458817', '78225642.8457*1479064931', '60948559.1326*1486245497', '73879840.3799*1482590307', '47978994.0217*1464213827', '89162751.4275*1471921160', '93579366.8708*1458009727', '39385777.6532*1462589092', '20907960.9733*1485342474', '65898444.7963*1461971752', '64117751.119*1486630638', '70072858.1647*1476466147', '15448860.8845*1464446202', '76899290.0157*1461531377', '2978393.21154*1486724355', '2085396.9253*1455650525', '75078409.0955*1464776819', '20443013.577*1453874541', '28083058.2513*1473030771', '7434202.06793*1476432906', '9659817.20247*1474827364', '6963063.34692*1463135868', '63160083.5015*1482784364', '879621.209467*1461215940', '76125267.5858*1485351985', '72811418.8602*1458030184', '48649360.1472*1480155325', '38787515.8599*1467463553', '33189110.7702*1469541958', '96014047.3098*1460542734', '95995726.716*1462520751', '31508988.2219*1474111642', '13132259.2355*1451816738', '13517254.6747*1478508651', '20377561.449*1487915573', '89643474.5631*1480398681', '59899615.353*1467180321', '41748034.2688*1481949226', '51592140.1193*1468036739', '34419864.759*1467380086', '33865478.5208*1483833860', '32784352.2038*1458670847', '58572592.1394*1470439810', '98042638.3594*1485501803', '92486790.6122*1462793261', '30664907.895*1457608995', '78214805.1995*1481624122', '18095321.8953*1476247660', '99360999.3226*1459451772', '44136912.8966*1482523656', '38382353.15*1468433715', '64938310.1659*1484483746', '31142232.8154*1485026134', '34035248.048*1457308695', '40067876.687*1481483829', '46929741.3442*1465447137', '9098161.55501*1486171952', '98606330.1961*1473111226', '27832758.8378*1466000617', '74658182.7255*1473767837', '38508965.7583*1460037450', '28323648.8827*1488107117', '1398104.61407*1462847179', '76020997.3109*1457037310', '95988837.468*1461180807', '37651742.3431*1468229492', '25359956.5632*1458770552', '96171301.5066*1469558592', '52663294.7276*1477969038', '76999109.5824*1469231310', '68355241.1866*1465384724', '67827774.12*1457975156', '32636443.578*1456625715', '51344155.5481*1455026228', '48904724.0021*1476986933', '83246378.2568*1486673920', '42379948.5698*1464372594', '70537951.2663*1462769221', '70718892.2703*1481681014', '33370918.6891*1468144350', '40534182.6324*1486269468', '15644054.4182*1467670121', '60106493.5834*1477159288', '51503696.9889*1472878317', '93418266.0823*1454189142', '43422736.1081*1472863760', '80045629.3178*1469075370', '30524648.249*1480030204', '78596930.3223*1458792211', '98004419.7416*1461485067', '93233786.4669*1461754400', '73883335.269*1477346578', '19267294.62*1466571988', '94565390.4055*1457800140', '47939676.4293*1475704466', '63687759.7034*1458775853', '14997111.4785*1458187487', '35726243.6663*1465590195', '66251271.5712*1478358512', '56928113.6028*1486383871', '13966356.791*1456121138', '36554059.491*1480679026', '85575829.0101*1458525246', '71990010.1128*1476955551', '21914819.9309*1460586997', '57159699.63*1463485782', '51289101.2173*1465647999', '27258693.8624*1473489093', '61361782.7778*1454622154', '52683411.1171*1451763169', '34166090.34*1462227428', '32440576.4471*1453427992', '42139155.8137*1467931672', '33998331.2989*1454918800', '87053222.6233*1452576922', '30544073.3541*1457386265', '16886001.5462*1475196297', '39697590.1833*1479366953', '47654361.9811*1461674031', '49386963.9175*1467765831', '68579401.8551*1481187984', '93892116.2176*1452648070', '40034517.363*1460730037', '31783581.7959*1455016538', '13373954.1942*1476200400', '54893726.1125*1475152972', '81811525.0717*1477500356', '11912370.9266*1465086517', '66140689.2807*1474082914', '2024355.17918*1478084307', '64702891.2354*1453082412', '58936634.9361*1480465472', '59721763.3675*1458287524', '86326552.4346*1467365994', '15416874.7064*1458405589', '77435620.6553*1461686168', '56952253.5008*1465221631', '99177886.5013*1463937233', '42524040.0244*1480877490', '59313071.1265*1478248924', '49314612.1762*1462989872', '52577543.3193*1465479522', '94701998.632*1467749667', '83985103.5058*1475307022', '79258605.1448*1471737982', '81261023.2334*1478554215', '95472586.6088*1461515316', '25378223.3002*1453147559', '41914863.5386*1477130976', '7891463.54445*1481535092', '45285013.4123*1472454861', '51522661.6873*1485592474', '85013772.812*1486804158', '32488755.1974*1460115601', '70610505.8169*1487976106', '80672312.6522*1470403405', '7856506.62062*1483926407', '27606019.3881*1481067427', '93623698.2126*1473182148', '42446787.7968*1474226908', '63605962.6158*1462554988', '50837566.5866*1454014755', '47040366.4852*1486274327', '97639160.3722*1456297436', '81726782.4626*1482724671', '14035422.4445*1479225434', '50362035.4185*1455163576', '75131445.233*1483077120', '82279226.7137*1475079205', '89621375.6199*1485643255', '78537572.9837*1475456620', '47237771.0555*1484237254', '62803933.9379*1474881666', '99462343.721*1473307070', '94575134.5836*1468863158', '37351095.4568*1466796501', '1122628.75867*1474696063', '38129501.4768*1475304347', '47135678.7074*1456827838', '5058469.16514*1455922050', '31741306.2032*1459742834', '91954701.0542*1484307989', '52251300.9418*1453236054', '70492941.159*1481707005', '32070546.7382*1485498360', '44005093.7748*1487696660', '88461887.4017*1452564626', '44558086.1381*1487033737', '15028044.4587*1465160327', '94977146.578*1476231115', '51046807.6471*1474459921', '97430814.8551*1454690832', '43298583.9458*1484563268', '54331375.2471*1486283481', '45374060.7508*1452413435', '16841909.688*1460522647', '47978994.0217*1464213827', '38724811.1578*1479708247', '667917.43775*1455011208', '28662922.4935*1463559935', '38128064.9634*1472651365', '35560615.7329*1460843898', '97303609.4331*1480574597', '70835198.6131*1483787207', '27421263.6122*1466467820', '46392003.0172*1463082870', '48196858.6384*1477567691', '5947705.40325*1483601565', '87082150.215*1452552002', '38630070.2012*1462540025', '69673219.9946*1486917225', '40774233.6309*1471265968', '11109073.0402*1475555239', '17204781.4569*1458929008', '42730607.3656*1452756579', '98652146.2255*1455026840', '27286596.0551*1480129051', '51910807.2755*1474503693', '44687326.3357*1466616259', '47764545.5833*1482699821', '99255992.3621*1479176236', '92157440.6298*1455063908', '59495640.9972*1475429256', '40482061.0311*1480203763', '11828298.3128*1455240825', '53434948.3105*1454444150', '59543749.3104*1471185482', '701671.461843*1474366403', '1228758.4908*1464452298', '15104771.2997*1481469932', '96595906.318*1461571072', '98853795.5334*1461195000', '18264598.6195*1455148066', '26000744.2419*1483538932', '64295885.3287*1458862298', '93637430.3378*1466954696', '91082623.6226*1467257908', '14516067.4311*1456581054', '60951456.5431*1453779361', '13077743.665*1468038655', '9188869.9385*1456455369', '31349447.0723*1471271934', '79938529.666*1474523233', '49751287.9131*1465909695', '21866147.4363*1466680080', '15805111.6006*1452438361', '99352436.0344*1460729422', '61820352.4753*1460016726', '66866444.8827*1464285685', '51344155.5481*1455026228', '36065748.5458*1483307417', '2355268.3099*1480963238', '68426487.5302*1483147494', '97790869.8585*1461091495', '32212947.3627*1459334181', '76727371.3981*1454337334', '46790.7866806*1480045530', '56012474.7686*1460714566', '83334658.8308*1476625337', '34793088.5413*1482523026', '88278350.9532*1481723140', '48680424.2221*1456672387', '32169045.3728*1469286406', '4522016.98972*1454344105', '75931304.2189*1458026078', '48018255.085*1486496149', '74263265.9538*1486573832', '70718892.2703*1481681014', '46204302.7837*1480823606', '80407745.4397*1477938803', '43682637.6611*1469035953', '49513506.6421*1484638319', '75372225.862*1475942181', '77370234.9261*1467252705', '36638946.9381*1463108993', '14957644.1005*1473251106', '93709691.6155*1460796736', '56045803.6089*1487588543', '58054639.1861*1484662929', '31577716.5882*1483143967', '97087264.5925*1477788331', '35444580.2944*1482567187', '17702066.4292*1484005797', '29914473.4879*1453610104', '42741697.9957*1470089165', '69979762.976*1460028125', '40647622.2441*1473754904', '59257952.5253*1478847070', '25515147.4014*1455543976', '61612416.5513*1458991244', '26524647.5948*1473017955', '13290646.1916*1457129165', '3103863.92781*1479176789', '59178768.6856*1460667614', '45332018.8564*1487852484', '50528473.627*1483147340', '53730283.8654*1456143816', '692300.604038*1467355600', '89178229.0705*1471390238', '13488080.6319*1486827063', '75313739.9216*1475616399', '16063793.3544*1462818631', '62803933.9379*1474881666', '54562159.866*1476101659', '7702411.04282*1486974712', '18291694.6931*1461741292', '62638249.9071*1466948404', '75542131.1136*1460200359', '71498286.953*1468740968', '20222841.38*1471732630', '24388242.3524*1462184852', '6070044.8611*1476538054', '24205868.696*1455882097', '44902171.5433*1461800639', '60706426.7367*1455400140', '18488840.28*1459896522', '71752723.8042*1475425901', '10446844.301*1459930249', '85823960.0809*1483374764', '70719502.0359*1461365427', '52862178.1727*1479816966', '45653467.7467*1476210899', '26495318.6105*1464003101', '26272957.7525*1479358881', '94710845.8924*1453455358', '36315044.5021*1479125058', '34114596.2168*1462492877', '15693010.3935*1455856975', '58591774.9607*1475174070', '44901645.0394*1458484651', '81624307.704*1487286760', '79842537.8573*1477629062', '69747546.9044*1456093808', '36820137.6137*1469236994', '44284321.2034*1470641321', '23036125.5345*1479238686', '18029961.7564*1483786875', '96493530.0062*1475639499', '55211566.7535*1469625789', '11163625.1241*1452287620', '15484735.2509*1463998015', '28592237.7637*1470819182', '16979503.9473*1471837585', '67146699.3385*1452451397', '86156950.5339*1469417431', '53354012.843*1487976639', '30565333.7526*1453423004', '33415277.5134*1456505011', '44647038.4922*1484865345', '17478168.9775*1479465450', '90123561.663*1482238749', '66392457.2961*1453145614', '81921715.5755*1460728454', '86784636.6809*1467842425', '45047037.8887*1486206672', '35444580.2944*1482567187', '92600671.7826*1459742924', '53270059.5622*1452537896', '26744747.8475*1474859071', '49921529.9611*1473970256', '35581676.9641*1487455431', '77550073.2566*1468568480', '43523690.3046*1480275931', '12453670.545*1468954483', '75872113.5404*1467817788', '43581011.6159*1477790786', '17702066.4292*1484005797', '10344110.7984*1475286153', '56410260.6733*1476172065', '8767592.95345*1475137192', '93089135.8778*1485950560', '23578332.3487*1460316218', '81721308.4674*1463169313', '52598529.9347*1455632296', '11443402.5601*1484110503', '98161271.8781*1454941641', '74074163.1005*1466728110', '75872113.5404*1467817788', '60819250.2914*1461515606', '71678397.4885*1487147664', '4883122.08788*1464320331', '80161913.702*1485074093', '47768337.5099*1486072473', '97974603.5916*1452595587', '21153097.5497*1481729948', '89585306.1458*1487662683', '89030687.3117*1487088478', '98297691.7578*1476785278', '1833349.01376*1475367357', '54014079.8809*1483934461', '47397505.7338*1472011806', '23967651.7454*1471066329', '94181314.1618*1462013653', '37905095.7574*1459836976', '91002182.7297*1475835472', '57481687.3869*1458591340', '1674846.44848*1468447776', '46188142.3976*1464218481', '64124746.4165*1471029509', '58697008.667*1468590369', '5865916.77035*1463273756', '33675171.8148*1465451360', '52842114.1717*1482607609', '83828638.9055*1469626930', '49430132.4061*1482172514', '32553120.5028*1473144227', '74203636.5064*1479957041', '32974260.8019*1484821041', '48922135.6733*1484162983', '40140679.7399*1462784866', '12204325.8971*1483682746', '68541325.4715*1472696548', '23699812.8185*1484283567', '19812357.2162*1457022868', '65745374.1394*1479776262', '37690600.499*1468939217', '51289101.2173*1465647999', '93842068.4585*1456673272', '30105996.7554*1479160302', '39775368.9384*1482398660', '67134250.4963*1475732858', '83977494.4094*1477598205', '29427269.0851*1453594313', '58874336.0544*1453561734', '2799676.55313*1476353255', '16886001.5462*1475196297', '17754800.1515*1458061761', '88677148.6882*1459143162', '91015745.2053*1477776660', '38341204.1313*1472117190', '93407509.0499*1487806742', '11803193.0619*1482863660', '76906895.7798*1469476107', '81771350.0081*1475872178', '53777935.0386*1467995205', '44128527.6298*1464886586', '33182409.8355*1480468660', '24425929.1809*1474762838', '71482297.9363*1474791425', '33032362.2348*1483381178', '68971315.4423*1460881975', '55943189.3708*1469643780', '32453961.1772*1481658839', '72898799.3609*1472634988', '18261221.5107*1473088762', '83068592.8104*1465902329', '65464061.5569*1461837730', '14079855.0189*1453299440', '94017246.8701*1466826703', '17348690.8851*1461132657', '16105353.1845*1474730337', '6838134.11951*1461250783', '13254790.9513*1484241922', '20601912.4618*1476797913', '28268632.6396*1464014039', '64939060.1841*1456123988', '67656570.4102*1470904721', '55340828.4761*1485080461', '76717311.6731*1483121663', '31228746.5541*1485750124', '31210610.5461*1456601363', '80560215.7683*1463127617', '46824110.3968*1451950245', '99091577.6306*1461735898', '98793934.7476*1453964999', '35116947.5765*1475842940', '25633697.3953*1475558321', '21378117.7889*1473839457', '60663117.3911*1479769566', '42887961.1212*1471572802', '38382353.15*1468433715', '88992561.2235*1466604188', '89876350.9556*1477563699', '81787690.3366*1453900665', '97218623.593*1472493755', '64248642.9178*1468954112', '31795365.2253*1458172637', '31870928.6431*1473680142', '43225516.1729*1461473853', '9516522.80787*1467508478', '17018454.0619*1469537154', '58244466.7336*1484940818', '42620936.0734*1486510390', '78838933.2346*1485132810', '38808003.7417*1460496018', '71908586.3342*1469670429', '36260464.4731*1486341583', '92330288.7781*1475075470', '13386800.3227*1478286700', '2102973.87761*1477626511', '14575626.5153*1482988793', '65744175.4451*1461580669', '66655757.9616*1468734655', '52507236.7254*1481896176', '49800164.2324*1458463034', '76806677.1158*1457384524', '75865767.0759*1485722833', '27001775.7996*1452125623', '19276717.0787*1484497032', '98635637.94*1466242212', '15565430.2891*1455124310', '79192910.8061*1454229565', '80384945.9591*1469711244', '75901441.2274*1477502384', '2800663.22775*1476463758', '44285619.2716*1469694108', '34245587.9424*1463896082', '74553837.4306*1477341513', '42244204.8498*1463600024', '59713275.063*1469329592', '10574124.8003*1463642550', '8853248.2442*1477732308', '33094223.1643*1479292764', '47667191.5237*1482353102', '80626066.0444*1465941286', '98964884.6679*1484233421', '81707217.7483*1470984582', '91092044.9186*1485667426', '84407327.2408*1455428264', '5575017.39178*1486456894', '7250387.88043*1456189648', '43413275.9777*1486023602', '69975711.8576*1480705609', '29670989.148*1470062992', '76710257.9317*1456057928', '38795865.6123*1476372489', '93649137.9696*1459001263', '85456547.5986*1486083849', '57591456.4163*1481631744', '95617770.0439*1477917906', '56430800.3671*1460436482', '65744175.4451*1461580669', '63160083.5015*1482784364', '91462114.7188*1486586442', '63409637.5047*1479497295', '18580846.9283*1477119995', '67075602.5916*1466103108', '13515965.9202*1458425740', '48843200.2288*1483399285', '93585664.6025*1457867513', '8112255.66735*1464570677', '40991556.2132*1463851726', '91134253.8294*1466454370', '96014047.3098*1460542734', '526223.131771*1470564684', '45617331.1159*1487746475', '25074197.1433*1458008022', '30691282.3947*1484322056', '8491665.05389*1470767311', '76321009.2216*1478611816', '18815006.6069*1473681219', '6974241.4111*1455861580', '93207725.6265*1470152106', '56046986.518*1459068695', '74553837.4306*1477341513', '28673667.0094*1478702623', '84385853.4245*1475314173', '54351614.0534*1453621894', '71152511.1972*1478272515', '31451897.294*1453955830', '29715554.1705*1474810042', '48264791.2794*1474290322', '87569590.6083*1472232329', '16621480.8842*1464945142', '25414910.2174*1483976663', '69954145.2162*1470312134', '1780085.97522*1456421920', '44465940.9731*1454636170', '63909703.7472*1480971999', '45696078.363*1483367805', '17763533.2503*1462441787', '20816224.7997*1473657550', '31274630.9343*1465472691', '91512849.8042*1473017319', '7807296.44345*1472961569', '83372957.6784*1451727555', '9356707.76521*1471108375', '56330035.1562*1472489099', '67546710.4084*1479533838', '57034333.5016*1459176457', '95626325.5317*1474264284', '37691796.209*1455019389', '74874340.4655*1487597165', '35413427.4289*1487201308', '57870596.9443*1482679889', '93402192.1956*1481019725', '45906414.5094*1463994609', '88516534.0403*1461192340', '99006277.6335*1454866548', '77502907.3591*1468765411', '57812397.5794*1482037167', '99549063.777*1474376694', '78214906.1015*1457478285', '66720262.1326*1457316671', '29018435.0455*1476446060', '91107353.6021*1471064572', '77314190.6426*1461571749', '70669649.6657*1471886048', '34736475.1851*1461536951', '36094362.0786*1456271288', '35329579.7095*1456713211', '10549607.7069*1468448794', '34154013.4603*1483915549', '8541006.06445*1478688025', '27238772.4662*1483152282', '32994178.6678*1462044144', '34221752.2828*1472802046', '37019451.3128*1479130129', '23199250.6719*1478615438', '83307989.6938*1475222113', '72631454.2951*1467601350', '14850697.1996*1470552236', '88928738.5208*1480984630', '96645539.2448*1469369389', '57109684.85*1477587213', '88099743.3971*1466258497', '71254900.61*1456402979', '63629000.4682*1467507927', '43847893.3665*1481459490', '80916575.5304*1465230292', '5507843.02019*1461628512', '46287384.9437*1461969360', '64258157.1211*1480780106', '73209960.0136*1453466342', '11945382.9246*1485076246', '91422712.1442*1471416905', '69599113.4104*1473884628', '47344535.9664*1473766712', '21763644.8568*1460052178', '56896224.1122*1476792498', '84412169.3246*1466904670', '53354378.2278*1457793913', '48280513.5335*1466310396', '53459300.6132*1488279177', '36370556.4387*1453553770', '27130260.9657*1458487878', '55928371.1229*1478312591', '76980067.5876*1472659350', '9361940.69287*1486133938', '15095854.8806*1475129801', '18564215.6725*1466407466', '43691178.3186*1467382183', '59226781.6957*1479341025', '27650952.439*1484848259', '23578332.3487*1460316218', '58625.2191816*1468479485', '64684716.6927*1480093079', '47900370.8628*1468110372', '46452622.1958*1458678223', '76726756.6343*1488009881', '64167439.6641*1477604320', '4351457.50618*1460384437', '64176521.2188*1479434506', '69509699.645*1475124450', '60780003.2218*1484681430', '21906421.9543*1464586818', '57320824.8858*1458149525', '52842006.2051*1479332402', '33928084.8825*1473577557', '9382818.47765*1477972912', '7052074.31428*1467328374', '21316181.3804*1479078418', '33761845.3375*1451783625', '13727352.9834*1452677155', '40591525.1025*1473117622', '50819105.8368*1476388461', '11803193.0619*1482863660', '58776514.8457*1483685771', '3691567.6641*1467880453', '25258741.5928*1456500980', '39754446.9507*1456074324', '1400086.44944*1457294484', '15945517.3687*1458052666', '48956507.8992*1470974157', '13172646.1935*1453331761', '28660404.4248*1476966452', '69907171.5606*1469662273', '54146004.87*1452888904', '79707555.1686*1488135261', '55434463.0436*1480612184', '37401149.0392*1478666824', '46103709.7755*1480486581', '81890751.7371*1470664513', '93477245.7229*1480076120', '8898534.08717*1476972871', '52477128.0519*1480952031', '50092943.5338*1470655978', '36859847.6577*1462492012', '71671342.2923*1477708328', '70120540.3804*1478031712', '13189839.0356*1461917734', '95744269.8656*1481103640', '6445344.18033*1460538294', '78076249.7288*1452244765', '6525555.85074*1487737150', '50140561.2575*1455148304', '33651115.6216*1467723817', '51261367.5187*1463651530', '49232004.51*1484436789', '74754951.1455*1453900307', '59903860.6866*1471592285', '94823261.686*1476190897', '87814811.5585*1478719759', '98063091.9571*1477244386', '83036127.7905*1478355520', '55135637.1353*1480131541', '23174676.824*1452858359', '46043573.9764*1456882567', '29922620.5123*1456997263', '43535221.2725*1474411864', '15379669.1006*1472426619', '9359247.9182*1480820508', '33371437.8536*1478092551', '40276526.998*1467655254', '80920210.3492*1459850388', '2098449.71018*1470855443', '49465041.5145*1463998790', '72733455.4577*1465936705', '47403624.733*1486343234', '1639867.73041*1477905530', '42353599.1188*1467632537', '83063808.9547*1462130130', '8138727.98216*1472012625', '27060117.8978*1486570213', '1665302.13195*1484546797', '70983704.1805*1487733265', '7676448.04519*1452616504', '31342305.8646*1477519058', '24494347.3639*1471393585', '89706954.4799*1478165663', '34002277.1927*1459160760', '48223778.7123*1480506027', '43372611.5102*1484012550', '42518355.9369*1464391526', '91592685.916*1466206275', '21197183.1228*1461025434', '79714105.0091*1478056851', '71660948.2801*1479671230', '24876786.8751*1451786023', '9384237.21785*1457157868', '95641489.696*1468586221', '18552535.481*1478734158', '74025273.1773*1455066168', '9902517.47562*1462173032', '66209247.325*1456084286', '42086775.4655*1452458152', '86834715.8509*1454046775', '9356707.76521*1471108375', '94646569.2956*1454781463', '38061238.4207*1462258520', '17406487.8011*1487553331', '66474823.2898*1453876393', '15349044.2284*1469873551', '92633328.4409*1461990798', '42915510.5033*1470262411', '34560668.6119*1479203847', '96327015.7411*1478166202', '15974169.8845*1459184170', '13125999.4211*1482744908', '90048114.9864*1486584484', '17401366.4962*1487480731', '59180167.1258*1458015127', '54854338.2399*1477260182', '380051.495521*1454609413', '67077052.1562*1479244225', '4783086.29175*1460999664', '1920007.81007*1462314666', '75606499.036*1454053805', '58591774.9607*1475174070', '55614185.2795*1469727643', '90091832.5668*1463471606', '25640807.9629*1457612920', '88215174.6492*1477936594', '29880885.7345*1475377760', '43753990.6709*1472029839', '68415895.9183*1474973566', '45046301.4495*1481220288', '72849245.1955*1459264890', '84277571.6352*1481186324', '94043059.1527*1479098448', '97413592.9664*1479393877', '83173851.4915*1474596388', '74112702.0522*1468996852', '87579613.7996*1467245904', '24378304.3267*1485370228', '9706347.67487*1456437330', '96377117.747*1471504989', '15967018.1207*1464939125', '82694609.2964*1470315276', '33543898.1486*1457652779', '47654361.9811*1461674031', '66488838.0112*1472554597', '66754486.7806*1458898099', '60212956.6296*1488017847', '98675355.1842*1456305443', '15693343.1012*1482430899', '87201295.0264*1469950543', '83478962.8475*1486347347', '98419262.8629*1481665386', '68626784.71*1477778264', '83964594.4564*1460200672', '68185662.3577*1488195323', '25629339.0504*1457446930', '71777492.7483*1480071237', '10072278.7753*1483203421', '40067876.687*1481483829', '80451875.4816*1473954938', '43748863.4913*1479284857', '42900917.2031*1456508915', '54899404.5313*1469655405', '13980317.1331*1483895948', '15095854.8806*1475129801', '3585621.1081*1453752395', '27268809.4003*1461274066', '26463914.864*1477010364', '32665776.7241*1453332005', '24309279.1264*1484691231', '64305559.8419*1454343810', '21365269.5936*1468911597', '54428405.7218*1465076240', '94055374.1168*1480395017', '8181753.92561*1488315492', '31795467.5962*1457505556', '44188434.75*1484666960', '31487046.6451*1460642155', '19004579.1422*1471427864', '49172347.1817*1487502516', '42952132.1187*1473081961', '5995672.96466*1452707474', '29973423.5797*1473417849', '24045525.869*1480454787', '16860716.2292*1471554105', '37651742.3431*1468229492', '96461109.7426*1483295487', '40991556.2132*1463851726', '97668175.6337*1474090048', '78459154.1167*1467280400', '82921460.6697*1464301235', '53444469.8409*1456551086', '55700809.3233*1472742490', '17784225.7118*1484436610', '80534282.1813*1457785663', '91708008.1531*1484380110', '37493606.0136*1474245760', '13260548.7335*1470994137', '37640513.4467*1487965733', '87223296.4388*1483391984', '81811525.0717*1477500356', '55340828.4761*1485080461', '66725586.312*1476087676', '77319578.0386*1473064721', '84828163.0488*1463711375', '87317125.3921*1465002170', '80045629.3178*1469075370', '19717953.4722*1466700832', '91988898.9702*1458844518', '82764840.5924*1484867871', '42447632.2824*1467517032', '53802954.4629*1453771251', '30671645.7339*1462876082', '19582882.9853*1467907258', '48988049.4896*1459024273', '57500031.0476*1476406691', '88433591.2973*1469047121', '76370743.2467*1453513492', '69219480.0284*1466197610', '14082376.684*1468299180', '46537994.2658*1471231013', '85400458.4223*1467812243', '91687610.5167*1452692188', '7956106.1456*1465444017', '47900370.8628*1468110372', '41220493.0401*1462677105', '77456470.6405*1471555412', '78157554.5068*1480203357', '18904520.4686*1460545207', '79523832.1802*1459386771', '78838933.2346*1485132810', '29868183.1073*1471195280', '8303512.22548*1480945619', '5058469.16514*1455922050', '62207621.4091*1480289842', '62618888.7134*1472781417', '69041806.8527*1457747972', '91618486.2284*1464336822', '9732719.52478*1453912917', '2500118.28603*1461958485', '42211982.3416*1471818290', '80045629.3178*1469075370', '2477555.35602*1467397899', '25980622.4785*1477656371', '55168300.136*1479629063', '53397111.6093*1468808807', '45362502.5358*1474014172', '979989.676716*1457370855', '7654378.67705*1462085778', '37203265.6864*1454118870', '90221720.3513*1476642234', '49172347.1817*1487502516', '65570622.3984*1466503129', '55850476.2046*1481727623', '376484.06604*1456075049', '45042294.2298*1469044780', '28570179.7281*1470799684', '41522398.5766*1460297005', '27080607.5995*1477918033', '10190143.3769*1467787430', '83700933.2769*1472177399', '84502912.1958*1460613862', '99449842.0648*1474632127', '54060711.2933*1464485697', '87914360.012*1478091117', '35042001.8971*1462668333', '5961540.61565*1461002487', '72136355.4719*1464723600', '10659621.3393*1465987726', '53594289.8827*1462276549', '95646273.3507*1483211638', '16351061.11*1467237713', '38355271.8298*1483289369', '96150799.8832*1456358599', '81212882.0561*1467337806', '40582605.9598*1454577825', '92843757.0934*1460409168', '6994851.11517*1465786054', '48176555.0657*1455009405', '11828298.3128*1455240825', '57550958.1485*1484890230', '67665665.78*1480657958', '14798632.4394*1478613090', '88478991.8584*1454287732', '99688407.6192*1470512483', '33126479.3581*1461641602', '30757071.7211*1480039622', '68880230.209*1464444579', '66705576.3294*1452394703', '54565070.4088*1484297659', '66209247.325*1456084286', '93606424.2766*1476237351', '13439760.2845*1487035301', '81581260.1408*1463501494', '391010.425267*1478047346', '30778535.3319*1475389646', '42833562.6594*1455422074', '3617968.66062*1465742926', '94202624.0606*1487940605', '95365216.5803*1458992428', '51054150.179*1483542946', '38860941.1496*1461335207', '29187667.8214*1456742361', '64824550.6985*1459249246', '20373535.1657*1467747443', '87454284.1735*1467165688', '20826022.9502*1480294867', '18226853.6629*1460937470', '57321780.8066*1458185526', '86466638.9405*1472010601', '41446824.8386*1471062600', '87673157.045*1484325805', '17529285.6082*1455208784', '39887858.0844*1472774990', '95539075.4215*1452175467', '67207894.7714*1463018388', '25219199.805*1467657326', '44171783.5213*1462910579', '14575626.5153*1482988793', '98977039.6314*1487755781', '72731595.4355*1468994757', '93633901.3213*1484273502', '77721639.1145*1486676799', '34688926.1258*1485665134', '80031107.754*1463262636', '87970882.2185*1465616515', '83607768.4712*1463512313', '71615142.8496*1458664283', '31835795.5611*1471120354', '31366501.7944*1483341900', '28948892.1527*1480713090', '49384247.9667*1479070630', '75209591.9972*1474867915', '38187474.7034*1468055758', '99144499.4301*1459100651', '69392853.5791*1472604281', '95969316.913*1475616618', '28680789.9015*1458894412', '7554434.97837*1480840485', '66951686.004*1461887466', '42814201.0101*1472411160', '87757946.8006*1474460213', '96950148.0491*1452727180', '41797875.9788*1454843655', '25925607.9999*1477211588', '37467299.0338*1477397664', '74442056.6778*1487670782', '63155626.4101*1462401003', '87585149.3396*1466562365', '74108055.8004*1465093811', '74652541.019*1463900565', '35254161.9929*1464791108', '43686545.0916*1475438545', '35254161.9929*1464791108', '14055873.7052*1459676512', '86824545.8863*1458053458', '5331163.89834*1458196268', '60454836.0738*1456976778', '31760400.8755*1477374547', '51659158.1397*1478387923', '88116451.5743*1456034247', '91176754.7286*1467494356', '64568943.9757*1455191444', '46975494.5064*1451947798', '84676770.5928*1465086932', '4448527.61292*1459408481', '94090505.8544*1477841138', '24138705.3756*1469342830', '33126479.3581*1461641602', '56918067.5109*1463825009', '50883437.2416*1472980006', '29206219.8568*1460204758', '97642327.3292*1474938968', '38129501.4768*1475304347', '32606683.9395*1461223310', '2910716.66624*1477446575', '57812397.5794*1482037167', '53482690.2622*1462928949', '90158788.0554*1482825673', '98785792.988*1485377691', '15896571.8573*1478822883', '94090074.8855*1469040783', '74449854.5779*1455989104', '13256178.5435*1472761952', '55434463.0436*1480612184', '26197890.4985*1466181736', '15598373.1316*1477818115', '72784209.6268*1453631592', '75620811.6264*1487604020', '80114799.4236*1459419493', '22584815.6107*1462011086', '36437890.0243*1481944496', '55695544.5575*1468653925', '16286492.4625*1475776677', '91003090.6114*1478010438', '20374163.1335*1485361372', '82188897.8325*1464110344', '30785508.2772*1452968455', '66908951.6802*1461176895', '83280110.993*1454482936', '92631508.3295*1483085803', '61539630.8801*1457786518', '82145225.8064*1488294907', '61600056.883*1459069264', '41791311.4333*1458624241', '18393630.9454*1482111604', '53775179.152*1460140610', '19864385.7165*1487922432', '2344304.90603*1487728355', '43652207.3481*1483233191', '1100981.62472*1476063173', '16378232.4717*1485967422', '58756189.3337*1480733421', '55702038.6737*1455142281', '68366268.9746*1479086933', '54902465.4619*1461401870', '28662922.4935*1463559935', '93295907.3327*1474697295', '16950018.4911*1487694175', '113609.418753*1466933721', '23716893.1946*1459507985', '32764877.3682*1465308903', '7626398.37981*1458631016', '93892116.2176*1452648070', '63081929.4937*1454194737', '19944187.1282*1468796505', '89863797.4299*1480453831', '62188081.667*1461938679', '14250433.838*1477404899', '68459487.1115*1460349200', '4398737.50649*1463757050', '59717916.8888*1480135468', '53174746.3855*1471480174', '55767529.8139*1458183851', '90251522.0652*1451656320', '76154642.8273*1487818374', '430743.747916*1478101007', '64504936.9712*1486630350', '30130535.8995*1481332470', '99751178.2301*1476933990', '72068911.655*1460698066', '87234736.4759*1465080690', '73449268.3109*1458549363', '54442582.8543*1452604122', '32048710.712*1487543037', '34991440.0276*1487659583', '88418103.8682*1482627083', '24700482.0778*1464232623', '30331074.0239*1453025083', '48817282.175*1462111958', '234877.797818*1456952139', '95689803.9734*1469810113', '52694927.0558*1460698353', '84726620.765*1461145974', '16078184.0977*1480271665', '38128064.9634*1472651365', '49324448.862*1472208688', '56378479.8906*1471425942', '41285270.023*1468644483', '81390244.1721*1458826916', '18447865.1886*1455665402', '80586195.4781*1452917783', '97431441.7024*1468460809', '15155343.2548*1460138611', '46235293.2842*1456846084', '63880486.6117*1483644426', '4040417.4531*1463471190', '16040028.0298*1464150151', '23575499.3653*1471153964', '96336784.7916*1472519446', '19458625.885*1454694213', '22209332.2318*1471018794', '90563723.335*1467104105', '71859685.9914*1482428479', '35554799.6621*1460520460', '49605867.8351*1487365740', '24793625.3021*1484781711', '81909613.092*1467829609', '250249.378079*1470247804', '59735877.6041*1478265450', '47746049.2571*1483286160', '94250126.3883*1453103934', '17818176.0347*1465795825', '33485793.9602*1453629550', '26872436.575*1457359893', '29195970.3285*1475937481', '95860862.1867*1463564700', '32440576.4471*1453427992', '86271163.9406*1474420426', '38543466.4294*1475589258', '46506049.0241*1456794960', '44870683.1762*1487681453', '68575992.1145*1465849725', '56397606.9756*1457352186', '96748154.6527*1456366084', '51547842.9063*1468719495', '33532047.0646*1467473257', '84977563.7086*1476711830', '40913816.3809*1456168944', '10388420.8402*1468136765', '77820249.3675*1472549963', '95800558.1113*1454193444', '26866226.8055*1455475665', '37634468.2137*1466647226', '29099143.218*1476677847', '15576025.6484*1481176933', '50337006.8329*1453860780', '46128230.3335*1454930090', '72733455.4577*1465936705', '30892434.1998*1455053009', '21009385.7307*1480462336', '42216971.932*1481233819', '44788546.661*1485962144', '8721542.25757*1458339046', '18948126.5753*1460196933', '18891475.4167*1465273758', '1785326.45646*1451687279', '85375910.4934*1463214331', '76724620.2439*1465146307', '14210741.9643*1454518723', '32153194.883*1453479488', '17053774.9279*1484555386', '46239780.136*1470556661', '22814499.4591*1473169342', '54759181.2789*1459653585', '78893492.4727*1452282201', '89525055.1045*1451714043', '55238464.6865*1478945537', '95800558.1113*1454193444', '47540589.7512*1459894383', '11597429.8116*1461698419', '39084444.1446*1459449075', '62564140.8899*1464604014', '18076061.3403*1486660994', '13299106.7607*1481317148', '46011777.507*1453679448', '36448370.6958*1470347962', '16479688.0125*1457013670', '36412208.0044*1470378213', '69933057.0275*1464150149', '84024566.9128*1476022204', '2458364.77027*1464606992', '49895310.7845*1457386437', '37353248.4247*1460353341', '69440790.6041*1470679262', '91833097.4955*1477928033', '96646083.6183*1479427490', '59465541.6683*1472018269', '33076876.5959*1460187328', '577807.846778*1484599672', '33885714.1627*1476802829', '13898208.6027*1483835663', '2199518.77127*1476005244', '63931541.2393*1485831844', '47659872.278*1462170302', '10957384.5896*1478295247', '80906210.0772*1475204260', '43374455.6477*1483276899', '56484536.5529*1467143976', '98785792.988*1485377691', '58389438.0534*1477024568', '85958842.3213*1452755328', '51093566.6713*1475949489', '98663046.7923*1488146738', '36215878.1123*1476364709', '53311750.6415*1463165993', '39147291.8496*1455819862', '96338883.127*1467115887', '50395778.7202*1486121709', '98394015.782*1475125319', '72821846.3687*1473425727', '27504450.9355*1463977294', '58892049.3555*1480981817', '92829966.9273*1456414509', '372025.228087*1482265166', '77654855.9904*1459657620', '4006270.02716*1479287795', '45923512.7294*1472155074', '42025696.9398*1452140012', '77575731.1964*1483750216', '16393464.3139*1463963800', '49395328.1539*1461862978', '77861700.7206*1469122248', '87496578.8698*1482418183', '92633328.4409*1461990798', '97424325.7662*1468883686', '51502956.5587*1456196338', '65390197.864*1460928483', '60445280.9408*1455168260', '2100825.19281*1464753577', '14604425.2269*1455165246', '57496823.1085*1483221382', '34648240.8914*1469794798', '84916466.7936*1466234067', '41596983.439*1464397334', '40328172.1781*1453977919', '81479577.2583*1457569913', '17342809.3475*1472549228', '74442056.6778*1487670782', '74245522.5864*1468974940', '21098047.8707*1474273337', '53790071.908*1470236522', '4565429.52615*1486455275', '24555482.2938*1470763368', '12181688.1584*1454140185', '57057881.6987*1467151751', '22681432.3639*1456932019', '27751726.5672*1455376717', '5716025.99608*1454272886', '88169153.3829*1463269853', '75939834.3654*1466332912', '53506855.994*1472591183', '69941559.6209*1482982241', '38683649.7645*1477189363', '30258116.8453*1457753862', '93683387.2352*1482083608', '67222929.4758*1457842306', '84057685.3721*1466335431', '38683649.7645*1477189363', '75159517.8445*1474528207', '89525055.1045*1451714043', '25054456.8884*1484991777', '77070217.6112*1467713068', '47589665.0758*1478203714', '89446457.1859*1466954989', '7889356.0429*1458460270', '54765591.5953*1452010591', '17191435.568*1451763638', '55023582.6886*1453390947', '47054711.9801*1456729774', '20381306.5741*1473183959', '4829411.65399*1481661548', '78673383.2531*1485398218', '30125234.8279*1469156433', '53629739.4265*1471846695', '96070558.8044*1484221767', '35171109.2715*1473153365', '86098410.896*1486598989', '38787515.8599*1467463553', '50526779.0658*1456185892', '9689650.27474*1462041061', '90289300.8544*1470626814', '50480306.9704*1453420059', '33652205.2631*1485124072', '39664038.1513*1469051591', '92117404.6677*1483182264', '52501894.4029*1465339603', '37568712.195*1471913897', '29236188.681*1473941822', '61005337.9686*1472216669', '31173160.4184*1460543490', '27416071.9935*1482331073', '82455232.8098*1482959404', '69174685.7137*1480502534', '43095648.8786*1477682484', '84639957.0473*1478279161', '71886237.924*1482562775', '18543535.3848*1454059035', '40523470.0659*1479690036', '88728900.1108*1479401954', '15323275.6307*1475062797', '29195970.3285*1475937481', '81779266.584*1479149677', '85809851.0905*1474132289', '80419186.2209*1487271551', '7783910.56224*1475580754', '81909613.092*1467829609', '44830187.1691*1452909057', '33277842.2887*1477374676', '2619854.75895*1455601472', '13065268.0585*1455568641', '80407745.4397*1477938803', '82827889.2413*1467981874', '62546980.0255*1468097172', '23005968.8485*1488315513', '16160536.4384*1484362925', '19086527.3692*1455560659', '51827805.1597*1458065987', '87696768.5818*1485128824', '49378028.4254*1477684899', '7098238.89263*1465856667', '18590863.7128*1476917684', '50731375.2088*1475174877', '24693612.4425*1452865700', '8305729.23174*1453103020', '99428882.4826*1482072074', '97542257.4641*1478463401', '67009183.5667*1484792554', '6167160.95437*1472062498', '44221126.9469*1462156032', '79296893.1702*1481063214', '15379669.1006*1472426619', '50903384.7807*1481932993', '80865533.2865*1481250242', '75418588.9652*1473456449', '17136968.1802*1451697649', '79551550.5303*1460995145', '58082800.4614*1480675497', '57996990.5525*1478107705', '77302896.8165*1458543585', '90094370.1484*1454557491', '94970289.7693*1480337951', '69440790.6041*1470679262', '45536795.9771*1476517885', '54892590.9119*1486307058', '84301660.9037*1474800592', '31659127.691*1456669739', '106305.036673*1469427392', '7769047.05538*1456179693', '92094882.7712*1467480272', '79370224.8093*1480982472', '98582719.9004*1465637699', '11453216.7665*1467346110', '61553662.2841*1460324653', '41264281.7684*1455676185', '31428583.1239*1457151062', '41055296.7787*1457359735', '25292865.2606*1481287810', '46929741.3442*1465447137', '45867509.6769*1457412365', '10148126.7865*1479969451', '45923512.7294*1472155074', '42536628.6442*1461465251', '11379084.5094*1455393975', '15379669.1006*1472426619', '66237066.1975*1482285190', '91748152.8092*1465626316', '6342851.34717*1456231595', '88325652.9759*1482749229', '74881116.3981*1483138205', '24205868.696*1455882097', '13351441.7798*1460019039', '34560668.6119*1479203847', '44155936.2615*1473020118', '55434463.0436*1480612184', '89200633.7529*1462727976', '20120187.8479*1456397849', '28828952.7547*1458239598', '40974220.4788*1462930618', '4143811.66611*1456835097', '52357914.2009*1454142069', '43386544.7408*1467211156', '51054270.8063*1471567235', '93649137.9696*1459001263', '59941656.0672*1464431852', '45645296.683*1484263075', '61079103.7698*1466968473', '79832188.7915*1453332112', '72264109.8265*1451623350', '26623771.9576*1465835684', '11337904.951*1481139265', '69376706.4977*1464684937', '96800465.9139*1458235126', '30384598.8361*1464386012', '96159734.1342*1468049263', '91386529.4935*1481730250', '99353499.8213*1469325649', '31673452.1173*1475507466', '75012552.0581*1456290271', '69918709.5168*1459756566', '22050435.6685*1485348917', '35172030.968*1468315293', '11668233.2883*1462306541', '9685906.53547*1480670841', '73104832.7631*1461386614', '7493565.79335*1467093514', '20858706.7145*1460431834', '28761085.5652*1464263098', '18039600.893*1483965834', '50140561.2575*1455148304', '79136577.5331*1452219471', '2800663.22775*1476463758', '86098410.896*1486598989', '70313844.8089*1486697368', '13352234.4522*1458434775', '32870410.1805*1487599699', '15799515.2622*1458660491', '68373489.34*1454232840', '76724620.2439*1465146307', '97523974.0719*1470133803', '71044270.0696*1486476520', '63212176.1639*1466889394', '67465401.6127*1464687961', '64167439.6641*1477604320', '74001168.0221*1481707161', '57623428.647*1478190567', '5511381.61096*1487326543', '29030500.4251*1465572790', '890790.067644*1477348152', '10410263.1336*1485440375', '81507671.8949*1463051693', '68286350.0245*1470049483', '22021841.1999*1464179294', '135437.229714*1468423999', '40507076.0078*1454083118', '46473256.8649*1458537387', '97344872.5725*1466076729', '60629312.0959*1481489311', '20265662.6637*1454253916', '54435236.0251*1485492231', '77502907.3591*1468765411', '82827889.2413*1467981874', '55037024.3214*1476568422', '41557549.8871*1468311941', '58007872.8565*1467674568', '19374549.7634*1457167098', '82318948.1944*1461756187', '66075786.9071*1459163721', '73314012.6997*1473634263', '64200252.5463*1478134375', '54541770.3366*1483624824', '82198921.401*1469548175', '4432651.64894*1457405839', '35448748.3015*1471496234', '40628876.9285*1483951242', '16656992.1129*1475356985', '30903941.8826*1484790228', '40549416.3033*1461314490', '79183528.8396*1454253293', '52158458.0582*1484026837', '67438491.1053*1484169969', '39830016.1238*1478561880', '12480282.0233*1464169904', '25079544.2871*1460441625', '50968373.1*1473159829', '8712580.10005*1462667403', '52749870.6402*1478831086', '17575914.2154*1463721299', '70603879.0336*1458816405', '65251854.9104*1485688176', '24149708.1497*1454815321', '261373.920703*1470009401', '38087753.3899*1477111612', '38083500.2509*1488183475', '26573458.3952*1483662118', '88175468.8124*1476044709', '78396316.7779*1483586745', '91630813.6837*1483150748', '23009250.2923*1478718238', '36094362.0786*1456271288', '21778342.7367*1473745314', '65494752.6908*1481278603', '32417901.1231*1465221086', '62404826.346*1452837255', '55273155.678*1478473945', '5520862.00177*1471795249', '89669411.1102*1451625058', '98652146.2255*1455026840', '83068653.5222*1467807964', '12628596.0151*1478179698', '75352384.6262*1483028967', '16420945.0035*1460253323', '18638617.2918*1483171159', '93992810.4611*1463690303', '96875293.1332*1486231167', '96929470.9911*1472523561', '2452353.592*1453273281', '8834798.40397*1484181511', '65773123.7855*1460362291', '24211827.5087*1463055446', '66295934.4287*1456787433', '84709609.6207*1487658331', '68772700.1787*1451986832', '89044016.2974*1465420537', '72733455.4577*1465936705', '63204420.3248*1461770562', '61984091.027*1470977070', '29034977.9841*1473902251', '54892590.9119*1486307058', '79548973.0369*1463970024', '46153289.102*1455432070', '39372147.1568*1470537888', '34930776.5862*1477964708', '63073202.928*1460228903', '41923565.1272*1457347854', '79192910.8061*1454229565', '67733616.704*1463099440', '58543111.698*1468951769', '50136923.9938*1480985415', '60502403.2895*1471512419', '40533338.5811*1460612549', '12291148.2056*1453794435', '53444469.8409*1456551086', '53362447.9035*1462111884', '4816464.5569*1487079762', '40944404.7957*1480064490', '89640657.1176*1461397871', '23414881.962*1488031693', '80187477.0184*1455061213', '81507671.8949*1463051693', '92099009.3327*1458904883', '90424441.1446*1469021303', '33032362.2348*1483381178', '8159296.17321*1483015754', '67333029.4611*1456616765', '23699812.8185*1484283567', '32405014.1491*1483948730', '32302186.218*1463420788', '18727597.9362*1476714513', '44772575.4903*1458510507', '78445983.6878*1461486147', '35817984.2016*1468278403', '4135130.47418*1466473272', '87914360.012*1478091117', '52603717.7102*1463036487', '9922787.19227*1460152703', '76134154.2129*1482240083', '42487558.0839*1471628037', '75303374.4817*1470009634', '27023358.4204*1472542451', '52577543.3193*1465479522', '15091508.7937*1460087243', '73509230.5583*1454131152', '46395152.1128*1464296678', '41957404.6129*1474998403', '91278269.627*1469487289', '76251681.7282*1473841236', '52598529.9347*1455632296', '64250912.2784*1462993430', '85669012.8056*1464833181', '37500119.1328*1455531621', '50518745.8366*1453701652', '98189956.619*1463805405', '43799373.1838*1472831721', '12798429.0409*1475059924', '94619215.7129*1478516250', '71997782.8005*1473322569', '43271523.2817*1460130671', '14542122.4351*1470062246', '43902157.9446*1477079136', '28603392.642*1476664006', '54455212.8147*1467847757', '66733373.4073*1468060836', '19090910.7189*1479366260', '79258722.4237*1479937899', '63383817.9947*1452618393', '999326.417049*1486427867', '17530344.9539*1487849816', '82808910.6338*1472591222', '63410553.1018*1477664883', '89372586.3317*1487605614', '52817404.3192*1459363878', '31598504.6537*1479868080', '74140714.3052*1487114692', '27251648.7792*1452567830', '36624835.3196*1485309068', '81507671.8949*1463051693', '58054639.1861*1484662929', '76143608.7713*1453507345', '59408433.5048*1465999176', '33845738.7862*1486410287', '53117043.4685*1481643167', '16873843.1182*1457075383', '25487121.8686*1459714071', '81477926.7867*1473237412', '11637233.327*1461318124', '16478948.6941*1462025349', '66058534.2282*1478519794', '68049594.1651*1462696179', '57270015.7757*1474898079', '43108455.8371*1476453778', '2076685.19898*1484101503', '91454093.0353*1479022012', '86672020.8895*1462489035', '90728509.1167*1464211384', '19855341.2896*1468460513', '42065696.197*1465764191', '67317155.4321*1451614183', '35023850.9633*1483791040', '21582268.8849*1471402615', '41959903.9304*1455237667', '94012121.977*1479616945', '40248709.8606*1459307565', '80183923.7654*1461843623', '67081582.8975*1459616539', '22085237.583*1482210470', '34715630.2208*1470423797', '49822745.9455*1463553054', '72352730.1221*1476572881', '51987541.052*1479245744', '10238665.9062*1463722533', '48307388.7811*1458019989', '47355856.3041*1479515537', '56916497.2261*1485951456', '29187652.6009*1470785443', '25117306.8201*1455116091', '98214229.0173*1456269742', '29456781.3491*1485124025', '92926778.5266*1455765648', '56916497.2261*1485951456', '71115799.7225*1467438290', '2481673.54111*1474710237', '76269481.7453*1452142949', '48680424.2221*1456672387', '26437305.4177*1479939874', '89540227.0964*1456264069', '83873882.1942*1480618639', '960806.454337*1455575812', '77662279.6552*1459862094', '20870479.4729*1463654391', '65075305.1769*1480169582', '96002062.2015*1470025692', '53070746.7973*1481025861', '89842442.8521*1472307251', '41525230.2982*1466358268', '82576397.3*1487734413', '75963407.5951*1453212741', '28933663.5341*1472277075', '70065803.4736*1451813728', '16420945.0035*1460253323', '94646569.2956*1454781463', '32918131.8498*1486733141', '69795224.344*1464679990', '87678457.4644*1472584971', '87954706.0183*1463098826', '40856996.6974*1474984866', '23872365.8768*1483827184', '89258489.795*1471398462', '23408131.9716*1454007716', '56835474.8566*1459178076', '47678782.5555*1456486196', '92157440.6298*1455063908', '93374682.427*1487884387', '13045168.4609*1454082444', '25929700.1042*1464203455', '14872488.1535*1460948113', '8547195.52857*1467657922', '23005968.8485*1488315513', '24852564.628*1480489668', '8112255.66735*1464570677', '65464061.5569*1461837730', '75159517.8445*1474528207', '60101650.3446*1486561877', '52162089.6102*1473831707', '44647164.9101*1454016298', '73512593.1274*1466850127', '31920448.9131*1462693370', '18039600.893*1483965834', '37999953.8672*1483472562', '70383142.36*1476829288', '31515547.0911*1482188645', '3415742.54978*1457131177', '56186144.227*1481765992', '50008324.7711*1487871526', '14250433.838*1477404899', '77756172.6834*1478092129', '76143608.7713*1453507345', '71520602.3013*1478430725', '52292487.433*1476396710', '46082740.3659*1486166613', '49441705.774*1478403373', '16587148.6344*1462617888', '27928366.2241*1484472696', '67938111.9747*1481312848', '16860716.2292*1471554105', '44149043.3328*1474944730', '69145972.3156*1469576718', '80512375.6817*1460002480', '17494498.7696*1463172567', '7297720.84102*1486494234', '1095478.23807*1480615067', '51447306.9966*1488118281', '68113475.2561*1459826717', '67077052.1562*1479244225', '84640876.7094*1470693697', '84950578.1592*1479150644', '3691567.6641*1467880453', '69959039.0878*1459145455', '38341204.1313*1472117190', '82477982.7055*1459213833', '73449452.789*1487100226', '88800619.5707*1474029032', '83370487.6525*1470928754', '41508530.1892*1481068517', '84247821.7006*1460650230', '77861700.7206*1469122248', '33028392.7724*1470219653', '57024467.5561*1462128933', '51565722.3134*1487940164', '43441577.7298*1485095680', '16056823.2382*1483648659', '93716648.6438*1487581696', '32333053.1142*1485189172', '29453082.798*1454708827', '83196434.5434*1458295554', '93295907.3327*1474697295', '53082033.8724*1462338390', '50528473.627*1483147340', '38597885.2249*1482997531', '63533.0879987*1477733736', '64851585.4811*1466638115', '16950018.4911*1487694175', '72961961.7341*1469510173', '26477670.7428*1465729649', '52362667.2616*1485162481', '89164723.1358*1463363680', '60175526.7865*1456599986', '53425753.468*1459124234', '91545230.2289*1451646892', '76020997.3109*1457037310', '81479577.2583*1457569913', '80867877.5995*1457866259', '70370304.9397*1477478427', '8401090.87211*1454529324', '60346073.7406*1478992492', '2953947.4491*1476234267', '11984482.2681*1477957353', '59769486.4354*1476405687', '50125846.2902*1485220778', '746308.657928*1483990167', '73361705.7781*1466499737', '84445397.2211*1470238946', '43728704.9148*1453517484', '78168762.7045*1465967414', '36206435.5095*1478357746', '55135637.1353*1480131541', '95807529.0008*1477035880', '85777067.6394*1469363311', '67866596.6463*1476574272', '73042882.0856*1464888282', '92252088.2137*1486896886', '1208707.64759*1467605644', '97957476.7472*1484104147', '94447545.0917*1482226048', '99213706.3456*1452343556', '65277439.5094*1486586867', '15258227.2639*1461124077', '77118317.1065*1478941079', '39263139.6697*1478368608', '83873882.1942*1480618639', '72068911.655*1460698066', '29381739.5187*1459419476', '26496100.615*1464837393', '20420063.7434*1485838837', '39323094.9643*1456610511', '27751726.5672*1455376717', '8943254.85235*1486602269', '27080607.5995*1477918033', '69456759.7701*1480421322', '24521539.314*1465695904', '10402653.3965*1482404482', '76661004.0074*1456509414', '55700809.3233*1472742490', '37735313.901*1482206548', '30641498.6021*1485425870', '75357890.7976*1456288436', '69428425.2112*1485324800', '76693516.7202*1468360026', '57719010.0224*1463303122', '51251603.8892*1463625173', '14684465.5513*1458611875', '26083885.3505*1480455767', '67715432.8614*1457208464', '65177558.2274*1461063265', '26871653.7299*1460636111', '73038389.1455*1451657744', '9706347.67487*1456437330', '38932647.824*1486688572', '33651115.6216*1467723817', '80506496.279*1453298606', '47413695.8166*1487740810', '73413934.4895*1473269183', '68612687.3145*1468890786', '69711464.7626*1466065296', '21316181.3804*1479078418', '45362502.5358*1474014172', '13185990.2018*1474173098', '2161691.70051*1470529849', '95793753.5739*1459930118', '6586499.1257*1479076145', '3586201.163*1473526705', '57015951.4785*1467536479', '87954706.0183*1463098826', '97786823.1397*1472963864', '14873894.8426*1482056186', '37610409.2895*1484657062', '18815006.6069*1473681219', '61358032.4699*1478327486', '91633272.358*1460267142', '63504356.4222*1483295223', '69430464.6731*1482306040', '83607768.4712*1463512313', '56435843.2432*1487427072', '54693989.8145*1465936332', '526223.131771*1470564684', '54276697.0743*1485717540', '10557922.4026*1473861890', '22570694.5192*1477913894', '35527599.3739*1466093500', '77705469.6582*1469165897', '67834526.2523*1451684342', '11265868.683*1467766007', '20008290.8822*1487828972', '66714505.9454*1463895517', '33783128.9868*1487974102', '27622505.914*1486885865', '39007802.693*1470642767', '40896050.4711*1462479999', '70433008.1355*1474924757', '65109648.6329*1451972489', '85327017.8509*1457377288', '7218659.0564*1472405189', '39419717.9403*1457863269', '64218945.7397*1467298131', '92816253.4812*1484821724', '16488414.1686*1481790426', '21534912.6998*1468105565', '35864577.5305*1473568251', '54177441.0525*1463115990', '85189386.5137*1465248810', '15858605.4005*1458728175', '95315762.6604*1453899785', '9780636.49418*1468156206', '98212139.0125*1461872958', '96660095.1832*1467573961', '56496349.4684*1485310854', '28163068.086*1461056062', '78398210.8703*1472612462', '51246774.1791*1455436764', '2257394.49903*1464744879', '45611166.9932*1466774039', '26170935.5278*1462713368', '31964337.6972*1463519243', '30182473.6313*1481161428', '57010375.123*1471016537', '44453347.9476*1464413990', '38982600.6241*1476128457', '45878534.798*1478583939', '83761257.0233*1455634429', '74067608.0729*1474489432', '26977534.9511*1466718775', '69070250.8578*1486425650', '88677148.6882*1459143162', '28654495.1478*1474638831', '82346493.9102*1476297236', '42893203.4969*1461079346', '99006872.4439*1471410637', '80619400.461*1477781532', '26071696.6809*1454808224', '4209132.90087*1485643310', '98110716.9312*1465225661', '42850893.407*1453918507', '35438201.9981*1453297000', '6615553.58057*1461877639', '85319479.7175*1477437329', '50833564.6885*1457760321', '29527358.3119*1471319174', '29877750.7501*1466445364', '47355856.3041*1479515537', '28680219.2034*1456228646', '26600710.6909*1464540588', '58135160.0093*1456093211', '64653117.5492*1470270544', '64657142.258*1451897630', '23835655.8979*1474528264', '21534912.6998*1468105565', '76140570.2824*1464593209', '34588983.1857*1485818908', '12922012.1094*1467608845', '20168799.4427*1472701619', '95330342.0981*1478147432', '82608814.3287*1473096725', '36118582.7874*1469903839', '44772575.4903*1458510507', '76006518.4865*1483042546', '78922767.2587*1485667155', '74944003.6708*1471375318', '11401275.0474*1479990040', '4710180.765*1467236730', '92067770.0756*1479143874', '5402308.57833*1468168585', '42915404.6248*1466730208', '23835259.9241*1467692082', '82846140.7861*1459535669', '37439923.3987*1476153002', '74873105.4572*1466666337', '1831966.98872*1468200310', '63664747.766*1462070113', '84316049.6456*1470508096', '73361705.7781*1466499737', '98883778.2248*1477424812', '96417467.1092*1483999708', '23570174.4826*1485717124', '54251474.4989*1475111063', '57156452.0084*1466451361', '14814244.1324*1470493409', '24567182.2439*1474758670', '8530512.89094*1478586028', '58087569.0054*1479605005', '37174026.8509*1464377667', '43294037.6688*1464400677', '431650.545298*1479012195', '78028375.8487*1456440893', '19878684.7384*1472067351', '31705922.2482*1475207693', '81464114.0168*1454591444', '44626914.9263*1472767056', '56952253.5008*1465221631', '64646543.9949*1482439322', '34957112.8849*1460902548', '53677236.4146*1482043202', '85850085.9085*1469074102', '32494220.915*1473179288', '20108183.2465*1482530694', '39831572.4699*1462595594', '28222504.2195*1454710290', '26071696.6809*1454808224', '83173851.4915*1474596388', '30641498.6021*1485425870', '31354239.6172*1470566260', '19378740.1554*1453305248', '49050398.3611*1474347203', '68797564.0381*1470275318', '33002083.7483*1451837092', '76331160.451*1459533105', '37501232.6044*1472253643', '786679.375876*1455453508', '82486733.3197*1465842707', '89258489.795*1471398462', '65094945.2966*1472111541', '37195928.5741*1465985955', '30808254.5092*1481652222', '84323291.5266*1454751380', '96389904.1675*1480688398', '22975610.1932*1461504780', '85950808.548*1454179603', '23190721.7352*1478092957', '48321975.0529*1465436073', '95490808.9501*1486133865', '87626440.4947*1454603259', '67559040.2854*1476855810', '44460934.789*1459775304', '65119152.9909*1470868847', '45781720.2155*1484074633', '8880467.03824*1474659550', '52350718.5105*1470353331', '93820344.5575*1452041765', '3340441.41332*1462699158', '27397597.5564*1468544171', '23835655.8979*1474528264', '46846758.3424*1477208853', '96546437.6836*1487966528', '56570100.4696*1485201743', '3070874.08233*1454275613', '70669649.6657*1471886048', '28199718.4235*1461750749', '35899556.4609*1463755986', '81782590.2431*1487499866', '51437852.5062*1453353213', '42575664.566*1468081706', '97824107.6899*1454617449', '43936787.9468*1481010748', '15533563.9434*1473892509', '87757946.8006*1474460213', '96019634.0437*1465969167', '17229648.272*1467800837', '33144333.9198*1468416721', '85827361.7266*1454966454', '44123737.3285*1457455848', '7831448.70803*1474018734', '57898573.7313*1484329887', '43084217.3427*1475708832', '78019796.9936*1475761391', '63510061.4099*1472089377', '43863210.2626*1456287900', '58877894.6661*1479867547', '56045803.6089*1487588543', '93623698.2126*1473182148', '47734805.4193*1453727410', '57209352.9229*1460151695', '60685680.2358*1468818428', '42211982.3416*1471818290', '99426768.8965*1458295437', '57433280.3225*1468402935', '17336652.5151*1456887144', '85669012.8056*1464833181', '78408857.2314*1476799235', '63199587.8429*1472322032', '18017500.0526*1470645188', '6862858.52187*1478356003', '49606550.3411*1456223773', '68355241.1866*1465384724', '90694173.13*1462255132', '76834800.4869*1477562953', '84298255.5629*1465850078', '95164413.8165*1488146854', '92436790.5754*1467962861', '31529887.4948*1482639854', '24665544.0282*1462446261', '46466297.6564*1464094076', '81410765.9244*1461831815', '71408613.1391*1460282414', '58032188.9836*1453895459', '37805908.415*1462955656', '8534328.78767*1456356935', '59721738.9463*1452665946', '123856.731363*1487912948', '32543114.1309*1464208480', '51094203.3323*1484867840', '99926820.533*1461910420', '6778164.99582*1463939659', '42225805.2145*1483402427', '89035297.4681*1474989384', '83007818.3184*1475054731', '42656361.6612*1470111782', '89574064.9648*1477299557', '1788655.89892*1466622705', '16587148.6344*1462617888', '84498118.9018*1452965865', '32447604.6158*1470421977', '90798231.1087*1475865470', '99150597.5808*1475444036', '96875293.1332*1486231167', '45090618.5572*1468151560', '50585167.4498*1456884796', '43299700.0651*1471782750', '31012985.594*1478357227', '99718150.1354*1463354901', '64954912.5587*1467483480', '62638249.9071*1466948404', '96374050.8206*1478752055', '93217619.7092*1484595730', '25115257.3353*1466221684', '18447865.1886*1455665402', '84775416.5394*1475774471', '42872676.8404*1461786443', '63846269.2245*1472389759', '69074102.4587*1483629383', '34320919.1499*1484245117', '34654522.1834*1463971628', '7136963.12315*1461117910', '61774549.0542*1485119692', '847048.698649*1478321187', '29463986.383*1457424725', '46734717.191*1461016480', '43184193.88*1472034401', '10443948.2859*1461581220', '35793494.2762*1454024678', '44094180.1082*1480212333', '86061038.234*1457838188', '26007945.8249*1473454468', '97811158.7984*1479849263', '34538153.4759*1485355964', '78713642.0183*1456408037', '69016503.9015*1479543640', '85977140.7472*1468036345', '71310720.7031*1467044098', '72750549.8391*1480195103', '4143811.66611*1456835097', '53445771.3525*1469858062', '15888773.9545*1477220516', '98924304.2606*1482819012', '8363405.50402*1485109294', '5388959.56857*1452939253', '35942685.5485*1482295243', '59818894.3568*1467232120', '38838528.2895*1476518634', '15286135.0854*1463278510', '26270751.324*1470908639', '1314505.85528*1467883843', '77618936.4936*1460842041', '66237066.1975*1482285190', '77431058.5994*1462896748', '39921657.9322*1486953369', '78575750.318*1481546863', '74612902.8948*1467570445', '91092044.9186*1485667426', '12820040.5433*1463771383', '2859760.54087*1487230867', '20511490.4061*1454356942', '83757738.6415*1459287627', '78617814.3777*1478839517', '14250433.838*1477404899', '51138981.796*1471474677', '83719480.0845*1477341785', '73105777.4913*1457957803', '36065449.6453*1480025182', '37011690.8418*1487560166', '54886050.5813*1454376407', '36260464.4731*1486341583', '55881394.1751*1461896003', '28692715.3816*1484859946', '21363097.3871*1483090011', '15462252.6514*1463377894', '5723689.62963*1486864833', '32009992.0216*1483159616', '53117552.5337*1482409439', '42380266.9498*1466629490', '81605702.8759*1466384720', '79842537.8573*1477629062', '47344535.9664*1473766712', '85272264.3126*1471640949', '75137472.1181*1470326622', '25797748.3005*1452677206', '70749969.9146*1463404971', '31086789.4135*1482676335', '39678532.4247*1463339472', '72327534.3714*1475177777', '55019811.6529*1479863578', '96327015.7411*1478166202', '59101672.7786*1460599904', '80648854.352*1479995668', '92465221.227*1465579185', '34188900.1903*1487003549', '93252791.8676*1459222319', '66712626.4033*1479730465', '65682604.0789*1477844230', '64071925.127*1487969349', '52787520.6045*1463858754', '92277728.1948*1469948456', '49409159.2346*1481980143', '639864.139323*1458159718', '54060711.2933*1464485697', '89075614.0427*1460440296', '57388968.468*1484253457', '27448208.8263*1458277134', '80836991.927*1470012664', '93665365.3013*1455322560', '42952132.1187*1473081961', '89505873.1232*1487343334', '3793669.13941*1464965095', '69374256.949*1481911514', '29568185.56*1454889017', '77504593.6646*1476567111', '61490160.7678*1476433083', '18891475.4167*1465273758', '90757243.8988*1462473378', '22835110.0577*1451910908', '21088702.7842*1452142954', '8707174.35515*1486583266', '43682637.6611*1469035953', '41973009.1739*1467006274', '66940652.683*1475459511', '4132816.39279*1486752990', '99792941.635*1475050191', '72148474.2275*1459417180', '57509492.3941*1456842380', '19144382.6417*1459309912', '41039630.9026*1478884407', '62065763.1429*1476277247', '88864850.2312*1453912891', '67319716.6374*1477260710', '21061628.5002*1485803845', '38962294.5261*1463672592', '40489961.7875*1476081337', '60122636.6771*1462332975', '273818.318591*1451710949', '13384155.7226*1470319346', '53238066.2164*1461516477', '15172774.4562*1478440209', '25115257.3353*1466221684', '46328555.1209*1465256763', '19354285.5973*1473604704', '30621743.0947*1463148023', '96545417.1131*1476041800', '84323291.5266*1454751380', '78317967.6858*1478019639', '64193975.3086*1461203057', '44751791.8417*1460605060', '8394995.70862*1462937136', '17763533.2503*1462441787', '93773830.068*1459207091', '22246456.923*1475276820', '15455621.698*1486226049', '3908894.72214*1466886648', '39298770.5392*1488214194', '32909470.4651*1466153017', '76570996.0553*1485943843', '30267746.1862*1457607792', '28585520.356*1470347623', '60068871.3833*1455446227', '67218155.6231*1475459910', '9655897.03525*1477337091', '64872881.9102*1486979981', '16320569.7129*1464732316', '67327055.2179*1454791437', '13730855.4481*1457065528', '2454584.38031*1487863655', '88310863.0798*1475820702', '77507301.0302*1456876285', '45874721.8437*1456472516', '7101551.31934*1471972850', '26521879.785*1478275253', '67072941.9909*1468423853', '31515547.0911*1482188645', '77002802.0319*1472250069', '71044270.0696*1486476520', '20335993.2148*1487022034', '45047037.8887*1486206672', '76500172.7073*1465834382', '70785099.01*1455171344', '45272131.1324*1479986056', '64468960.4179*1461962804', '6743543.15071*1459825417', '6887819.81127*1453482876', '97301569.1608*1485204516', '26007945.8249*1473454468', '80114799.4236*1459419493', '55997616.104*1467283468', '25107001.7012*1479412764', '13273355.3867*1482040247', '10761990.0508*1476357211', '99864340.94*1486324101', '41355257.4249*1476122333', '71387503.9726*1471261666', '67937780.0158*1453689175', '44449478.7873*1466639260', '78963182.8532*1454831361', '81091749.123*1481595789', '43081768.9162*1475416352', '2631646.01616*1476017034', '76629765.9812*1458558092', '24113745.6898*1474589939', '13015825.6997*1471087427', '93241578.0072*1476932646', '74067608.0729*1474489432', '86228734.6275*1470056208', '96493530.0062*1475639499', '41666768.4093*1473753969', '99482926.212*1463440328', '35606394.8194*1476935974', '46207264.6073*1475312218', '15209265.8675*1470921242', '24507233.449*1480345416', '34992543.6867*1463923947', '40774233.6309*1471265968', '75726255.947*1472472376', '31617131.2055*1475004112', '21271880.9127*1455241006', '75664282.2083*1468268706', '97252738.0237*1454234791', '81927554.506*1479161467', '11967809.8954*1481862316', '36094362.0786*1456271288', '55100492.9424*1475877035', '69857369.1607*1460940956', '95952187.3452*1476779810', '98592316.5386*1460305469', '59528508.3649*1457105355', '34050207.0849*1455948503', '99726497.8061*1457946056', '84383484.0676*1486469301', '27759079.1342*1459876146', '43879718.8293*1474017427', '23556802.1863*1453671943', '64140398.4437*1454500577', '31764018.1497*1467943151', '20358105.4583*1472400376', '534316.664734*1486302472', '99845807.67*1475337637', '11614589.8782*1483214021', '52895857.5865*1474195043', '49822745.9455*1463553054', '71470606.0458*1466826213', '35554799.6621*1460520460', '24045525.869*1480454787', '71372056.5108*1473249521', '71671342.2923*1477708328', '23463873.3936*1451798070', '8347768.76828*1482917547', '57915784.0915*1471247811', '85456547.5986*1486083849', '85290768.9448*1454373491', '90167656.7667*1483692105', '91405273.0369*1472512901', '10668302.1285*1479104192', '7500825.85954*1456496268', '20343620.4501*1487476702', '24075051.1592*1452799626', '14794618.9342*1480665171', '64854062.3459*1473050078', '75083968.6986*1483033720', '94202624.0606*1487940605', '64305559.8419*1454343810', '37439923.3987*1476153002', '50219093.3084*1472225333', '67009183.5667*1484792554', '98969128.3652*1478385162', '42458346.7464*1471021662', '81723451.8895*1483354292', '19950991.0608*1469327045', '71144868.5094*1458022715', '79849440.6445*1468574675', '8019312.86798*1466353797', '43529971.2894*1463885590', '89491295.3275*1460121610', '8159296.17321*1483015754', '18751106.2129*1464443405', '37995287.8489*1466690116', '60110613.1051*1487736761', '54549532.3029*1463268521', '46941047.1969*1486050359', '5495034.35884*1483251800', '57410399.5014*1456101368', '69243220.5432*1453774470', '48347267.9693*1479595970', '8414078.08034*1458785663', '33417739.4272*1472264379', '99184163.4955*1475568365', '25489273.0842*1461198218', '42615754.0084*1454450413', '83550294.72*1479076852', '29187652.6009*1470785443', '75500169.2571*1457634862', '48346894.2106*1468916983', '11958471.5074*1483457246', '43230550.4056*1462049436', '7889356.0429*1458460270', '54276697.0743*1485717540', '43549025.1252*1472780692', '54428940.5878*1470566519', '99729038.0266*1465097105', '37203265.6864*1454118870', '46304384.0323*1461104848', '83862067.8805*1487083690', '95000641.5339*1451948169', '1919177.12206*1452687642', '35353772.1256*1460590908', '64246149.5219*1465649384', '8460801.83339*1471193383', '75145863.994*1464038622', '44696867.7785*1470831718', '90681584.8874*1487350889', '63917403.3095*1455712092', '15888773.9545*1477220516', '65876726.6047*1470869224', '96887711.6003*1467931486', '19370967.774*1455586074', '47764545.5833*1482699821', '72373366.5065*1487685234', '67015284.661*1477149279', '51640147.9739*1467536119', '70934910.121*1481722944', '38387266.4398*1456148146', '69941559.6209*1482982241', '17801966.3564*1482780596', '45765351.9657*1485220498', '15433457.3496*1484009720', '52161817.9414*1456286971', '10797198.2881*1455023074', '38864454.1403*1480594653', '74822757.5456*1483601817', '70387621.194*1460726392', '4406397.03801*1475973112', '22726649.8241*1454318265', '26201848.3871*1457712502', '14133586.6684*1479961203', '51092006.6712*1461960789', '14863642.4266*1473586125', '19498817.5148*1479133517', '86826370.786*1475073509', '61820440.5636*1452167835', '89842442.8521*1472307251', '78319392.016*1470868989', '92493526.5539*1483602117', '73676082.4252*1483944984', '33618502.1118*1481899694', '18596923.0965*1472341708', '73042882.0856*1464888282', '62398886.265*1471856621', '56043532.5065*1477733893', '73483905.8968*1455878574', '96826713.2759*1473030221', '18017500.0526*1470645188', '3547869.0042*1476845374', '42580802.1837*1474988425', '56186144.227*1481765992', '46445489.5315*1479273324', '91971336.9584*1482270067', '10647315.8024*1477776120', '25892096.4317*1453218526', '78163302.5649*1464678900', '5470750.25185*1479062203', '89012091.52*1457813906', '30469696.2359*1454614969', '27293598.7805*1452000769', '44040479.1712*1473584003', '61106761.2007*1465341162', '60919126.3495*1482876227', '69828575.981*1480494802', '39492087.0161*1460026288', '85792160.0926*1456655197', '65566412.4033*1453266315', '7520300.84858*1480160797', '24388242.3524*1462184852', '98350311.3005*1465866755', '69174685.7137*1480502534', '42694988.0956*1485747626', '78392171.1577*1473301889', '1287597.54509*1478584127', '58333454.0888*1477954559', '6316501.93094*1475695941', '45227813.2121*1456008997', '51111765.7015*1484503600', '76321009.2216*1478611816', '31705922.2482*1475207693', '83000217.3855*1463998042', '90887254.4609*1481215943', '44130401.4698*1485356689', '72233939.2276*1474116999', '42424486.8602*1452794520', '46180268.3167*1483811839', '3432403.82506*1466094387', '29611281.4145*1470572004', '76131705.0887*1471548962', '39664038.1513*1469051591', '23009250.2923*1478718238', '93476057.2514*1456379135', '13971493.6766*1475706825', '20229571.5247*1478382978', '19012608.5773*1463980128', '50649941.3379*1454084975', '19430975.1729*1486033074', '90343733.6696*1487226052', '13222256.1308*1485370971', '786679.375876*1455453508', '15167102.6746*1454198086', '16874843.5849*1483876875', '64730750.9661*1456296943', '94166551.3776*1466279075', '23646581.4931*1456310033', '7702411.04282*1486974712', '76789935.5325*1466106340', '220317.432271*1468010718', '85710597.4787*1477524999', '1298617.9735*1488153831', '94012121.977*1479616945', '86375221.2549*1467383462', '46243093.0577*1483401708', '73971625.4228*1453042488', '10889775.7119*1472529656', '66474823.2898*1453876393', '7626398.37981*1458631016', '93025076.2407*1478036525', '64237519.0917*1480238513', '5928771.41704*1483346964', '65575206.2226*1486714056', '6634418.20662*1463498939', '21921388.0401*1481258143', '23967651.7454*1471066329', '31920177.5982*1484288549', '19950991.0608*1469327045', '86721455.447*1480605385', '3399137.69074*1459121386', '59803180.2976*1465288957', '73208226.692*1470673729', '80430104.6485*1458119550', '32915053.9179*1473446786', '90513957.261*1472865842', '94710845.8924*1453455358', '78837604.1801*1461323545', '27761358.8979*1467972111', '23326921.0426*1455184640', '50585167.4498*1456884796', '76410399.6558*1458124061', '66684856.487*1470663762', '54956636.6671*1458518659', '43686545.0916*1475438545', '1831966.98872*1468200310', '33690930.0111*1456450030', '13042106.3482*1465640673', '10077251.0717*1454153576', '92183586.3414*1453845989', '34347729.3319*1459125454', '47030293.3045*1452076829', '43581011.6159*1477790786', '41195095.3793*1463636151', '35329579.7095*1456713211', '41000429.9448*1458941797', '74034779.9726*1487945362', '8909140.60055*1474840664', '17386893.9957*1469023605', '62111816.2331*1485798801', '97786160.9476*1482062953', '57024533.8973*1467624688', '80121512.5888*1486338503', '99144499.4301*1459100651', '90289300.8544*1470626814', '82779066.4405*1454811064', '38265066.1285*1475405514', '86672020.8895*1462489035', '35942685.5485*1482295243', '82145225.8064*1488294907', '22035437.1678*1460132211', '90158788.0554*1482825673', '59975001.8711*1479528715', '27308686.204*1456099742', '22852421.1515*1462052125', '30016058.6888*1469362970', '25496518.6392*1463968490', '95861958.0258*1457972463', '77353288.1388*1465921496', '31146538.0298*1462621846', '20453175.8324*1470190398', '71042740.0511*1468645484', '73173154.2012*1480835854', '24494347.3639*1471393585', '69041806.8527*1457747972', '61669637.5592*1474144440', '85022200.0119*1452718442', '28603392.642*1476664006', '88913695.8922*1484935659', '39639530.9694*1486267631', '77259011.4089*1454283589', '3952540.73304*1482227442', '92382731.6621*1461604252', '62414752.3914*1474635681', '21061324.0631*1452356280', '71553327.1027*1476550341', '15349044.2284*1469873551', '58568884.2617*1460892993', '81091749.123*1481595789', '71372056.5108*1473249521', '76779669.1322*1488310882', '18963092.377*1460085681', '26872436.575*1457359893', '82841323.7032*1487580268', '59721738.9463*1452665946', '4191633.37254*1475266522', '62288220.5802*1462507021', '24738043.531*1468284308', '93025076.2407*1478036525', '52819905.9342*1476343777', '18068341.8824*1468766751', '20343620.4501*1487476702', '74841885.4333*1461649645', '46281731.5368*1484454672', '18968755.4333*1486671383', '96016962.5912*1466984819', '51377135.6136*1466489747', '31865381.2148*1487073215', '42424486.8602*1452794520', '24793357.22*1473069986', '97007130.8265*1473807549', '90172285.2379*1482276785', '72028477.4981*1475197880', '20231312.9804*1455388310', '93725404.3491*1463884577', '90425806.3499*1485661134', '6503222.10632*1462779102', '49409159.2346*1481980143', '3105519.23952*1473095166', '5304917.93672*1455898140', '72023131.4994*1478591291', '30200348.3437*1470265753', '64778783.9365*1480850955', '39263139.6697*1478368608', '11682247.9767*1474284947', '25236873.8672*1468238016', '41829184.3193*1465657265', '69064093.1649*1453908840', '50731375.2088*1475174877', '71051253.8672*1478339900', '44460934.789*1459775304', '81010241.0091*1478138994', '23696798.1093*1474300962', '67116770.525*1459950627', '92020905.0777*1460046738', '90420199.3478*1456252402', '68339054.0181*1456769911', '92237303.529*1460743261', '29611281.4145*1470572004', '61384721.7253*1474172102', '95467031.4177*1455289338', '63840324.5325*1468539516', '71275386.3687*1478493539', '45657871.1318*1465156280', '49631360.6637*1452387967', '14621681.8272*1455638582', '14035422.4445*1479225434', '95340082.6508*1455612649', '59408433.5048*1465999176', '20558805.9063*1481556993', '89576942.603*1453432440', '71144868.5094*1458022715', '53114393.748*1456003191', '32006603.0844*1484272794', '7170102.0748*1463583325', '18261221.5107*1473088762', '85456547.5986*1486083849', '1920152.9983*1485651368', '39008493.7872*1484893770', '79195998.5955*1464230258', '56916497.2261*1485951456', '14435698.1195*1482173925', '71729193.5863*1476481240', '44046154.6848*1459779761', '56370476.5509*1463712632', '39676320.7373*1479561722', '85985595.6077*1465897402', '26530276.6658*1475711956', '75341429.3085*1478665076', '16417900.6199*1465268587', '58237199.9099*1457985697', '63676472.6631*1463264315', '26185327.1345*1454632043', '35745456.5672*1487171107', '49453002.8552*1478102796', '59186620.7815*1451803874', '73076315.545*1482532065', '89781218.9952*1487090468', '6051319.00684*1467088075', '97515048.2293*1472012412', '38117810.8726*1485166278', '70370304.9397*1477478427', '32918131.8498*1486733141', '38360017.3942*1470441262', '26075908.7961*1486826980', '88355118.838*1460021205', '44236064.908*1477911144', '61567848.0273*1470660796', '54251474.4989*1475111063', '58807194.8961*1483606251', '83173851.4915*1474596388', '54765591.5953*1452010591', '43888173.3352*1469980972', '51135758.6801*1475397103', '67465401.6127*1464687961', '83686880.7738*1468848163', '61422745.4302*1458198293', '43042265.7317*1474708238', '86101968.1369*1464743139', '60365967.9401*1462661383', '99682618.2568*1457931355', '56045803.6089*1487588543', '40795753.4957*1479425416', '52357914.2009*1454142069', '89770429.3481*1476563436', '89265301.3033*1481279740', '27805571.5451*1473241391', '18278811.7563*1479250073', '37711345.8437*1484205548', '3965662.76881*1473719687', '22191472.7501*1484129214', '57039027.5832*1481900836', '38395984.6584*1452741984', '66704974.4219*1485705477', '25556180.7008*1465886531', '27632751.9439*1480343474', '54403579.875*1460995142', '47438684.2096*1453180858', '76539491.7671*1477740219', '61079103.7698*1466968473', '62206706.8047*1482380631', '64992360.7121*1463100685', '4677811.28339*1485070684', '34217050.0504*1478177971', '18104718.977*1464072950', '38508965.7583*1460037450', '391010.425267*1478047346', '84445397.2211*1470238946', '14873894.8426*1482056186', '16687802.7987*1457798277', '34217050.0504*1478177971', '93932059.911*1453053218', '25346652.8968*1462783774', '34377186.4645*1454686405', '34663198.3056*1466031045', '10202581.0343*1460081347', '96983119.6445*1466636956', '68797564.0381*1470275318', '31648993.7233*1483397521', '6905548.52172*1451871549', '41508530.1892*1481068517', '28389383.6517*1474421746', '37708966.5542*1453524851', '53397747.3005*1458246166', '32764877.3682*1465308903', '27890532.733*1466610816', '48216348.6326*1470958687', '79812867.3068*1452747674', '14822833.9099*1484059482', '5128290.51614*1469789894', '796304.821793*1481059935', '46202144.5447*1486653446', '44157204.8749*1460412697', '78108509.3229*1468641501', '70320243.9815*1478834027', '3871758.1268*1469757601', '32462389.013*1479728610', '37172288.442*1459405354', '41794115.3061*1480628716', '31074926.1086*1464057941', '30258116.8453*1457753862', '3415742.54978*1457131177', '10100708.2532*1487745738', '49895310.7845*1457386437', '74517592.4077*1487500062', '1781345.14046*1453931943', '33076574.3956*1481224529', '8608742.46241*1475538902', '51009055.7208*1478791482', '30228562.179*1455685882', '98110716.9312*1465225661', '49468330.4973*1475287032', '84841620.3804*1484343211', '48583861.5549*1477335944', '73831864.7902*1471340943', '5862514.04828*1467618895', '57729329.3108*1456556262', '2874677.63036*1457506357', '9803307.06285*1478373009', '33533993.5043*1484269234', '47678782.5555*1456486196', '91279381.8413*1462439363', '83907741.6791*1469657123', '92902608.6812*1463642311', '81139663.628*1452196091', '38179832.255*1470296530', '56321913.0056*1458847215', '89646623.3727*1472135635', '73712073.5959*1464868998', '96114677.4895*1455378963', '43642009.4077*1486889022', '99996831.0841*1488213267', '51697599.0788*1479266689', '43837264.8905*1487958755', '50859133.0687*1463431525', '78066945.4928*1477036772', '8707163.8915*1483588627', '33456725.6204*1452560864', '77377503.982*1475164926', '50365743.6901*1477120469', '26555529.4625*1466513720', '90670678.8155*1458398931', '32915053.9179*1473446786', '19855495.1118*1480041139', '91604045.8698*1476434768', '63800018.2696*1467146097', '72270048.7858*1467888938', '13627635.0612*1456180387', '37834452.1852*1467049292', '90985928.5528*1465946023', '36782003.0368*1478265321', '2709043.95279*1468905991', '13975008.8455*1461719422', '16378232.4717*1485967422', '92432993.1624*1478196685', '78391235.5155*1486257097', '62732428.972*1461905268', '94166551.3776*1466279075', '50903384.7807*1481932993', '93380340.7097*1462855376', '17936695.8628*1462262940', '21858468.2566*1461129070', '67079347.1143*1485688156', '66655757.9616*1468734655', '58529578.9013*1462003627', '48699608.6809*1471135271', '63481423.7704*1454704001', '63383817.9947*1452618393', '12487718.4318*1486365265', '96869791.5033*1480917354', '25595210.0066*1468690977', '42449321.8316*1475688775', '70233741.163*1475977995', '95126682.9*1482246824', '16979503.9473*1471837585', '847048.698649*1478321187', '66472689.3181*1480373441', '77231319.0386*1485428471', '48242150.2286*1458333930', '10455102.2205*1471753149', '99114259.5957*1453496513', '48838735.9788*1451937070', '52889443.076*1452180614', '6799538.76232*1462497767', '28269571.472*1480066595', '78043857.5945*1473506952', '27220432.2075*1463625875', '60956449.5503*1452766808', '95384660.2171*1485717623', '91771451.2778*1482007804', '77603090.1435*1460872119', '29099143.218*1476677847', '60343217.7049*1479388024', '29973423.5797*1473417849', '48053686.932*1480123338', '8513952.84621*1487819551', '96332890.1072*1481122893', '20907960.9733*1485342474', '83637347.2206*1473241025', '11023464.8844*1483352455', '58776514.8457*1483685771', '24567182.2439*1474758670', '34728054.3895*1471471745', '91979737.406*1461820557', '36632494.7086*1472882290', '95781605.7673*1459377896', '66704974.4219*1485705477', '86096599.9341*1462492602', '17168634.5622*1475978739', '76820760.7924*1453299731', '59592441.1239*1483815357', '66382840.9166*1454029730', '33782582.0628*1456561415', '21365269.5936*1468911597', '91687610.5167*1452692188', '17406487.8011*1487553331', '16887712.0036*1467395133', '99037540.7753*1471750379', '60754855.4114*1453944332', '1950932.25957*1472299272', '11315078.3179*1463276347', '69933057.0275*1464150149', '4440050.29981*1470026319', '6776988.63819*1476994936', '26516312.2685*1468568676', '43604135.1969*1478038202', '60784256.4314*1486134184', '88236551.5721*1464751169', '78904798.7771*1483792510', '24425929.1809*1474762838', '4483966.53161*1459025780', '83325409.0158*1464476496', '56496349.4684*1485310854', '32251480.2938*1451921545', '55463491.5061*1475282117', '73729930.3273*1480688634', '41612941.3265*1462806860', '99335354.6322*1473344064', '18019591.71*1470919516', '14850888.0651*1471725152', '12588230.2319*1480110594', '17185311.2848*1453246994', '98302277.6216*1454927400', '85405324.2162*1456284096', '27686860.4245*1458683935', '71337066.3597*1457697219', '38035341.9324*1475449625', '81212882.0561*1467337806', '98170067.8122*1456520059', '34347729.3319*1459125454', '14922588.6158*1484517730', '63062971.9896*1466698089', '1784056.51043*1451754280', '34065107.8024*1452462825', '35058655.4936*1475764983', '98486812.2808*1458154904', '76282219.5044*1468119498', '22908643.725*1481091632', '31508988.2219*1474111642', '21109897.3144*1461763661', '13578280.2674*1471253184', '6370144.45765*1455609672', '28910002.6492*1475263897', '28592237.7637*1470819182', '63943725.2249*1482830467', '3163402.14419*1485476940', '25102724.3822*1458478120', '87171401.3258*1473062706', '87696768.5818*1485128824', '84640876.7094*1470693697', '69386058.7551*1465696691', '22693518.2432*1478753151', '74108055.8004*1465093811', '56117387.251*1456856988', '47035257.797*1481696067', '16655919.6917*1459861690', '10288511.237*1466193825', '26298421.7544*1473807542', '21205402.4665*1474728050', '30664907.895*1457608995', '48849085.1986*1461214190', '34626346.9806*1479253117', '51545025.0543*1466034228', '88744566.5977*1465233237', '72125373.4605*1452379508', '30130535.8995*1481332470', '34424968.4066*1468943966', '92297879.915*1456147065', '56245228.4508*1452260276', '15876713.5261*1487062787', '11352211.0916*1464560100', '61315413.8722*1474464531', '67563362.2056*1482010826', '16157111.68*1469408240', '64567654.7108*1474720021', '90458298.2103*1453486849', '15462252.6514*1463377894', '14938618.2822*1472482469', '51697599.0788*1479266689', '57273512.2567*1462664917', '24575773.7998*1481531001', '58409878.3411*1466928119', '35822016.983*1473952196', '70680959.0388*1455047051', '52862178.1727*1479816966', '44168529.4191*1471656259', '34924937.102*1476184586', '2257394.49903*1464744879', '27252770.0308*1458824831', '3586201.163*1473526705', '89643474.5631*1480398681', '40459776.4877*1453758649', '84449102.2042*1466683703', '40062924.0947*1456612839', '40549416.3033*1461314490', '92451093.4666*1482749336', '24957950.2853*1485567817', '7474959.33302*1479238018', '91279381.8413*1462439363', '59027350.7803*1475873169', '39871145.3849*1459374470', '37382193.3433*1456558254', '57039027.5832*1481900836', '96546437.6836*1487966528', '41892082.5761*1458601233', '76999818.143*1465621384', '19469642.1687*1455226510', '65039831.5978*1459714935', '74486802.2976*1451905921', '11163625.1241*1452287620', '31243736.6029*1481341660', '7218589.89123*1488193847', '34672661.3781*1459270081', '51835994.8803*1473114069', '5557888.52282*1454097399', '50480306.9704*1453420059', '80051106.5146*1457698520', '39349574.6487*1476318595', '79038023.1869*1469714839', '44214897.4001*1463616646', '72023131.4994*1478591291', '10876493.7005*1469275995', '72324847.0352*1484689768', '42458346.7464*1471021662', '69052826.2204*1460592484', '70125720.5961*1479013828', '16860716.2292*1471554105', '43581011.6159*1477790786', '48903183.346*1483976968', '57989754.7319*1469351841', '39281205.5729*1477153134', '6146207.50372*1483068021', '55072943.6791*1461276740', '16941456.6165*1467269005', '15069548.0512*1473131600', '80457037.2396*1485228569', '49395328.1539*1461862978', '85564873.9978*1478182220', '38083500.2509*1488183475', '36315044.5021*1479125058', '11352106.2089*1482011366', '66472689.3181*1480373441', '94575134.5836*1468863158', '2256168.45051*1474523662', '28062657.9935*1455077902', '36082069.4099*1459679738', '37970388.2558*1469984605', '53268301.6454*1475854235', '2376255.49556*1475612155', '38864454.1403*1480594653', '88320507.9593*1458518143', '46758690.2919*1473165719', '75630058.9799*1459878681', '24881713.9912*1486535133', '28181113.4172*1481302721', '4779412.12442*1487751382', '40583276.3721*1474848672', '54901109.9356*1483978014', '93939745.3608*1464308596', '47472078.6711*1463788343', '43884999.7318*1486681276', '56643796.2837*1458664013', '70546884.3913*1488157698', '21901742.5043*1463988643', '60365967.9401*1462661383', '84027817.7083*1460437660', '37589086.961*1451730738', '21861556.1291*1482753756', '26083885.3505*1480455767', '12004267.1627*1482864358', '57159699.63*1463485782', '38522573.0462*1477975734', '96284902.892*1454864257', '59473855.8674*1478098108', '77575731.1964*1483750216', '58287781.0998*1453771003', '64747259.531*1479735269', '744240.689642*1467871746', '23759794.8812*1473500264', '89164723.1358*1463363680', '11966447.1765*1465377155', '15598373.1316*1477818115', '78809477.6749*1451611951', '70830672.8868*1480004235', '70815921.4595*1473121822', '16453896.8318*1465713640', '36892062.2894*1462409606', '99718150.1354*1463354901', '33076876.5959*1460187328', '28026783.6473*1461091814', '6866426.48311*1462118289', '78503513.8797*1482935594', '86337287.4109*1460380080', '77302896.8165*1458543585', '23540368.1961*1475262855', '60663117.3911*1479769566', '41857405.1455*1456928572', '46755734.9784*1470333172', '18852484.0632*1451735657', '536608.13503*1472244357', '80690225.3367*1470704803', '87579613.7996*1467245904', '66977006.0282*1459093491', '81091087.6598*1477745796', '38172025.2729*1472469250', '7098238.89263*1465856667', '28948892.1527*1480713090', '3114095.34428*1464957168', '53328486.5056*1455769501', '29709093.628*1454505804', '82897686.9409*1457937533', '6130211.5255*1459856402', '74508372.3694*1456936901', '75500169.2571*1457634862', '31594903.6724*1462508946', '71149545.7552*1482284385', '15533563.9434*1473892509', '36629874.8752*1475691034', '89968135.183*1460193322', '7032280.92574*1486816001', '74431095.8182*1468435324', '13578280.2674*1471253184', '95969316.913*1475616618', '33565896.7971*1487759844', '29827716.1596*1469614134', '36527483.0752*1452010149', '51468070.5854*1485227780', '80348565.9178*1472219496', '91763444.7866*1478477418', '4461267.24732*1468194198', '88039905.9155*1463651945', '88235287.9085*1484648418', '62154177.5383*1472386394', '49363032.9974*1479844593', '23938917.6502*1457843191', '42053605.9571*1481336532', '75959980.3591*1470664766', '21088702.7842*1452142954', '47964516.7344*1484920508', '92631508.3295*1483085803', '92223054.588*1487502960', '7864082.9472*1488070574', '5143347.72418*1458015980', '91968134.168*1453559946', '72579283.4439*1488026426', '28026796.5543*1455193502', '66750224.0262*1466050266', '50109793.0804*1454116165', '32228635.7859*1461727509', '66457742.7195*1474614756', '74106162.1623*1476755813', '84007239.3221*1456131995', '21367014.0498*1480098400', '12419506.3528*1476587473', '3163402.14419*1485476940', '66848813.7005*1453661080', '94193839.3336*1455700246', '57109684.85*1477587213', '11453216.7665*1467346110', '7487424.38288*1487431577', '89164723.1358*1463363680', '31615944.7365*1481241860', '73879840.3799*1482590307', '47362088.8185*1483048387', '24140500.3559*1462429937', '67087822.8052*1470167024', '45308234.0311*1477803771', '34137509.5491*1486700437', '8961024.67501*1467960464', '38780432.6206*1455034816', '25839776.85*1460463590', '46130751.5406*1452412119', '62457960.6735*1456124267', '91668525.9144*1453880837', '2232690.77874*1467061800', '82318948.1944*1461756187', '46153289.102*1455432070', '21550835.0543*1470993810', '13617588.1601*1481407912', '65245311.4403*1481320989', '16251288.3722*1472349289', '57183440.4629*1454569239', '97807799.9021*1482572220', '89621375.6199*1485643255', '6017121.06207*1463751606', '33938983.9125*1478814086', '96503352.7831*1471812860', '68577759.3482*1485770304', '71880594.8913*1469788511', '5559825.92341*1458081844', '22944728.7945*1479233299', '48528954.9606*1486819922', '55928371.1229*1478312591', '63350274.8321*1487146510', '16126692.4936*1484724222', '63212176.1639*1466889394', '79136577.5331*1452219471', '12944761.8248*1465355977', '28026783.6473*1461091814', '10254587.691*1479176697', '22910599.9561*1479682670', '46737190.7154*1469388993', '6158289.92448*1473399767', '60677593.1226*1477116377', '74633435.8115*1453461809', '79115776.6867*1473815937', '3689438.4576*1470820549', '72720602.3836*1457313615', '74595826.4361*1484624920', '76484937.5127*1479492587', '73676058.7133*1476580612', '92536265.9294*1461709646', '33927585.3372*1471828669', '14988113.0891*1466791321', '51111765.7015*1484503600', '47911649.8354*1483923040', '55943189.3708*1469643780', '67937780.0158*1453689175', '2098449.71018*1470855443', '8112255.66735*1464570677', '39962596.6354*1487681419', '58591955.692*1467546747', '53802954.4629*1453771251', '13975008.8455*1461719422', '94627902.2845*1485430751', '59818894.3568*1467232120', '5915847.97598*1484801445', '89781218.9952*1487090468', '51279343.4286*1471241304', '20765244.7947*1479171682', '40024808.4487*1464321870', '47402577.6858*1488162845', '8149794.38766*1480705117', '45688607.0221*1460655865', '15286135.0854*1463278510', '72577433.0246*1484368300', '46975494.5064*1451947798', '59975001.8711*1479528715', '66696938.5107*1484948272', '39281687.8425*1476187058', '59209980.9984*1488213611', '83136987.2649*1474173392', '6050407.27058*1473942706', '49800164.2324*1458463034', '2013383.96344*1470230851', '10783361.4164*1458532652', '77440531.3542*1478422992', '74853907.7543*1458042302', '87937701.8234*1468513288', '5947705.40325*1483601565', '74263265.9538*1486573832', '43311826.1085*1476258862', '98649291.2477*1457732124', '79849440.6445*1468574675', '49966611.1573*1461023788', '78837604.1801*1461323545', '40067889.8285*1452633689', '23872365.8768*1483827184', '7522898.05854*1467759656', '18078565.9097*1453745718', '78788984.4787*1476649684', '21856877.7528*1457518045', '5160765.14155*1468046867', '57041738.2359*1461859735', '9833257.25202*1487731859', '63057882.2181*1469580816', '73139764.7085*1464406959', '81373330.2778*1464014483', '56741493.5092*1461228085', '78759957.8396*1453260668', '76020997.3109*1457037310', '34435746.0297*1476671498', '34365192.441*1469318659', '18632085.2642*1458314037', '33066794.9221*1473766683', '42656361.6612*1470111782', '55944885.2546*1464288667', '90597420.3661*1464575049', '60988920.812*1454965703', '65876726.6047*1470869224', '94161532.9042*1471944919', '2344304.90603*1487728355', '73383401.641*1457228199', '91107353.6021*1471064572', '24388242.3524*1462184852', '59583485.6535*1468996470', '48242150.2286*1458333930', '80533879.2547*1473656406', '98063024.6896*1476056659', '80619400.461*1477781532', '22303391.4452*1475597077', '62216815.687*1461984565', '33675171.8148*1465451360', '75820738.9417*1483681601', '97204033.6938*1457849154', '61048505.1581*1462659007', '20767673.9435*1456238833', '49404681.3007*1478358243', '87491193.358*1453251516', '57039027.5832*1481900836', '32191568.2872*1472466662', '49761794.8845*1478493217', '11992897.6116*1458901000', '59769486.4354*1476405687', '99006277.6335*1454866548', '2958062.12842*1456807495', '65648406.2363*1455158174', '54428940.5878*1470566519', '19592175.1259*1455395665', '5303110.48826*1453186277', '98525719.3435*1480914258', '14247936.7799*1486623148', '63880486.6117*1483644426', '24910639.1158*1456089156', '13351632.9103*1462973305', '80672312.6522*1470403405', '57514846.1927*1455801023', '90647435.9827*1468101724', '49453002.8552*1478102796', '60462710.9216*1472306731', '12278854.668*1473556146', '74384913.72*1455974855', '83001336.7853*1483943235', '30303631.9073*1465313059', '42999753.1455*1476968773', '99252661.1645*1485164700', '98606330.1961*1473111226', '56803089.4001*1487016525', '11110763.4738*1468685011', '8707163.8915*1483588627', '7094246.65713*1481124440', '76661004.0074*1456509414', '72574827.7226*1459238822', '85863078.9335*1469281735', '49111565.5968*1461583775', '47237771.0555*1484237254', '16351061.11*1467237713', '2529800.43418*1458841635', '42487558.0839*1471628037', '78596930.3223*1458792211', '34624101.7148*1473505758', '25219674.1962*1473959224', '32228261.1169*1477406016', '46103709.7755*1480486581', '55309537.0102*1478882378', '33923165.2789*1459743335', '97175388.4172*1465171671', '60882892.6282*1461128664', '30182473.6313*1481161428', '73600169.3122*1456119639', '37610409.2895*1484657062', '91545230.2289*1451646892', '65142446.256*1485219612', '81139663.628*1452196091', '25184576.0823*1452714322', '77738051.2996*1467443136', '2098449.71018*1470855443', '77146413.4146*1451647352', '98528135.6044*1486036523', '85319479.7175*1477437329', '77373727.3486*1459142317', '4155022.64652*1475692040', '20935321.4626*1458843696', '94968120.1605*1475897545', '3749294.36318*1476641043', '88759494.6445*1471749640', '42656361.6612*1470111782', '82158631.454*1469507347', '68091514.6978*1483346000', '86818016.8289*1471065449', '80867877.5995*1457866259', '3875426.16101*1484205875', '27239089.2535*1466574574', '74885386.3984*1469955065', '95265104.068*1461088810', '53016511.173*1463233750', '90758827.3044*1485031171', '29973423.5797*1473417849', '12282478.7767*1469345557', '55322385.2197*1459888776', '54537568.4498*1481132549', '57284706.4449*1478888218', '45587058.8782*1471356413', '44569561.2167*1456510217', '82764840.5924*1484867871', '28406190.8614*1456902769', '73430247.7344*1459203750', '85375910.4934*1463214331', '29598944.5255*1479703393', '79437144.6464*1468773873', '89122139.7077*1469362442', '87036484.8466*1468306571', '65248526.7447*1483658318', '37640513.4467*1487965733', '27968412.935*1486153663', '50697348.1517*1462003045', '64103303.6942*1480633811', '24932501.4326*1471329387', '47746049.2571*1483286160', '89991721.2313*1452216360', '79316290.7014*1480616093', '36632494.7086*1472882290', '77187072.0082*1483478801', '32606683.9395*1461223310', '56518473.2633*1473131133', '62838167.2168*1482225555', '19190074.186*1472788616', '22744873.4124*1474512902', '75113323.5794*1457689085', '77187072.0082*1483478801', '11794016.7285*1455154618', '52380639.9826*1477678397', '8612498.40209*1466837249', '31732789.1555*1472061220', '16733108.2028*1472323839', '37324204.148*1452757700', '8418037.73131*1452195076', '69041806.8527*1457747972', '14957644.1005*1473251106', '43608777.6953*1470112763', '15069548.0512*1473131600', '3547869.0042*1476845374', '27490302.2088*1479487212', '13045168.4609*1454082444', '63384544.8254*1471925648', '33360671.8991*1473669546', '66842942.21*1458356486', '17738940.7876*1456535235', '88184417.7284*1460762304', '36305244.7175*1466167215', '81158683.9869*1461366495', '91284914.4562*1452570571', '86067736.4486*1468376643', '44891265.4578*1473590470', '98850116.9372*1480002856', '15924279.5643*1485360871', '17478168.9775*1479465450', '17575914.2154*1463721299', '50208103.8346*1452227880', '27491523.4034*1465341119', '71828845.8959*1484663458', '28594652.86*1466656505', '29914473.4879*1453610104', '92843757.0934*1460409168', '15565430.2891*1455124310', '43653135.5523*1451798752', '28392430.3943*1459006200', '17696591.6754*1483055142', '67666788.1291*1471524661', '76483913.8649*1460194342', '59375089.4272*1480252283', '31756758.1871*1472229123', '31598504.6537*1479868080', '17129914.9854*1480666341', '95905194.8398*1478657335', '13449456.587*1476525123', '8460801.83339*1471193383', '73512593.1274*1466850127', '27425207.5374*1457153808', '63160083.5015*1482784364', '91630813.6837*1483150748', '30086606.8834*1482867859', '63186033.7549*1465614206', '76278128.8609*1474093170', '57156452.0084*1466451361', '84244241.1358*1459447294', '92150506.5995*1457334689', '11163625.1241*1452287620', '11868578.3374*1459177715', '36315044.5021*1479125058', '62215795.1607*1480816664', '38012030.5888*1482645069', '90578690.3162*1460303860', '49073349.1797*1475446595', '25911839.7069*1480472652', '77603090.1435*1460872119', '50145628.3051*1459341832', '25457689.5071*1488299995', '22122056.1815*1484507581', '29310391.5795*1462834297', '39459540.7511*1459193502', '44738134.3218*1472004874', '9868674.44574*1477136236', '97281699.5543*1467279034', '98507877.3218*1484378707', '85777067.6394*1469363311', '7783910.56224*1475580754', '2199518.77127*1476005244', '59605524.7909*1457927949', '60866155.5475*1458220650', '74640284.8075*1465146265', '55731686.7835*1483700101', '28276139.9785*1476172029', '82202952.1408*1484836565', '81701248.2517*1483114299', '65746146.0345*1464931071', '66276334.5905*1454423951', '19168467.3839*1461110093', '52120970.3834*1454628770', '61939478.5875*1478213922', '27006370.5137*1486111540', '56391918.4998*1460303726', '46860513.0984*1470095663', '43847893.3665*1481459490', '25648056.9011*1458245776', '4648439.7244*1453816126', '7856506.62062*1483926407', '51138981.796*1471474677', '18317919.3522*1476030783', '73236587.8728*1484877603', '67176544.0812*1474712999', '65245311.4403*1481320989', '87185366.7938*1461418648', '47693721.7599*1477279763', '99542150.952*1484222223', '33744387.8518*1455496864', '96667977.0227*1466542837', '50859133.0687*1463431525', '56947914.3038*1487672538', '21161762.4967*1478044742', '54453989.0006*1483010083', '91875709.5173*1483435555', '50307438.4342*1458484304', '71859685.9914*1482428479', '94800512.879*1455664942', '21072254.2864*1454352689', '3105519.23952*1473095166', '72821846.3687*1473425727', '71812441.1886*1478302065', '10891560.5334*1454774360', '90335709.3352*1474855875', '30200348.3437*1470265753', '50518745.8366*1453701652', '80457037.2396*1485228569', '11376105.5303*1473714998', '49315144.6711*1461696297', '77481050.221*1469284557', '106305.036673*1469427392', '41842366.8493*1481104645', '70438993.6847*1478842482', '55049792.8835*1462441021', '98824228.098*1485941326', '28761085.5652*1464263098', '46474692.4708*1484230026', '77066963.1591*1468237272', '56067042.2541*1452383896', '60598301.0073*1474831215', '46070545.3461*1474976547', '95814030.424*1473211379', '37177825.3529*1461579221', '34034091.1111*1456033117', '4781492.01121*1453268867', '98612080.9437*1463293191', '42580802.1837*1474988425', '27258693.8624*1473489093', '76839839.3461*1486928663', '7911961.77505*1462340013', '43642009.4077*1486889022', '63426710.8701*1480542415', '62932122.637*1467103884', '28382005.638*1487675569', '91532627.5667*1465708340', '32265318.2082*1468255288', '79547394.0408*1477739837', '7248774.25741*1466814754', '25359956.5632*1458770552', '61939478.5875*1478213922', '71050828.8975*1458010329', '69074102.4587*1483629383', '41324990.075*1469886051', '18573798.453*1461426005', '64503997.5731*1463581062', '44637332.6062*1486631176', '97874226.5227*1453473378', '36820137.6137*1469236994', '38684455.712*1471624305', '16804191.8481*1469010532', '96299489.113*1482457453', '12173728.6025*1483431557', '81726782.4626*1482724671', '85580308.3098*1482294090', '16655919.6917*1459861690', '8069548.26203*1453996456', '78290672.2877*1453560825', '66696938.5107*1484948272', '77632260.7721*1455608978', '18709426.3591*1481694187', '57492297.0172*1477517909', '54893726.1125*1475152972', '71074195.6643*1463705956', '6500229.09954*1454497331', '70233741.163*1475977995', '36451791.7306*1479378400', '59769214.8871*1475776898', '29792950.6034*1457162563', '39051774.8638*1470931454', '37581616.3619*1461909753', '95472586.6088*1461515316', '51169613.2716*1479696532', '51914580.2363*1472482217', '35777494.5556*1452854090', '82360488.9448*1454682947', '36266580.0342*1480548587', '93703811.6542*1473110223', '47961870.7636*1461646085', '8181753.92561*1488315492', '53600811.9198*1484478543', '46929741.3442*1465447137', '50220282.6313*1480734606', '68426487.5302*1483147494', '57007.9584623*1485128752', '63880025.9963*1465371851', '26807253.1427*1468230199', '25496518.6392*1463968490', '83176911.5388*1477470537', '72293422.7161*1479586239', '27407758.3787*1466935405', '75040669.4381*1456786777', '16747969.7718*1483163953', '87082150.215*1452552002', '39939271.5865*1487733734', '86922466.7631*1474836043', '85210122.7581*1458243835', '57209352.9229*1460151695', '29973122.9672*1468251877', '25663529.175*1462797082', '99482765.5438*1475366665', '69376706.4977*1464684937', '75934726.0652*1468315646', '87223296.4388*1483391984', '34580769.0242*1458513952', '890790.067644*1477348152', '63723676.739*1473042571', '94094512.8505*1481405160', '16422891.6353*1485836882', '58044373.4257*1474689388', '96887711.6003*1467931486', '22904138.627*1466181976', '33256036.4577*1467003579', '72383020.1772*1472440877', '33697353.2743*1472098408', '35128811.8967*1460302455', '23234124.8305*1466594938', '89842442.8521*1472307251', '77816850.687*1468206636', '83450751.9655*1463202675', '83968402.2335*1486633947', '62587513.5801*1465479931', '75882041.7742*1476922117', '94345265.2684*1469277324', '10352086.1256*1473516956', '43705985.1595*1462766811', '2876679.81848*1452316951', '11418880.3204*1464688808', '93972012.3533*1484030988', '29236188.681*1473941822', '75337902.5617*1480717110', '94615405.7843*1479925704', '73632060.0051*1461889582', '50458366.6304*1451713915', '92099009.3327*1458904883', '63528996.9782*1476787498', '39613792.4766*1470311730', '16217260.6463*1468914467', '38683649.7645*1477189363', '29403710.3638*1471959809', '74508372.3694*1456936901', '92954866.0069*1462815518', '27647229.7156*1468965367', '99286402.5773*1479105566', '25292865.2606*1481287810', '67057206.5969*1468871484', '51522661.6873*1485592474', '93810406.9374*1470535453', '29877750.7501*1466445364', '94519675.3167*1459933353', '10797198.2881*1455023074', '56946612.8694*1454198440', '23221066.278*1473880092', '60677593.1226*1477116377', '22312973.7896*1456269040', '56570100.4696*1485201743', '45587058.8782*1471356413', '36704714.8277*1474776219', '59700383.7715*1471320556', '6053629.27477*1456308665', '20055190.2636*1477768468', '65315502.3478*1464239137', '50220282.6313*1480734606', '18673684.724*1453008418', '26566311.6395*1465258672', '55463491.5061*1475282117', '32494220.915*1473179288', '74508372.3694*1456936901', '58529578.9013*1462003627', '96374050.8206*1478752055', '35782976.3213*1483927903', '56199000.3035*1476546185', '72098068.189*1467611827', '65445341.2391*1457094160', '89355294.2701*1478815175', '16227717.1467*1473831133', '7581895.34698*1473532595', '86212701.8507*1468827770', '85233184.0727*1458797975', '33641626.8735*1482918980', '27258693.8624*1473489093', '52561012.6799*1459392886', '11614589.8782*1483214021', '15876713.5261*1487062787', '33707789.3665*1477523779', '91134253.8294*1466454370', '80620542.0119*1457947990', '43322905.7807*1469216170', '76659164.4521*1464757153', '67286274.8838*1478997024', '92252088.2137*1486896886', '91148116.4261*1473674076', '99530822.7292*1484919061', '83280110.993*1454482936', '43854102.7011*1464186923', '95528783.9349*1485248401', '45981448.0917*1456700762', '30284647.261*1473952985', '48583861.5549*1477335944', '88864850.2312*1453912891', '6838134.11951*1461250783', '83912092.2271*1454506025', '88478991.8584*1454287732', '97204033.6938*1457849154', '97301569.1608*1485204516', '93842068.4585*1456673272', '76134154.2129*1482240083', '49473904.6224*1483036668', '46633112.8381*1456649712', '67338114.4482*1483045616', '34974285.102*1453585041', '10454394.8771*1485654624', '51222884.8239*1485529957', '11637233.327*1461318124', '62414752.3914*1474635681', '23938917.6502*1457843191', '24899504.1657*1471856824', '44671317.4353*1460051329', '85119475.7299*1481008727', '89525055.1045*1451714043', '33028392.7724*1470219653', '33216869.0204*1479128305', '68053221.3194*1486317191', '78953393.0393*1452820570', '64218945.7397*1467298131', '54004854.1227*1483412446', '84891733.3326*1483537792', '9188869.9385*1456455369', '65577516.6791*1479644469', '41612941.3265*1462806860', '19950991.0608*1469327045', '33984835.2122*1466863694', '48336899.2536*1467541141', '55291691.0153*1478111425', '90936301.3158*1471275650', '77427201.0813*1469078769', '36482461.1022*1466385810', '64264394.1194*1463634424', '50307438.4342*1458484304', '14081197.29*1465201706', '93461862.1516*1455962879', '32619153.4572*1474638968', '9018894.3366*1459309032', '10151814.7414*1480353238', '77471136.1536*1456893554', '78396316.7779*1483586745', '50092943.5338*1470655978', '1713259.55924*1468583835', '36234857.1187*1474694573', '97790869.8585*1461091495', '6694267.78387*1455369687', '501455.849764*1467076670', '69203479.6092*1456025353', '21378117.7889*1473839457', '6365864.74411*1483074134', '60833.2960605*1465358385', '19795318.861*1486806633', '43604135.1969*1478038202', '60784256.4314*1486134184', '21898840.6493*1455905428', '85517241.1638*1467121831', '68817665.5134*1456251757', '75931304.2189*1458026078', '56035372.092*1455091782', '85618322.3237*1480017227', '82013465.1276*1479094259', '37425779.1246*1474395138', '19688453.8619*1460210745', '25481179.5287*1471855341', '16163434.7567*1481533335', '88099872.5826*1467208813', '57817194.8715*1468950936', '48347267.9693*1479595970', '2085396.9253*1455650525', '2974087.26289*1473082848', '74140714.3052*1487114692', '24138499.5385*1484143884', '45557991.6568*1470811068', '82486534.1257*1485039954', '94525928.7603*1466887139', '46730101.4*1466536991', '88468313.4491*1470060954', '75297784.3061*1474706236', '68666092.2455*1476380636', '44109377.7166*1453894016', '10100708.2532*1487745738', '84213623.3492*1462169189', '38672595.3375*1487341251', '30451616.1309*1452129494', '43936787.9468*1481010748', '35970213.8999*1474873842', '57989754.7319*1469351841', '82344571.1837*1481579074', '85083089.3115*1464039617', '37390704.6577*1478155657', '91824376.9535*1470333728', '374273.526822*1464002139', '12773377.7698*1466819026', '30524648.249*1480030204', '82765839.498*1474602919', '51262946.597*1460975253', '8055073.58881*1468584001', '38248665.5371*1482305492', '70934910.121*1481722944', '53451629.2621*1485259402', '41249204.1466*1466569099', '94116956.1336*1471329633', '77044100.0759*1486746610', '66376717.8547*1468840535', '90424441.1446*1469021303', '67759622.6224*1472508232', '66624985.4234*1476799369', '61258337.4399*1458093093', '13399430.9646*1462932435', '92432993.1624*1478196685', '96539570.9476*1462458360', '75280765.383*1453815049', '33068554.5326*1481876143', '58620931.0763*1456352931', '15945517.3687*1458052666', '13083819.6519*1458728602', '48018255.085*1486496149', '60369378.2813*1466731706', '42174728.0488*1475182171', '18305178.3993*1474043245', '65527950.9565*1457714920', '11867264.5719*1479304008', '99419530.0607*1455603545', '31117538.6747*1459478937', '93250674.0404*1474110948', '17457155.278*1474812892', '99580278.1693*1462715815', '2800663.22775*1476463758', '67846794.524*1463181398', '37389324.9343*1471441015', '61702967.6411*1481148755', '20318163.847*1477635642', '71144868.5094*1458022715', '23093085.4567*1488261866', '81158683.9869*1461366495', '61789248.0638*1466382180', '25743007.4215*1486845540', '67079347.1143*1485688156', '70033600.1509*1467129558', '73925462.4339*1465686783', '56354274.7926*1466819824', '4278693.08682*1454610523', '64512136.2009*1473360884', '35702005.5204*1451816310', '87885272.3448*1487395937', '3811016.02964*1464462524', '30808169.3235*1480335965', '45536795.9771*1476517885', '97626373.7479*1482367075', '41876734.6574*1487782094', '85836754.2241*1481467632', '73906637.4862*1470767268', '80672312.6522*1470403405', '12658576.4681*1469703201', '71467898.0859*1464793236', '92436790.5754*1467962861', '35899556.4609*1463755986', '28030217.2814*1466454739', '60228291.0066*1464785291', '12370153.2352*1474703191', '2777550.09469*1487812677', '72879583.9435*1452578263', '94725881.2706*1476662094', '14342689.7833*1481891773', '9153347.79816*1488036164', '89483321.362*1487070185', '57160314.0107*1474495132', '17129914.9854*1480666341', '4688709.35791*1467115474', '70120540.3804*1478031712', '21848553.2456*1453163186', '71695324.2886*1476804954', '67176544.0812*1474712999', '74122304.1579*1479097701', '73577624.4154*1470485675', '2958062.12842*1456807495', '92583584.8566*1468900195', '47035257.797*1481696067', '79168754.8446*1485871297', '13975008.8455*1461719422', '97430814.8551*1454690832', '54146890.4139*1487806079', '24887432.617*1459444108', '83368744.379*1453033290', '32016772.6368*1457307473', '89502816.4258*1480977436', '97006294.3254*1486338843', '90929929.7799*1476523072', '44967703.1079*1454871449', '13506552.53*1472902690', '96949229.1948*1454113546', '33509829.5008*1480723382', '67640048.7075*1461517006', '60329085.2092*1482138241', '31354239.6172*1470566260', '76806677.1158*1457384524', '28268632.6396*1464014039', '90091832.5668*1463471606', '42783804.6238*1454805916', '97218623.593*1472493755', '27618864.7202*1468097464', '38424873.41*1484451054', '15352759.4492*1476924835', '69386058.7551*1465696691', '82827889.2413*1467981874', '23238658.3875*1461283377', '37568712.195*1471913897', '80554669.7637*1470518612', '8943254.85235*1486602269', '68113475.2561*1459826717', '3206282.15596*1470109871', '77695337.7596*1473968722', '57681431.3852*1464192943', '82448346.6613*1470231906', '40874099.9101*1465229444', '58484088.2076*1487641471', '83862067.8805*1487083690', '51169613.2716*1479696532', '78368167.333*1483616470', '84189361.6613*1461140170', '8510062.84529*1458934467', '42933485.9452*1469623267', '44923134.0054*1451994509', '19037622.8825*1461692075', '7800323.30109*1471421594', '63925621.6514*1480653099', '61711702.8476*1466381495', '42518355.9369*1464391526', '89576942.603*1453432440', '38292538.9471*1474270077', '54820223.2694*1472966939', '35606394.8194*1476935974', '87171401.3258*1473062706', '74283069.8986*1458849935', '92223054.588*1487502960', '20283991.2883*1463586311', '2461889.6874*1464068895', '36451791.7306*1479378400', '72363069.3299*1454424539', '67482359.4288*1469363547', '13227495.0924*1457330292', '86666437.1327*1467121344', '48627151.8947*1479896072', '23238658.3875*1461283377', '28910002.6492*1475263897', '69080660.3791*1464344767', '54830722.6299*1474103560', '23254586.1626*1461352506', '1716172.8979*1483280629', '72387851.2622*1479419815', '31865381.2148*1487073215', '14797228.3711*1453676312', '33080614.1083*1466552489', '67055192.0593*1487032827', '51928152.316*1483113273', '41355257.4249*1476122333', '71004574.8268*1477312660', '59212298.2643*1479157145', '84764316.9749*1485461790', '23646581.4931*1456310033', '93378933.4654*1467276092', '94273102.9855*1486366916', '46395152.1128*1464296678', '40067889.8285*1452633689', '70481429.6765*1482149702', '11891597.486*1472127565', '35170806.8219*1466005374', '69931094.2813*1455204975', '68541325.4715*1472696548', '26872436.575*1457359893', '40189516.8117*1452798084', '83898762.7133*1460300122', '92663432.7001*1486332399', '65370246.2963*1453605272', '47021763.7662*1466288109', '1919177.12206*1452687642', '8491665.05389*1470767311', '80133163.0426*1455584371', '8613039.52132*1452793336', '36944818.3768*1464506639', '60936179.9818*1474421352', '88556416.6689*1451642296', '485485.584361*1487595935', '88051770.8975*1461424531', '84159696.8331*1457476803', '25892096.4317*1453218526', '52051558.6886*1487548766', '71254900.61*1456402979', '40030225.2612*1462840610', '14837044.1845*1463196663', '63062971.9896*1466698089', '17457155.278*1474812892', '23947051.1027*1465478038', '51914086.196*1460196279', '34805216.4496*1464771270', '8480013.09792*1474684805', '35296499.3443*1467156992', '68745442.1838*1484666506', '89883417.7765*1483767063', '98308803.1085*1482284300', '17386785.993*1472943217', '57472365.8117*1484345865', '91284914.4562*1452570571', '44079767.9571*1464431015', '70481429.6765*1482149702', '14542122.4351*1470062246', '42542169.8532*1457744397', '69815271.4807*1480527782', '92082672.6792*1473807209', '4686147.77692*1458990609', '87904879.3353*1459270738', '37523258.2092*1484026040', '45144433.6515*1461880187', '94892841.2005*1484705919', '61939197.677*1460566109', '23699812.8185*1484283567', '46086550.6728*1476404894', '55614185.2795*1469727643', '64740221.3598*1467130709', '1585303.8512*1461410549', '56527905.2039*1464957443', '17287026.2407*1469743919', '18242355.874*1485894819', '89502816.4258*1480977436', '88735224.264*1452581752', '53914011.7579*1488224292', '7783910.56224*1475580754', '31167721.8031*1459056940', '14785188.0475*1475289657', '36058612.4287*1465050879', '46808161.5729*1464553949', '41901810.5389*1482286786', '42868796.4507*1456126595', '37401149.0392*1478666824', '53688836.5169*1476847928', '10036070.0533*1467332183', '51060977.7494*1461608246', '8295521.53069*1485300062', '77934772.8426*1484242512', '26253450.5093*1456709083', '99133867.1033*1464044953', '61922271.2489*1476121429', '62285077.3053*1453870486', '75606499.036*1454053805', '93358315.7953*1468437640', '43050867.946*1481505937', '38808003.7417*1460496018', '41892082.5761*1458601233', '25079544.2871*1460441625', '64399117.4507*1456450203', '21646831.9998*1471002881', '2245703.80403*1474375645', '20055190.2636*1477768468', '78838933.2346*1485132810', '78122934.4525*1467918347', '93250674.0404*1474110948', '49475025.0694*1466093864', '97882197.3289*1466260849', '77756172.6834*1478092129', '95983311.2653*1464504221', '65493758.767*1480217559', '45978298.236*1482100476', '69909382.7792*1479943814', '56463250.8344*1484660064', '59673433.3308*1452424028', '49708890.6881*1486785116', '46105294.4611*1482100215', '60318331.2816*1460992972', '23667021.9377*1463591804', '54562159.866*1476101659', '80273107.9661*1465088686', '99534064.3341*1483565251', '39510901.5206*1452879203', '80916575.5304*1465230292', '88265223.2986*1464748965', '74853907.7543*1458042302', '8530512.89094*1478586028', '99601299.4764*1472664766', '70667982.4033*1466338585', '80273107.9661*1465088686', '19812357.2162*1457022868', '15116310.466*1469025060', '4651514.61774*1455695958', '43881462.0366*1453007746', '6586499.1257*1479076145', '89576942.603*1453432440', '96484317.6792*1459077300', '84776834.7014*1487743916', '72646623.4494*1471228354', '53238066.2164*1461516477', '94561594.9915*1456422910', '9019925.82208*1464668906', '87223296.4388*1483391984', '36728383.3348*1483814052', '8843073.9451*1461385606', '32012644.6602*1456325461', '37950791.5473*1471920770', '67482359.4288*1469363547', '22570694.5192*1477913894', '3688522.17078*1470329879', '95453970.1964*1480894688', '38564451.4254*1462627441', '83089605.31*1456746316', '98834498.0871*1453700935', '9381825.4506*1480259083', '38464729.5197*1455751592', '94748821.5496*1482790176', '2745956.99788*1454776876', '80378205.0409*1457975195', '49555908.2145*1475492217', '22590534.09*1471172968', '15087028.8003*1459392381', '12204325.8971*1483682746', '13399430.9646*1462932435', '80708894.578*1467597322', '82280051.203*1484397073', '68339054.0181*1456769911', '38061238.4207*1462258520', '2527150.91832*1455891256', '55959297.6985*1452815816', '32222515.3173*1471117663', '11577944.8804*1454438746', '46304384.0323*1461104848', '58000902.8023*1473884505', '29182591.4551*1475513729', '76348332.5797*1479578760', '87030304.8501*1475412214', '23309843.1089*1467799415', '5961540.61565*1461002487', '86514653.2632*1459418462', '47051905.4333*1476899716', '96332050.0521*1465531163', '8019312.86798*1466353797', '68864635.4072*1474808515', '53883583.7558*1478242655', '72484561.231*1478467686', '79449292.9874*1486363777', '14247936.7799*1486623148', '49207985.0055*1457584969', '18291694.6931*1461741292', '2409544.44377*1468351732', '69553267.5327*1477122241', '77133785.4574*1483059504', '63186033.7549*1465614206', '66032014.8299*1476698091', '55322385.2197*1459888776', '15878243.1326*1476350423', '82663448.5507*1473272789', '28452914.6865*1467415031', '2452353.592*1453273281', '11311505.5998*1466613744', '51910807.2755*1474503693', '37834452.1852*1467049292', '62546980.0255*1468097172', '82630561.2889*1461911466', '66358000.5532*1453522238', '77695337.7596*1473968722', '47746049.2571*1483286160', '89434043.9695*1453801809', '5015743.26828*1471823078', '2449102.60525*1473200307', '17492475.2893*1466582118', '54562159.866*1476101659', '66302503.5249*1467259002', '8994401.06241*1457553896', '36224245.6699*1484860932', '46774866.0261*1481732023', '89009508.7255*1484800332', '90085340.1668*1469268298', '51592140.1193*1468036739', '31182233.709*1452007063', '47599599.2091*1478608805', '31451897.294*1453955830', '95772711.2791*1476344014', '52283585.8889*1457441256', '6667726.26671*1456392874', '60353233.6116*1459885089', '45309575.41*1483780515', '2024723.45488*1480475220', '47472100.6359*1487645551', '89314985.5418*1467451083', '77165691.9044*1465698086', '76485387.5309*1470194613', '13205458.531*1475420105', '86693651.8919*1460672579', '82346493.9102*1476297236', '75881205.9038*1459974594', '36847643.0572*1486086586', '32478322.5091*1452047038', '34156907.4423*1454413032', '64140398.4437*1454500577', '39429662.1648*1460913122', '68447760.9715*1466366160', '75963407.5951*1453212741', '82497802.9915*1468403389', '56160709.1619*1454153688', '43094339.8752*1474879917', '78163302.5649*1464678900', '51222884.8239*1485529957', '19506765.0346*1483749508', '28770485.9873*1452168758', '58886892.7727*1472728769', '1437847.06962*1479938216', '95018573.4784*1483874687', '77572352.4201*1463652756', '48649360.1472*1480155325', '91397403.4764*1488319813', '1927619.16497*1472279708', '41370745.5245*1468929555', '12815703.2658*1477051689', '43441577.7298*1485095680', '92117404.6677*1483182264', '27650952.439*1484848259', '60965261.8197*1462006190', '78403119.0534*1484111886', '33974381.9013*1454861709', '68366268.9746*1479086933', '71654727.8695*1474485651', '90340250.2364*1488254769', '91600544.5399*1480325502', '40628876.9285*1483951242', '26797295.5614*1484048061', '22570694.5192*1477913894', '47142579.6302*1470725646', '91979737.406*1461820557', '78464792.3116*1461805615', '44615817.8495*1472837555', '37807284.4531*1463791369', '71050828.8975*1458010329', '23993436.552*1463540745', '69811188.4986*1455389623', '69392853.5791*1472604281', '47964516.7344*1484920508', '25115257.3353*1466221684', '87030304.8501*1475412214', '8072315.50501*1460109789', '83828336.4107*1466860466', '4142545.25758*1463973604', '17118191.5471*1484831578', '5811372.39504*1484847491', '28578096.8142*1458870498', '26007945.8249*1473454468', '43191438.2153*1458652912', '63009793.5858*1478386938', '20558805.9063*1481556993', '74391406.3312*1482688625', '8745150.29088*1468021301', '61895412.152*1459809099', '42419414.7487*1477291496', '72483604.9693*1480509028', '64248642.9178*1468954112', '7741394.1709*1480624493', '93374682.427*1487884387', '56465672.3321*1465648715', '72644985.1453*1484275257', '95759164.5545*1458005643', '85902096.7847*1459953327', '31487046.6451*1460642155', '1890482.03476*1455532555', '76737130.5401*1465468862', '7098238.89263*1465856667', '55623739.6889*1459823390', '18095321.8953*1476247660', '48522517.4732*1475853267', '19474973.1269*1452823522', '80211991.8731*1453165799', '38795865.6123*1476372489', '90167656.7667*1483692105', '52273651.8134*1470501176', '24044070.0732*1481982416', '98815181.6567*1470121486', '15763665.3223*1456892618', '38984712.0692*1452125095', '5331163.89834*1458196268', '55648937.2709*1475491203', '83340663.7688*1456503886', '39401308.3473*1467621277', '67482359.4288*1469363547', '12628596.0151*1478179698', '13189839.0356*1461917734', '65905314.9249*1485208165', '41791311.4333*1458624241', '33696783.2092*1478078169', '90309757.0232*1473912584', '38174045.6296*1472736532', '60948559.1326*1486245497', '59899615.353*1467180321', '20061124.9501*1475378684', '85684725.4726*1474158991', '84887158.2245*1456081366', '10795384.7832*1454208886', '81974861.0191*1465631420', '52477128.0519*1480952031', '62423339.1625*1471835445', '97659575.8041*1462179459', '24986921.8428*1454991381', '84890286.5719*1473866754', '10452745.9081*1464481357', '26797295.5614*1484048061', '85985595.6077*1465897402', '77550073.2566*1468568480', '39199684.1359*1457533850', '78691891.638*1481032514', '47520153.2827*1466870854', '22726649.8241*1454318265', '42053605.9571*1481336532', '40859007.5134*1459873358', '77044100.0759*1486746610', '960806.454337*1455575812', '51640147.9739*1467536119', '48795230.4285*1473818160', '27109993.9452*1463211979', '75415478.4253*1469080309', '49315144.6711*1461696297', '16847055.3516*1465136534', '10452745.9081*1464481357', '21830071.7802*1486908153', '96696547.0994*1486260979', '61702967.6411*1481148755', '31698118.7637*1465484059', '60491602.5134*1488070752', '42884037.5442*1463663972', '46985677.9552*1476212865', '71254900.61*1456402979', '80071769.2233*1482812553', '55322385.2197*1459888776', '68918565.732*1459696369', '60614356.1461*1473547457', '78464792.3116*1461805615', '62825271.0591*1455378859', '42353599.1188*1467632537', '88655978.2096*1468120668', '73831864.7902*1471340943', '27896884.558*1458859746', '80808115.4987*1464745850', '75483328.4831*1471687894', '6274813.43582*1460254622', '63155626.4101*1462401003', '48540084.7388*1484936589', '43895866.8295*1476023782', '63257713.3861*1462573466', '21850095.4484*1459776343', '26600710.6909*1464540588', '37565195.6159*1463122189', '93409788.6115*1465654334', '92414842.7975*1466487602', '20421341.6699*1458789930', '1798285.06847*1477571985', '21395302.0003*1485344728', '84330491.5591*1477344285', '22624244.6696*1466580876', '11867264.5719*1479304008', '14794618.9342*1480665171', '89841270.2565*1488029395', '66108349.0964*1472802889', '77861700.7206*1469122248', '59045021.7904*1471586467', '64212953.1749*1480475199', '98026745.1573*1476395285', '12975521.4471*1477279580', '26253450.5093*1456709083', '24045525.869*1480454787', '39706936.9128*1471575259', '94339722.3612*1485205548', '43652207.3481*1483233191', '71654727.8695*1474485651', '60177092.4569*1478098317', '36206435.5095*1478357746', '45025815.135*1468692367', '19226432.4153*1473400507', '83908758.2238*1464044849', '69170779.1374*1467567992', '80897645.8983*1465761699', '20630039.8755*1479872717', '98954066.3207*1470639642', '9678245.34289*1454335269', '34865281.6999*1460053950', '67839014.6885*1457265420', '77900905.2077*1485879471', '56841625.1852*1483592994', '94559124.1666*1483599265', '22330745.6666*1458884024', '38497611.5518*1487075326', '49057804.5841*1478956693', '49556650.2074*1480952789', '20243782.5944*1466270950', '5559825.92341*1458081844', '31353195.5415*1481539497', '87227186.0264*1485263422', '62955844.9563*1472544959', '12926454.7262*1485395279', '36314188.6874*1463249637', '96646083.6183*1479427490', '60365967.9401*1462661383', '58529578.9013*1462003627', '98825701.2758*1482804587', '12550044.834*1480183336', '79838284.1033*1463715663', '44827787.3759*1456854665', '32303443.1235*1464789777', '87751882.5046*1462815784', '46122582.238*1456813556', '29222333.0249*1461960956', '97424325.7662*1468883686', '99235085.0878*1465986443', '67891816.8247*1458974874', '47217432.0883*1469776948', '30785508.2772*1452968455', '85342048.0236*1478112268', '81139663.628*1452196091', '70701628.4737*1458564745', '44816658.4346*1460670046', '2191423.64814*1468968537', '8414078.08034*1458785663', '97822728.6036*1486183100', '14740536.3759*1479234164', '28455076.7912*1457666502', '72132289.8662*1485439095', '32405014.1491*1483948730', '65103379.7835*1464141833', '76370743.2467*1453513492', '41622006.4807*1474446745', '68880977.3425*1454957526', '47499280.9788*1456109496', '76635791.5994*1487851132', '38864753.2886*1470966262', '2100825.19281*1464753577', '11868578.3374*1459177715', '83914144.7895*1458531196', '55340828.4761*1485080461', '82254845.8658*1476785604', '68540107.3619*1458595878', '69823054.0044*1483259653', '16873843.1182*1457075383', '33543898.1486*1457652779', '2942121.71365*1473037389', '85069593.4684*1469463928', '37735313.901*1482206548', '75286154.7372*1459867139', '1920990.02867*1459229398', '53562217.4205*1470812899', '95772711.2791*1476344014', '47986835.9358*1478396539', '26197890.4985*1466181736', '77738051.2996*1467443136', '52817404.3192*1459363878', '68817665.5134*1456251757', '29542659.0446*1478234591', '27239089.2535*1466574574', '9439599.56143*1452753019', '70836345.6032*1457377052', '79883274.672*1456446767', '968451.415474*1453294928', '67180278.4511*1486629509', '65794125.5879*1475312323', '95675420.4103*1463089812', '83964594.4564*1460200672', '74146962.3701*1472952230', '15887556.5598*1458456102', '19858830.168*1462077436', '52561012.6799*1459392886', '40734097.4445*1460952109', '42860554.0821*1483947780', '64726925.4378*1475860032', '76834800.4869*1477562953', '18603102.761*1472170929', '64043874.3311*1482682067', '56966641.2927*1474525751', '41195778.998*1473105204', '92026709.9324*1468407061', '76154642.8273*1487818374', '65445341.2391*1457094160', '58409182.1099*1469400943', '8530512.89094*1478586028', '89440139.4876*1460090385', '76784609.0776*1471194511', '28369019.9223*1474803041', '28564515.8398*1468003801', '75081279.8567*1468409114', '27635023.3801*1485293729', '6580155.44484*1457634565', '10957384.5896*1478295247', '95661815.2736*1482199372', '45773044.542*1466476765', '85070530.4653*1476526588', '69979762.976*1460028125', '61143973.0371*1464598249', '25054456.8884*1484991777', '43011748.8761*1475982160', '57518197.9667*1455473892', '70642978.705*1480561019', '53328486.5056*1455769501', '38622441.0914*1453890445', '72702557.7642*1473339304', '6752644.13003*1469317311', '38875505.6026*1470719916', '89253002.9162*1476901325', '69374256.949*1481911514', '69087973.0136*1457676654', '65998005.1491*1478678871', '17738940.7876*1456535235', '79655618.361*1457755035', '67176311.629*1453064064', '48440507.0386*1467438272', '45042294.2298*1469044780', '266114.627659*1468479109', '38412174.5815*1464080759', '23623267.9177*1479692073', '32441001.8613*1463520565', '62077681.1885*1487817593', '21582268.8849*1471402615', '66451221.2487*1476103846', '99630370.4403*1483286264', '31244141.7381*1462872571', '12014857.3604*1487428187', '12624196.6715*1471049218', '48336899.2536*1467541141', '15391361.0048*1462955185', '34630140.2902*1487465120', '69828575.981*1480494802', '54537568.4498*1481132549', '12301399.866*1481704241', '59425855.3046*1475009896', '86410068.8479*1457109459', '47663962.2652*1474961540', '40189516.8117*1452798084', '30698916.6255*1466485105', '24651449.9565*1486788917', '491365.535008*1462594050', '46555903.039*1463111804', '3914782.62886*1453256965', '66233912.0816*1478893766', '69958003.3785*1487450650', '18393630.9454*1482111604', '86804342.8251*1459730023', '35052325.1513*1453206175', '68540107.3619*1458595878', '65923087.311*1462872288', '77353288.1388*1465921496', '1759411.8348*1487073712', '34645548.5156*1460681485', '14872488.1535*1460948113', '4937758.87166*1455285711', '57004655.2947*1475809824', '89162751.4275*1471921160', '21319893.7833*1476836470', '31905725.2591*1452073276', '18029961.7564*1483786875', '708537.35258*1462123795', '35014034.6251*1483122267', '78193436.7757*1475807191', '96016962.5912*1466984819', '32205234.0881*1470004346', '47000955.15*1452287606', '79061957.986*1481955209', '18977827.7612*1474932788', '56596454.4977*1472012045', '55476891.911*1473638473', '1391922.82416*1485224317', '10275515.5466*1455886292', '30694921.6906*1488190209', '32878189.3824*1463031204', '92816253.4812*1484821724', '46105294.4611*1482100215', '34056905.6237*1456247725', '29834088.8104*1486225852', '84323291.5266*1454751380', '65585537.321*1485702152', '53445880.4785*1464320942', '88389351.9282*1465449338', '43108455.8371*1476453778', '11967809.8954*1481862316', '67081582.8975*1459616539', '50355147.2582*1468487480', '20350011.4469*1458297813', '273920.33823*1453361398', '82410655.5795*1486972113', '52873155.9516*1459259390', '31558302.7706*1468433077', '42315805.586*1468473766', '71245171.4418*1465097444', '49449338.3774*1465931088', '60168629.5006*1480699008', '20120187.8479*1456397849', '50092943.5338*1470655978', '62399434.5143*1463348123', '93325422.1372*1469477901', '55660237.637*1470966514', '24899504.1657*1471856824', '14364820.0774*1461657206', '46544588.1115*1456404959', '98874581.5531*1469212867', '79631388.7298*1478245803', '65249045.9106*1474993259', '57661397.9958*1479897167', '51910807.2755*1474503693', '87931648.2069*1465043539', '66472689.3181*1480373441', '71025045.6437*1487841447', '6448220.39189*1469994470', '62207621.4091*1480289842', '50307389.2958*1466306977', '56003131.5986*1473930591', '18131967.4788*1486903320', '44751791.8417*1460605060', '46813455.9896*1477333863', '42580802.1837*1474988425', '3547869.0042*1476845374', '83898762.7133*1460300122', '18765545.5136*1464549288', '60353233.6116*1459885089', '78596930.3223*1458792211', '57928485.6762*1486356856', '33077499.035*1469363628', '2376255.49556*1475612155', '18239195.0579*1483684868', '71842768.6835*1487100107', '20647363.714*1486717521', '5692167.63992*1454901947', '59592441.1239*1483815357', '5015743.26828*1471823078', '33654250.7442*1487136179', '38967732.1801*1479854206', '14994009.4493*1452635710', '41003523.1376*1463195840', '37439923.3987*1476153002', '93942436.6333*1474940736', '27610179.1975*1461152456', '15775223.08*1458158480', '19536498.2227*1480959292', '63152161.653*1463770557', '65577516.6791*1479644469', '51455122.7032*1471013001', '95656048.4572*1469007730', '68222507.9983*1453631116', '81581260.1408*1463501494', '15974169.8845*1459184170', '19592175.1259*1455395665', '20524051.2489*1452495313', '81419881.6467*1452821166', '18242355.874*1485894819', '786679.375876*1455453508', '84995797.454*1458983663', '61285034.2229*1472102023', '95036757.9133*1461114030', '87498949.2435*1472419499', '72620360.8586*1455822710', '32012644.6602*1456325461', '56262583.4617*1476230469', '27618864.7202*1468097464', '27504450.9355*1463977294', '10725079.6058*1452855899', '63873821.7154*1457386138', '19580969.7453*1472171430', '76912777.4137*1453671680', '98825701.2758*1482804587', '29053323.0756*1487848669', '87412614.039*1468803396', '28629241.3938*1485301421', '46846758.3424*1477208853', '57415590.9796*1473571235', '52603717.7102*1463036487', '44800059.3528*1486506037', '62253825.1981*1456154434', '47316821.09*1488138834', '66209247.325*1456084286', '14642006.9688*1484977679', '48029243.251*1471837505', '74028207.8818*1482418666', '19881849.5181*1482774563', '36033587.7984*1480889112', '91391357.7442*1465977967', '4385265.35825*1479134290', '69911776.9596*1477411388', '3965662.76881*1473719687', '52477128.0519*1480952031', '19878684.7384*1472067351', '72628900.3331*1484064406', '29922620.5123*1456997263', '1979133.33152*1456639374', '30427395.1217*1481224573', '93506364.1729*1469116954', '45205482.4501*1476871808', '85070530.4653*1476526588', '34106307.8046*1451921618', '40983612.9494*1459175940', '7985525.74028*1482665100', '31802292.1331*1479795956', '94476606.9243*1457930957', '65075305.1769*1480169582', '64992360.7121*1463100685', '89900780.1339*1485138228', '88265223.2986*1464748965', '73010313.7449*1486063842', '7932744.64129*1481087894', '94988804.5531*1470723659', '10891560.5334*1454774360', '77511569.9675*1455786136', '30854169.5889*1487270433', '13171976.2871*1478377784', '47990940.7714*1456698124', '59375725.4624*1483099344', '52160349.6746*1470390062', '84423067.983*1453049807', '87491193.358*1453251516', '1314505.85528*1467883843', '88438112.5281*1469150441', '74527504.5007*1462775199', '80103154.7019*1455011818', '13981596.8997*1453597829', '75084136.0517*1471329238', '43386544.7408*1467211156', '77070217.6112*1467713068', '1848869.93776*1452947627', '60220786.8947*1452372730', '77721839.1246*1477752696', '2723596.25911*1459715329', '14258801.4867*1464952201', '80920210.3492*1459850388', '96720882.2312*1488307814', '38869401.6999*1453563835', '41104188.5104*1474369793', '40471741.1488*1471447711', '37101168.1865*1466832707', '81390244.1721*1458826916', '98152400.0264*1478441926', '526223.131771*1470564684', '30303971.4507*1466794692', '1716172.8979*1483280629', '22555464.0175*1473273269', '80832108.4643*1477705740', '49945185.547*1457837762', '7297720.84102*1486494234', '91934984.7474*1454436760', '5142644.70549*1459466962', '65161324.7601*1456977037', '91719507.58*1463231510', '72879583.9435*1452578263', '27041774.7687*1459578491', '85809851.0905*1474132289', '39419717.9403*1457863269', '68294832.7065*1453177484', '73801407.5113*1483198389', '73589602.4739*1476770151', '74449250.0478*1460107953', '41522398.5766*1460297005', '41781499.7678*1451940407', '48336899.2536*1467541141', '73526371.0532*1453033841', '24455568.8636*1483051161', '89012091.52*1457813906', '20455618.2622*1453022075', '25487121.8686*1459714071', '29770090.0387*1475244707', '35719453.9382*1469686278', '30196116.898*1466879965', '92631508.3295*1483085803', '46916566.8455*1478881215', '20265662.6637*1454253916', '35739280.5049*1487181702', '3625856.28175*1466775364', '63155626.4101*1462401003', '34367562.1398*1470247222', '95961486.7419*1457504351', '41025019.5384*1472291782', '4449273.57075*1476346187', '32066973.3164*1470839853', '67316550.6795*1480613661', '77070101.1854*1451849291', '27014709.8923*1454925128', '57033374.7588*1460315268', '71859685.9914*1482428479', '83001336.7853*1483943235', '92074180.6201*1460896173', '92384025.4056*1465534076', '49974907.9385*1472264510', '35266378.4275*1456418962', '34424968.4066*1468943966', '98146815.2421*1481905788', '26536680.5038*1483508470', '31330626.4652*1468582835', '65811699.1054*1458906241', '97009803.32*1471930726', '38429051.1097*1483320859', '87454284.1735*1467165688', '57007382.765*1459752182', '35691143.6449*1453397681', '94299933.6473*1467322448', '85902733.6903*1463236431', '5490974.52051*1463709859', '54010578.7447*1476597522', '92748986.6879*1481645621', '41384359.8685*1459634287', '83361740.2642*1465686239', '91624062.6638*1472983704', '35458607.6301*1472134804', '69756322.7316*1488100331', '66346746.2436*1472390407', '42678397.402*1451969401', '84239421.5313*1485343124', '16834855.7317*1469529922', '23719743.4015*1487653008', '15849289.7386*1480711609', '76140570.2824*1464593209', '63683686.4715*1466518183', '8534328.78767*1456356935', '65057205.1709*1487437955', '17321466.1412*1463951459', '56435843.2432*1487427072', '64962667.5946*1452653775', '12605832.4815*1479663924', '31763746.7088*1469679675', '64759636.4461*1470239313', '23947051.1027*1465478038', '26185178.2001*1479553311', '67103423.6782*1473207817', '63617979.8228*1453475784', '16907493.6664*1471382893', '23830110.9722*1474074638', '49599860.9224*1488295605', '26069511.1083*1478202980', '12670250.1674*1484064550', '88556416.6689*1451642296', '12958520.6294*1464827550', '40290464.3775*1457416827', '61256713.6453*1485594597', '48948388.7366*1465449276', '11580070.643*1452794792', '10159965.6991*1465419075', '89585306.1458*1487662683', '8909140.60055*1474840664', '11808362.8485*1456247047', '91507734.8774*1475004617', '77618936.4936*1460842041', '71023458.5251*1456853934', '92094882.7712*1467480272', '60956449.5503*1452766808', '77076798.8053*1453708867', '91977121.9474*1453109334', '57301422.6779*1454442500', '56626467.3335*1481962075', '18756738.2523*1470725539', '27504450.9355*1463977294', '24021656.1386*1463424789', '99530822.7292*1484919061', '79785043.482*1485502937', '63878838.0383*1463631463', '57007.9584623*1485128752', '17494498.7696*1463172567', '26085460.62*1467465626', '23791642.028*1464379353', '61190843.5106*1479427103', '63152161.653*1463770557', '50731375.2088*1475174877', '86632399.2676*1452921598', '12419506.3528*1476587473', '71104760.4268*1486534487', '12550044.834*1480183336', '6753814.72977*1459446726', '62215795.1607*1480816664', '35726243.6663*1465590195', '85069593.4684*1469463928', '16227519.3201*1479648048', '76383971.2818*1475513648', '84498118.9018*1452965865', '20099871.11*1471468136', '83067643.5096*1463240347', '25208910.4839*1462935809', '56091502.0712*1462345726', '91532627.5667*1465708340', '10029953.1016*1480405261', '36077199.7731*1457815336', '43084217.3427*1475708832', '78214805.1995*1481624122', '40841735.5868*1454133166', '68721710.0389*1452128987', '71337066.3597*1457697219', '68053164.2*1470744941', '75963407.5951*1453212741', '23369111.8971*1483999357', '4855355.44281*1481195513', '65264045.5562*1459557827', '48790165.6763*1459795085', '52120970.3834*1454628770', '89434043.9695*1453801809', '30372407.957*1472739110', '88417876.3497*1462850667', '17704521.602*1461864636', '53198405.5425*1457575905', '46445489.5315*1479273324', '30327193.6972*1478055299', '59180167.1258*1458015127', '21508113.0782*1472865599', '90681584.8874*1487350889', '69404473.7719*1473107521', '71752723.8042*1475425901', '91636711.8385*1474710722', '31732789.1555*1472061220', '13295375.0686*1466627385', '39042647.7608*1473316510', '37786691.5657*1486842856', '92382731.6621*1461604252', '14393515.9834*1472989654', '14157231.6025*1479433223', '82253699.5251*1482716886', '2091402.41406*1487353203', '7191412.57471*1458165031', '43958197.1602*1485306756', '54455212.8147*1467847757', '35650218.0339*1462931382', '17417485.5484*1465518865', '48710406.0087*1476111242', '26777535.2361*1457224472', '7218885.9484*1472564614', '979989.676716*1457370855', '62185852.969*1472385965', '57928485.6762*1486356856', '30778535.3319*1475389646', '25457689.5071*1488299995', '25975500.9628*1482163658', '82084522.3582*1462305858', '25609866.4749*1468906794', '18104718.977*1464072950', '6316501.93094*1475695941', '57057881.6987*1467151751', '26585896.4574*1453278609', '79424893.3919*1478993462', '99235085.0878*1465986443', '60282642.4241*1480679178', '65527950.9565*1457714920', '1357596.94667*1462996371', '47344535.9664*1473766712', '77494867.2665*1456530231', '15323275.6307*1475062797', '13730855.4481*1457065528', '62257408.4154*1475253827', '41612941.3265*1462806860', '62806039.4796*1487236019', '23839263.5193*1462583101', '16479688.0125*1457013670', '90597420.3661*1464575049', '70404525.5811*1468086059', '60790043.0177*1467691249', '17053774.9279*1484555386', '78392171.1577*1473301889', '32019019.782*1461344593', '71566885.8567*1458417780', '98265031.1566*1480677908', '59065017.7166*1467296318', '49945185.547*1457837762', '19168467.3839*1461110093', '76737130.5401*1465468862', '98618098.0277*1452559315', '5220534.27577*1487208431', '87036484.8466*1468306571', '21830958.4098*1472871180', '64059074.6966*1472839731', '75901441.2274*1477502384', '44647038.4922*1484865345', '75131445.233*1483077120', '59495640.9972*1475429256', '43372611.5102*1484012550', '12351787.527*1468111749', '85296407.5546*1460887307', '85661835.8602*1452856501', '44342202.5256*1467667562', '15321435.0967*1467643383', '29321058.8284*1460667729', '18093164.7075*1477187875', '75928827.4693*1473010629', '31173652.12*1483123984', '69828575.981*1480494802', '66488838.0112*1472554597', '38887644.1068*1451866109', '31010351.8388*1481549381', '87171401.3258*1473062706', '11812910.8249*1455364167', '90758827.3044*1485031171', '28228057.966*1487437414', '74431095.8182*1468435324', '96014907.1285*1469174490', '76410399.6558*1458124061', '74937296.6416*1453185203', '17300899.0335*1482097169', '83525259.0324*1466760845', '85564503.0131*1483969235', '44606646.022*1457699479', '47992239.7251*1476916676', '71520602.3013*1478430725', '63728756.9081*1475452527', '81787690.3366*1453900665', '12128853.9078*1478227392', '48029243.251*1471837505', '67166825.8986*1485149042', '2723596.25911*1459715329', '62005349.7451*1474305259', '80615046.9165*1459932043', '48583861.5549*1477335944', '1285384.56237*1480182133', '59226781.6957*1479341025', '80393477.5514*1460976527', '26530276.6658*1475711956', '32322101.1744*1475248005', '10734303.6865*1473924244', '73484246.2875*1466278895', '71886237.924*1482562775', '31668796.997*1466825018', '87264957.6093*1467381078', '81673187.6108*1474503740', '46958157.4771*1472610448', '67241130.3826*1474675300', '65570622.3984*1466503129', '85930799.7503*1472689199', '85390899.6502*1476545182', '56391918.4998*1460303726', '65951605.1197*1473527218', '4686147.77692*1458990609', '28369880.6333*1486697713', '60944446.8792*1487496057', '83307989.6938*1475222113', '56818152.8136*1455690819', '52355396.8338*1476346886', '31244141.7381*1462872571', '55960297.5488*1486161089', '39579252.1888*1469208281', '58482484.382*1464004286', '4648439.7244*1453816126', '66715410.9943*1477060798', '38290100.7862*1475819131', '98134135.6138*1483302281', '83908758.2238*1464044849', '73146839.1291*1476092722', '6833000.89482*1471636115', '17575914.2154*1463721299', '24910096.4222*1470231043', '46758094.3895*1470711621', '13627635.0612*1456180387', '7269784.24235*1453547182', '22216755.6652*1470632623', '44162846.6367*1461166422', '85878825.6576*1469659465', '69509699.645*1475124450', '49135251.2447*1484868100', '99828282.1016*1459064337', '34626346.9806*1479253117', '99354694.5914*1458947329', '92117404.6677*1483182264', '33209952.8525*1484216773', '78067560.4004*1472631049', '36539717.8984*1482335200', '79258722.4237*1479937899', '29030500.4251*1465572790', '27795840.2522*1456193229', '41999515.4055*1465533840', '49345624.1275*1462780120', '41056200.8042*1453688183', '27571351.6141*1463125313', '75651645.0523*1487044041', '69659516.4118*1481885121', '59583485.6535*1468996470', '46737190.7154*1469388993', '36257355.3934*1470709933', '33225707.7061*1453789032', '79434608.0601*1476861196', '71735660.8973*1476138972', '58754835.8185*1458479561', '83548048.1231*1477617645', '8182620.68458*1466084483', '75872113.5404*1467817788', '60288992.254*1465770805', '4136457.89866*1479096456', '54323520.5362*1456507272', '45114421.9998*1465173985', '15116310.466*1469025060', '37568712.195*1471913897', '32552837.57*1472140697', '62546980.0255*1468097172', '75357890.7976*1456288436', '98339795.5973*1452566959', '57172144.811*1454520979', '19700181.2521*1452975753', '88478032.6804*1462373810', '55211566.7535*1469625789', '95805685.6325*1486083885', '24957950.2853*1485567817', '88516534.0403*1461192340', '60282642.4241*1480679178', '9104705.17748*1474093279', '83583775.975*1475984852', '8952980.63814*1478513778', '57362473.0997*1462734014', '45599355.9552*1467804760', '7493565.79335*1467093514', '33675171.8148*1465451360', '5148243.02744*1466712092', '42885657.4896*1468744139', '29880885.7345*1475377760', '10465371.8965*1466724230', '28222504.2195*1454710290', '60281188.5817*1455459294', '56330663.3521*1472231192', '98924304.2606*1482819012', '1665302.13195*1484546797', '57564528.1337*1459321557', '68972140.7438*1475785779', '53777935.0386*1467995205', '69659516.4118*1481885121', '16715367.2565*1465121376', '73602483.4229*1465127931', '15670968.3853*1470234999', '15751307.8532*1451844588', '48399.0968299*1471215593', '16715367.2565*1465121376', '36260464.4731*1486341583', '83325409.0158*1464476496', '92493924.1806*1472123477', '18229418.7859*1462843455', '63171407.9535*1462243897', '15849133.5444*1467960348', '62652348.0122*1482101449', '28660404.4248*1476966452', '8090088.39602*1472710128', '96159734.1342*1468049263', '16172879.6623*1457694012', '26392102.9109*1470844872', '2529094.46846*1473790435', '96482696.0901*1487364725', '22500027.0795*1478158215', '75422354.7058*1467227713', '88985379.9847*1460576438', '7950073.68168*1456588753', '54380847.671*1472172747', '41508530.1892*1481068517', '37101168.1865*1466832707', '87505669.2457*1455755842', '83354671.9542*1485113412', '17869130.5628*1478578796', '64778783.9365*1480850955', '58014440.1536*1479075772', '51947652.2609*1457357840', '65923087.311*1462872288', '63410553.1018*1477664883', '83432386.0664*1475595684', '30182473.6313*1481161428', '69440788.2679*1457832779', '15837235.1744*1464613506', '55854562.0161*1469248698', '48461278.5436*1461859835', '96973735.017*1453700380', '55731686.7835*1483700101', '8036160.04141*1486374440', '74646059.7523*1483869538', '17936695.8628*1462262940', '1197406.69281*1486780010', '6945997.71239*1465786328', '28321735.0634*1463795217', '33014853.3316*1485774035', '26396436.8041*1480114729', '47040366.4852*1486274327', '87749474.8611*1471526206', '44853731.4818*1478504455', '2191423.64814*1468968537', '62112192.9368*1460972939', '78207410.668*1469607556', '16825367.2606*1483933539', '87576502.729*1464607498', '19585046.9099*1461843055', '11069864.739*1458760420', '27407758.3787*1466935405', '34217050.0504*1478177971', '25487121.8686*1459714071', '79323044.6381*1455023121', '155834.820877*1470096015', '35840241.334*1477790832', '68218688.2584*1484577607', '2262428.99704*1457831748', '69016503.9015*1479543640', '75630058.9799*1459878681', '86721455.447*1480605385', '99144499.4301*1459100651', '8837242.92071*1463289862', '89991721.2313*1452216360', '40774233.6309*1471265968', '87400700.6384*1464297084', '73173154.2012*1480835854', '67665665.78*1480657958', '29372772.0488*1479100880', '83727875.4866*1456867979', '42997286.7679*1480719578', '78021971.1233*1461663548', '32525924.0428*1464805379', '23835259.9241*1467692082', '37834452.1852*1467049292', '66237066.1975*1482285190', '44046154.6848*1459779761', '12378857.1214*1475797134', '43887510.3363*1476328295', '39330031.0293*1483473147', '67812910.1976*1485826983', '53484157.5472*1472768096', '86653833.5772*1457733955', '30276426.9997*1481772131', '93438440.475*1458878665', '98254222.3689*1458727899', '56617891.594*1465412118', '49324896.2186*1466756821', '68612687.3145*1468890786', '18084850.9933*1483841652', '7822656.57797*1456116912', '13161213.6013*1473292100', '11717808.8507*1475098794', '56643796.2837*1458664013', '46070545.3461*1474976547', '48988049.4896*1459024273', '96949229.1948*1454113546', '7548796.25093*1480798690', '30618608.7369*1480824195', '16950018.4911*1487694175', '78768309.3157*1470196229', '5021689.34882*1482320511', '63583820.5452*1465570252', '44447378.5724*1470494344', '78754435.5165*1486017830', '31741306.2032*1459742834', '84709609.6207*1487658331', '17832039.7842*1470598239', '59326251.5508*1460139735', '40955163.5707*1471910784', '8909684.93561*1479918695', '19590085.5009*1480976906', '4466616.77463*1456209859', '73856215.4022*1455601031', '77898462.2261*1486405211', '81311903.292*1459968159', '7461349.23452*1460809214', '47044212.0571*1473518577', '76020997.3109*1457037310', '76348332.5797*1479578760', '34365192.441*1469318659', '51246774.1791*1455436764', '87256448.157*1473437387', '82013465.1276*1479094259', '74978729.049*1488062828', '95695842.3596*1471426928', '77165691.9044*1465698086', '6336035.09065*1467503413', '63204728.5204*1466117397', '11265868.683*1467766007', '9961760.01144*1467514159', '38091236.8358*1469516222', '41220493.0401*1462677105', '87572312.7005*1466593343', '11109073.0402*1475555239', '76175127.8931*1458492797', '43441577.7298*1485095680', '25986704.0953*1485071639', '78464792.3116*1461805615', '2844528.41183*1467766201', '82608814.3287*1473096725', '72925150.6402*1472431196', '76631974.0975*1472298379', '43041146.4827*1455554279', '71640924.0487*1454070027', '51169613.2716*1479696532', '26437305.4177*1479939874', '34732215.8062*1462195304', '21133726.7216*1480403018', '34050207.0849*1455948503', '63879023.1711*1458106638', '21349930.4052*1474658277', '36053624.0149*1451737630', '40507076.0078*1454083118', '10303453.3755*1464678881', '51592140.1193*1468036739', '25327428.1721*1476261070', '127687.382861*1472174390', '81859125.7436*1480462120', '80717181.1154*1455777606', '70313844.8089*1486697368', '51928152.316*1483113273', '99617852.0927*1471424429', '89036449.7461*1481013706', '47499280.9788*1456109496', '86590513.9849*1454912455', '35702005.5204*1451816310', '2308851.81705*1482341096', '57648139.2943*1478203799', '28530568.8821*1485308489', '58958104.4984*1470816854', '40482620.2315*1466192199', '9747955.9353*1458885262', '98225767.3481*1458165617', '1780085.97522*1456421920', '44888346.6715*1462429168', '57002267.069*1476224878', '64843761.3739*1482676569', '53804986.1216*1451940275', '87751882.5046*1462815784', '39587118.8869*1474928443', '71520602.3013*1478430725', '56464791.4593*1464783149', '90250156.0706*1453680882', '11376105.5303*1473714998', '29709093.628*1454505804', '78673383.2531*1485398218', '29321058.8284*1460667729', '534316.664734*1486302472', '60810565.637*1457750519', '23234350.212*1453187208', '66457742.7195*1474614756', '52830465.0421*1457162383', '74527504.5007*1462775199', '91630813.6837*1483150748', '82318948.1944*1461756187', '999939.013158*1459188221', '4883487.86866*1452080674', '27513502.862*1478459378', '50362035.4185*1455163576', '68142376.8741*1475998210', '46035247.9983*1470161072', '10202581.0343*1460081347', '79272340.9526*1460686648', '19703473.9413*1488146027', '41003523.1376*1463195840', '69174685.7137*1480502534', '72949881.4229*1469362258', '36125635.617*1480180836', '29834088.8104*1486225852', '47571167.3952*1465276374', '87480025.3804*1462550422', '43934402.7246*1482134688', '70472946.8869*1475278079', '19032584.8805*1463872513', '31515294.125*1468290124', '31191208.111*1485015144', '26104732.9259*1475082600', '26000744.2419*1483538932', '19402017.3942*1481341272', '84121151.1158*1459365480', '78028375.8487*1456440893', '96929470.9911*1472523561', '25969074.1107*1484052112', '62215795.1607*1480816664', '41025019.5384*1472291782', '88865905.4334*1473839311', '2958062.12842*1456807495', '2115457.38109*1468807114', '84656814.0071*1471272023', '16715367.2565*1465121376', '26231875.0501*1457758290', '15988063.0348*1478610639', '45587058.8782*1471356413', '25117306.8201*1455116091', '42136070.4947*1461381635', '81373330.2778*1464014483', '14163642.4176*1453424792', '659702.818746*1456614442', '71399618.1518*1468053449', '99542150.952*1484222223', '32770814.5823*1463484544', '29080785.7616*1472355578', '63800018.2696*1467146097', '68817665.5134*1456251757', '29816836.3016*1487223382', '49324896.2186*1466756821', '27080607.5995*1477918033', '34127403.5307*1476915300', '21572695.6082*1478481644', '45164331.2175*1472742985', '85564503.0131*1483969235', '3800266.11396*1485052357', '4616842.91788*1475564383', '96634082.4245*1468721049', '52936881.5592*1466984643', '47992239.7251*1476916676', '86902863.5819*1453868103', '56574369.636*1471921370', '75137472.1181*1470326622', '58044373.4257*1474689388', '33076832.7846*1465710871', '2772265.92349*1460258550', '24326814.7141*1481350606', '12006871.1519*1467658286', '50145628.3051*1459341832', '58720378.6994*1464548406', '52663294.7276*1477969038', '61100497.2113*1470987062', '92384025.4056*1465534076', '41599929.3549*1459925478', '22106903.9431*1461519033', '44662266.9566*1463248783', '90367390.9428*1455960887', '94659087.4629*1475355050', '99580278.1693*1462715815', '73676058.7133*1476580612', '4648439.7244*1453816126', '16733108.2028*1472323839', '21861556.1291*1482753756', '79239892.0462*1473466690', '71174784.7185*1455070314', '30436310.6936*1458961505', '28307366.505*1451982108', '78837604.1801*1461323545', '50612840.5796*1454323619', '77064684.6065*1472272935', '35847223.0268*1468498408', '63723676.739*1473042571', '20055190.2636*1477768468', '76032488.9444*1464616225', '65290852.8245*1469067541', '82243880.0182*1477620544', '20377561.449*1487915573', '54854338.2399*1477260182', '41525230.2982*1466358268', '96284902.892*1454864257', '4388441.87844*1455702216', '12193393.1902*1458371850', '73569582.415*1470969794', '80777542.539*1452848232', '38521569.7998*1471942737', '14921968.5585*1472395516', '85390899.6502*1476545182', '32494220.915*1473179288', '85119475.7299*1481008727', '49384247.9667*1479070630', '7578576.25484*1467270962', '71700345.8797*1453920355', '74146962.3701*1472952230', '43549025.1252*1472780692', '46487618.2148*1463810846', '2942121.71365*1473037389', '48227666.0168*1480579366', '92606099.4332*1486700138', '74388.5880796*1485429453', '1406697.50858*1465708857', '62185852.969*1472385965', '58150151.4982*1484296391', '96784463.8612*1481257591', '24108288.8822*1458794400', '69178946.6534*1461206402', '39192502.1879*1464442076', '24651449.9565*1486788917', '3489991.20182*1457382562', '87581678.8654*1458608623', '5492516.91243*1480425403', '18239195.0579*1483684868', '92362578.8193*1485505588', '7250387.88043*1456189648', '4040417.4531*1463471190', '60003148.8954*1456188385', '56241976.7084*1479326886', '71149545.7552*1482284385', '30515789.4046*1458780732', '40374496.7179*1470980612', '29080785.7616*1472355578', '44037712.7113*1456907775', '34624101.7148*1473505758', '37448047.5502*1457902945', '18922023.3249*1468522511', '9678245.34289*1454335269', '27534271.8782*1452058953', '45227813.2121*1456008997', '85088957.3607*1456146009', '89505873.1232*1487343334', '99449842.0648*1474632127', '89983261.6022*1457453427', '18673684.724*1453008418', '63197259.1632*1463376194', '64250912.2784*1462993430', '41794115.3061*1480628716', '24278153.9388*1481895579', '68160966.4545*1462677749', '50262114.777*1486390677', '71880594.8913*1469788511', '15421996.7016*1458452523', '96707698.2741*1481535169', '56241976.7084*1479326886', '71004574.8268*1477312660', '40252856.7967*1460596579', '79115776.6867*1473815937', '98701588.3964*1486935220', '20929460.579*1454548375', '97033642.5255*1480254275', '90047508.1402*1457985179', '57860969.3441*1468116160', '23793323.4501*1469054271', '70984862.449*1460029775', '56835474.8566*1459178076', '97288906.1426*1454435643', '43711046.1809*1464590990', '81023132.8694*1485301993', '30823165.6428*1453227295', '9476195.27712*1476172765', '4432651.64894*1457405839', '3301145.62535*1467624350', '61780067.0193*1483183042', '98989110.9513*1476867696', '37387421.1976*1465400207', '88355118.838*1460021205', '73526371.0532*1453033841', '8096537.38217*1456862784', '56035372.092*1455091782', '46056230.3344*1473742665', '62744017.333*1479266514', '86035867.3249*1480300104', '11710526.0941*1460694180', '52577543.3193*1465479522', '9221372.7207*1479292033', '11992897.6116*1458901000', '51582321.8684*1478244648', '17118191.5471*1484831578', '52315405.592*1475434519', '5220534.27577*1487208431', '72522406.1086*1461679226', '34792828.4224*1487140210', '10344110.7984*1475286153', '96825767.8004*1458965370', '34147376.1978*1479383376', '54404663.2584*1470521017', '89640657.1176*1461397871', '95255692.3149*1462579693', '37195335.608*1478685848', '7625556.70056*1465659914', '34674064.261*1487078319', '87935678.4962*1479688478', '42909314.0897*1470945080', '50262114.777*1486390677', '21020393.7951*1459590149', '60212956.6296*1488017847', '65102764.7909*1469763920', '47589665.0758*1478203714', '65640844.9928*1484229982', '12237989.9971*1479182828', '24278153.9388*1481895579', '20907960.9733*1485342474', '99504522.9394*1462106212', '47867049.8549*1458855814', '78707464.0029*1470828917', '93437819.9852*1464581022', '91015745.2053*1477776660', '40034273.4086*1454410566', '86705862.9293*1467872513', '36629874.8752*1475691034', '70642978.705*1480561019', '34974285.102*1453585041', '48517882.4714*1478048800', '49986958.8765*1471008516', '7932744.64129*1481087894', '72028477.4981*1475197880', '82070993.1936*1470973030', '41807938.3511*1484316641', '9982426.09459*1470987518', '47362088.8185*1483048387', '47356033.2708*1457356315', '56621412.0014*1467194767', '4469714.05281*1458722089', '26744747.8475*1474859071', '33066794.9221*1473766683', '89075614.0427*1460440296', '95727411.7995*1488062362', '80133163.0426*1455584371', '33578633.9998*1470155264', '30150914.5028*1476821052', '20716762.8716*1484211505', '91003090.6114*1478010438', '60110613.1051*1487736761', '9162018.62854*1482621469', '59006102.2926*1469433641', '95590.8272685*1477421526', '77561199.1253*1468368981', '78977513.8639*1476158446', '84841620.3804*1484343211', '23124051.9048*1452655684', '16888434.4071*1458561300', '8401066.94195*1477304547', '37422088.173*1458086796', '38864753.2886*1470966262', '78928661.6927*1484163794', '51653534.9229*1467964170', '68063652.6229*1453134142', '58373382.294*1484370088', '47582568.5568*1469796800', '60288992.254*1465770805', '95099295.7256*1470454806', '76381979.3496*1478038272', '52830465.0421*1457162383', '35726243.6663*1465590195', '12336676.3398*1461092905', '83157211.5725*1467375041', '91196122.7906*1483538677', '90692525.4927*1468684843', '76631974.0975*1472298379', '84298220.1862*1471964984', '44896537.4565*1459477311', '39462563.642*1473112721', '47681069.2668*1468885152', '87362805.3135*1457602368', '78368167.333*1483616470', '80747321.8054*1488293852', '60886941.8164*1456504414', '33371437.8536*1478092551', '39969385.4006*1478568563', '90094370.1484*1454557491', '20640697.5772*1468978806', '78223293.4679*1461631200', '51455122.7032*1471013001', '87954706.0183*1463098826', '48699608.6809*1471135271', '19638928.74*1454587535', '24319820.5297*1468232872', '4493397.30999*1472152533', '65600831.9268*1453925637', '5388959.56857*1452939253', '22471458.6299*1461273469', '53506855.994*1472591183', '82551905.5183*1479725023', '87877122.0339*1466215401', '89968135.183*1460193322', '29321699.1148*1455025518', '92078809.2697*1484840397', '59721738.9463*1452665946', '89841270.2565*1488029395', '38543466.4294*1475589258', '80802273.744*1464768990', '42126421.8802*1483553720', '83908758.2238*1464044849', '35211020.0814*1484720867', '22393469.0181*1462562531', '52959101.7319*1465467169', '71828845.8959*1484663458', '64740221.3598*1467130709', '5221264.79735*1468793612', '79311597.5554*1466753519', '95048148.6313*1453820622', '68459487.1115*1460349200', '66343970.9546*1465823013', '8403850.26456*1488056835', '51462275.9642*1468126059', '74001168.0221*1481707161', '50528473.627*1483147340', '2934523.73143*1456089984', '27312228.0626*1456297023', '56343248.3519*1480660251', '46452622.1958*1458678223', '40848737.1414*1474388912', '24755268.8431*1458028342', '67839014.6885*1457265420', '62188081.667*1461938679', '97496047.126*1469914449', '92511685.3046*1462925012', '18947357.2999*1475348880', '12037558.7939*1467798298', '46590956.3279*1455261256', '29025117.3717*1486420347', '14081197.29*1465201706', '25104990.4892*1482165395', '55023582.6886*1453390947', '16130431.6993*1480024461', '24246248.4543*1452253151', '97424325.7662*1468883686', '67997052.3513*1464398460', '77231319.0386*1485428471', '7877326.49246*1462063975', '95617770.0439*1477917906', '51271689.6717*1478979896', '63583820.5452*1465570252', '58082800.4614*1480675497', '44214897.4001*1463616646', '35536334.5681*1484966674', '74171474.3751*1481998156', '53804986.1216*1451940275', '29489479.8716*1466244182', '87426316.3732*1452054902', '26224128.5495*1475883402', '39372147.1568*1470537888', '27448208.8263*1458277134', '17008774.1551*1463300876', '89169380.4482*1481429636', '52098535.1173*1469584712', '16833901.6638*1486640319', '66376717.8547*1468840535', '98368732.6422*1463604858', '67948061.2667*1454959270', '52842114.1717*1482607609', '30501829.4066*1480330225', '45970849.8201*1453224654', '75078409.0955*1464776819', '14850697.1996*1470552236', '31795467.5962*1457505556', '44453347.9476*1464413990', '29053323.0756*1487848669', '43582977.2218*1476680831', '42308043.6181*1480198875', '72272398.8191*1475803015', '90269909.2833*1481467631', '45582755.1217*1486328538', '34674064.261*1487078319', '43060495.1994*1482437151', '99688407.6192*1470512483', '1406697.50858*1465708857', '23502970.3975*1466492591', '82455232.8098*1482959404', '73209960.0136*1453466342', '61953671.812*1465220792', '48627151.8947*1479896072', '67260685.8834*1481573609', '42092014.2641*1487960824', '74822757.5456*1483601817', '18043157.547*1487731853', '53390015.0864*1484755301', '71715095.6447*1460811252', '64710535.4204*1476441671', '87937701.8234*1468513288', '87227186.0264*1485263422', '85599014.1729*1452599920', '47161955.764*1468888480', '50969663.5777*1462729163', '30350423.8956*1463568017', '87697647.6102*1484203016', '38072199.2457*1460948548', '96950148.0491*1452727180', '47449513.5297*1455540640', '35183757.0781*1475090917', '21866094.4626*1472457984', '86811463.8431*1465732568', '63234941.2813*1463261678', '46036433.1997*1452555711', '34584679.1582*1461079234', '2248485.43819*1451982880', '13276228.3782*1466431645', '32070546.7382*1485498360', '32828024.8871*1452208898', '68366268.9746*1479086933', '82034533.7748*1478611757', '59899615.353*1467180321', '22944728.7945*1479233299', '95528783.9349*1485248401', '994925.435607*1464418633', '73865976.2416*1472330926', '6053629.27477*1456308665', '71997782.8005*1473322569', '12193393.1902*1458371850', '69146764.3804*1483354478', '10543018.5619*1474953257', '42741697.9957*1470089165', '71822136.5955*1485414827', '44312579.4894*1485562885', '99630370.4403*1483286264', '1287597.54509*1478584127', '51318758.243*1467991587', '8271264.15601*1456986556', '87581678.8654*1458608623', '52448750.9469*1464309159', '59246066.3368*1469596015', '33028392.7724*1470219653', '52364079.8982*1473743159', '67321762.1391*1475631812', '38521569.7998*1471942737', '87117810.1769*1462486276', '88984710.0592*1456395729', '3670169.1275*1459134195', '68916332.786*1484815997', '29169852.2549*1458699184', '95610387.9748*1453558558', '83502911.1644*1478446966', '34957112.8849*1460902548', '99688407.6192*1470512483', '37784273.9867*1475268455', '75846506.2676*1481462670', '54908669.859*1484182748', '72049921.2443*1468961676', '28282456.4044*1467711624', '79862932.21*1468242572', '93531600.0744*1468056791', '5253865.91166*1474289378', '47961870.7636*1461646085', '60583872.1436*1462184742', '24056536.4449*1484402335', '26566311.6395*1465258672', '36320472.7989*1479736899', '76287097.2576*1465861067', '71822136.5955*1485414827', '21557691.1172*1478251826', '92382731.6621*1461604252', '67794088.4634*1460325884', '87234017.6239*1472974950', '75433503.5244*1470729646', '46188142.3976*1464218481', '79266389.7262*1484403756', '99354137.1763*1486387956', '73980545.9143*1477082991', '90578690.3162*1460303860', '31482812.1293*1473925079', '66002645.6082*1460617269', '92465221.227*1465579185', '37995287.8489*1466690116', '76650627.0153*1469009767', '25219199.805*1467657326', '10576320.4497*1463047209', '71177750.0643*1465042273', '3056995.82064*1457790592', '17695327.899*1465207963', '92355605.8034*1486600054', '64258157.1211*1480780106', '93992810.4611*1463690303', '51125324.7342*1451997801', '76779669.1322*1488310882', '76635791.5994*1487851132', '63577162.9639*1459148859', '6316501.93094*1475695941', '14582603.4313*1467044431', '66297341.2722*1481958211', '62400038.6095*1472909726', '18865722.4279*1454226350', '93973354.2478*1475736321', '29481406.7851*1483408534', '91144901.853*1468435909', '49556627.6518*1455739985', '84679747.6904*1472110936', '71846689.3575*1466112620', '33298335.2431*1453968432', '47840372.1481*1486078018', '53094624.2901*1476931897', '45970849.8201*1453224654', '50654848.4794*1477604257', '30258116.8453*1457753862', '17282106.5784*1467042301', '16700393.5807*1454618548', '93773830.068*1459207091', '7857840.47799*1453615656', '91624062.6638*1472983704', '63080236.6055*1465008499', '5161591.87901*1466927795', '3505648.33503*1463539269', '41543590.3855*1471951187', '88668079.9684*1472041655', '59210037.7498*1482543175', '46039395.7594*1462917917', '27759079.1342*1459876146', '18317919.3522*1476030783', '19080308.596*1455994733', '42860554.0821*1483947780', '87337331.7404*1468039325', '66051209.6705*1484490925', '75726255.947*1472472376', '89595728.0082*1468251621', '57089905.126*1476154352', '92657517.3346*1460123918', '54344825.0706*1452438952', '78663822.0384*1463677753', '77435620.6553*1461686168', '92863477.2075*1481684635', '14922588.6158*1484517730', '60866155.5475*1458220650', '11480411.2247*1473778084', '64059074.6966*1472839731', '72327534.3714*1475177777', '24347419.9659*1483825169', '37926268.3208*1486944402', '38935126.1364*1482230672', '93827392.3224*1469650528', '36408539.913*1463372958', '16198297.0253*1452206051', '31865381.2148*1487073215', '63409637.5047*1479497295', '55100492.9424*1475877035', '9659817.20247*1474827364', '3914782.62886*1453256965', '47520153.2827*1466870854', '37508008.1082*1482405233', '5866982.09831*1457397504', '99210304.0448*1477719554', '93721897.0204*1482757364', '41541455.8469*1488122110', '13312053.6202*1459610870', '97127350.9496*1473113331', '37578854.5209*1455964958', '25284989.0042*1468340148', '31428583.1239*1457151062', '28345059.2314*1451803879', '83195425.651*1471385858', '69430464.6731*1482306040', '40533338.5811*1460612549', '98394015.782*1475125319', '86899201.5296*1474249007', '64504936.9712*1486630350', '22471458.6299*1461273469', '83665714.547*1480877578', '17136968.1802*1451697649', '6410110.70358*1459875124', '44155936.2615*1473020118', '86098410.896*1486598989', '57817194.8715*1468950936', '12724055.0587*1461017696', '29849988.3831*1467845311', '35777494.5556*1452854090', '35745456.5672*1487171107', '32447604.6158*1470421977', '4503215.83466*1486872260', '85827361.7266*1454966454', '370758.810013*1467851147', '64830700.3591*1488034140', '55156846.4473*1452399973', '23254171.774*1475918849', '64212953.1749*1480475199', '96539570.9476*1462458360', '83659543.8625*1469055418', '55945548.7386*1484150578', '57041738.2359*1461859735', '73363444.2597*1458138671', '78066945.4928*1477036772', '91977121.9474*1453109334', '20052506.4654*1464382943', '83362917.1088*1460955735', '45475686.3317*1456542382', '72627384.6896*1454463929', '1079659.09516*1459917887', '49345971.4851*1462477876', '32553120.5028*1473144227', '50177472.8671*1452672902', '35232454.3123*1473520625', '46207264.6073*1475312218', '85944448.2941*1471793924', '51455122.7032*1471013001', '45362502.5358*1474014172', '39117114.0936*1473913617', '22544502.2037*1472996675', '26431450.5603*1453023061', '90556981.0949*1487045888', '2256168.45051*1474523662', '39920633.614*1463356807', '18498322.1795*1487139411', '56691736.9623*1474836135', '36082069.4099*1459679738', '32122553.1783*1485498424', '60781141.2941*1473597112', '48817282.175*1462111958', '44647164.9101*1454016298', '92829024.3924*1459713634', '41999515.4055*1465533840'], balance_amt_all=['20112648.7419*1462383970', '76161258.7397*1478136108', '18146674.8834*1465695904', '2663318.8783*1476955551', '12060786.9847*1470647917', '58186307.4454*1468048947', '31347110.238*1452816239', '67331940.3641*1468547747', '76464874.5687*1472411160', '24521707.3883*1458539976', '61949415.4957*1455522621', '88222682.1238*1463771383', '15676232.4937*1462344575', '81171372.521*1483302924', '86745277.6612*1464335852', '44399922.6188*1462083228', '31687572.4641*1467527448', '8302088.46546*1474273158', '10255507.2701*1455745939', '48960303.9602*1464452091', '21182494.348*1482822662', '43553831.5394*1477208853', '75501037.0382*1459915257', '29277974.0989*1476343777', '42195634.0178*1476662094', '37855079.9505*1462173032', '33376930.6735*1453290881', '89745451.9131*1477200953', '44233733.0939*1453689175', '52624513.9332*1455751592', '45226295.7648*1470651624', '28630862.3486*1460266258', '77360091.585*1456249594', '57929294.9267*1466904670', '78953907.5351*1459201442', '43206954.0652*1452119971', '39187240.1567*1459259673', '48351717.5339*1474879917', '53297172.7487*1468711626', '68926991.0678*1463929366', '7094821.85848*1454002554', '1045342.16572*1478584127', '95556967.9201*1470628379', '76265127.2995*1479980717', '89028605.434*1481557857', '53474514.0208*1474658277', '25513638.1481*1468954483', '17740169.5561*1452405394', '82501303.6554*1481209893', '48824916.0048*1487022197', '59534679.4851*1470537888', '87542745.8811*1462149244', '11386002.6081*1476364017', '31244161.0159*1487734413', '79127513.5196*1472006245', '20694533.7728*1487965733', '51353633.5165*1472426879', '60764972.7195*1459198975', '88459480.4456*1478562172', '76464874.5687*1472411160', '81563365.9331*1460750221', '79318826.2139*1482730117', '39223106.3818*1464057941', '69396261.417*1470684073', '9864440.97797*1470406523', '69609268.6773*1477079202', '92783836.6412*1477563699', '25207370.228*1452125095', '10893666.6517*1468773873', '67219146.4873*1486264947', '48458181.9545*1481595789', '54456519.3431*1483211638', '3555118.07418*1469955065', '52777609.5281*1477504480', '75411533.6052*1462607979', '51603380.5636*1472174390', '85669303.6247*1463312173', '28320619.1813*1474999838', '89625227.5489*1486363777', '67381395.7695*1461249959', '12887237.655*1453098110', '27883021.1772*1453709033', '52013714.9418*1472444422', '71851366.3692*1486154663', '13914956.505*1475378684', '3178191.22029*1482295243', '4319878.74855*1466408899', '49688729.3329*1482465613', '64550982.7546*1455765648', '59221794.181*1452968455', '33180040.8054*1472280453', '92241958.2835*1479492587', '11561070.8708*1469295631', '3941477.33979*1456880895', '57780001.2574*1462111884', '40449205.5686*1479560259', '78918472.1495*1461990798', '65037792.4352*1459968159', '93838980.2433*1472385965', '23545068.8407*1456986556', '862917.489952*1474812541', '58788245.497*1482056186', '45034149.3775*1481535169', '41117099.5588*1485200541', '73302732.4956*1456472516', '73553798.6614*1483926341', '7015289.72254*1483307602', '96979319.2572*1462908723', '64581686.3984*1463980128', '23668089.9853*1469055418', '60587113.3517*1484472887', '9509772.26817*1474702980', '29857872.4288*1474940736', '35500000.1211*1464563772', '16564165.3928*1456548289', '90966475.2894*1472987703', '13670894.6168*1483791040', '79314416.5579*1453845989', '65986134.8072*1483999357', '67575233.6379*1470958687', '77647330.2135*1473803238', '61411703.4293*1457140502', '22399498.1408*1455318216', '71117506.1651*1458271141', '11485899.7022*1455171344', '53211625.5917*1475047002', '98165574.71*1458025711', '48290262.3621*1459854002', '88913589.6075*1467003579', '50042697.1457*1472645803', '46667207.7752*1479433223', '68302180.5875*1454159939', '79686094.3668*1463998790', '6868645.73978*1468229492', '34209594.6994*1464301288', '37175052.9481*1458906241', '6978984.7451*1483199984', '46480750.3339*1480462336', '4890975.19424*1471065449', '65206827.983*1483966148', '28475264.2893*1483544160', '17740169.5561*1452405394', '89056229.6533*1465390354', '20784026.468*1455023121', '14250812.6423*1487759915', '33901428.5576*1474737666', '50777106.6801*1480002856', '53122125.9345*1452413435', '7374570.10464*1475234127', '66037051.8968*1464742244', '2879638.88767*1475286661', '69502487.8571*1483903896', '94560268.9929*1459862094', '71418274.6154*1461519905', '98307626.7488*1467613149', '47162328.0023*1484893770', '6814175.6635*1461313763', '98853188.6551*1470337438', '289937.848207*1468120460', '11137356.195*1477752060', '6254143.95004*1483593350', '26396716.9446*1481796763', '11175669.9316*1483317858', '35176923.1095*1458362067', '5431705.73994*1483662118', '22940945.8255*1484394873', '8050069.5669*1474240582', '79896160.5546*1464208480', '80491056.3243*1456193229', '30489111.8656*1486953312', '62127193.6752*1486478769', '23592907.0681*1478561078', '44565072.1492*1456103885', '40180157.6426*1472651110', '36578415.7872*1471886048', '29040322.4939*1462258520', '89565605.5612*1456274073', '95679793.6152*1457744397', '54069914.6409*1473144227', '62954111.2823*1467610671', '35209082.2015*1453621894', '29999708.4364*1472659350', '3121348.76462*1484414747', '39211261.907*1455881593', '75574453.1136*1465029725', '70429292.9348*1484624891', '92106202.1545*1484082868', '69226592.8396*1482904557', '95012528.4798*1485334410', '23383550.942*1467896655', '12516801.3058*1486697570', '7312143.49089*1474544535', '59123678.5767*1468522511', '52270728.1512*1460200360', '91628771.7367*1478830092', '12288023.0052*1476829288', '93715729.9444*1473963389', '52952748.1799*1453890445', '77873605.1851*1481728579', '50271754.4365*1461145974', '66028191.7309*1456510217', '97670044.8558*1473397746', '48547539.4767*1483767063', '73323135.5413*1478831086', '50976094.4578*1470390268', '69146357.8876*1485643255', '87173984.3171*1464520726', '4274383.96983*1451881008', '35997494.1928*1453457120', '73435008.1389*1464182203', '27971825.1948*1472489099', '33852341.7928*1455376451', '23571081.6972*1472050715', '92831693.0255*1481597023', '34882570.1542*1459517925', '37105475.8656*1472029624', '8113085.15592*1481036592', '81077243.9756*1453968432', '412768.538182*1488077301', '55172083.9645*1488146854', '63490964.0232*1468251028', '91010301.2197*1471228354', '66936467.4645*1479672155', '41511296.8323*1476172029', '60784857.6578*1459173798', '56549330.379*1487405204', '94980047.1504*1483602117', '72087142.6106*1470275318', '93831765.4973*1465449790', '1025711.07779*1470025692', '53682571.3346*1452604122', '10682201.3541*1451704165', '83245829.5633*1478674633', '8869635.92343*1482044336', '98658434.5881*1455047319', '92702869.4731*1463493959', '86644864.6824*1478640603', '79156424.2424*1479031178', '54778341.0033*1470402332', '29141454.3577*1478794927', '52388055.7882*1464699702', '26443041.3883*1481112283', '36469673.3944*1474676527', '35338409.3133*1473130248', '75817053.3399*1463958332', '99274731.1659*1483749508', '10857515.5083*1463651530', '80700709.044*1472264628', '83080314.3819*1457273482', '60508515.5983*1460107953', '71699727.6657*1464072950', '4958845.06743*1475051931', '12149652.5474*1478992492', '40889100.077*1466215521', '2454355.51588*1457712502', '91653990.5154*1482521867', '36988948.4137*1475204260', '30261195.8772*1475300352', '59596684.335*1486380807', '54432315.3558*1482337598', '23627864.6354*1457453427', '18507287.6836*1485403125', '56926970.6529*1487402938', '15680900.2015*1471385858', '92024677.9365*1469532439', '20647072.8719*1464494966', '8550834.40427*1473783403', '35624983.4423*1453912917', '36545808.6293*1476154144', '43994523.5907*1475764983', '63501655.1895*1465377155', '59939408.747*1486845540', '21810609.9622*1467265828', '46070986.5308*1457736654', '99500341.4626*1467261920', '89220253.7742*1471218839', '34070747.7818*1467546952', '1252168.6584*1483586745', '40449205.5686*1479560259', '85812518.5692*1460756076', '80723045.6058*1460305469', '41452644.2304*1466345863', '85963262.596*1470296530', '75078563.0442*1453513833', '54212314.8893*1472552067', '31921709.0597*1483794305', '6864957.43557*1464931071', '79127513.5196*1472006245', '98552262.8956*1466590713', '54469302.8403*1461206402', '81785156.5256*1485162481', '88710984.3155*1468086059', '91881321.0565*1453343936', '12887237.655*1453098110', '65991207.9335*1453103934', '77713768.1184*1465683117', '87414169.4691*1452578263', '65793273.168*1480201150', '29921775.3267*1474411864', '42189655.0808*1466283123', '38173264.1384*1467318996', '71033146.8943*1472858414', '5541790.13983*1469044780', '28988319.7318*1485985018', '70968548.8327*1486457504', '72771852.2341*1467237713', '87969519.7855*1469463781', '14235387.4568*1464611701', '22532125.3287*1475684092', '90214830.2674*1480065724', '28391968.0585*1455444657', '95300628.4725*1472867294', '75517739.6104*1488293852', '27252716.3328*1456164248', '48050868.3928*1484143884', '2353910.42458*1484595730', '13659819.7926*1487233955', '20845118.0977*1476187553', '59367347.5088*1474689388', '56863035.0947*1484562265', '33230845.4717*1479182828', '13417430.5708*1477468057', '21278894.3713*1483435555', '74283134.9051*1458093093', '74090397.6521*1458080011', '12589407.2405*1469522184', '95597765.7248*1465886531', '2004110.29356*1479939874', '17822718.3286*1453299753', '21187326.8962*1456921458', '48214730.0172*1461079234', '14836822.0596*1459959727', '42465654.1015*1468422952', '5813731.46999*1474587624', '17544803.0644*1483310665', '94848978.4181*1452333329', '8457676.27452*1486202283', '16589062.7047*1462665453', '71759941.5083*1454551879', '24725095.601*1471586467', '48356278.1179*1453523098', '62407701.2144*1467283468', '37574346.1133*1456535235', '62123818.842*1472712923', '93620609.645*1468055758', '93508970.5042*1485095913', '8081223.68732*1482013737', '35565455.8624*1468808003', '76052134.1394*1486889022', '42504986.1162*1487353203', '17838518.3106*1453201571', '922673.664184*1452288867', '44077314.4757*1456359902', '97485959.8889*1460728454', '53598589.6523*1482274753', '66530223.1149*1481876143', '79788439.5822*1452581752', '71935305.2697*1474692794', '65171320.9407*1487668202', '30248273.4369*1479149677', '30679980.8341*1454917435', '98288729.6432*1459953327', '34904291.1605*1481896082', '10347084.0167*1486687096', '81968483.0001*1454650397', '75130829.0712*1483462180', '67765814.0992*1466674779', '7430176.26206*1452008988', '73289821.7599*1464753577', '86427707.52*1488021081', '60261065.5513*1472041655', '87316278.4607*1487910366', '14228172.3761*1480659475', '91592575.1255*1453496513', '88326350.1452*1455007512', '96134282.4623*1460327808', '83566501.784*1481291978', '40445791.6225*1487662579', '39692299.1209*1484666960', '26164692.3223*1477549371', '14942304.63*1455808865', '14869408.3041*1478091117', '32488849.9385*1455921753', '59474356.8122*1459222319', '67553535.0518*1485078416', '54232609.0865*1463746160', '82944009.3823*1480983820', '61191087.6483*1463923947', '23545068.8407*1456986556', '55199360.6595*1471231013', '61391944.4826*1474586612', '68924744.5418*1473676997', '97844176.3257*1466886648', '741282.24753*1473556146', '65249791.4563*1453372768', '44120809.1241*1469883062', '25717454.1512*1462436536', '74270918.4037*1470486524', '83186182.1087*1481510794', '34829033.4279*1479738009', '15203309.5061*1474368068', '35679762.3081*1460730540', '47969218.6746*1478754471', '81226157.7241*1463838931', '797727.845526*1481025861', '91967288.0375*1477684899', '92556749.1694*1466234067', '99403756.2017*1467780176', '1320193.45697*1465621384', '49271155.0904*1472232133', '72982268.5911*1458987833', '52991833.2452*1475491203', '41319130.8035*1487877064', '90509758.8625*1477604257', '69336161.683*1470265753', '49284630.5051*1474310976', '97928820.539*1465050879', '94701834.5481*1468380414', '82844633.5852*1467039912', '64579019.4272*1471552539', '22069954.8558*1454577825', '15614414.2595*1485836882', '77610687.3467*1471155157', '76905028.3021*1458159718', '5166480.25669*1468442606', '46841223.4739*1467658286', '57701685.4911*1488294907', '80163238.9469*1484088064', '18408125.2465*1487550522', '91139746.3626*1454946135', '78907418.2944*1477327809', '26249082.515*1469174490', '2988313.11771*1461895726', '28969840.2448*1461198205', '38889044.8759*1463356807', '56863035.0947*1484562265', '42987661.2077*1457294484', '37105475.8656*1472029624', '5014713.5921*1487431318', '9141041.49553*1456252683', '29185234.7669*1453944332', '37030879.8365*1452655684', '53900814.4926*1467629313', '53198247.6859*1461773549', '84150625.6996*1468720741', '58981716.6482*1484364602', '66754917.5958*1453771251', '27015809.2721*1453945396', '3171407.0563*1469063080', '87357842.8516*1484077876', '13068919.9882*1481053585', '90905861.9445*1486974712', '95370657.4604*1467144343', '62199855.4978*1461464892', '81296949.8766*1476931897', '6632586.90067*1475687538', '9187725.18944*1456885760', '4070457.60315*1454456460', '95627020.4628*1467236668', '7118335.00927*1474733576', '24907957.0068*1473923367', '58535248.2076*1472016535', '78671451.4181*1455241006', '82834318.7316*1464957168', '30354696.5333*1481800321', '85391117.4476*1454627808', '47546549.889*1475348880', '52561557.2031*1484736473', '10881240.2911*1456608749', '52102670.5577*1457199486', '72814297.9739*1454479886', '34938504.0746*1462979862', '87688152.41*1459896522', '11214968.5581*1484182748', '36099015.3423*1469405937', '84435816.85*1452113567', '16588976.8078*1452327342', '97304291.8071*1473432942', '23215958.3403*1478842685', '68572378.3107*1472299272', '63276893.4623*1471248360', '31130284.5046*1452975707', '18071486.61*1452831852', '3801653.39336*1459025780', '59892039.8175*1462526398', '75881132.2221*1467988992', '5583178.48599*1465251598', '52442428.6924*1486925563', '11284533.5153*1486817398', '92310409.2499*1452346423', '6032772.1745*1470974157', '69396681.2829*1460796736', '65973413.386*1475487386', '27131194.891*1470830277', '52522081.9759*1466124316', '20532895.0257*1462246953', '56056587.0382*1464687961', '78019189.7762*1454904515', '45059750.6703*1464782630', '17475022.2567*1476158446', '21882212.4431*1464900508', '70919292.9195*1455376717', '37051590.0554*1481240542', '67437791.4951*1454902806', '22424363.3524*1470821942', '99138886.1557*1478339900', '69456792.1489*1484071939', '65958369.9587*1457732124', '68073819.3279*1461129070', '81146458.5462*1481965506', '2206656.55829*1468251877', '67827626.0523*1473624237', '77928651.3806*1455697338', '45106044.4076*1480877578', '68924744.5418*1473676997', '88461447.4072*1467560493', '61090601.1482*1467310155', '78987890.1098*1473117622', '50525629.5595*1470723659', '92315984.2814*1473769762', '25453910.8749*1471962101', '18303472.7288*1478287047', '77682964.4192*1474388912', '97782849.0638*1464320331', '44536480.8203*1484291921', '71163299.176*1483606251', '51999153.4216*1471431857', '42302311.9518*1463180841', '22747551.6654*1475819131', '8113085.15592*1481036592', '34843250.7576*1476916676', '70307280.3777*1485403942', '26428650.3586*1465957423', '49370100.8923*1474553993', '11346374.1505*1462469782', '35544180.2549*1471805603', '4331438.94732*1467907854', '48893088.7855*1466636956', '95722666.4642*1468580839', '89808298.4053*1462885248', '48162626.1282*1451744398', '22732134.5928*1468325414', '99596117.5046*1460338235', '51719576.8655*1487510444', '20090169.4441*1475846389', '2268440.52731*1462627441', '77647330.2135*1473803238', '13171936.0808*1457275266', '47196893.3577*1468686693', '72170915.4493*1465230292', '5680772.18305*1474147915', '3533507.10096*1477064596', '94886168.4123*1455856975', '75649356.4412*1485425870', '56244367.7093*1481334081', '22835642.1133*1480705117', '81070011.6735*1463309367', '48793155.4402*1477776660', '27461775.3325*1469040783', '48901136.6066*1477327645', '51597686.952*1462720927', '42168264.4178*1487788525', '31465755.1533*1473030221', '37735535.659*1472554597', '19135350.0295*1452672902', '62767915.2448*1461644791', '12361946.0646*1454331975', '171476.966266*1476802829', '42159628.6753*1459270738', '8291298.76436*1458415505', '12149415.4413*1469111040', '93238579.5346*1455161152', '27789418.8785*1479850639', '28602397.027*1462524869', '65982669.7873*1458478120', '19738101.3299*1479836450', '9239265.44963*1469017087', '6632586.90067*1475687538', '26222785.8202*1472378164', '53970746.2977*1480337951', '68040871.2762*1467527703', '77077987.2173*1461735974', '83760093.8006*1478121095', '65067114.0806*1454252796', '83299979.2014*1469964065', '95624056.984*1470717408', '48896447.5693*1476659308', '20172511.8282*1484887812', '7942219.96148*1469170262', '21129917.9065*1461945735', '78682634.6443*1455028846', '80922518.0206*1465353493', '98928432.1023*1471633395', '84766296.7349*1459417091', '83838519.9391*1473962513', '9643201.46406*1475947766', '28853208.5803*1486924490', '6505147.92137*1466852996', '14098716.8034*1455496864', '22041629.3648*1472665022', '73274952.9896*1481349785', '63200380.4427*1475612679', '32183490.9023*1452990876', '44612205.2887*1457302840', '37965331.5757*1469831449', '57156912.7949*1457375605', '8829834.74208*1482700847', '91443799.3323*1474485651', '20523235.7957*1458958989', '587589.890402*1485267186', '35550356.2793*1485610113', '6366490.38902*1480271031', '75887341.1653*1465848897', '64074371.7338*1467375041', '39119094.6144*1461019168', '3600053.97276*1464337755', '60497920.2094*1467929039', '95331859.3014*1465518865', '50046661.8258*1484242512', '11741885.2285*1460069670', '60026978.3215*1461265529', '73568494.2763*1486931588', '40713679.4671*1471365913', '63096482.4632*1478970880', '63142296.7517*1457784165', '4360803.0256*1469529922', '35500247.2293*1471555412', '12366002.6294*1458992428', '65391518.8357*1477350296', '91191469.2331*1473216006', '19731564.6532*1482055414', '68862745.9276*1484327382', '34678742.5524*1466469303', '54808328.2213*1456074324', '44600251.3792*1473511477', '77803845.0002*1474791425', '62411646.6052*1454863159', '74129545.5643*1468341568', '45099261.6315*1473705521', '24701877.3741*1470439810', '72228773.1214*1459705100', '25506776.3784*1486470340', '42893452.5686*1487448705', '34510097.7423*1460138611', '63276893.4623*1471248360', '46406752.4753*1473045439', '47970028.3559*1480665171', '87268471.3115*1477119995', '31934068.3624*1485169111', '15757146.611*1473742453', '58744531.1078*1465654668', '27686036.2304*1454323619', '19705330.7537*1478685848', '72451927.9178*1459730023', '71634505.7514*1467301884', '20786863.4839*1464590990', '93143285.0887*1482943079', '74003643.3481*1480660251', '72395289.9152*1452174336', '59154629.5288*1469714839', '53138051.6474*1457068593', '2096242.19603*1480877490', '99678292.9018*1479970552', '85438014.5727*1458628866', '27916329.1254*1465713640', '86763489.4906*1472530594', '57936177.0944*1462723885', '32819872.5615*1456547588', '96979319.2572*1462908723', '7657450.89736*1474218104', '12691626.9325*1454409671', '28580269.8711*1458295554', '63506736.2084*1460436482', '94990792.0967*1457392699', '63096482.4632*1478970880', '76326657.9892*1461698419', '80224410.628*1469506346', '9688998.73915*1469213474', '20116326.9622*1487110425', '20112648.7419*1462383970', '34168387.1526*1473996898', '27092330.3114*1486553274', '30054712.3678*1452356249', '64514394.4503*1477041119', '77873605.1851*1481728579', '68194524.744*1464580461', '36342987.626*1483059504', '54739341.6709*1457791328', '19665394.0035*1454029420', '66224545.1194*1463647086', '99099876.2517*1455974855', '118263.40072*1472000961', '14170915.1412*1456303924', '64723919.7467*1482706455', '62721966.3212*1461180807', '40113562.3589*1472111541', '76962041.9039*1479956926', '57267571.3349*1461822377', '74593851.9703*1464214374', '90203881.9108*1474676805', '69190157.7646*1488304300', '33700959.643*1483656720', '56394288.7734*1476711830', '57692877.8584*1470360410', '73530720.3358*1456851509', '36321066.1365*1467987090', '66005241.5683*1461863665', '49996670.4414*1479497295', '97179876.1425*1466552489', '88177221.8772*1477700433', '17234625.9156*1454664482', '44139591.907*1471812860', '49969694.9426*1471885404', '56369708.2825*1481804226', '45845221.84*1457902945', '47330144.5388*1459957054', '92918915.6046*1486584484', '96405706.443*1459326817', '20314000.94*1465876215', '62123818.842*1472712923', '88113319.4201*1478674087', '29463440.1059*1456686177', '71868844.0765*1452768022', '47876173.4486*1476613838', '65280131.9751*1474758670', '92483354.496*1471445443', '23101336.0593*1457668470', '14780513.1882*1466400231', '66509968.7475*1459386771', '72103154.517*1477633906', '27825131.489*1452341096', '5321044.67382*1453062175', '70106057.0356*1464321870', '95385757.6155*1486623148', '74143353.2734*1453698824', '9318941.50812*1465212852', '26159620.4172*1455171614', '99099876.2517*1455974855', '98562221.0291*1468809304', '40454703.2452*1461531377', '89976340.0593*1479064931', '41545883.3225*1460842041', '5691266.35825*1458001110', '59832997.035*1487851146', '11787444.6335*1478065412', '62957890.7698*1471749640', '61563782.1846*1461641937', '86156586.4353*1465257194', '69435840.6507*1458177046', '4012789.6508*1471850906', '76650620.146*1454927567', '61828637.5632*1474059510', '61456592.0483*1461195000', '5552307.97057*1466302737', '13517.9560216*1484444914', '71116325.357*1463240347', '70101594.3206*1471172968', '14791806.2709*1453931943', '54864813.3636*1470573781', '70782279.1404*1472529656', '58619703.8918*1476901382', '46221872.9585*1459756566', '2208141.21371*1457313615', '76336468.1858*1485278432', '87451120.4048*1453932043', '71659927.509*1456080555', '15964889.2874*1476026075', '57644744.1404*1457507542', '16888284.0277*1461263414', '37132887.2501*1484546797', '13691837.1803*1459616539', '36815473.1769*1474932788', '27686036.2304*1454323619', '51898473.127*1484149008', '8923453.30482*1457120256', '55508043.0089*1487940164', '91644310.1532*1479964789', '3378230.53796*1475090917', '11147318.592*1484627238', '45284947.8599*1454234791', '53696867.376*1457331811', '84575493.46*1472012412', '20893480.8884*1477031974', '47920541.8714*1482353102', '87029961.8813*1460566109', '69174077.7102*1474898079', '36947745.7804*1474728050', '90234988.6156*1484109302', '71314646.5791*1464015844', '67795033.2384*1461515316', '48224351.3074*1472259903', '42195634.0178*1476662094', '86664090.1993*1455815009', '13127208.1902*1485484873', '38893001.4227*1481467631', '49439611.6075*1463594975', '54704361.2256*1466918112', '19187100.6191*1477224971', '99747202.3014*1478295834', '75760412.6484*1482163658', '9746523.24164*1476595150', '37967377.2737*1461063265', '73837027.3717*1470060954', '73936821.1417*1456405136', '40096059.0001*1454399632', '33059632.2353*1487884387', '34165723.6335*1460469174', '384735.582064*1465365864', '67343977.8437*1487612106', '94226937.8226*1475390338', '51617228.0946*1488195719', '53138051.6474*1457068593', '5289729.4218*1469357843', '80689447.3735*1485790278', '71935105.6467*1477442797', '94226937.8226*1475390338', '847702.298724*1464577672', '41002392.9649*1460126155', '61837617.3468*1477555341', '77210268.0238*1474694573', '96271950.2516*1471925648', '13995820.0811*1478748976', '53150601.229*1454930246', '94477446.1904*1478466464', '88686336.891*1463690516', '74721714.6975*1455349984', '81506427.0402*1455568641', '62035809.5386*1465986582', '90755000.4123*1479616945', '30663757.3091*1486343234', '30748150.9017*1467241182', '43008515.471*1483295487', '34535049.2821*1456123988', '75549893.5995*1483147494', '35839322.1378*1469075370', '42368202.5141*1461669026', '43865012.345*1465897572', '38653469.5284*1471426928', '47202053.0005*1454975992', '49708419.6291*1468612967', '20500667.6998*1483696071', '1049521.37894*1477776120', '81913489.5186*1479253117', '67085582.7044*1456468100', '92114783.7596*1487352159', '93076947.4072*1458053458', '79657405.6413*1474261815', '42371230.9479*1465407137', '35967751.3466*1452714322', '88486815.0462*1483431557', '40065592.2311*1477743182', '8708498.74946*1455475665', '54259406.6532*1480532726', '86382671.1579*1468916983', '85939857.9591*1459066998', '66779757.3373*1478954251', '42293147.8831*1487841885', '7531436.86637*1462830407', '8708498.74946*1455475665', '48764426.1033*1473050157', '39238770.8085*1483518120', '47538897.8551*1469794798', '21179502.0889*1454930090', '78233024.537*1466936553', '97113187.9364*1452648070', '50410416.5437*1481241860', '19006504.7849*1457867513', '97819051.8242*1460918494', '84150625.6996*1468720741', '32093972.6658*1473189371', '35900548.8139*1456364297', '14760811.5452*1459930249', '69328946.6898*1465486158', '22194765.1076*1468644483', '57030442.2758*1468765411', '23951835.1502*1474323603', '28143080.0616*1454506025', '87721710.0998*1459876146', '26862847.1159*1463564700', '5412198.71993*1472163178', '25946441.3682*1482659850', '84736584.0841*1481321700', '80820934.1406*1484840397', '39713640.0217*1481800198', '5139449.98531*1482442799', '87110435.3697*1451996117', '14373689.837*1486732074', '19436494.3492*1478997024', '23874347.8505*1470280524', '18178851.4247*1480658632', '45060579.7096*1486166613', '93469607.6296*1455388310', '20384779.3608*1487230867', '46722852.502*1452652212', '53716599.7991*1459550471', '51800757.3529*1483648659', '96353104.0202*1485018370', '65567137.4563*1455099366', '71323611.0437*1465835684', '71046454.888*1458611875', '77087119.2934*1476770151', '67531488.8079*1463200192', '40783179.3707*1455237667', '63802319.5561*1462186275', '96889638.3907*1467808323', '65980083.5639*1463895517', '33286295.5917*1478668217', '45715028.7424*1474288205', '12539958.288*1466907845', '19204888.8946*1485220778', '16638467.8364*1480562505', '78367750.626*1460353341', '52181115.1137*1476767279', '74358986.3341*1452611559', '67036967.7365*1480468252', '57189440.8372*1465123048', '42166954.1981*1477374676', '53884188.5074*1475139007', '98796852.8497*1479683895', '64859178.1486*1481041422', '93809470.9*1466239651', '79471146.0651*1487548766', '38361176.2687*1486297640', '74048791.5055*1473656406', '97007224.0193*1466666691', '77850076.2298*1473414876', '46480750.3339*1480462336', '33197374.6227*1455299230', '34946690.4486*1480241858', '11198802.4487*1465086932', '36104080.8776*1451866109', '75650850.2869*1475276782', '29063360.1217*1468060836', '77990720.0375*1483167464', '58178990.5892*1458976500', '48456389.5422*1483307417', '49772911.6518*1462775721', '69899786.418*1478032243', '6456956.43465*1471917573', '35734387.8556*1459132726', '85315533.1976*1466639260', '90015019.298*1485128005', '58250246.4211*1479465592', '58713781.4408*1457714920', '47124129.9428*1488186396', '74250787.1406*1452879203', '71320226.3122*1474706236', '92642058.2506*1487997212', '72942537.3139*1456836602', '10452405.9158*1452650689', '38610295.0352*1471331774', '63231271.1079*1456680773', '85606375.7616*1467879372', '17832130.7663*1487199104', '68061986.2825*1453758649', '47152587.2551*1456882567', '92541963.0889*1456685834', '8860811.05248*1451990236', '62849820.7777*1467199181', '32786645.9519*1481124440', '96287992.094*1460302455', '94168032.0617*1451739437', '88537969.5805*1486092109', '71770787.9651*1454779263', '13641730.7057*1473395411', '9052563.50401*1481643167', '2830126.84041*1456143816', '33185160.6213*1476901325', '43004862.0392*1462267954', '62384785.209*1453332112', '49994833.0946*1468614918', '24939471.4783*1470155264', '33015184.4543*1479407549', '91442494.7443*1477024568', '5776413.66687*1475605932', '1512940.0822*1471329387', '91451087.4948*1488315093', '29894455.0024*1464738437', '21754761.0847*1457675085', '76772344.6105*1453420059', '17380973.6998*1453701543', '69509013.1829*1466270950', '18508437.7354*1456811901', '38088574.5678*1479203847', '79460396.8227*1471781801', '68472110.9369*1457984559', '29277974.0989*1476343777', '84889910.275*1460145199', '25404975.963*1452150718', '99427572.8314*1458750578', '93633863.7672*1467208813', '3011281.44191*1456976059', '1834814.80595*1460194342', '48523031.7582*1454426778', '15718700.9949*1453522238', '3467151.57841*1481560952', '73044063.9856*1463269853', '53500043.4338*1469191167', '72592271.6205*1472004874', '96863233.0874*1479743465', '96795424.741*1464560100', '82987810.2577*1466034349', '18818600.962*1453860255', '51080397.1099*1471473377', '44372742.3412*1486561877', '5541790.13983*1469044780', '65478088.4096*1466760845', '24889982.4637*1473067590', '62881504.1574*1472881426', '5416349.57053*1466691986', '98877767.7176*1487661706', '80117990.4568*1465743242', '64857818.6389*1476584144', '34783758.8628*1464332609', '23330089.8486*1454140970', '37010485.9637*1486381413', '79657405.6413*1474261815', '10497934.8783*1480364782', '52576403.7663*1466825018', '16487315.6306*1454164927', '70168359.6239*1468156206', '4018903.14539*1475182171', '84165711.5678*1454925128', '12939662.9837*1461189132', '22063691.4933*1487836481', '9955770.88954*1454337334', '7791147.79404*1477320523', '52125720.374*1480809608', '9630699.41627*1470693697', '44757725.2634*1480217559', '92693950.882*1454346239', '19631232.6207*1473017955', '29723223.4378*1483097630', '46826589.7538*1451754280', '70715503.6038*1488062362', '59065781.2919*1461577921', '37088777.6238*1477469262', '74450310.9207*1465827219', '40074298.9439*1455378963', '8924369.51805*1459449075', '10093220.9476*1454014755', '67040619.6676*1475773137', '41541981.5161*1456985398', '83766837.5129*1469777097', '40130991.5945*1467007344', '66741010.7055*1472149357', '50007088.0074*1467218820', '37000283.1561*1476295065', '57594715.0127*1465616515', '57063259.3383*1470074388', '99190618.7607*1476387894', '95087542.2024*1459193502', '76239528.0986*1473241025', '46035916.4785*1487974617', '93566297.955*1476883859', '88083495.4253*1468126059', '54636569.5698*1453587326', '55267402.4956*1473506286', '91015967.8794*1466239281', '50250596.4688*1464301967', '78971818.4165*1481257591', '68289000.9998*1453398547', '59423852.476*1456003191', '67777155.5766*1452334132', '33477601.4915*1475282117', '60000693.5278*1473753803', '48257876.0815*1458547049', '20414957.8356*1453661080', '30776938.9166*1465455754', '3121348.76462*1484414747', '69453719.0795*1456089984', '28927023.5459*1471539718', '64677273.8319*1484316641', '8369345.84617*1479298914', '41147148.4542*1464002139', '73431171.3157*1468174720', '77796952.9353*1461221108', '40483996.1014*1463667095', '26063083.9318*1451656253', '37156097.4661*1478661741', '32930796.9007*1469269255', '5325292.30404*1473437387', '71446146.9103*1455905915', '65337071.9216*1471730203', '31464019.0245*1459932043', '88881037.9447*1458858064', '3888160.90959*1485350387', '4879697.21782*1479097701', '81506427.0402*1455568641', '45284947.8599*1454234791', '27551257.4474*1461452536', '65400354.8033*1462106212', '35200172.9318*1480850955', '53523904.8156*1481362743', '33447742.2497*1458356486', '14778821.4247*1485879471', '11698260.3393*1472355578', '27637475.6531*1474871092', '83134231.7254*1454774360', '63989683.906*1460315268', '61088775.0631*1470246741', '37000283.1561*1476295065', '7426499.11909*1462770053', '4692898.36183*1487199223', '66920118.6014*1476792498', '57676642.7681*1464506639', '43224075.7066*1472508021', '46589415.4479*1458245776', '93581546.9602*1470298861', '22373665.3312*1467760761', '38117710.9211*1465505988', '97153565.5369*1472349962', '92424283.6836*1474460213', '85218527.7448*1463334054', '99391716.1311*1462769221', '41834841.3219*1464004286', '62848548.7678*1468255288', '62376455.5674*1486317191', '29590099.7493*1460389559', '7598982.72536*1460966037', '91358472.6477*1487670782', '218706.788306*1469345557', '20643748.7502*1487114895', '2027114.08191*1473105204', '53598589.6523*1482274753', '21337544.0166*1484079692', '8987665.48151*1460925128', '21392076.386*1472532804', '61356267.255*1477720255', '96347317.1491*1455019422', '61994695.5206*1465087911', '36321066.1365*1467987090', '70623844.7249*1472591222', '96483338.4544*1468766751', '34631496.0295*1467443136', '11516277.0013*1486033074', '85644875.3009*1454229565', '15569721.817*1459327391', '41375018.1381*1461986187', '33393930.4994*1463540745', '68689211.2212*1462338390', '23383550.942*1467896655', '18928149.9888*1479176697', '78725827.9111*1462276744', '60206859.9759*1474459921', '80953579.2627*1467799415', '77720499.3968*1453486849', '24868172.6753*1466332912', '66273733.8166*1479844593', '55415562.3819*1453848928', '50482184.3473*1472878317', '80851962.1911*1460851111', '28716599.8232*1456413285', '69005385.1106*1455852426', '1836276.66752*1454270358', '94708574.2634*1465723274', '12149652.5474*1478992492', '52893927.1039*1455832036', '58678200.1232*1457412365', '50847812.5864*1465718342', '69775208.7264*1470360943', '54091756.5235*1487939074', '14509681.3778*1479004749', '63233699.5042*1471771346', '28685527.7061*1458295437', '3226734.9118*1471016537', '44197059.8649*1462294273', '35820295.6328*1461541886', '20685158.2482*1467595107', '38186661.4379*1466892793', '14894217.6374*1478138994', '37574346.1133*1456535235', '34321663.5381*1481074236', '31464019.0245*1459932043', '54986404.917*1453895459', '75741036.3321*1465315212', '41977829.679*1466027738', '30327839.589*1484205548', '54653227.7002*1484258313', '42895215.6664*1463068973', '25971952.4361*1483874687', '71371850.3329*1470747662', '43393402.9804*1474637712', '22509120.5578*1467536479', '86657360.9525*1452635710', '65756845.8513*1462458360', '73484879.8942*1457197692', '80082220.0378*1461581835', '38337085.8721*1453605272', '36610749.1298*1471333041', '73636142.8405*1475043285', '67270690.7203*1484860562', '57155994.1557*1466860466', '82146999.217*1477512614', '34783758.8628*1464332609', '2107491.3567*1461294559', '91020117.2587*1455776452', '80331528.9169*1468110372', '65602861.0374*1458405589', '29466130.2607*1484547516', '93238534.7534*1487365740', '20951204.1548*1458727899', '28397187.6342*1483430299', '45133874.6517*1460796420', '47383485.6141*1462943699', '1370243.99298*1474431780', '36819487.0924*1480053656', '95385757.6155*1486623148', '74193030.9716*1479098448', '23592907.0681*1478561078', '43119330.8845*1465921496', '33879691.2944*1487125638', '24442960.8233*1471021662', '66263000.4644*1473299427', '30633153.6555*1453080014', '48349640.2373*1466301333', '499612.640447*1458254212', '18654332.7206*1477893073', '99827541.7016*1466878400', '25056836.2482*1467179948', '39223996.6432*1464219144', '2140399.4676*1455738843', '17899059.9437*1478362297', '2989478.57569*1458167732', '45312530.4904*1462303496', '17271360.7594*1454728612', '73258263.6261*1473061673', '42972170.3311*1468584001', '60840901.507*1481857884', '97078482.8746*1471852533', '59657450.5963*1486957106', '52420829.8341*1466366160', '27944231.9187*1472225346', '70759400.2908*1479007929', '86741833.483*1473950752', '66554702.204*1469363628', '88111271.6494*1469738793', '9553222.89488*1464698367', '84036858.0705*1466569099', '38264953.4302*1468448794', '74033194.7541*1467842258', '22768314.8078*1475425901', '18964803.7307*1485071639', '73274590.8087*1464705876', '84336407.8736*1452968117', '36647498.2413*1458903709', '81317213.7387*1471942265', '65966453.2699*1458119550', '53573313.9741*1480951462', '98100049.6371*1459264890', '11137356.195*1477752060', '11055924.6643*1476087676', '4926577.73619*1478428834', '34379689.0759*1485215515', '91329380.0974*1482329788', '16823881.0273*1480315999', '83822477.489*1453022075', '97570480.8723*1452937982', '23215325.1986*1485923174', '55360499.125*1465909695', '17902955.064*1467244784', '93046558.665*1478197467', '85638015.5853*1479292764', '87266867.7856*1485370767', '48100653.2662*1452536984', '33609960.4722*1468030250', '31799362.3421*1460864711', '70812958.9654*1475776898', '56369102.7748*1468378950', '87488078.6145*1487122962', '5786529.71847*1472034401', '65712613.8575*1457956050', '64582480.727*1475529729', '43714647.4775*1465423958', '85040873.234*1458154904', '3033092.17796*1485310854', '58958919.3423*1472906706', '1892494.16698*1477201748', '75132587.9059*1461860066', '99176125.8268*1482567187', '18221756.1997*1466412159', '15758497.5865*1487441646', '62630816.093*1457525051', '70975813.3378*1476422520', '13664046.8642*1465614206', '73619546.0829*1472110936', '53202574.9787*1480719578', '64903060.2595*1452539800', '49163485.218*1478591291', '96199298.0382*1484332344', '96935328.5833*1481484187', '45156239.2587*1485487496', '96977104.454*1454824673', '7149703.13025*1454083118', '90602731.2314*1467415031', '57063259.3383*1470074388', '64443372.59*1469157612', '79305178.2179*1476680831', '75948767.175*1469889307', '49946130.9616*1473681219', '70689337.0765*1457348733', '88964471.3238*1475790764', '6612609.17535*1462266365', '97858581.202*1482409439', '72410132.257*1487527668', '22023546.6859*1461552428', '6218152.79867*1456991938', '45562949.8296*1478133262', '78069953.8988*1473959224', '51729466.8137*1474162606', '67252794.6894*1452779418', '56274660.1362*1475708832', '86261904.6105*1456754811', '18213091.8574*1457089218', '10001398.846*1487236019', '90958352.4294*1459467502', '57732128.6235*1482830973', '77846189.6184*1485377691', '9673956.77767*1453782354', '2888994.80943*1467401969', '26225298.5562*1486050359', '73323135.5413*1478831086', '18900842.1162*1477406016', '83262671.0343*1472029839', '62698124.1744*1454547014', '32239757.4655*1486534436', '73509244.756*1465686239', '76508082.8711*1476974214', '25654234.5952*1454307713', '12940062.8369*1466954696', '20121167.9838*1468228401', '34325151.1408*1471545829', '25361541.1108*1468863158', '84871001.1592*1469838746', '48896447.5693*1476659308', '66364770.7543*1458086796', '735065.043466*1484253457', '75539472.0396*1484414591', '55415562.3819*1453848928', '76347651.0466*1477207043', '42723658.1752*1467408338', '69227744.887*1478178972', '69834761.4825*1472639441', '34903526.7594*1462634565', '21569033.4556*1456817576', '79289188.2268*1455091782', '7901011.89513*1487976639', '33660057.3516*1469363659', '85397222.8564*1459972787', '67517969.1541*1485328621', '46281152.7862*1485031171', '9864440.97797*1470406523', '83203318.5445*1471166606', '57082619.3064*1478900236', '27908423.5307*1478570492', '92232929.8711*1486700138', '75537948.7366*1452528419', '38532924.5559*1461105104', '74390744.4758*1471492142', '40026356.6197*1459207091', '79358650.1937*1460940707', '57676642.7681*1464506639', '87794012.8539*1482757364', '57837972.1367*1455473336', '52250880.4471*1473974358', '90476439.596*1473260875', '48759930.2098*1482404482', '83357494.3218*1472640946', '26105620.7901*1466966210', '91139746.3626*1454946135', '74809333.2684*1484805216', '52699181.5756*1461386614', '23455238.3139*1474638958', '38608398.5604*1483933539', '78481332.7742*1462508946', '8887265.05104*1467570445', '11989995.6049*1472291782', '55457014.396*1479425416', '35247994.989*1484387383', '444181.576264*1461826961', '97013000.9486*1452807355', '71109099.598*1456148146', '36159100.3078*1473037616', '38048713.4452*1461986917', '12835569.4999*1469537154', '74142050.305*1485643310', '16071784.0924*1456709554', '96834449.7728*1451860532', '99647988.7359*1452353837', '28093660.6741*1478678842', '21082337.6191*1462925012', '33403742.133*1483273017', '62473459.5006*1473571235', '1795220.37705*1478441481', '282398.039795*1483090219', '38788034.3778*1465659914', '30630452.5144*1454360848', '28295653.2991*1457012287', '66387344.9812*1460692516', '77300737.7401*1476359917', '24017330.6426*1465631420', '15496102.4141*1488279350', '59167254.7781*1487650200', '37039088.259*1466536991', '72451927.9178*1459730023', '94292554.6392*1457584969', '76340799.2704*1472578636', '76971946.4627*1458333715', '16623777.4843*1452167835', '81809064.4565*1471818290', '51058373.0663*1485910750', '18539868.3383*1463067300', '15248960.9971*1470266996', '22771547.4701*1461530870', '22143117.8182*1478386938', '48966622.686*1483955727', '32990834.9701*1475528811', '95793293.3455*1479128305', '45461537.0677*1486842856', '81324703.6758*1470763368', '61018872.3713*1476093110', '72454229.1614*1483791354', '18969184.9716*1460363487', '11810224.7211*1475886827', '97609260.2419*1451995791', '20693761.6631*1455292421', '43935689.1723*1478098317', '38715317.2316*1484724222', '61255858.2041*1471087427', '33764691.8213*1455768519', '12269460.1293*1481561918', '78834250.3192*1461729029', '98011568.0559*1462048714', '90135128.4283*1475934588', '59431928.6084*1467049089', '33391531.6649*1466270247', '59180767.8461*1452960650', '70156130.9616*1472829855', '18603806.362*1468119841', '51289572.7001*1460267142', '61023525.5129*1479075772', '92562119.8352*1456607527', '87115512.1221*1467851627', '98853188.6551*1470337438', '80563961.1136*1483685771', '81369826.4163*1486420347', '64967789.7081*1461356627', '26298634.1078*1480581133', '52124253.1817*1453932661', '86747086.1576*1469423465', '86163521.3945*1463492998', '29767898.4312*1465480589', '74283134.9051*1458093093', '16901058.9003*1486153663', '39316349.0394*1481681342', '95087542.2024*1459193502', '10830809.1155*1461225587', '10768314.4504*1474202301', '86832974.9072*1471872151', '75016945.6234*1455937730', '58362393.6467*1468409114', '78367750.626*1460353341', '60375787.6858*1470718154', '46826589.7538*1451754280', '71117506.1651*1458271141', '64226087.099*1453876393', '20041465.6107*1459056940', '56194109.1718*1461632921', '8718822.245*1469764471', '43575011.8164*1471775205', '59490270.419*1462424576', '44642409.0107*1471536483', '51941894.3518*1462750728', '69443135.4346*1459609059', '53649211.7666*1488031693', '86805686.1313*1479126434', '60161807.1101*1457313892', '96979319.2572*1462908723', '41604777.3664*1486630847', '19266644.9962*1453098407', '39591968.5417*1458246166', '25564426.5791*1482210470', '33121189.6289*1462896748', '49994833.0946*1468614918', '59819496.7707*1459176457', '58641244.8673*1479176236', '84515337.2468*1481759297', '60713585.78*1458565064', '21036879.9521*1460954706', '7351569.07489*1464232257', '61726438.8757*1469023605', '88872444.7523*1454316695', '68072237.4575*1454308703', '25627986.6208*1475886884', '76962041.9039*1479956926', '5296187.84934*1485343124', '91644310.1532*1479964789', '84479038.3618*1478888218', '77870747.0444*1481696067', '12223926.3853*1462930618', '43159873.5981*1469438275', '10873878.1209*1452356280', '10775065.8525*1456126595', '12121971.425*1486094949', '3440662.97386*1471725152', '31888954.6062*1472376802', '57009753.3955*1483183042', '46551084.0706*1484860932', '34484812.4058*1453789740', '72141163.8032*1463788343', '34815719.3275*1461280113', '77655465.8697*1465479931', '79293248.0142*1456658943', '49159775.8003*1454007716', '77016940.5391*1474961540', '45034149.3775*1481535169', '6622128.96755*1456422910', '96852918.9016*1486979018', '42304598.2709*1455369581', '74297659.7397*1465708857', '88217811.3826*1467287499', '17419700.005*1455082313', '51184936.8326*1455735272', '26423885.7324*1483074134', '17944690.5537*1456968823', '1906294.39153*1483116199', '10150296.4866*1484368300', '18542290.8833*1462987107', '87101205.2451*1485893379', '83736895.2246*1458739337', '58938456.0802*1464773046', '10629933.6167*1454178197', '50413484.5558*1475367226', '21331858.0562*1483962560', '84036858.0705*1466569099', '56599281.2549*1473520051', '86582227.6756*1458183851', '8577798.82841*1453278609', '39738414.9253*1463485782', '68710192.0831*1454444150', '56369708.2825*1481804226', '28580269.8711*1458295554', '68208472.6801*1483199238', '66571386.2507*1469476107', '46499993.0024*1479731545', '71498129.6061*1465141344', '23237244.794*1476344014', '99534274.6323*1486736608', '61477122.0791*1486111540', '35062158.348*1487837423', '10068244.3532*1473275925', '71053768.2556*1455277245', '68672085.8214*1464186923', '23239715.6888*1456902769', '9082344.44329*1462093134', '58042689.6294*1483088016', '69870761.7887*1461426005', '85869416.2682*1459830891', '9238269.40088*1477424812', '90226554.4185*1461456397', '41383971.3833*1473514790', '2441385.9739*1467770969', '75630644.6229*1487100226', '15774649.7136*1465916333', '6509732.89041*1480734606', '72638865.0921*1462497767', '58996771.41*1484503600', '73732214.6236*1452921598', '92114783.7596*1487352159', '44293023.2166*1472968298', '42157319.8109*1482997531', '85202715.3346*1485099835', '77109917.0473*1476115826', '36578415.7872*1471886048', '77368803.2611*1483277921', '7512741.66746*1477269834', '83255539.1125*1455357736', '55209791.864*1485128752', '93501400.2542*1462815518', '7808513.64888*1462602522', '31101902.2107*1461083748', '2040888.05964*1478893766', '48515914.8262*1464099685', '5409609.02068*1464643565', '36566253.3828*1487539117', '14555338.2898*1482174872', '2454355.51588*1457712502', '50364814.288*1454244594', '76819322.4904*1461881529', '44199841.4724*1465549270', '34477355.2618*1462041061', '8396722.20607*1454918800', '19972793.8576*1474604198', '30455199.0046*1469234459', '50635590.5992*1482983563', '73169522.2469*1478897191', '38085626.9828*1470644537', '62067848.4761*1458836939', '61198886.4513*1466181868', '58796299.2364*1470809179', '52314608.2621*1461235999', '13991161.0588*1460922116', '28311368.8745*1467278017', '19897413.5274*1452865700', '52522081.9759*1466124316', '24227175.6275*1482804587', '37068145.1557*1469796800', '56031655.81*1470497477', '36575046.5256*1473921386', '41976531.7412*1482655853', '32416503.4272*1461559689', '41080475.8451*1479795956', '59212682.4461*1467329584', '81554094.5888*1475979761', '60890723.456*1459507985', '94481622.5907*1468574675', '78157313.2724*1476664006', '55073716.6513*1460430595', '82221640.0793*1472172747', '18391811.647*1481320989', '28677039.8178*1483690926', '29404142.0129*1484094640', '30260098.2935*1459422386', '9451702.20513*1484345865', '66270323.6906*1469507347', '33434776.8808*1457432565', '99724288.0904*1484299508', '54131396.2595*1472732054', '82501303.6554*1481209893', '4005836.05829*1454130410', '8242966.95828*1476526588', '15399751.2877*1473952196', '64455862.277*1482206548', '1444539.00198*1475010101', '9412205.42725*1463339472', '60484898.0906*1473778084', '62744205.404*1466181736', '40941702.4104*1452668562', '82769619.858*1477153835', '94558557.3328*1485389662', '67575233.6379*1470958687', '90675119.6415*1487138107', '28919419.5739*1480835911', '37000283.1561*1476295065', '28967324.1515*1476052789', '27843119.9168*1477015693', '26582809.8083*1478514041', '74793852.6932*1478749685', '70324333.9857*1460140610', '34046939.1404*1487471205', '39119094.6144*1461019168', '24132717.3603*1459377896', '49850040.3359*1486550089', '49522761.0571*1459509883', '9164975.63668*1456539510', '72409071.7924*1478272515', '12621497.8978*1467395133', '39080452.4235*1455303231', '39859470.93*1452732126', '27461775.3325*1469040783', '64722249.9559*1469839942', '30691412.3384*1457455848', '70904080.5065*1487893453', '78857333.0134*1485332353', '27776879.5004*1477788331', '15071237.7897*1453425660', '95918035.1672*1456884796', '14140817.382*1458760420', '80491273.7164*1462617888', '53500043.4338*1469191167', '84490937.7666*1471160839', '42999316.1263*1472252821', '76264652.3326*1453558558', '9272699.86293*1475518882', '2961691.0656*1456104541', '79837851.4992*1466235461', '30491901.7747*1451948169', '51709094.1427*1467832352', '47330144.5388*1459957054', '4668267.4347*1476567111', '74500195.6964*1452379508', '82836994.4635*1454216853', '76006113.7415*1459515893', '61527660.5089*1466886639', '79241878.8179*1467699949', '47992777.2061*1476655837', '33178415.5064*1467643383', '24864692.8011*1461969360', '66732050.2501*1461283377', '93344603.5912*1484205875', '44612205.2887*1457302840', '51072116.7056*1457080501', '30419376.7697*1462317529', '66465404.1547*1487328431', '45283506.3475*1471461074', '15678486.1559*1456189018', '22665686.0846*1458326321', '42476624.5125*1465945073', '69927275.8148*1481104645', '9150785.91673*1457882295', '89625227.5489*1486363777', '49358421.9064*1473182148', '60099199.1194*1476030783', '34458616.9233*1466398587', '74944132.7036*1460503168', '73340936.638*1471476882', '37467754.1713*1481871110', '13995820.0811*1478748976', '37288322.8006*1456907775', '94039073.1452*1474043245', '59542107.5111*1479915356', '87087395.7127*1479099409', '45181774.6915*1463367411', '91928864.4779*1458670547', '62240158.6059*1455784536', '65005985.03*1486022269', '80831776.1632*1470868847', '32339903.7673*1486903320', '76176242.7025*1458859746', '3478912.99404*1473831707', '87683933.1277*1470817648', '78649643.9106*1479075545', '52083834.8012*1463963368', '10452405.9158*1452650689', '54764643.8987*1472574723', '13532903.8859*1465479522', '28948677.5589*1464268076', '16171331.4921*1464888282', '662795.566333*1471441015', '3045214.75635*1470868989', '97946161.3748*1487223382', '73544787.956*1470655978', '29155333.9908*1488104805', '40499219.3134*1476095630', '10881240.2911*1456608749', '66583531.4878*1471473775', '41524086.9597*1470148718', '99469429.5039*1453559946', '22563996.1981*1451687279', '9451702.20513*1484345865', '71339265.3266*1484380110', '47569805.9771*1466174885', '64186321.3994*1456450203', '14093517.3991*1485117525', '77733814.828*1458282622', '35776496.2618*1480527782', '40058135.0286*1454447518', '77707798.2391*1454253916', '66191547.1227*1467723817', '33403742.133*1483273017', '55478627.2866*1487395937', '34395766.4742*1485398022', '73171666.294*1476179137', '73024401.6895*1472482217', '55521494.3526*1456730665', '94904234.1159*1457930957', '87137952.9555*1475809824', '34179755.7365*1481440909', '60799464.3655*1477756260', '3419528.46254*1478660350', '80647134.7678*1484147419', '22486929.0963*1452757700', '93517434.5465*1464957443', '99703424.8624*1475845026', '75873280.8573*1461864636', '72076512.2532*1469883820', '76173302.7196*1461955488', '73211745.9898*1467614920', '50055428.9683*1478290849', '65287850.1616*1479182317', '54366814.3386*1456672387', '20076491.7533*1465220792', '80479329.9468*1461128473', '10614495.573*1479925704', '73431171.3157*1468174720', '55238281.231*1485461790', '68302180.5875*1454159939', '88097087.3385*1461452865', '69703531.5298*1469362258', '27212129.539*1457094160', '54653227.7002*1484258313', '28250815.9881*1460324653', '92935829.2541*1472089377', '61658081.9745*1469986624', '29857872.4288*1474940736', '90388741.7476*1462128933', '11019816.4793*1456089156', '72192312.9122*1482402046', '40776335.9087*1452218280', '24497413.2588*1483018139', '63369054.6312*1466694059', '72409071.7924*1478272515', '84611888.1176*1469914449', '89678597.8527*1453534129', '42642460.7332*1458187487', '45746069.0075*1487330687', '60227213.7905*1467766007', '86266636.7017*1461617708', '85573609.7516*1475263897', '77487475.257*1480093079', '28242433.7322*1465248810', '76114481.8429*1485351985', '65206465.6015*1476356027', '30979933.3988*1484836565', '87907622.441*1478002644', '78470628.3202*1454643414', '83841119.0499*1474831215', '29767898.4312*1465480589', '60461215.4424*1468155888', '49819884.1567*1471298424', '44167314.8192*1484992764', '11516277.0013*1486033074', '95659788.9923*1465221086', '98782828.8462*1471670989', '31807975.7491*1487874625', '1015330.40217*1451949746', '10377555.5128*1469243596', '49199282.8027*1486347903', '10723226.2804*1475429256', '26466514.4597*1463498939', '51122305.7362*1452903894', '41356276.2357*1484660868', '71312329.8047*1479734139', '30419376.7697*1462317529', '34209594.6994*1464301288', '11112243.839*1459361883', '13453030.5575*1460348989', '7937981.66988*1485232490', '4879697.21782*1479097701', '24374105.3675*1455786985', '49566366.3969*1480670841', '6638469.73785*1474805438', '32529197.3332*1484134890', '58633322.105*1479905531', '89497148.5283*1476477456', '32779060.6701*1468487480', '90752337.3342*1485450857', '61713708.2815*1466480743', '47196893.3577*1468686693', '32159795.2338*1476670552', '89513689.6534*1472329739', '89060632.2013*1468845716', '14228172.3761*1480659475', '58394618.5829*1483362949', '68672085.8214*1464186923', '51929332.5738*1454463929', '81585893.9882*1453053092', '9164975.63668*1456539510', '42786057.2401*1455204975', '17987237.436*1469212867', '56211669.6262*1473500874', '29178823.9082*1465732568', '40196880.6726*1482177480', '84421733.9497*1470611685', '5195116.94755*1487595488', '35820295.6328*1461541886', '68560810.2659*1472806883', '43438521.5088*1464604014', '85518476.8018*1480284809', '49000325.5007*1473055115', '38612754.7682*1457965722', '42194201.2839*1474800592', '75955689.0589*1482723552', '84189064.9014*1487649036', '45846982.1624*1473027213', '19705330.7537*1478685848', '24321710.1443*1480914093', '89281716.5939*1459419476', '42859657.4404*1472298379', '12689478.3072*1456075049', '48103878.0925*1479684322', '91801048.0453*1479253511', '71217869.5399*1482889629', '86365366.1956*1482146934', '15947762.6099*1481087894', '50669288.3019*1466949406', '84575493.46*1472012412', '1640882.96693*1452531371', '81730559.5598*1472710128', '9702808.20974*1487465120', '82149426.2409*1480024461', '8027674.26796*1484624920', '64495701.867*1468544171', '48103878.0925*1479684322', '71527533.5884*1468793128', '25201592.178*1487794550', '6579216.23263*1487848490', '26872018.5645*1454053805', '79451544.6128*1456706558', '51876473.3114*1483969235', '73169522.2469*1478897191', '37629452.8811*1487226052', '88222682.1238*1463771383', '69703531.5298*1469362258', '11407122.7204*1459530014', '26322776.8659*1467708359', '54402696.764*1458780732', '38401874.9521*1454927400', '52973067.4756*1482789027', '76161258.7397*1478136108', '57434551.8978*1487871526', '57625904.686*1465491845', '94008128.2974*1454519860', '12504770.3434*1458081844', '92220370.1146*1487490041', '43733755.61*1457347854', '83323021.521*1461462576', '47813258.329*1462305196', '83260216.3456*1481220288', '92704452.2724*1463225983', '52777609.5281*1477504480', '85279259.4835*1461564862', '40975041.59*1472943217', '34403348.9784*1452280846', '58075437.9395*1471889806', '70835075.1864*1458220650', '67657866.3961*1454309437', '20951009.389*1470014427', '95370760.5364*1455634429', '41654122.3396*1480363317', '56131177.8552*1474110631', '26019115.0678*1465484059', '86699341.8773*1487595935', '75704437.4107*1481556993', '98515884.6422*1460021205', '39920536.0226*1456446767', '38721728.1055*1471244459', '63555884.6037*1479427490', '51029676.3248*1475314173', '31041731.9188*1465757742', '88186919.9936*1455730097', '70859227.0158*1458990609', '88712934.2329*1468251621', '96714678.2599*1464413990', '22139003.6145*1483113273', '40358470.8532*1453502539', '74769664.1736*1477001630', '1131251.37784*1482732848', '3060605.4361*1474224523', '10653472.2252*1487003839', '21644128.4498*1473575380', '53901021.5472*1453770702', '52312647.6586*1468684395', '47883561.1778*1470924750', '71439864.6189*1475481366', '23555438.1598*1467265045', '79241878.8179*1467699949', '6441156.89463*1455400140', '20470352.0227*1465516606', '16286229.6959*1474173392', '94704687.2151*1457357840', '85218527.7448*1463334054', '71446146.9103*1455905915', '44919282.8068*1471117663', '52050641.1855*1487032827', '85441583.2773*1470151211', '22591162.332*1486106261', '18532974.0497*1457511120', '36280721.4911*1464162696', '53627187.6002*1474047085', '65492390.9946*1454282198', '75902619.2404*1453106140', '9509772.26817*1474702980', '22665686.0846*1458326321', '40979324.6359*1472905888', '27971825.1948*1472489099', '82272424.7727*1476007562', '97369081.8135*1455210973', '88844540.0592*1482238749', '18776492.0372*1469541912', '8738300.95406*1474533788', '70155187.3873*1460496018', '2826578.05736*1457526215', '53598589.6523*1482274753', '35006501.837*1483558510', '49566366.3969*1480670841', '75960092.9449*1485984906', '79278801.3745*1470987518', '36046665.8329*1455862076', '11105751.9928*1486810617', '72817185.1923*1463573925', '62588651.2283*1463671986', '9141041.49553*1456252683', '46912709.4378*1461344593', '62303673.5597*1478440209', '64385176.2195*1462822684', '74769664.1736*1477001630', '51103804.922*1464382943', '87110435.3697*1451996117', '86533088.3442*1454296396', '31560822.351*1466491136', '73562654.8203*1469790518', '95522031.9185*1482100215', '9528136.23918*1479086887', '86049173.6893*1458026078', '65678878.1085*1469067541', '84573235.4747*1452420153', '98135070.7824*1465062924', '4517502.41634*1461462941', '61391944.4826*1474586612', '1311082.98624*1453224654', '1130425.84449*1466954989', '92935829.2541*1472089377', '76021526.2399*1452056349', '59466428.6933*1458205908', '53150601.229*1454930246', '99679026.5518*1483742146', '92834886.6366*1487020342', '27893691.944*1483954045', '85218527.7448*1463334054', '34962344.9501*1486548023', '75456221.7113*1465718696', '28947002.1079*1465992022', '12223926.3853*1462930618', '73287270.2245*1468900202', '38318162.112*1472682691', '18796415.7935*1471793924', '39112700.965*1451910519', '3341621.17006*1459289045', '52924900.3104*1476375260', '60478083.4939*1469200422', '88507489.7887*1484106767', '69443135.4346*1459609059', '36046665.8329*1455862076', '36866475.4917*1454133166', '94032888.0251*1483090886', '62663912.5509*1456847109', '23275047.2622*1467136294', '76336468.1858*1485278432', '38530769.0951*1468128169', '94199560.4246*1474173098', '31148601.3596*1462766811', '17235171.2329*1466751331', '80145710.8835*1458277912', '56599680.1491*1479986056', '89073971.0564*1452959901', '4647032.12452*1484399374', '14363382.3525*1484460505', '37842284.6586*1457831748', '46457855.6467*1473110984', '8647680.56858*1461057913', '90752337.3342*1485450857', '88686336.891*1463690516', '33719617.0116*1463951887', '32899768.7341*1475692040', '78261267.5326*1482607609', '69645422.1735*1481851997', '63532760.5316*1485974856', '75439684.7747*1454651265', '52179832.5528*1472541934', '67561134.6018*1473346941', '64785767.3369*1485241416', '46730310.8658*1486083885', '93715729.9444*1473963389', '88496896.1238*1479233299', '87138189.3899*1464380430', '53685159.5418*1471295099', '217276.843028*1460980860', '27173670.6345*1487457638', '70569359.5827*1464225850', '42009337.8067*1475872178', '79159313.3955*1479737281', '20652016.7285*1483472562', '23683587.481*1461740019', '64718885.5171*1467076670', '14444584.3442*1473542592', '9688998.73915*1469213474', '77567345.7389*1473081961', '26249082.515*1469174490', '74064512.8093*1464406959', '95235546.8519*1485866634', '68418964.2186*1458732780', '22229488.8032*1456185892', '11804128.9427*1468888480', '64675882.7971*1476041800', '3636605.59026*1462122974', '75533384.1735*1467270962', '13681662.3072*1475498899', '74156573.635*1476254037', '72035782.1193*1475066418', '67331940.3641*1468547747', '24860934.8689*1473816335', '51825295.4662*1455717073', '94685722.05*1484848259', '13295206.7663*1476928390', '57213904.9022*1458171227', '85279259.4835*1461564862', '77105007.5813*1470608278', '13887279.6621*1469313933', '24231334.6812*1480977436', '28602397.027*1462524869', '4699200.69977*1486262568', '10072842.8403*1470713621', '77312986.1631*1461008724', '31799362.3421*1460864711', '10742436.148*1472323839', '53491701.8775*1453615656', '93801961.1155*1470193247', '47130021.3044*1468906794', '36869526.3841*1477456253', '12784345.5927*1457044104', '3396822.16373*1462995356', '85418975.7106*1460051329', '92480572.6935*1479780984', '78525407.7049*1459161153', '51118969.5806*1481678013', '73160020.0272*1485186803', '62336814.4617*1452174023', '57115838.0363*1486017830', '41478653.4995*1465288957', '11884402.3945*1465146307', '81369526.124*1470808377', '15892172.292*1471795249', '19365623.5935*1465761699', '33280759.1719*1479938216', '12589407.2405*1469522184', '47480640.9747*1460749565', '16064945.9032*1465730240', '18359445.1696*1478631477', '16464264.7078*1478312591', '26932751.0698*1466794692', '69848220.7421*1488224292', '64923010.5738*1486599838', '9222655.84307*1481932993', '37045597.1393*1461424531', '19669684.6904*1455298313', '76816922.9164*1463315638', '70907110.5613*1456019812', '22079247.8203*1454523984', '10120273.5606*1466681406', '26794816.5718*1463520565', '14527738.5797*1460081347', '57339134.8563*1487728355', '49099889.2921*1470107660', '12921423.6753*1477460886', '29469531.3504*1473159829', '2868342.82444*1482409181', '22591162.332*1486106261', '94094686.2281*1460694180', '83260216.3456*1481220288', '80624163.2316*1469670429', '85792134.0144*1461581220', '71634505.7514*1467301884', '55251822.3506*1457782817', '77909209.662*1486418193', '94943476.6081*1463690303', '51381491.9422*1461661691', '36266845.3988*1475319816', '23559715.8616*1455169524', '32990834.9701*1475528811', '42941166.9457*1459850388', '47605266.3738*1481388056', '90991818.9301*1460508515', '53393791.0886*1462085778', '44308477.0351*1452908307', '24096496.3395*1475062797', '38218194.8786*1479892182', '92556749.1694*1466234067', '40752421.0325*1452076829', '73568494.2763*1486931588', '36531052.9304*1465105342', '70849462.832*1462649574', '61501956.4313*1486420690', '80851962.1911*1460851111', '74724336.5292*1485076246', '77655465.8697*1465479931', '28988319.7318*1485985018', '1836276.66752*1454270358', '51603380.5636*1472174390', '61465972.212*1478441926', '14846398.897*1481535092', '77400376.4954*1468460513', '67765814.0992*1466674779', '72147021.8677*1456247725', '58947952.3001*1456373404', '50472225.2008*1460999664', '4496590.05033*1460254622', '76783448.548*1478155657', '69625119.9397*1482719681', '64637544.1941*1471253184', '28580269.8711*1458295554', '34828574.9743*1480030204', '6909793.56052*1460274366', '11804128.9427*1468888480', '55478627.2866*1487395937', '30377619.2348*1485494697', '45927664.702*1484200087', '88890447.4947*1469025060', '66581581.7165*1459130015', '19131482.0735*1467410827', '13381455.352*1468973229', '60289312.4523*1482791532', '18180685.8351*1466431645', '4306822.1114*1468479109', '36128012.0099*1464397334', '9859081.70688*1470057435', '12633421.7481*1473030771', '70812958.9654*1475776898', '51691675.8519*1464720462', '87200192.6241*1487708133', '80439521.5948*1454793536', '86541317.2789*1481507568', '45846982.1624*1473027213', '2410007.74191*1472508232', '11839794.2222*1487242306', '19116954.855*1475978739', '79946646.6455*1464404644', '83640306.2515*1465493018', '60508515.5983*1460107953', '63464484.2372*1453146650', '60671392.1766*1469788511', '75758157.9591*1476825866', '30216070.1019*1453679448', '6309288.09219*1469362970', '49722260.4283*1469162550', '77252678.4909*1460131809', '38108739.062*1460798849', '69164044.8941*1456731099', '59945069.3725*1470740235', '42233985.8408*1460655865', '52957665.0546*1475712072', '26727015.6434*1455240825', '94118015.7271*1475456620', '22811884.847*1472141310', '53536580.8199*1469374501', '10881240.2911*1456608749', '26380767.146*1454518723', '93562225.9463*1479064809', '26396716.9446*1481796763', '55365562.5196*1454966454', '90677495.8954*1458465335', '48473016.8523*1474697295', '53481792.4888*1451940275', '73018698.7471*1480474495', '48321252.7822*1457384524', '62035809.5386*1465986582', '42504986.1162*1487353203', '8248309.84612*1471333910', '43217266.6954*1462614711', '25817339.9452*1461646085', '6422325.85619*1487484256', '24825812.6095*1477348152', '65860966.0883*1471067687', '73817514.4783*1462626187', '26039257.8861*1464962331', '13078796.6631*1471400650', '24589675.1164*1461418648', '61797861.234*1488077237', '30466421.9524*1471629386', '90751405.2054*1474103560', '28536659.1446*1467755429', '99017558.9193*1487668068', '50697897.8032*1483906388', '31695897.1173*1472832641', '77850076.2298*1473414876', '33185768.6036*1455510595', '656285.858367*1467907258', '6786253.51535*1453965348', '16660855.0225*1469084072', '48335176.6549*1462329812', '93631498.1297*1479676058', '63523613.5719*1474638207', '61006354.4653*1481395026', '80078020.6985*1453465523', '47708290.6799*1463195840', '76453396.9596*1470598239', '36292850.4577*1456729774', '2888994.80943*1467401969', '11038047.1042*1465534156', '30618562.5669*1474926477', '40713679.4671*1471365913', '27065213.2075*1488310882', '55746609.2313*1482367075', '18573486.1899*1486387308', '95254565.9251*1468740968', '23561339.5439*1460310409', '67323489.7427*1462544052', '89312754.5521*1469548175', '16907896.1566*1472325742', '28536659.1446*1467755429', '72772427.1889*1486226049', '55040057.3268*1462367103', '47681106.1069*1487588543', '95453455.3974*1460437660', '69732991.9138*1482778011', '49359490.086*1465020548', '52116568.5662*1483742448', '61297146.0657*1465548677', '345559.201072*1483792275', '60305786.7874*1473942706', '67686797.0283*1458052666', '86112667.0774*1485485325', '48875622.5846*1473913617', '48764426.1033*1473050157', '16648353.4567*1467843555', '85792134.0144*1461581220', '69752535.0244*1484765740', '60699190.5542*1466850404', '22616110.2273*1463591804', '12121971.425*1486094949', '75056029.9051*1474421352', '8081223.68732*1482013737', '24965068.833*1487736761', '44836827.3122*1456669739', '41145221.174*1469417431', '49091809.3248*1452125623', '18760505.2221*1453899702', '52822301.0393*1464548406', '90537185.1768*1486586442', '31624333.742*1481894637', '33032649.1506*1472279708', '51571131.4199*1454591444', '74998441.184*1455180106', '23083298.4434*1452144735', '86699341.8773*1487595935', '84490937.7666*1471160839', '79094909.6121*1457818565', '51173150.9463*1480139027', '58147770.0159*1485039954', '28311368.8745*1467278017', '42629352.6874*1468039325', '65206465.6015*1476356027', '10175931.5164*1479866485', '5265684.28583*1465828323', '40375443.6781*1483902395', '78620014.1104*1482770468', '83751082.435*1458599033', '40910656.0417*1455428264', '69164044.8941*1456731099', '77610687.3467*1471155157', '78973597.824*1474130843', '12114500.9015*1463939659', '47343378.919*1484170061', '46199590.916*1465724824', '72328502.5308*1484131680', '13934399.7815*1455443280', '61058921.8077*1460599904', '38275689.0216*1456041293', '73792460.9501*1455628630', '61620276.5449*1466363717', '79125920.7319*1452099865', '5007045.37506*1474419553', '42137556.6165*1478010438', '78151874.533*1485747626', '85450523.8228*1467160550', '51593172.8218*1487033733', '29813987.1726*1473876784', '28491372.7351*1487073712', '99678292.9018*1479970552', '77377732.9492*1475268455', '85872182.6465*1475459910', '84163415.2457*1483415949', '32310475.2353*1471333161', '51365492.5152*1451783625', '93715729.9444*1473963389', '1507210.8364*1482867859', '13999232.3014*1459795085', '60261065.5513*1472041655', '48380556.4616*1484225562', '13887279.6621*1469313933', '77739341.3084*1486206672', '1926210.76505*1486764228', '2868342.82444*1482409181', '24684854.9766*1474675300', '96961551.0028*1484652622', '20110985.7004*1475821231', '18605998.5669*1478242655', '28121120.5776*1461140170', '44075562.9727*1463884577', '1640882.96693*1452531371', '13213881.4676*1462473378', '34824757.1414*1474491307', '9641609.5382*1473935098', '54165627.3782*1482606347', '93536862.2768*1482854470', '38036245.4754*1487849816', '68067315.5862*1467129558', '87953469.0476*1482320511', '55312521.5971*1470744941', '5161996.67596*1469810362', '42230039.3663*1480616093', '66676092.4939*1454707744', '88532145.268*1462768700', '70869092.4336*1462064570', '99471899.2196*1487469419', '74829541.5779*1456998924', '92886696.6247*1486717521', '43386075.8624*1472360507', '58284870.7386*1464607294', '39741710.466*1459002087', '62844240.7638*1478047346', '16960919.2614*1470202138', '72582064.7831*1461118595', '78331710.867*1456204236', '81573072.5087*1474305259', '34422661.5183*1470088190', '32882663.3812*1487496057', '48349640.2373*1466301333', '77913954.3294*1459163721', '60478083.4939*1469200422', '19738101.3299*1479836450', '40760430.7471*1471232617', '77062675.9592*1467611946', '99959974.0811*1460887245', '30489111.8656*1486953312', '31655548.0077*1460349200', '15003670.4346*1453066114', '84268924.2586*1481008727', '36224714.1348*1461536951', '84871001.1592*1469838746', '90754630.8562*1462568556', '32408459.9336*1476384923', '93718314.8823*1468968537', '76656763.9218*1466258497', '55251822.3506*1457782817', '46753495.5099*1477793564', '6638188.24062*1478449422', '56328366.7071*1461164437', '44238075.5234*1465770805', '19119049.9746*1473884628', '71418274.6154*1461519905', '91474688.9077*1481778551', '50199760.8647*1464613506', '19514451.3049*1479134768', '56031655.81*1470497477', '28716599.8232*1456413285', '42166579.3642*1463266795', '8572735.12207*1453227295', '1366897.2551*1478397725', '37531548.489*1483559726', '56100054.5056*1463135868', '39015734.7029*1459768328', '43974356.2957*1483279463', '5912527.08131*1479286568', '5289729.4218*1469357843', '35247994.989*1484387383', '87492531.7852*1464945142', '4207194.90872*1456848431', '64296054.5316*1453335819', '82524199.0877*1476485876', '34643967.6978*1459249246', '14513212.2016*1456179693', '14098716.8034*1455496864', '46886334.0676*1462864455', '66374382.2155*1484459834', '63369234.5239*1463139242', '5612862.6688*1473023788', '77720499.3968*1453486849', '18964366.6582*1478884407', '78950113.6834*1480039622', '96021066.6112*1466466544', '31790500.4877*1452615816', '51336804.8127*1457506357', '12820674.1972*1477841138', '82099920.1792*1466000617', '36476117.9019*1472546298', '91239045.3469*1456715365', '75847319.9474*1481641632', '33854958.1214*1454090337', '76922939.68*1483743149', '37973136.6478*1475312218', '35217790.1779*1459715329', '9507271.79389*1483773577', '18532974.0497*1457511120', '41565322.2779*1458827572', '26164692.3223*1477549371', '3132378.11494*1462676593', '43008515.471*1483295487', '68086433.6162*1482688625', '47700223.5024*1483976968', '28646393.6363*1475531931', '79133208.1736*1454018760', '72435197.8258*1454706140', '9924208.38751*1457427533', '87571605.7428*1462217082', '98063265.4336*1452782503', '18294269.8967*1473322569', '48688341.9296*1481010261', '77849998.914*1487962485', '73723043.3113*1465908232', '71340592.7943*1464176629', '99777541.5359*1470421977', '86266636.7017*1461617708', '64611322.2385*1472951005', '46841223.4739*1467658286', '41356276.2357*1484660868', '2540423.1724*1463036487', '3467151.57841*1481560952', '11818906.1394*1453047910', '8124262.68729*1481755730', '2594664.53178*1468722351', '82836994.4635*1454216853', '26225298.5562*1486050359', '60818157.1194*1456610511', '75067400.5447*1467115887', '45720434.4438*1451982880', '29534182.261*1456290271', '19869611.3695*1461056746', '55457014.396*1479425416', '40822649.23*1471150778', '1442221.23604*1473887962', '29183011.8658*1477244386', '59741667.2185*1481913382', '63433853.1945*1456495496', '27586043.3922*1483457939', '63950244.5289*1464141290', '81324703.6758*1470763368', '90714500.691*1486799053', '3533507.10096*1477064596', '41812803.8806*1468692367', '35457230.084*1465625350', '74132620.6887*1475853267', '42903279.6423*1470815802', '48535500.5081*1471966273', '57884378.7096*1465531163', '16586408.8625*1467226839', '46716752.4136*1464744879', '96758948.9884*1453996456', '58787277.9422*1479225434', '13575620.5903*1466070618', '71498129.6061*1465141344', '14587084.6103*1474693058', '61997523.886*1468218769', '87547072.5905*1484116638', '56452036.1969*1473922064', '46923187.2473*1452909693', '62426791.4266*1474588080', '36083502.9221*1476904047', '48523031.7582*1454426778', '99378203.4409*1473273269', '77583899.5541*1452119398', '89811324.0474*1456531422', '11417825.8731*1472349289', '63634600.6503*1476496162', '11366025.5491*1461770562', '7430176.26206*1452008988', '23498232.4757*1455894400', '48680876.6159*1471313561', '81841484.4128*1465603853', '24353211.7382*1481503407', '7696835.94634*1467629328', '34558737.5678*1483951020', '42705172.0501*1468592156', '71009922.4695*1465487172', '16429861.426*1465786328', '85254631.4421*1473680142', '29063360.1217*1468060836', '55300071.2539*1458991244', '60469294.1469*1456508222', '64838995.7477*1487933267', '52139080.3256*1463386161', '70215158.8144*1451614183', '64251760.6018*1459610208', '1906294.39153*1483116199', '38319550.6382*1472698013', '13190539.887*1485472522', '57510562.9766*1483232867', '7351569.07489*1464232257', '23620718.746*1480033002', '73129907.289*1484490925', '75903063.6773*1484691231', '84564906.436*1453353213', '72431342.8032*1473131133', '43575011.8164*1471775205', '48777156.9147*1457674441', '82864565.1049*1478586864', '7767849.68342*1487287296', '34372543.1894*1477519058', '88441647.9791*1469214671', '47723484.9584*1456789816', '60095160.281*1453025083', '60375733.4231*1474049418', '86603347.0131*1460623735', '15757146.611*1473742453', '48731832.7283*1466279075', '38528704.6038*1464378903', '41713359.9683*1471195280', '33797842.0503*1470312134', '78031589.8396*1462554557', '71959037.5731*1466883479', '81330037.2563*1480823081', '51453813.5174*1464450373', '68014669.1879*1461348377', '44575716.499*1455898140', '64629650.6745*1458360720', '16276836.5021*1461191450', '91694199.6573*1457666502', '70421412.6731*1452505965', '23881288.0693*1459024273', '65383064.9538*1454529324', '52570022.0278*1459779761', '36083502.9221*1476904047', '72395289.9152*1452174336', '94343283.4932*1485823067', '54264812.0733*1464377667', '35106432.1542*1484472696', '61391944.4826*1474586612', '63722798.0955*1455954995', '63484867.3825*1468793021', '33312589.051*1454601221', '8756410.7815*1468764182', '73418637.338*1484344849', '53602995.6356*1466140425', '66828386.7225*1485950894', '57063259.3383*1470074388', '68560810.2659*1472806883', '37426251.7085*1476939820', '46828319.6081*1461009531', '82016660.7408*1481954643', '82045523.6945*1470725539', '49303173.5256*1459674313', '82161118.4274*1459453516', '99947728.0863*1468974940', '63325231.0665*1472062498', '33436254.4983*1479013966', '31799362.3421*1460864711', '23576385.9398*1481844506', '42987661.2077*1457294484', '26607626.9765*1455696589', '84798816.1184*1488213267', '75291194.0163*1460698353', '59431928.6084*1467049089', '67228184.2725*1481905788', '26734971.6546*1475366665', '32518432.5681*1481494075', '79848459.8958*1485501803', '4371461.46684*1477797862', '51965075.8724*1456475407', '66139806.0199*1461236377', '14509681.3778*1479004749', '34618277.4446*1483681601', '82364143.006*1474290322', '91105468.9167*1452324397', '65850632.0271*1479368891', '48968327.567*1475426734', '60076768.0553*1482503104', '14728805.7014*1455922358', '9245693.83073*1479161467', '14787041.6681*1477148147', '63127170.9214*1462935809', '25056836.2482*1467179948', '65712613.8575*1457956050', '92232929.8711*1486700138', '60375787.6858*1470718154', '9065751.27412*1487011118', '11019377.8171*1473689258', '56628053.2632*1467005506', '17544803.0644*1483310665', '16564165.3928*1456548289', '49868492.8735*1465111452', '2281274.11158*1485026134', '42299302.5974*1485108841', '77909209.662*1486418193', '25971952.4361*1483874687', '2820163.39994*1478028237', '77201308.6202*1469277324', '22501373.6876*1470726619', '51691675.8519*1464720462', '31896487.2816*1458710304', '16138873.9918*1477424364', '64341801.8741*1456450030', '16582015.4309*1476138972', '1629635.01204*1480594653', '37741005.1768*1487663110', '43186165.0877*1480064490', '77850076.2298*1473414876', '96852918.9016*1486979018', '80050119.053*1458308175', '55974982.1024*1452054902', '69992067.1652*1481996556', '86701204.5967*1467691249', '32811648.2877*1486260979', '64893584.1454*1474029799', '88623988.0096*1487816688', '79479225.7329*1458770552', '83740641.7222*1475568365', '6820108.5423*1460578418', '12887237.655*1453098110', '63719691.099*1452067003', '35561145.9673*1474220655', '27441160.2608*1464965095', '35338409.3133*1473130248', '98481811.1078*1479957041', '69908080.4313*1458752337', '78836792.0872*1457545541', '45476803.0446*1476187058', '70569359.5827*1464225850', '78970651.7463*1465355977', '10964419.4923*1475774471', '55439264.2834*1468010270', '54664050.5853*1484665770', '46054978.4824*1479358881', '43786908.7241*1472670872', '50957176.7895*1468525781', '10093220.9476*1454014755', '12958869.0529*1479539996', '83882455.63*1484229982', '38040471.2135*1480885334', '67827626.0523*1473624237', '3419528.46254*1478660350', '81354552.377*1453894016', '7359046.86957*1453332005', '4647956.48099*1469618846', '31687572.4641*1467527448', '68114680.7242*1458783171', '74685833.9515*1452337219', '68428738.6529*1479937899', '86533036.4385*1468951769', '34458616.9233*1466398587', '77418620.8223*1462357123', '59474313.2289*1454063448', '34265199.4621*1483716932', '30376735.8493*1477178866', '41375018.1381*1461986187', '21287177.2835*1484645183', '38496109.7856*1485920279', '52602749.3424*1460313931', '78822122.4248*1464105310', '8960179.83439*1486455540', '60282859.7133*1457316671', '84862839.2449*1476387933', '91129427.4597*1457452741', '96358206.31*1478454523', '4669892.60012*1451803874', '55056570.3099*1460046738', '62772920.6217*1457313946', '70870899.4628*1458135603', '71163299.176*1483606251', '52663151.4917*1480645408', '95900807.4678*1474434530', '10150296.4866*1484368300', '80716110.3639*1488191778', '25936086.8619*1476192675', '70968548.8327*1486457504', '74142050.305*1485643310', '79856708.2043*1477111612', '52133195.019*1472405189', '54771400.5629*1471427864', '33131360.1202*1456677100', '36545808.6293*1476154144', '74850304.2712*1475830358', '56671384.2503*1470343699', '68086433.6162*1482688625', '81160025.5994*1477146254', '41709938.4029*1481487910', '48221259.6634*1477656371', '98645230.5123*1453274408', '22747551.6654*1475819131', '22041629.3648*1472665022', '2230611.60901*1485826983', '76381344.6499*1459218006', '36348177.4638*1473225322', '41692196.4822*1459138392', '88234624.063*1476841879', '82834318.7316*1464957168', '87598806.3534*1451714043', '40532073.4075*1479448593', '28378832.3888*1462025349', '43004246.9614*1457975156', '68924180.314*1479168238', '65506031.2114*1469510886', '17587895.3144*1479570140', '67683152.454*1463768912', '6366490.38902*1480271031', '79246807.6238*1468105565', '15854975.7805*1463434328', '62028431.9598*1486072421', '54196704.5415*1482523656', '87051734.9802*1477782362', '88169839.3048*1456864957', '57446550.66*1464627852', '26314005.3432*1486788917', '65262113.6189*1455086878', '37621768.9646*1479133517', '95873192.7046*1454467269', '67536288.6962*1468648745', '35253585.4196*1453023061', '64872604.3226*1475868644', '67576963.8571*1480156204', '51969263.5682*1473754904', '1518175.99896*1479265488', '15485904.0054*1464141833', '12967398.4553*1483508149', '1055490.98035*1457569913', '89811324.0474*1456531422', '30792395.4671*1466984643', '92875278.4355*1465809987', '15041655.9065*1466877143', '85438014.5727*1458628866', '69707399.6941*1453752395', '98442617.3164*1457147809', '46999500.029*1458608623', '33652070.371*1462783774', '1893548.23412*1472528626', '62721966.3212*1461180807', '68136954.3979*1476069830', '35081123.7059*1478036525', '2988313.11771*1461895726', '54128277.0043*1482958826', '58106258.7239*1463970024', '92875278.4355*1465809987', '66686312.4046*1474290553', '32132113.7986*1472011806', '98666943.7173*1468526672', '28040286.4161*1485342474', '21737256.8255*1485339204', '90608159.8886*1451970969', '97965387.4966*1470699057', '1225435.161*1456225890', '96483338.4544*1468766751', '73099351.5714*1466588750', '43390490.3938*1481533335', '12583635.4847*1483814052', '31239080.6217*1485731467', '73156134.5592*1462930748', '91055408.935*1473269183', '91414551.9389*1481863215', '19790837.5039*1477035880', '96415922.9699*1467382183', '24446846.2415*1463817777', '15804655.1536*1479012195', '70302997.2153*1482037167', '49184374.7572*1479767061', '76642212.9*1458444845', '60375787.6858*1470718154', '71297031.6853*1473137080', '43381772.3582*1458121995', '65958369.9587*1457732124', '15405064.7546*1463799339', '74850304.2712*1475830358', '91670289.2591*1480649043', '16558020.9947*1476600536', '56547856.7484*1453064412', '94844431.0433*1471025980', '96835531.1045*1478973572', '12070260.7597*1464862498', '55521494.3526*1456730665', '87629296.8632*1458631016', '218706.788306*1469345557', '49554413.24*1452634324', '30767054.882*1481483829', '42210655.781*1483838940', '54071609.137*1485847265', '7536742.21645*1478245803', '28189169.1642*1463046480', '62336814.4617*1452174023', '98761654.3922*1467921969', '68843159.6091*1485195012', '46055513.4711*1474554935', '15758497.5865*1487441646', '59537681.1182*1464149745', '90231388.6765*1466638115', '83481181.2531*1468194198', '14273868.7036*1487267005', '82185358.6149*1480889112', '76922939.68*1483743149', '10748258.805*1461896518', '2477769.72115*1476847928', '47483653.6944*1474988445', '13681662.3072*1475498899', '40579437.3889*1452566959', '85457842.9144*1484812665', '30229504.2316*1486752990', '57549997.9943*1476976337', '83203568.6737*1463249168', '53588777.3588*1487543037', '78671451.4181*1455241006', '37866758.9789*1470400025', '98877767.7176*1487661706', '12236232.3239*1481788597', '51283366.8187*1483706962', '39223106.3818*1464057941', '16898679.3292*1462244120', '41153182.3173*1453134142', '13816900.8166*1481661548', '89296535.4022*1479881016', '87003821.6205*1467040701', '86676760.3811*1466754823', '99756307.5096*1478848078', '97473939.8165*1455019389', '32230957.6976*1478941079', '38771892.6847*1471753149', '81498631.0665*1479710579', '90823377.452*1471504989', '97278150.1997*1471701643', '7498315.74911*1470921242', '77636782.2528*1472306731', '11212565.4173*1458898099', '62735306.5416*1457541566', '26943139.7017*1476973394', '71638529.3703*1464168815', '33280122.6462*1488324413', '9735334.7757*1485290854', '3565351.91119*1460177099', '66756637.0088*1480906358', '57776300.3249*1461326112', '76081228.995*1472389759', '14985745.5404*1485567817', '89968206.3663*1485300062', '86510728.6461*1487413184', '64027622.6582*1461688585', '23840078.3162*1452438952', '65287850.1616*1479182317', '7619038.85412*1478615438', '26606885.4645*1484859946', '24269831.5122*1483648227', '38897853.7301*1461369451', '45780680.8845*1458049270', '80250712.7213*1481483763', '39579886.612*1485959066', '95709674.6959*1452618393', '25423345.5344*1469442492', '62824208.849*1476121429', '62167054.5529*1472097977', '90916913.9733*1477212115', '50146935.5462*1459676512', '97618283.1517*1474913077', '8662738.12084*1468913113', '33506387.4574*1477705407', '66139806.0199*1461236377', '98796579.831*1468598043', '3533507.10096*1477064596', '67223307.4138*1487922432', '48174124.149*1451893789', '44329461.1557*1461516477', '59832335.3349*1462824726', '24456180.5663*1487567794', '11084582.2715*1460464582', '43108556.0003*1482556639', '87547072.5905*1484116638', '41180138.2877*1484497032', '3694514.27684*1480212333', '37039088.259*1466536991', '61411703.4293*1457140502', '73992442.1539*1463196663', '80101751.0062*1453539206', '25244801.2421*1462847337', '33973142.4102*1486469301', '23561339.5439*1460310409', '17176949.8498*1485858690', '94163063.9257*1460628879', '65057797.2067*1478338616', '60040967.2969*1474044948', '96956113.0642*1458855814', '33755995.6557*1471640949', '10515728.5459*1455152563', '15029390.5529*1476658818', '42476624.5125*1465945073', '6213503.94856*1464606992', '8235988.36485*1472386394', '18573600.8108*1484868100', '99068085.9083*1478310663', '67442844.9135*1479872717', '95414096.3556*1461321727', '86046448.9323*1478586617', '98562221.0291*1468809304', '5208402.47673*1483666591', '98986360.4564*1465657265', '63369234.5239*1463139242', '41098483.989*1480489668', '68302180.5875*1454159939', '51110822.7473*1467205953', '43390490.3938*1481533335', '31484661.102*1487476702', '49518278.7176*1478397318', '2107491.3567*1461294559', '59581187.4753*1475007140', '33286295.5917*1478668217', '62145254.022*1483616470', '91339123.184*1476122333', '10999143.9026*1469196956', '19215898.2575*1468539516', '58684815.3662*1486338503', '78971818.4165*1481257591', '2540423.1724*1463036487', '98336893.3742*1470474509', '19772019.7786*1455878574', '73230998.1825*1479774613', '52184774.242*1457187172', '98091838.2836*1485812245', '11055924.6643*1476087676', '62772920.6217*1457313946', '22509120.5578*1467536479', '85146246.151*1462923231', '16921679.1472*1484385450', '75130829.0712*1483462180', '62379270.9993*1454743217', '30818557.4306*1473385120', '24321710.1443*1480914093', '88744125.5219*1452451397', '15572718.8555*1484427757', '25731776.4397*1477060238', '78527132.0579*1485219612', '34506568.0672*1451800725', '1626597.66492*1478322518', '30825818.3896*1486455275', '15071237.7897*1453425660', '96758190.7387*1480391619', '75113570.4399*1466819026', '70584092.8439*1453466660', '93437732.1041*1477804679', '62830928.9536*1475712185', '99094571.9362*1465823914', '39712716.6435*1463519243', '83090276.4741*1458466293', '5925585.15938*1483792918', '93332667.9608*1462740361', '50406593.3627*1467800837', '6496729.02235*1478472189', '98060328.2067*1473925079', '75750243.5551*1478573972', '87492531.7852*1464945142', '33714119.9089*1480372228', '38060578.5636*1462276549', '73604734.4149*1487806079', '25418849.4948*1470613958', '77346274.167*1482770888', '88953670.0063*1452525036', '14808858.6973*1485594597', '25257137.6933*1462438357', '9041072.42141*1481684635', '70269561.2469*1468720152', '44789969.3793*1480693228', '86221443.9493*1464391526', '302347.438796*1486400140', '89056229.6533*1465390354', '11561070.8708*1469295631', '3385550.52216*1463910293', '13381455.352*1468973229', '78822122.4248*1464105310', '46856955.2808*1469547432', '19492320.0281*1473058837', '51058373.0663*1485910750', '53880092.6318*1465451603', '67360685.7634*1474144919', '49240939.9941*1471697833', '71759941.5083*1454551879', '89484061.2641*1474400644', '82317929.6206*1478586028', '95370657.4604*1467144343', '31651706.5082*1459748022', '79657405.6413*1474261815', '84681574.1415*1452660679', '73935679.4479*1460668165', '87241941.9821*1464107365', '69459708.3058*1483012879', '64046322.9559*1487990857', '85869416.2682*1459830891', '12449933.8898*1476546185', '48314182.966*1482687678', '23440274.108*1479605011', '87137952.9555*1475809824', '83558064.9957*1459590149', '88163403.3494*1457872106', '69146566.8396*1481694187', '76116482.6901*1470868307', '14351434.9642*1454955349', '89379837.6771*1475369898', '79788439.5822*1452581752', '58394618.5829*1483362949', '42158678.5233*1483835663', '37056929.759*1456520059', '15354450.0987*1451874436', '20090169.4441*1475846389', '96669064.6669*1454454649', '97433409.9162*1466617345', '84791722.5095*1485254285', '80485077.6253*1476549412', '78157313.2724*1476664006', '86363736.8843*1467257908', '21760468.5578*1472761952', '44216482.4641*1481790426', '9507271.79389*1483773577', '779560.304345*1487645551', '51195621.9137*1475558321', '92886696.6247*1486717521', '76048608.4201*1467888068', '6456956.43465*1471917573', '47605266.3738*1481388056', '90729492.066*1482742588', '28638126.499*1462832393', '90549312.2717*1455989104', '27593617.5653*1477232637', '89532437.9129*1485601049', '1925718.07969*1469564089', '2846681.30712*1487695972', '44783400.3886*1487070185', '76564146.18*1454255517', '9528136.23918*1479086887', '53015838.9812*1460746045', '77598682.8298*1459001263', '47651756.8542*1469625789', '71491549.4371*1485064244', '65770364.617*1476028603', '51820988.6722*1453745718', '1447656.23883*1460235288', '34662098.8084*1465824617', '65651403.4909*1472983704', '84671546.6193*1466574574', '98165205.4938*1458699184', '64970788.1789*1482305492', '68101368.4235*1461515444', '19994594.4662*1475763799', '94558557.3328*1485389662', '2410007.74191*1472508232', '23270783.0132*1457075383', '29064185.0125*1463436025', '61363836.3338*1483990167', '83766837.5129*1469777097', '73424154.0749*1454777253', '58338576.0766*1462668333', '37382595.6332*1471750379', '87626001.0719*1487841447', '17944690.5537*1456968823', '17274299.6798*1471828669', '46199590.916*1465724824', '5618402.6286*1488134169', '14170915.1412*1456303924', '49861694.7429*1471569908', '11138246.1816*1461215940', '14006591.6563*1476103846', '10072842.8403*1470713621', '77545515.5898*1462402759', '28975202.5753*1462889313', '19872889.7784*1469364936', '82274925.7994*1471418985', '93737303.2451*1474871413', '6642316.36418*1474638831', '88765682.1899*1479069308', '8435871.2872*1481448496', '80445045.46*1455938890', '54502767.6129*1462579693', '34993036.0262*1453912891', '42218203.1642*1488069600', '70784883.527*1476006192', '85021411.7252*1466665566', '75750243.5551*1478573972', '42453581.1596*1470828917', '89561876.5661*1487397357', '32597001.3259*1460988013', '35433858.3154*1465201706', '23247568.1174*1463118212', '62122395.0145*1463463180', '1561778.40291*1456288436', '27935303.6*1483230759', '65555592.5692*1462316929', '64074495.1618*1458366638', '44399922.6188*1462083228', '85021411.7252*1466665566', '23520289.5666*1454888909', '8308362.84739*1467716594', '15711940.9217*1463900541', '40414006.3895*1461663548', '24018531.8862*1451735657', '30221090.3745*1473468473', '47989243.2056*1482539191', '47218809.8438*1468230199', '85442660.4974*1483700101', '97841720.4333*1457975195', '19729609.5868*1486649895', '71059729.4991*1468120668', '41864032.5573*1470440858', '28536659.1446*1467755429', '75838728.6047*1460976527', '13008130.4734*1459444108', '10255507.2701*1455745939', '70329705.186*1487357829', '97333450.8488*1466054384', '27987121.3248*1458527483', '82724071.3556*1462937136', '69848163.1312*1480472652', '44426745.2018*1469580816', '79821513.8163*1473822106', '59830056.1125*1482523026', '61836698.4947*1458664283', '9673956.77767*1453782354', '81146458.5462*1481965506', '43638969.226*1477025632', '54165627.3782*1482606347', '38318162.112*1472682691', '52392096.7086*1461293353', '32288344.8016*1484717867', '55789174.6814*1476915300', '44530543.3954*1480548587', '81309802.1304*1464462524', '34815719.3275*1461280113', '95627020.4628*1467236668', '77218415.7601*1486168809', '74155452.7124*1486474655', '8900979.45529*1477574218', '24250544.7228*1487308841', '36708916.657*1463111804', '38386328.7897*1483209771', '68121445.3491*1485667155', '20697193.4348*1488199042', '49210909.257*1472531241', '32238900.2391*1486213428', '1054985.81296*1486819922', '64074495.1618*1458366638', '2961691.0656*1456104541', '10931901.9811*1463769945', '52054359.4987*1468402935', '91258000.3181*1457162563', '97421042.9157*1484595567', '6151387.59684*1481065419', '8900690.63146*1467747443', '27401989.7064*1455964958', '49439611.6075*1463594975', '43376410.0515*1459633073', '3327476.37007*1463553054', '74292393.6778*1468677163', '21082337.6191*1462925012', '13068919.9882*1481053585', '76691224.1334*1478548413', '4662681.71962*1456387692', '90084153.6488*1469882471', '55590220.0839*1486481907', '39112700.965*1451910519', '27766393.9398*1468685011', '52093050.6308*1475438545', '2030180.53075*1454160043', '80755256.7828*1453576375', '75630644.6229*1487100226', '90711950.4307*1466870854', '89316369.3734*1451842707', '21465897.001*1454520979', '41010865.3548*1474082914', '68268229.9101*1457472289', '77459474.7603*1481489311', '85572940.9441*1482469284', '21883794.9296*1459221090', '18180685.8351*1466431645', '60901383.224*1476612282', '48405624.873*1470994137', '20159546.2475*1487362371', '49125812.4463*1477963871', '77153822.7913*1465856667', '5813731.46999*1474587624', '53716599.7991*1459550471', '21752957.3912*1466112620', '68485347.3839*1477072856', '42066534.9743*1464783859', '68072237.4575*1454308703', '49432924.3317*1459558573', '21278894.3713*1483435555', '13556673.4518*1453687587', '53886579.212*1469984605', '96476864.9666*1472986135', '96087696.6808*1480675318', '84180967.9378*1473198293', '37597903.879*1451905921', '2229330.63609*1477641464', '42791738.6364*1483471781', '38108739.062*1460798849', '48547539.4767*1483767063', '6804121.9526*1457755035', '11017495.8161*1473127905', '48269743.5467*1456112756', '48667889.6678*1459018458', '36659799.8728*1473095166', '59832997.035*1487851146', '10510543.0401*1458983580', '73273816.2432*1488183475', '20755670.7466*1472736532', '94071317.7885*1456846084', '49236328.2958*1452741984', '63486859.6117*1457815336', '40170478.1809*1456892618', '60494887.8912*1452287217', '43504158.2312*1456153760', '46906061.2759*1457446853', '51720764.3851*1470908639', '46117766.82*1479366260', '12374027.2959*1473725544', '39921361.8036*1474848672', '11810224.7211*1475886827', '98453316.8101*1485228569', '57632315.0981*1484077297', '68384710.9616*1461630737', '62824208.849*1476121429', '99331192.5664*1476395285', '46952835.2524*1462784866', '1793283.39768*1477639868', '70793321.9541*1470435965', '39187240.1567*1459259673', '76006113.7415*1459515893', '32151097.2642*1460937470', '66171203.7524*1463261678', '16681568.3348*1452945101', '41342521.5562*1469278271', '74035816.1059*1465798762', '95957025.2934*1484524702', '85895393.2028*1483592994', '66850907.881*1458674135', '63471345.7947*1474714536', '31573916.5515*1484115841', '45940806.8599*1454059035', '74648203.0342*1459451772', '32151097.2642*1460937470', '8944057.25797*1481950366', '51418771.1968*1472454861', '42895215.6664*1463068973', '85005953.0879*1477186673', '63219672.9107*1478735326', '26466514.4597*1463498939', '87347353.1101*1474100847', '1483843.03367*1476968773', '63403998.2208*1464085772', '72521501.1454*1476063173', '44347598.3204*1468715550', '72627297.4369*1457760321', '53530150.2023*1472820449', '33679401.6453*1470195597', '35104596.4035*1462847179', '73910687.1881*1470552236', '16867283.5578*1468877045', '58734823.5516*1453260668', '5208402.47673*1483666591', '57082619.3064*1478900236', '72971585.3346*1483283112', '59952940.1409*1459463363', '80250712.7213*1481483763', '30422728.8492*1475883524', '71749442.624*1478281513', '14717350.1127*1487774199', '97365494.0548*1454805916', '40927661.3095*1454787953', '58045987.6051*1466793084', '50093309.0369*1476540306', '41356276.2357*1484660868', '16561751.7119*1474003112', '34403348.9784*1452280846', '98496600.4419*1460893998', '90509758.8625*1477604257', '26380767.146*1454518723', '28389350.4614*1478056851', '57566438.4437*1484660064', '57936097.0239*1471951187', '58619703.8918*1476901382', '32859322.194*1464850969', '45094120.4477*1469451065', '3897003.20025*1452507859', '39088990.7467*1460325884', '84435816.85*1452113567', '83658444.0667*1457278468', '40937793.7074*1478275253', '20354762.0873*1458878665', '6573932.90014*1470414049', '80109032.1189*1465472691', '90275912.8709*1483553720', '73807495.7207*1461837730', '46369288.8893*1455077902', '75247896.5889*1476328295', '79002317.8937*1454116165', '72215971.7722*1479533685', '42844658.0273*1455608978', '46861124.5225*1476258070', '37406639.7165*1462993430', '75130829.0712*1483462180', '48439702.0122*1460197501', '99946140.8864*1488225550', '66711775.706*1463731487', '22486929.0963*1452757700', '24585893.4524*1453917717', '89044889.5496*1465160327', '72458990.8205*1487270433', '97024683.2408*1483694077', '11597204.9644*1473231598', '65850632.0271*1479368891', '19320282.0072*1478242046', '94184513.0203*1470270544', '55859580.6477*1459298982', '55327541.9187*1467749667', '53864449.5275*1475535991', '11401741.1709*1458417780', '38201739.6372*1477638493', '65567736.4888*1484756668', '10902984.7346*1460088088', '58646021.0397*1462598541', '30332640.7714*1483280629', '32542320.6039*1460449728', '29782880.7629*1453560825', '76172426.7916*1459049036', '90895900.2244*1475860032', '60478083.4939*1469200422', '2449353.86226*1467871746', '73716059.5594*1474939345', '37921287.4445*1472897154', '95635964.4222*1463751606', '49236328.2958*1452741984', '22765309.3811*1465822242', '89810660.7087*1454875861', '99739557.3378*1468928139', '14841020.5946*1476950986', '30388979.5216*1468954112', '74216305.7274*1485381356', '83341467.447*1468036345', '70419964.8362*1481258310', '41805671.7484*1479346611', '47668950.2266*1454706673', '85343812.271*1475444036', '83401996.51*1473139082', '27015809.2721*1453945396', '32769196.3942*1465297971', '91174323.8342*1471386487', '19666812.233*1455712092', '93718489.6305*1454955816', '92956685.7032*1481122971', '84112886.1616*1479245744', '19804215.3784*1461272340', '12754412.6065*1484705919', '36673181.0298*1463392107', '46512653.5552*1478396539', '46466172.9982*1458382239', '51674728.5157*1453507345', '12511036.4686*1464679990', '37400775.9637*1468144898', '21754761.0847*1457675085', '17707334.7873*1487567795', '43040753.3069*1466485105', '9579948.9644*1487890514', '849632.118531*1465333929', '63161546.8711*1458764822', '90010322.1216*1471154090', '89241819.373*1481081215', '71389296.6306*1472742490', '82752837.6116*1478979896', '48915884.1884*1452804298', '20617343.8291*1464087228', '58958919.3423*1472906706', '4039376.0519*1462531839', '12593603.5607*1484091996', '51170442.1111*1477995168', '70197492.7312*1461707118', '58549357.1447*1456252402', '62901208.1481*1472019055', '32779060.6701*1468487480', '39204750.9195*1486972113', '24939471.4783*1470155264', '97997265.9782*1476538054', '42881979.554*1472457984', '40499219.3134*1476095630', '66173583.9285*1487668949', '91758211.7684*1487271551', '66727903.441*1457562150', '59212682.4461*1467329584', '45461537.0677*1486842856', '94685942.9549*1462834297', '89513689.6534*1472329739', '73235175.5704*1467804760', '51578475.5741*1472504020', '13198152.2595*1463169313', '84521429.8952*1459585568', '76256523.0953*1477305295', '32506216.2746*1468232872', '1447656.23883*1460235288', '80145710.8835*1458277912', '78389595.7491*1477733736', '99692021.5598*1465190897', '68208084.3282*1465110753', '26798435.4896*1455068143', '67392083.5001*1467573961', '49052055.1287*1482062953', '17420058.5706*1463712632', '36815684.2546*1487568097', '86023233.4527*1475797134', '55172083.9645*1488146854', '79317774.3449*1485192466', '49994833.0946*1468614918', '5618402.6286*1488134169', '94198870.5164*1464257722', '71868844.0765*1452768022', '77796952.9353*1461221108', '43003968.0941*1468223932', '32798096.704*1451762811', '20845118.0977*1476187553', '25244801.2421*1462847337', '86470081.2495*1466184151', '1392076.02665*1465536516', '51361623.1669*1466580876', '33825480.4378*1470607571', '30656314.4597*1475708899', '30466121.8855*1470293286', '61214170.2446*1468040427', '59021021.777*1452929465', '76057556.559*1464616225', '55341124.2534*1483458864', '32468556.6319*1456272370', '62551103.8456*1463625875', '29985815.4102*1456569587', '98275100.0665*1458579599', '21222056.2938*1468721049', '99082733.7184*1481127920', '90384215.4634*1474024282', '534105.792209*1467480272', '83961403.0296*1482010826', '82221640.0793*1472172747', '12610921.4381*1476237351', '67042966.31*1461960789', '69670186.7796*1478842663', '80562399.7877*1473337341', '52250880.4471*1473974358', '30466421.9524*1471629386', '47409062.5669*1470691069', '45466510.1955*1455045975', '94412803.754*1465533840', '38667797.5696*1470673729', '74480974.2122*1480574561', '68771495.546*1472142361', '36070610.0145*1488142338', '13810091.682*1469463735', '99917456.6405*1461110843', '55932497.8537*1474672445', '31462229.2474*1482716886', '87701746.7054*1462164203', '75805962.6895*1453690171', '4044969.13112*1453676312', '71178846.7049*1474420426', '67231104.5146*1484891021', '47419197.6319*1462780120', '72382030.0218*1483983032', '10001398.846*1487236019', '90058332.2107*1484636963', '48959229.5883*1462573466', '54672341.8154*1451858456', '30207037.8811*1468631478', '29813987.1726*1473876784', '62372184.7214*1467121831', '68113166.4446*1453517484', '28328049.1785*1455685882', '55073716.6513*1460430595', '91795362.1516*1455659233', '99215571.2529*1486317003', '35389920.6664*1464623470', '28491372.7351*1487073712', '302208.621262*1484945175', '81469911.0877*1472817387', '81406121.3902*1478147432', '9857972.24691*1487599699', '43924632.4535*1466812943', '98237775.8726*1479304008', '34811370.3742*1459613912', '55705686.2248*1456510080', '99378203.4409*1473273269', '68040871.2762*1467527703', '73283277.1522*1472837555', '98782315.5255*1485720392', '65567137.4563*1455099366', '67020962.2655*1455384453', '33973142.4102*1486469301', '83769212.4953*1456196338', '17927734.0038*1474791853', '35246403.5361*1466053432', '16404614.4862*1485370228', '51578475.5741*1472504020', '14834543.4846*1467692082', '84782065.383*1481478070', '44028835.042*1472335071', '83503818.3001*1465173985', '66691074.5616*1453084155', '54978772.2292*1475322717', '2888994.80943*1467401969', '85812518.5692*1460756076', '83255539.1125*1455357736', '28571541.6915*1460440296', '10294931.9286*1454189142', '21338864.2679*1484150578', '81469911.0877*1472817387', '90084845.2902*1465890197', '39175958.1057*1457131177', '69769720.209*1452273534', '58994196.7724*1480988378', '11104671.9594*1451997801', '92407430.898*1476175856', '61297146.0657*1465548677', '33121189.6289*1462896748', '67085582.7044*1456468100', '10060277.4428*1452753019', '42002864.9403*1453441705', '61409531.1143*1465233237', '87904492.2839*1460382868', '3971278.63611*1471273805', '48351717.5339*1474879917', '2213066.37318*1456081366', '54072689.6547*1455276167', '43616074.6972*1472161357', '72171041.227*1456347059', '22283919.0268*1483015754', '75464758.6297*1482244068', '40309819.0057*1479708247', '31199220.6085*1474614756', '22194778.1715*1483841652', '23668089.9853*1469055418', '14090904.2999*1456486196', '91440132.8499*1475206295', '13794527.1983*1463851768', '89354139.2179*1484935659', '92729920.6667*1468168005', '97369081.8135*1455210973', '1614281.71698*1473717103', '85518476.8018*1480284809', '21639639.4327*1485688176', '39859470.93*1452732126', '36066372.7832*1464953573', '57968026.6529*1485661134', '68084961.0552*1465346303', '71638529.3703*1464168815', '40668601.5994*1465967414', '76034044.8563*1466653759', '7015289.72254*1483307602', '87120900.2168*1452830255', '24210791.4819*1454140557', '55803577.1904*1473845147', '81045.4824318*1461960956', '86091822.156*1466579245', '24142762.2775*1487818374', '56556834.1594*1483641070', '47467807.6579*1459409801', '24193138.9116*1454836268', '64923010.5738*1486599838', '61411703.4293*1457140502', '99427503.7312*1454603259', '32680640.7424*1463958097', '66849830.6858*1487709102', '13041647.3499*1481567877', '74997482.7321*1464757153', '19789820.2759*1464288667', '12631310.4323*1454967030', '66173583.9285*1487668949', '74292393.6778*1468677163', '7528560.22805*1459557175', '99096670.6952*1480824195', '35588764.0389*1482570962', '72057664.2803*1484252915', '61552598.1548*1463231993', '90244288.1337*1474425409', '30776938.9166*1465455754', '6107498.06678*1459090944', '95806711.1667*1462573393', '20509483.1303*1453738708', '43937656.2289*1454594623', '16487315.6306*1454164927', '21245090.7102*1483395846', '92254152.7069*1453822041', '79738674.2631*1458221033', '23990902.4197*1469858062', '13681662.3072*1475498899', '17245567.434*1456786777', '52949778.4413*1474438429', '2007705.49825*1484754101', '31244583.1581*1458484304', '8584362.27134*1483233028', '74648203.0342*1459451772', '81116613.6332*1484216773', '65046599.917*1473089822', '42736769.5717*1478613090', '14502772.5131*1465842707', '46054978.4824*1479358881', '38624018.2526*1475369025', '93021003.8795*1482472677', '50963649.6318*1453610104', '67469922.5528*1470591744', '62304963.003*1487079590', '58016927.9915*1483750295', '72967983.2165*1478834027', '12784345.5927*1457044104', '21338864.2679*1484150578', '83341467.447*1468036345', '22668489.4587*1461221489', '99447046.245*1462563432', '3520191.53514*1481469737', '89666329.2277*1485667426', '98265665.69*1471855341', '1640882.96693*1452531371', '23936209.4933*1478279161', '37862482.0688*1485296699', '41307428.349*1465136534', '52445995.305*1464344767', '46672497.5694*1480203763', '38509921.6902*1472232329', '22206268.0643*1464934763', '66453802.5573*1470231906', '17786219.4649*1457166548', '94724383.6248*1488195323', '66248746.476*1455886292', '3175879.0992*1473425727', '33679401.6453*1470195597', '51353633.5165*1472426879', '47480640.9747*1460749565', '20544171.3957*1463168740', '61880877.1842*1476231115', '57009753.3955*1483183042', '60987331.9742*1455758327', '4654470.79912*1484397073', '99844263.6196*1481869012', '87631300.6167*1464277982', '12520091.5649*1479378400', '83391233.1489*1463825009', '13794527.1983*1463851768', '76116482.6901*1470868307', '34656389.3142*1470530088', '41628146.9963*1456559381', '82757331.2816*1466513720', '69405888.5325*1483384043', '15347882.9398*1460053950', '33955005.2116*1486285764', '77924376.1714*1456305443', '25863234.8064*1475841735', '69873253.3966*1485393084', '51260931.4799*1470239313', '10146916.2639*1476380636', '26877585.5557*1456684415', '78396363.6625*1462917020', '53425206.9926*1466750935', '1893548.23412*1472528626', '56569346.6635*1457808504', '85308125.5255*1479420626', '78114318.2118*1460350385', '11946978.1565*1476678185', '11359367.0556*1463914113', '4764334.23253*1465171671', '40937793.7074*1478275253', '39203037.2748*1454131563', '73000463.882*1475877035', '93076947.4072*1458053458', '69396261.417*1470684073', '34928214.5046*1465293941', '14927737.9569*1459399708', '62122395.0145*1463463180', '28108252.5499*1473652085', '70156130.9616*1472829855', '9853777.7685*1475746064', '24868172.6753*1466332912', '63374492.4768*1480045530', '25571426.618*1485291631', '46090094.8802*1469734319', '91432736.7086*1451924664', '89318503.7499*1468763523', '17038265.4798*1478504455', '60491918.8731*1455825453', '74998692.4327*1477826210', '32968586.9542*1477389423', '87591395.3574*1472595128', '16089025.3039*1470234999', '18858102.8061*1476787498', '50525629.5595*1470723659', '61432944.4897*1466112214', '25651379.1294*1463930382', '38602402.0776*1484775074', '11346374.1505*1462469782', '27551257.4474*1461452536', '32537416.7752*1481642413', '59474313.2289*1454063448', '14757737.8339*1478132573', '84111944.2241*1456057928', '45530659.309*1484451054', '80169379.4315*1459416969', '11112243.839*1459361883', '35263198.1774*1482245788', '60256279.791*1474818816', '44700353.9375*1480940841', '28969840.2448*1461198205', '54462599.6131*1482823977', '92541963.0889*1456685834', '902932.660257*1486759164', '97061262.977*1458904883', '47723484.9584*1456789816', '92260679.8405*1473117614', '22639783.5852*1468596472', '3520191.53514*1481469737', '39591968.5417*1458246166', '19728165.0971*1487450336', '8545538.96743*1483313037', '40294232.0352*1480986308', '80689447.3735*1485790278', '59517476.3719*1471737456', '75010356.6128*1454194033', '87907622.441*1478002644', '94048231.8719*1480723382', '53117332.6542*1479258101', '25093872.1983*1462888686', '36355963.3929*1479637413', '30459347.5771*1454707721', '1776806.00595*1464357264', '13024300.6596*1453482876', '53685159.5418*1471295099', '55169364.721*1481336532', '27003688.1106*1464844011', '55620041.0767*1463417701', '4473070.81838*1454307056', '82326055.6943*1470030793', '73418637.338*1484344849', '28122892.4373*1478568563', '26289131.1636*1470799684', '35967751.3466*1452714322', '18054589.0476*1485877274', '69240732.6012*1461308718', '87683933.1277*1470817648', '27898005.9732*1453709300', '21218115.0859*1480857510', '74566869.5402*1456093211', '697518.571184*1460813044', '34723828.211*1464506671', '76679799.1034*1482876227', '75411533.6052*1462607979', '75110761.6926*1487972829', '42881979.554*1472457984', '79867781.7952*1460948113', '60161807.1101*1457313892', '63031210.0137*1482110701', '69660346.4013*1481569469', '39948870.3231*1472689910', '46753689.1803*1470645299', '46956853.7863*1468231242', '70156130.9616*1472829855', '5897169.7524*1475050191', '58958919.3423*1472906706', '88953670.0063*1452525036', '84935964.3606*1474403822', '32899768.7341*1475692040', '32680146.902*1453899785', '23023728.2825*1483357755', '58284309.4528*1487059955', '85202715.3346*1485099835', '17444936.5291*1451871549', '46616907.7733*1463757050', '62945965.045*1474570798', '28295653.2991*1457012287', '6814175.6635*1461313763', '9082344.44329*1462093134', '51049254.6522*1468376643', '73191665.4191*1468049263', '86495687.8789*1465902329', '16062339.956*1468054396', '95923418.1025*1480527352', '20157422.1309*1484830064', '16699659.8163*1479416187', '74090397.6521*1458080011', '31449165.0112*1457386756', '57570886.9118*1486796862', '99228627.1711*1477153134', '11098297.0701*1453008152', '48145173.6396*1467279034', '15934385.1609*1484787732', '25946433.295*1473020118', '23237244.794*1476344014', '37382595.6332*1471750379', '90948981.3883*1458328457', '94979756.5695*1474737585', '25132686.6486*1455473023', '27849242.5097*1459507836', '80765421.062*1484156958', '81075644.2651*1462697943', '66577024.4218*1475247509', '34815719.3275*1461280113', '56275344.9521*1473782346', '61954147.5969*1461119237', '19084828.0742*1455954519', '77852299.9318*1476074447', '45315597.3359*1465308903', '32693833.4949*1483538932', '88988899.1066*1486267283', '49271155.0904*1472232133', '83219411.4043*1454253293', '5325292.30404*1473437387', '43159873.5981*1469438275', '70623844.7249*1472591222', '69264101.9438*1470719916', '59596684.335*1486380807', '47020331.3636*1456713211', '58684815.3662*1486338503', '32876097.4237*1454029730', '32955240.8985*1481718567', '36005905.3484*1456704274', '84871001.1592*1469838746', '72637769.5448*1471624305', '3226734.9118*1471016537', '28915602.3552*1459501059', '68245306.8893*1481299079', '75915357.1116*1461056062', '58150978.8748*1453935267', '57477665.8582*1471512419', '45433561.0286*1487764170', '55216182.2172*1464398460', '52250880.4471*1473974358', '95354727.8828*1465268587', '47154381.092*1470973030', '97304425.5586*1479379036', '50301256.8002*1478825015', '44022163.9502*1485139760', '89283682.3367*1454896006', '84690952.3641*1478970530', '65788695.5577*1484666506', '30130717.4914*1480851082', '358986.940119*1481722944', '71046454.888*1458611875', '45133311.1426*1471320556', '32468556.6319*1456272370', '82219471.5315*1481034516', '23999888.7047*1465674298', '96134282.4623*1460327808', '63638178.4507*1455945430', '79848459.8958*1485501803', '14666990.7784*1484335927', '78870914.9271*1458297813', '8272221.59776*1457866259', '98115661.972*1488057763', '29144068.1191*1484431565', '81539727.7963*1452202758', '72086924.7777*1485021850', '10842272.3094*1466012588', '41102871.8797*1471628037', '42210655.781*1483838940', '777715.186992*1486535133', '75010356.6128*1454194033', '10145634.4513*1456496268', '65249791.4563*1453372768', '64593762.1783*1470711975', '82855179.9884*1481614607', '22143117.8182*1478386938', '91878128.0193*1480666059', '37903429.5147*1458101547', '6612609.17535*1462266365', '3533507.10096*1477064596', '53536580.8199*1469374501', '91783782.1164*1460698066', '27015809.2721*1453945396', '60083942.2407*1458463034', '40058135.0286*1454447518', '34946690.4486*1480241858', '97521653.2833*1479130129', '29120363.4235*1477435190', '1688394.09243*1482402510', '63120065.7138*1458666607', '16442591.4326*1460522647', '44053463.062*1475331741', '81034250.0545*1479995668', '5409609.02068*1464643565', '48136394.6167*1459125454', '64807665.6094*1479687734', '30106413.6685*1475584589', '96720556.9144*1485014311', '76565902.1348*1476372193', '93363021.7565*1477788135', '3010916.22751*1460996884', '40326173.731*1477502384', '28696623.0869*1486414770', '35734387.8556*1459132726', '67257970.0565*1476986933', '82285638.6006*1485251334', '72254525.6563*1459867139', '4285299.16042*1465735775', '68524598.7634*1475631812', '85001670.1572*1488070752', '38264953.4302*1468448794', '50401572.7137*1488155202', '30485569.3416*1466467820', '8866768.17625*1481539497', '58192750.3113*1477079136', '72123355.2639*1473360715', '91313825.0511*1459556763', '31200976.239*1459890269', '33400696.3459*1462270770', '86261904.6105*1456754811', '52501174.4615*1461194102', '99822058.3123*1469980972', '8889348.44643*1486780010', '87462185.9733*1462542960', '98165205.4938*1458699184', '3173095.29358*1454980051', '42299302.5974*1485108841', '33349059.7013*1457390686', '19215898.2575*1468539516', '51124017.8421*1456403249', '45027842.7248*1477627246', '7726426.88993*1465035053', '48824916.0048*1487022197', '67870763.8713*1458595326', '53167618.688*1455458503', '81770454.8604*1472028589', '43209534.711*1459779776', '16702222.2221*1458938574', '4216634.58786*1465241988', '59770889.1069*1457956584', '76221691.589*1474288100', '87372743.7338*1473060700', '74371081.6214*1480396421', '37846296.9214*1453056246', '45847559.0643*1486244729', '19856691.6955*1475580821', '43995289.2025*1477499632', '42203326.9242*1456297436', '10401219.6262*1460253323', '27916329.1254*1465713640', '44530543.3954*1480548587', '51289572.7001*1460267142', '66037380.6218*1457755659', '89877826.9878*1468797530', '79389884.5086*1460389055', '68996528.9531*1471356039', '8396722.20607*1454918800', '25573555.8791*1466015085', '73969980.0515*1461649645', '42323433.3807*1485773530', '75844480.888*1478363323', '59180767.8461*1452960650', '59596684.335*1486380807', '99251130.6764*1470276920', '76650620.146*1454927567', '19098642.084*1463150244', '94829762.865*1451906236', '58950727.0822*1487480761', '85685484.6988*1475264050', '75006557.4132*1480114729', '40113562.3589*1472111541', '51336804.8127*1457506357', '82303175.2198*1469990335', '13024300.6596*1453482876', '90841107.0943*1475455035', '55744525.7351*1482282603', '83848568.5025*1463165993', '20640681.6625*1479185862', '62059715.9257*1454901281', '31575842.123*1464037966', '80250712.7213*1481483763', '66465404.1547*1487328431', '99703184.0695*1476802368', '83579893.5186*1479808616', '6978984.7451*1483199984', '88656585.2622*1461312458', '69137260.1477*1453563835', '33125077.1307*1486341583', '43928522.4749*1472923272', '55737290.8833*1486413684', '90057351.559*1487816751', '40388268.4917*1482784364', '21610504.8564*1486347717', '87204741.3238*1481161428', '69226592.8396*1482904557', '99447046.245*1462563432', '56253461.5824*1461227126', '39569150.4377*1462184742', '64518006.0764*1486978079', '89395112.3153*1481407912', '31290721.7234*1478961978', '41811046.5407*1455389623', '14683409.519*1482744908', '28628448.2419*1455162773', '55661067.2935*1479807719', '63989683.906*1460315268', '10405415.5225*1475686709', '27916329.1254*1465713640', '59039050.6631*1468763211', '87945748.2563*1473276222', '98275100.0665*1458579599', '1453805.77613*1452425880', '20348106.5205*1462794732', '468318.275791*1454686405', '90392059.0929*1484165866', '71217869.5399*1482889629', '78643697.2189*1476642234', '32538401.0955*1487467153', '78110739.858*1455023074', '77194334.1616*1472665606', '6296440.68912*1456800228', '88965137.7303*1459309912', '2206656.55829*1468251877', '78966557.8517*1459418462', '68864787.981*1469845449', '60212761.1423*1475595684', '92389581.6*1483866387', '54864813.3636*1470573781', '42664882.0276*1471567235', '92702869.4731*1463493959', '17168917.8923*1474481563', '3074419.68074*1464502578', '16225038.5738*1470275677', '769079.18221*1476384391', '18359445.1696*1478631477', '44988375.7955*1453236054', '52130105.2192*1477592990', '18573600.8108*1484868100', '80653440.5139*1473590470', '62103880.8359*1456855183', '9633512.66767*1464965835', '69943890.9697*1486766298', '2736329.8221*1453033290', '16273823.5619*1454724059', '71215465.9904*1463795217', '51440215.297*1460657625', '11033049.7655*1473985724', '14871479.8535*1479578760', '72086924.7777*1485021850', '39416005.1733*1460576438', '92560579.8954*1486274327', '5786529.71847*1472034401', '55893518.7027*1465861067', '86026252.0489*1462174702', '36668358.3665*1474111642', '16166031.5709*1463755986', '77884953.1692*1456203896', '9161559.51943*1480279596', '47202053.0005*1454975992', '43107358.7193*1486398683', '93738298.5595*1486908153', '20724312.2366*1478347652', '23498232.4757*1455894400', '27825131.489*1452341096', '82355464.0509*1454271200', '94184513.0203*1470270544', '28317297.4944*1459878681', '572574.987286*1467691490', '60261065.5513*1472041655', '14613104.0784*1475412378', '31085244.4344*1455879439', '91872605.8191*1457583482', '16619205.493*1487239540', '74075160.7727*1473107471', '70449682.2354*1457286431', '62091334.6517*1461228085', '1065626.65087*1478533697', '3419528.46254*1478660350', '91451087.4948*1488315093', '41825308.1461*1461121135', '15701603.9526*1472018269', '87933281.416*1466430471', '52973957.0074*1468136765', '70270870.5638*1453684671', '42605273.6521*1472061894', '76021526.2399*1452056349', '57569140.9566*1454369701', '49969694.9426*1471885404', '7611854.49726*1452214123', '64042980.9136*1480291351', '2822093.91449*1457379724', '71935305.2697*1474692794', '33838528.9378*1476580612', '23878942.1651*1459751488', '72164036.1511*1480486581', '28826686.2515*1486710201', '37965331.5757*1469831449', '24521707.3883*1458539976', '73189051.918*1472341708', '66184606.3696*1468525293', '3806282.48477*1460016726', '99427503.7312*1454603259', '33710637.0069*1465795825', '40284801.8629*1473647611', '47899764.5868*1465640673', '17038265.4798*1478504455', '17314044.1961*1479689821', '91605013.0956*1475451843', '15365965.8517*1463097005', '82571304.0197*1466236147', '12358436.1745*1479323131', '82922713.2862*1473884505', '43895574.9097*1476254133', '2931432.40423*1452909057', '6298722.92989*1468418437', '87079401.2689*1481624122', '45032413.8362*1479869612', '60403593.9504*1481585673', '54166997.9285*1483129201', '85504299.9739*1464688808', '77162256.4433*1485092355', '21754761.0847*1457675085', '24756304.5342*1455760440', '37400775.9637*1468144898', '5341380.00427*1488034140', '96863233.0874*1479743465', '26978031.9433*1460831397', '14629460.1054*1476611724', '78114357.3085*1478493217', '30500491.1437*1456148988', '61220376.3832*1454520818', '52178467.858*1463037592', '22290742.627*1469479559', '6627088.45596*1463664305', '31041731.9188*1465757742', '3623947.99443*1480110826', '52083834.8012*1463963368', '20468624.6377*1466159902', '65005985.03*1486022269', '71649552.1649*1456397856', '66214301.2944*1467680355', '62508014.9064*1487885193', '16340293.0759*1476258862', '11359367.0556*1463914113', '52314608.2621*1461235999', '16016267.3619*1453798480', '693934.816798*1455382735', '4517502.41634*1461462941', '44261890.657*1487809573', '86532954.5668*1482420649', '83186182.1087*1481510794', '57032632.7368*1488213421', '67270690.7203*1484860562', '57115838.0363*1486017830', '22806911.9765*1473042251', '91547190.187*1474591701', '21485413.5882*1479902363', '89896170.8501*1486641047', '33710637.0069*1465795825', '18577912.6803*1457270809', '99682080.2747*1459089710', '67188484.2109*1479170600', '33714557.1692*1455375933', '53282151.1224*1483460047', '9254889.30751*1474207246', '64385176.2195*1462822684', '63879917.2214*1468380256', '7577199.52864*1474608246', '89745451.9131*1477200953', '38111230.4231*1476023782', '16198833.8661*1484110503', '71562110.4579*1483354292', '90639473.799*1476640353', '39923913.7304*1455060771', '23819071.221*1484169512', '62334265.2272*1452371127', '96127671.6319*1462255132', '83166417.3442*1482982472', '3478912.99404*1473831707', '36869526.3841*1477456253', '93501400.2542*1462815518', '17176949.8498*1485858690', '11858550.2178*1464667310', '36280721.4911*1464162696', '57009753.3955*1483183042', '22668489.4587*1461221489', '99703424.8624*1475845026', '62260850.3796*1473790435', '70801183.389*1461060775', '62332893.9173*1456701689', '31347110.238*1452816239', '70350790.3184*1462520751', '10623106.792*1464485697', '19528703.9839*1467594620', '72772427.1889*1486226049', '31011262.8888*1486411825', '59496061.8204*1477120469', '81410931.769*1477490798', '90437980.801*1480183336', '47467807.6579*1459409801', '21926347.8953*1485913028', '25651401.0046*1470583781', '56369102.7748*1468378950', '61477122.0791*1486111540', '13213881.4676*1462473378', '16094.8638653*1483027802', '88052526.6868*1455735573', '65942268.6817*1456132712', '25093872.1983*1462888686', '36996741.9616*1455065233', '1453805.77613*1452425880', '8891093.67508*1452438361', '1028503.47487*1478791482', '1965230.5827*1462253767', '31773424.5546*1457454385', '43211419.6905*1476119462', '46886937.2298*1452692188', '31485334.2584*1464238448', '99534274.6323*1486736608', '96434000.4726*1483824528', '97784067.3767*1453601341', '3310800.78376*1459457952', '45350396.2874*1456242225', '72732736.9464*1462340013', '39211261.907*1455881593', '25673360.5849*1457612920', '42088500.9666*1484370088', '98498462.8781*1474128879', '91329380.0974*1482329788', '2866191.75586*1461727509', '41209122.2645*1464638879', '88234624.063*1476841879', '85702992.2338*1471424875', '41338575.5376*1455960887', '96765314.8993*1465076240', '15399751.2877*1473952196', '22690669.7869*1461415710', '41156816.1006*1487965559', '79650806.7796*1465008499', '52442428.6924*1486925563', '48439702.0122*1460197501', '60658507.6154*1465720083', '98498462.8781*1474128879', '62740356.7586*1457405839', '17362060.3617*1475433038', '11148816.1375*1458884024', '93408530.4482*1481583223', '69540839.0327*1471120354', '98709334.5*1479524402', '32597001.3259*1460988013', '89340694.7315*1468890786', '34458616.9233*1466398587', '96602725.2951*1482165395', '50984176.1506*1466803142', '16232529.248*1485255898', '15796687.1887*1477370494', '75873536.4571*1470006970', '35900548.8139*1456364297', '21465897.001*1454520979', '81681856.6024*1454142069', '35720499.6554*1456124267', '50710874.4704*1487328753', '28605337.3718*1459473239', '1965230.5827*1462253767', '96050520.9333*1461941291', '93920041.387*1470102373', '12169505.279*1487212319', '55597068.9813*1479605005', '33248469.1961*1472138806', '34151618.6498*1453553770', '75537948.7366*1452528419', '90207859.0065*1458637096', '14942304.63*1455808865', '10241770.6829*1484867871', '44961508.9281*1459143162', '45751341.4956*1481536478', '52083834.8012*1463963368', '85343812.271*1475444036', '31076179.9488*1480180836', '31664089.6591*1482341096', '47989243.2056*1482539191', '55904394.9396*1482467270', '71897782.8768*1473207817', '83760093.8006*1478121095', '35239731.2015*1468010718', '94848978.4181*1452333329', '83119244.6716*1473836500', '76915735.6466*1457153808', '71014147.4006*1457064565', '94712707.2277*1461203057', '5776413.66687*1475605932', '12565218.4941*1476671498', '38813148.2432*1487309954', '97819051.8242*1460918494', '23559715.8616*1455169524', '31721939.3961*1469464766', '91592575.1255*1453496513', '55217451.9946*1480493557', '70759400.2908*1479007929', '96163010.9914*1477364457', '38373222.6684*1473748830', '18379048.1237*1454646644', '90900349.7797*1459405354', '65991207.9335*1453103934', '45050229.1924*1478896645', '68825591.2709*1481580115', '36819487.0924*1480053656', '40996855.0569*1465018226', '26008135.8061*1475111063', '59835509.1672*1465128238', '8176031.88892*1455609672', '5216667.47998*1459392381', '90030953.6853*1481958211', '57644744.1404*1457507542', '11194247.2659*1461628512', '44789969.3793*1480693228', '37178054.284*1458983663', '64405551.3933*1484576700', '35003598.8349*1456449595', '63116834.8683*1481187027', '48667889.6678*1459018458', '90057351.559*1487816751', '58186307.4454*1468048947', '20580819.6264*1454009796', '1089580.40323*1480856511', '89137945.2757*1487604020', '92185559.318*1464791108', '20691393.8693*1463006957', '85002965.3316*1487140210', '59048792.6291*1453294928', '16660855.0225*1469084072', '84165711.5678*1454925128', '50130514.818*1464785291', '41812803.8806*1468692367', '20633777.4135*1469116954', '88326350.1452*1455007512', '42874568.4289*1461838350', '19665394.0035*1454029420', '71902178.1246*1469159535', '61214170.2446*1468040427', '9098949.67424*1478833704', '92254152.7069*1453822041', '62402668.9336*1464057461', '33157962.7901*1458484651', '99848202.5779*1454870450', '47604392.0694*1467667562', '68825591.2709*1481580115', '47840098.1948*1473200307', '29120363.4235*1477435190', '82394689.6963*1453920355', '66214301.2944*1467680355', '66465404.1547*1487328431', '91020117.2587*1455776452', '2864661.88916*1452141507', '2618087.93091*1454482656', '54985955.0573*1473153365', '3091827.04911*1483068021', '7426499.11909*1462770053', '93993372.7746*1485166278', '46737083.8311*1484221767', '65497964.3675*1451744337', '10738096.8328*1475329699', '68407216.1804*1456271288', '11409453.0679*1459836976', '80440300.7111*1457832779', '7753939.69186*1468891591', '36083502.9221*1476904047', '86544565.7705*1458743001', '60160240.389*1461577668', '4500190.21704*1473902251', '24846773.7167*1463100685', '2561429.17692*1459385987', '11331464.579*1467055202', '47813258.329*1462305196', '50984176.1506*1466803142', '43896536.1143*1458531196', '89668648.8804*1465053447', '89833021.1555*1475937481', '63129563.8702*1470136068', '91670289.2591*1480649043', '87055786.8574*1484377113', '93836017.5736*1478001578', '8048173.90274*1480679026', '60497920.2094*1467929039', '88111271.6494*1469738793', '70244763.1482*1463607385', '8082397.64796*1469107211', '28967324.1515*1476052789', '6820108.5423*1460578418', '38905161.5287*1466258198', '86261904.6105*1456754811', '75537948.7366*1452528419', '50330543.7113*1468924910', '54074246.3766*1484332585', '69420731.0135*1481027512', '21713213.0013*1467555347', '34445020.7007*1476072894', '40677169.8441*1485391258', '6475787.10355*1485204516', '34946690.4486*1480241858', '1894996.07945*1467889982', '76594627.3866*1473520625', '23800893.1403*1457151062', '77961614.6331*1462183024', '44536480.8203*1484291921', '902932.660257*1486759164', '41524086.9597*1470148718', '38742476.7816*1470121486', '8482196.83426*1480313917', '59423085.6422*1487822169', '42247967.1212*1480395017', '6269726.33366*1473859005', '69371007.2096*1458708645', '98299500.3973*1477136771', '15362664.5716*1454128720', '82149426.2409*1480024461', '84517869.1604*1487437414', '98091239.298*1459713634', '99068085.9083*1478310663', '1680853.53017*1465341119', '30156430.77*1456547343', '36348535.1831*1458456102', '62630816.093*1457525051', '10561698.9106*1476927392', '54739341.6709*1457791328', '87093922.9067*1474944730', '3806282.48477*1460016726', '19182041.4678*1485492231', '68319532.4129*1463625173', '90916913.9733*1477212115', '8572735.12207*1453227295', '60615210.6449*1452383896', '21562649.874*1462932035', '80047993.299*1482186580', '98852980.1813*1461971752', '23215325.1986*1485923174', '58713781.4408*1457714920', '18818600.962*1453860255', '83697336.1311*1485220498', '21803635.0845*1465585702', '34831696.3944*1481838125', '93238579.5346*1455161152', '78031589.8396*1462554557', '75291194.0163*1460698353', '16738073.6999*1475389646', '1214648.1344*1487966528', '35248940.6425*1476799369', '56569346.6635*1457808504', '35006501.837*1483558510', '85114549.7527*1458507479', '60378057.1684*1470966514', '50199760.8647*1464613506', '73553798.6614*1483926341', '7400891.56672*1452659005', '45089620.5243*1462479999', '25301188.4216*1463471606', '1965230.5827*1462253767', '22999582.2415*1484706978', '98798073.2091*1453680882', '82060703.6902*1457228199', '26169405.4507*1485138228', '11122014.6647*1464476496', '54178049.5308*1482908661', '23862888.5928*1465816892', '38117710.9211*1465505988', '24038004.01*1488012521', '9423896.98118*1468967489', '38240305.51*1477108456', '64521093.5878*1451816310', '41792751.791*1484855712', '69145424.2992*1451863272', '58633322.105*1479905531', '58633322.105*1479905531', '27432933.4808*1484410589', '7593338.77161*1466297501', '97894745.9853*1465121376', '31496576.9687*1452010591', '7351569.07489*1464232257', '24998590.0922*1486998789', '25513638.1481*1468954483', '22532125.3287*1475684092', '67404775.2638*1478678871', '86415710.1385*1477911144', '99215571.2529*1486317003', '95316076.4242*1457022973', '98111978.578*1459330864', '70154745.1958*1475164673', '69084602.5143*1483965834', '94648926.2195*1472549228', '87113739.2345*1483346964', '4794170.69441*1464992533', '70958029.4151*1480403018', '99914887.9788*1454889017', '76270059.6155*1457513470', '90340421.1518*1452665649', '59123678.5767*1468522511', '88922128.781*1482859523', '58731738.8308*1487765777', '99260999.1245*1474642563', '96569513.1691*1453482148', '18097634.8153*1453265382', '7432667.3109*1476486022', '56244367.7093*1481334081', '16322252.8026*1478512813', '22338032.2923*1480745986', '35499329.3791*1470557720', '63907252.1498*1453672945', '60491918.8731*1455825453', '33659304.5772*1470566260', '38893001.4227*1481467631', '40170480.2915*1457370855', '10744421.1044*1479813586', '81205296.8951*1472519446', '77384375.6678*1466570370', '88111271.6494*1469738793', '16693093.9191*1457129165', '75057216.4553*1453918507', '35493557.6612*1481862316', '70572409.7618*1463616646', '81700202.4377*1482335200', '67409807.4086*1477291496', '3381444.34739*1483921069', '33719617.0116*1463951887', '78017742.1398*1474973566', '79848459.8958*1485501803', '76565902.1348*1476372193', '29185234.7669*1453944332', '89307592.6478*1457331301', '26944120.6165*1483646778', '39992101.9329*1478402633', '55893518.7027*1465861067', '31875634.7894*1459535669', '56822454.0496*1485962144', '14984620.7182*1469200695', '14774808.8032*1486175590', '21926347.8953*1485913028', '15339743.6121*1467035631', '25126998.4113*1475612155', '87941164.5524*1454398140', '25361541.1108*1468863158', '28073989.4932*1460258550', '76956939.8353*1487632452', '20076491.7533*1465220792', '30703174.7582*1485300656', '84173178.1848*1485318177', '30084008.4541*1485037650', '2753534.78765*1485525753', '91313825.0511*1459556763', '2290754.73069*1458927017', '82908150.4221*1481899694', '62758249.6422*1485273781', '83558064.9957*1459590149', '13784662.7245*1452690354', '88507489.7887*1484106767', '25524941.7974*1464110344', '45883169.8611*1486945042', '97736624.1242*1454373576', '80074021.9303*1454517590', '59157690.9616*1459696369', '94848734.4369*1469077451', '64514394.4503*1477041119', '31236906.3788*1452546192', '13830554.1359*1473278585', '57560407.8185*1468721554', '3060605.4361*1474224523', '17235171.2329*1466751331', '1614281.71698*1473717103', '31503238.5031*1477872838', '35720499.6554*1456124267', '36893027.9586*1460586997', '62678770.1165*1483832780', '33835050.7608*1468630970', '90596714.3841*1483944219', '87858232.7931*1454605135', '31879188.1692*1473088653', '69247663.9757*1458933198', '46861124.5225*1476258070', '77947483.9922*1482286786', '3065721.8797*1456520305', '63081616.6897*1460232511', '92595120.0188*1454694213', '81680861.2186*1458268178', '10000572.6672*1467438272', '16660855.0225*1469084072', '49220890.6819*1468965367', '83795231.1971*1465931088', '22102194.5232*1458152117', '72215067.2093*1475004112', '24571478.0516*1457938254', '12134965.3315*1481057052', '45308267.2271*1487705685', '43895574.9097*1476254133', '22591162.332*1486106261', '69687206.386*1463183731', '82852857.9934*1475075470', '65956129.9643*1469516222', '94546983.0746*1479012511', '48103878.0925*1479684322', '27546349.2217*1463968490', '39243700.3796*1478419802', '80674003.3155*1456296943', '64880057.0932*1484877603', '52765735.2874*1461637924', '64549017.3931*1472512901', '47038384.7758*1474654355', '44574636.4358*1456507272', '40668601.5994*1465967414', '15838468.6881*1458794400', '3440662.97386*1471725152', '29221616.2645*1454632631', '1065626.65087*1478533697', '62343130.7804*1475438878', '19376651.7796*1477515946', '92579791.8153*1466757409', '37104.4044091*1454565609', '12121971.425*1486094949', '67587262.8353*1467298131', '19279025.4704*1481590094', '88048748.5238*1475009896', '82963519.5677*1461686946', '85401321.782*1457689257', '33228209.0849*1487229534', '23763493.0923*1471573760', '77063.5814128*1482546993', '21179502.0889*1454930090', '58107970.8586*1475582135', '72426708.5911*1459229398', '18054589.0476*1485877274', '36005905.3484*1456704274', '25864720.537*1473795277', '69562134.8499*1480945619', '99374367.9318*1451616403', '32685525.4686*1486953073', '52335622.6608*1474106069', '77795007.2521*1485652407', '20363180.8517*1454843655', '43096594.3635*1473459715', '75458959.063*1455096542', '45050229.1924*1478896645', '50644469.3623*1476563436', '64514394.4503*1477041119', '4914121.56338*1478761358', '67331940.3641*1468547747', '87634522.7293*1487006262', '36011389.2712*1482791977', '93718314.8823*1468968537', '59614492.8773*1485562885', '16198833.8661*1484110503', '77271740.2053*1479778867', '33907302.4544*1473029774', '33360745.3988*1467847757', '48391823.5825*1481370664', '23926012.7944*1467129052', '50531764.8525*1458196268', '78396363.6625*1462917020', '30084008.4541*1485037650', '3352495.63096*1476698091', '92717949.0841*1461741292', '40795850.1707*1482550800', '63719691.099*1452067003', '20587521.1035*1466071243', '29411408.1035*1467346110', '29921775.3267*1474411864', '12269460.1293*1481561918', '4093491.99388*1463351602', '83090276.4741*1458466293', '44575716.499*1455898140', '24799334.0234*1485941326', '43545185.4186*1461161818', '62091334.6517*1461228085', '12191086.659*1486673920', '33393930.4994*1463540745', '46369288.8893*1455077902', '90779874.4561*1481350606', '60547535.0017*1469703201', '2618087.93091*1454482656', '19431834.4319*1458517683', '87465917.7226*1484454672', '48731832.7283*1466279075', '95873192.7046*1454467269', '83357494.3218*1472640946', '46302080.6746*1455186495', '15248960.9971*1470266996', '7432667.3109*1476486022', '23391982.4634*1463895930', '84329690.6646*1461244582', '90583065.5227*1477437329', '33452748.8604*1457749015', '27460610.2293*1455586074', '3381444.34739*1483921069', '48622646.0157*1453194521', '36539471.4646*1468003801', '82394689.6963*1453920355', '21505101.4374*1475405514', '43344966.5932*1458624241', '88496896.1238*1479233299', '9788797.74949*1460132211', '61018872.3713*1476093110', '84295545.5961*1461385652', '10830809.1155*1461225587', '67112650.7657*1483289369', '69135181.8033*1458564745', '46077972.9322*1472544959', '31462229.2474*1482716886', '59187968.2954*1459878665', '20294779.8536*1456867842', '22200686.0753*1470096015', '94008128.2974*1454519860', '20009519.9677*1478163114', '99678292.9018*1479970552', '69135181.8033*1458564745', '80109032.1189*1465472691', '61284558.6587*1463600634', '22139003.6145*1483113273', '64732106.3707*1471094379', '58641244.8673*1479176236', '56131177.8552*1474110631', '25598497.4812*1457652779', '12803566.3539*1485109216', '85441583.2773*1470151211', '49850040.3359*1486550089', '28360065.0863*1475272838', '65850632.0271*1479368891', '22190794.4558*1471635422', '41145221.174*1469417431', '36022923.2947*1462310046', '23053404.3841*1456180387', '43720762.8785*1460596579', '59807021.8631*1479528715', '84075943.7467*1478625744', '92974897.092*1454965703', '55084968.5937*1477954559', '72035782.1193*1475066418', '90099531.3799*1466219810', '16094.8638653*1483027802', '9285253.03496*1456269040', '57312953.5722*1472756654', '18651108.7668*1473097187', '92729377.2384*1475616399', '69770379.6092*1473930327', '65487806.6326*1479676474', '64748119.6108*1475514221', '4362556.22279*1463963800', '46849013.7458*1465708813', '88937150.2952*1485552220', '37041754.7407*1464969440', '4926577.73619*1478428834', '12396681.5238*1463566591', '38933469.8644*1476193741', '29183011.8658*1477244386', '1376752.02112*1458112248', '72190124.3116*1485722833', '21195838.3896*1476970348', '7591495.19756*1462677105', '2035332.62928*1478090690', '7253976.41411*1472440877', '88362139.3209*1474421746', '80397678.4545*1478165663', '36900820.5231*1475941106', '26165336.8586*1484230814', '30804913.9892*1480203357', '33755995.6557*1471640949', '37926748.2472*1475732858', '8595562.35811*1474531823', '79366224.2465*1487828972', '69435840.6507*1458177046', '47668950.2266*1454706673', '21475182.4811*1452718442', '25044059.3723*1481730250', '62260950.9882*1485344728', '23874347.8505*1470280524', '18213091.8574*1457089218', '19178385.0312*1480252283', '23933690.964*1470231484', '61703127.6047*1458243835', '66700677.5353*1459344359', '34656389.3142*1470530088', '36857291.1299*1488261866', '92679499.9507*1457171686', '74407773.1226*1483051161', '14515979.9296*1470787429', '92965475.829*1468818428', '13499982.1854*1481233819', '27079758.7069*1473114680', '66686312.4046*1474290553', '1461565.38558*1457509152', '12593603.5607*1484091996', '65758102.8595*1478805173', '28909540.0318*1461318124', '44265928.5164*1452387967', '78151874.533*1485747626', '27593617.5653*1477232637', '20116326.9622*1487110425', '93201084.6265*1476649684', '68295054.6447*1470928754', '84453816.8202*1478358243', '75048635.2398*1463446635', '2972832.64437*1451966880', '30261195.8772*1475300352', '27576302.3572*1465304436', '74390744.4758*1471492142', '62561273.8953*1478924651', '39717941.0454*1473861890', '52927469.5435*1459659469', '18900842.1162*1477406016', '98206963.516*1487851016', '73340936.638*1471476882', '62305226.3367*1470541877', '230802.360214*1476836470', '61950343.3817*1487908413', '13978920.0698*1467596452', '71379596.2025*1464643137', '32709002.0342*1456513761', '76221691.589*1474288100', '50364249.5743*1475229412', '64270815.2593*1486534463', '52093050.6308*1475438545', '12319864.9321*1481573474', '59048792.6291*1453294928', '18393468.8891*1475897545', '75057216.4553*1453918507', '55691291.8351*1457330292', '44209902.7525*1477771397', '68907951.2659*1485535033', '20544171.3957*1463168740', '26877585.5557*1456684415', '35263198.1774*1482245788', '98548011.5995*1459859709', '54277344.6803*1459917887', '60808479.4291*1483694361', '99055017.969*1483869538', '68996528.9531*1471356039', '54986404.917*1453895459', '98782828.8462*1471670989', '64074371.7338*1467375041', '47510521.6426*1462734014', '82501303.6554*1481209893', '56822454.0496*1485962144', '43924632.4535*1466812943', '33445163.8853*1453007746', '43545185.4186*1461161818', '10150296.4866*1484368300', '18015430.6884*1463685542', '39294623.662*1463278510', '90425804.3373*1455261256', '43995289.2025*1477499632', '21218115.0859*1480857510', '29183011.8658*1477244386', '46886334.0676*1462864455', '13009510.6235*1466612287', '59884446.1239*1463488980', '33708429.8179*1484934410', '46309537.7745*1477027811', '27879079.5234*1458425740', '46195494.7776*1476593550', '28948677.5589*1464268076', '74739544.5503*1464379353', '14293901.4068*1463240248', '75951835.9904*1474696066', '41390412.8523*1458430291', '52501174.4615*1461194102', '40532073.4075*1479448593', '48479392.6727*1453519353', '74454370.3783*1468808807', '14841020.5946*1476950986', '73199226.1558*1487608188', '30723068.9238*1467761894', '70445791.9261*1462111958', '27366008.2974*1453379610', '82812269.6065*1481982265', '32726600.8172*1472790670', '62796689.9649*1470492011', '48417166.0454*1478256538', '14170915.1412*1456303924', '55084968.5937*1477954559', '25888659.649*1472502559', '74894665.0678*1469517287', '7728872.11888*1486133938', '50318735.3529*1473164291', '32017013.3343*1482608153', '74562481.3552*1458035784', '73820996.4466*1453236124', '7176361.60485*1483524934', '49903855.0305*1474385047', '87369820.5524*1461434777', '38182675.1104*1475355050', '40648003.3824*1468375584', '53882180.0052*1470387302', '45431512.1271*1459740711', '19514451.3049*1479134768', '25402508.0551*1465292812', '20212591.9464*1466680080', '33280759.1719*1479938216', '98770312.5103*1478728206', '25422742.4275*1477082991', '6472397.99098*1455058460', '88005602.6783*1464639901', '72435197.8258*1454706140', '3041358.02493*1469236994', '99939268.6592*1478387923', '2722335.44497*1481148755', '14007834.7365*1484845132', '8901244.09499*1456997074', '21883485.6235*1462829855', '27605214.4995*1485838837', '57032632.7368*1488213421', '68073819.3279*1461129070', '62418857.3121*1467438290', '85115219.5253*1488020609', '60830619.1683*1465981242', '45505842.6991*1466622705', '54586842.0218*1476005244', '38357310.451*1457158951', '57434551.8978*1487871526', '46890565.2675*1477051689', '11212693.391*1472617449', '51340862.8466*1468235816', '32980582.9208*1470345793', '29144068.1191*1484431565', '44450873.25*1463143424', '89627689.3939*1486433351', '70125550.6595*1484507581', '23646385.1569*1456588753', '33376930.6735*1453290881', '56328366.7071*1461164437', '34422661.5183*1470088190', '33951029.3514*1474264284', '19127197.9707*1485346430', '19225058.995*1471166830', '27460610.2293*1455586074', '59763826.2862*1477296283', '74500195.6964*1452379508', '48358324.9125*1473082848', '13127208.1902*1485484873', '6960658.47534*1467713068', '65280131.9751*1474758670', '21535951.0759*1479465450', '45089620.5243*1462479999', '46923187.2473*1452909693', '12189550.9778*1475088290', '53880092.6318*1465451603', '58035187.1893*1473124166', '34510097.7423*1460138611', '87577005.3017*1486269468', '14509681.3778*1479004749', '20157422.1309*1484830064', '3361759.28054*1484937982', '48416096.9621*1476540977', '39773123.3149*1479504306', '99956437.1938*1463714610', '92258355.8986*1480536134', '8292983.65471*1468713614', '77961318.5819*1478067840', '88362737.5225*1459558505', '48690090.1851*1462129302', '31010995.7365*1462505345', '62407856.1822*1485301993', '43137492.6663*1471979887', '86382671.1579*1468916983', '36039219.9042*1485796285', '94354121.362*1476546775', '19098642.084*1463150244', '72528903.8903*1480391713', '70635644.2005*1469938054', '76384095.2107*1452073276', '50784994.6158*1456440893', '43585581.8389*1454811064', '32136925.8731*1455165246', '57428039.3378*1462241128', '11214968.5581*1484182748', '42330561.3765*1456846869', '7942219.96148*1469170262', '40445791.6225*1487662579', '93831765.4973*1465449790', '24864692.8011*1461969360', '56969165.0833*1487998416', '74124326.1932*1456028278', '95132790.9227*1459151022', '80439521.5948*1454793536', '26432774.4767*1470740287', '23196512.7314*1479735380', '44575716.499*1455898140', '4926577.73619*1478428834', '41825308.1461*1461121135', '55661067.2935*1479807719', '45106044.4076*1480877578', '50960382.0993*1483876875', '61368622.6122*1487539191', '60375787.6858*1470718154', '7791424.65764*1477796187', '75915357.1116*1461056062', '81070011.6735*1463309367', '61578152.4619*1479316267', '10873878.1209*1452356280', '46901922.4885*1463348123', '94390298.7961*1476646213', '90382222.0021*1486809600', '74134672.3631*1452458152', '22881208.6572*1463672854', '61388070.2104*1472236510', '95300628.4725*1472867294', '52116567.4051*1483337555', '2972832.64437*1451966880', '6951677.52548*1452821166', '81448006.3291*1452532683', '88748351.9932*1459826717', '96228106.4089*1487943834', '40423429.6427*1475488313', '35209082.2015*1453621894', '42362245.586*1481200652', '42168264.4178*1487788525', '59076345.3908*1469520908', '61090601.1482*1467310155', '10024683.3252*1455285940', '86699341.8773*1487595935', '99508947.5252*1458185961', '57701828.9629*1455745719', '74611358.0075*1453801809', '5587840.41602*1485050726', '227324.233734*1485659102', '871001.062508*1481732023', '83844805.1392*1480758732', '90057351.559*1487816751', '66189855.7382*1485493046', '24617447.6542*1467807964', '45993401.3048*1462039515', '75420503.6414*1472929429', '11019377.8171*1473689258', '36845484.3477*1484608583', '11138246.1816*1461215940', '9651823.53015*1483193298', '21287177.2835*1484645183', '46923295.6953*1452736674', '61447358.4546*1470228900', '67432467.1902*1467236730', '68159463.5967*1462044144', '65966453.2699*1458119550', '74617755.6227*1478534160', '1350099.25675*1468015079', '13916575.0668*1479990617', '43504158.2312*1456153760', '88759519.9992*1453466342', '5472602.72016*1463037390', '72982268.5911*1458987833', '52921791.9336*1477121720', '42605973.4433*1470326622', '6298722.92989*1468418437', '23986169.342*1466638717', '75302434.5666*1477705740', '23936209.4933*1478279161', '14779011.9981*1460998935', '9456209.52273*1452480396', '49284630.5051*1474310976', '91881321.0565*1453343936', '27605214.4995*1485838837', '75494867.9345*1475258614', '45256775.6141*1472643211', '5326772.76968*1464279022', '94816475.5767*1464021651', '82199966.1825*1481032514', '97670044.8558*1473397746', '80264094.2683*1470236522', '40445791.6225*1487662579', '99959974.0811*1460887245', '2864661.88916*1452141507', '48439702.0122*1460197501', '42002864.9403*1453441705', '11368026.7016*1466152258', '68118454.1677*1484949597', '43206954.0652*1452119971', '85545483.7857*1471301221', '73018698.7471*1480474495', '32294460.7551*1482348101', '10114243.4765*1481573609', '15296101.9309*1486347347', '23683587.481*1461740019', '19528703.9839*1467594620', '24762387.373*1465086517', '7253976.41411*1472440877', '72147021.8677*1456247725', '36555211.6541*1483269854', '53536580.8199*1469374501', '5552307.97057*1466302737', '24956946.6462*1454494659', '79215547.8118*1479888076', '67658247.4595*1461211648', '77451540.454*1461276740', '74519938.2181*1466426832', '6077659.73913*1475513729', '66643173.7437*1471843981', '62866485.7666*1455739985', '58646021.0397*1462598541', '28402693.627*1462639826', '75517739.6104*1488293852', '33714557.1692*1455375933', '79460396.8227*1471781801', '96080828.2026*1472431196', '23234722.4827*1476565559', '52544271.5553*1460257727', '99511699.7028*1464771403', '79502577.718*1487189322', '35594349.341*1464814809', '44589016.6104*1480184542', '8887265.05104*1467570445', '43381723.5268*1458680665', '33294711.7904*1470811068', '87898047.1535*1478611816', '31049642.4266*1452763603', '40074867.7899*1458211674', '80793474.1584*1486591054', '79036726.5673*1483570528', '22996606.1092*1471601601', '1998276.10353*1478149175', '51999153.4216*1471431857', '59595606.8667*1473224753', '37831009.0785*1451816738', '23683587.481*1461740019', '99756307.5096*1478848078', '23401190.8728*1478552636', '3045214.75635*1470868989', '91915520.6016*1464969486', '5472602.72016*1463037390', '98493254.3993*1486067985', '35451041.1158*1461081903', '75131538.9562*1460840850', '64723919.7467*1482706455', '28389350.4614*1478056851', '58439757.1353*1454957526', '12242427.6611*1467222370', '24573799.1642*1485963476', '8710014.39411*1488162434', '23878942.1651*1459751488', '83015739.664*1480947885', '19158158.3315*1471812457', '74593851.9703*1464214374', '14378137.7986*1478658431', '35011508.9241*1458903372', '51080397.1099*1471473377', '83756923.8723*1454063311', '89945128.1607*1467931486', '65567137.4563*1455099366', '42823855.8315*1470711336', '9878455.31413*1483164533', '93491214.6043*1475706825', '18651108.7668*1473097187', '20041465.6107*1459056940', '12047855.8666*1464039617', '1605707.80086*1462726677', '13453030.5575*1460348989', '2888994.80943*1467401969', '55328834.4881*1466576989', '71342062.547*1458847692', '54068085.4452*1468878988', '11090662.8223*1481784424', '74845933.4879*1478858355', '30406534.0523*1455607149', '61130122.4798*1468592242', '74454370.3783*1468808807', '48556824.3134*1483936276', '37478903.2618*1474158991', '63214704.6862*1464297084', '89565605.5612*1456274073', '20633777.4135*1469116954', '3675592.70311*1473211379', '74558631.1787*1479509416', '81950027.4496*1485717124', '12169505.279*1487212319', '77219574.8436*1464482588', '80455421.9752*1451955575', '32425033.6884*1461931681', '8264220.6663*1474989093', '39992101.9329*1478402633', '43458066.0969*1480160797', '14887860.244*1465534076', '85297648.6323*1467982083', '32786645.9519*1481124440', '73723027.3745*1467597322', '55932497.8537*1474672445', '68061986.2825*1453758649', '19790837.5039*1477035880', '34132361.587*1475978780', '65941283.0981*1474744511', '96513167.9145*1470314550', '47712462.6292*1487869596', '52546401.3793*1469960952', '32908841.1525*1483286264', '85279259.4835*1461564862', '6525548.69821*1483811839', '67789653.0194*1465695267', '43137492.6663*1471979887', '56846179.6477*1464431852', '86743464.9951*1476512007', '30723068.9238*1467761894', '91607225.9349*1468588006', '46246071.4203*1472155449', '68319532.4129*1463625173', '82954273.7399*1453029252', '52129635.3549*1477450908', '33828456.3348*1459775304', '1320193.45697*1465621384', '23215325.1986*1485923174', '91055408.935*1473269183', '76174753.3712*1476340708', '92189611.1232*1460258679', '98942745.0905*1484696663', '51862479.8439*1452796793', '33352600.5242*1476434768', '40180157.6426*1472651110', '70908890.51*1477001077', '83751082.435*1458599033', '38277739.261*1456395729', '58734823.5516*1453260668', '88187100.5657*1457817411', '11586640.6596*1482111604', '12835569.4999*1469537154', '31148601.3596*1462766811', '82144064.697*1464877077', '52440522.5094*1487800363', '15027829.5968*1487751382', '46121899.2034*1477905530', '84194651.6747*1475795717', '62824208.849*1476121429', '19994594.4662*1475763799', '52798268.2298*1458149525', '70569630.3153*1456120658', '33879691.2944*1487125638', '60040967.2969*1474044948', '85496360.5888*1468939174', '66870001.4342*1480840485', '52952748.1799*1453890445', '28787970.2707*1471329238', '64707256.1753*1487124402', '94818668.796*1486700437', '57692877.8584*1470360410', '61880877.1842*1476231115', '97178682.868*1461490874', '404634.288569*1487110075', '29260870.7078*1470169004', '98611243.6584*1483476481', '85963262.596*1470296530', '71297031.6853*1473137080', '54128277.0043*1482958826', '24831195.5261*1482420265', '84715445.5974*1461608246', '54555207.0306*1459270081', '45505842.6991*1466622705', '87372743.7338*1473060700', '24998590.0922*1486998789', '55026650.8089*1480717405', '47280032.8174*1484887029', '69665168.7713*1461473853', '69227744.887*1478178972', '57827844.6559*1474432557', '74117356.3186*1453590343', '50847812.5864*1465718342', '94057048.3744*1466222415', '92847583.9763*1452753785', '79986835.3651*1466696903', '15594650.7115*1466359853', '19614633.1308*1469789894', '10315832.0348*1479847079', '83656056.6214*1462413880', '74998441.184*1455180106', '10264430.0751*1452231411', '27056111.9274*1455393975', '36039219.9042*1485796285', '91484381.5152*1476574272', '54269105.6677*1474518263', '45857207.2344*1458525246', '42257212.1865*1462751110', '57362283.5332*1485393127', '35500247.2293*1471555412', '4274879.33236*1475098794', '92835268.5*1477421526', '11416019.4613*1476470616', '18213091.8574*1457089218', '82326055.6943*1470030793', '65576941.6285*1468037109', '22999582.2415*1484706978', '90593858.1996*1459415239', '80694346.0191*1455745179', '735065.043466*1484253457', '4738755.17831*1470461529', '32408459.9336*1476384923', '18038643.0752*1466306977', '45059750.6703*1464782630', '31754495.1107*1452917783', '25562272.9316*1471375318', '42586727.9094*1461132657', '2541377.88853*1468787750', '39991761.5801*1475691034', '52025410.2953*1466385810', '51080397.1099*1471473377', '64558634.5541*1486629509', '55722043.2947*1456732913', '44480020.5671*1470636126', '11787444.6335*1478065412', '86327181.1475*1475050380', '50211015.9363*1469482085', '13588144.6582*1458471098', '36131032.4312*1469956457', '80078020.6985*1453465523', '25321054.3923*1471870712', '98265665.69*1471855341', '63219672.9107*1478735326', '3504081.13295*1463715663', '78245597.1409*1467451083', '64903060.2595*1452539800', '58458821.7465*1454163022', '83080314.3819*1457273482', '29042272.2222*1469559259', '48824916.0048*1487022197', '64152705.2191*1457147675', '68121445.3491*1485667155', '36225925.1396*1462589092', '46923187.2473*1452909693', '74134672.3631*1452458152', '31354067.1225*1468423999', '70441955.8858*1456987665', '358986.940119*1481722944', '30033676.9906*1487520614', '47524850.1164*1461958485', '67827626.0523*1473624237', '70780527.2654*1460592484', '41338575.5376*1455960887', '76627857.3375*1485486929', '29487182.0173*1459471192', '56430790.6382*1483123984', '29796370.7232*1482863660', '47668950.2266*1454706673', '46195494.7776*1476593550', '81763883.7576*1467446567', '84479038.3618*1478888218', '34151618.6498*1453553770', '51098504.8423*1454618548', '24061521.9034*1487053978', '13556673.4518*1453687587', '90484120.8019*1463634424', '42159628.6753*1459270738', '4239375.57748*1477860631', '89283682.3367*1454896006', '89565605.5612*1456274073', '40526191.0116*1476441041', '83478342.2389*1469054271', '80179896.4282*1460286271', '1112140.22823*1476182556', '73073220.3141*1485086031', '83214034.5092*1473479639', '85788590.6664*1470056208', '45639913.0962*1468464454', '38602402.0776*1484775074', '49754650.799*1482925961', '32343034.9198*1481839659', '7618377.29901*1460106655', '89581409.5625*1480526842', '47899764.5868*1465640673', '86532954.5668*1482420649', '27003688.1106*1464844011', '31413528.8884*1460811252', '12460305.3949*1477708328', '71506845.5196*1458491540', '94184513.0203*1470270544', '75555414.7407*1462630358', '79946646.6455*1464404644', '5911848.76474*1487127625', '25592801.4158*1475631836', '45133311.1426*1471320556', '78832584.1605*1467995205', '82185631.3371*1468570097', '66068127.3092*1473948922', '63501655.1895*1465377155', '42203326.9242*1456297436', '24825812.6095*1477348152', '50936757.529*1453382695', '45639913.0962*1468464454', '71635873.8882*1463973604', '74134672.3631*1452458152', '89521721.0097*1480547687', '69562134.8499*1480945619', '51291226.1971*1485398218', '33624764.6536*1462118289', '90024914.5821*1459111910', '9611728.81302*1468169953', '27401989.7064*1455964958', '43137492.6663*1471979887', '82544357.5504*1469558592', '68924744.5418*1473676997', '4632877.87205*1472665296', '66335782.8203*1464915423', '20685158.2482*1467595107', '67188484.2109*1479170600', '11831395.6451*1459471185', '66300754.304*1455584371', '34056087.6601*1467657326', '25352415.8347*1454523030', '42742693.2753*1464309159', '4615222.32957*1452707474', '88193063.6659*1455068032', '76564146.18*1454255517', '39223996.6432*1464219144', '9087406.77299*1470076623', '15664442.1593*1482766112', '97515531.2154*1477111298', '79837851.4992*1466235461', '42293147.8831*1487841885', '37041754.7407*1464969440', '39020225.0617*1468933377', '41849057.4916*1474781026', '99574853.2228*1487641471', '81785156.5256*1485162481', '19565267.058*1487675569', '45902622.5944*1469477901', '39569150.4377*1462184742', '9741588.04411*1460037450', '8954885.01648*1458314037', '76378460.3227*1466984819', '94977164.49*1485665134', '32044486.8496*1458847215', '4391666.81006*1486110660', '69512918.9362*1487505178', '54382726.2926*1466699291', '46667207.7752*1479433223', '25736190.3218*1466912530', '4705377.24611*1454817075', '93657009.3427*1465834843', '37046054.8117*1466846112', '29701970.2917*1473626957', '10255507.2701*1455745939', '16582015.4309*1476138972', '21926347.8953*1485913028', '71981161.985*1452837255', '65555592.5692*1462316929', '18832546.8487*1462664917', '45527212.31*1451700694', '572574.987286*1467691490', '39172964.5789*1457167098', '71466982.0215*1461679226', '58372584.2304*1474856814', '85456964.0583*1452195076', '22831167.1082*1478125349', '58018068.6227*1479682670', '4807553.06063*1469499135', '53807383.4417*1452841216', '94009117.7222*1467491963', '33641783.262*1453000153', '45815448.0306*1485224317', '37771976.2172*1472958374', '63821402.1045*1484233421', '31678183.2724*1481953760', '64246723.222*1477162450', '82726975.7148*1486936926', '60699190.5542*1466850404', '95923418.1025*1480527352', '61456592.0483*1461195000', '55956533.1531*1457944584', '8662636.48262*1456051300', '17740169.5561*1452405394', '99519274.4297*1481190864', '7514041.8878*1451849291', '99679026.5518*1483742146', '20587521.1035*1466071243', '79479915.6147*1452853320', '97422200.5022*1471410637', '76269041.6458*1463826925', '10561698.9106*1476927392', '5552307.97057*1466302737', '41569854.7067*1459052977', '28367568.4969*1461364883', '96271950.2516*1471925648', '62167054.5529*1472097977', '69899600.5508*1453576992', '54907992.2002*1451954552', '87071940.3248*1452501428', '49271155.0904*1472232133', '28840217.6537*1486052064', '7767849.68342*1487287296', '16142476.3335*1463949299', '97612728.0825*1484201923', '63486859.6117*1457815336', '29844968.0471*1462056694', '20845118.0977*1476187553', '25704260.0766*1465201015', '18964803.7307*1485071639', '14666990.7784*1484335927', '53643068.0514*1469860370', '12744494.4368*1468871484', '31921709.0597*1483794305', '47875319.023*1457951708', '19442810.0058*1456565956', '30875761.3622*1486896886', '10321151.903*1454376407', '13641730.7057*1473395411', '33489130.9111*1482257507', '1099910.55806*1459661799', '21912851.0935*1473103298', '39859470.93*1452732126', '7953488.74371*1476407389', '94477446.1904*1478466464', '93088970.4928*1478734158', '54954127.0461*1475882421', '48133269.1266*1488238959', '14799076.4763*1471782750', '43393402.9804*1474637712', '17707334.7873*1487567795', '29534182.261*1456290271', '31200976.239*1459890269', '93371966.8292*1484781711', '9480634.86075*1467088075', '30226423.4604*1455021748', '71938526.6093*1480311859', '40454703.2452*1461531377', '4702577.09599*1469327045', '34589863.0221*1487863655', '38551156.1682*1478789621', '10572581.6948*1455125230', '82337316.6476*1471272023', '16600860.0802*1461930554', '83324893.4468*1475614162', '89616935.248*1483067824', '96540256.7138*1461988604', '67437791.4951*1454902806', '44224783.3849*1470711621', '26553909.164*1455203368', '16391776.9224*1452567830', '17059538.1314*1462843455', '85542109.4231*1464534028', '67223307.4138*1487922432', '16016267.3619*1453798480', '52777158.669*1454036981', '64514394.4503*1477041119', '45099261.6315*1473705521', '43393402.9804*1474637712', '90384215.4634*1474024282', '51525753.6136*1470727243', '2206656.55829*1468251877', '48515914.8262*1464099685', '38608398.5604*1483933539', '69405888.5325*1483384043', '12290480.7241*1487792633', '78587423.8893*1487889371', '34209594.6994*1464301288', '90966475.2894*1472987703', '71693848.0875*1473749391', '87115371.8543*1481009427', '15711940.9217*1463900541', '91467279.1417*1463654391', '91641954.0401*1454402474', '87969519.7855*1469463781', '91376500.1419*1476799235', '83766837.5129*1469777097', '34209594.6994*1464301288', '5400190.93326*1467093514', '87634522.7293*1487006262', '4764334.23253*1465171671', '832333.046497*1466070416', '67935861.8359*1466531690', '19772019.7786*1455878574', '96125772.0943*1466473272', '54364780.5195*1458165617', '79142496.9033*1461692075', '97410920.0956*1452694779', '55222192.0774*1475790181', '32581268.6174*1464642869', '39222387.4433*1452156026', '31926314.2812*1462872288', '9710797.99321*1475516454', '1099910.55806*1459661799', '87600399.4036*1457811006', '63643311.9313*1458655546', '30033676.9906*1487520614', '75817053.3399*1463958332', '13978920.0698*1467596452', '8457676.27452*1486202283', '5866308.30691*1463291330', '9710797.99321*1475516454', '83203318.5445*1471166606', '13636301.3551*1474528207', '70870899.4628*1458135603', '42591545.6313*1453390947', '14149622.8778*1479341294', '87530041.1225*1478078169', '77622338.5632*1462627383', '61130122.4798*1468592242', '18520797.1012*1488107117', '72193510.1749*1480835854', '2665210.30589*1471340943', '1225435.161*1456225890', '66581581.7165*1459130015', '82161118.4274*1459453516', '1932091.20437*1477719554', '1431420.34572*1453585041', '53323571.4471*1463308927', '74820361.4604*1466541114', '74292393.6778*1468677163', '8597946.28633*1458789930', '88085579.9751*1471018794', '90714500.691*1486799053', '53356350.4583*1468868348', '10974004.889*1475501647', '64046322.9559*1487990857', '20588442.2378*1453411981', '54248130.253*1475599318', '69149588.1377*1477660978', '20534133.0404*1487003549', '26088132.178*1458546378', '15008143.4767*1482389874', '92627135.8323*1462986228', '57155994.1557*1466860466', '50329954.445*1464965038', '61348477.1249*1485990532', '30261195.8772*1475300352', '17059538.1314*1462843455', '85413950.3773*1480705609', '42175684.0828*1452360188', '98307626.7488*1467613149', '33190357.0759*1474593366', '80604807.6509*1483173641', '36348177.4638*1473225322', '16787205.9823*1474598635', '16184715.0703*1455327637', '66068127.3092*1473948922', '50670219.1111*1487290568', '21824818.4078*1480344418', '60752139.2396*1472330926', '64832303.106*1458963183', '16010528.1562*1464123771', '20022819.9219*1461624399', '70975813.3378*1476422520', '14122294.551*1466161005', '38393400.7466*1470626814', '819859.336044*1471837585', '73604734.4149*1487806079', '23655862.3096*1487004524', '12289859.2364*1482449696', '19361816.0074*1486939537', '56481326.8926*1469388993', '83640306.2515*1465493018', '94448405.5241*1476212865', '17802557.7588*1463748030', '64389625.285*1484324149', '36643457.9775*1484325248', '46238925.6547*1458819366', '71014147.4006*1457064565', '38481174.8486*1457382562', '63184285.1785*1466055479', '88188266.7002*1464586818', '74402923.8232*1474284781', '87381158.0062*1456581054', '41541981.5161*1456985398', '89877826.9878*1468797530', '66703685.1986*1476499562', '74402923.8232*1474284781', '89044889.5496*1465160327', '38777570.2243*1487629195', '91383332.5062*1471543142', '65926700.548*1481429636', '28886984.4365*1474497803', '10285196.0683*1459108205', '45726139.0133*1484777702', '73709437.7152*1478677211', '49184374.7572*1479767061', '2652574.63102*1479339382', '78786426.6537*1475270215', '36022923.2947*1462310046', '48523031.7582*1454426778', '8043948.39619*1476043298', '86080549.4597*1472989654', '41994956.1169*1466558353', '46548106.1931*1459729631', '13496649.2144*1452336906', '40349574.3612*1471218352', '20998341.4896*1480655997', '37406639.7165*1462993430', '44286320.7817*1479515354', '55217451.9946*1480493557', '93906558.4891*1477413640', '54319487.8197*1465889433', '25503838.8586*1467655254', '38048713.4452*1461986917', '5042857.02783*1454208886', '76959295.7858*1475994622', '33181109.8751*1458925292', '49073673.2875*1462114650', '7050767.42496*1467450808', '57417903.4187*1476441671', '3888160.90959*1485350387', '4668267.4347*1476567111', '4584004.61783*1484060540', '84766296.7349*1459417091', '99177737.0464*1474632127', '51124017.8421*1456403249', '24393944.5861*1458601233', '28484703.7289*1466690116', '19751794.8975*1473996827', '96327714.3739*1478344463', '51929332.5738*1454463929', '58929264.298*1467232120', '19856691.6955*1475580821', '39088990.7467*1460325884', '769079.18221*1476384391', '69702.9767954*1486589395', '24085823.1628*1473383446', '54068085.4452*1468878988', '80490472.4526*1473222016', '1685210.06967*1472820251', '54764643.8987*1472574723', '35008451.8203*1479733692', '86435714.5764*1482057663', '18178851.4247*1480658632', '16613614.5518*1481531001', '86779615.5639*1477733893', '10540072.2824*1456109496', '73581369.3782*1481700002', '33094541.8672*1461859835', '44528511.9146*1461788187', '52388055.7882*1464699702', '36774658.5351*1460428156', '32292595.7662*1457634565', '62851981.4471*1476907500', '13198152.2595*1463169313', '43224075.7066*1472508021', '78584596.5132*1476241454', '7811280.3251*1462263164', '17683745.7991*1478429696', '30630452.5144*1454360848', '42381094.0984*1466137640', '25865366.0248*1454371001', '99146994.7899*1468744139', '2652574.63102*1479339382', '30969399.6606*1480976906', '66560676.6665*1471868208', '83593243.5108*1459570625', '39223996.6432*1464219144', '9187725.18944*1456885760', '82173307.6272*1471116401', '77796952.9353*1461221108', '57095110.3509*1465447351', '96795424.741*1464560100', '39901716.5754*1463271681', '10990741.7806*1472909726', '60469294.1469*1456508222', '87087395.7127*1479099409', '69955463.5979*1473031363', '37156097.4661*1478661741', '12412851.9355*1480677908', '11561070.8708*1469295631', '33659304.5772*1470566260', '76770491.3972*1483851321', '17351619.1206*1476846883', '59614859.9468*1456869262', '302208.621262*1484945175', '61618471.6589*1463262636', '69573228.0982*1487326543', '23150605.8498*1476134834', '1423692.39109*1453455358', '11403467.1267*1481467632', '39551965.9355*1478693180', '92808272.8514*1468038655', '62274114.4597*1474665914', '21246477.0367*1463352785', '3011281.44191*1456976059', '94707675.0055*1462369262', '12353732.8616*1453002236', '53375334.537*1465849725', '89962983.6585*1477297468', '44600251.3792*1473511477', '40447822.9378*1453683097', '11810224.7211*1475886827', '28462663.8341*1472814889', '61714590.6086*1480805551', '52776556.5258*1471374875', '95624056.984*1470717408', '48590028.3063*1487743916', '38936270.3253*1483233191', '66577024.4218*1475247509', '78233024.537*1466936553', '73820996.4466*1453236124', '64261541.0627*1463801183', '14683409.519*1482744908', '31049642.4266*1452763603', '96417280.7833*1479776262', '32409377.2203*1480383971', '56547457.5362*1466244182', '30865172.6282*1474867915', '27252716.3328*1456164248', '55902995.6264*1459102358', '66178372.3334*1486604593', '73935679.4479*1460668165', '37882438.0396*1488012633', '99130802.0929*1473524627', '8689531.3148*1459291712', '86800125.4821*1480713090', '5412198.71993*1472163178', '6077659.73913*1475513729', '171000.248982*1470641321', '71428724.3455*1459894383', '88507489.7887*1484106767', '49199282.8027*1486347903', '48417166.0454*1478256538', '32993777.8929*1477782793', '73887500.2367*1480868474', '36643457.9775*1484325248', '23028977.5582*1469322219', '43322232.565*1473042571', '59547802.134*1474708238', '55818055.4975*1471636115', '49093985.6846*1476429624', '51525646.6639*1480605385', '40543241.5407*1485839802', '75472143.0923*1465478038', '1370243.99298*1474431780', '87889043.6878*1457726663', '49335879.8134*1465946023', '92627135.8323*1462986228', '14126757.9704*1469119047', '40857306.5922*1474605856', '58565336.2998*1484478543', '7260247.71761*1475639499', '91414551.9389*1481863215', '52671785.7798*1466718775', '17732752.2689*1456493772', '25864321.7426*1470572004', '5337795.0957*1469948456', '68278446.7984*1452288890', '41802866.9486*1460690889', '82111216.1147*1468111749', '47387769.2039*1471778439', '21166103.9203*1463762284', '49147368.1334*1452606334', '30811746.2921*1467156992', '16970607.0637*1484026040', '52451537.4507*1470049483', '78970651.7463*1465355977', '32990834.9701*1475528811', '49964424.7618*1480223872', '67409807.4086*1477291496', '86413108.8641*1487376769', '82752837.6116*1478979896', '46578042.8921*1466454370', '31425237.6693*1476433083', '32993777.8929*1477782793', '230802.360214*1476836470', '18279566.8072*1477159288', '95070913.3723*1484090218', '12681440.8124*1467756158', '62304963.003*1487079590', '56452036.1969*1473922064', '45089620.5243*1462479999', '62172867.5377*1453951117', '91915520.6016*1464969486', '51171459.7152*1462177576', '53539942.4504*1457416827', '71693848.0875*1473749391', '65727632.5415*1477203103', '49523261.4848*1484378707', '43612365.6947*1472623623', '88784416.0845*1458683935', '71981161.985*1452837255', '61501956.4313*1486420690', '47417976.1223*1461051878', '87659069.174*1466050266', '8360209.46674*1485056015', '74953901.9065*1468483537', '10616497.7182*1468284308', '40196880.6726*1482177480', '86038212.9054*1452853235', '77684304.352*1451906285', '75456221.7113*1465718696', '75266454.6793*1477881447', '76125426.9766*1454131152', '64558634.5541*1486629509', '56253461.5824*1461227126', '10902984.7346*1460088088', '36131032.4312*1469956457', '12353732.8616*1453002236', '84075943.7467*1478625744', '90392441.3025*1487685234', '88097087.3385*1461452865', '58725016.0822*1455061213', '98493254.3993*1486067985', '28426337.088*1467609609', '31839365.4008*1456533353', '31096621.707*1484031625', '42791738.6364*1483471781', '18376072.0681*1459025940', '77016940.5391*1474961540', '6475787.10355*1485204516', '67110505.2372*1469246439', '38303590.5187*1466850127', '33907302.4544*1473029774', '68930672.4369*1479553147', '52671785.7798*1466718775', '36610749.1298*1471333041', '54586842.0218*1476005244', '74778298.4072*1479879394', '71439864.6189*1475481366', '23510203.4504*1480964691', '78767622.7862*1480780471', '12267450.0041*1477411388', '80653440.5139*1473590470', '81179584.7686*1481502401', '23982910.2463*1479170028', '52973957.0074*1468136765', '35239731.2015*1468010718', '6229197.26978*1463207368', '1000605.86721*1485050184', '28317297.4944*1459878681', '98288729.6432*1459953327', '16844375.0414*1476094970', '69713946.5658*1485871297', '7512741.66746*1477269834', '69405888.5325*1483384043', '98118148.839*1465987452', '11331464.579*1467055202', '92254696.775*1486535478', '51316381.8352*1460670046', '75991615.9937*1481221163', '70653244.4556*1463181398', '27124316.3826*1476182472', '68194524.744*1464580461', '20640681.6625*1479185862', '25594476.7347*1485164700', '98709334.5*1479524402', '54669082.9142*1484907457', '99956437.1938*1463714610', '69634738.1506*1476404894', '45796602.3578*1482777237', '18253624.1447*1478001714', '64261541.0627*1463801183', '55830397.2556*1455171455', '77973303.9008*1478723870', '94039073.1452*1474043245', '4543805.18693*1454016298', '40465257.9153*1465788655', '34184196.7528*1466576738', '61037268.8387*1483922856', '16589062.7047*1462665453', '7430176.26206*1452008988', '19616981.0573*1469649533', '63466562.7493*1469580247', '57717273.5248*1464368083', '59970227.1625*1485124025', '74786181.2584*1467280400', '41177819.4387*1487343334', '80282420.9318*1467761541', '36297358.6473*1481399430', '90340389.2305*1456153415', '38411981.2259*1466814754', '33892136.0889*1468513288', '99215571.2529*1486317003', '63466562.7493*1469580247', '31517657.4619*1484936589', '49094038.3234*1466610816', '42923777.7786*1471657370', '46136921.8667*1472712186', '4117410.20126*1453215470', '73192076.7642*1478337309', '57736160.4888*1475054438', '11307447.0447*1472523561', '58026230.5419*1481305708', '91139746.3626*1454946135', '58535248.2076*1472016535', '28386242.7466*1458854066', '74934575.3129*1459557827', '99228627.1711*1477153134', '63791433.7768*1470026319', '29017190.044*1472330447', '6847394.93072*1464047091', '90010322.1216*1471154090', '73509244.756*1465686239', '38469310.2181*1463858754', '42356730.8693*1479929818', '66453802.5573*1470231906', '20375080.5358*1477937376', '44788691.1864*1479155057', '49256392.8226*1452449327', '7426499.11909*1462770053', '84573235.4747*1452420153', '2140399.4676*1455738843', '38351445.0802*1468229011', '82223459.6064*1473946158', '32136925.8731*1455165246', '16809843.4887*1458517692', '8096411.82134*1474625975', '10520259.9925*1466968473', '38486009.2226*1479565308', '21063226.8916*1453700935', '96723062.8295*1462850728', '32574430.2443*1480454724', '74124348.9247*1462314666', '48261782.6061*1453573999', '2989478.57569*1458167732', '36417858.8783*1454198086', '60305786.7874*1473942706', '2748153.05022*1456713286', '15754969.1399*1482601220', '36473854.4953*1452586837', '94548554.6353*1467682053', '89050159.4445*1481448883', '60484898.0906*1473778084', '83916247.4054*1486342561', '33641783.262*1453000153', '1366897.2551*1478397725', '46395733.8002*1477562953', '27015809.2721*1453945396', '39992101.9329*1478402633', '51029046.3754*1456410631', '29999708.4364*1472659350', '59134456.27*1473616313', '63690161.312*1466712092', '13675258.1252*1487942154', '44978406.438*1453304242', '51744312.2137*1458872690', '16586408.8625*1467226839', '60928385.2988*1480015271', '19551554.4522*1474195043', '38667797.5696*1470673729', '55327541.9187*1467749667', '86049173.6893*1458026078', '47510521.6426*1462734014', '21485413.5882*1479902363', '25305481.2567*1488008597', '29178823.9082*1465732568', '5412198.71993*1472163178', '8578850.03048*1457758290', '15964175.389*1463770557', '55311528.0456*1481592312', '70568728.2144*1465985516', '78572536.8869*1460079901', '77554862.2235*1468290124', '66724265.7542*1483521030', '20746728.7236*1472117190', '59517476.3719*1471737456', '83080314.3819*1457273482', '65699842.3989*1467380086', '12127757.8484*1457602368', '73073220.3141*1485086031', '49565302.3185*1474928443', '20553360.334*1469287006', '74769748.183*1467589743', '10499221.8626*1485141450', '22591162.332*1486106261', '54543484.5957*1459515879', '68453046.463*1468796505', '48108214.1135*1454505804', '21120709.3297*1475943151', '94118015.7271*1475456620', '50401572.7137*1488155202', '5014713.5921*1487431318', '55262519.0367*1464203426', '65613701.4107*1465088686', '49073673.2875*1462114650', '34056087.6601*1467657326', '97858257.4643*1477167047', '73404415.0787*1476453778', '71320226.3122*1474706236', '32882663.3812*1487496057', '99738940.946*1480780106', '73061003.9893*1465416045', '37935590.8798*1470641834', '81406121.3902*1478147432', '6767135.33552*1460916890', '1776273.77097*1486692921', '57150892.8321*1473823162', '12619499.3035*1464806469', '30188061.19*1458792211', '27541887.6341*1460114409', '18253624.1447*1478001714', '39123855.172*1465019770', '69005385.1106*1455852426', '59855842.8027*1480155325', '12114500.9015*1463939659', '38905161.5287*1466258198', '83854913.6876*1463108993', '93566297.955*1476883859', '51999153.4216*1471431857', '404634.288569*1487110075', '61006392.5326*1463581062', '74193030.9716*1479098448', '12368704.3*1487427072', '44916530.8107*1463577233', '38213472.8751*1456700762', '14134399.6519*1456676710', '16273823.5619*1454724059', '34086675.222*1466401646', '26316408.1418*1479714519', '28969840.2448*1461198205', '34259801.1271*1482760525', '76297488.9716*1469940189', '74685833.9515*1452337219', '19126972.5335*1466320214', '41156816.1006*1487965559', '118263.40072*1472000961', '92875278.4355*1465809987', '85413950.3773*1480705609', '96372130.9979*1476935974', '80831776.1632*1470868847', '59029290.5746*1477256023', '23951835.1502*1474323603', '4661388.39306*1456846126', '17351619.1206*1476846883', '20776271.5397*1460362291', '94133088.4281*1487188201', '6312346.96338*1467397899', '90900349.7797*1459405354', '63535309.8659*1457952375', '81700202.4377*1482335200', '93069071.5024*1480907963', '47937890.0828*1475635485', '30426917.1758*1486815018', '79856708.2043*1477111612', '68672085.8214*1464186923', '51770970.7047*1472231192', '97188598.2192*1460143893', '21730072.9061*1459644830', '22811884.847*1472141310', '1077801.19859*1471901714', '20293093.5152*1467725770', '14228172.3761*1480659475', '98442290.7826*1461583775', '71973327.8967*1469394641', '72035782.1193*1475066418', '52749398.1649*1478976360', '19893046.3034*1478645164', '62407701.2144*1467283468', '63231271.1079*1456680773', '71959037.5731*1466883479', '49693736.7191*1470004346', '79277138.8494*1472323220', '13502143.6546*1453397681', '84424837.9609*1475446595', '62976127.7064*1470831718', '75120347.2554*1481944496', '55415562.3819*1453848928', '69035572.4073*1464444579', '36545808.6293*1476154144', '90776627.7327*1462621846', '24085823.1628*1473383446', '9549051.98603*1470278187', '76176242.7025*1458859746', '20532895.0257*1462246953', '88623988.0096*1487816688', '73359345.1022*1473488788', '98776551.3014*1458957020', '16068366.7774*1482230672', '14250812.6423*1487759915', '84683187.6566*1463851726', '50635590.5992*1482983563', '87659069.174*1466050266', '68345798.7178*1467146097', '11164567.8882*1488171614', '13453030.5575*1460348989', '77685554.7148*1483357089', '10557761.5049*1470361215', '90822092.2887*1470764082', '1012011.65595*1476687057', '64933174.2653*1456178265', '22958056.5675*1477795932', '47918810.1711*1476804954', '10902984.7346*1460088088', '51358748.0197*1467401481', '51240793.3021*1462722056', '54502767.6129*1462579693', '72805468.7803*1488056604', '26963982.5932*1487817593', '33617367.701*1473360307', '85890510.7295*1486102683', '70503341.4991*1453307354', '82807217.2784*1487958755', '90058332.2107*1484636963', '76959295.7858*1475994622', '92747857.1374*1464407285', '52178396.3042*1470161072', '27944231.9187*1472225346', '56532480.9039*1483802642', '87119476.3178*1487240469', '62390170.3433*1475129801', '18185769.0575*1474547633', '34473874.3141*1466065296', '67755332.1818*1452819141', '1029437.11249*1465646059', '73396515.4299*1464283684', '80121351.5295*1484477840', '54739341.6709*1457791328', '2846681.30712*1487695972', '60921637.8395*1479453346', '47937890.0828*1475635485', '45001747.5017*1473809092', '98083054.1733*1471832856', '17606351.7984*1477553481', '44572891.4003*1466520500', '52116568.5662*1483742448', '2444194.55913*1475081707', '11138241.85*1471669561', '87418668.6991*1471840130', '50179946.3162*1469433453', '76114481.8429*1485351985', '4659684.99384*1457194169', '29985815.4102*1456569587', '48893088.7855*1466636956', '52139080.3256*1463386161', '14780513.1882*1466400231', '87591395.3574*1472595128', '12242427.6611*1467222370', '40664350.3438*1478966501', '59474313.2289*1454063448', '20640681.6625*1479185862', '58738648.8712*1483159493', '59519963.9924*1457208464', '86021255.8477*1483004848', '79010614.3849*1481247727', '58949739.5799*1462306541', '4922190.84592*1458991429', '9880284.8916*1482609802', '3065721.8797*1456520305', '99406498.554*1474635681', '88490746.6003*1472986195', '59872793.9782*1484187219', '63931033.2833*1487862065', '48314182.966*1482687678', '65767165.2458*1476297236', '12114663.515*1467332183', '39943972.376*1463404971', '21720017.0049*1486387956', '44145044.9239*1459542366', '19614633.1308*1469789894', '27455913.8839*1461613469', '33819678.6658*1478092129', '43159873.5981*1469438275', '36940941.8432*1452260276', '57663388.869*1474808515', '60911703.714*1456325461', '10192874.1727*1470869224', '11164567.8882*1488171614', '12990898.6526*1457831573', '45001747.5017*1473809092', '23940672.6758*1481053123', '14515979.9296*1470787429', '30804913.9892*1480203357', '78865223.8079*1456791998', '71078383.9175*1480633811', '48959229.5883*1462573466', '18461757.5699*1459071808', '41864032.5573*1470440858', '31218774.2341*1472701619', '14774808.8032*1486175590', '12820674.1972*1477841138', '32699159.4549*1460955969', '58597863.9176*1461519033', '12319864.9321*1481573474', '90484120.8019*1463634424', '76173302.7196*1461955488', '14799076.4763*1471782750', '23386176.6557*1486322072', '40423429.6427*1475488313', '67308854.1808*1467778802', '99138886.1557*1478339900', '73075675.6549*1451950245', '33488234.325*1477333863', '35003598.8349*1456449595', '5869008.63314*1478241964', '23874347.8505*1470280524', '62866485.7666*1455739985', '25003379.429*1462383572', '12801880.2864*1454991381', '69203205.8552*1481700303', '95806711.1667*1462573393', '83214034.5092*1473479639', '35389920.6664*1464623470', '31896487.2816*1458710304', '51058373.0663*1485910750', '91279912.8515*1477120469', '43896536.1143*1458531196', '17634010.5248*1453594313', '65788695.5577*1484666506', '12903525.9011*1458394097', '73273816.2432*1488183475', '48744196.0107*1455748894', '42884088.692*1487208431', '24831195.5261*1482420265', '28536659.1446*1467755429', '51072116.7056*1457080501', '85333419.7226*1455707008', '74371081.6214*1480396421', '94229536.5862*1456653600', '7696835.94634*1467629328', '24885752.5651*1484950393', '88589483.3584*1485080461', '78114318.2118*1460350385', '90084153.6488*1469882471', '64186321.3994*1456450203', '37853720.2849*1475740862', '25515970.1838*1462123823', '90881589.7916*1483895948', '49091809.3248*1452125623', '8959260.5197*1473873639', '11057492.5735*1456379135', '63031210.0137*1482110701', '60040967.2969*1474044948', '34499061.5869*1485263422', '44977399.8241*1483684868', '656285.858367*1467907258', '44167314.8192*1484992764', '44089910.4328*1485447971', '96624504.9565*1453547526', '1518175.99896*1479265488', '42899408.762*1451657744', '70541694.0283*1475507466', '38889044.8759*1463356807', '87313337.4992*1481176933', '88107714.393*1452845969', '8482196.83426*1480313917', '10990741.7806*1472909726', '83203318.5445*1471166606', '32611833.2197*1472551975', '44620047.6575*1464298350', '50403391.2792*1480748380', '81832072.2727*1477386816', '34395766.4742*1485398022', '76174753.3712*1476340708', '22771547.4701*1461530870', '16558020.9947*1476600536', '4007140.47498*1453789032', '36739685.0249*1487071005', '4496590.05033*1460254622', '4794170.69441*1464992533', '1834814.80595*1460194342', '51160137.0932*1463951459', '60289312.4523*1482791532', '42362245.586*1481200652', '12389960.1623*1475088315', '75101332.4867*1468994757', '89050159.4445*1481448883', '77743325.6605*1482647254', '46616907.7733*1463757050', '76915735.6466*1457153808', '69759181.7282*1478097905', '3467151.57841*1481560952', '9266228.17752*1462310139', '78798237.1248*1474889695', '27541154.5337*1482437151', '37370189.4407*1468435324', '49210909.257*1472531241', '84647509.2111*1456251757', '37057714.0407*1461118420', '8327458.58378*1476111977', '29463440.1059*1456686177', '97429732.1402*1465986443', '38111230.4231*1476023782', '34265199.4621*1483716932', '1131251.37784*1482732848', '42629352.6874*1468039325', '92556749.1694*1466234067', '66691695.8089*1463125313', '32908454.5532*1476412872', '21036879.9521*1460954706', '10857515.5083*1463651530', '49150244.0788*1480184662', '33677785.467*1453701652', '6464775.65212*1485220349', '90903440.4974*1479233177', '19178385.0312*1480252283', '21429060.7377*1463779071', '41619945.1183*1470256393', '63719691.099*1452067003', '40665651.7201*1473360884', '46919473.0458*1482183284', '27712920.2771*1456782576', '48013289.3591*1479663924', '70645219.4427*1456397849', '71699727.6657*1464072950', '96034302.483*1461544958', '47140604.8272*1484936478', '96892289.2007*1466781718', '12890071.6786*1477207580', '65158418.5898*1468772906', '92389581.6*1483866387', '26039257.8861*1464962331', '25567800.9331*1465850078', '16615666.075*1486894760', '81851790.1648*1470518612', '52181115.1137*1476767279', '24066606.4278*1456099742', '39334155.3925*1464341059', '40713679.4671*1471365913', '35457230.084*1465625350', '94483321.411*1459992879', '66643684.8769*1473096725', '44199841.4724*1465549270', '2075678.25453*1452142954', '7050767.42496*1467450808', '26118675.3231*1458950731', '92995351.4406*1469975651', '69584448.7124*1462381279', '61399285.2695*1455142281', '6623500.15734*1478665076', '44407764.504*1468793612', '15925185.7365*1467670121', '12753126.2277*1463178130', '80455421.9752*1451955575', '19029159.8145*1455081992', '44197059.8649*1462294273', '52929332.848*1483951242', '45838609.8079*1481425255', '58549357.1447*1456252402', '86435714.5764*1482057663', '39551965.9355*1478693180', '76057556.559*1464616225', '80539533.1353*1488323462', '91418341.4356*1455775162', '16842453.5226*1466603936', '38602402.0776*1484775074', '84873141.6777*1479251031', '69543795.2998*1464783149', '39223106.3818*1464057941', '73082650.7444*1462955185', '57452359.7267*1476590353', '12679407.2992*1475397103', '59651979.8032*1471254138', '46076780.2427*1488171301', '11831395.6451*1459471185', '56864929.6738*1472626770', '98852980.1813*1461971752', '67228184.2725*1481905788', '1443307.96632*1485469991', '64862326.2142*1462377381', '93161584.4968*1480589617', '40435607.1027*1473101824', '13679196.3794*1475286153', '66431663.8038*1467693671', '50046661.8258*1484242512', '87075177.8954*1483992010', '80425113.7992*1475807191', '97819051.8242*1460918494', '26877585.5557*1456684415', '4654470.79912*1484397073', '45883169.8611*1486945042', '21534448.9906*1465626316', '21188864.9608*1463461833', '64443372.59*1469157612', '50697897.8032*1483906388', '53112030.7196*1480386090', '10084105.1771*1452028034', '73073220.3141*1485086031', '17047752.5276*1474395138', '68155613.1004*1459478937', '67652533.5008*1486210501', '74302537.7545*1470660796', '1131251.37784*1482732848', '92679499.9507*1457171686', '32518432.5681*1481494075', '48321252.7822*1457384524', '99565486.6988*1483833860', '118263.40072*1472000961', '77898136.652*1471425942', '99914887.9788*1454889017', '62945965.045*1474570798', '29844968.0471*1462056694', '35854947.2823*1459888414', '73887500.2367*1480868474', '99978607.4561*1468479485', '78037662.6547*1468503308', '59364825.9603*1462932435', '21179502.0889*1454930090', '81226157.7241*1463838931', '41010968.8491*1454040583', '4508121.46336*1453701607', '33151268.0402*1472177399', '8409460.17735*1458246490', '8960179.83439*1486455540', '41026572.9367*1457404416', '82077328.3684*1487057733', '56659081.1969*1456648121', '43296258.0717*1454710290', '56031655.81*1470497477', '89808298.4053*1462885248', '48915884.1884*1452804298', '27596965.4942*1482284385', '29042272.2222*1469559259', '55738794.2613*1472932169', '52392096.7086*1461293353', '44464928.5448*1469688357', '65664460.435*1475918849', '46209625.4203*1451844588', '34046939.1404*1487471205', '42736769.5717*1478613090', '15041655.9065*1466877143', '21182494.348*1482822662', '92258355.8986*1480536134', '67929546.8602*1488007092', '12863735.3313*1481622876', '59830056.1125*1482523026', '39238111.5152*1460714566', '42476624.5125*1465945073', '10842272.3094*1466012588', '5882688.07033*1478718238', '25888659.649*1472502559', '29530381.6549*1484733007', '14354577.4253*1455026840', '18379048.1237*1454646644', '3707140.02928*1452013132', '10321151.903*1454376407', '10073646.3163*1456605827', '73579920.701*1477711399', '50130667.1554*1463196691', '96716361.1666*1478265450', '40018054.5831*1456681014', '95215164.4765*1471665052', '64677273.8319*1484316641', '28718936.9737*1455459294', '87832077.4155*1462492012', '29983166.4096*1483855729', '73843107.0432*1473585990', '80445045.46*1455938890', '53573313.9741*1480951462', '4919977.26238*1473113331', '15594650.7115*1466359853', '95573588.5698*1461750749', '63956027.4171*1478666824', '58277520.7312*1459808162', '44509420.309*1454194737', '89508484.8839*1481132549', '15360553.699*1463786047', '94609413.3913*1481998156', '36329247.7247*1455928065', '36039219.9042*1485796285', '48889759.9515*1487171107', '17683745.7991*1478429696', '65758671.0087*1471219031', '75873536.4571*1470006970', '2715361.40065*1484799927', '17997431.974*1456505011', '16652093.2127*1485376608', '59482212.2868*1469030662', '66804838.1681*1483538677', '78587423.8893*1487889371', '89137945.2757*1487604020', '10561698.9106*1476927392', '68843159.6091*1485195012', '77273826.1393*1468101724', '64770782.6278*1474074638', '36046665.8329*1455862076', '69396261.417*1470684073', '92041899.4701*1461273469', '82245012.1094*1461880187', '2045735.87026*1457793913', '90154248.5035*1480614256', '94993840.4081*1485426796', '21882212.4431*1464900508', '10964419.4923*1475774471', '77062675.9592*1467611946', '4473070.81838*1454307056', '14886937.0486*1477304455', '24849172.707*1460962339', '10073646.3163*1456605827', '5042857.02783*1454208886', '20693559.6543*1475820702', '28602397.027*1462524869', '19365623.5935*1465761699', '30111365.2455*1487681453', '88187100.5657*1457817411', '42486742.698*1453284558', '77610687.3467*1471155157', '55550203.8773*1472171430', '17997431.974*1456505011', '89625830.5631*1482916477', '9098949.67424*1478833704', '10768314.4504*1474202301', '64395912.4118*1475686178', '24860934.8689*1473816335', '99096670.6952*1480824195', '47544813.1257*1452536032', '36252350.8651*1464723600', '81632180.2297*1476871808', '80101751.0062*1453539206', '58477891.8898*1464879813', '57049383.1299*1485155682', '23695929.5748*1465479172', '98649425.4191*1478826553', '18029606.4043*1462965918', '42257212.1865*1462751110', '86737334.6146*1483491558', '49908769.3715*1475106915', '42672135.1719*1477732308', '49210909.257*1472531241', '4239375.57748*1477860631', '93850796.4966*1459446726', '29373713.7856*1464442076', '23561339.5439*1460310409', '43063597.7098*1466561029', '35235040.1125*1470767315', '36858583.3293*1451897630', '1337976.98396*1478253589', '99737.8528217*1456025353', '67414228.7983*1476867895', '9041072.42141*1481684635', '91018127.1742*1465317468', '52927407.7717*1479176789', '12155972.9687*1458678223', '98301430.0671*1487605614', '1370243.99298*1474431780', '2863307.27284*1485370971', '7765505.3232*1454941641', '37862482.0688*1485296699', '87722220.1306*1479303671', '11346374.1505*1462469782', '43101428.5819*1469168643', '78896969.6945*1463887612', '76258127.0553*1472365954', '27927194.7981*1468053449', '86365366.1956*1482146934', '70150849.0761*1488072749', '74129545.5643*1468341568', '29636420.5045*1460908428', '217276.843028*1460980860', '9222261.18832*1457504351', '59123678.5767*1468522511', '54593588.732*1460834613', '40795850.1707*1482550800', '55974982.1024*1452054902', '26556329.3549*1454927056', '22424363.3524*1470821942', '37761526.7797*1465280701', '97304291.8071*1473432942', '8687298.9214*1486460437', '2290754.73069*1458927017', '85853533.2922*1461840772', '78114357.3085*1478493217', '66849830.6858*1487709102', '56433045.5073*1472317457', '83919354.626*1478481644', '14014540.4356*1464163660', '6366490.38902*1480271031', '32920177.4743*1456719327', '88129003.3615*1454352689', '21546242.9955*1472046389', '32017013.3343*1482608153', '10109001.9159*1484010588', '8778649.14579*1486637387', '61695486.9391*1467296318', '22314002.0921*1488315513', '36252350.8651*1464723600', '69848163.1312*1480472652', '22194778.1715*1483841652', '78151874.533*1485747626', '34397856.2908*1468437660', '30966967.6236*1478988568', '66433110.0923*1478483780', '34473874.3141*1466065296', '39943972.376*1463404971', '96830454.5369*1457035780', '19277629.3925*1453331761', '25843925.755*1457388348', '35548877.7566*1470615575', '77780164.1596*1459946451', '51365492.5152*1451783625', '34938539.9472*1466468349', '52128745.1394*1479969451', '45476803.0446*1476187058', '63719691.099*1452067003', '47700578.5858*1451937070', '4890975.19424*1471065449', '71069909.0538*1484801445', '87988399.1564*1479870420', '64465003.5319*1473904506', '32401787.2276*1460090385', '35682861.3013*1467989657', '41342739.5394*1473301889', '72751981.362*1458015127', '78870191.8252*1456366084', '44536480.8203*1484291921', '77078223.2914*1464213827', '93344603.5912*1484205875', '49584069.2458*1485695042', '22424363.3524*1470821942', '44104982.7406*1473208162', '42745307.0218*1485717540', '91418341.4356*1455775162', '64616180.6486*1464263098', '51800757.3529*1483648659', '43378107.4504*1486127798', '49000936.9999*1457798277', '51029676.3248*1475314173', '47202053.0005*1454975992', '30211066.8193*1452140012', '36208054.3498*1458452523', '90963319.3097*1456509414', '34867856.0203*1477964708', '1793283.39768*1477639868', '77826751.7092*1479629063', '94642578.996*1471344918', '52924900.3104*1476375260', '52576403.7663*1466825018', '55095506.5865*1457555348', '26428650.3586*1465957423', '63466562.7493*1469580247', '91642864.5499*1477985534', '48302360.6252*1453233195', '17985803.9343*1484527166', '12158977.6696*1456011274', '88193063.6659*1455068032', '44744969.9431*1467960464', '12158977.6696*1456011274', '87316990.0274*1483136324', '9955415.69918*1466546424', '36066372.7832*1464953573', '10427722.0907*1481899534', '33882037.9344*1466753950', '32597001.3259*1460988013', '69684596.4718*1486631176', '12939662.9837*1461189132', '27541154.5337*1482437151', '68560810.2659*1472806883', '93021003.8795*1482472677', '64893584.1454*1474029799', '96778017.652*1456310033', '15947878.3923*1474347822', '55188276.2316*1457553896', '13995820.0811*1478748976', '55803577.1904*1473845147', '13499982.1854*1481233819', '42605273.6521*1472061894', '68450443.5623*1471390238', '6344154.72363*1465229444', '64923010.5738*1486599838', '58374869.5125*1455208784', '69998124.3095*1466323308', '26483005.6879*1455479540', '11440260.2093*1456421920', '32409377.2203*1480383971', '89616935.248*1483067824', '44574636.4358*1456507272', '70176561.421*1484030988', '38351445.0802*1468229011', '6091256.5246*1471445654', '2615756.6734*1485458082', '36857291.1299*1488261866', '12863735.3313*1481622876', '36531052.9304*1465105342', '3694514.27684*1480212333', '44955464.7388*1456189648', '84263198.4547*1452635199', '88085579.9751*1471018794', '87530041.1225*1478078169', '82443704.8755*1454264344', '87241941.9821*1464107365', '7726426.88993*1465035053', '11098297.0701*1453008152', '80922518.0206*1465353493', '7226423.81265*1473626948', '62035809.5386*1465986582', '49370100.8923*1474553993', '40653296.1227*1454902631', '70349939.206*1485717623', '8866768.17625*1481539497', '19751794.8975*1473996827', '54432315.3558*1482337598', '29931017.8459*1486366916', '1233488.60759*1457931355', '49199282.8027*1486347903', '40096059.0001*1454399632', '73190969.6771*1486441248', '88177221.8772*1477700433', '48680876.6159*1471313561', '46121899.2034*1477905530', '67528360.844*1486036523', '34265199.4621*1483716932', '99403756.2017*1467780176', '63031210.0137*1482110701', '9673956.77767*1453782354', '84573235.4747*1452420153', '74402923.8232*1474284781', '93631498.1297*1479676058', '38509921.6902*1472232329', '95620155.5392*1479590085', '54530661.59*1465860113', '92793395.6547*1454831361', '10339663.3649*1486072473', '89062238.2923*1468588807', '66951727.7947*1451654044', '37265223.3775*1452595587', '82757331.2816*1466513720', '23378671.5542*1465516699', '59283553.5996*1473171133', '69660198.8856*1462030231', '86325425.8893*1473527218', '58444222.297*1463994609', '8314361.00891*1482867817', '4506801.76279*1464901352', '23427193.7099*1453999266', '52699181.5756*1461386614', '94845395.9696*1466679066', '37854512.6727*1478683718', '77105007.5813*1470608278', '44460735.2947*1483786875', '91642864.5499*1477985534', '87055786.8574*1484377113', '94250754.5897*1467893360', '2007705.49825*1484754101', '22877474.0481*1481024609', '3355736.85765*1452581508', '90015019.298*1485128005', '35555453.0709*1478634406', '73518504.6612*1472170929', '77873605.1851*1481728579', '84180995.0618*1474458345', '24612490.5839*1479234164', '84490937.7666*1471160839', '12689478.3072*1456075049', '23498232.4757*1455894400', '32920177.4743*1456719327', '64880057.0932*1484877603', '87110435.3697*1451996117', '12106708.1997*1480608659', '10768314.4504*1474202301', '73992442.1539*1463196663', '57488103.9183*1479782158', '68309226.461*1470355663', '76309861.7492*1473655601', '227324.233734*1485659102', '68159463.5967*1462044144', '73911423.6529*1483234339', '72416957.0436*1472781417', '97894945.2033*1483537792', '41977829.679*1466027738', '41342521.5562*1469278271', '18865429.6359*1477148559', '27883021.1772*1453709033', '99423553.5655*1467008442', '74250787.1406*1452879203', '41805671.7484*1479346611', '47293415.5019*1472570853', '19631232.6207*1473017955', '68699795.2259*1460847350', '10768945.0289*1481506293', '17352821.9793*1451778692', '50422886.2358*1476987326', '5988321.79349*1465449276', '10294931.9286*1454189142', '82443704.8755*1454264344', '8722397.9493*1487175601', '42723658.1752*1467408338', '59884446.1239*1463488980', '35555453.0709*1478634406', '30633153.6555*1453080014', '36839304.7115*1480967940', '89856483.4455*1487662683', '48667889.6678*1459018458', '36756402.9467*1454656442', '86343566.7729*1475312323', '40483996.1014*1463667095', '36342987.626*1483059504', '49879544.6113*1473050078', '27441160.2608*1464965095', '59622330.0502*1471678032', '15956385.9306*1484181511', '41143799.5512*1467247719', '37629452.8811*1487226052', '23028912.4552*1487161948', '99886607.9317*1474523662', '50514723.889*1458873282', '51029821.6691*1453463349', '26877585.5557*1456684415', '85918307.5857*1483923040', '63504392.7204*1476034988', '81354552.377*1453894016', '66374858.5022*1465032833', '39213691.6051*1458302673', '94292554.6392*1457584969', '47546549.889*1475348880', '14664533.2959*1478681652', '39948870.3231*1472689910', '25055552.05*1461277210', '44450873.25*1463143424', '77826751.7092*1479629063', '85657162.8738*1471964984', '20331952.7319*1469284557', '61777895.9127*1451965797', '76008505.4406*1461152456', '1715068.06744*1481051483', '29469531.3504*1473159829', '77803845.0002*1474791425', '82035070.1733*1479918695', '26582809.8083*1478514041', '38059552.3137*1452175467', '99253365.6946*1453206175', '73061003.9893*1465416045', '12191086.659*1486673920', '5911848.76474*1487127625', '81055941.805*1483042546', '63120065.7138*1458666607', '84939886.3465*1459708155', '6425974.38124*1480193422', '72215067.2093*1475004112', '34811370.3742*1459613912', '73732214.6236*1452921598', '57837261.2703*1480162967', '77636782.2528*1472306731', '61298087.1706*1459329538', '87701746.7054*1462164203', '94807887.5355*1464416839', '8607497.44386*1468182616', '14134399.6519*1456676710', '30835574.9947*1479561722', '41383971.3833*1473514790', '57836408.1041*1458828641', '17987237.436*1469212867', '52265806.3256*1456963654', '89283436.2502*1452794792', '85343812.271*1475444036', '10301068.5438*1459374470', '63988588.4947*1473110223', '16089025.3039*1470234999', '67404775.2638*1478678871', '4662681.71962*1456387692', '17206537.8959*1478294588', '52624513.9332*1455751592', '67040619.6676*1475773137', '51336804.8127*1457506357', '75978429.8878*1464793236', '3117085.19203*1469743919', '61469083.8788*1485166235', '54664050.5853*1484665770', '64590342.9696*1470639632', '53905895.3904*1452301670', '73969980.0515*1461649645', '60818157.1194*1456610511', '5069553.17118*1464094071', '11084582.2715*1460464582', '41812803.8806*1468692367', '52009879.5764*1485508286', '47169735.2032*1474899318', '57156912.7949*1457375605', '82737243.439*1484064607', '86805686.1313*1479126434', '66184606.3696*1468525293', '14187350.3194*1455626153', '90275912.8709*1483553720', '15819326.9882*1469584712', '4177176.37229*1474446745', '53913639.4733*1475033693', '36121140.2157*1472868286', '72380116.593*1459145455', '45784169.5293*1460085681', '15729156.9246*1479818801', '35434883.1744*1486324101', '58044388.3588*1460439844', '93408530.4482*1481583223', '99930362.9914*1471533810', '95763195.4483*1479238018', '99440701.1698*1485500376', '90369535.564*1487219415', '98867012.208*1480952789', '70419964.8362*1481258310', '94168032.0617*1451739437', '80674003.3155*1456296943', '15041655.9065*1466877143', '43274000.3669*1453631436', '75056029.9051*1474421352', '20685158.2482*1467595107', '43760985.1473*1458165031', '57254770.1846*1466050966', '74285703.2684*1483410876', '32568584.9349*1472717024', '57912784.5285*1455771738', '7968489.16805*1458371850', '14787041.6681*1477148147', '70198917.2318*1457446930', '37494721.2256*1477499076', '30324472.8052*1461616667', '66964828.8389*1475492217', '75537948.7366*1452528419', '58141254.5067*1465952443', '37853720.2849*1475740862', '19323330.4829*1452955306', '96273295.687*1453016507', '5439213.68704*1476625337', '14446626.8437*1461517870', '61172574.1903*1474803041', '11977037.4706*1470884033', '86952632.9554*1452495175', '63403998.2208*1464085772', '81839493.3422*1469150441', '42178566.6236*1458670847', '94607057.4181*1461096926', '92729920.6667*1468168005', '90849918.2243*1454299175', '93238534.7534*1487365740', '49477355.3484*1470523142', '66813200.8152*1485498360', '32588727.3472*1464067601', '62215831.6994*1483105769', '54725154.5452*1466229638', '48357277.1549*1477136236', '47541581.3932*1466257500', '40752421.0325*1452076829', '45315597.3359*1465308903', '55974982.1024*1452054902', '46406752.4753*1473045439', '39338356.7451*1479595970', '33670426.0594*1466003047', '51726794.4484*1460881925', '69719592.5587*1473748610', '53157528.3808*1477421456', '91342066.8463*1481267100', '87401155.7393*1462792975', '8662738.12084*1468913113', '66817407.0338*1478356003', '21988130.0325*1472627836', '96016074.0495*1485498424', '4699200.69977*1486262568', '37771976.2172*1472958374', '99336251.9586*1485529957', '46667207.7752*1479433223', '36274250.9785*1471930975', '12250745.8936*1486817904', '86737334.6146*1483491558', '31200976.239*1459890269', '93687893.6093*1458365301', '15967067.2027*1456179726', '85768175.3503*1455620175', '60362605.6712*1479567425', '43882212.8446*1463649108', '83728703.5336*1460019039', '66964828.8389*1475492217', '89916766.9279*1477648528', '87524010.0535*1474972546', '66437080.7573*1464684937', '6032772.1745*1470974157', '74585712.6808*1467300775', '65390394.1607*1480141589', '28295653.2991*1457012287', '97772767.6757*1461862978', '44077314.4757*1456359902', '8884189.13226*1455632296', '18146674.8834*1465695904', '49604151.7355*1462182029', '51969263.5682*1473754904', '6431331.63544*1461580669', '61994695.5206*1465087911', '52799310.0947*1484614091', '14351061.9747*1472123477', '18578228.194*1486732184', '46931363.2145*1481645388', '45573907.8795*1454370320', '85456964.0583*1452195076', '48851755.7473*1466534660', '83756923.8723*1454063311', '36348535.1831*1458456102', '56231792.062*1482380648', '55434246.5361*1469679775', '9239265.44963*1469017087', '73283277.1522*1472837555', '97305323.8593*1478866558', '48662727.9643*1476538184', '2764066.33175*1456529310', '58938456.0802*1464773046', '97844176.3257*1466886648', '24631825.5787*1487733734', '4216634.58786*1465241988', '962404.196499*1455575812', '51283366.8187*1483706962', '19046739.0161*1479849263', '76008505.4406*1461152456', '65337071.9216*1471730203', '43483289.644*1462797082', '12389960.1623*1475088315', '16908499.8166*1478830077', '95453455.3974*1460437660', '37853720.2849*1475740862', '67301733.1093*1474172102', '93582143.9926*1468568480', '90269761.925*1475966864', '9756784.73257*1476353255', '33019061.6749*1477437780', '51358748.0197*1467401481', '77554862.2235*1468290124', '24118501.5597*1475196297', '61356267.255*1477720255', '62508014.9064*1487885193', '63930518.5762*1485304508', '98135070.7824*1465062924', '47448100.6098*1478906970', '10269310.8088*1453042834', '69005385.1106*1455852426', '81014361.316*1470501176', '20623012.4685*1469745895', '96053006.3853*1474727582', '57884746.3777*1477288093', '89354139.2179*1484935659', '69899315.969*1455635498', '38361176.2687*1486297640', '98877767.7176*1487661706', '35993525.8538*1481274453', '19751794.8975*1473996827', '51049254.6522*1468376643', '83790169.5375*1467507927', '11417825.8731*1472349289', '50109100.0611*1474643154', '83294895.8845*1471986269', '83245829.5633*1478674633', '20118495.2083*1476899774', '9955415.69918*1466546424', '36996741.9616*1455065233', '70759400.2908*1479007929', '18484127.8451*1462779102', '1468302.46954*1453953288', '65341561.209*1465273758', '70118465.4189*1456223773', '8235988.36485*1472386394', '99200387.1948*1456756967', '44589016.6104*1480184542', '95419651.5827*1465654334', '54456519.3431*1483211638', '66616063.6379*1475640018', '7593338.77161*1466297501', '69584448.7124*1462381279', '89655736.3663*1478705433', '88115826.0282*1460303860', '10405402.3544*1455080942', '41620472.3831*1469342830', '2864661.88916*1452141507', '44287508.5992*1471480174', '35776496.2618*1480527782', '60708543.3725*1467635508', '28646393.6363*1475531931', '19665394.0035*1454029420', '88927227.7856*1455011208', '20582433.7598*1466487602', '22745252.347*1455611975', '52116568.5662*1483742448', '31464636.5785*1471029509', '37732349.0573*1470813734', '94691173.2452*1477341785', '58394618.5829*1483362949', '33352600.5242*1476434768', '35115375.9043*1461589171', '18063723.2025*1452492711', '10072842.8403*1470713621', '66640054.437*1467001731', '50463703.319*1476230469', '72189393.0439*1482816355', '63720855.6189*1482936025', '52184774.242*1457187172', '92055475.7731*1452923510', '62215831.6994*1483105769', '87292777.4928*1478578796', '57663388.869*1474808515', '21446132.2267*1458860918', '15230749.8508*1487114692', '21475182.4811*1452718442', '28840217.6537*1486052064', '10974004.889*1475501647', '63721323.4809*1455679068', '38863760.6151*1483484926', '6968794.35021*1464868998', '24470950.7406*1454549075', '51316381.8352*1460670046', '80486143.1915*1455290987', '98275100.0665*1458579599', '31339842.9273*1483052846', '46782932.8994*1479171682', '26423885.7324*1483074134', '34466828.2868*1465312576', '74090397.6521*1458080011', '56049370.9408*1462727976', '58491572.7225*1462469031', '71935305.2697*1474692794', '63907252.1498*1453672945', '74784316.203*1475416352', '56249029.941*1472071201', '86064816.0213*1465166019', '61365052.3422*1469220902', '20707658.9889*1471606212', '97945109.9404*1475276820', '66560676.6665*1471868208', '60699190.5542*1466850404', '63090155.8203*1471303970', '7188202.80812*1470679262', '48949902.1768*1482235774', '31639060.6159*1465050908', '32915760.782*1474602919', '55188276.2316*1457553896', '86872377.9566*1484272794', '54704361.2256*1466918112', '33053118.3225*1485592474', '51528971.6421*1464044953', '9613442.24488*1458567682', '17576220.8524*1481317148', '25333624.6158*1478219245', '52184774.242*1457187172', '36297358.6473*1481399430', '42044096.468*1465329895', '38863760.6151*1483484926', '37731490.1367*1464122694', '7368833.61982*1455653691', '30031630.2231*1486405211', '98580697.5861*1485428471', '11554174.7998*1477910013', '16132542.4016*1453521824', '78154682.2927*1465647999', '73910687.1881*1470552236', '80445045.46*1455938890', '57780001.2574*1462111884', '85214202.4131*1483163953', '20549548.8412*1468077395', '27344610.8294*1477957353', '3117085.19203*1469743919', '43231849.2295*1461128664', '82188365.9507*1454443781', '2175375.85967*1480427548', '76381344.6499*1459218006', '49184374.7572*1479767061', '2838053.87011*1479999731', '35326345.7218*1463047209', '86978817.9701*1477861943', '71656956.4383*1473584003', '25863234.8064*1475841735', '7799372.50471*1476343022', '48335176.6549*1462329812', '35716615.8905*1465097444', '88765682.1899*1479069308', '65285664.8537*1479648048', '68346710.713*1476935042', '55222192.0774*1475790181', '61527660.5089*1466886639', '27893691.944*1483954045', '74819980.6624*1467215637', '66374382.2155*1484459834', '23150605.8498*1476134834', '10120273.5606*1466681406', '30226423.4604*1455021748', '61298087.1706*1459329538', '81335684.1857*1477033093', '39793695.6012*1482227442', '2475846.66662*1484877558', '36755079.9017*1459658436', '85418975.7106*1460051329', '77647621.3021*1470012338', '64838995.7477*1487933267', '41849815.6571*1465057205', '61023710.2503*1467149436', '60713585.78*1458565064', '24706005.6337*1458532652', '62797249.4303*1455945213', '23649468.298*1467851147', '77275141.8909*1487733265', '62945965.045*1474570798', '93581546.9602*1470298861', '43601284.6736*1478652828', '86794447.0575*1483658318', '35451041.1158*1461081903', '83022853.1223*1459955613', '44209902.7525*1477771397', '55821714.3403*1457323764', '10073646.3163*1456605827', '35731315.1036*1469268298', '60596265.0664*1459861690', '34903526.7594*1462634565', '46209625.4203*1451844588', '76891164.6166*1486163854', '70827992.3573*1467541141', '33489130.9111*1482257507', '40349574.3612*1471218352', '12149415.4413*1469111040', '34711391.9102*1460200359', '27123614.7387*1488235718', '63433853.1945*1456495496', '2120016.83797*1463874747', '2328533.12094*1477140014', '62551103.8456*1463625875', '63397667.7967*1461726416', '36739685.0249*1487071005', '33230845.4717*1479182828', '39257058.3865*1464747496', '92916742.6884*1469859783', '82282736.4622*1470230851', '45679584.6412*1469777413', '7432667.3109*1476486022', '13575620.5903*1466070618', '28402693.627*1462639826', '94352473.1281*1461034780', '17382166.1587*1485151447', '79837851.4992*1466235461', '34477355.2618*1462041061', '1453805.77613*1452425880', '83876125.847*1470764886', '41619945.1183*1470256393', '78971818.4165*1481257591', '60692608.012*1468098533', '37866758.9789*1470400025', '25130300.3325*1455063908', '58362393.6467*1468409114', '38573362.0936*1455221730', '3378230.53796*1475090917', '65491499.8248*1483285594', '96458958.2406*1463888131', '29806050.6684*1474444277', '98984736.3737*1454791437', '92729920.6667*1468168005', '58899514.3105*1472643398', '95305129.0097*1461465251', '69955463.5979*1473031363', '28927023.5459*1471539718', '56258407.141*1477060798', '1795220.37705*1478441481', '48423168.6373*1456794960', '38490895.8782*1479819729', '17895864.3853*1468340148', '63447653.6655*1478092551', '99757383.5801*1470945080', '84613058.7724*1486257097', '3719045.96316*1456716664', '71173782.2581*1482812553', '32938514.6959*1465359192', '65087598.8089*1483794668', '39187240.1567*1459259673', '69817725.7802*1487873701', '35006501.837*1483558510', '60152219.1544*1478654287', '8523408.02296*1487181430', '17050980.1886*1462210325', '20588442.2378*1453411981', '76161055.7374*1458796114', '7359046.86957*1453332005', '22069954.8558*1454577825', '46406752.4753*1473045439', '70780527.2654*1460592484', '95854876.0865*1467264447', '11485899.7022*1455171344', '28367568.4969*1461364883', '69899786.418*1478032243', '80491056.3243*1456193229', '22877474.0481*1481024609', '26396716.9446*1481796763', '5618402.6286*1488134169', '8647680.56858*1461057913', '20968752.1537*1469776948', '24916888.2105*1473411852', '36357642.847*1488306067', '40857306.5922*1474605856', '49688781.9762*1481896176', '13007227.6218*1459875124', '84173178.1848*1485318177', '26943139.7017*1476973394', '43681559.8879*1480145612', '84512193.3614*1471366486', '47846547.1094*1482204854', '87993801.1731*1470725646', '95070913.3723*1484090218', '861648.081459*1457806389', '15768122.1589*1483450158', '1015330.40217*1451949746', '33196579.9003*1469317311', '47875319.023*1457951708', '30324472.8052*1461616667', '65563376.8518*1465611937', '79801637.1496*1460590908', '52788985.1233*1453865218', '69459708.3058*1483012879', '31449165.0112*1457386756', '35720499.6554*1456124267', '48675679.03*1470171697', '22835642.1133*1480705117', '97965387.4966*1470699057', '92067339.298*1466926640', '34632829.6523*1452441177', '17176949.8498*1485858690', '5990674.94964*1487332484', '76099442.0605*1455154618', '57510562.9766*1483232867', '98469676.6659*1487350889', '335842.940207*1465071140', '59283553.5996*1473171133', '18402000.8342*1474651070', '37959464.4385*1463716705', '92114783.7596*1487352159', '84398147.6705*1462426982', '13051912.716*1481905700', '60403593.9504*1481585673', '39695020.7125*1483934461', '42642460.7332*1458187487', '77162256.4433*1485092355', '16832998.865*1485128077', '79358650.1937*1460940707', '19565267.058*1487675569', '79466590.8295*1459776343', '7171889.8801*1454531087', '42299302.5974*1485108841', '46233915.2439*1472875589', '46912709.4378*1461344593', '23842137.203*1470785443', '61578152.4619*1479316267', '61563782.1846*1461641937', '29652577.9264*1481707161', '82060703.6902*1457228199', '9414311.72829*1487740810', '78953907.5351*1459201442', '54264812.0733*1464377667', '16300469.6505*1484472068', '78767770.6849*1469572151', '23649468.298*1467851147', '85333419.7226*1455707008', '98288273.4811*1467172699', '81573281.8792*1474045474', '4692190.73707*1468473766', '2868342.82444*1482409181', '82113064.3165*1485507315', '97013000.9486*1452807355', '25673360.5849*1457612920', '29120363.4235*1477435190', '41420925.2076*1453561734', '98942745.0905*1484696663', '35062158.348*1487837423', '24920990.7006*1479703237', '71749442.624*1478281513', '95168863.2477*1468361049', '61031609.079*1452463430', '62343130.7804*1475438878', '1443307.96632*1485469991', '12288023.0052*1476829288', '1725686.47832*1460910220', '27593617.5653*1477232637', '50110125.9861*1487581696', '49393204.7345*1455910201', '85203490.0913*1478702623', '47969218.6746*1478754471', '51602149.5233*1459934967', '47538897.8551*1469794798', '40795850.1707*1482550800', '48174124.149*1451893789', '13007227.6218*1459875124', '73887500.2367*1480868474', '20434177.8554*1479217649', '82733154.6612*1487100107', '42371230.9479*1465407137', '43352871.1978*1486153211', '73157975.299*1486171952', '64807665.6094*1479687734', '30624367.4864*1461251785', '43458066.0969*1480160797', '28462663.8341*1472814889', '40540207.9327*1452345021', '82303175.2198*1469990335', '11418131.1841*1454308486', '54015539.3266*1486642093', '21287177.2835*1484645183', '91068118.4089*1480289842', '79775216.5189*1476068210', '25858187.6948*1462840610', '96834449.7728*1451860532', '37979083.9448*1478178183', '4978402.52706*1473315398', '60378057.1684*1470966514', '57560407.8185*1468721554', '18475636.2339*1485259402', '6525548.69821*1483811839', '19277629.3925*1453331761', '92563594.4395*1471194511', '86906043.4997*1479094259', '74998441.184*1455180106', '45573907.8795*1454370320', '15638001.425*1476243208', '26687618.3422*1458180319', '44881438.0094*1460475838', '31346537.4346*1487343906', '68391229.1294*1465419075', '52924900.3104*1476375260', '50472225.2008*1460999664', '57425256.8209*1460718388', '44988375.7955*1453236054', '62965169.8173*1483987206', '82664768.0628*1472694661', '39923913.7304*1455060771', '21220006.5086*1459950627', '70653244.4556*1463181398', '25130300.3325*1455063908', '68751079.005*1457863269', '41356862.6315*1482072074', '90750166.5331*1477053068', '26322776.8659*1467708359', '35685067.3832*1462576618', '97916441.8639*1484651043', '50364249.5743*1475229412', '30476571.3183*1455436764', '37178054.284*1458983663', '48718977.9598*1454530986', '22877474.0481*1481024609', '95300628.4725*1472867294', '46282103.2941*1458214196', '88222720.4922*1471640420', '44661950.1107*1487960824', '77514996.9926*1481890568', '81500457.4943*1453175254', '84798816.1184*1488213267', '83220730.3799*1467914568', '73770969.4284*1469893396', '36292850.4577*1456729774', '38842653.6322*1452946274', '27553110.4284*1485829485', '7400891.56672*1452659005', '93219459.7716*1485778001', '33853188.6394*1483811634', '24719442.9766*1464997374', '34388970.176*1466353797', '1781958.78316*1469670356', '16287067.7578*1484821041', '18818600.962*1453860255', '43165348.5882*1463859525', '22811884.847*1472141310', '9705808.4671*1466260849', '34887265.8627*1459403517', '13551.9773688*1470525446', '54593588.732*1460834613', '32581268.6174*1464642869', '5620796.12616*1452801801', '52463244.8196*1472090281', '72228773.1214*1459705100', '24207954.9982*1465834382', '43186165.0877*1480064490', '34547959.6047*1468151560', '64203994.8899*1459207032', '7100534.45315*1476981819', '62145254.022*1483616470', '9703562.00216*1458486364', '45063403.4637*1461887466', '14134390.5681*1452693028', '76159341.7021*1460728584', '98063265.4336*1452782503', '71468698.8778*1485386064', '24318392.9075*1473968722', '61770525.2444*1481996033', '77998643.7802*1462509571', '29128711.7196*1485113412', '40569084.2568*1461187550', '31080301.6042*1459974594', '30226423.4604*1455021748', '9478423.81687*1483565251', '73568494.2763*1486931588', '8991685.2671*1486126701', '31687572.4641*1467527448', '76453396.9596*1470598239', '38812324.9272*1479370894', '19279025.4704*1481590094', '83147715.7266*1467153028', '9955291.42238*1468200913', '9638462.94408*1474879575', '587589.890402*1485267186', '87113739.2345*1483346964', '29054330.8165*1459222993', '68064373.5855*1487044041', '17707334.7873*1487567795', '71854281.7092*1482709512', '78157005.0367*1460128805', '68920502.3745*1479835293', '84183928.9409*1462446757', '86533036.4385*1468951769', '39662987.621*1472390407', '78295640.498*1485115941', '42330561.3765*1456846869', '56328366.7071*1461164437', '34167583.5148*1468615461', '85872182.6465*1475459910', '30154440.0772*1460463590', '37846296.9214*1453056246', '4616336.64646*1473681260', '54166997.9285*1483129201', '36982073.2665*1462339312', '97841720.4333*1457975195', '87142391.3533*1457689085', '16662240.3676*1485243537', '7953488.74371*1476407389', '96669064.6669*1454454649', '1444539.00198*1475010101', '48515914.8262*1464099685', '34167583.5148*1468615461', '18376072.0681*1459025940', '86657360.9525*1452635710', '35661698.2181*1484329445', '16589062.7047*1462665453', '63116834.8683*1481187027', '76594627.3866*1473520625', '86331403.3712*1476316890', '98091239.298*1459713634', '65005985.03*1486022269', '10444348.1757*1476299959', '68453046.463*1468796505', '74390744.4758*1471492142', '92510437.0247*1473339304', '98200969.7693*1460952109', '71825719.2712*1479671230', '13551.9773688*1470525446', '6455416.79104*1481982416', '7253976.41411*1472440877', '57246570.2858*1458543585', '65899918.9957*1458249037', '59119989.0687*1480025182', '61432944.4897*1466112214', '59167254.7781*1487650200', '82918014.4627*1468734655', '27150918.1975*1458495391', '24038004.01*1488012521', '6541423.10738*1459093907', '20652016.7285*1483472562', '68771495.546*1472142361', '59547802.134*1474708238', '6042259.57904*1460139735', '87465917.7226*1484454672', '54598460.5975*1474819687', '11148816.1375*1458884024', '47539518.7292*1480811690', '16378519.2587*1477928033', '95132790.9227*1459151022', '76897693.7249*1453122069', '50051819.5774*1457698520', '1540585.61627*1454419884', '16110413.7985*1486096172', '34221429.8949*1469208281', '64883735.2425*1481895579', '30110831.5051*1479401753', '37866758.9789*1470400025', '61583585.2446*1461591469', '15838468.6881*1458794400', '37965331.5757*1469831449', '54535878.5164*1478166202', '15399751.2877*1473952196', '16718234.1725*1460975253', '36359720.3879*1485290437', '70958029.4151*1480403018', '77383153.8505*1465559814', '63375366.8733*1469950543', '78822122.4248*1464105310', '13209882.5486*1485361372', '21542311.0399*1461538441', '34831696.3944*1481838125', '20008243.7137*1469686278', '20411542.6159*1484253484', '90966475.2894*1472987703', '31404170.6801*1466075974', '75052178.7259*1484433617', '89689176.0156*1485074093', '10914158.278*1486969601', '84651013.6208*1463431525', '28767117.6448*1473747696', '64128729.2818*1466347846', '21643583.7805*1483621568', '30537183.8633*1467043289', '79497016.7357*1488319813', '7376695.57632*1459485137', '12744494.4368*1468871484', '38357464.9222*1455693764', '24283785.6705*1460674661', '63471345.7947*1474714536', '50956636.9308*1469697234', '52597459.6994*1466562365', '93631498.1297*1479676058', '70269561.2469*1468720152', '48535500.5081*1471966273', '43937656.2289*1454594623', '15729156.9246*1479818801', '37531548.489*1483559726', '51446024.3658*1476092722', '19789820.2759*1464288667', '11659353.8589*1453386744', '93920041.387*1470102373', '41005483.9387*1470819182', '2989478.57569*1458167732', '42844374.6079*1457765861', '61006354.4653*1481395026', '35263111.4217*1482173925', '30630452.5144*1454360848', '90351472.4933*1476043976', '73843107.0432*1473585990', '39921361.8036*1474848672', '80974076.2097*1480428575', '63038309.91*1466093864', '98984736.3737*1454791437', '1348992.32678*1460636111', '97946161.3748*1487223382', '25321054.3923*1471870712', '31639060.6159*1465050908', '32811648.2877*1486260979', '86555110.6933*1465400207', '93610213.3441*1476158920', '68970542.5493*1458929008', '3971278.63611*1471273805', '31901642.8053*1473506952', '88623988.0096*1487816688', '4982434.39173*1481806246', '9490148.15465*1487500062', '81617115.3214*1483269589', '89916766.9279*1477648528', '36755079.9017*1459658436', '86427707.52*1488021081', '16027854.7665*1473241391', '77162256.4433*1485092355', '62714058.7339*1484740867', '10226208.6713*1459384011', '76392377.1771*1484563639', '93120206.5809*1487812740', '72399306.8919*1476255624', '44467313.616*1461843055', '91590410.656*1453361398', '2594664.53178*1468722351', '82146999.217*1477512614', '37584644.6925*1465146265', '23264808.2538*1482982241', '76221691.589*1474288100', '4189304.33149*1472778773', '35186908.1224*1476210899', '52637325.0209*1459250533', '96973474.1087*1470420940', '53900814.4926*1467629313', '17475022.2567*1476158446', '70021353.3143*1470402205', '36868627.3692*1463202675', '66804838.1681*1483538677', '73864016.387*1455864599', '90740086.2578*1452287620', '72582064.7831*1461118595', '52689498.1999*1466206275', '63791433.7768*1470026319', '16648498.2838*1488222010', '60511370.9858*1485637529', '22746023.9285*1479283005', '6959645.12973*1461169513', '74778298.4072*1479879394', '80562399.7877*1473337341', '73370270.7743*1475837084', '85457842.9144*1484812665', '55199439.5463*1461607890', '41976531.7412*1482655853', '23627864.6354*1457453427', '17384285.4312*1463214331', '59614492.8773*1485562885', '92532714.437*1468803396', '70314756.5984*1472277075', '64102342.7487*1484684377', '44987890.9089*1470194613', '3385550.52216*1463910293', '12898172.8555*1485341337', '84259447.6843*1481000161', '24524922.657*1466193825', '10456267.4272*1461500043', '4681902.9453*1462747521', '65269239.1999*1479310813', '83192621.9127*1471471745', '84564906.436*1453353213', '27088612.6432*1475420105', '55842669.2049*1476572881', '23520289.5666*1454888909', '36476117.9019*1472546298', '53176293.3426*1478618704', '14984620.7182*1469200695', '93351473.5936*1456269238', '54292211.0253*1486506037', '62199855.4978*1461464892', '85638015.5853*1479292764', '23763493.0923*1471573760', '37831009.0785*1451816738', '24617447.6542*1467807964', '19462475.3364*1472127565', '44209902.7525*1477771397', '40783179.3707*1455237667', '25558409.3094*1451752723', '44836266.5518*1467878220', '20998341.4896*1480655997', '76347651.0466*1477207043', '63186118.2202*1460774926', '99946140.8864*1488225550', '5866308.30691*1463291330', '16068366.7774*1482230672', '90469006.8067*1473272789', '11878993.7819*1476224878', '57632315.0981*1484077297', '76269041.6458*1463826925', '17501646.2697*1458326204', '62077102.5542*1471213973', '77855166.2174*1473248832', '18475636.2339*1485259402', '21562649.874*1462932035', '32537416.7752*1481642413', '84621335.6679*1478321187', '51627654.3039*1452468868', '79421334.5204*1452918657', '54904730.123*1470340240', '27739061.8828*1453424792', '86091822.156*1466579245', '56827532.216*1454864257', '23555438.1598*1467265045', '8960179.83439*1486455540', '64068419.5537*1484687389', '97473939.8165*1455019389', '90469006.8067*1473272789', '32908454.5532*1476412872', '67782362.5896*1467276548', '60764972.7195*1459198975', '95132790.9227*1459151022', '39211261.907*1455881593', '9167851.57144*1461122065', '44511613.3615*1460557653', '60840901.507*1481857884', '76939313.3095*1470767268', '87626001.0719*1487841447', '58678200.1232*1457412365', '71496151.4527*1469411837', '84081623.9402*1455047051', '24321710.1443*1480914093', '44955464.7388*1456189648', '29366738.2145*1459224722', '14629460.1054*1476611724', '14502772.5131*1465842707', '79686094.3668*1463998790', '10011228.0808*1480568552', '9956530.71159*1481224529', '67678523.2841*1457359735', '38742476.7816*1470121486', '17812115.7263*1455512405', '26787189.1501*1473799084', '96476864.9666*1472986135', '33168166.0976*1477346578', '45726139.0133*1484777702', '97945109.9404*1475276820', '60105353.2425*1484815997', '73937630.2072*1486162158', '63116834.8683*1481187027', '28462663.8341*1472814889', '15991534.6228*1465671524', '12073424.1546*1487156974', '72715939.1212*1483941970', '48777156.9147*1457674441', '77275141.8909*1487733265', '48245892.6301*1481091632', '67308854.1808*1467778802', '18178851.4247*1480658632', '33567375.7345*1482225555', '78775095.1637*1481896308', '10291942.6213*1485439095', '87071940.3248*1452501428', '69684596.4718*1486631176', '83232946.2837*1481891459', '44210684.1807*1485801525', '5670599.37304*1455601031', '68550121.3672*1486827063', '88633521.3815*1479388024', '94056721.9946*1482418666', '15830655.4809*1481847867', '27401989.7064*1455964958', '94542951.6769*1452134995', '50393860.3096*1458841635', '82547245.9366*1459142317', '78184002.2396*1456639374', '65973413.386*1475487386', '97245858.184*1457750519', '25423070.9846*1470361031', '41349488.3454*1462440719', '64882709.7277*1467044431', '20112648.7419*1462383970', '79471146.0651*1487548766', '58027483.2578*1463157418', '57884378.7096*1465531163', '75078563.0442*1453513833', '76767332.7421*1488299995', '57317432.7962*1469318659', '12011497.5298*1468862006', '46480750.3339*1480462336', '73911423.6529*1483234339', '33280122.6462*1488324413', '96198663.4719*1481182888', '64341801.8741*1456450030', '11340214.4796*1481469932', '14373689.837*1486732074', '98318660.0629*1456812678', '38610295.0352*1471331774', '8815039.81209*1485236479', '82807217.2784*1487958755', '33207740.6029*1455819831', '29767898.4312*1465480589', '56394288.7734*1476711830', '88043067.6536*1480598836', '96335161.9234*1464239137', '39052214.9197*1470247804', '40910656.0417*1455428264', '74784316.203*1475416352', '48050868.3928*1484143884', '32581268.6174*1464642869', '53038711.1483*1470904721', '62975857.8943*1470293632', '24226887.6851*1468993430', '25863234.8064*1475841735', '86978817.9701*1477861943', '17922715.5064*1475010812', '92386222.1018*1459309032', '49050079.1888*1456334038', '63821402.1045*1484233421', '45846982.1624*1473027213', '17832130.7663*1487199104', '78888624.7403*1482056736', '17561814.8888*1454413032', '82433313.7189*1479407133', '72192312.9122*1482402046', '93038561.3617*1467780174', '8718822.245*1469764471', '31281587.8304*1477836558', '91216861.7876*1483048387', '58682406.471*1468119498', '15354103.4245*1468311941', '86794447.0575*1483658318', '82835232.425*1463337679', '78568819.4192*1461352506', '80974076.2097*1480428575', '18796415.7935*1471793924', '1461565.38558*1457509152', '25123401.0592*1482967176', '88193063.6659*1455068032', '88084863.471*1459259390', '85001670.1572*1488070752', '18507184.2596*1470889240', '16256766.3029*1452336993', '87696386.7836*1458415790', '86942220.6194*1475125319', '33197374.6227*1455299230', '54978772.2292*1475322717', '68965997.7276*1479570522', '18303472.7288*1478287047', '7015289.72254*1483307602', '51825295.4662*1455717073', '8738300.95406*1474533788', '36947745.7804*1474728050', '23683587.481*1461740019', '9878455.31413*1483164533', '69973717.8717*1485473614', '48041112.3033*1475137192', '39120905.6218*1484531362', '34630629.937*1470679703', '27568723.8218*1472802889', '97593172.5981*1467736192', '74201791.3428*1467829609', '37597903.879*1451905921', '65337071.9216*1471730203', '74819980.6624*1467215637', '11138241.85*1471669561', '18391811.647*1481320989', '32685525.4686*1486953073', '95385089.8328*1461641602', '17541034.1249*1470172937', '27156873.047*1481166968', '68569923.0929*1457800140', '40447822.9378*1453683097', '58106258.7239*1463970024', '90382222.0021*1486809600', '69899500.7296*1465672569', '47254010.4028*1483394208', '29529432.2731*1471856824', '10291942.6213*1485439095', '4535388.01978*1487597165', '87794012.8539*1482757364', '49584069.2458*1485695042', '85442660.4974*1483700101', '65681464.7426*1486528418', '32506216.2746*1468232872', '78870191.8252*1456366084', '77076711.2329*1476077154', '68268743.6239*1461014012', '65598941.2446*1477887330', '74685833.9515*1452337219', '95418229.3357*1456370992', '46357990.3359*1465837350', '46649339.9655*1460502363', '51725117.2122*1474225832', '53087648.5528*1455091667', '96820422.5895*1454438240', '24684854.9766*1474675300', '63415646.0155*1473286352', '59551788.8065*1484043244', '87162634.4635*1484111886', '47140604.8272*1484936478', '24998590.0922*1486998789', '36566253.3828*1487539117', '86266035.1103*1454292029', '94546983.0746*1479012511', '1101289.79914*1454385701', '4702291.2807*1464748965', '63433328.0578*1462910206', '41078036.9646*1458901000', '8901244.09499*1456997074', '1404955.02153*1463354901', '21549400.9814*1451607860', '96125772.0943*1466473272', '26593713.411*1487124250', '95168540.6903*1477307527', '91917003.0057*1474969260', '2229330.63609*1477641464', '47280032.8174*1484887029', '77873605.1851*1481728579', '30779989.5964*1470294589', '29404142.0129*1484094640', '67782362.5896*1467276548', '2374168.26026*1472067351', '60502008.8103*1479078418', '44407764.504*1468793612', '46274533.7902*1488062828', '10120273.5606*1466681406', '74769664.1736*1477001630', '35104596.4035*1462847179', '90030953.6853*1481958211', '69336552.5568*1452348638', '13642937.902*1453249205', '93571196.7334*1486383847', '75847319.9474*1481641632', '18928149.9888*1479176697', '24155770.9917*1475817668', '1168323.6371*1483112731', '40499219.3134*1476095630', '39471987.9224*1459334181', '92861703.9146*1482726380', '90498686.8714*1478248924', '61088775.0631*1470246741', '16564165.3928*1456548289', '37494721.2256*1477499076', '72254525.6563*1459867139', '15354450.0987*1451874436', '98720410.3928*1487874190', '17310201.9012*1475289657', '23386010.4084*1459113523', '54416481.5645*1487519801', '88913589.6075*1467003579', '76174753.3712*1476340708', '61190171.4573*1478106825', '50605390.0616*1456556262', '69899786.418*1478032243', '57452359.7267*1476590353', '85767911.4124*1485036424', '64579019.4272*1471552539', '29806050.6684*1474444277', '21129917.9065*1461945735', '92254696.775*1486535478', '29951584.9101*1459268827', '15111777.0443*1461929325', '21569033.4556*1456817576', '9222655.84307*1481932993', '88537969.5805*1486092109', '77815547.9263*1474347203', '1045342.16572*1478584127', '14455056.2218*1482918980', '6464775.65212*1485220349', '52466059.0737*1454683445', '86089201.37*1478892578', '30476571.3183*1455436764', '96347317.1491*1455019422', '19666812.233*1455712092', '13934399.7815*1455443280', '95679793.6152*1457744397', '98381702.6075*1459320441', '51781270.48*1467463553', '72521501.1454*1476063173', '52689498.1999*1466206275', '11401741.1709*1458417780', '73059593.0747*1462021189', '20331952.7319*1469284557', '64405551.3933*1484576700', '1206763.87797*1468637073', '9810794.49223*1480918299', '28475264.2893*1483544160', '66178372.3334*1486604593', '43786908.7241*1472670872', '89967798.2734*1460026288', '40203747.3503*1462195074', '31076179.9488*1480180836', '68171128.7078*1462989872', '17071546.9958*1463474390', '19856691.6955*1475580821', '72858050.3477*1478931653', '82168230.0448*1452939253', '62178849.514*1467111099', '60150942.4829*1458185526', '47541581.3932*1466257500', '83245829.5633*1478674633', '96586437.0848*1474261121', '30656314.4597*1475708899', '19361816.0074*1486939537', '2594664.53178*1468722351', '58018068.6227*1479682670', '30261722.9407*1455148304', '41080475.8451*1479795956', '85401321.782*1457689257', '73460094.1675*1474245760', '94746869.7313*1461748617', '54091756.5235*1487939074', '39966625.5957*1487608894', '74993331.0615*1468195072', '3021992.34175*1482498588', '18269120.2916*1466775364', '55438165.1565*1461091814', '56697177.9525*1472741733', '9699288.3936*1466454739', '3600053.97276*1464337755', '64013188.6517*1486199789', '6037697.7224*1457224525', '6312346.96338*1467397899', '72967983.2165*1478834027', '73044063.9856*1463269853', '69120284.6989*1457227828', '52557961.4474*1459114165', '51332325.5883*1484436275', '86232585.7365*1487075326', '10796762.1082*1455011479', '76679799.1034*1482876227', '53765167.2083*1460764269', '45630860.8536*1460883378', '45751341.4956*1481536478', '73603215.584*1483972350', '87213016.7692*1478342313', '83953157.8917*1475589274', '58514846.7051*1486826980', '4153441.9336*1478357746', '6505147.92137*1466852996', '5502731.07443*1472921840', '786259.274728*1472370502', '52915326.1199*1471933236', '10141434.791*1484720867', '66079096.9928*1477951817', '38905161.5287*1466258198', '8027674.26796*1484624920', '50936757.529*1453382695', '61220376.3832*1454520818', '41811046.5407*1455389623', '62336814.4617*1452174023', '28914033.6092*1458218979', '88107714.393*1452845969', '71314646.5791*1464015844', '49994833.0946*1468614918', '10128076.8383*1466381495', '83377750.2116*1484012550', '29529432.2731*1471856824', '33489130.9111*1482257507', '61465972.212*1478441926', '73099351.5714*1466588750', '66037380.6218*1457755659', '52420829.8341*1466366160', '34630629.937*1470679703', '46023780.9012*1472976974', '95722666.4642*1468580839', '98709334.5*1479524402', '6296440.68912*1456800228', '3419528.46254*1478660350', '15462222.7527*1454607964', '69652983.7107*1476172765', '4360803.0256*1469529922', '16462240.9299*1462816163', '21187326.8962*1456921458', '49439611.6075*1463594975', '27048275.2235*1472631049', '61418250.8189*1484800332', '71215465.9904*1463795217', '21488706.4876*1478574106', '64754319.9103*1470074785', '19098642.084*1463150244', '99822058.3123*1469980972', '77685554.7148*1483357089', '54562377.5587*1471526206', '92407430.898*1476175856', '98782315.5255*1485720392', '9504975.44349*1464386012', '65478088.4096*1466760845', '10444348.1757*1476299959', '42449678.6245*1469750859', '13068919.9882*1481053585', '65699842.3989*1467380086', '34618277.4446*1483681601', '19551554.4522*1474195043', '2444194.55913*1475081707', '6051359.06501*1457989802', '37046338.6217*1479491878', '88005602.6783*1464639901', '41565322.2779*1458827572', '90620615.8916*1475346075', '62766957.1312*1453082412', '73235175.5704*1467804760', '19288657.8033*1457334689', '12060786.9847*1470647917', '8545538.96743*1483313037', '17931582.1011*1488217898', '26844528.0282*1459134195', '11366025.5491*1461770562', '49370100.8923*1474553993', '42316433.2308*1457518045', '20746728.7236*1472117190', '79610609.7595*1470581623', '10510543.0401*1458983580', '58899514.3105*1472643398', '96184548.3261*1453883399', '88765682.1899*1479069308', '15440822.8106*1468201731', '89616935.248*1483067824', '44788628.8721*1456476630', '59180767.8461*1452960650', '1465801.24731*1455222928', '78786426.6537*1475270215', '27893591.644*1463608559', '12412851.9355*1480677908', '9161559.51943*1480279596', '42586727.9094*1461132657', '8465867.40507*1453299440', '92901105.5182*1469247377', '25969530.3458*1474260325', '62848548.7678*1468255288', '84889910.275*1460145199', '23028912.4552*1487161948', '94216837.8185*1466887139', '58622824.3527*1472365161', '77336991.9422*1478804442', '24974732.9539*1485928235', '15422485.4377*1477374547', '36498784.9902*1468416721', '69973717.8717*1485473614', '99247484.2731*1462003045', '54257515.4436*1451710949', '97804409.3486*1474186327', '75067400.5447*1467115887', '4508121.46336*1453701607', '28115322.7378*1461544927', '22381935.4009*1476507451', '48948944.9114*1455022964', '58875753.0823*1468507731', '1679384.81235*1460321599', '5612862.6688*1473023788', '90099531.3799*1466219810', '32073974.2953*1474287812', '27789418.8785*1479850639', '90392441.3025*1487685234', '13794527.1983*1463851768', '81969433.5339*1481820941', '57736160.4888*1475054438', '39773123.3149*1479504306', '61402843.0569*1473237412', '58619703.8918*1476901382', '31094566.9718*1469009767', '10955769.7095*1455702216', '8457676.27452*1486202283', '99026024.6626*1481462670', '79311330.0107*1480041139', '79183884.4133*1461486147', '92619748.1828*1456741560', '638096.611254*1475023015', '46309537.7745*1477027811', '82016660.7408*1481954643', '67613926.0587*1482954598', '37426251.7085*1476939820', '29647535.7858*1482639854', '73798512.0115*1486616823', '21643583.7805*1483621568', '39046325.2514*1455168260', '42486742.698*1453284558', '73484879.8942*1457197692', '83919354.626*1478481644', '69328946.6898*1465486158', '90285628.4897*1453268867', '45927664.702*1484200087', '20755670.7466*1472736532', '45527212.31*1451700694', '85530025.9188*1487851132', '18292141.0544*1482284703', '7611854.49726*1452214123', '93581546.9602*1470298861', '84173178.1848*1485318177', '87224267.2701*1479929991', '52637325.0209*1459250533', '41831087.1826*1478611757', '24017094.3502*1468313518', '21858367.1994*1472780692', '15711940.9217*1463900541', '55238281.231*1485461790', '63856926.6786*1466305406', '5192866.94618*1477635642', '45973860.8993*1475481616', '36460836.3181*1486952358', '28948677.5589*1464268076', '49135233.9457*1484241922', '34372543.1894*1477519058', '32669242.5245*1471088026', '41377675.7309*1463685208', '94704687.2151*1457357840', '18632116.4302*1479806662', '68159463.5967*1462044144', '34807395.0422*1475683713', '43438521.5088*1464604014', '412768.538182*1488077301', '13845528.4296*1456243234', '49264568.591*1474211922', '10060277.4428*1452753019', '23634633.1495*1469331324', '32602864.9111*1475073509', '62714058.7339*1484740867', '25669188.9652*1460384437', '37615185.132*1460797185', '67036967.7365*1480468252', '30687203.3082*1475803015', '66783869.9895*1475555239', '72303129.813*1479224683', '76163215.5182*1476500452', '12593603.5607*1484091996', '91358472.6477*1487670782', '98752552.7216*1473097922', '61618471.6589*1463262636', '65231622.553*1452398529', '27724796.6631*1475736321', '44120809.1241*1469883062', '73460094.1675*1474245760', '91461889.0677*1478107705', '70635644.2005*1469938054', '2072196.97026*1453908840', '44245232.6426*1463520999', '50984176.1506*1466803142', '77724100.8989*1471837505', '60122529.6105*1483581909', '80397678.4545*1478165663', '92072443.1382*1486126266', '39101571.673*1467573356', '36321066.1365*1467987090', '11366025.5491*1461770562', '26088132.178*1458546378', '57439138.655*1475397378', '61663765.311*1466666337', '26019115.0678*1465484059', '54091756.5235*1487939074', '94229536.5862*1456653600', '50271754.4365*1461145974', '57341926.1497*1457576443', '86080549.4597*1472989654', '17015034.8559*1462241475', '39249521.6279*1454343810', '10775065.8525*1456126595', '25220604.5739*1479645046', '20314000.94*1465876215', '64027622.6582*1461688585', '72592271.6205*1472004874', '67020962.2655*1455384453', '63720855.6189*1482936025', '6131351.37928*1472216669', '84611888.1176*1469914449', '36131032.4312*1469956457', '20426158.2745*1479646396', '13127208.1902*1485484873', '60752139.2396*1472330926', '58535248.2076*1472016535', '77506941.4067*1457849154', '43240488.1816*1476151447', '41873764.2978*1459547401', '83656056.6214*1462413880', '74199098.3634*1464506276', '83686066.1439*1461969114', '86558968.9784*1483171159', '42001718.1506*1461512761', '36858583.3293*1451897630', '36178445.6232*1468293875', '34810441.3321*1483037476', '9482423.87883*1478882378', '14846398.897*1481535092', '56884533.7507*1471421594', '9607430.67394*1453631592', '99725118.826*1474229929', '13242041.5526*1452747674', '61172574.1903*1474803041', '10766284.135*1469925795', '32518432.5681*1481494075', '10771603.7809*1472961569', '41866067.2399*1473009096', '67042326.2333*1484402335', '70027002.5394*1454221896', '35565455.8624*1468808003', '38412096.425*1482038156', '62240158.6059*1455784536', '98942745.0905*1484696663', '42899408.762*1451657744', '21204061.5101*1452823522', '54074246.3766*1484332585', '44612205.2887*1457302840', '10776111.6784*1452888817', '8050069.5669*1474240582', '80668996.6254*1452351161', '57082619.3064*1478900236', '54471305.8145*1470138225', '37862482.0688*1485296699', '5150049.10166*1461820557', '65956129.9643*1469516222', '57837972.1367*1455473336', '56834081.4325*1458805679', '34308716.5266*1469096465', '99821409.7679*1470903523', '55918777.2932*1484862598', '21639639.4327*1485688176', '7953488.74371*1476407389', '16730713.4809*1471551543', '21085150.4956*1474614461', '53602995.6356*1466140425', '26063083.9318*1451656253', '30789660.014*1452687642', '48949902.1768*1482235774', '32574430.2443*1480454724', '56253461.5824*1461227126', '87889043.6878*1457726663', '94081728.6893*1474523233', '15177708.7369*1468905991', '14228172.3761*1480659475', '91467279.1417*1463654391', '9735334.7757*1485290854', '87418668.6991*1471840130', '19776634.7143*1473847925', '5618751.4615*1461518432', '36982073.2665*1462339312', '19872889.7784*1469364936', '30770860.0535*1465582716', '23169635.0462*1468464524', '63757258.0675*1469231310', '99060219.3198*1467410742', '1485013.67086*1476391589', '83688658.049*1459576981', '69951672.4705*1481720490', '21625225.9297*1465985955', '81045.4824318*1461960956', '12621497.8978*1467395133', '68595162.3681*1481010706', '877982.435468*1457885869', '47813258.329*1462305196', '40677169.8441*1485391258', '39920044.7631*1465398458', '49879544.6113*1473050078', '8113085.15592*1481036592', '66270323.6906*1469507347', '59104133.4656*1456599986', '1443307.96632*1485469991', '52184774.242*1457187172', '72754780.5597*1458984517', '61348477.1249*1485990532', '78272910.0972*1466538268', '82113064.3165*1485507315', '85001670.1572*1488070752', '81839493.3422*1469150441', '72086924.7777*1485021850', '91445677.6474*1458406724', '18408125.2465*1487550522', '50431652.0277*1459875603', '10456267.4272*1461500043', '36960021.5382*1469315738', '653692.59119*1462232505', '37876993.9795*1464019431', '99395070.6665*1470642767', '38179208.1926*1487033737', '45131227.2302*1476190897', '31272354.5319*1479553311', '53074351.1449*1486346153', '30492662.4652*1451969401', '5166480.25669*1468442606', '12474726.7585*1478847591', '73455178.018*1455860802', '15354450.0987*1451874436', '19116954.855*1475978739', '69328946.6898*1465486158', '97421042.9157*1484595567', '23233964.7252*1456916919', '40018054.5831*1456681014', '75949195.0729*1483153840', '82274925.7994*1471418985', '92769830.2594*1453787516', '35794134.3356*1485553702', '79159313.3955*1479737281', '16725446.5547*1487015107', '62549629.6644*1459853692', '12434043.8962*1472520772', '60764972.7195*1459198975', '93801961.1155*1470193247', '18508437.7354*1456811901', '30776938.9166*1465455754', '40653296.1227*1454902631', '29983166.4096*1483855729', '94979756.5695*1474737585', '81741865.0156*1452080674', '18367743.0908*1487147736', '43014380.0468*1482622775', '82019757.4685*1472627582', '69899315.969*1455635498', '83756923.8723*1454063311', '6518435.72399*1460109789', '61348477.1249*1485990532', '96409707.5447*1462693370', '91376500.1419*1476799235', '35993525.8538*1481274453', '84935964.3606*1474403822', '83740641.7222*1475568365', '39913834.709*1487143203', '59319282.2529*1467883843', '39689048.1278*1471010145', '74025552.2016*1460069714', '44509420.309*1454194737', '64054640.3568*1453844453', '62669721.9156*1472692457', '88965137.7303*1459309912', '79224437.455*1484771671', '71117506.1651*1458271141', '42031772.2692*1454581911', '75257613.6019*1453045271', '16552404.0963*1477887823', '16898679.3292*1462244120', '20604175.1064*1464497189', '23467117.8*1483943235', '37045012.7076*1461665673', '13642937.902*1453249205', '9607430.67394*1453631592', '70666250.052*1462052228', '36900820.5231*1475941106', '99374367.9318*1451616403', '34301868.5327*1477160606', '39249521.6279*1454343810', '21134133.7169*1482029622', '19623928.9111*1472222532', '94548554.6353*1467682053', '95331859.3014*1465518865', '39913834.709*1487143203', '57915269.8238*1456209859', '55222192.0774*1475790181', '91313825.0511*1459556763', '21187326.8962*1456921458', '13921316.6147*1486739613', '69174077.7102*1474898079', '2933542.47005*1462017352', '42819578.936*1482867695', '72631991.4046*1485400073', '49708419.6291*1468612967', '39112700.965*1451910519', '18515123.9178*1481836171', '88953670.0063*1452525036', '38153474.4301*1474503693', '22716401.5901*1470152106', '8206255.63261*1452975753', '79727830.1988*1456343442', '76242616.3156*1460119211', '64611322.2385*1472951005', '82113064.3165*1485507315', '50183253.7101*1480071845', '99068085.9083*1478310663', '28377976.5652*1467580385', '16898679.3292*1462244120', '33286295.5917*1478668217', '67160711.562*1463791369', '28641948.7364*1486570213', '70062509.7754*1451921618', '77284208.0877*1486954823', '69673255.5772*1458157813', '45032413.8362*1479869612', '81913489.5186*1479253117', '22354638.8722*1483457246', '64518006.0764*1486978079', '20553360.334*1469287006', '28737029.1804*1457778563', '63036515.3578*1475584075', '64385176.2195*1462822684', '26932751.0698*1466794692', '27770921.6948*1467618767', '41864032.5573*1470440858', '32170450.3157*1485803845', '80083655.5628*1484897390', '63112432.2939*1474718536', '37462568.7285*1454650427', '66306880.1419*1487748501', '30922528.7125*1468727201', '59832997.035*1487851146', '77787136.7589*1483785312', '45815448.0306*1485224317', '27568723.8218*1472802889', '38201739.6372*1477638493', '89678597.8527*1453534129', '54586817.8666*1488193847', '60219713.1634*1467787430', '98867012.208*1480952789', '19403994.1024*1485470316', '14195142.1117*1484340376', '52128745.1394*1479969451', '59405607.9781*1469886051', '65287850.1616*1479182317', '34831696.3944*1481838125', '95381983.7623*1471398462', '8508225.13081*1472375680', '82946378.6758*1485833584', '84517869.1604*1487437414', '69459601.5553*1482988793', '47417976.1223*1461051878', '35176923.1095*1458362067', '34903526.7594*1462634565', '85869416.2682*1459830891', '21245090.7102*1483395846', '69459708.3058*1483012879', '83916247.4054*1486342561', '15154359.8744*1454461342', '27441160.2608*1464965095', '47651759.3808*1481186324', '71527533.5884*1468793128', '50095360.1778*1478068955', '56119346.2328*1479438751', '73518504.6612*1472170929', '86443565.3649*1455071711', '98782315.5255*1485720392', '19603138.5699*1453842931', '41258898.6421*1454396814', '9478423.81687*1483565251', '55972032.7243*1456286971', '39865359.604*1474242440', '29366738.2145*1459224722', '3155980.29433*1485918475', '54485350.6776*1461710950', '17580442.2343*1474132289', '52410123.2217*1451947798', '17203993.6778*1474110948', '80282420.9318*1467761541', '1986764.2586*1454024678', '91280269.699*1453629550', '18807106.98*1484517730', '39334155.3925*1464341059', '2175375.85967*1480427548', '64302244.2014*1467359365', '45562949.8296*1478133262', '65476610.2047*1459028555', '2663318.8783*1476955551', '1443544.51268*1458033159', '78793171.1747*1485088010', '23000816.569*1451798070', '36960021.5382*1469315738', '25381865.0138*1456238833', '44117589.1743*1478467686', '6623500.15734*1478665076', '14126757.9704*1469119047', '70791166.1963*1461274066', '5583178.48599*1465251598', '15569060.8824*1482635936', '58370909.086*1460617269', '23668089.9853*1469055418', '38135695.3698*1457655373', '24210791.4819*1454140557', '97928233.4431*1461385606', '34552736.772*1481368397', '41708702.089*1465130277', '79358650.1937*1460940707', '30248273.4369*1479149677', '95049768.7196*1481860350', '60692608.012*1468098533', '21485413.5882*1479902363', '8839083.17283*1466147348', '97433409.9162*1466617345', '62678770.1165*1483832780', '92743.4201355*1465941286', '36546807.7898*1466960468', '15768122.1589*1483450158', '72458990.8205*1487270433', '534105.792209*1467480272', '40677169.8441*1485391258', '1444539.00198*1475010101', '32928203.3214*1452058953', '17985803.9343*1484527166', '47254010.4028*1483394208', '64069264.0876*1458682987', '28628448.2419*1455162773', '28947002.1079*1465992022', '71202857.8816*1468645484', '19408237.3982*1464504647', '19108004.0404*1475977995', '29421380.1922*1481723140', '72141163.8032*1463788343', '63528386.6263*1461459138', '14273868.7036*1487267005', '1000605.86721*1485050184', '79241878.8179*1467699949', '26582809.8083*1478514041', '61850318.4345*1473999329', '59490270.419*1462424576', '1226001.9517*1485149042', '77218415.7601*1486168809', '89266955.1453*1486785116', '21625225.9297*1465985955', '26483005.6879*1455479540', '11212693.391*1472617449', '62767915.2448*1461644791', '65793273.168*1480201150', '80082220.0378*1461581835', '81098867.0262*1455285711', '9837430.1843*1477778155', '59562356.0049*1457946056', '14732481.8657*1458176678', '6901151.34368*1452164189', '68064373.5855*1487044041', '15664442.1593*1482766112', '46826589.7538*1451754280', '73911423.6529*1483234339', '10069573.1546*1472152533', '12059728.1602*1463998015', '67301733.1093*1474172102', '75098539.5328*1471484227', '51441037.7974*1464916946', '66448678.3431*1471257654', '67017067.0805*1460866578', '61284558.6587*1463600634', '45432673.69*1479630183', '24762387.373*1465086517', '82726975.7148*1486936926', '43261792.2839*1461612083', '94019194.8247*1484555386', '70629042.7077*1467918347', '84450180.6989*1463374327', '99542371.8847*1458479561', '36819487.0924*1480053656', '94558557.3328*1485389662', '41342521.5562*1469278271', '45055280.5118*1481424220', '17354925.3371*1473329581', '76269041.6458*1463826925', '88129003.3615*1454352689', '15010904.4169*1479292033', '63176295.9593*1468054118', '56433045.5073*1472317457', '57163019.3756*1464169904', '64226087.099*1453876393', '63733123.1253*1484572816', '97521653.2833*1479130129', '94843728.474*1478126286', '30151414.6775*1476204977', '4175457.45018*1472831721', '41010865.3548*1474082914', '46961417.8475*1451940407', '86747086.1576*1469423465', '55856353.5935*1484847491', '64838995.7477*1487933267', '14184199.6039*1483004273', '65793202.5431*1457377052', '1688394.09243*1482402510', '90883296.8414*1461017459', '14885184.6917*1468939217', '85391117.4476*1454627808', '15664994.2789*1459896428', '76572096.1106*1461368605', '16002873.4728*1484644105', '80282420.9318*1467761541', '16867283.5578*1468877045', '99395045.584*1482354538', '66640054.437*1467001731', '48458181.9545*1481595789', '34335599.7819*1466181976', '99268742.1291*1478537498', '51904494.5525*1473247816', '18724136.2624*1469281735', '14424902.8046*1488135261', '50413484.5558*1475367226', '61418250.8189*1484800332', '74819980.6624*1467215637', '3941477.33979*1456880895', '8291298.76436*1458415505', '92704452.2724*1463225983', '3385550.52216*1463910293', '72458990.8205*1487270433', '50737306.0299*1468423853', '65357267.5075*1482240083', '27576302.3572*1465304436', '35573237.9688*1468249418', '55199439.5463*1461607890', '90058332.2107*1484636963', '12288023.0052*1476829288', '35900548.8139*1456364297', '20840478.8819*1471944919', '51171459.7152*1462177576', '12373765.9623*1487071312', '38318162.112*1472682691', '57439138.655*1475397378', '77567345.7389*1473081961', '11423105.4713*1480391564', '10264430.0751*1452231411', '33828456.3348*1459775304', '44707418.0022*1452430786', '12534271.7442*1471247811', '33190357.0759*1474593366', '48245892.6301*1481091632', '69239412.2835*1469881217', '96640820.2503*1460152703', '7400891.56672*1452659005', '33469538.6211*1469319129', '10551715.5607*1482199372', '3074419.68074*1464502578', '82852857.9934*1475075470', '63739560.3942*1487982842', '74998692.4327*1477826210', '59283553.5996*1473171133', '23386176.6557*1486322072', '57787657.6766*1464455530', '22115226.4806*1453949206', '37282593.6662*1477173709', '13921316.6147*1486739613', '17985803.9343*1484527166', '95635964.4222*1463751606', '64857818.6389*1476584144', '85702992.2338*1471424875', '50697897.8032*1483906388', '2931432.40423*1452909057', '74567173.6904*1484052112', '71339265.3266*1484380110', '36862469.04*1478912655', '46499993.0024*1479731545', '96198663.4719*1481182888', '34361292.9317*1453035181', '94168816.9773*1472151466', '81617115.3214*1483269589', '98091838.2836*1485812245', '60465254.2751*1476946268', '5691266.35825*1458001110', '64025837.7383*1475367357', '93021003.8795*1482472677', '48133542.0706*1478234591', '54011565.8609*1480398681', '53215023.7416*1474518718', '62767915.2448*1461644791', '67036967.7365*1480468252', '17820930.5475*1453501265', '28108252.5499*1473652085', '90606439.0076*1486065155', '47140604.8272*1484936478', '85040873.234*1458154904', '72664406.746*1452958646', '18046255.532*1472556968', '24771484.6917*1466791321', '68084296.3291*1465217609', '72331641.6076*1481980143', '46085467.3081*1463189920', '33721593.9128*1457637849', '71491549.4371*1485064244', '80716110.3639*1488191778', '5565961.01846*1460393734', '65727632.5415*1477203103', '18269120.2916*1466775364', '56047863.3662*1481231970', '87115371.8543*1481009427', '45490671.1981*1483090011', '38889044.8759*1463356807', '86907911.6148*1457699479', '50699992.8023*1482306040', '54586817.8666*1488193847', '17267583.583*1468156331', '71912323.4649*1483601565', '32120529.509*1458849935', '2302809.45695*1459409036', '60723580.0058*1478731971', '95770435.1249*1467964170', '11416202.7251*1454438746', '78798237.1248*1474889695', '28627076.3896*1478407078', '93788726.5848*1467393368', '36132849.1228*1478446966', '87524010.0535*1474972546', '61006392.5326*1463581062', '90531637.9897*1488279177', '42485501.6019*1482362326', '8662738.12084*1468913113', '79811341.7729*1468579894', '12036981.4661*1482676569', '55598975.3846*1454525754', '48396512.5323*1482834564', '73082650.7444*1462955185', '52812757.9573*1486182940', '92562119.8352*1456607527', '39112700.965*1451910519', '16391776.9224*1452567830', '36164458.3101*1461268143', '57901137.8282*1463607766', '43476839.6766*1478373191', '76172426.7916*1459049036', '56258407.141*1477060798', '27095693.0947*1454508604', '94707675.0055*1462369262', '88964471.3238*1475790764', '30588924.2478*1486195088', '78822674.7961*1460941272', '57718414.0497*1456373893', '94574214.4774*1471930726', '3687183.63727*1477778116', '11394595.4606*1456017027', '2269576.82579*1460074150', '89056229.6533*1465390354', '34895006.845*1456856988', '87642959.151*1485119692', '59770889.1069*1457956584', '97195950.2381*1453447625', '30155072.5774*1465570252', '40570600.1454*1463668413', '34763928.4977*1483561078', '30466421.9524*1471629386', '13527801.3077*1454636170', '80170603.0386*1462897353', '13718450.5171*1465454270', '78671451.4181*1455241006', '83099164.5047*1453402630', '71053959.2587*1475463748', '3171407.0563*1469063080', '57625904.686*1465491845', '65623468.3633*1471939629', '93645816.7016*1471855092', '43239505.9806*1454913596', '17032473.7753*1482365664', '55187455.5131*1476847329', '85005953.0879*1477186673', '74739544.5503*1464379353', '65385550.1021*1452703682', '49903855.0305*1474385047', '16558020.9947*1476600536', '28641948.7364*1486570213', '10981706.4754*1478200158', '46759144.676*1452240664', '76108655.9612*1484592819', '96273295.687*1453016507', '27972341.9752*1480961108', '12534271.7442*1471247811', '45022776.0253*1467007714', '99331192.5664*1476395285', '73992442.1539*1463196663', '97422200.5022*1471410637', '24553644.8587*1471265968', '3352495.63096*1476698091', '29599328.5142*1488213606', '15846205.9087*1473294012', '64947445.9379*1460681485', '18539868.3383*1463067300', '78083246.0698*1452680665', '2613061.70197*1468433077', '27962380.0423*1473604704', '16286229.6959*1474173392', '17821354.6742*1463368105', '55691291.8351*1457330292', '4692190.73707*1468473766', '14184199.6039*1483004273', '1631117.97626*1462782607', '48849534.3485*1460044058', '45226295.7648*1470651624', '77884953.1692*1456203896', '12619499.3035*1464806469', '10842272.3094*1466012588', '14841020.5946*1476950986', '23028977.5582*1469322219', '50042697.1457*1472645803', '21181530.6769*1478762136', '8710014.39411*1488162434', '40888923.3976*1485237508', '29247823.1372*1486363750', '88186919.9936*1455730097', '52445995.305*1464344767', '15804655.1536*1479012195', '87875910.3652*1478664775', '21508068.4896*1459415736', '89283436.2502*1452794792', '30753862.0358*1457985996', '24217077.8972*1456592331', '77928651.3806*1455697338', '31465755.1533*1473030221', '59643775.6253*1485042915', '29935408.146*1487540505', '45940806.8599*1454059035', '39542344.274*1479865440', '43481686.4229*1453505090', '59523495.7027*1461311365', '10347084.0167*1486687096', '65986134.8072*1483999357', '9507271.79389*1483773577', '4070457.60315*1454456460', '33529147.2411*1474906926', '88169839.3048*1456864957', '26166206.9699*1462973305', '63186118.2202*1460774926', '74035816.1059*1465798762', '1192181.43732*1457912143', '7254369.25319*1477212101', '47444159.8069*1472253643', '54985955.0573*1473153365', '96794802.3399*1456500081', '40729823.6606*1484288549', '99190618.7607*1476387894', '62240158.6059*1455784536', '64261541.0627*1463801183', '21077018.8719*1462372141', '31244161.0159*1487734413', '85917276.0941*1455228531', '10744421.1044*1479813586', '38958965.5612*1466339030', '70629042.7077*1467918347', '7036212.24089*1469008550', '37407172.3211*1484662929', '92704452.2724*1463225983', '81782514.061*1453902404', '94199560.4246*1474173098', '99024626.7796*1464777994', '20134472.6088*1456356935', '58370909.086*1460617269', '3086041.15509*1477866483', '91279912.8515*1477120469', '20640681.6625*1479185862', '29479603.6878*1483149902', '82188365.9507*1454443781', '58491572.7225*1462469031', '4615222.32957*1452707474', '6978984.7451*1483199984', '84862839.2449*1476387933', '88360907.139*1479642513', '68384710.9616*1461630737', '42330561.3765*1456846869', '3000873.60899*1484948272', '36460836.3181*1486952358', '51441037.7974*1464916946', '85401321.782*1457689257', '45751341.4956*1481536478', '32798096.704*1451762811', '82274925.7994*1471418985', '90271790.4293*1482891028', '88084863.471*1459259390', '71314646.5791*1464015844', '82836994.4635*1454216853', '71391986.2176*1472439515', '45221042.7788*1478050148', '33721593.9128*1457637849', '90054515.1472*1482724671', '55217451.9946*1480493557', '44467313.616*1461843055', '92898032.7655*1472472376', '85918307.5857*1483923040', '85944347.8801*1473353578', '44210684.1807*1485801525', '90400302.127*1483036668', '57691183.262*1456408037', '24857286.7119*1452848232', '52798268.2298*1458149525', '14757737.8339*1478132573', '11442403.5436*1474512902', '70350790.3184*1462520751', '48416096.9621*1476540977', '46551084.0706*1484860932', '81062454.9362*1457022868', '91279912.8515*1477120469', '49083067.2741*1461504281', '15629051.4943*1457786518', '2030180.53075*1454160043', '55414602.3953*1451798752', '12829156.1013*1470009401', '23249200.1387*1486086317', '36314890.1831*1476248684', '29999373.9118*1481019725', '56253461.5824*1461227126', '72410132.257*1487527668', '65067114.0806*1454252796', '48504285.756*1461114030', '33393930.4994*1463540745', '48815108.0092*1455650525', '67409807.4086*1477291496', '61824794.8308*1486634234', '79215547.8118*1479888076', '59876906.05*1462810156', '57837134.6944*1476603362', '49850040.3359*1486550089', '12921423.6753*1477460886', '73302732.4956*1456472516', '56369708.2825*1481804226', '31152772.9596*1456471085', '28914033.6092*1458218979', '75955689.0589*1482723552', '18054589.0476*1485877274', '18292141.0544*1482284703', '94223287.5714*1464837393', '57936177.0944*1462723885', '87642959.151*1485119692', '9788797.74949*1460132211', '57093307.838*1475567758', '70791166.1963*1461274066', '74489213.6528*1458815844', '94607057.4181*1461096926', '45527212.31*1451700694', '46090094.8802*1469734319', '35253585.4196*1453023061', '92974897.092*1454965703', '9594285.1693*1485124965', '63894147.3275*1471464304', '16691870.7241*1465972388', '6804121.9526*1457755035', '67257970.0565*1476986933', '2961691.0656*1456104541', '19480548.4221*1461002487', '81296949.8766*1476931897', '66581581.7165*1459130015', '64882709.7277*1467044431', '55956533.1531*1457944584', '92389581.6*1483866387', '27712920.2771*1456782576', '41026572.9367*1457404416', '69476036.1621*1465468862', '91445677.6474*1458406724', '90727898.0671*1470821232', '52822301.0393*1464548406', '78156094.436*1474762838', '80455421.9752*1451955575', '27893691.944*1483954045', '41281744.3193*1476472938', '10747391.0015*1482153232', '83910850.8156*1477818115', '37908152.7008*1479832836', '29723223.4378*1483097630', '88633521.3815*1479388024', '2201035.82649*1482101449', '19469147.5531*1484689768', '37917790.3849*1462519758', '4093491.99388*1463351602', '87292777.4928*1478578796', '99500764.1088*1462502256', '71825719.2712*1479671230', '24210791.4819*1454140557', '63200380.4427*1475612679', '57736160.4888*1475054438', '72303129.813*1479224683', '68572378.3107*1472299272', '67528360.844*1486036523', '51331052.4451*1475248005', '28049972.3225*1481957841', '57156912.7949*1457375605', '71046454.888*1458611875', '95276542.3143*1463268521', '83190434.4603*1472155065', '53094503.8124*1482572494', '27879079.5234*1458425740', '47605266.3738*1481388056', '45943775.2408*1462162002', '55519632.7846*1473024542', '87075177.8954*1483992010', '52724585.3188*1479736899', '47723484.9584*1456789816', '48667889.6678*1459018458', '31236906.3788*1452546192', '99703424.8624*1475845026', '93491214.6043*1475706825', '93738298.5595*1486908153', '40873507.36*1453652357', '11198802.4487*1465086932', '77545515.5898*1462402759', '43808728.7029*1453996241', '6279255.30081*1451937986', '21337544.0166*1484079692', '87683933.1277*1470817648', '82733154.6612*1487100107', '14874776.3605*1473743159', '56100054.5056*1463135868', '23449541.7261*1475304347', '13478270.522*1474643349', '8545538.96743*1483313037', '37343808.55*1467480441', '91216861.7876*1483048387', '8821109.32256*1454450413', '93189522.8806*1473892509', '36121140.2157*1472868286', '53901021.5472*1453770702', '37282593.6662*1477173709', '10141434.791*1484720867', '52777609.5281*1477504480', '66037380.6218*1457755659', '76816922.9164*1463315638', '59431928.6084*1467049089', '35235040.1125*1470767315', '54125521.3215*1473532595', '2058248.95929*1482199656', '3675592.70311*1473211379', '68926991.0678*1463929366', '99526321.191*1460310156', '26522432.5548*1484050924', '95505604.4208*1485940642', '31236906.3788*1452546192', '30969399.6606*1480976906', '9741588.04411*1460037450', '12160605.0783*1473485460', '92000111.3396*1487511155', '47467807.6579*1459409801', '34558737.5678*1483951020', '29373713.7856*1464442076', '1680853.53017*1465341119', '30570962.6417*1465011945', '70176561.421*1484030988', '10510543.0401*1458983580', '90173037.8435*1487257164', '15010904.4169*1479292033', '30753862.0358*1457985996', '13973497.4551*1486302472', '40715389.8406*1482444172', '43882212.8446*1463649108', '4692898.36183*1487199223', '99487080.8258*1452538235', '74228329.6669*1472010232', '25028337.0755*1453424996', '75055722.42*1468299180', '28590410.2716*1473421779', '73396515.4299*1464283684', '10331779.1042*1473809325', '80601446.1746*1469408240', '58535248.2076*1472016535', '49093985.6846*1476429624', '61703127.6047*1458243835', '75128838.0702*1470156867', '42018905.8879*1486086586', '80350109.055*1480123338', '54416481.5645*1487519801', '51302283.0361*1477429021', '67392083.5001*1467573961', '6431538.45225*1462917917', '76627857.3375*1485486929', '18507184.2596*1470889240', '51956251.0768*1481374365', '24851583.4275*1469994470', '34677734.8853*1473493246', '51092197.801*1470916768', '12803566.3539*1485109216', '79140630.9011*1453064064', '92966881.5101*1471140707', '70168359.6239*1468156206', '27739061.8828*1453424792', '71035139.2858*1484208985', '14728944.5499*1486695404', '34585106.8602*1478195272', '65400354.8033*1462106212', '82733154.6612*1487100107', '29803252.2845*1464417250', '45440223.6305*1484711284', '94240780.8973*1468662447', '17623061.286*1453298606', '18015430.6884*1463685542', '94542951.6769*1452134995', '23951835.1502*1474323603', '76116482.6901*1470868307', '18379048.1237*1454646644', '90027474.924*1488257607', '42485501.6019*1482362326', '33168166.0976*1477346578', '94707675.0055*1462369262', '30678847.7341*1487731859', '71466982.0215*1461679226', '19705330.7537*1478685848', '78265808.6622*1456121138', '97410920.0956*1452694779', '5680772.18305*1474147915', '47254010.4028*1483394208', '79986835.3651*1466696903', '31413528.8884*1460811252', '64228101.5358*1461490009', '36011389.2712*1482791977', '93508970.5042*1485095913', '66703685.1986*1476499562', '22732134.5928*1468325414', '63096482.4632*1478970880', '57082278.9516*1480984052', '2685114.05225*1478839517', '63484867.3825*1468793021', '68039207.0749*1476575878', '60674648.8242*1468437640', '46919473.0458*1482183284', '61363836.3338*1483990167', '30151414.6775*1476204977', '66374382.2155*1484459834', '93920041.387*1470102373', '38620668.6676*1467028530', '83081258.0301*1469724080', '37571068.4822*1471659335', '56549330.379*1487405204', '69227744.887*1478178972', '18916221.4766*1485201743', '42848994.4349*1460590162', '79810098.7262*1452208898', '93302385.3159*1458518143', '16586408.8625*1467226839', '89281716.5939*1459419476', '98091955.3108*1468690977', '27392631.0402*1459392886', '47280032.8174*1484887029', '7498315.74911*1470921242', '83624848.5398*1487981619', '61656708.0604*1458777812', '2072196.97026*1453908840', '78786426.6537*1475270215', '51616772.7621*1481992312', '72454369.7464*1479298660', '17645787.7559*1483551180', '82956723.0197*1484356660', '49224369.9195*1472967024', '75006557.4132*1480114729', '2010706.9469*1471926279', '1147887.89183*1456867979', '36874535.7069*1465637699', '68569923.0929*1457800140', '21605356.4997*1488190955', '47154381.092*1470973030', '2353910.42458*1484595730', '70419964.8362*1481258310', '92142121.8608*1465763961', '90779874.4561*1481350606', '777715.186992*1486535133', '21220006.5086*1459950627', '86555110.6933*1465400207', '48622646.0157*1453194521', '37161073.8237*1454188631', '14184199.6039*1483004273', '91244967.8521*1461885482', '36117959.9652*1459825417', '2058660.57857*1476350423', '51282230.9242*1462441787', '16071784.0924*1456709554', '91605013.0956*1475451843', '78156094.436*1474762838', '18834324.9389*1475274548', '91812839.1366*1457005981', '44286320.7817*1479515354', '92241958.2835*1479492587', '89731570.8255*1480475199', '62508014.9064*1487885193', '97737292.6009*1477740219', '41455035.9049*1473766683', '47419197.6319*1462780120', '11019816.4793*1456089156', '39793695.6012*1482227442', '28046899.6063*1463887434', '9681629.99743*1460954913', '23028977.5582*1469322219', '58026230.5419*1481305708', '60465254.2751*1476946268', '73018698.7471*1480474495', '30261195.7894*1475835472', '71217869.5399*1482889629', '50529969.0818*1463586311', '37068145.1557*1469796800', '41977829.679*1466027738', '18335393.5258*1478134375', '70689337.0765*1457348733', '78426226.0375*1468693801', '17234625.9156*1454664482', '36104080.8776*1451866109', '1725686.47832*1460910220', '35854947.2823*1459888414', '12389960.1623*1475088315', '37426251.7085*1476939820', '59243299.8328*1472225333', '19304288.3306*1465569561', '3528396.89562*1481654430', '75838728.6047*1460976527', '90382222.0021*1486809600', '61954147.5969*1461119237', '87591395.3574*1472595128', '17802557.7588*1463748030', '92159219.7622*1471102838', '87679165.8031*1456519900', '55341124.2534*1483458864', '75574453.1136*1465029725', '94390298.7961*1476646213', '78389595.7491*1477733736', '50330570.0671*1479331967', '94612614.4268*1468229917', '69908080.4313*1458752337', '26921614.8511*1452543678', '60289312.4523*1482791532', '54704361.2256*1466918112', '34460569.9663*1477010364', '53588777.3588*1487543037', '23021631.9599*1476244453', '31386948.8619*1479273324', '3520191.53514*1481469737', '99490634.5773*1476545182', '16302515.465*1468278403', '51029676.3248*1475314173', '53474514.0208*1474658277', '97365494.0548*1454805916', '95900807.4678*1474434530', '38864055.7227*1460469708', '16779213.64*1474289147', '87669636.3639*1454618861', '41010865.3548*1474082914', '9955415.69918*1466546424', '94920702.8542*1482526329', '45927664.702*1484200087', '26921614.8511*1452543678', '45431512.1271*1459740711', '74676882.8997*1485700498', '14992686.4035*1452303377', '41510397.6832*1458008687', '33447742.2497*1458356486', '8825186.63435*1457307473', '94977164.49*1485665134', '24038004.01*1488012521', '8597946.28633*1458789930', '27150918.1975*1458495391', '18442857.763*1461410549', '62984317.2949*1479063454', '20544171.3957*1463168740', '46117766.82*1479366260', '16256766.3029*1452336993', '6292916.81697*1472090731', '9661250.54606*1477498838', '41390412.8523*1458430291', '65400354.8033*1462106212', '77898136.652*1471425942', '10824152.3404*1467931672', '51691675.8519*1464720462', '88234624.063*1476841879', '20951009.389*1470014427', '99773239.4917*1485573477', '95635964.4222*1463751606', '43381772.3582*1458121995', '60753970.5798*1473807209', '61159430.6954*1475411525', '34397856.2908*1468437660', '96907316.9213*1465227224', '10747391.0015*1482153232', '94390298.7961*1476646213', '49908769.3715*1475106915', '9222261.18832*1457504351', '15203309.5061*1474368068', '54327853.0243*1481459490', '58140367.9057*1473088762', '56599680.1491*1479986056', '36815684.2546*1487568097', '25543241.2869*1454392588', '32339903.7673*1486903320', '87137402.719*1486365265', '48050868.3928*1484143884', '10377555.5128*1469243596', '61294164.446*1470493409', '2269576.82579*1460074150', '85002965.3316*1487140210', '78137993.5306*1468955277', '31148601.3596*1462766811', '56544256.1292*1480615067', '6632586.90067*1475687538', '92562119.8352*1456607527', '61563782.1846*1461641937', '31464636.5785*1471029509', '67198275.8109*1481765992', '61797861.234*1488077237', '11990711.8548*1456093808', '76081228.995*1472389759', '65385550.1021*1452703682', '11194247.2659*1461628512', '19279025.4704*1481590094', '56659081.1969*1456648121', '2722335.44497*1481148755', '80716110.3639*1488191778', '96640820.2503*1460152703', '83080314.3819*1457273482', '97582591.9106*1477664883', '85573609.7516*1475263897', '82864565.1049*1478586864', '74218356.8908*1473370854', '11166488.7571*1464939125', '16378519.2587*1477928033', '71455262.7726*1460081395', '86334194.5963*1485055465', '14893711.9211*1456084286', '28975202.5753*1462889313', '14774808.8032*1486175590', '72380116.593*1459145455', '92644690.4945*1469329592', '76256950.3286*1465977967', '75247896.5889*1476328295', '96053006.3853*1474727582', '68135008.7097*1487016525', '72215067.2093*1475004112', '18879471.9288*1468168585', '29054330.8165*1459222993', '7728105.83811*1457701573', '89230328.1515*1475538902', '79094909.6121*1457818565', '92654363.7947*1469080309', '99391716.1311*1462769221', '9509772.26817*1474702980', '41565322.2779*1458827572', '73160020.0272*1485186803', '26571233.6187*1476464887', '30922528.7125*1468727201', '49594792.1674*1464446202', '48890650.8438*1481726644', '29108468.2739*1480196725', '14992686.4035*1452303377', '51898473.127*1484149008', '58147770.0159*1485039954', '85815226.8604*1488122110', '5680772.18305*1474147915', '87071940.3248*1452501428', '16013125.3418*1461686168', '51800757.3529*1483648659', '71864719.4867*1479782378', '38011066.881*1481342026', '5171055.4672*1474340104', '65885138.6934*1487764370', '78689286.9926*1486059405', '41000392.9105*1455740267', '81187455.0973*1473961379', '54232609.0865*1463746160', '72664406.746*1452958646', '33953738.3672*1454435643', '25627986.6208*1475886884', '72372536.024*1454972021', '57570886.9118*1486796862', '8261608.29676*1478403373', '83822477.489*1453022075', '28969840.2448*1461198205', '86266035.1103*1454292029', '12551707.4399*1461938679', '86023233.4527*1475797134', '94979756.5695*1474737585', '77200258.1449*1454991125', '36460836.3181*1486952358', '50055428.9683*1478290849', '24066606.4278*1456099742', '95659788.9923*1465221086', '78331710.867*1456204236', '66817407.0338*1478356003', '67594653.9894*1457733955', '17168917.8923*1474481563', '74574919.6039*1454628770', '79986835.3651*1466696903', '60282859.7133*1457316671', '52000353.2394*1461117910', '77312986.1631*1461008724', '47441768.8855*1468653925', '14905931.1835*1457424474', '75467238.1779*1462373810', '51104337.2196*1464036484', '90752337.3342*1485450857', '16809843.4887*1458517692', '25201592.178*1487794550', '48269743.5467*1456112756', '85341264.0042*1459050851', '44238075.5234*1465770805', '8081223.68732*1482013737', '59199220.8648*1474339654', '99513084.9405*1463440328', '53094503.8124*1482572494', '51365612.9801*1486949606', '19288657.8033*1457334689', '20700110.4453*1459447294', '51365612.9801*1486949606', '36348177.4638*1473225322', '40206592.2554*1462446261', '30106413.6685*1475584589', '41085034.3379*1451869294', '16132542.4016*1453521824', '75738286.3433*1483027976', '1768743.67772*1487790718', '73211745.9898*1467614920', '69137260.1477*1453563835', '34763251.5899*1483347894', '41620472.3831*1469342830', '40653296.1227*1454902631', '23655862.3096*1487004524', '19897413.5274*1452865700', '60465254.2751*1476946268', '194122.583459*1464443405', '91991271.3732*1452223485', '64483196.6517*1478084307', '75113570.4399*1466819026', '41377675.7309*1463685208', '22438906.2547*1464995913', '18034761.3158*1466083353', '76336468.1858*1485278432', '13336502.2757*1479515537', '25564426.5791*1482210470', '78694147.3544*1478355520', '73762063.7003*1465969458', '13618688.4522*1457634862', '17235171.2329*1466751331', '53138051.6474*1457068593', '87993801.1731*1470725646', '90107127.2197*1462954797', '28151674.435*1471942452', '98649425.4191*1478826553', '46499993.0024*1479731545', '66173583.9285*1487668949', '21128770.7792*1454427713', '4331438.94732*1467907854', '83688658.049*1459576981', '81368477.9763*1458720091', '66640054.437*1467001731', '94253474.7476*1458718615', '23649468.298*1467851147', '227324.233734*1485659102', '26466514.4597*1463498939', '18327800.6744*1460605060', '53295789.7105*1475337637', '84329690.6646*1461244582', '41147148.4542*1464002139', '28460069.6061*1463639500', '57477665.8582*1471512419', '17683745.7991*1478429696', '31754495.1107*1452917783', '4296781.6533*1479601646', '98990679.609*1464038622', '6622128.96755*1456422910', '54771400.5629*1471427864', '8397398.03748*1461889582', '56100054.5056*1463135868', '8307989.52696*1481759630', '64341801.8741*1456450030', '96053006.3853*1474727582', '91795362.1516*1455659233', '53757079.033*1463794107', '75010356.6128*1454194033', '86639422.3846*1464014376', '61243220.8628*1467567992', '22194765.1076*1468644483', '69838980.3025*1480428263', '39551965.9355*1478693180', '78889254.9698*1460893540', '10680793.3201*1455769501', '21988130.0325*1472627836', '41002971.5768*1477304547', '13041647.3499*1481567877', '27766393.9398*1468685011', '43870918.8848*1471973173', '9068029.8295*1480618639', '83543079.4834*1467328374', '11698260.3393*1472355578', '77713768.1184*1465683117', '6312346.96338*1467397899', '62965169.8173*1483987206', '92465017.1857*1454500577', '53221758.2021*1474762014', '63299703.1171*1457182935', '1877729.30533*1467761617', '80826971.5285*1467383462', '55972032.7243*1456286971', '94560418.7958*1453299731', '94861390.0871*1486872565', '68942760.5617*1474300962', '68524598.7634*1475631812', '87369165.0263*1475880721', '33779288.8434*1487078319', '10974004.889*1475501647', '29985815.4102*1456569587', '38179208.1926*1487033737', '67827626.0523*1473624237', '46906061.2759*1457446853', '73258263.6261*1473061673', '42844658.0273*1455608978', '49577536.3846*1461544515', '10881613.4085*1484979399', '84889910.275*1460145199', '12843749.3015*1463195413', '34168501.8901*1479703393', '22329947.4679*1455542569', '77815547.9263*1474347203', '4007140.47498*1453789032', '93302385.3159*1458518143', '45431512.1271*1459740711', '68582655.1022*1465033246', '30419376.7697*1462317529', '17634720.7934*1459188221', '59109718.2142*1484129214', '29327369.5497*1481842323', '67013884.3971*1478295247', '82721877.5709*1461249060', '51719576.8655*1487510444', '61527660.5089*1466886639', '22057539.6282*1459287627', '28250815.9881*1460324653', '171476.966266*1476802829', '28974831.7636*1465649384', '55216182.2172*1464398460', '51888545.3498*1454291835', '86360330.7986*1467121344', '877982.435468*1457885869', '7765505.3232*1454941641', '76042485.4724*1468040505', '60186550.5242*1468259753', '53749509.7631*1459084434', '81034250.0545*1479995668', '18408125.2465*1487550522', '5472602.72016*1463037390', '85021411.7252*1466665566', '85060635.9315*1463600024', '21639672.8951*1459148090', '9710797.99321*1475516454', '66068127.3092*1473948922', '64579019.4272*1471552539', '90216340.8316*1473464940', '47444159.8069*1472253643', '55426674.2597*1462262940', '16392532.3271*1460236160', '51447955.8522*1454981874', '42874568.4289*1461838350', '31130284.5046*1452975707', '75832398.2542*1455092972', '99395070.6665*1470642767', '74407773.1226*1483051161', '28078648.5592*1482572220', '59490270.419*1462424576', '30191370.0101*1454703257', '37997855.1149*1482590307', '18316150.1388*1487812677', '5809168.51038*1458110481', '67514783.9843*1483261347', '13916575.0668*1479990617', '12583635.4847*1483814052', '9216187.23274*1468906460', '46221872.9585*1459756566', '58031798.8814*1461088810', '41825308.1461*1461121135', '85647699.6424*1485170291', '11787444.6335*1478065412', '69476036.1621*1465468862', '55789174.6814*1476915300', '95508133.4689*1480455767', '23559715.8616*1455169524', '9651823.53015*1483193298', '86071189.8079*1463660299', '80689447.3735*1485790278', '75873280.8573*1461864636', '80282420.9318*1467761541', '74361251.3881*1453186277', '53117332.6542*1479258101', '69543795.2998*1464783149', '74778298.4072*1479879394', '2685114.05225*1478839517', '78470628.3202*1454643414', '46259973.7536*1455961709', '17105022.142*1467552611', '62254842.9855*1459234811', '99844263.6196*1481869012', '73315132.4507*1459303488', '24077158.1806*1468943966', '18756018.636*1463948617', '15674397.8354*1479644469', '28767117.6448*1473747696', '79810098.7262*1452208898', '18883728.2745*1453169901', '89708018.435*1463575710', '69551098.8739*1470697032', '76105027.7411*1476568422', '33953738.3672*1454435643', '13999232.3014*1459795085', '71498129.6061*1465141344', '99892567.4292*1473342484', '87093922.9067*1474944730', '22438906.2547*1464995913', '66229180.3406*1465529263', '77082947.4708*1451684342', '55434246.5361*1469679775', '40493538.6919*1476804597', '66530223.1149*1481876143', '41143799.5512*1467247719', '97712113.4449*1453165799', '42186218.298*1473912584', '9613442.24488*1458567682', '86081813.7281*1463631463', '2058248.95929*1482199656', '29427612.288*1471238864', '98481811.1078*1479957041', '44257492.8474*1484241929', '79300219.2531*1487385662', '61777895.9127*1451965797', '69652983.7107*1476172765', '36394997.4882*1460040844', '64550982.7546*1455765648', '79811341.7729*1468579894', '1354033.79829*1488256441', '65337071.9216*1471730203', '36815473.1769*1474932788', '4039376.0519*1462531839', '15892587.9495*1462699158', '15846205.9087*1473294012', '54068085.4452*1468878988', '91484381.5152*1476574272', '849632.118531*1465333929', '95806711.1667*1462573393', '52788985.1233*1453865218', '57560407.8185*1468721554', '72215971.7722*1479533685', '7190943.84467*1484237254', '58192750.3113*1477079136', '40198323.2872*1478307038', '41849815.6571*1465057205', '54076940.6478*1487755781', '67594653.9894*1457733955', '39172964.5789*1457167098', '17968345.139*1464014039', '23169635.0462*1468464524', '91139.063833*1472180522', '79804341.6162*1477758772', '85445870.8642*1462381923', '78352672.1133*1477441153', '29599328.5142*1488213606', '49000325.5007*1473055115', '5942656.35255*1465345653', '73008687.4028*1475324712', '6978984.7451*1483199984', '65712613.8575*1457956050', '90234988.6156*1484109302', '42465654.1015*1468422952', '10339663.3649*1486072473', '42537277.9517*1487782094', '7196357.05529*1479660451', '40698483.3943*1466916711', '99253365.6946*1453206175', '13646528.7087*1478459378', '66804161.4602*1463289862', '63375366.8733*1469950543', '92644690.4945*1469329592', '99513084.9405*1463440328', '8672495.09618*1465845888', '76663121.2373*1483822821', '9746523.24164*1476595150', '895927.17305*1476677847', '53964139.4837*1462955656', '84696614.1411*1459397440', '77063.5814128*1482546993', '47562161.938*1455148066', '41356862.6315*1482072074', '15678486.1559*1456189018', '58738648.8712*1483159493', '14652426.0699*1472696548', '7156628.16332*1481923550', '25731776.4397*1477060238', '21643583.7805*1483621568', '80562399.7877*1473337341', '24732750.2381*1470702326', '70584092.8439*1453466660', '66616063.6379*1475640018', '91467959.0551*1483499781', '78019189.7762*1454904515', '37478903.2618*1474158991', '10842717.7038*1465946051', '61695486.9391*1467296318', '63365608.0821*1452190462', '75420503.6414*1472929429', '97267149.853*1479686359', '70613864.8925*1475810753', '64025837.7383*1475367357', '7498315.74911*1470921242', '71973327.8967*1469394641', '85919135.9254*1467728921', '44419329.9684*1455786136', '80820934.1406*1484840397', '44286320.7817*1479515354', '76971946.4627*1458333715', '65750142.7182*1464053021', '61409531.1143*1465233237', '86015072.7184*1463377894', '8467771.28221*1483827184', '40621997.7915*1471215929', '73258263.6261*1473061673', '84600219.5862*1476657577', '96415922.9699*1467382183', '407900.998696*1484193383', '51446024.3658*1476092722', '21489121.8128*1454336252', '21063226.8916*1453700935', '21883485.6235*1462829855', '9480634.86075*1467088075', '75917416.9636*1479300991', '82019757.4685*1472627582', '76256523.0953*1477305295', '12031040.5363*1473371206', '32044486.8496*1458847215', '27220917.1241*1464336822', '21429060.7377*1463779071', '8825186.63435*1457307473', '4093491.99388*1463351602', '64443372.59*1469157612', '27366008.2974*1453379610', '38040471.2135*1480885334', '75016945.6234*1455937730', '73082650.7444*1462955185', '35110510.7497*1471579623', '29942766.2363*1452180614', '6254143.95004*1483593350', '80601446.1746*1469408240', '38455259.0333*1480935704', '12134965.3315*1481057052', '86885855.6923*1463623343', '16695764.3789*1477844230', '94816475.5767*1464021651', '70324333.9857*1460140610', '93189522.8806*1473892509', '56604279.4767*1480110594', '31049642.4266*1452763603', '40729823.6606*1484288549', '92702869.4731*1463493959', '12358436.1745*1479323131', '93831765.4973*1465449790', '3396822.16373*1462995356', '45315342.3434*1465225661', '96368918.3938*1472037677', '13242041.5526*1452747674', '26449333.1939*1458460270', '12358436.1745*1479323131', '63200380.4427*1475612679', '84069548.3583*1488190209', '33472133.973*1481846439', '23998559.3133*1488029395', '31041731.9188*1465757742', '13078796.6631*1471400650', '67262852.2701*1477122241', '32766766.7107*1483152282', '29381568.2339*1483510199', '34911980.9242*1484483746', '13481188.9005*1477128329', '17420058.5706*1463712632', '76403846.0192*1480098400', '13575620.5903*1466070618', '72150057.2795*1467501319', '92121151.3221*1464506829', '39539727.6073*1464613402', '54211968.1339*1471919757', '41566851.8451*1464481357', '73423657.2545*1456231595', '88872444.7523*1454316695', '32749612.2357*1483114299', '80792423.4808*1463210302', '49064491.1489*1458663545', '77210268.0238*1474694573', '11423105.4713*1480391564', '45630860.8536*1460883378', '8397398.03748*1461889582', '88712934.2329*1468251621', '63501545.5714*1477897061', '82738683.3881*1465914736', '84450180.6989*1463374327', '33670426.0594*1466003047', '2376210.10975*1459276227', '35499329.3791*1470557720', '95939692.2017*1472012625', '92541963.0889*1456685834', '52722378.2983*1479534600', '76161258.7397*1478136108', '75101332.4867*1468994757', '38761187.8698*1455801023', '69870761.7887*1461426005', '19223436.661*1459069264', '56437035.4498*1487364725', '17419700.005*1455082313', '68743640.5353*1451994509', '68771495.546*1472142361', '67969979.4002*1482270067', '62758249.6422*1485273781', '5725868.99974*1477404999', '3396822.16373*1462995356', '28630862.3486*1460266258', '13316320.3794*1482067052', '55341124.2534*1483458864', '1337976.98396*1478253589', '16822048.391*1465309456', '78352672.1133*1477441153', '70986965.3377*1488153831', '82272424.7727*1476007562', '39352179.5501*1462702259', '44476144.8519*1480071237', '6042259.57904*1460139735', '84505307.076*1487502960', '34484812.4058*1453789740', '38182675.1104*1475355050', '79277726.9696*1470185855', '76464874.5687*1472411160', '6213503.94856*1464606992', '83323021.521*1461462576', '38863760.6151*1483484926', '15752680.3712*1480259083', '27003688.1106*1464844011', '42330561.3765*1456846869', '72303129.813*1479224683', '24589675.1164*1461418648', '86089201.37*1478892578', '55410894.9609*1455417919', '55737290.8833*1486413684', '89689176.0156*1485074093', '39046325.2514*1455168260', '36498784.9902*1468416721', '33721593.9128*1457637849', '12769278.8593*1479536435', '62863164.9271*1458138671', '16702222.2221*1458938574', '37381339.1162*1456709083', '31203545.5375*1482558086', '75750243.5551*1478573972', '14660155.1765*1467363654', '32144374.3056*1476396602', '91903832.2202*1456673272', '69899500.7296*1465672569', '16641561.5784*1484707058', '1685210.06967*1472820251', '36266845.3988*1475319816', '61662420.6216*1486077881', '7449249.30625*1462583101', '71935105.6467*1477442797', '85391117.4476*1454627808', '13496649.2144*1452336906', '22708413.8489*1459873358', '16462240.9299*1462816163', '84961523.7202*1474192493', '36394997.4882*1460040844', '82046552.4568*1462439363', '28975202.5753*1462889313', '42371230.9479*1465407137', '84914285.0487*1478281211', '25858187.6948*1462840610', '46090094.8802*1469734319', '85518476.8018*1480284809', '73370270.7743*1475837084', '51032358.0325*1472397477', '32980582.9208*1470345793', '53222166.4527*1458019989', '91881321.0565*1453343936', '34806256.5111*1457843191', '4300908.69929*1472011052', '82034001.2513*1457359893', '97210174.5086*1487483572', '91442494.7443*1477024568', '67204395.5984*1483750216', '27576302.3572*1465304436', '51104337.2196*1464036484', '81766789.7883*1457214410', '40449205.5686*1479560259', '86510728.6461*1487413184', '46615416.53*1482888299', '99856112.718*1454354848', '22231906.4346*1488262567', '97712113.4449*1453165799', '85114549.7527*1458507479', '15071237.7897*1453425660', '70184489.3613*1472665609', '21393807.4428*1484199365', '20862708.5877*1457853068', '12059728.1602*1463998015', '54953517.1768*1464789777', '34809104.199*1474660904', '9699288.3936*1466454739', '80050119.053*1458308175', '76326657.9892*1461698419', '12434043.8962*1472520772', '6311117.19992*1456614442', '29042272.2222*1469559259', '78870914.9271*1458297813', '16879255.024*1463063248', '72814297.9739*1454479886', '73837027.3717*1470060954', '59212682.4461*1467329584', '71559569.9658*1460121610', '30491901.7747*1451948169', '34962344.9501*1486548023', '75074364.0592*1487107995', '76453396.9596*1470598239', '81700202.4377*1482335200', '51104337.2196*1464036484', '54364780.5195*1458165617', '49160776.5483*1470576284', '54196704.5415*1482523656', '35263198.1774*1482245788', '19187100.6191*1477224971', '32518432.5681*1481494075', '98553876.446*1481821682', '40421101.6697*1484995791', '92893149.7904*1478111425', '87605402.7772*1477757845', '86046448.9323*1478586617', '77377732.9492*1475268455', '79305178.2179*1476680831', '22951713.6142*1477279945', '51400154.1044*1456308665', '29590099.7493*1460389559', '34986953.387*1465363844', '33973142.4102*1486469301', '79998318.2209*1458546762', '2541377.88853*1468787750', '57837637.6085*1479793265', '31855717.1016*1463127617', '52812757.9573*1486182940', '64518006.0764*1486978079', '32798096.704*1451762811', '73013170.2075*1459307565', '7696835.94634*1467629328', '87137402.719*1486365265', '15892587.9495*1462699158', '42791738.6364*1483471781', '62343130.7804*1475438878', '30860489.3042*1465384724', '6587420.78618*1462855376', '19872889.7784*1469364936', '58674870.1508*1461917734', '50380756.1121*1483276899', '47712462.6292*1487869596', '65505273.7142*1459417799', '57691183.262*1456408037', '92527905.2242*1462767474', '22716401.5901*1470152106', '29236700.7742*1460087243', '22273220.8399*1453460459', '95597765.7248*1465886531', '39262254.3685*1475590579', '63490964.0232*1468251028', '194122.583459*1464443405', '38812324.9272*1479370894', '4039376.0519*1462531839', '24088332.8162*1458521031', '48423168.6373*1456794960', '6342162.31815*1483213211', '82922713.2862*1473884505', '1348992.32678*1460636111', '71602288.2544*1458434775', '76099442.0605*1455154618', '71202857.8816*1468645484', '35436369.518*1458333930', '95381983.7623*1471398462', '39686208.0461*1463687230', '38788034.3778*1465659914', '62473459.5006*1473571235', '19728165.0971*1487450336', '9841128.97991*1461356140', '21722405.4587*1479104485', '80479329.9468*1461128473', '33376930.6735*1453290881', '44419329.9684*1455786136', '1894996.07945*1467889982', '2376904.65425*1454097399', '31200976.239*1459890269', '67777155.5766*1452334132', '56612819.2326*1472493755', '30630452.5144*1454360848', '2007705.49825*1484754101', '24487847.3551*1480717110', '96843234.8*1477131992', '50433170.1292*1462016512', '37855079.9505*1462173032', '47123093.1974*1461176895', '72367334.7911*1457863946', '73829955.2598*1453246994', '91915520.6016*1464969486', '2838053.87011*1479999731', '3914180.69239*1458477070', '48145173.6396*1467279034', '76656763.9218*1466258497', '54598460.5975*1474819687', '65383064.9538*1454529324', '26764594.8918*1470245638', '45726139.0133*1484777702', '60546724.5242*1485248401', '34179755.7365*1481440909', '78446793.1618*1451954918', '23840078.3162*1452438952', '70428203.5928*1461896003', '52637325.0209*1459250533', '5530632.80542*1469463928', '12863735.3313*1481622876', '45639913.0962*1468464454', '50725671.0144*1474695758', '68120902.5792*1458383038', '61255858.2041*1471087427', '43937656.2289*1454594623', '15854975.7805*1463434328', '33503894.6939*1487201308', '13549101.0737*1470066219', '52561557.2031*1484736473', '460014.061704*1468928939', '40198323.2872*1478307038', '32548294.6213*1472902690', '85315533.1976*1466639260', '23689572.1855*1479514944', '20090169.4441*1475846389', '34530025.4485*1472743619', '58026230.5419*1481305708', '60469294.1469*1456508222', '42700134.9503*1473276409', '75903063.6773*1484691231', '44919282.8068*1471117663', '78484063.794*1481727623', '75139442.7772*1481224573', '50095360.1778*1478068955', '72251889.8445*1483421217', '7121351.04749*1477626085', '35094202.9036*1459197191', '45420446.8451*1483976663', '26307204.844*1458570263', '41831087.1826*1478611757', '77355442.1669*1454808224', '24940638.7544*1482556846', '95624056.984*1470717408', '34656389.3142*1470530088', '93722865.0681*1462893269', '63504392.7204*1476034988', '12681949.6965*1463642550', '52133195.019*1472405189', '14535575.3601*1482695076', '74156573.635*1476254037', '11138241.85*1471669561', '94707675.0055*1462369262', '68862745.9276*1484327382', '69674411.89*1476997890', '87381158.0062*1456581054', '22190794.4558*1471635422', '87183770.9723*1458061761', '1507210.8364*1482867859', '74193030.9716*1479098448', '23769676.4229*1454357108', '66691074.5616*1453084155', '3226284.23655*1479284857', '10073603.6434*1465138645', '10069573.1546*1472152533', '13584874.518*1455543976', '68031713.7167*1473565140', '56116739.2669*1454009509', '53719929.8541*1466093500', '73328954.1285*1469071986', '4922190.84592*1458991429', '10616497.7182*1468284308', '31058636.5347*1464668906', '55141788.6214*1456752051', '38624018.2526*1475369025', '43648867.8344*1472250069', '17245567.434*1456786777', '40196880.6726*1482177480', '53675911.398*1453398592', '78620014.1104*1482770468', '50095360.1778*1478068955', '78836792.0872*1457545541', '73435008.1389*1464182203', '74820361.4604*1466541114', '50263470.5948*1473220985', '34654801.9267*1465696691', '57947898.3764*1477336682', '18738859.3426*1475938746', '49000936.9999*1457798277', '79687707.2107*1487455431', '769079.18221*1476384391', '6868645.73978*1468229492', '83688658.049*1459576981', '17419700.005*1455082313', '31656035.1662*1454213960', '44265486.664*1471513716', '74607812.9805*1465477760', '52899368.7052*1464704731', '17047752.5276*1474395138', '86788149.2942*1486496149', '65555592.5692*1462316929', '60206859.9759*1474459921', '92159219.7622*1471102838', '61223439.686*1454414086', '54469302.8403*1461206402', '14457503.4808*1482959404', '18916221.4766*1485201743', '91691153.3228*1460145720', '40801008.2569*1472952230', '21720017.0049*1486387956', '3577543.3343*1476845374', '59221794.181*1452968455', '79140630.9011*1453064064', '53900814.4926*1467629313', '37088777.6238*1477469262', '50211015.9363*1469482085', '1512940.0822*1471329387', '5670599.37304*1455601031', '68118454.1677*1484949597', '73709437.7152*1478677211', '10904618.762*1485306756', '67561134.6018*1473346941', '19436494.3492*1478997024', '30434897.2529*1464186351', '27927194.7981*1468053449', '58514846.7051*1486826980', '27776879.5004*1477788331', '59408171.7845*1453677354', '58605679.8634*1460464725', '68340271.4933*1465080690', '69035572.4073*1464444579', '55597068.9813*1479605005', '34763928.4977*1483561078', '95744053.4076*1469317664', '57836408.1041*1458828641', '10301068.5438*1459374470', '92254152.7069*1453822041', '48915884.1884*1452804298', '64903060.2595*1452539800', '72367334.7911*1457863946', '53062755.2166*1483251800', '71078383.9175*1480633811', '19436494.3492*1478997024', '53215023.7416*1474518718', '31544411.9384*1454153688', '91445677.6474*1458406724', '24940638.7544*1482556846', '18634389.0116*1468885152', '18738859.3426*1475938746', '6960658.47534*1467713068', '1168323.6371*1483112731', '43325749.7475*1478212106', '27317120.6242*1486682218', '77622338.5632*1462627383', '11828784.5*1461806783', '25055552.05*1461277210', '99756307.5096*1478848078', '37105475.8656*1472029624', '25571426.618*1485291631', '4208424.53111*1482402335', '96569513.1691*1453482148', '83866028.2519*1482286619', '37462568.7285*1454650427', '55246215.2718*1478179698', '69744977.973*1483478683', '1836276.66752*1454270358', '47918810.1711*1476804954', '42088500.9666*1484370088', '20293093.5152*1467725770', '27825131.489*1452341096', '17322958.2013*1457502458', '35246403.5361*1466053432', '48243357.8519*1459745673', '99403756.2017*1467780176', '54071609.137*1485847265', '96126421.4483*1470062246', '24288077.0225*1455163576', '38059552.3137*1452175467', '94724383.6248*1488195323', '52884633.339*1487181702', '11386002.6081*1476364017', '86655784.933*1452781794', '88888648.16*1470641121', '87683933.1277*1470817648', '73008687.4028*1475324712', '98532048.4766*1480894774', '87969519.7855*1469463781', '65793273.168*1480201150', '59835509.1672*1465128238', '40752421.0325*1452076829', '89877826.9878*1468797530', '20207594.4493*1480129051', '45118919.3002*1478202980', '45420446.8451*1483976663', '98796579.831*1468598043', '3074419.68074*1464502578', '94843728.474*1478126286', '32766766.7107*1483152282', '58633322.105*1479905531', '22381935.4009*1476507451', '96335161.9234*1464239137', '23646385.1569*1456588753', '1009937.25329*1475412214', '27255364.8938*1475979664', '20515227.9729*1468883686', '70352862.8158*1485789546', '8954885.01648*1458314037', '2753534.78765*1485525753', '67067976.7693*1472807386', '25865366.0248*1454371001', '83656056.6214*1462413880', '95908236.7977*1455822710', '66028191.7309*1456510217', '78584596.5132*1476241454', '407821.492278*1486688572', '46431056.1103*1463458117', '49908769.3715*1475106915', '9878455.31413*1483164533', '12070260.7597*1464862498', '83694173.6235*1463610146', '11878993.7819*1476224878', '32017013.3343*1482608153', '35661698.2181*1484329445', '96199298.0382*1484332344', '18577912.6803*1457270809', '51475097.7973*1452492151', '16588976.8078*1452327342', '9222261.18832*1457504351', '36005905.3484*1456704274', '23794008.1457*1472469250', '871001.062508*1481732023', '51091714.1813*1461250783', '87108113.3353*1473516770', '66214301.2944*1467680355', '10062565.6944*1472454736', '22057539.6282*1459287627', '47581545.3129*1480947190', '37949571.2983*1463172567', '47689126.2167*1467909650', '63214704.6862*1464297084', '18442857.763*1461410549', '38933469.8644*1476193741', '9592191.68676*1477629062', '49726868.4297*1454344105', '55208269.4313*1467238650', '59892039.8175*1462526398', '23141915.7174*1461314490', '5813731.46999*1474587624', '78793171.1747*1485088010', '77928651.3806*1455697338', '87547072.5905*1484116638', '797727.845526*1481025861', '73509244.756*1465686239', '99519274.4297*1481190864', '12363785.4567*1454973312', '82844633.5852*1467039912', '94448405.5241*1476212865', '81116613.6332*1484216773', '60486369.622*1475400641', '46136921.8667*1472712186', '49523261.4848*1484378707', '70604062.7723*1463822303', '82326055.6943*1470030793', '3011281.44191*1456976059', '31080301.6042*1459974594', '65493932.8901*1487146781', '58374869.5125*1455208784', '44879587.9879*1464768990', '17465756.9777*1467473257', '65485216.3371*1461085274', '59431732.7911*1474806579', '52494755.4954*1453605592', '31032277.5499*1459656656', '30054712.3678*1452356249', '47162328.0023*1484893770', '60615210.6449*1452383896', '86138802.9445*1480666341', '6511658.0224*1456233402', '30301569.7488*1471261666', '56821483.4898*1482980090', '93993372.7746*1485166278', '10904618.762*1485306756', '57837972.1367*1455473336', '70106057.0356*1464321870', '83022853.1223*1459955613', '93867414.349*1458290707', '52073993.2187*1474859071', '81146458.5462*1481965506', '35230893.9281*1465435488', '71559569.9658*1460121610', '35544180.2549*1471805603', '1443544.51268*1458033159', '11740477.9383*1475222113', '5534856.66829*1455695958', '871001.062508*1481732023', '11417787.1196*1486746610', '40423429.6427*1475488313', '61399285.2695*1455142281', '59306696.81*1487819551', '90716910.2366*1487751573', '47383485.6141*1462943699', '10331779.1042*1473809325', '11818906.1394*1453047910', '47697237.3792*1472832392', '90400302.127*1483036668', '89565605.5612*1456274073', '30376735.8493*1477178866', '36161590.6551*1479238686', '65558326.9776*1484977719', '22826336.4595*1459215478', '19204888.8946*1485220778', '29764667.2719*1474840664', '37853720.2849*1475740862', '5661716.29983*1460612549', '8291298.76436*1458415505', '66086821.8772*1472675098', '62698124.1744*1454547014', '14504774.2379*1487499866', '48906728.5646*1471765911', '29098058.2179*1475433545', '41785912.6076*1478235619', '30890814.7894*1460718848', '46841223.4739*1467658286', '36868627.3692*1463202675', '26008998.325*1474606661', '62123818.842*1472712923', '19174888.6652*1455962879', '50737306.0299*1468423853', '15218379.7888*1476682943', '48296433.1116*1477969038', '84075943.7467*1478625744', '39243700.3796*1478419802', '36377132.9959*1472651365', '78377112.1347*1454565042', '45838609.8079*1481425255', '69899315.969*1455635498', '4274383.96983*1451881008', '79389884.5086*1460389055', '8015240.27262*1474028292', '24606646.1771*1484297659', '82563890.5329*1457813906', '15296101.9309*1486347347', '48314182.966*1482687678', '20090678.495*1473719687', '60861109.8892*1488295605', '39475885.8003*1476116162', '40889100.077*1466215521', '79142496.9033*1461692075', '29236700.7742*1460087243', '77941460.2172*1478251826', '74003643.3481*1480660251', '15261387.7862*1466683703', '16700665.7683*1457899637', '80425113.7992*1475807191', '20041465.6107*1459056940', '58746010.1795*1479076852', '30033676.9906*1487520614', '60362605.6712*1479567425', '15678486.1559*1456189018', '33606546.4147*1454684450', '59243703.2705*1464455933', '40142679.538*1462222169', '66849830.6858*1487709102', '86023233.4527*1475797134', '45032854.1921*1464125545', '21626497.2051*1482011366', '61294164.446*1470493409', '43895574.9097*1476254133', '14014540.4356*1464163660', '12134965.3315*1481057052', '29144068.1191*1484431565', '47097353.0395*1454410566', '94854570.36*1483478801', '47448100.6098*1478906970', '18889167.495*1454375737', '42002864.9403*1453441705', '72942537.3139*1456836602', '28676208.1431*1477620544', '39334155.3925*1464341059', '36566253.3828*1487539117', '39316349.0394*1481681342', '32093972.6658*1473189371', '30151414.6775*1476204977', '39120905.6218*1484531362', '6456956.43465*1471917573', '88797402.1383*1477745796', '47218809.8438*1468230199', '32769196.3942*1465297971', '17902955.064*1467244784', '58408150.0446*1456997263', '79466590.8295*1459776343', '7006986.12709*1453922682', '29454663.2061*1462550422', '64185576.6517*1453657764', '79094909.6121*1457818565', '74528338.4015*1456928572', '4730920.39608*1484811344', '90799256.2146*1475157590', '44916530.8107*1463577233', '50110125.9861*1487581696', '42672135.1719*1477732308', '12728521.7365*1461268775', '16822048.391*1465309456', '76256523.0953*1477305295', '46512653.5552*1478396539', '43760985.1473*1458165031', '28244861.6929*1467209211', '15096876.0132*1463245366', '19084828.0742*1455954519', '6820108.5423*1460578418', '46952835.2524*1462784866', '81950027.4496*1485717124', '35716615.8905*1465097444', '2099603.80289*1477450368', '4661388.39306*1456846126', '47130021.3044*1468906794', '51170442.1111*1477995168', '52178467.858*1463037592', '45877135.2397*1468391673', '13499982.1854*1481233819', '32132113.7986*1472011806', '92269920.3535*1468306571', '34591904.3556*1469363311', '54376300.6877*1468186982', '7924603.95925*1487696660', '55956533.1531*1457944584', '55141788.6214*1456752051', '48675679.03*1470171697', '9120979.02933*1458198293', '14167792.9101*1460051261', '46867544.8194*1485505588', '20121167.9838*1468228401', '7611854.49726*1452214123', '50694680.3636*1483791841', '65982669.7873*1458478120', '80078020.6985*1453465523', '15821883.28*1470844872', '15575065.7189*1486307058', '71408030.09*1471527523', '73798512.0115*1486616823', '23723328.9617*1454912304', '38897853.7301*1461369451', '21391624.1494*1459932328', '85961494.8837*1465082482', '60164072.8357*1481665386', '79215547.8118*1479888076', '65902833.9598*1465444017', '33503894.6939*1487201308', '20413763.0429*1457940440', '90583478.031*1478513778', '11442403.5436*1474512902', '13041647.3499*1481567877', '40479485.7978*1470089165', '18799191.3121*1476056659', '77720499.3968*1453486849', '20582433.7598*1466487602', '92637667.8887*1487382267', '86041253.1459*1461478153', '51820988.6722*1453745718', '89678597.8527*1453534129', '25244801.2421*1462847337', '3694514.27684*1480212333', '63798275.6041*1480343474', '6587420.78618*1462855376', '94168032.0617*1451739437', '28402693.627*1462639826', '74367988.0676*1487318377', '51302283.0361*1477429021', '39875839.6305*1472264379', '63120065.7138*1458666607', '79196854.6428*1470316482', '48654914.7016*1487708618', '10778216.7058*1457386437', '54978772.2292*1475322717', '91461889.0677*1478107705', '77772745.9164*1465246239', '53765167.2083*1460764269', '35424783.3277*1455395665', '33797842.0503*1470312134', '28018909.3577*1456269742', '65549175.0247*1472387967', '10304406.138*1483792510', '43882212.8446*1463649108', '75533384.1735*1467270962', '38612754.7682*1457965722', '68340271.4933*1465080690', '61399285.2695*1455142281', '40943326.6416*1456062402', '55290718.6224*1466989230', '15502752.0407*1472966553', '47981082.6094*1456744814', '61284558.6587*1463600634', '14373689.837*1486732074', '28677039.8178*1483690926', '77866870.8029*1457386138', '30679980.8341*1454917435', '46138359.3246*1476531097', '11105751.9928*1486810617', '70272386.4338*1467904070', '80440300.7111*1457832779', '12047855.8666*1464039617', '16803973.522*1470408450', '9729702.63519*1485770304', '4920873.28057*1465590923', '70402963.0986*1474036168', '82364143.006*1474290322', '30324472.8052*1461616667', '18316150.1388*1487812677', '17168917.8923*1474481563', '85005953.0879*1477186673', '42675538.4105*1467917283', '36580829.9899*1480660524', '58335563.4463*1456853934', '74489213.6528*1458815844', '16671267.8857*1453008418', '52090681.809*1477036772', '3555118.07418*1469955065', '86046448.9323*1478586617', '3636605.59026*1462122974', '62400260.6377*1459341832', '37584644.6925*1465146265', '64115892.6771*1459820984', '86081813.7281*1463631463', '12958869.0529*1479539996', '20314000.94*1465876215', '16016267.3619*1453798480', '77456441.1261*1482751982', '42643637.8153*1467466736', '44426745.2018*1469580816', '69912009.1272*1458965370', '6642316.36418*1474638831', '81119197.3779*1469950586', '26643203.3839*1476466147', '30663757.3091*1486343234', '94954351.4363*1461674031', '40791175.3314*1487096714', '30216070.1019*1453679448', '70031691.17*1466445364', '39717941.0454*1473861890', '3716251.46164*1473466690', '57063259.3383*1470074388', '55746609.2313*1482367075', '27049920.4035*1466593343', '36539471.4646*1468003801', '76114481.8429*1485351985', '12501846.8762*1456147936', '24444749.2188*1476189644', '69573228.0982*1487326543', '86526007.6692*1466555419', '7799372.50471*1476343022', '89225507.3983*1469390872', '62127193.6752*1486478769', '21182494.348*1482822662', '36238078.4947*1482843802', '31449165.0112*1457386756', '79389884.5086*1460389055', '22380433.3678*1465800087', '32093972.6658*1473189371', '90916913.9733*1477212115', '70960799.6824*1471872498', '18242584.1486*1466259281', '83697336.1311*1485220498', '89294987.4436*1485555987', '49236328.2958*1452741984', '90226554.4185*1461456397', '13086091.8309*1458030184', '8545538.96743*1483313037', '45034149.3775*1481535169', '66809286.6822*1470012664', '63506736.2084*1460436482', '19361816.0074*1486939537', '16287067.7578*1484821041', '41599384.7473*1486093644', '42247967.1212*1480395017', '74946484.1688*1484718852', '14778821.4247*1485879471', '65158418.5898*1468772906', '41096653.4607*1470390062', '13734912.2493*1466084483', '12236232.3239*1481788597', '45022776.0253*1467007714', '15617.2762814*1462715815', '27949897.4498*1465056190', '50694184.4346*1452069224', '87313337.4992*1481176933', '36329247.7247*1455928065', '48580707.8103*1468699196', '12516801.3058*1486697570', '22023546.6859*1461552428', '39579886.612*1485959066', '4517502.41634*1461462941', '49000325.5007*1473055115', '21246130.3511*1478833143', '1341200.45288*1454791106', '18859233.4845*1485601875', '9956530.71159*1481224529', '22276074.5772*1461366495', '27026503.989*1460300122', '46246071.4203*1472155449', '5213949.27241*1464479817', '67795033.2384*1461515316', '7252622.4478*1468827770', '24857286.7119*1452848232', '39376131.8054*1472787089', '90839294.794*1474721033', '83476103.0798*1454947258', '11597204.9644*1473231598', '64242238.8766*1476253102', '38634523.3797*1480703742', '35081123.7059*1478036525', '51726794.4484*1460881925', '47989243.2056*1482539191', '54941562.728*1468046867', '71033146.8943*1472858414', '35854947.2823*1459888414', '16302515.465*1468278403', '88957002.5774*1475124450', '60161807.1101*1457313892', '32805170.9945*1456649712', '77312986.1631*1461008724', '53719929.8541*1466093500', '60495032.4567*1487502516', '19240807.7*1472522802', '67584086.512*1459763762', '97004481.0655*1465277326', '78725827.9111*1462276744', '4692898.36183*1487199223', '29156884.8346*1470250437', '76989125.545*1477136944', '98165574.71*1458025711', '81859486.5259*1474015189', '34477355.2618*1462041061', '28025801.8071*1478945537', '21610504.8564*1486347717', '38386328.7897*1483209771', '58268388.9761*1452575051', '5890160.82643*1457458817', '89976340.0593*1479064931', '37648579.0985*1486245497', '37997855.1149*1482590307', '77078223.2914*1464213827', '77093318.3706*1471921160', '77189306.773*1458009727', '36225925.1396*1462589092', '28040286.4161*1485342474', '98852980.1813*1461971752', '20005238.7946*1486630638', '26643203.3839*1476466147', '49594792.1674*1464446202', '40454703.2452*1461531377', '41909183.0114*1486724355', '48815108.0092*1455650525', '42875793.4428*1464776819', '14910013.9878*1453874541', '12633421.7481*1473030771', '83402374.8411*1476432906', '72451993.7311*1474827364', '56100054.5056*1463135868', '40388268.4917*1482784364', '11138246.1816*1461215940', '76114481.8429*1485351985', '13086091.8309*1458030184', '59855842.8027*1480155325', '51781270.48*1467463553', '44480601.7946*1469541958', '56162078.16*1460542734', '70350790.3184*1462520751', '36668358.3665*1474111642', '37831009.0785*1451816738', '45981398.7843*1478508651', '71042695.4408*1487915573', '54011565.8609*1480398681', '46714134.0492*1467180321', '80089576.586*1481949226', '66708710.6566*1468036739', '65699842.3989*1467380086', '99565486.6988*1483833860', '42178566.6236*1458670847', '24701877.3741*1470439810', '79848459.8958*1485501803', '8820012.90744*1462793261', '95189948.6192*1457608995', '87079401.2689*1481624122', '32552118.1994*1476247660', '74648203.0342*1459451772', '54196704.5415*1482523656', '99594846.9393*1468433715', '34911980.9242*1484483746', '2281274.11158*1485026134', '53205244.1901*1457308695', '30767054.882*1481483829', '32402354.5216*1465447137', '73157975.299*1486171952', '2825277.10742*1473111226', '82099920.1792*1466000617', '49729098.2292*1473767837', '9741588.04411*1460037450', '18520797.1012*1488107117', '35104596.4035*1462847179', '96598476.2082*1457037310', '62721966.3212*1461180807', '6868645.73978*1468229492', '79479225.7329*1458770552', '82544357.5504*1469558592', '48296433.1116*1477969038', '63757258.0675*1469231310', '30860489.3042*1465384724', '43004246.9614*1457975156', '37779249.6272*1456625715', '46345325.2454*1455026228', '67257970.0565*1476986933', '12191086.659*1486673920', '82962241.0976*1464372594', '99391716.1311*1462769221', '30134240.278*1481681014', '57786364.1138*1468144350', '87577005.3017*1486269468', '15925185.7365*1467670121', '18279566.8072*1477159288', '50482184.3473*1472878317', '10294931.9286*1454189142', '87597261.048*1472863760', '35839322.1378*1469075370', '34828574.9743*1480030204', '30188061.19*1458792211', '56936274.1184*1461485067', '21376510.5671*1461754400', '33168166.0976*1477346578', '96246683.4478*1466571988', '68569923.0929*1457800140', '80593880.667*1475704466', '71327736.8875*1458775853', '42642460.7332*1458187487', '49796278.7784*1465590195', '87731228.6437*1478358512', '32971514.4631*1486383871', '78265808.6622*1456121138', '8048173.90274*1480679026', '45857207.2344*1458525246', '2663318.8783*1476955551', '36893027.9586*1460586997', '39738414.9253*1463485782', '78154682.2927*1465647999', '2122675.89463*1473489093', '40655817.4048*1454622154', '39472240.9442*1451763169', '63318724.8041*1462227428', '89591683.4991*1453427992', '10824152.3404*1467931672', '8396722.20607*1454918800', '50881691.6899*1452576922', '70783947.1051*1457386265', '24118501.5597*1475196297', '44982947.4873*1479366953', '94954351.4363*1461674031', '78905719.7471*1467765831', '2145534.93791*1481187984', '97113187.9364*1452648070', '96333016.1915*1460730037', '62887017.3277*1455016538', '90319380.9964*1476200400', '68369007.1329*1475152972', '81903167.9997*1477500356', '24762387.373*1465086517', '41010865.3548*1474082914', '64483196.6517*1478084307', '62766957.1312*1453082412', '40883689.4199*1480465472', '56701699.3435*1458287524', '30537599.5979*1467365994', '65602861.0374*1458405589', '16013125.3418*1461686168', '19498931.6083*1465221631', '65570797.1668*1463937233', '2096242.19603*1480877490', '90498686.8714*1478248924', '68171128.7078*1462989872', '13532903.8859*1465479522', '55327541.9187*1467749667', '5396987.61673*1475307022', '7116698.70231*1471737982', '14934459.4584*1478554215', '67795033.2384*1461515316', '44922615.5099*1453147559', '51682424.8536*1477130976', '14846398.897*1481535092', '51418771.1968*1472454861', '33053118.3225*1485592474', '87842900.2242*1486804158', '98379430.509*1460115601', '44329480.98*1487976106', '20217556.1149*1470403405', '10554611.5645*1483926407', '22321796.1604*1481067427', '49358421.9064*1473182148', '86111637.2466*1474226908', '62542770.1871*1462554988', '10093220.9476*1454014755', '92560579.8954*1486274327', '42203326.9242*1456297436', '90054515.1472*1482724671', '58787277.9422*1479225434', '24288077.0225*1455163576', '46160350.1171*1483077120', '94895838.5494*1475079205', '69146357.8876*1485643255', '94118015.7271*1475456620', '7190943.84467*1484237254', '51714161.9149*1474881666', '12761406.3599*1473307070', '25361541.1108*1468863158', '27909256.9878*1466796501', '73360725.9997*1474696063', '23449541.7261*1475304347', '4716546.95705*1456827838', '65397307.3669*1455922050', '19943056.1715*1459742834', '57772113.1744*1484307989', '44988375.7955*1453236054', '83448272.8102*1481707005', '66813200.8152*1485498360', '7924603.95925*1487696660', '127776.198993*1452564626', '38179208.1926*1487033737', '89044889.5496*1465160327', '61880877.1842*1476231115', '60206859.9759*1474459921', '54879384.4284*1454690832', '56832788.9953*1484563268', '14879112.8144*1486283481', '53122125.9345*1452413435', '16442591.4326*1460522647', '77078223.2914*1464213827', '40309819.0057*1479708247', '88927227.7856*1455011208', '58791413.0142*1463559935', '36377132.9959*1472651365', '34368654.7678*1460843898', '23858677.3643*1480574597', '77950249.6031*1483787207', '30485569.3416*1466467820', '35361041.872*1463082870', '33329382.5913*1477567691', '71912323.4649*1483601565', '9982236.97957*1452552002', '4432846.79006*1462540025', '61353523.922*1486917225', '24553644.8587*1471265968', '66783869.9895*1475555239', '68970542.5493*1458929008', '35716974.6802*1452756579', '14354577.4253*1455026840', '20207594.4493*1480129051', '38153474.4301*1474503693', '13367089.1052*1466616259', '38892106.8131*1482699821', '58641244.8673*1479176236', '25130300.3325*1455063908', '10723226.2804*1475429256', '46672497.5694*1480203763', '26727015.6434*1455240825', '68710192.0831*1454444150', '30755494.2986*1471185482', '19777178.2947*1474366403', '97517248.6063*1464452298', '11340214.4796*1481469932', '4049885.32595*1461571072', '61456592.0483*1461195000', '47562161.938*1455148066', '32693833.4949*1483538932', '12907593.2514*1458862298', '12940062.8369*1466954696', '86363736.8843*1467257908', '87381158.0062*1456581054', '16686395.0142*1453779361', '92808272.8514*1468038655', '79998452.7794*1456455369', '43054650.8617*1471271934', '94081728.6893*1474523233', '55360499.125*1465909695', '20212591.9464*1466680080', '8891093.67508*1452438361', '39224352.5704*1460729422', '3806282.48477*1460016726', '76293074.8464*1464285685', '46345325.2454*1455026228', '48456389.5422*1483307417', '22676242.5833*1480963238', '75549893.5995*1483147494', '54028427.4058*1461091495', '39471987.9224*1459334181', '9955770.88954*1454337334', '63374492.4768*1480045530', '39238111.5152*1460714566', '5439213.68704*1476625337', '59830056.1125*1482523026', '29421380.1922*1481723140', '54366814.3386*1456672387', '95889195.5769*1469286406', '49726868.4297*1454344105', '86049173.6893*1458026078', '86788149.2942*1486496149', '40032875.28*1486573832', '30134240.278*1481681014', '59468125.8429*1480823606', '93856249.7262*1477938803', '82600181.1896*1469035953', '9335679.61145*1484638319', '76304489.0779*1475942181', '98414492.8227*1467252705', '83854913.6876*1463108993', '44383170.4954*1473251106', '69396681.2829*1460796736', '47681106.1069*1487588543', '37407172.3211*1484662929', '39358275.3823*1483143967', '27776879.5004*1477788331', '99176125.8268*1482567187', '26171163.5291*1484005797', '50963649.6318*1453610104', '40479485.7978*1470089165', '66904268.7437*1460028125', '51969263.5682*1473754904', '88425180.4432*1478847070', '13584874.518*1455543976', '55300071.2539*1458991244', '19631232.6207*1473017955', '16693093.9191*1457129165', '52927407.7717*1479176789', '35721008.8367*1460667614', '69779105.4374*1487852484', '44524327.2014*1483147340', '2830126.84041*1456143816', '19729260.8268*1467355600', '68450443.5623*1471390238', '68550121.3672*1486827063', '92729377.2384*1475616399', '89685528.8122*1462818631', '51714161.9149*1474881666', '44617492.7662*1476101659', '90905861.9445*1486974712', '92717949.0841*1461741292', '33215110.1056*1466948404', '34711391.9102*1460200359', '95254565.9251*1468740968', '18804698.7373*1471732630', '79598582.1989*1462184852', '97997265.9782*1476538054', '90299378.7291*1455882097', '40007472.5051*1461800639', '6441156.89463*1455400140', '87688152.41*1459896522', '22768314.8078*1475425901', '14760811.5452*1459930249', '88500066.5424*1483374764', '92677034.482*1461365427', '21734885.3475*1479816966', '35186908.1224*1476210899', '79810467.9467*1464003101', '46054978.4824*1479358881', '1423692.39109*1453455358', '66869339.3329*1479125058', '52705922.9974*1462492877', '94886168.4123*1455856975', '96941554.3991*1475174070', '33157962.7901*1458484651', '7684994.64981*1487286760', '9592191.68676*1477629062', '11990711.8548*1456093808', '3041358.02493*1469236994', '171000.248982*1470641321', '36161590.6551*1479238686', '44460735.2947*1483786875', '7260247.71761*1475639499', '47651756.8542*1469625789', '90740086.2578*1452287620', '12059728.1602*1463998015', '41005483.9387*1470819182', '819859.336044*1471837585', '88744125.5219*1452451397', '41145221.174*1469417431', '7901011.89513*1487976639', '73516685.2812*1453423004', '17997431.974*1456505011', '53925943.6232*1484865345', '21535951.0759*1479465450', '88844540.0592*1482238749', '17617007.1425*1453145614', '97485959.8889*1460728454', '37973748.6797*1467842425', '77739341.3084*1486206672', '99176125.8268*1482567187', '38505475.1229*1459742924', '40625928.5898*1452537896', '52073993.2187*1474859071', '90703267.3385*1473970256', '79687707.2107*1487455431', '93582143.9926*1468568480', '33713581.7258*1480275931', '25513638.1481*1468954483', '61377774.6128*1467817788', '1335630.74236*1477790786', '26171163.5291*1484005797', '13679196.3794*1475286153', '58189227.1362*1476172065', '48041112.3033*1475137192', '573071.452144*1485950560', '60794745.3678*1460316218', '13198152.2595*1463169313', '8884189.13226*1455632296', '16198833.8661*1484110503', '7765505.3232*1454941641', '34985557.8599*1466728110', '61377774.6128*1467817788', '39207170.762*1461515606', '20495998.8618*1487147664', '97782849.0638*1464320331', '89689176.0156*1485074093', '10339663.3649*1486072473', '37265223.3775*1452595587', '55138950.0879*1481729948', '89856483.4455*1487662683', '30008464.2989*1487088478', '81986135.309*1476785278', '64025837.7383*1475367357', '39695020.7125*1483934461', '32132113.7986*1472011806', '77528788.3288*1471066329', '71036042.4601*1462013653', '11409453.0679*1459836976', '30261195.7894*1475835472', '13791409.7293*1458591340', '6726202.58732*1468447776', '88024288.6456*1464218481', '31464636.5785*1471029509', '16275047.7672*1468590369', '71491832.1535*1463273756', '47129528.4264*1465451360', '78261267.5326*1482607609', '32726268.166*1469626930', '22399934.7755*1482172514', '54069914.6409*1473144227', '98481811.1078*1479957041', '16287067.7578*1484821041', '8361256.26578*1484162983', '46952835.2524*1462784866', '73400190.4166*1483682746', '14652426.0699*1472696548', '50944207.2217*1484283567', '81062454.9362*1457022868', '96417280.7833*1479776262', '14885184.6917*1468939217', '78154682.2927*1465647999', '91903832.2202*1456673272', '17562266.4611*1479160302', '79576107.5052*1482398660', '37926748.2472*1475732858', '77125750.5534*1477598205', '17634010.5248*1453594313', '41420925.2076*1453561734', '9756784.73257*1476353255', '24118501.5597*1475196297', '87183770.9723*1458061761', '44961508.9281*1459143162', '48793155.4402*1477776660', '20746728.7236*1472117190', '89445004.4701*1487806742', '29796370.7232*1482863660', '66571386.2507*1469476107', '42009337.8067*1475872178', '78832584.1605*1467995205', '57269954.0822*1464886586', '7050330.64417*1480468660', '78156094.436*1474762838', '77803845.0002*1474791425', '88286877.015*1483381178', '78649187.2781*1460881975', '39132168.1962*1469643780', '77174309.3363*1481658839', '45785224.1748*1472634988', '58140367.9057*1473088762', '86495687.8789*1465902329', '73807495.7207*1461837730', '8465867.40507*1453299440', '407980.440048*1466826703', '42586727.9094*1461132657', '13129524.8699*1474730337', '51091714.1813*1461250783', '49135233.9457*1484241922', '44021298.2177*1476797913', '17968345.139*1464014039', '34535049.2821*1456123988', '53038711.1483*1470904721', '88589483.3584*1485080461', '71330377.6412*1483121663', '75939055.3352*1485750124', '20269407.7153*1456601363', '31855717.1016*1463127617', '73075675.6549*1451950245', '45384886.3056*1461735898', '1923087.88895*1453964999', '83041571.5166*1475842940', '51195621.9137*1475558321', '65995846.5525*1473839457', '73224196.2625*1479769566', '36078482.5898*1471572802', '99594846.9393*1468433715', '252723.437169*1466604188', '92783836.6412*1477563699', '36717160.092*1453900665', '56612819.2326*1472493755', '30388979.5216*1468954112', '89073174.2587*1458172637', '85254631.4421*1473680142', '69665168.7713*1461473853', '2178594.56046*1467508478', '12835569.4999*1469537154', '22474498.1114*1484940818', '47337766.0694*1486510390', '58718540.3626*1485132810', '70155187.3873*1460496018', '80624163.2316*1469670429', '33125077.1307*1486341583', '82852857.9934*1475075470', '55143838.0551*1478286700', '57860470.1408*1477626511', '69459601.5553*1482988793', '6431331.63544*1461580669', '82918014.4627*1468734655', '49688781.9762*1481896176', '60083942.2407*1458463034', '48321252.7822*1457384524', '72190124.3116*1485722833', '49091809.3248*1452125623', '41180138.2877*1484497032', '87064417.4033*1466242212', '14105914.3014*1455124310', '85644875.3009*1454229565', '2335137.43969*1469711244', '40326173.731*1477502384', '30038415.2283*1476463758', '53301210.3808*1469694108', '61296587.0523*1463896082', '81928002.9646*1477341513', '85060635.9315*1463600024', '92644690.4945*1469329592', '12681949.6965*1463642550', '42672135.1719*1477732308', '85638015.5853*1479292764', '47920541.8714*1482353102', '92743.4201355*1465941286', '63821402.1045*1484233421', '69504616.0839*1470984582', '89666329.2277*1485667426', '40910656.0417*1455428264', '23995927.3258*1486456894', '44955464.7388*1456189648', '67919301.555*1486023602', '85413950.3773*1480705609', '30393323.6204*1470062992', '84111944.2241*1456057928', '58128294.3441*1476372489', '77598682.8298*1459001263', '42790892.6799*1486083849', '17457500.956*1481631744', '14688919.367*1477917906', '63506736.2084*1460436482', '6431331.63544*1461580669', '40388268.4917*1482784364', '90537185.1768*1486586442', '49996670.4414*1479497295', '87268471.3115*1477119995', '10349219.2307*1466103108', '27879079.5234*1458425740', '98469752.2083*1483399285', '19006504.7849*1457867513', '15046758.6854*1464570677', '84683187.6566*1463851726', '46578042.8921*1466454370', '56162078.16*1460542734', '16968168.1792*1470564684', '65733274.9466*1487746475', '6082647.82748*1458008022', '39427381.7095*1484322056', '68191281.8395*1470767311', '87898047.1535*1478611816', '49946130.9616*1473681219', '99356302.029*1455861580', '22716401.5901*1470152106', '70087391.5313*1459068695', '81928002.9646*1477341513', '85203490.0913*1478702623', '51029676.3248*1475314173', '35209082.2015*1453621894', '72409071.7924*1478272515', '92129636.2383*1453955830', '15972308.1781*1474810042', '82364143.006*1474290322', '38509921.6902*1472232329', '87492531.7852*1464945142', '45420446.8451*1483976663', '33797842.0503*1470312134', '11440260.2093*1456421920', '13527801.3077*1454636170', '7338342.21358*1480971999', '3124585.49593*1483367805', '51282230.9242*1462441787', '33981879.3143*1473657550', '80109032.1189*1465472691', '11919363.0248*1473017319', '10771603.7809*1472961569', '48166326.9459*1451727555', '64144606.0573*1471108375', '27971825.1948*1472489099', '8309521.05836*1479533838', '59819496.7707*1459176457', '33951029.3514*1474264284', '97473939.8165*1455019389', '4535388.01978*1487597165', '33503894.6939*1487201308', '8068132.80984*1482679889', '29999373.9118*1481019725', '58444222.297*1463994609', '27536321.1583*1461192340', '91931755.4718*1454866548', '57030442.2758*1468765411', '70302997.2153*1482037167', '56776441.6697*1474376694', '84813984.4231*1457478285', '60282859.7133*1457316671', '48558040.1813*1476446060', '14585780.5187*1471064572', '2756920.02783*1461571749', '36578415.7872*1471886048', '36224714.1348*1461536951', '68407216.1804*1456271288', '47020331.3636*1456713211', '38264953.4302*1468448794', '98040137.3834*1483915549', '249175.784017*1478688025', '32766766.7107*1483152282', '68159463.5967*1462044144', '34105802.4108*1472802046', '97521653.2833*1479130129', '7619038.85412*1478615438', '11740477.9383*1475222113', '19174888.7609*1467601350', '73910687.1881*1470552236', '70392619.4734*1480984630', '94244356.6844*1469369389', '98505513.3403*1477587213', '76656763.9218*1466258497', '14373379.8505*1456402979', '83790169.5375*1467507927', '54327853.0243*1481459490', '72170915.4493*1465230292', '11194247.2659*1461628512', '24864692.8011*1461969360', '99738940.946*1480780106', '88759519.9992*1453466342', '74724336.5292*1485076246', '46700560.9877*1471416905', '19119049.9746*1473884628', '44663440.4598*1473766712', '95633802.5226*1460052178', '66920118.6014*1476792498', '57929294.9267*1466904670', '2045735.87026*1457793913', '64964255.1993*1466310396', '90531637.9897*1488279177', '34151618.6498*1453553770', '40612897.3781*1458487878', '16464264.7078*1478312591', '29999708.4364*1472659350', '7728872.11888*1486133938', '62390170.3433*1475129801', '50940157.0551*1466407466', '96415922.9699*1467382183', '82042296.7467*1479341025', '94685722.05*1484848259', '60794745.3678*1460316218', '99978607.4561*1468479485', '77487475.257*1480093079', '80331528.9169*1468110372', '12155972.9687*1458678223', '1376099.57754*1488009881', '20004068.8011*1477604320', '25669188.9652*1460384437', '39627382.1433*1479434506', '88957002.5774*1475124450', '76649823.6909*1484681430', '88188266.7002*1464586818', '52798268.2298*1458149525', '59896146.4042*1479332402', '61697524.3569*1473577557', '95396848.216*1477972912', '83543079.4834*1467328374', '60502008.8103*1479078418', '51365492.5152*1451783625', '31160141.378*1452677155', '78987890.1098*1473117622', '32902405.4603*1476388461', '29796370.7232*1482863660', '80563961.1136*1483685771', '13968903.3822*1467880453', '81726256.5488*1456500980', '54808328.2213*1456074324', '42987661.2077*1457294484', '67686797.0283*1458052666', '6032772.1745*1470974157', '19277629.3925*1453331761', '15513983.4855*1476966452', '79192319.6357*1469662273', '83112463.5546*1452888904', '14424902.8046*1488135261', '74012479.8344*1480612184', '63956027.4171*1478666824', '72164036.1511*1480486581', '82710372.172*1470664513', '38089676.9082*1480076120', '65783785.8533*1476972871', '39653866.1862*1480952031', '73544787.956*1470655978', '87832077.4155*1462492012', '12460305.3949*1477708328', '36391299.4124*1478031712', '58674870.1508*1461917734', '26338334.433*1481103640', '65174865.5537*1460538294', '69867176.2749*1452244765', '97764734.1734*1487737150', '30261722.9407*1455148304', '66191547.1227*1467723817', '10857515.5083*1463651530', '82360286.875*1484436789', '5950137.36522*1453900307', '27727798.7682*1471592285', '45131227.2302*1476190897', '83967565.3718*1478719759', '29183011.8658*1477244386', '78694147.3544*1478355520', '93586261.5898*1480131541', '82652275.5729*1452858359', '47152587.2551*1456882567', '58408150.0446*1456997263', '29921775.3267*1474411864', '61274264.3395*1472426619', '65413278.1287*1480820508', '63447653.6655*1478092551', '25503838.8586*1467655254', '42941166.9457*1459850388', '38172948.1459*1470855443', '79686094.3668*1463998790', '50003188.2633*1465936705', '30663757.3091*1486343234', '46121899.2034*1477905530', '69536054.8806*1467632537', '75355919.0866*1462130130', '95939692.2017*1472012625', '28641948.7364*1486570213', '37132887.2501*1484546797', '77275141.8909*1487733265', '8425549.64514*1452616504', '34372543.1894*1477519058', '15359865.7702*1471393585', '80397678.4545*1478165663', '82359.9750905*1459160760', '87068509.6483*1480506027', '83377750.2116*1484012550', '86221443.9493*1464391526', '52689498.1999*1466206275', '93578021.5745*1461025434', '28389350.4614*1478056851', '71825719.2712*1479671230', '68815725.0423*1451786023', '68223793.8604*1457157868', '86391150.6913*1468586221', '93088970.4928*1478734158', '72280350.3794*1455066168', '37855079.9505*1462173032', '14893711.9211*1456084286', '74134672.3631*1452458152', '92670526.4907*1454046775', '64144606.0573*1471108375', '37962680.8617*1454781463', '29040322.4939*1462258520', '83445831.8951*1487553331', '64226087.099*1453876393', '97164903.2383*1469873551', '78918472.1495*1461990798', '62372222.6952*1470262411', '38088574.5678*1479203847', '54535878.5164*1478166202', '50006516.7264*1459184170', '14683409.519*1482744908', '92918915.6046*1486584484', '77088986.9757*1487480731', '72751981.362*1458015127', '33881960.686*1477260182', '30328531.7199*1454609413', '50317370.6489*1479244225', '50472225.2008*1460999664', '74124348.9247*1462314666', '26872018.5645*1454053805', '96941554.3991*1475174070', '17554406.5693*1469727643', '25301188.4216*1463471606', '25673360.5849*1457612920', '72757443.4724*1477936594', '34402892.0023*1475377760', '83262671.0343*1472029839', '78017742.1398*1474973566', '83260216.3456*1481220288', '98100049.6371*1459264890', '47651759.3808*1481186324', '74193030.9716*1479098448', '10627942.8455*1479393877', '77234451.4878*1474596388', '41619649.8784*1468996852', '29897937.0235*1467245904', '16404614.4862*1485370228', '49105672.4756*1456437330', '90823377.452*1471504989', '11166488.7571*1464939125', '66330977.8417*1470315276', '25598497.4812*1457652779', '94954351.4363*1461674031', '37735535.659*1472554597', '11212565.4173*1458898099', '28279424.0642*1488017847', '77924376.1714*1456305443', '8016832.30347*1482430899', '63375366.8733*1469950543', '15296101.9309*1486347347', '60164072.8357*1481665386', '52240351.952*1477778264', '62408550.1248*1460200672', '94724383.6248*1488195323', '70198917.2318*1457446930', '44476144.8519*1480071237', '15428629.9152*1483203421', '30767054.882*1481483829', '80099398.1969*1473954938', '3226284.23655*1479284857', '50072258.7456*1456508915', '64123472.1936*1469655405', '90881589.7916*1483895948', '62390170.3433*1475129801', '69707399.6941*1453752395', '70791166.1963*1461274066', '34460569.9663*1477010364', '7359046.86957*1453332005', '75903063.6773*1484691231', '39249521.6279*1454343810', '99443023.9234*1468911597', '96765314.8993*1465076240', '42247967.1212*1480395017', '3803707.64249*1488315492', '58576915.6706*1457505556', '39692299.1209*1484666960', '93672620.0803*1460642155', '54771400.5629*1471427864', '60495032.4567*1487502516', '77567345.7389*1473081961', '4615222.32957*1452707474', '30198492.8459*1473417849', '76862171.0664*1480454787', '69507001.327*1471554105', '6868645.73978*1468229492', '43008515.471*1483295487', '84683187.6566*1463851726', '59731702.5318*1474090048', '74786181.2584*1467280400', '77068238.4524*1464301235', '95129933.4484*1456551086', '71389296.6306*1472742490', '88445321.2864*1484436610', '27493367.4211*1457785663', '71339265.3266*1484380110', '73460094.1675*1474245760', '48405624.873*1470994137', '20694533.7728*1487965733', '61636736.9041*1483391984', '81903167.9997*1477500356', '88589483.3584*1485080461', '11055924.6643*1476087676', '64675265.2899*1473064721', '64550340.1406*1463711375', '5385489.25321*1465002170', '35839322.1378*1469075370', '97777783.3898*1466700832', '46581817.6827*1458844518', '10241770.6829*1484867871', '56092267.7448*1467517032', '66754917.5958*1453771251', '23475697.6754*1462876082', '656285.858367*1467907258', '23881288.0693*1459024273', '95325609.0492*1476406691', '80162102.2157*1469047121', '18086123.2087*1453513492', '84091122.305*1466197610', '75055722.42*1468299180', '55199360.6595*1471231013', '50443807.3495*1467812243', '46886937.2298*1452692188', '65902833.9598*1465444017', '80331528.9169*1468110372', '7591495.19756*1462677105', '35500247.2293*1471555412', '30804913.9892*1480203357', '74857844.9514*1460545207', '66509968.7475*1459386771', '58718540.3626*1485132810', '41713359.9683*1471195280', '69562134.8499*1480945619', '65397307.3669*1455922050', '91068118.4089*1480289842', '72416957.0436*1472781417', '4328030.21304*1457747972', '27220917.1241*1464336822', '35624983.4423*1453912917', '47524850.1164*1461958485', '81809064.4565*1471818290', '35839322.1378*1469075370', '6312346.96338*1467397899', '48221259.6634*1477656371', '77826751.7092*1479629063', '74454370.3783*1468808807', '13999978.661*1474014172', '40170480.2915*1457370855', '53393791.0886*1462085778', '71602920.5652*1454118870', '78643697.2189*1476642234', '60495032.4567*1487502516', '68200114.8008*1466503129', '78484063.794*1481727623', '12689478.3072*1456075049', '5541790.13983*1469044780', '26289131.1636*1470799684', '65334810.3039*1460297005', '30444160.4148*1477918033', '60219713.1634*1467787430', '33151268.0402*1472177399', '45494500.0201*1460613862', '99177737.0464*1474632127', '10623106.792*1464485697', '14869408.3041*1478091117', '58338576.0766*1462668333', '19480548.4221*1461002487', '36252350.8651*1464723600', '90534662.6545*1465987726', '38060578.5636*1462276549', '54456519.3431*1483211638', '72771852.2341*1467237713', '67112650.7657*1483289369', '56702616.8441*1456358599', '93740287.4336*1467337806', '22069954.8558*1454577825', '91891543.6649*1460409168', '71640547.4745*1465786054', '72117831.956*1455009405', '26727015.6434*1455240825', '72961012.9699*1484890230', '69329897.9573*1480657958', '42736769.5717*1478613090', '1436482.80452*1454287732', '14243273.5766*1470512483', '95385089.8328*1461641602', '78950113.6834*1480039622', '69035572.4073*1464444579', '36947925.2288*1452394703', '24606646.1771*1484297659', '14893711.9211*1456084286', '12610921.4381*1476237351', '5990007.92682*1487035301', '70762729.0175*1463501494', '62844240.7638*1478047346', '16738073.6999*1475389646', '20908975.0263*1455422074', '43001063.6211*1465742926', '81296075.0558*1487940605', '12366002.6294*1458992428', '31002954.0687*1483542946', '72118798.0121*1461335207', '14159294.5625*1456742361', '34643967.6978*1459249246', '8900690.63146*1467747443', '92340451.4679*1467165688', '33620618.3564*1480294867', '32151097.2642*1460937470', '60150942.4829*1458185526', '27639607.623*1472010601', '73612572.1906*1471062600', '3595635.30312*1484325805', '58374869.5125*1455208784', '20883420.8983*1472774990', '38059552.3137*1452175467', '45927648.0003*1463018388', '34056087.6601*1467657326', '58337352.8708*1462910579', '69459601.5553*1482988793', '54076940.6478*1487755781', '75101332.4867*1468994757', '88891208.7412*1484273502', '29211569.9496*1486676799', '94977164.49*1485665134', '61618471.6589*1463262636', '57594715.0127*1465616515', '17963252.1347*1463512313', '61836698.4947*1458664283', '69540839.0327*1471120354', '48497361.6398*1483341900', '86800125.4821*1480713090', '50340895.8812*1479070630', '30865172.6282*1474867915', '93620609.645*1468055758', '81473291.4891*1459100651', '58625227.4061*1472604281', '35723879.4949*1475616618', '59965403.2148*1458894412', '66870001.4342*1480840485', '45063403.4637*1461887466', '76464874.5687*1472411160', '92424283.6836*1474460213', '86910652.9409*1452727180', '20363180.8517*1454843655', '38204011.7264*1477211588', '14494079.7372*1477397664', '91358472.6477*1487670782', '66493444.2541*1462401003', '52597459.6994*1466562365', '52138195.6439*1465093811', '53336342.6504*1463900565', '92185559.318*1464791108', '52093050.6308*1475438545', '92185559.318*1464791108', '50146935.5462*1459676512', '93076947.4072*1458053458', '50531764.8525*1458196268', '25792724.9513*1456976778', '15422485.4377*1477374547', '99939268.6592*1478387923', '81511222.8302*1456034247', '22490738.1156*1467494356', '50636297.9902*1455191444', '52410123.2217*1451947798', '11198802.4487*1465086932', '2725790.17385*1459408481', '12820674.1972*1477841138', '41620472.3831*1469342830', '95385089.8328*1461641602', '83391233.1489*1463825009', '73633179.0219*1472980006', '9530484.57235*1460204758', '6072440.93035*1474938968', '23449541.7261*1475304347', '80709795.7483*1461223310', '30506219.6548*1477446575', '70302997.2153*1482037167', '86442990.9797*1462928949', '68293759.5174*1482825673', '77846189.6184*1485377691', '30532137.4721*1478822883', '27461775.3325*1469040783', '90549312.2717*1455989104', '21760468.5578*1472761952', '74012479.8344*1480612184', '62744205.404*1466181736', '83910850.8156*1477818115', '9607430.67394*1453631592', '89137945.2757*1487604020', '99293486.9266*1459419493', '21393721.7704*1462011086', '75120347.2554*1481944496', '47441768.8855*1468653925', '52248663.4031*1475776677', '42137556.6165*1478010438', '13209882.5486*1485361372', '25524941.7974*1464110344', '59221794.181*1452968455', '47123093.1974*1461176895', '27423215.452*1454482936', '92417483.1657*1483085803', '15629051.4943*1457786518', '57701685.4911*1488294907', '19223436.661*1459069264', '43344966.5932*1458624241', '11586640.6596*1482111604', '70324333.9857*1460140610', '67223307.4138*1487922432', '57339134.8563*1487728355', '38936270.3253*1483233191', '72521501.1454*1476063173', '9810237.2539*1485967422', '38492244.4451*1480733421', '61399285.2695*1455142281', '15164763.0293*1479086933', '12765914.5039*1461401870', '58791413.0142*1463559935', '48473016.8523*1474697295', '90701040.1403*1487694175', '96918504.1929*1466933721', '60890723.456*1459507985', '45315597.3359*1465308903', '87629296.8632*1458631016', '97113187.9364*1452648070', '44509420.309*1454194737', '68453046.463*1468796505', '6563915.43109*1480453831', '12551707.4399*1461938679', '52218165.5695*1477404899', '31655548.0077*1460349200', '46616907.7733*1463757050', '14719643.1182*1480135468', '44287508.5992*1471480174', '86582227.6756*1458183851', '29791574.9576*1451656320', '24142762.2775*1487818374', '68966509.9291*1478101007', '21177856.4307*1486630350', '18890278.2914*1481332470', '52831068.4879*1476933990', '91783782.1164*1460698066', '68340271.4933*1465080690', '86534454.827*1458549363', '53682571.3346*1452604122', '53588777.3588*1487543037', '52737714.4453*1487659583', '71461761.0017*1482627083', '64081989.168*1464232623', '60095160.281*1453025083', '70445791.9261*1462111958', '52964063.0693*1456952139', '46092170.6371*1469810113', '75291194.0163*1460698353', '50271754.4365*1461145974', '86255404.8665*1480271665', '36377132.9959*1472651365', '40809422.6476*1472208688', '77898136.652*1471425942', '22194765.1076*1468644483', '17046014.402*1458826916', '71162279.554*1455665402', '31754495.1107*1452917783', '74561917.1203*1468460809', '34510097.7423*1460138611', '94071317.7885*1456846084', '81726533.1602*1483644426', '35186980.5998*1463471190', '67952348.9504*1464150151', '74009794.225*1471153964', '81205296.8951*1472519446', '92595120.0188*1454694213', '88085579.9751*1471018794', '23536811.513*1467104105', '90291973.3306*1482428479', '25689463.0043*1460520460', '93238534.7534*1487365740', '93371966.8292*1484781711', '74201791.3428*1467829609', '39052214.9197*1470247804', '96716361.1666*1478265450', '33307012.2897*1483286160', '65991207.9335*1453103934', '33710637.0069*1465795825', '91280269.699*1453629550', '82034001.2513*1457359893', '89833021.1555*1475937481', '26862847.1159*1463564700', '89591683.4991*1453427992', '71178846.7049*1474420426', '30801647.3032*1475589258', '48423168.6373*1456794960', '30111365.2455*1487681453', '53375334.537*1465849725', '31505950.1368*1457352186', '78870191.8252*1456366084', '33825819.1865*1468719495', '17465756.9777*1467473257', '56394288.7734*1476711830', '65156586.9841*1456168944', '52973957.0074*1468136765', '46134495.5742*1472549963', '68135282.615*1454193444', '8708498.74946*1455475665', '18335118.556*1466647226', '895927.17305*1476677847', '87313337.4992*1481176933', '78799879.9494*1453860780', '21179502.0889*1454930090', '50003188.2633*1465936705', '72268605.4126*1455053009', '46480750.3339*1480462336', '13499982.1854*1481233819', '56822454.0496*1485962144', '72837308.3938*1458339046', '48998781.5615*1460196933', '65341561.209*1465273758', '22563996.1981*1451687279', '17384285.4312*1463214331', '11884402.3945*1465146307', '26380767.146*1454518723', '43336752.3458*1453479488', '94019194.8247*1484555386', '20484011.6217*1470556661', '34138743.912*1473169342', '36409078.7854*1459653585', '51883951.3099*1452282201', '87598806.3534*1451714043', '28025801.8071*1478945537', '68135282.615*1454193444', '71428724.3455*1459894383', '76326657.9892*1461698419', '8924369.51805*1459449075', '43438521.5088*1464604014', '7082008.47851*1486660994', '17576220.8524*1481317148', '30216070.1019*1453679448', '61437030.8316*1470347962', '97926405.7388*1457013670', '85905247.9764*1470378213', '27810597.9145*1464150149', '6370360.04086*1476022204', '6213503.94856*1464606992', '10778216.7058*1457386437', '78367750.626*1460353341', '7188202.80812*1470679262', '16378519.2587*1477928033', '63555884.6037*1479427490', '15701603.9526*1472018269', '69822650.2022*1460187328', '98729060.7652*1484599672', '171476.966266*1476802829', '42158678.5233*1483835663', '54586842.0218*1476005244', '4541049.36342*1485831844', '92969132.4474*1462170302', '67013884.3971*1478295247', '36988948.4137*1475204260', '50380756.1121*1483276899', '6246151.78867*1467143976', '77846189.6184*1485377691', '91442494.7443*1477024568', '87343715.8341*1452755328', '81428661.4017*1475949489', '94708823.5477*1488146738', '27560961.4831*1476364709', '83848568.5025*1463165993', '97233970.0526*1455819862', '75067400.5447*1467115887', '14368691.6309*1486121709', '86942220.6194*1475125319', '3175879.0992*1473425727', '49830424.6478*1463977294', '56710496.323*1480981817', '92168571.8803*1456414509', '3556768.17116*1482265166', '10518154.648*1459657620', '8028536.74768*1479287795', '95991054.8936*1472155074', '30211066.8193*1452140012', '67204395.5984*1483750216', '4362556.22279*1463963800', '97772767.6757*1461862978', '58467145.7853*1469122248', '62969380.3886*1482418183', '78918472.1495*1461990798', '20515227.9729*1468883686', '83769212.4953*1456196338', '46442877.7342*1460928483', '39046325.2514*1455168260', '73289821.7599*1464753577', '32136925.8731*1455165246', '62062536.4125*1483221382', '47538897.8551*1469794798', '92556749.1694*1466234067', '36128012.0099*1464397334', '8948424.87601*1453977919', '1055490.98035*1457569913', '94648926.2195*1472549228', '91358472.6477*1487670782', '99947728.0863*1468974940', '94258284.0288*1474273337', '80264094.2683*1470236522', '30825818.3896*1486455275', '81324703.6758*1470763368', '81691732.7176*1454140185', '69374552.3588*1467151751', '47050086.607*1456932019', '70919292.9195*1455376717', '20930586.7466*1454272886', '73044063.9856*1463269853', '24868172.6753*1466332912', '1237620.57927*1472591183', '23264808.2538*1482982241', '90298711.9977*1477189363', '99180488.6306*1457753862', '59016621.3277*1482083608', '79163968.4121*1457842306', '79490874.9943*1466335431', '90298711.9977*1477189363', '13636301.3551*1474528207', '87598806.3534*1451714043', '63057359.9299*1484991777', '6960658.47534*1467713068', '10312310.7752*1478203714', '1130425.84449*1466954989', '26449333.1939*1458460270', '31496576.9687*1452010591', '22517648.4138*1451763638', '42591545.6313*1453390947', '36292850.4577*1456729774', '57666692.5638*1473183959', '13816900.8166*1481661548', '51291226.1971*1485398218', '86724086.882*1469156433', '34050415.5635*1471846695', '46737083.8311*1484221767', '54985955.0573*1473153365', '42044262.8489*1486598989', '51781270.48*1467463553', '22229488.8032*1456185892', '34477355.2618*1462041061', '38393400.7466*1470626814', '76772344.6105*1453420059', '96011923.2963*1485124072', '23090065.5249*1469051591', '59240625.5853*1483182264', '68789131.9641*1465339603', '46554143.0351*1471913897', '48108394.7829*1473941822', '6131351.37928*1472216669', '75642316.7987*1460543490', '70178338.8564*1482331073', '14457503.4808*1482959404', '17702290.064*1480502534', '96926110.3586*1477682484', '23936209.4933*1478279161', '566581.206043*1482562775', '45940806.8599*1454059035', '88005943.4926*1479690036', '80446466.5985*1479401954', '24096496.3395*1475062797', '89833021.1555*1475937481', '30248273.4369*1479149677', '17580442.2343*1474132289', '91758211.7684*1487271551', '20350335.7336*1475580754', '74201791.3428*1467829609', '2931432.40423*1452909057', '42166954.1981*1477374676', '47831529.6051*1455601472', '81506427.0402*1455568641', '93856249.7262*1477938803', '78448563.1836*1467981874', '125561.724059*1468097172', '22314002.0921*1488315513', '69539487.4695*1484362925', '17838557.6266*1455560659', '17443602.4043*1458065987', '82032710.1256*1485128824', '91967288.0375*1477684899', '77153822.7913*1465856667', '16390099.9199*1476917684', '89545239.7112*1475174877', '19897413.5274*1452865700', '38715024.2424*1453103020', '41356862.6315*1482072074', '57349385.8207*1478463401', '34930884.6836*1484792554', '63325231.0665*1472062498', '16326626.4045*1462156032', '5570453.65258*1481063214', '61274264.3395*1472426619', '9222655.84307*1481932993', '65068072.2688*1481250242', '7122116.56421*1473456449', '51795715.1239*1451697649', '94261840.1925*1460995145', '85227232.5957*1480675497', '91461889.0677*1478107705', '57246570.2858*1458543585', '33803923.7592*1454557491', '53970746.2977*1480337951', '7188202.80812*1470679262', '38927933.6963*1476517885', '15575065.7189*1486307058', '42194201.2839*1474800592', '44836827.3122*1456669739', '69545361.4051*1469427392', '14513212.2016*1456179693', '534105.792209*1467480272', '64976631.1487*1480982472', '36874535.7069*1465637699', '29411408.1035*1467346110', '28250815.9881*1460324653', '40159519.7073*1455676185', '23800893.1403*1457151062', '67678523.2841*1457359735', '21431025.3062*1481287810', '32402354.5216*1465447137', '58678200.1232*1457412365', '52128745.1394*1479969451', '95991054.8936*1472155074', '95305129.0097*1461465251', '27056111.9274*1455393975', '61274264.3395*1472426619', '40752545.0836*1482285190', '21534448.9906*1465626316', '73423657.2545*1456231595', '26505422.4912*1482749229', '97637275.9462*1483138205', '90299378.7291*1455882097', '83728703.5336*1460019039', '38088574.5678*1479203847', '25946433.295*1473020118', '74012479.8344*1480612184', '56049370.9408*1462727976', '70645219.4427*1456397849', '57010128.2406*1458239598', '12223926.3853*1462930618', '65558510.0686*1456835097', '81681856.6024*1454142069', '23411148.5492*1467211156', '42664882.0276*1471567235', '77598682.8298*1459001263', '56846179.6477*1464431852', '71063124.7869*1484263075', '10520259.9925*1466968473', '62384785.209*1453332112', '91843278.6796*1451623350', '71323611.0437*1465835684', '77884472.8285*1481139265', '66437080.7573*1464684937', '88405266.1543*1458235126', '9504975.44349*1464386012', '73191665.4191*1468049263', '25044059.3723*1481730250', '13472699.4583*1469325649', '70541694.0283*1475507466', '29534182.261*1456290271', '46221872.9585*1459756566', '84390510.1798*1485348917', '48168724.7956*1468315293', '58949739.5799*1462306541', '49566366.3969*1480670841', '52699181.5756*1461386614', '5400190.93326*1467093514', '15325487.431*1460431834', '64616180.6486*1464263098', '69084602.5143*1483965834', '30261722.9407*1455148304', '93267936.8825*1452219471', '30038415.2283*1476463758', '42044262.8489*1486598989', '25311409.255*1486697368', '71602288.2544*1458434775', '9857972.24691*1487599699', '4292226.57336*1458660491', '2836514.31813*1454232840', '11884402.3945*1465146307', '30875776.4612*1470133803', '3133432.82296*1486476520', '64278631.3398*1466889394', '56056587.0382*1464687961', '20004068.8011*1477604320', '29652577.9264*1481707161', '6252283.15278*1478190567', '69573228.0982*1487326543', '55084550.9971*1465572790', '24825812.6095*1477348152', '87455478.2841*1485440375', '10214143.4856*1463051693', '52451537.4507*1470049483', '30782661.8984*1464179294', '31354067.1225*1468423999', '7149703.13025*1454083118', '80969154.3432*1458537387', '18311987.5989*1466076729', '77459474.7603*1481489311', '77707798.2391*1454253916', '19182041.4678*1485492231', '57030442.2758*1468765411', '78448563.1836*1467981874', '76105027.7411*1476568422', '15354103.4245*1468311941', '87205009.4745*1467674568', '39172964.5789*1457167098', '92784707.1199*1461756187', '77913954.3294*1459163721', '32733710.3237*1473634263', '18335393.5258*1478134375', '42521099.4914*1483624824', '89312754.5521*1469548175', '62740356.7586*1457405839', '86884762.7243*1471496234', '52929332.848*1483951242', '13285533.3153*1475356985', '31979153.7999*1484790228', '23141915.7174*1461314490', '83219411.4043*1454253293', '77788201.1379*1484026837', '51246711.4339*1484169969', '5786813.78206*1478561880', '57163019.3756*1464169904', '86675014.1948*1460441625', '29469531.3504*1473159829', '84753710.6293*1462667403', '73323135.5413*1478831086', '16104338.0131*1463721299', '86018890.0741*1458816405', '21639639.4327*1485688176', '7702824.90663*1454815321', '12829156.1013*1470009401', '79856708.2043*1477111612', '73273816.2432*1488183475', '5431705.73994*1483662118', '68530905.8759*1476044709', '1252168.6584*1483586745', '99528021.2764*1483150748', '5882688.07033*1478718238', '68407216.1804*1456271288', '55273165.5488*1473745314', '39197605.7411*1481278603', '95659788.9923*1465221086', '71981161.985*1452837255', '62693902.0171*1478473945', '15892172.292*1471795249', '44828311.9331*1451625058', '14354577.4253*1455026840', '24617447.6542*1467807964', '55246215.2718*1478179698', '88420379.1491*1483028967', '10401219.6262*1460253323', '86558968.9784*1483171159', '94943476.6081*1463690303', '69151503.2508*1486231167', '11307447.0447*1472523561', '26470597.5989*1453273281', '15956385.9306*1484181511', '20776271.5397*1460362291', '23531588.2666*1463055446', '78025448.3948*1456787433', '37622110.9655*1487658331', '61424188.5626*1451986832', '11319738.3554*1465420537', '50003188.2633*1465936705', '11366025.5491*1461770562', '42671635.9674*1470977070', '4500190.21704*1473902251', '15575065.7189*1486307058', '58106258.7239*1463970024', '80868518.686*1455432070', '59534679.4851*1470537888', '34867856.0203*1477964708', '97517584.9495*1460228903', '43733755.61*1457347854', '85644875.3009*1454229565', '99829361.8732*1463099440', '86533036.4385*1468951769', '80437198.743*1480985415', '57477665.8582*1471512419', '5661716.29983*1460612549', '20763430.979*1453794435', '95129933.4484*1456551086', '57780001.2574*1462111884', '76861128.9644*1487079762', '43186165.0877*1480064490', '9671856.36863*1461397871', '53649211.7666*1488031693', '58725016.0822*1455061213', '10214143.4856*1463051693', '97061262.977*1458904883', '20068784.399*1469021303', '88286877.015*1483381178', '22283919.0268*1483015754', '69306661.4397*1456616765', '50944207.2217*1484283567', '44276942.6316*1483948730', '37795467.7574*1463420788', '77571733.2991*1476714513', '85704871.1417*1458510507', '79183884.4133*1461486147', '16302515.465*1468278403', '96125772.0943*1466473272', '14869408.3041*1478091117', '2540423.1724*1463036487', '96640820.2503*1460152703', '65357267.5075*1482240083', '41102871.8797*1471628037', '71801482.2536*1470009634', '66263607.233*1472542451', '13532903.8859*1465479522', '29236700.7742*1460087243', '76125426.9766*1454131152', '37366971.8287*1464296678', '60585089.6826*1474998403', '37830167.7695*1469487289', '76615461.4784*1473841236', '8884189.13226*1455632296', '37406639.7165*1462993430', '96026195.5192*1464833181', '7055704.50605*1455531621', '33677785.467*1453701652', '9831151.71078*1463805405', '4175457.45018*1472831721', '67357932.9323*1475059924', '82776531.7746*1478516250', '18294269.8967*1473322569', '93446335.318*1460130671', '96126421.4483*1470062246', '58192750.3113*1477079136', '78157313.2724*1476664006', '33360745.3988*1467847757', '29063360.1217*1468060836', '46117766.82*1479366260', '68428738.6529*1479937899', '95709674.6959*1452618393', '97677596.3561*1486427867', '38036245.4754*1487849816', '70623844.7249*1472591222', '97582591.9106*1477664883', '98301430.0671*1487605614', '84675608.7468*1459363878', '98824169.07*1479868080', '15230749.8508*1487114692', '16391776.9224*1452567830', '39137007.1092*1485309068', '10214143.4856*1463051693', '37407172.3211*1484662929', '51674728.5157*1453507345', '64525619.8455*1465999176', '66966603.1677*1486410287', '9052563.50401*1481643167', '23270783.0132*1457075383', '25039762.2951*1459714071', '61402843.0569*1473237412', '28909540.0318*1461318124', '28378832.3888*1462025349', '14355691.7449*1478519794', '67757221.7984*1462696179', '69174077.7102*1474898079', '73404415.0787*1476453778', '14666684.2376*1484101503', '32205876.6411*1479022012', '86654383.0933*1462489035', '50926697.4876*1464211384', '77400376.4954*1468460513', '19389257.9364*1465764191', '70215158.8144*1451614183', '13670894.6168*1483791040', '22920776.7766*1471402615', '40783179.3707*1455237667', '90755000.4123*1479616945', '73013170.2075*1459307565', '34663564.6018*1461843623', '13691837.1803*1459616539', '25564426.5791*1482210470', '64518153.0709*1470423797', '3327476.37007*1463553054', '55842669.2049*1476572881', '84112886.1616*1479245744', '30614129.3933*1463722533', '53222166.4527*1458019989', '13336502.2757*1479515537', '89487634.5995*1485951456', '23842137.203*1470785443', '3565981.41163*1455116091', '28018909.3577*1456269742', '59970227.1625*1485124025', '64550982.7546*1455765648', '89487634.5995*1485951456', '62418857.3121*1467438290', '93232542.9594*1474710237', '47921801.6018*1452142949', '54366814.3386*1456672387', '2004110.29356*1479939874', '26523794.5413*1456264069', '9068029.8295*1480618639', '962404.196499*1455575812', '94560268.9929*1459862094', '91467279.1417*1463654391', '30755615.6058*1480169582', '1025711.07779*1470025692', '797727.845526*1481025861', '94314768.4552*1472307251', '60241255.7244*1466358268', '31244161.0159*1487734413', '20506511.2528*1453212741', '70314756.5984*1472277075', '93636937.3579*1451813728', '10401219.6262*1460253323', '37962680.8617*1454781463', '34397898.8385*1486733141', '12511036.4686*1464679990', '49032020.7316*1472584971', '992387.345545*1463098826', '39800811.6178*1474984866', '8467771.28221*1483827184', '95381983.7623*1471398462', '49159775.8003*1454007716', '49904715.9065*1459178076', '14090904.2999*1456486196', '25130300.3325*1455063908', '33059632.2353*1487884387', '80553770.3436*1454082444', '23469867.7692*1464203455', '79867781.7952*1460948113', '40772978.2288*1467657922', '22314002.0921*1488315513', '41098483.989*1480489668', '15046758.6854*1464570677', '73807495.7207*1461837730', '13636301.3551*1474528207', '44372742.3412*1486561877', '3478912.99404*1473831707', '4543805.18693*1454016298', '38303590.5187*1466850127', '96409707.5447*1462693370', '69084602.5143*1483965834', '20652016.7285*1483472562', '12288023.0052*1476829288', '33316046.1445*1482188645', '39175958.1057*1457131177', '67198275.8109*1481765992', '57434551.8978*1487871526', '52218165.5695*1477404899', '33819678.6658*1478092129', '51674728.5157*1453507345', '66179064.5325*1478430725', '24052987.3219*1476396710', '45060579.7096*1486166613', '8261608.29676*1478403373', '80491273.7164*1462617888', '35106432.1542*1484472696', '59448674.3604*1481312848', '69507001.327*1471554105', '87093922.9067*1474944730', '96332026.0789*1469576718', '27156651.6106*1460002480', '37949571.2983*1463172567', '52282332.0926*1486494234', '56544256.1292*1480615067', '60764753.8167*1488118281', '88748351.9932*1459826717', '50317370.6489*1479244225', '9630699.41627*1470693697', '32573814.3832*1479150644', '13968903.3822*1467880453', '72380116.593*1459145455', '20746728.7236*1472117190', '13027258.5856*1459213833', '75630644.6229*1487100226', '89691315.2889*1474029032', '68295054.6447*1470928754', '15069167.8473*1481068517', '43671154.916*1460650230', '58467145.7853*1469122248', '44914761.1394*1470219653', '90388741.7476*1462128933', '55508043.0089*1487940164', '61660657.7218*1485095680', '51800757.3529*1483648659', '50110125.9861*1487581696', '93121282.2817*1485189172', '33030572.692*1454708827', '28580269.8711*1458295554', '48473016.8523*1474697295', '68689211.2212*1462338390', '44524327.2014*1483147340', '42157319.8109*1482997531', '78389595.7491*1477733736', '90231388.6765*1466638115', '90701040.1403*1487694175', '73308349.526*1469510173', '50244351.3964*1465729649', '81785156.5256*1485162481', '25729414.7743*1463363680', '59104133.4656*1456599986', '38025867.0706*1459124234', '839759.974617*1451646892', '96598476.2082*1457037310', '1055490.98035*1457569913', '8272221.59776*1457866259', '49243400.8132*1477478427', '65383064.9538*1454529324', '12149652.5474*1478992492', '69252403.3581*1476234267', '27344610.8294*1477957353', '19834654.819*1476405687', '19204888.8946*1485220778', '61363836.3338*1483990167', '13186521.6708*1466499737', '26105526.7651*1470238946', '68113166.4446*1453517484', '40668601.5994*1465967414', '4153441.9336*1478357746', '93586261.5898*1480131541', '19790837.5039*1477035880', '34591904.3556*1469363311', '91484381.5152*1476574272', '16171331.4921*1464888282', '30875761.3622*1486896886', '39794876.0315*1467605644', '1436482.0342*1484104147', '74695132.1326*1482226048', '81033765.432*1452343556', '69399122.1921*1486586867', '11702020.6879*1461124077', '32230957.6976*1478941079', '33159691.4497*1478368608', '9068029.8295*1480618639', '91783782.1164*1460698066', '89281716.5939*1459419476', '94223287.5714*1464837393', '27605214.4995*1485838837', '60818157.1194*1456610511', '70919292.9195*1455376717', '69090096.9834*1486602269', '30444160.4148*1477918033', '15975555.3128*1480421322', '18146674.8834*1465695904', '48759930.2098*1482404482', '90963319.3097*1456509414', '71389296.6306*1472742490', '64455862.277*1482206548', '75649356.4412*1485425870', '1561778.40291*1456288436', '96104648.4282*1485324800', '94584273.6265*1468360026', '12083814.488*1463303122', '68319532.4129*1463625173', '71046454.888*1458611875', '95508133.4689*1480455767', '59519963.9924*1457208464', '37967377.2737*1461063265', '1348992.32678*1460636111', '42899408.762*1451657744', '49105672.4756*1456437330', '407821.492278*1486688572', '66191547.1227*1467723817', '17623061.286*1453298606', '9414311.72829*1487740810', '91055408.935*1473269183', '89340694.7315*1468890786', '34473874.3141*1466065296', '60502008.8103*1479078418', '13999978.661*1474014172', '94199560.4246*1474173098', '86238189.7648*1470529849', '43168702.7335*1459930118', '40531780.0855*1479076145', '88208803.8461*1473526705', '22509120.5578*1467536479', '992387.345545*1463098826', '14909127.1698*1472963864', '58788245.497*1482056186', '62516710.3785*1484657062', '49946130.9616*1473681219', '75184580.8272*1478327486', '51289572.7001*1460267142', '34736555.3639*1483295223', '50699992.8023*1482306040', '17963252.1347*1463512313', '12368704.3*1487427072', '9252002.9793*1465936332', '16968168.1792*1470564684', '42745307.0218*1485717540', '39717941.0454*1473861890', '41741112.8603*1477913894', '53719929.8541*1466093500', '90572459.3081*1469165897', '77082947.4708*1451684342', '60227213.7905*1467766007', '79366224.2465*1487828972', '65980083.5639*1463895517', '13737191.7201*1487974102', '30264690.4889*1486885865', '99395070.6665*1470642767', '45089620.5243*1462479999', '44287245.5198*1474924757', '7256928.32997*1451972489', '76472162.0902*1457377288', '52133195.019*1472405189', '68751079.005*1457863269', '67587262.8353*1467298131', '56063298.3368*1484821724', '44216482.4641*1481790426', '79246807.6238*1468105565', '18002838.8707*1473568251', '50540902.4797*1463115990', '28242433.7322*1465248810', '52328751.8989*1458728175', '32680146.902*1453899785', '70168359.6239*1468156206', '70969232.752*1461872958', '67392083.5001*1467573961', '3033092.17796*1485310854', '75915357.1116*1461056062', '17045191.029*1472612462', '30476571.3183*1455436764', '46716752.4136*1464744879', '46172127.2233*1466774039', '94315606.1058*1462713368', '39712716.6435*1463519243', '87204741.3238*1481161428', '3226734.9118*1471016537', '96714678.2599*1464413990', '50824752.8288*1476128457', '8607408.59647*1478583939', '95370760.5364*1455634429', '84882673.3238*1474489432', '52671785.7798*1466718775', '71344276.3981*1486425650', '44961508.9281*1459143162', '6642316.36418*1474638831', '65767165.2458*1476297236', '19878112.6554*1461079346', '97422200.5022*1471410637', '56587949.6985*1477781532', '77355442.1669*1454808224', '74142050.305*1485643310', '45315342.3434*1465225661', '75057216.4553*1453918507', '95131144.8965*1453297000', '33295320.0304*1461877639', '90583065.5227*1477437329', '72627297.4369*1457760321', '69521383.0521*1471319174', '70031691.17*1466445364', '13336502.2757*1479515537', '71929462.4762*1456228646', '990999.230095*1464540588', '74566869.5402*1456093211', '94184513.0203*1470270544', '36858583.3293*1451897630', '27275349.172*1474528264', '79246807.6238*1468105565', '69237645.8673*1464593209', '67955318.5466*1485818908', '72671497.8065*1467608845', '31218774.2341*1472701619', '81406121.3902*1478147432', '66643684.8769*1473096725', '58717120.0819*1469903839', '85704871.1417*1458510507', '81055941.805*1483042546', '68121445.3491*1485667155', '25562272.9316*1471375318', '23029615.1577*1479990040', '67432467.1902*1467236730', '87001276.2619*1479143874', '18879471.9288*1468168585', '51589903.9078*1466730208', '14834543.4846*1467692082', '31875634.7894*1459535669', '47191246.7678*1476153002', '61663765.311*1466666337', '24283954.5595*1468200310', '81801475.8099*1462070113', '12714363.3302*1470508096', '13186521.6708*1466499737', '9238269.40088*1477424812', '90308407.6947*1483999708', '81950027.4496*1485717124', '26008135.8061*1475111063', '22049700.8239*1466451361', '61294164.446*1470493409', '65280131.9751*1474758670', '82317929.6206*1478586028', '55597068.9813*1479605005', '54264812.0733*1464377667', '14066359.8228*1464400677', '15804655.1536*1479012195', '50784994.6158*1456440893', '2374168.26026*1472067351', '56066986.6048*1475207693', '51571131.4199*1454591444', '47532721.3778*1472767056', '19498931.6083*1465221631', '485431.753118*1482439322', '56678279.993*1460902548', '63721987.3563*1482043202', '85503879.7109*1469074102', '36760606.8587*1473179288', '13591485.9751*1482530694', '75928387.5569*1462595594', '43296258.0717*1454710290', '77355442.1669*1454808224', '77234451.4878*1474596388', '75649356.4412*1485425870', '33659304.5772*1470566260', '80443615.3292*1453305248', '77815547.9263*1474347203', '72087142.6106*1470275318', '96537698.7096*1451837092', '30664447.4373*1459533105', '47444159.8069*1472253643', '22357794.2639*1455453508', '14502772.5131*1465842707', '95381983.7623*1471398462', '40113562.3589*1472111541', '21625225.9297*1465985955', '60628947.0048*1481652222', '74610127.6064*1454751380', '68906809.4525*1480688398', '38691314.4358*1461504780', '44720971.8641*1454179603', '31884781.6467*1478092957', '62223406.3373*1465436073', '17929309.1867*1486133865', '99427503.7312*1454603259', '55193213.9602*1476855810', '33828456.3348*1459775304', '80831776.1632*1470868847', '27824272.1025*1484074633', '27145843.1427*1474659550', '11083542.1658*1470353331', '50723345.6296*1452041765', '15892587.9495*1462699158', '64495701.867*1468544171', '27275349.172*1474528264', '43553831.5394*1477208853', '1214648.1344*1487966528', '18916221.4766*1485201743', '80531289.0185*1454275613', '36578415.7872*1471886048', '95573588.5698*1461750749', '16166031.5709*1463755986', '14504774.2379*1487499866', '84564906.436*1453353213', '33145381.5503*1468081706', '98283931.1798*1454617449', '84388856.6081*1481010748', '93189522.8806*1473892509', '92424283.6836*1474460213', '55694947.9248*1465969167', '50406593.3627*1467800837', '36498784.9902*1468416721', '55365562.5196*1454966454', '30691412.3384*1457455848', '22603775.1743*1474018734', '6920481.04538*1484329887', '56274660.1362*1475708832', '60285262.7996*1475761391', '92935829.2541*1472089377', '26777577.8965*1456287900', '10928229.7003*1479867547', '47681106.1069*1487588543', '49358421.9064*1473182148', '62925399.605*1453727410', '81399396.2655*1460151695', '92965475.829*1468818428', '81809064.4565*1471818290', '28685527.7061*1458295437', '52054359.4987*1468402935', '77475207.7153*1456887144', '96026195.5192*1464833181', '91376500.1419*1476799235', '96981572.7175*1472322032', '80889885.9182*1470645188', '66817407.0338*1478356003', '70118465.4189*1456223773', '30860489.3042*1465384724', '96127671.6319*1462255132', '46395733.8002*1477562953', '25567800.9331*1465850078', '55172083.9645*1488146854', '63723182.8191*1467962861', '29647535.7858*1482639854', '40206592.2554*1462446261', '30045803.5659*1464094076', '84140146.1695*1461831815', '3497701.13011*1460282414', '54986404.917*1453895459', '53964139.4837*1462955656', '20134472.6088*1456356935', '92408710.4905*1452665946', '88457968.5907*1487912948', '79896160.5546*1464208480', '6160431.14501*1484867840', '76266584.9469*1461910420', '12114500.9015*1463939659', '85644357.1475*1483402427', '62869041.8205*1474989384', '73229180.875*1475054731', '28023811.6779*1470111782', '76123656.5135*1477299557', '45505842.6991*1466622705', '80491273.7164*1462617888', '77087629.3271*1452965865', '99777541.5359*1470421977', '8995038.83265*1475865470', '85343812.271*1475444036', '69151503.2508*1486231167', '34547959.6047*1468151560', '95918035.1672*1456884796', '14799076.4763*1471782750', '37617903.5641*1478357227', '1404955.02153*1463354901', '30421739.156*1467483480', '33215110.1056*1466948404', '9585225.75614*1478752055', '2353910.42458*1484595730', '1604731.21417*1466221684', '71162279.554*1455665402', '10964419.4923*1475774471', '28652040.3775*1461786443', '76081228.995*1472389759', '78805267.4246*1483629383', '61436484.617*1484245117', '7788068.46232*1463971628', '52000353.2394*1461117910', '87642959.151*1485119692', '84621335.6679*1478321187', '49119335.8234*1457424725', '90554979.8163*1461016480', '5786529.71847*1472034401', '85792134.0144*1461581220', '1986764.2586*1454024678', '3694514.27684*1480212333', '50659975.0518*1457838188', '77528142.7352*1473454468', '19046739.0161*1479849263', '90439542.754*1485355964', '57691183.262*1456408037', '81032254.2042*1479543640', '83341467.447*1468036345', '8030103.96169*1467044098', '96742569.0861*1480195103', '65558510.0686*1456835097', '23990902.4197*1469858062', '65429337.5416*1477220516', '28937361.3952*1482819012', '16587110.6177*1485109294', '82168230.0448*1452939253', '3178191.22029*1482295243', '58929264.298*1467232120', '4050045.69112*1476518634', '39294623.662*1463278510', '51720764.3851*1470908639', '59319282.2529*1467883843', '41545883.3225*1460842041', '40752545.0836*1482285190', '33121189.6289*1462896748', '96464860.773*1486953369', '29456124.6061*1481546863', '8887265.05104*1467570445', '89666329.2277*1485667426', '88222682.1238*1463771383', '20384779.3608*1487230867', '45409533.931*1454356942', '22057539.6282*1459287627', '2685114.05225*1478839517', '52218165.5695*1477404899', '36959277.9487*1471474677', '94691173.2452*1477341785', '64123185.0064*1457957803', '59119989.0687*1480025182', '79485996.6715*1487560166', '10321151.903*1454376407', '33125077.1307*1486341583', '70428203.5928*1461896003', '26606885.4645*1484859946', '45490671.1981*1483090011', '86015072.7184*1463377894', '11865987.7546*1486864833', '55791798.3894*1483159616', '97858581.202*1482409439', '82146055.9136*1466629490', '1307133.41965*1466384720', '9592191.68676*1477629062', '44663440.4598*1473766712', '33755995.6557*1471640949', '42605973.4433*1470326622', '71137259.6999*1452677206', '39943972.376*1463404971', '11042395.624*1482676335', '9412205.42725*1463339472', '96287246.7587*1475177777', '69843241.889*1479863578', '54535878.5164*1478166202', '61058921.8077*1460599904', '81034250.0545*1479995668', '22383358.1074*1465579185', '20534133.0404*1487003549', '59474356.8122*1459222319', '10323572.1102*1479730465', '16695764.3789*1477844230', '96857261.5931*1487969349', '38469310.2181*1463858754', '5337795.0957*1469948456', '72331641.6076*1481980143', '76905028.3021*1458159718', '10623106.792*1464485697', '28571541.6915*1460440296', '735065.043466*1484253457', '85525066.5906*1458277134', '66809286.6822*1470012664', '73843686.2638*1455322560', '77567345.7389*1473081961', '41177819.4387*1487343334', '27441160.2608*1464965095', '59549954.775*1481911514', '99914887.9788*1454889017', '4668267.4347*1476567111', '31425237.6693*1476433083', '65341561.209*1465273758', '13213881.4676*1462473378', '85430569.4661*1451910908', '2075678.25453*1452142954', '20996198.8834*1486583266', '82600181.1896*1469035953', '71428413.9596*1467006274', '15993649.0671*1475459511', '30229504.2316*1486752990', '5897169.7524*1475050191', '15805751.072*1459417180', '51619777.5013*1456842380', '88965137.7303*1459309912', '18964366.6582*1478884407', '61070995.5483*1476277247', '34993036.0262*1453912891', '55288156.0169*1477260710', '32170450.3157*1485803845', '32867193.4971*1463672592', '53276151.9364*1476081337', '10140604.9987*1462332975', '54257515.4436*1451710949', '71939394.0061*1470319346', '44329461.1557*1461516477', '62303673.5597*1478440209', '1604731.21417*1466221684', '83798489.9329*1465256763', '27962380.0423*1473604704', '13984716.5678*1463148023', '64675882.7971*1476041800', '74610127.6064*1454751380', '43341921.8322*1478019639', '94712707.2277*1461203057', '18327800.6744*1460605060', '82724071.3556*1462937136', '51282230.9242*1462441787', '40026356.6197*1459207091', '97945109.9404*1475276820', '72772427.1889*1486226049', '97844176.3257*1466886648', '76535996.3732*1488214194', '55238157.7659*1466153017', '49854040.6956*1485943843', '99506521.2809*1457607792', '77183738.0327*1470347623', '32057454.2943*1455446227', '85872182.6465*1475459910', '16633846.6168*1477337091', '48679143.4054*1486979981', '16568256.894*1464732316', '98984736.3737*1454791437', '84793006.214*1457065528', '34589863.0221*1487863655', '20693559.6543*1475820702', '41562577.203*1456876285', '73302732.4956*1456472516', '6528434.91826*1471972850', '40937793.7074*1478275253', '50737306.0299*1468423853', '33316046.1445*1482188645', '43648867.8344*1472250069', '3133432.82296*1486476520', '126287.563911*1487022034', '77739341.3084*1486206672', '24207954.9982*1465834382', '11485899.7022*1455171344', '56599680.1491*1479986056', '68619024.519*1461962804', '36117959.9652*1459825417', '13024300.6596*1453482876', '6475787.10355*1485204516', '77528142.7352*1473454468', '99293486.9266*1459419493', '62407701.2144*1467283468', '40459261.5349*1479412764', '85528249.972*1482040247', '82393774.1003*1476357211', '35434883.1744*1486324101', '91339123.184*1476122333', '30301569.7488*1471261666', '44233733.0939*1453689175', '85315533.1976*1466639260', '92793395.6547*1454831361', '48458181.9545*1481595789', '74784316.203*1475416352', '72011792.8853*1476017034', '46157702.2439*1458558092', '34706732.0391*1474589939', '61255858.2041*1471087427', '47675445.4685*1476932646', '84882673.3238*1474489432', '85788590.6664*1470056208', '7260247.71761*1475639499', '47743148.6536*1473753969', '99513084.9405*1463440328', '96372130.9979*1476935974', '37973136.6478*1475312218', '7498315.74911*1470921242', '39309003.5022*1480345416', '61191087.6483*1463923947', '24553644.8587*1471265968', '92898032.7655*1472472376', '72215067.2093*1475004112', '78671451.4181*1455241006', '71353019.6742*1468268706', '45284947.8599*1454234791', '9245693.83073*1479161467', '35493557.6612*1481862316', '68407216.1804*1456271288', '73000463.882*1475877035', '24075096.7672*1460940956', '58796057.861*1476779810', '80723045.6058*1460305469', '42903796.5098*1457105355', '46611061.9256*1455948503', '59562356.0049*1457946056', '33973142.4102*1486469301', '87721710.0998*1459876146', '88784958.0745*1474017427', '30374646.2081*1453671943', '92465017.1857*1454500577', '66050865.6264*1467943151', '99278623.9288*1472400376', '13973497.4551*1486302472', '53295789.7105*1475337637', '20347385.6706*1483214021', '19551554.4522*1474195043', '3327476.37007*1463553054', '39525016.4535*1466826213', '25689463.0043*1460520460', '76862171.0664*1480454787', '90048920.9672*1473249521', '12460305.3949*1477708328', '23000816.569*1451798070', '83495912.2542*1482917547', '12534271.7442*1471247811', '42790892.6799*1486083849', '73251531.1296*1454373491', '19118437.1359*1483692105', '64549017.3931*1472512901', '17139092.3295*1479104192', '10145634.4513*1456496268', '31484661.102*1487476702', '95623657.6849*1452799626', '47970028.3559*1480665171', '49879544.6113*1473050078', '66081589.7586*1483033720', '81296075.0558*1487940605', '39249521.6279*1454343810', '47191246.7678*1476153002', '59243299.8328*1472225333', '34930884.6836*1484792554', '83073128.039*1478385162', '24442960.8233*1471021662', '71562110.4579*1483354292', '4702577.09599*1469327045', '67632690.7865*1458022715', '94481622.5907*1468574675', '34388970.176*1466353797', '54517341.9657*1463885590', '71559569.9658*1460121610', '22283919.0268*1483015754', '194122.583459*1464443405', '28484703.7289*1466690116', '24965068.833*1487736761', '95276542.3143*1463268521', '26225298.5562*1486050359', '53062755.2166*1483251800', '18034459.3647*1456101368', '42270927.777*1453774470', '39338356.7451*1479595970', '25407431.8093*1458785663', '39875839.6305*1472264379', '83740641.7222*1475568365', '34439078.762*1461198218', '8821109.32256*1454450413', '58746010.1795*1479076852', '23842137.203*1470785443', '13618688.4522*1457634862', '86382671.1579*1468916983', '22354638.8722*1483457246', '67656980.9516*1462049436', '26449333.1939*1458460270', '42745307.0218*1485717540', '21858367.1994*1472780692', '13470832.0559*1470566519', '67680288.9017*1465097105', '71602920.5652*1454118870', '78464041.4635*1461104848', '25193652.4356*1487083690', '30491901.7747*1451948169', '30789660.014*1452687642', '79801637.1496*1460590908', '28974831.7636*1465649384', '1692986.23968*1471193383', '98990679.609*1464038622', '62976127.7064*1470831718', '98469676.6659*1487350889', '19666812.233*1455712092', '65429337.5416*1477220516', '10192874.1727*1470869224', '89945128.1607*1467931486', '27460610.2293*1455586074', '38892106.8131*1482699821', '90392441.3025*1487685234', '86108181.5665*1477149279', '89679315.6318*1467536119', '358986.940119*1481722944', '71109099.598*1456148146', '23264808.2538*1482982241', '16904137.9613*1482780596', '83697336.1311*1485220498', '99298783.7028*1484009720', '55972032.7243*1456286971', '78110739.858*1455023074', '1629635.01204*1480594653', '75746075.0221*1483601817', '93322545.7181*1460726392', '64615279.8319*1475973112', '55582392.1828*1454318265', '2454355.51588*1457712502', '27892067.7151*1479961203', '67042966.31*1461960789', '14501047.4542*1473586125', '37621768.9646*1479133517', '32602864.9111*1475073509', '16623777.4843*1452167835', '94314768.4552*1472307251', '3045214.75635*1470868989', '94980047.1504*1483602117', '91950385.4058*1483944984', '82908150.4221*1481899694', '73189051.918*1472341708', '16171331.4921*1464888282', '56269874.2094*1471856621', '86779615.5639*1477733893', '19772019.7786*1455878574', '31465755.1533*1473030221', '80889885.9182*1470645188', '3577543.3343*1476845374', '74324428.8399*1474988425', '67198275.8109*1481765992', '31386948.8619*1479273324', '67969979.4002*1482270067', '1049521.37894*1477776120', '27123095.6723*1453218526', '86166657.1988*1464678900', '93244457.1685*1479062203', '82563890.5329*1457813906', '36803654.1234*1454614969', '54669431.4024*1452000769', '71656956.4383*1473584003', '55504708.7102*1465341162', '76679799.1034*1482876227', '24741168.4639*1480494802', '89967798.2734*1460026288', '33684158.441*1456655197', '6776793.38922*1453266315', '43458066.0969*1480160797', '79598582.1989*1462184852', '85968915.55*1465866755', '17702290.064*1480502534', '78151874.533*1485747626', '41342739.5394*1473301889', '1045342.16572*1478584127', '55084968.5937*1477954559', '58608228.7683*1475695941', '56218377.3754*1456008997', '58996771.41*1484503600', '87898047.1535*1478611816', '56066986.6048*1475207693', '53566057.0952*1463998042', '36435373.0094*1481215943', '4543070.88518*1485356689', '77254993.645*1474116999', '63152933.3473*1452794520', '6525548.69821*1483811839', '69378439.318*1466094387', '25864321.7426*1470572004', '38141645.9936*1471548962', '23090065.5249*1469051591', '5882688.07033*1478718238', '11057492.5735*1456379135', '93491214.6043*1475706825', '58336608.2296*1478382978', '64581686.3984*1463980128', '37028287.2045*1454084975', '11516277.0013*1486033074', '37629452.8811*1487226052', '2863307.27284*1485370971', '22357794.2639*1455453508', '36417858.8783*1454198086', '50960382.0993*1483876875', '80674003.3155*1456296943', '48731832.7283*1466279075', '96778017.652*1456310033', '90905861.9445*1486974712', '33878888.121*1466106340', '35239731.2015*1468010718', '74759485.5821*1477524999', '70986965.3377*1488153831', '90755000.4123*1479616945', '80826971.5285*1467383462', '58388068.241*1483401708', '58362486.2747*1453042488', '70782279.1404*1472529656', '64226087.099*1453876393', '87629296.8632*1458631016', '35081123.7059*1478036525', '54272135.1196*1480238513', '87113739.2345*1483346964', '12857231.6598*1486714056', '26466514.4597*1463498939', '5115190.94501*1481258143', '77528788.3288*1471066329', '40729823.6606*1484288549', '4702577.09599*1469327045', '51525646.6639*1480605385', '96394180.8763*1459121386', '41478653.4995*1465288957', '38667797.5696*1470673729', '65966453.2699*1458119550', '15532691.5973*1473446786', '26212514.1982*1472865842', '1423692.39109*1453455358', '44182759.1962*1461323545', '66812446.8559*1467972111', '45628445.0231*1455184640', '95918035.1672*1456884796', '34461855.572*1458124061', '95895347.6685*1470663762', '96030094.4922*1458518659', '52093050.6308*1475438545', '24283954.5595*1468200310', '64341801.8741*1456450030', '47899764.5868*1465640673', '84595004.6099*1454153576', '79314416.5579*1453845989', '48136394.6167*1459125454', '40752421.0325*1452076829', '1335630.74236*1477790786', '64427694.6705*1463636151', '47020331.3636*1456713211', '97023072.0485*1458941797', '4008645.45625*1487945362', '29764667.2719*1474840664', '61726438.8757*1469023605', '50183003.0954*1485798801', '49052055.1287*1482062953', '81880021.0509*1467624688', '58684815.3662*1486338503', '81473291.4891*1459100651', '38393400.7466*1470626814', '43585581.8389*1454811064', '21505101.4374*1475405514', '86654383.0933*1462489035', '3178191.22029*1482295243', '57701685.4911*1488294907', '9788797.74949*1460132211', '68293759.5174*1482825673', '59807021.8631*1479528715', '24066606.4278*1456099742', '54840989.0506*1462052125', '6309288.09219*1469362970', '27546349.2217*1463968490', '98094621.105*1457972463', '43119330.8845*1465921496', '90776627.7327*1462621846', '96114843.3272*1470190398', '71202857.8816*1468645484', '72193510.1749*1480835854', '15359865.7702*1471393585', '4328030.21304*1457747972', '6475094.18952*1474144440', '21475182.4811*1452718442', '78157313.2724*1476664006', '89354139.2179*1484935659', '3074127.47961*1486267631', '98898847.7631*1454283589', '39793695.6012*1482227442', '78546489.1204*1461604252', '99406498.554*1474635681', '10873878.1209*1452356280', '49786967.3368*1476550341', '97164903.2383*1469873551', '26106304.0392*1460892993', '48458181.9545*1481595789', '90048920.9672*1473249521', '27065213.2075*1488310882', '45784169.5293*1460085681', '82034001.2513*1457359893', '41619850.0762*1487580268', '92408710.4905*1452665946', '60616226.197*1475266522', '57767841.3493*1462507021', '10616497.7182*1468284308', '35081123.7059*1478036525', '29277974.0989*1476343777', '96483338.4544*1468766751', '31484661.102*1487476702', '73969980.0515*1461649645', '87465917.7226*1484454672', '56226756.7213*1486671383', '76378460.3227*1466984819', '17786663.4507*1466489747', '65393820.0205*1487073215', '63152933.3473*1452794520', '19423841.8954*1473069986', '46418011.5837*1473807549', '39035123.9012*1482276785', '36637974.4844*1475197880', '93469607.6296*1455388310', '44075562.9727*1463884577', '57968026.6529*1485661134', '18484127.8451*1462779102', '72331641.6076*1481980143', '36659799.8728*1473095166', '44575716.499*1455898140', '49163485.218*1478591291', '69336161.683*1470265753', '35200172.9318*1480850955', '33159691.4497*1478368608', '62408422.0525*1474284947', '95505586.698*1468238016', '98986360.4564*1465657265', '2072196.97026*1453908840', '89545239.7112*1475174877', '99138886.1557*1478339900', '33828456.3348*1459775304', '14894217.6374*1478138994', '68942760.5617*1474300962', '21220006.5086*1459950627', '55056570.3099*1460046738', '58549357.1447*1456252402', '37455763.958*1456769911', '55360389.9344*1460743261', '25864321.7426*1470572004', '67301733.1093*1474172102', '1471222.90682*1455289338', '19215898.2575*1468539516', '80668760.4644*1478493539', '24747390.3417*1465156280', '44265928.5164*1452387967', '91308095.7361*1455638582', '58787277.9422*1479225434', '60604004.4807*1455612649', '64525619.8455*1465999176', '75704437.4107*1481556993', '83972474.7368*1453432440', '67632690.7865*1458022715', '59423852.476*1456003191', '86872377.9566*1484272794', '93060499.8743*1463583325', '58140367.9057*1473088762', '42790892.6799*1486083849', '26288206.8882*1485651368', '47162328.0023*1484893770', '76161070.3306*1464230258', '89487634.5995*1485951456', '35263111.4217*1482173925', '55446558.1647*1476481240', '52570022.0278*1459779761', '17420058.5706*1463712632', '30835574.9947*1479561722', '68610480.1669*1465897402', '8647265.38586*1475711956', '6623500.15734*1478665076', '95354727.8828*1465268587', '45291498.3958*1457985697', '19831918.7357*1463264315', '91725570.3395*1454632043', '48889759.9515*1487171107', '36053793.2832*1478102796', '4669892.60012*1451803874', '765403.48552*1482532065', '12868640.644*1487090468', '9480634.86075*1467088075', '84575493.46*1472012412', '93993372.7746*1485166278', '49243400.8132*1477478427', '34397898.8385*1486733141', '33597023.1093*1470441262', '58514846.7051*1486826980', '98515884.6422*1460021205', '86415710.1385*1477911144', '74302537.7545*1470660796', '26008135.8061*1475111063', '71163299.176*1483606251', '77234451.4878*1474596388', '31496576.9687*1452010591', '99822058.3123*1469980972', '12679407.2992*1475397103', '56056587.0382*1464687961', '86531764.3287*1468848163', '9120979.02933*1458198293', '59547802.134*1474708238', '62955389.7002*1464743139', '66853775.8182*1462661383', '1233488.60759*1457931355', '47681106.1069*1487588543', '55457014.396*1479425416', '81681856.6024*1454142069', '50644469.3623*1476563436', '21777838.5159*1481279740', '16027854.7665*1473241391', '12797658.7491*1479250073', '30327839.589*1484205548', '20090678.495*1473719687', '59109718.2142*1484129214', '49007612.6594*1481900836', '49236328.2958*1452741984', '42503522.4004*1485705477', '95597765.7248*1465886531', '63798275.6041*1480343474', '17323143.9108*1460995142', '58140155.974*1453180858', '97737292.6009*1477740219', '10520259.9925*1466968473', '59624377.7051*1482380631', '24846773.7167*1463100685', '86417429.5361*1485070684', '62064802.5793*1478177971', '71699727.6657*1464072950', '9741588.04411*1460037450', '62844240.7638*1478047346', '26105526.7651*1470238946', '58788245.497*1482056186', '49000936.9999*1457798277', '62064802.5793*1478177971', '74943636.6335*1453053218', '33652070.371*1462783774', '468318.275791*1454686405', '32777938.3151*1466031045', '14527738.5797*1460081347', '48893088.7855*1466636956', '72087142.6106*1470275318', '49175142.0762*1483397521', '17444936.5291*1451871549', '15069167.8473*1481068517', '88362139.3209*1474421746', '99896811.4967*1453524851', '39591968.5417*1458246166', '45315597.3359*1465308903', '49094038.3234*1466610816', '67575233.6379*1470958687', '13242041.5526*1452747674', '52953864.8069*1484059482', '19614633.1308*1469789894', '19241712.3917*1481059935', '9318855.74543*1486653446', '74747940.9907*1460412697', '36772457.8868*1468641501', '72967983.2165*1478834027', '15773573.8907*1469757601', '68282848.0547*1479728610', '90900349.7797*1459405354', '79106523.5147*1480628716', '39223106.3818*1464057941', '99180488.6306*1457753862', '39175958.1057*1457131177', '70233701.406*1487745738', '10778216.7058*1457386437', '9490148.15465*1487500062', '14791806.2709*1453931943', '9956530.71159*1481224529', '89230328.1515*1475538902', '1028503.47487*1478791482', '28328049.1785*1455685882', '45315342.3434*1465225661', '89091977.8487*1475287032', '2395600.22251*1484343211', '49127612.402*1477335944', '2665210.30589*1471340943', '63488594.5155*1467618895', '50605390.0616*1456556262', '51336804.8127*1457506357', '61065559.4949*1478373009', '5479259.27433*1484269234', '14090904.2999*1456486196', '82046552.4568*1462439363', '89379569.2464*1469657123', '85643650.2377*1463642311', '64451579.2426*1452196091', '85963262.596*1470296530', '32044486.8496*1458847215', '20070853.8604*1472135635', '6968794.35021*1464868998', '40074298.9439*1455378963', '76052134.1394*1486889022', '84798816.1184*1488213267', '35215234.9851*1479266689', '82807217.2784*1487958755', '84651013.6208*1463431525', '52090681.809*1477036772', '67740337.4612*1483588627', '75378641.805*1452560864', '40983051.8268*1475164926', '59496061.8204*1477120469', '82757331.2816*1466513720', '39405274.7958*1458398931', '15532691.5973*1473446786', '79311330.0107*1480041139', '33352600.5242*1476434768', '68345798.7178*1467146097', '58341369.7936*1467888938', '23053404.3841*1456180387', '41898692.1253*1467049292', '49335879.8134*1465946023', '14917907.6368*1478265321', '15177708.7369*1468905991', '50192823.2839*1461719422', '9810237.2539*1485967422', '94652126.4957*1478196685', '84613058.7724*1486257097', '27359654.0297*1461905268', '48731832.7283*1466279075', '9222655.84307*1481932993', '6587420.78618*1462855376', '55426674.2597*1462262940', '68073819.3279*1461129070', '48961652.2669*1485688156', '82918014.4627*1468734655', '52042080.377*1462003627', '7417813.14504*1471135271', '38950317.4117*1454704001', '95709674.6959*1452618393', '87137402.719*1486365265', '50385215.644*1480917354', '98091955.3108*1468690977', '6881836.85409*1475688775', '19108004.0404*1475977995', '87164538.1418*1482246824', '819859.336044*1471837585', '84621335.6679*1478321187', '82448877.2575*1480373441', '98580697.5861*1485428471', '35436369.518*1458333930', '38771892.6847*1471753149', '91592575.1255*1453496513', '47700578.5858*1451937070', '29942766.2363*1452180614', '72638865.0921*1462497767', '23903544.9622*1480066595', '31901642.8053*1473506952', '62551103.8456*1463625875', '14111064.1248*1452766808', '70349939.206*1485717623', '64568511.7118*1482007804', '60892393.4462*1460872119', '895927.17305*1476677847', '88633521.3815*1479388024', '30198492.8459*1473417849', '80350109.055*1480123338', '59306696.81*1487819551', '49836357.9002*1481122893', '28040286.4161*1485342474', '76239528.0986*1473241025', '20333160.6676*1483352455', '80563961.1136*1483685771', '65280131.9751*1474758670', '83192621.9127*1471471745', '5150049.10166*1461820557', '29069203.9527*1472882290', '24132717.3603*1459377896', '42503522.4004*1485705477', '34881549.9181*1462492602', '19116954.855*1475978739', '94560418.7958*1453299731', '40294948.6097*1483815357', '32876097.4237*1454029730', '31157892.3578*1456561415', '99443023.9234*1468911597', '46886937.2298*1452692188', '83445831.8951*1487553331', '12621497.8978*1467395133', '37382595.6332*1471750379', '29185234.7669*1453944332', '68572378.3107*1472299272', '96174345.4347*1463276347', '27810597.9145*1464150149', '63791433.7768*1470026319', '9007241.74544*1476994936', '79468311.1863*1468568676', '10336031.1054*1478038202', '24904054.2112*1486134184', '12426430.2715*1464751169', '10304406.138*1483792510', '78156094.436*1474762838', '3801653.39336*1459025780', '11122014.6647*1464476496', '3033092.17796*1485310854', '19027172.2921*1451921545', '33477601.4915*1475282117', '46996393.6687*1480688634', '89773854.6409*1462806860', '28336882.5084*1473344064', '74889754.6993*1470919516', '3440662.97386*1471725152', '56604279.4767*1480110594', '73829955.2598*1453246994', '38401874.9521*1454927400', '18575073.9695*1456284096', '88784416.0845*1458683935', '25230590.8817*1457697219', '93779697.5113*1475449625', '93740287.4336*1467337806', '37056929.759*1456520059', '48136394.6167*1459125454', '18807106.98*1484517730', '81290970.3775*1466698089', '46826589.7538*1451754280', '27972711.508*1452462825', '43994523.5907*1475764983', '85040873.234*1458154904', '58682406.471*1468119498', '48245892.6301*1481091632', '36668358.3665*1474111642', '77577643.3355*1461763661', '64637544.1941*1471253184', '8176031.88892*1455609672', '85573609.7516*1475263897', '41005483.9387*1470819182', '78353759.569*1482830467', '17175688.9113*1485476940', '65982669.7873*1458478120', '45729008.1193*1473062706', '82032710.1256*1485128824', '9630699.41627*1470693697', '34654801.9267*1465696691', '9556018.49613*1478753151', '52138195.6439*1465093811', '34895006.845*1456856988', '77870747.0444*1481696067', '60596265.0664*1459861690', '24524922.657*1466193825', '95176086.4249*1473807542', '36947745.7804*1474728050', '95189948.6192*1457608995', '93800771.3854*1461214190', '81913489.5186*1479253117', '17696523.2685*1466034228', '61409531.1143*1465233237', '74500195.6964*1452379508', '18890278.2914*1481332470', '24077158.1806*1468943966', '88188385.5711*1456147065', '36940941.8432*1452260276', '40258119.9236*1487062787', '96795424.741*1464560100', '71477428.1088*1474464531', '83961403.0296*1482010826', '80601446.1746*1469408240', '84970000.5499*1474720021', '77720499.3968*1453486849', '86015072.7184*1463377894', '94347414.5105*1472482469', '35215234.9851*1479266689', '18832546.8487*1462664917', '16613614.5518*1481531001', '34632633.7635*1466928119', '15399751.2877*1473952196', '84081623.9402*1455047051', '21734885.3475*1479816966', '56321906.7012*1471656259', '84431277.7597*1476184586', '46716752.4136*1464744879', '78244920.3106*1458824831', '88208803.8461*1473526705', '54011565.8609*1480398681', '68061986.2825*1453758649', '15261387.7862*1466683703', '79266864.0729*1456612839', '23141915.7174*1461314490', '9180082.83886*1482749336', '14985745.5404*1485567817', '95763195.4483*1479238018', '82046552.4568*1462439363', '40490234.887*1475873169', '10301068.5438*1459374470', '7800393.11823*1456558254', '49007612.6594*1481900836', '1214648.1344*1487966528', '24393944.5861*1458601233', '1320193.45697*1465621384', '84395873.7504*1455226510', '19072428.0892*1459714935', '37597903.879*1451905921', '90740086.2578*1452287620', '68891623.3282*1481341660', '54586817.8666*1488193847', '54555207.0306*1459270081', '59265503.1934*1473114069', '2376904.65425*1454097399', '76772344.6105*1453420059', '50051819.5774*1457698520', '29227216.1743*1476318595', '59154629.5288*1469714839', '70572409.7618*1463616646', '49163485.218*1478591291', '68539217.6523*1469275995', '19469147.5531*1484689768', '24442960.8233*1471021662', '70780527.2654*1460592484', '59734281.1159*1479013828', '69507001.327*1471554105', '1335630.74236*1477790786', '47700223.5024*1483976968', '5196307.23294*1469351841', '99228627.1711*1477153134', '3091827.04911*1483068021', '77451540.454*1461276740', '27720920.0936*1467269005', '10866934.365*1473131600', '98453316.8101*1485228569', '97772767.6757*1461862978', '49234016.9188*1478182220', '73273816.2432*1488183475', '66869339.3329*1479125058', '21626497.2051*1482011366', '82448877.2575*1480373441', '25361541.1108*1468863158', '99886607.9317*1474523662', '46369288.8893*1455077902', '38620166.0709*1459679738', '53886579.212*1469984605', '69631385.38*1475854235', '25126998.4113*1475612155', '1629635.01204*1480594653', '93302385.3159*1458518143', '66354993.2603*1473165719', '28317297.4944*1459878681', '777715.186992*1486535133', '80743822.6002*1481302721', '15027829.5968*1487751382', '39921361.8036*1474848672', '53458525.5945*1483978014', '26557805.8379*1464308596', '72141163.8032*1463788343', '2252435.38137*1486681276', '75326546.8344*1458664013', '47761968.3943*1488157698', '70763074.967*1463988643', '66853775.8182*1462661383', '95453455.3974*1460437660', '22783857.6912*1451730738', '85956117.5666*1482753756', '95508133.4689*1480455767', '34386017.1899*1482864358', '39738414.9253*1463485782', '69795298.8614*1477975734', '56827532.216*1454864257', '60922933.312*1478098108', '67204395.5984*1483750216', '70542814.2353*1453771003', '7888836.76732*1479735269', '2449353.86226*1467871746', '15874520.4952*1473500264', '25729414.7743*1463363680', '63501655.1895*1465377155', '83910850.8156*1477818115', '60401992.2174*1451611951', '831670.115318*1480004235', '56310687.0663*1473121822', '27916329.1254*1465713640', '56068163.9372*1462409606', '1404955.02153*1463354901', '69822650.2022*1460187328', '55438165.1565*1461091814', '33624764.6536*1462118289', '58459737.2949*1482935594', '73367185.2375*1460380080', '57246570.2858*1458543585', '24569240.5308*1475262855', '73224196.2625*1479769566', '74528338.4015*1456928572', '41660762.4748*1470333172', '24018531.8862*1451735657', '89486045.4885*1472244357', '39173100.7547*1470704803', '29897937.0235*1467245904', '22549981.5446*1459093491', '88797402.1383*1477745796', '23794008.1457*1472469250', '77153822.7913*1465856667', '86800125.4821*1480713090', '82834318.7316*1464957168', '10680793.3201*1455769501', '48108214.1135*1454505804', '39288071.9775*1457937533', '5314463.62722*1459856402', '94390552.2142*1456936901', '13618688.4522*1457634862', '78481332.7742*1462508946', '27596965.4942*1482284385', '93189522.8806*1473892509', '39991761.5801*1475691034', '34301984.5229*1460193322', '10441282.1641*1486816001', '37370189.4407*1468435324', '64637544.1941*1471253184', '35723879.4949*1475616618', '27422063.2638*1487759844', '53147757.0432*1469614134', '47464333.857*1452010149', '47477540.4166*1485227780', '81580603.9836*1472219496', '58490079.2252*1478477418', '83481181.2531*1468194198', '73533225.1626*1463651945', '49499420.4466*1484648418', '8235988.36485*1472386394', '66273733.8166*1479844593', '34806256.5111*1457843191', '55169364.721*1481336532', '98323430.5734*1470664766', '2075678.25453*1452142954', '55659974.3953*1484920508', '92417483.1657*1483085803', '84505307.076*1487502960', '10898688.6261*1488070574', '51296481.9559*1458015980', '99469429.5039*1453559946', '54156780.0553*1488026426', '28397212.3076*1455193502', '87659069.174*1466050266', '79002317.8937*1454116165', '2866191.75586*1461727509', '31199220.6085*1474614756', '65875652.8018*1476755813', '46755323.7312*1456131995', '76403846.0192*1480098400', '86267250.3797*1476587473', '17175688.9113*1485476940', '20414957.8356*1453661080', '53331651.3759*1455700246', '98505513.3403*1477587213', '29411408.1035*1467346110', '75485269.3026*1487431577', '25729414.7743*1463363680', '50410416.5437*1481241860', '37997855.1149*1482590307', '91216861.7876*1483048387', '51310754.3082*1462429937', '47738306.0582*1470167024', '27727457.7255*1477803771', '94818668.796*1486700437', '44744969.9431*1467960464', '41324270.5953*1455034816', '30154440.0772*1460463590', '90597350.9283*1452412119', '35720499.6554*1456124267', '29441843.414*1453880837', '32874337.514*1467061800', '92784707.1199*1461756187', '80868518.686*1455432070', '25252489.3199*1470993810', '89395112.3153*1481407912', '18391811.647*1481320989', '11417825.8731*1472349289', '78449727.7881*1454569239', '28078648.5592*1482572220', '69146357.8876*1485643255', '95635964.4222*1463751606', '13285802.2438*1478814086', '44139591.907*1471812860', '9729702.63519*1485770304', '60671392.1766*1469788511', '12504770.3434*1458081844', '88496896.1238*1479233299', '1054985.81296*1486819922', '16464264.7078*1478312591', '36450200.4002*1487146510', '38715317.2316*1484724222', '64278631.3398*1466889394', '93267936.8825*1452219471', '78970651.7463*1465355977', '55438165.1565*1461091814', '18928149.9888*1479176697', '58018068.6227*1479682670', '56481326.8926*1469388993', '75269500.2431*1473399767', '55500794.3374*1477116377', '47336002.3663*1453461809', '838589.981327*1473815937', '94256951.3696*1470820549', '2208141.21371*1457313615', '8027674.26796*1484624920', '92241958.2835*1479492587', '33838528.9378*1476580612', '49485593.9541*1461709646', '17274299.6798*1471828669', '24771484.6917*1466791321', '58996771.41*1484503600', '85918307.5857*1483923040', '39132168.1962*1469643780', '44233733.0939*1453689175', '38172948.1459*1470855443', '15046758.6854*1464570677', '91263753.4346*1487681419', '32039981.3593*1467546747', '66754917.5958*1453771251', '50192823.2839*1461719422', '30479958.9209*1485430751', '58929264.298*1467232120', '71069909.0538*1484801445', '12868640.644*1487090468', '13078369.7837*1471241304', '46782932.8994*1479171682', '70106057.0356*1464321870', '24910594.7199*1488162845', '22835642.1133*1480705117', '42233985.8408*1460655865', '39294623.662*1463278510', '10150296.4866*1484368300', '52410123.2217*1451947798', '59807021.8631*1479528715', '3000873.60899*1484948272', '45476803.0446*1476187058', '24380792.9436*1488213611', '16286229.6959*1474173392', '60305786.7874*1473942706', '60083942.2407*1458463034', '82282736.4622*1470230851', '24706005.6337*1458532652', '46789651.7973*1478422992', '8097757.47694*1458042302', '33892136.0889*1468513288', '71912323.4649*1483601565', '40032875.28*1486573832', '16340293.0759*1476258862', '65958369.9587*1457732124', '94481622.5907*1468574675', '38701014.4278*1461023788', '44182759.1962*1461323545', '62617696.8125*1452633689', '8467771.28221*1483827184', '47008773.255*1467759656', '51820988.6722*1453745718', '93201084.6265*1476649684', '42316433.2308*1457518045', '54941562.728*1468046867', '16678982.0657*1461859735', '30678847.7341*1487731859', '44426745.2018*1469580816', '74064512.8093*1464406959', '5415252.8005*1464014483', '62091334.6517*1461228085', '58734823.5516*1453260668', '96598476.2082*1457037310', '12565218.4941*1476671498', '57317432.7962*1469318659', '8954885.01648*1458314037', '41455035.9049*1473766683', '28023811.6779*1470111782', '19789820.2759*1464288667', '51800992.7581*1464575049', '92974897.092*1454965703', '10192874.1727*1470869224', '20840478.8819*1471944919', '57339134.8563*1487728355', '82060703.6902*1457228199', '14585780.5187*1471064572', '79598582.1989*1462184852', '16221140.5458*1468996470', '35436369.518*1458333930', '74048791.5055*1473656406', '18799191.3121*1476056659', '56587949.6985*1477781532', '32549395.7388*1475597077', '10013520.7035*1461984565', '47129528.4264*1465451360', '34618277.4446*1483681601', '77506941.4067*1457849154', '13450539.6128*1462659007', '25381865.0138*1456238833', '84453816.8202*1478358243', '4905001.25304*1453251516', '49007612.6594*1481900836', '19387153.7096*1472466662', '78114357.3085*1478493217', '41078036.9646*1458901000', '19834654.819*1476405687', '91931755.4718*1454866548', '83901697.5486*1456807495', '4291524.97511*1455158174', '13470832.0559*1470566519', '35424783.3277*1455395665', '74361251.3881*1453186277', '80837123.259*1480914258', '95385757.6155*1486623148', '81726533.1602*1483644426', '11019816.4793*1456089156', '26166206.9699*1462973305', '20217556.1149*1470403405', '38761187.8698*1455801023', '77273826.1393*1468101724', '36053793.2832*1478102796', '77636782.2528*1472306731', '741282.24753*1473556146', '99099876.2517*1455974855', '23467117.8*1483943235', '77650479.916*1465313059', '1483843.03367*1476968773', '25594476.7347*1485164700', '2825277.10742*1473111226', '68135008.7097*1487016525', '27766393.9398*1468685011', '67740337.4612*1483588627', '32786645.9519*1481124440', '90963319.3097*1456509414', '6529570.16095*1459238822', '18724136.2624*1469281735', '98442290.7826*1461583775', '7190943.84467*1484237254', '72771852.2341*1467237713', '50393860.3096*1458841635', '41102871.8797*1471628037', '30188061.19*1458792211', '16863019.4698*1473505758', '78069953.8988*1473959224', '18900842.1162*1477406016', '72164036.1511*1480486581', '9482423.87883*1478882378', '84745373.624*1459743335', '4764334.23253*1465171671', '43231849.2295*1461128664', '87204741.3238*1481161428', '20956279.7617*1456119639', '62516710.3785*1484657062', '839759.974617*1451646892', '78527132.0579*1485219612', '64451579.2426*1452196091', '35967751.3466*1452714322', '34631496.0295*1467443136', '38172948.1459*1470855443', '90755785.0398*1451647352', '67528360.844*1486036523', '90583065.5227*1477437329', '82547245.9366*1459142317', '32899768.7341*1475692040', '35917391.8354*1458843696', '18393468.8891*1475897545', '67711544.734*1476641043', '62957890.7698*1471749640', '28023811.6779*1470111782', '66270323.6906*1469507347', '93038100.737*1483346000', '4890975.19424*1471065449', '8272221.59776*1457866259', '93344603.5912*1484205875', '84671546.6193*1466574574', '3555118.07418*1469955065', '58031798.8814*1461088810', '37619438.0108*1463233750', '46281152.7862*1485031171', '30198492.8459*1473417849', '218706.788306*1469345557', '90117220.1848*1459888776', '89508484.8839*1481132549', '84479038.3618*1478888218', '49401558.8201*1471356413', '66028191.7309*1456510217', '10241770.6829*1484867871', '23239715.6888*1456902769', '19177554.022*1459203750', '17384285.4312*1463214331', '34168501.8901*1479703393', '10893666.6517*1468773873', '56721299.545*1469362442', '92269920.3535*1468306571', '86794447.0575*1483658318', '20694533.7728*1487965733', '16901058.9003*1486153663', '99247484.2731*1462003045', '71078383.9175*1480633811', '1512940.0822*1471329387', '33307012.2897*1483286160', '68244732.748*1452216360', '42230039.3663*1480616093', '29069203.9527*1472882290', '94854570.36*1483478801', '80709795.7483*1461223310', '72431342.8032*1473131133', '33567375.7345*1482225555', '75861666.443*1472788616', '11442403.5436*1474512902', '87142391.3533*1457689085', '94854570.36*1483478801', '76099442.0605*1455154618', '32595199.6548*1477678397', '89481255.4507*1466837249', '72047548.8802*1472061220', '10742436.148*1472323839', '22486929.0963*1452757700', '85456964.0583*1452195076', '4328030.21304*1457747972', '44383170.4954*1473251106', '331711.268657*1470112763', '10866934.365*1473131600', '3577543.3343*1476845374', '73164364.8095*1479487212', '80553770.3436*1454082444', '96271950.2516*1471925648', '16867633.2927*1473669546', '33447742.2497*1458356486', '37574346.1133*1456535235', '38730605.591*1460762304', '59997351.6061*1466167215', '22276074.5772*1461366495', '14152233.9722*1452570571', '51049254.6522*1468376643', '80653440.5139*1473590470', '50777106.6801*1480002856', '48249241.8587*1485360871', '21535951.0759*1479465450', '16104338.0131*1463721299', '31894925.5808*1452227880', '1680853.53017*1465341119', '37054885.0889*1484663458', '30238642.4607*1466656505', '50963649.6318*1453610104', '91891543.6649*1460409168', '14105914.3014*1455124310', '55414602.3953*1451798752', '92101740.0906*1459006200', '53666752.414*1483055142', '30881190.7634*1471524661', '1834814.80595*1460194342', '19178385.0312*1480252283', '6169564.59295*1472229123', '98824169.07*1479868080', '86138802.9445*1480666341', '93001623.6016*1478657335', '10451332.5086*1476525123', '1692986.23968*1471193383', '38303590.5187*1466850127', '76915735.6466*1457153808', '40388268.4917*1482784364', '99528021.2764*1483150748', '1507210.8364*1482867859', '13664046.8642*1465614206', '3744439.26805*1474093170', '22049700.8239*1466451361', '20700110.4453*1459447294', '19288657.8033*1457334689', '90740086.2578*1452287620', '78802915.8846*1459177715', '66869339.3329*1479125058', '4283596.12162*1480816664', '49337169.3418*1482645069', '88115826.0282*1460303860', '84424837.9609*1475446595', '69848163.1312*1480472652', '60892393.4462*1460872119', '62400260.6377*1459341832', '76767332.7421*1488299995', '70125550.6595*1484507581', '94685942.9549*1462834297', '95087542.2024*1459193502', '72592271.6205*1472004874', '48357277.1549*1477136236', '48145173.6396*1467279034', '49523261.4848*1484378707', '34591904.3556*1469363311', '20350335.7336*1475580754', '54586842.0218*1476005244', '60906190.4145*1457927949', '70835075.1864*1458220650', '37584644.6925*1465146265', '85442660.4974*1483700101', '41511296.8323*1476172029', '30979933.3988*1484836565', '32749612.2357*1483114299', '6864957.43557*1464931071', '38932190.8555*1454423951', '95020612.8207*1461110093', '74574919.6039*1454628770', '11257141.0625*1478213922', '61477122.0791*1486111540', '27735127.6162*1460303726', '7199642.3643*1470095663', '54327853.0243*1481459490', '46589415.4479*1458245776', '87262060.5073*1453816126', '10554611.5645*1483926407', '36959277.9487*1471474677', '60099199.1194*1476030783', '64880057.0932*1484877603', '47206367.3753*1474712999', '18391811.647*1481320989', '24589675.1164*1461418648', '19447756.776*1477279763', '86169068.4546*1484222223', '14098716.8034*1455496864', '12052150.4355*1466542837', '84651013.6208*1463431525', '60055035.3576*1487672538', '38290848.1049*1478044742', '95352975.4516*1483010083', '21278894.3713*1483435555', '31244583.1581*1458484304', '90291973.3306*1482428479', '26568934.7369*1455664942', '88129003.3615*1454352689', '36659799.8728*1473095166', '3175879.0992*1473425727', '44440038.2306*1478302065', '83134231.7254*1454774360', '88573591.0804*1474855875', '69336161.683*1470265753', '33677785.467*1453701652', '98453316.8101*1485228569', '90113875.2264*1473714998', '29539824.8707*1461696297', '20331952.7319*1469284557', '69545361.4051*1469427392', '69927275.8148*1481104645', '3428239.73821*1478842482', '1237016.77633*1462441021', '24799334.0234*1485941326', '64616180.6486*1464263098', '68394889.1282*1484230026', '6912200.93332*1468237272', '60615210.6449*1452383896', '83841119.0499*1474831215', '24437694.7603*1474976547', '3675592.70311*1473211379', '50704855.6201*1461579221', '62056623.0753*1456033117', '90285628.4897*1453268867', '323612.36319*1463293191', '74324428.8399*1474988425', '2122675.89463*1473489093', '66247077.299*1486928663', '72732736.9464*1462340013', '76052134.1394*1486889022', '63423478.5644*1480542415', '182194.620854*1467103884', '19565267.058*1487675569', '22592654.105*1465708340', '62848548.7678*1468255288', '3951001.57506*1477739837', '38411981.2259*1466814754', '79479225.7329*1458770552', '11257141.0625*1478213922', '79657900.5511*1458010329', '78805267.4246*1483629383', '59405607.9781*1469886051', '69870761.7887*1461426005', '61006392.5326*1463581062', '69684596.4718*1486631176', '5510497.84231*1453473378', '3041358.02493*1469236994', '72637769.5448*1471624305', '57432984.5793*1469010532', '35296541.3071*1482457453', '88486815.0462*1483431557', '90054515.1472*1482724671', '66611895.3291*1482294090', '60596265.0664*1459861690', '96758948.9884*1453996456', '29782880.7629*1453560825', '3000873.60899*1484948272', '42844658.0273*1455608978', '69146566.8396*1481694187', '57417448.0804*1477517909', '68369007.1329*1475152972', '48046548.2391*1463705956', '92101304.076*1454497331', '19108004.0404*1475977995', '12520091.5649*1479378400', '70812958.9654*1475776898', '91258000.3181*1457162563', '2946788.38573*1470931454', '49304133.0719*1461909753', '67795033.2384*1461515316', '25831045.4214*1479696532', '73024401.6895*1472482217', '58279398.2226*1452854090', '43517868.5415*1454682947', '44530543.3954*1480548587', '63988588.4947*1473110223', '25817339.9452*1461646085', '3803707.64249*1488315492', '58565336.2998*1484478543', '32402354.5216*1465447137', '6509732.89041*1480734606', '75549893.5995*1483147494', '55209791.864*1485128752', '17512743.3212*1465371851', '47218809.8438*1468230199', '27546349.2217*1463968490', '59610252.2491*1477470537', '83432413.6563*1479586239', '98474892.4081*1466935405', '17245567.434*1456786777', '85214202.4131*1483163953', '9982236.97957*1452552002', '24631825.5787*1487733734', '1363492.47438*1474836043', '61703127.6047*1458243835', '81399396.2655*1460151695', '2206656.55829*1468251877', '43483289.644*1462797082', '26734971.6546*1475366665', '66437080.7573*1464684937', '50028211.3551*1468315646', '61636736.9041*1483391984', '95794640.7348*1458513952', '24825812.6095*1477348152', '43322232.565*1473042571', '87498718.413*1481405160', '15614414.2595*1485836882', '59367347.5088*1474689388', '89945128.1607*1467931486', '34335599.7819*1466181976', '88913589.6075*1467003579', '7253976.41411*1472440877', '14106370.2065*1472098408', '96287992.094*1460302455', '6742230.02321*1466594938', '94314768.4552*1472307251', '45139438.9896*1468206636', '36868627.3692*1463202675', '76019401.338*1486633947', '77655465.8697*1465479931', '40591645.7128*1476922117', '77201308.6202*1469277324', '85415357.8925*1473516956', '31148601.3596*1462766811', '70342713.1009*1452316951', '85504299.9739*1464688808', '70176561.421*1484030988', '48108394.7829*1473941822', '24487847.3551*1480717110', '10614495.573*1479925704', '8397398.03748*1461889582', '99518743.8099*1451713915', '97061262.977*1458904883', '18858102.8061*1476787498', '50318482.6468*1470311730', '25047501.5151*1468914467', '90298711.9977*1477189363', '49897941.0129*1471959809', '94390552.2142*1456936901', '93501400.2542*1462815518', '49220890.6819*1468965367', '57316519.3998*1479105566', '21431025.3062*1481287810', '12744494.4368*1468871484', '33053118.3225*1485592474', '35340413.6382*1470535453', '70031691.17*1466445364', '72129982.9339*1459933353', '78110739.858*1455023074', '67275135.7591*1454198440', '32431194.066*1473880092', '55500794.3374*1477116377', '9285253.03496*1456269040', '18916221.4766*1485201743', '49401558.8201*1471356413', '69764697.1738*1474776219', '45133311.1426*1471320556', '51400154.1044*1456308665', '99384160.8585*1477768468', '96335161.9234*1464239137', '6509732.89041*1480734606', '16671267.8857*1453008418', '57269327.1262*1465258672', '33477601.4915*1475282117', '36760606.8587*1473179288', '94390552.2142*1456936901', '52042080.377*1462003627', '9585225.75614*1478752055', '63286538.2457*1483927903', '12449933.8898*1476546185', '91177559.671*1467611827', '27212129.539*1457094160', '8715667.30589*1478815175', '41803245.1455*1473831133', '54125521.3215*1473532595', '7252622.4478*1468827770', '31889711.3805*1458797975', '14455056.2218*1482918980', '2122675.89463*1473489093', '27392631.0402*1459392886', '20347385.6706*1483214021', '40258119.9236*1487062787', '76744536.7046*1477523779', '46578042.8921*1466454370', '18438335.5777*1457947990', '6954912.46963*1469216170', '74997482.7321*1464757153', '19436494.3492*1478997024', '30875761.3622*1486896886', '57466787.286*1473674076', '39239258.7234*1484919061', '27423215.452*1454482936', '68672085.8214*1464186923', '60546724.5242*1485248401', '38213472.8751*1456700762', '32156939.3055*1473952985', '49127612.402*1477335944', '34993036.0262*1453912891', '51091714.1813*1461250783', '28143080.0616*1454506025', '1436482.80452*1454287732', '77506941.4067*1457849154', '6475787.10355*1485204516', '91903832.2202*1456673272', '65357267.5075*1482240083', '90400302.127*1483036668', '32805170.9945*1456649712', '84435487.7816*1483045616', '1431420.34572*1453585041', '64642116.1086*1485654624', '99336251.9586*1485529957', '28909540.0318*1461318124', '99406498.554*1474635681', '34806256.5111*1457843191', '29529432.2731*1471856824', '85418975.7106*1460051329', '84268924.2586*1481008727', '87598806.3534*1451714043', '44914761.1394*1470219653', '95793293.3455*1479128305', '62376455.5674*1486317191', '87826205.9459*1452820570', '67587262.8353*1467298131', '73635778.2625*1483412446', '97894945.2033*1483537792', '79998452.7794*1456455369', '15674397.8354*1479644469', '89773854.6409*1462806860', '4702577.09599*1469327045', '85394019.5419*1466863694', '70827992.3573*1467541141', '92893149.7904*1478111425', '42613837.6798*1471275650', '58330320.0139*1469078769', '52025410.2953*1466385810', '90484120.8019*1463634424', '31244583.1581*1458484304', '35433858.3154*1465201706', '19174888.6652*1455962879', '26733596.0297*1474638968', '92386222.1018*1459309032', '67575890.8662*1480353238', '50162449.2405*1456893554', '1252168.6584*1483586745', '73544787.956*1470655978', '70319601.1291*1468583835', '77210268.0238*1474694573', '54028427.4058*1461091495', '79989564.4217*1455369687', '64718885.5171*1467076670', '99737.8528217*1456025353', '65995846.5525*1473839457', '26423885.7324*1483074134', '60063459.0323*1465358385', '31980168.4998*1486806633', '10336031.1054*1478038202', '24904054.2112*1486134184', '41687356.552*1455905428', '62372184.7214*1467121831', '84647509.2111*1456251757', '86049173.6893*1458026078', '79289188.2268*1455091782', '21749235.1725*1480017227', '86906043.4997*1479094259', '17047752.5276*1474395138', '91567977.8922*1460210745', '98265665.69*1471855341', '43390490.3938*1481533335', '93633863.7672*1467208813', '44040504.4309*1468950936', '39338356.7451*1479595970', '48815108.0092*1455650525', '48358324.9125*1473082848', '15230749.8508*1487114692', '48050868.3928*1484143884', '33294711.7904*1470811068', '58147770.0159*1485039954', '94216837.8185*1466887139', '37039088.259*1466536991', '73837027.3717*1470060954', '71320226.3122*1474706236', '10146916.2639*1476380636', '81354552.377*1453894016', '70233701.406*1487745738', '79961166.249*1462169189', '88800504.2614*1487341251', '81665440.1836*1452129494', '84388856.6081*1481010748', '95578723.4234*1474873842', '5196307.23294*1469351841', '85004739.7487*1481579074', '12047855.8666*1464039617', '76783448.548*1478155657', '57902708.6459*1470333728', '41147148.4542*1464002139', '75113570.4399*1466819026', '34828574.9743*1480030204', '32915760.782*1474602919', '16718234.1725*1460975253', '42972170.3311*1468584001', '64970788.1789*1482305492', '358986.940119*1481722944', '18475636.2339*1485259402', '84036858.0705*1466569099', '92802998.3302*1471329633', '11417787.1196*1486746610', '42824991.1833*1468840535', '20068784.399*1469021303', '2410007.74191*1472508232', '35248940.6425*1476799369', '74283134.9051*1458093093', '59364825.9603*1462932435', '94652126.4957*1478196685', '65756845.8513*1462458360', '65835218.41*1453815049', '66530223.1149*1481876143', '66876852.2049*1456352931', '67686797.0283*1458052666', '50775431.2011*1458728602', '86788149.2942*1486496149', '25191543.0325*1466731706', '4018903.14539*1475182171', '94039073.1452*1474043245', '58713781.4408*1457714920', '98237775.8726*1479304008', '304179.042657*1455603545', '68155613.1004*1459478937', '17203993.6778*1474110948', '53201526.8965*1474812892', '15617.2762814*1462715815', '30038415.2283*1476463758', '70653244.4556*1463181398', '662795.566333*1471441015', '2722335.44497*1481148755', '5192866.94618*1477635642', '67632690.7865*1458022715', '36857291.1299*1488261866', '22276074.5772*1461366495', '3846530.82991*1466382180', '59939408.747*1486845540', '48961652.2669*1485688156', '68067315.5862*1467129558', '16513157.9742*1465686783', '79154218.7858*1466819824', '95673232.7663*1454610523', '40665651.7201*1473360884', '64521093.5878*1451816310', '55478627.2866*1487395937', '81309802.1304*1464462524', '30781747.2178*1480335965', '38927933.6963*1476517885', '55746609.2313*1482367075', '42537277.9517*1487782094', '11403467.1267*1481467632', '76939313.3095*1470767268', '20217556.1149*1470403405', '60547535.0017*1469703201', '75978429.8878*1464793236', '63723182.8191*1467962861', '16166031.5709*1463755986', '9699288.3936*1466454739', '50130514.818*1464785291', '67545749.2808*1474703191', '18316150.1388*1487812677', '87414169.4691*1452578263', '42195634.0178*1476662094', '83455331.4755*1481891773', '87400616.5156*1488036164', '44783400.3886*1487070185', '95294068.2237*1474495132', '86138802.9445*1480666341', '32948218.5918*1467115474', '36391299.4124*1478031712', '83777974.654*1453163186', '47918810.1711*1476804954', '47206367.3753*1474712999', '4879697.21782*1479097701', '35942994.5179*1470485675', '83901697.5486*1456807495', '51614691.9277*1468900195', '77870747.0444*1481696067', '69713946.5658*1485871297', '50192823.2839*1461719422', '54879384.4284*1454690832', '73604734.4149*1487806079', '13008130.4734*1459444108', '2736329.8221*1453033290', '8825186.63435*1457307473', '24231334.6812*1480977436', '92712454.6355*1486338843', '47624269.4578*1476523072', '49310053.1716*1454871449', '32548294.6213*1472902690', '13675413.3762*1454113546', '94048231.8719*1480723382', '20772047.7459*1461517006', '5081552.81374*1482138241', '33659304.5772*1470566260', '48321252.7822*1457384524', '17968345.139*1464014039', '25301188.4216*1463471606', '97365494.0548*1454805916', '56612819.2326*1472493755', '67226340.6832*1468097464', '45530659.309*1484451054', '33899797.8334*1476924835', '34654801.9267*1465696691', '78448563.1836*1467981874', '66732050.2501*1461283377', '46554143.0351*1471913897', '81851790.1648*1470518612', '69090096.9834*1486602269', '88748351.9932*1459826717', '56195301.1362*1470109871', '24318392.9075*1473968722', '41410337.3628*1464192943', '66453802.5573*1470231906', '6344154.72363*1465229444', '99574853.2228*1487641471', '25193652.4356*1487083690', '25831045.4214*1479696532', '62145254.022*1483616470', '28121120.5776*1461140170', '24687091.0818*1458934467', '48804477.5622*1469623267', '68743640.5353*1451994509', '79142496.9033*1461692075', '56884533.7507*1471421594', '289846.367792*1480653099', '10128076.8383*1466381495', '86221443.9493*1464391526', '83972474.7368*1453432440', '68922213.2537*1474270077', '43096186.7555*1472966939', '96372130.9979*1476935974', '45729008.1193*1473062706', '32120529.509*1458849935', '84505307.076*1487502960', '50529969.0818*1463586311', '50996202.7271*1464068895', '12520091.5649*1479378400', '11510457.5165*1454424539', '24223534.6728*1469363547', '55691291.8351*1457330292', '86360330.7986*1467121344', '74186684.4424*1479896072', '66732050.2501*1461283377', '85573609.7516*1475263897', '52445995.305*1464344767', '90751405.2054*1474103560', '78568819.4192*1461352506', '30332640.7714*1483280629', '15749900.2883*1479419815', '65393820.0205*1487073215', '4044969.13112*1453676312', '97179876.1425*1466552489', '52050641.1855*1487032827', '22139003.6145*1483113273', '91339123.184*1476122333', '19369630.304*1477312660', '32917914.829*1479157145', '55238281.231*1485461790', '96778017.652*1456310033', '53049049.9944*1467276092', '29931017.8459*1486366916', '37366971.8287*1464296678', '62617696.8125*1452633689', '57551269.2401*1482149702', '19462475.3364*1472127565', '99804198.1862*1466005374', '42786057.2401*1455204975', '14652426.0699*1472696548', '82034001.2513*1457359893', '50181894.4316*1452798084', '27026503.989*1460300122', '82914620.1313*1486332399', '38337085.8721*1453605272', '81787018.5615*1466288109', '30789660.014*1452687642', '68191281.8395*1470767311', '66300754.304*1455584371', '99595698.8234*1452793336', '57676642.7681*1464506639', '75056029.9051*1474421352', '11100036.1124*1451642296', '86699341.8773*1487595935', '37045597.1393*1461424531', '98533587.4257*1457476803', '27123095.6723*1453218526', '79471146.0651*1487548766', '14373379.8505*1456402979', '25858187.6948*1462840610', '73992442.1539*1463196663', '81290970.3775*1466698089', '53201526.8965*1474812892', '75472143.0923*1465478038', '87008939.0351*1460196279', '68218066.9679*1464771270', '29197923.8908*1474684805', '30811746.2921*1467156992', '65788695.5577*1484666506', '48547539.4767*1483767063', '98470563.9356*1482284300', '40975041.59*1472943217', '9451702.20513*1484345865', '14152233.9722*1452570571', '26485307.4659*1464431015', '57551269.2401*1482149702', '96126421.4483*1470062246', '95679793.6152*1457744397', '35776496.2618*1480527782', '60753970.5798*1473807209', '70859227.0158*1458990609', '42159628.6753*1459270738', '16970607.0637*1484026040', '82245012.1094*1461880187', '12754412.6065*1484705919', '87029961.8813*1460566109', '50944207.2217*1484283567', '69634738.1506*1476404894', '17554406.5693*1469727643', '51954441.4373*1467130709', '18442857.763*1461410549', '93517434.5465*1464957443', '3117085.19203*1469743919', '47579662.5968*1485894819', '24231334.6812*1480977436', '79788439.5822*1452581752', '69848220.7421*1488224292', '20350335.7336*1475580754', '20041465.6107*1459056940', '17310201.9012*1475289657', '97928820.539*1465050879', '82517994.8614*1464553949', '77947483.9922*1482286786', '10775065.8525*1456126595', '63956027.4171*1478666824', '2477769.72115*1476847928', '12114663.515*1467332183', '84715445.5974*1461608246', '89968206.3663*1485300062', '50046661.8258*1484242512', '37381339.1162*1456709083', '51528971.6421*1464044953', '62824208.849*1476121429', '93197662.5965*1453870486', '26872018.5645*1454053805', '60674648.8242*1468437640', '59009440.9012*1481505937', '70155187.3873*1460496018', '24393944.5861*1458601233', '86675014.1948*1460441625', '64186321.3994*1456450203', '132763.034239*1471002881', '8405129.9939*1474375645', '99384160.8585*1477768468', '58718540.3626*1485132810', '70629042.7077*1467918347', '17203993.6778*1474110948', '63038309.91*1466093864', '9705808.4671*1466260849', '33819678.6658*1478092129', '28066038.7843*1464504221', '44757725.2634*1480217559', '53164722.2168*1482100476', '16637035.0625*1479943814', '57566438.4437*1484660064', '43280235.8697*1452424028', '89266955.1453*1486785116', '95522031.9185*1482100215', '44450597.748*1460992972', '22616110.2273*1463591804', '44617492.7662*1476101659', '65613701.4107*1465088686', '9478423.81687*1483565251', '74250787.1406*1452879203', '72170915.4493*1465230292', '4702291.2807*1464748965', '8097757.47694*1458042302', '82317929.6206*1478586028', '70810739.4377*1472664766', '15315541.2098*1466338585', '65613701.4107*1465088686', '81062454.9362*1457022868', '88890447.4947*1469025060', '5534856.66829*1455695958', '33445163.8853*1453007746', '40531780.0855*1479076145', '83972474.7368*1453432440', '46783774.9738*1459077300', '48590028.3063*1487743916', '91010301.2197*1471228354', '44329461.1557*1461516477', '6622128.96755*1456422910', '31058636.5347*1464668906', '61636736.9041*1483391984', '12583635.4847*1483814052', '97928233.4431*1461385606', '60911703.714*1456325461', '23112291.9968*1471920770', '24223534.6728*1469363547', '41741112.8603*1477913894', '94033156.2578*1470329879', '3880719.03351*1480894688', '2268440.52731*1462627441', '45282223.7145*1456746316', '21063226.8916*1453700935', '15752680.3712*1480259083', '52624513.9332*1455751592', '55217989.3675*1482790176', '75857490.5445*1454776876', '97841720.4333*1457975195', '66964828.8389*1475492217', '70101594.3206*1471172968', '5216667.47998*1459392381', '73400190.4166*1483682746', '59364825.9603*1462932435', '73723027.3745*1467597322', '4654470.79912*1484397073', '37455763.958*1456769911', '29040322.4939*1462258520', '64997102.6119*1455891256', '27307294.327*1452815816', '44919282.8068*1471117663', '11416202.7251*1454438746', '78464041.4635*1461104848', '82922713.2862*1473884505', '6077659.73913*1475513729', '14871479.8535*1479578760', '1009937.25329*1475412214', '80953579.2627*1467799415', '19480548.4221*1461002487', '78966557.8517*1459418462', '93382950.3194*1476899716', '57884378.7096*1465531163', '34388970.176*1466353797', '57663388.869*1474808515', '18605998.5669*1478242655', '44117589.1743*1478467686', '89625227.5489*1486363777', '95385757.6155*1486623148', '94292554.6392*1457584969', '92717949.0841*1461741292', '35528640.0353*1468351732', '67262852.2701*1477122241', '36342987.626*1483059504', '13664046.8642*1465614206', '3352495.63096*1476698091', '90117220.1848*1459888776', '2058660.57857*1476350423', '90469006.8067*1473272789', '90602731.2314*1467415031', '26470597.5989*1453273281', '99717808.4462*1466613744', '38153474.4301*1474503693', '41898692.1253*1467049292', '125561.724059*1468097172', '20079321.5858*1461911466', '15718700.9949*1453522238', '24318392.9075*1473968722', '33307012.2897*1483286160', '74611358.0075*1453801809', '21720594.796*1471823078', '47840098.1948*1473200307', '82835316.7692*1466582118', '44617492.7662*1476101659', '96083611.2334*1467259002', '55188276.2316*1457553896', '46551084.0706*1484860932', '871001.062508*1481732023', '61418250.8189*1484800332', '35731315.1036*1469268298', '66708710.6566*1468036739', '84537046.263*1452007063', '4265208.87749*1478608805', '92129636.2383*1453955830', '23237244.794*1476344014', '79105493.5671*1457441256', '55435265.8139*1456392874', '93397072.3336*1459885089', '87947565.2649*1483780515', '48909203.3316*1480475220', '779560.304345*1487645551', '78245597.1409*1467451083', '14852018.501*1465698086', '44987890.9089*1470194613', '27088612.6432*1475420105', '13645543.4396*1460672579', '65767165.2458*1476297236', '31080301.6042*1459974594', '42018905.8879*1486086586', '27239091.9753*1452047038', '17561814.8888*1454413032', '92465017.1857*1454500577', '99196252.1429*1460913122', '52420829.8341*1466366160', '20506511.2528*1453212741', '93551084.3909*1468403389', '31544411.9384*1454153688', '48351717.5339*1474879917', '86166657.1988*1464678900', '99336251.9586*1485529957', '99274731.1659*1483749508', '82974276.0334*1452168758', '53936360.2676*1472728769', '33280759.1719*1479938216', '25971952.4361*1483874687', '63696851.3648*1463652756', '59855842.8027*1480155325', '79497016.7357*1488319813', '33032649.1506*1472279708', '76529494.6943*1468929555', '46890565.2675*1477051689', '61660657.7218*1485095680', '59240625.5853*1483182264', '94685722.05*1484848259', '4029114.86331*1462006190', '87162634.4635*1484111886', '65532357.5249*1454861709', '15164763.0293*1479086933', '91443799.3323*1474485651', '54427544.1552*1488254769', '12242490.5085*1480325502', '52929332.848*1483951242', '26547615.5521*1484048061', '41741112.8603*1477913894', '87993801.1731*1470725646', '5150049.10166*1461820557', '59179994.6317*1461805615', '73283277.1522*1472837555', '67160711.562*1463791369', '79657900.5511*1458010329', '33393930.4994*1463540745', '41811046.5407*1455389623', '58625227.4061*1472604281', '55659974.3953*1484920508', '1604731.21417*1466221684', '1009937.25329*1475412214', '6518435.72399*1460109789', '57155994.1557*1466860466', '71635873.8882*1463973604', '92110972.0863*1484831578', '55856353.5935*1484847491', '93889316.0986*1458870498', '77528142.7352*1473454468', '68949421.0751*1458652912', '22143117.8182*1478386938', '75704437.4107*1481556993', '68086433.6162*1482688625', '65790224.3654*1468021301', '44153909.9686*1459809099', '67409807.4086*1477291496', '55190271.2492*1480509028', '30388979.5216*1468954112', '34459026.6199*1480624493', '33059632.2353*1487884387', '78281061.6901*1465648715', '69139165.1393*1484275257', '7949135.34249*1458005643', '98288729.6432*1459953327', '93672620.0803*1460642155', '93988161.9923*1455532555', '69476036.1621*1465468862', '77153822.7913*1465856667', '42201465.784*1459823390', '32552118.1994*1476247660', '74132620.6887*1475853267', '21204061.5101*1452823522', '97712113.4449*1453165799', '58128294.3441*1476372489', '19118437.1359*1483692105', '81014361.316*1470501176', '6455416.79104*1481982416', '38742476.7816*1470121486', '40170478.1809*1456892618', '25207370.228*1452125095', '50531764.8525*1458196268', '52991833.2452*1475491203', '42827887.6632*1456503886', '93622963.8804*1467621277', '24223534.6728*1469363547', '55246215.2718*1478179698', '58674870.1508*1461917734', '57820431.6287*1485208165', '43344966.5932*1458624241', '87530041.1225*1478078169', '42186218.298*1473912584', '20755670.7466*1472736532', '37648579.0985*1486245497', '46714134.0492*1467180321', '13914956.505*1475378684', '37478903.2618*1474158991', '2213066.37318*1456081366', '5042857.02783*1454208886', '24017330.6426*1465631420', '39653866.1862*1480952031', '99098661.5902*1471835445', '69997557.0221*1462179459', '12801880.2864*1454991381', '18150086.2536*1473866754', '41566851.8451*1464481357', '26547615.5521*1484048061', '68610480.1669*1465897402', '93582143.9926*1468568480', '80157574.33*1457533850', '82199966.1825*1481032514', '90711950.4307*1466870854', '55582392.1828*1454318265', '55169364.721*1481336532', '22708413.8489*1459873358', '11417787.1196*1486746610', '962404.196499*1455575812', '89679315.6318*1467536119', '66443400.7755*1473818160', '69024602.7827*1463211979', '92654363.7947*1469080309', '29539824.8707*1461696297', '41307428.349*1465136534', '41566851.8451*1464481357', '93738298.5595*1486908153', '32811648.2877*1486260979', '2722335.44497*1481148755', '26019115.0678*1465484059', '85001670.1572*1488070752', '70508121.9161*1463663972', '94448405.5241*1476212865', '14373379.8505*1456402979', '71173782.2581*1482812553', '90117220.1848*1459888776', '59157690.9616*1459696369', '28121792.5098*1473547457', '59179994.6317*1461805615', '597815.439481*1455378859', '69536054.8806*1467632537', '71059729.4991*1468120668', '2665210.30589*1471340943', '76176242.7025*1458859746', '71322984.3853*1464745850', '40695958.6906*1471687894', '4496590.05033*1460254622', '66493444.2541*1462401003', '31517657.4619*1484936589', '38111230.4231*1476023782', '48959229.5883*1462573466', '79466590.8295*1459776343', '990999.230095*1464540588', '21163521.7119*1463122189', '95419651.5827*1465654334', '20582433.7598*1466487602', '8597946.28633*1458789930', '25275920.3438*1477571985', '62260950.9882*1485344728', '89249555.2511*1477344285', '51361623.1669*1466580876', '98237775.8726*1479304008', '47970028.3559*1480665171', '23998559.3133*1488029395', '27568723.8218*1472802889', '58467145.7853*1469122248', '24725095.601*1471586467', '89731570.8255*1480475199', '99331192.5664*1476395285', '30502269.8568*1477279580', '37381339.1162*1456709083', '76862171.0664*1480454787', '84082464.8115*1471575259', '65130790.1366*1485205548', '38936270.3253*1483233191', '91443799.3323*1474485651', '43935689.1723*1478098317', '4153441.9336*1478357746', '41812803.8806*1468692367', '35145642.9178*1473400507', '54499118.6*1464044849', '61243220.8628*1467567992', '19365623.5935*1465761699', '67442844.9135*1479872717', '91192673.6577*1470639642', '52708105.0144*1454335269', '15347882.9398*1460053950', '44189324.4472*1457265420', '14778821.4247*1485879471', '85895393.2028*1483592994', '60590799.6698*1483599265', '11148816.1375*1458884024', '86232585.7365*1487075326', '66710885.8375*1478956693', '98867012.208*1480952789', '69509013.1829*1466270950', '12504770.3434*1458081844', '8866768.17625*1481539497', '34499061.5869*1485263422', '46077972.9322*1472544959', '3124760.58536*1485395279', '93090760.3615*1463249637', '63555884.6037*1479427490', '66853775.8182*1462661383', '52042080.377*1462003627', '24227175.6275*1482804587', '90437980.801*1480183336', '3504081.13295*1463715663', '24327626.9785*1456854665', '54953517.1768*1464789777', '99243099.8357*1462815784', '48201887.3237*1456813556', '81045.4824318*1461960956', '20515227.9729*1468883686', '97429732.1402*1465986443', '86238525.5983*1458974874', '20968752.1537*1469776948', '59221794.181*1452968455', '90868181.2054*1478112268', '64451579.2426*1452196091', '69135181.8033*1458564745', '51316381.8352*1460670046', '93718314.8823*1468968537', '25407431.8093*1458785663', '50296801.3673*1486183100', '24612490.5839*1479234164', '91694199.6573*1457666502', '10291942.6213*1485439095', '44276942.6316*1483948730', '15485904.0054*1464141833', '18086123.2087*1453513492', '4177176.37229*1474446745', '58439757.1353*1454957526', '10540072.2824*1456109496', '85530025.9188*1487851132', '16456325.2749*1470966262', '73289821.7599*1464753577', '78802915.8846*1459177715', '43896536.1143*1458531196', '88589483.3584*1485080461', '19340947.2714*1476785604', '34459956.2067*1458595878', '81598570.9096*1483259653', '23270783.0132*1457075383', '25598497.4812*1457652779', '10946186.3793*1473037389', '5530632.80542*1469463928', '64455862.277*1482206548', '72254525.6563*1459867139', '72426708.5911*1459229398', '24924662.8276*1470812899', '23237244.794*1476344014', '46512653.5552*1478396539', '62744205.404*1466181736', '34631496.0295*1467443136', '84675608.7468*1459363878', '84647509.2111*1456251757', '48133542.0706*1478234591', '84671546.6193*1466574574', '10060277.4428*1452753019', '65793202.5431*1457377052', '39920536.0226*1456446767', '59048792.6291*1453294928', '64558634.5541*1486629509', '86343566.7729*1475312323', '42270183.6142*1463089812', '62408550.1248*1460200672', '40801008.2569*1472952230', '36348535.1831*1458456102', '71277820.0522*1462077436', '27392631.0402*1459392886', '98200969.7693*1460952109', '67474018.6174*1483947780', '90895900.2244*1475860032', '46395733.8002*1477562953', '73518504.6612*1472170929', '95832561.8689*1482682067', '90904387.5816*1474525751', '2027114.08191*1473105204', '83354870.3933*1468407061', '24142762.2775*1487818374', '27212129.539*1457094160', '91430031.7272*1469400943', '82317929.6206*1478586028', '32401787.2276*1460090385', '92563594.4395*1471194511', '61172574.1903*1474803041', '36539471.4646*1468003801', '58362393.6467*1468409114', '37295724.2438*1485293729', '32292595.7662*1457634565', '67013884.3971*1478295247', '10551715.5607*1482199372', '42762136.5477*1466476765', '8242966.95828*1476526588', '66904268.7437*1460028125', '52489323.2237*1464598249', '63057359.9299*1484991777', '49105150.7996*1475982160', '8730882.22372*1455473892', '45917872.7009*1480561019', '10680793.3201*1455769501', '52952748.1799*1453890445', '92510437.0247*1473339304', '33196579.9003*1469317311', '69264101.9438*1470719916', '33185160.6213*1476901325', '59549954.775*1481911514', '8172805.65385*1457676654', '67404775.2638*1478678871', '37574346.1133*1456535235', '6804121.9526*1457755035', '79140630.9011*1453064064', '10000572.6672*1467438272', '5541790.13983*1469044780', '4306822.1114*1468479109', '1697035.02726*1464080759', '95512487.9821*1479692073', '26794816.5718*1463520565', '26963982.5932*1487817593', '22920776.7766*1471402615', '14006591.6563*1476103846', '32908841.1525*1483286264', '85136633.5103*1462872571', '72530393.0754*1487428187', '27524224.5842*1471049218', '70827992.3573*1467541141', '73082650.7444*1462955185', '9702808.20974*1487465120', '24741168.4639*1480494802', '89508484.8839*1481132549', '98686651.6976*1481704241', '88048748.5238*1475009896', '82124236.4952*1457109459', '77016940.5391*1474961540', '50181894.4316*1452798084', '43040753.3069*1466485105', '26314005.3432*1486788917', '97469714.8352*1462594050', '36708916.657*1463111804', '93259774.6114*1453256965', '2040888.05964*1478893766', '80190702.58*1487450650', '11586640.6596*1482111604', '72451927.9178*1459730023', '99253365.6946*1453206175', '34459956.2067*1458595878', '31926314.2812*1462872288', '43119330.8845*1465921496', '28491372.7351*1487073712', '64947445.9379*1460681485', '79867781.7952*1460948113', '81098867.0262*1455285711', '87137952.9555*1475809824', '77093318.3706*1471921160', '230802.360214*1476836470', '76384095.2107*1452073276', '44460735.2947*1483786875', '98876751.1525*1462123795', '11549966.1537*1483122267', '80425113.7992*1475807191', '76378460.3227*1466984819', '49693736.7191*1470004346', '69703616.2427*1452287606', '42521193.7385*1481955209', '36815473.1769*1474932788', '37638918.418*1472012045', '41278291.3689*1473638473', '45815448.0306*1485224317', '66248746.476*1455886292', '84069548.3583*1488190209', '49262665.6774*1463031204', '56063298.3368*1484821724', '95522031.9185*1482100215', '72147021.8677*1456247725', '82584653.109*1486225852', '74610127.6064*1454751380', '42161293.2514*1485702152', '61703874.1649*1464320942', '8443758.08312*1465449338', '73404415.0787*1476453778', '35493557.6612*1481862316', '13691837.1803*1459616539', '32779060.6701*1468487480', '78870914.9271*1458297813', '91590410.656*1453361398', '39204750.9195*1486972113', '88084863.471*1459259390', '2613061.70197*1468433077', '4692190.73707*1468473766', '35716615.8905*1465097444', '83795231.1971*1465931088', '40638757.6145*1480699008', '70645219.4427*1456397849', '73544787.956*1470655978', '46901922.4885*1463348123', '45902622.5944*1469477901', '60378057.1684*1470966514', '29529432.2731*1471856824', '6118208.03121*1461657206', '57764829.4827*1456404959', '17987237.436*1469212867', '7536742.21645*1478245803', '92013191.5568*1474993259', '46686983.497*1479897167', '38153474.4301*1474503693', '19688164.9257*1465043539', '82448877.2575*1480373441', '87626001.0719*1487841447', '24851583.4275*1469994470', '91068118.4089*1480289842', '18038643.0752*1466306977', '20269440.2503*1473930591', '32339903.7673*1486903320', '18327800.6744*1460605060', '33488234.325*1477333863', '74324428.8399*1474988425', '3577543.3343*1476845374', '27026503.989*1460300122', '796193.452993*1464549288', '93397072.3336*1459885089', '30188061.19*1458792211', '75637866.5281*1486356856', '66554702.204*1469363628', '25126998.4113*1475612155', '44977399.8241*1483684868', '82733154.6612*1487100107', '92886696.6247*1486717521', '95534030.9709*1454901947', '40294948.6097*1483815357', '21720594.796*1471823078', '60159201.6154*1487136179', '17815425.2408*1479854206', '86657360.9525*1452635710', '47708290.6799*1463195840', '47191246.7678*1476153002', '29857872.4288*1474940736', '76008505.4406*1461152456', '30176068.0637*1458158480', '78536167.9168*1480959292', '15964175.389*1463770557', '15674397.8354*1479644469', '91306168.3485*1471013001', '56886934.5368*1469007730', '4865989.77523*1453631116', '70762729.0175*1463501494', '50006516.7264*1459184170', '35424783.3277*1455395665', '83878924.0499*1452495313', '6951677.52548*1452821166', '47579662.5968*1485894819', '22357794.2639*1455453508', '37178054.284*1458983663', '29015240.1967*1472102023', '48504285.756*1461114030', '44791066.6684*1472419499', '95908236.7977*1455822710', '60911703.714*1456325461', '50463703.319*1476230469', '67226340.6832*1468097464', '49830424.6478*1463977294', '7825229.93052*1452855899', '77866870.8029*1457386138', '55550203.8773*1472171430', '73913981.558*1453671680', '24227175.6275*1482804587', '40574438.6029*1487848669', '92532714.437*1468803396', '73892027.055*1485301421', '43553831.5394*1477208853', '62473459.5006*1473571235', '2540423.1724*1463036487', '54292211.0253*1486506037', '55253658.0404*1456154434', '73424027.2585*1488138834', '14893711.9211*1456084286', '56497031.1605*1484977679', '77724100.8989*1471837505', '94056721.9946*1482418666', '5096404.2981*1482774563', '82185358.6149*1480889112', '76256950.3286*1465977967', '3153453.32497*1479134290', '12267450.0041*1477411388', '20090678.495*1473719687', '39653866.1862*1480952031', '2374168.26026*1472067351', '15475675.524*1484064406', '58408150.0446*1456997263', '78184002.2396*1456639374', '75139442.7772*1481224573', '20633777.4135*1469116954', '81632180.2297*1476871808', '8242966.95828*1476526588', '70062509.7754*1451921618', '82360187.0016*1459175940', '50025417.4934*1482665100', '41080475.8451*1479795956', '94904234.1159*1457930957', '30755615.6058*1480169582', '24846773.7167*1463100685', '26169405.4507*1485138228', '4702291.2807*1464748965', '43613731.6901*1486063842', '15947762.6099*1481087894', '50525629.5595*1470723659', '83134231.7254*1454774360', '44419329.9684*1455786136', '72458990.8205*1487270433', '5383354.21703*1478377784', '1987361.10263*1456698124', '31399349.9439*1483099344', '41096653.4607*1470390062', '22409812.4896*1453049807', '4905001.25304*1453251516', '59319282.2529*1467883843', '81839493.3422*1469150441', '9839341.30192*1462775199', '25276726.9713*1455011818', '91582954.7976*1453597829', '28787970.2707*1471329238', '23411148.5492*1467211156', '6960658.47534*1467713068', '57551953.1555*1452947627', '52910737.3575*1452372730', '65013838.9359*1477752696', '35217790.1779*1459715329', '5701072.77171*1464952201', '42941166.9457*1459850388', '37736960.7147*1488307814', '69137260.1477*1453563835', '55522559.9412*1474369793', '83460410.485*1471447711', '43107194.5882*1466832707', '17046014.402*1458826916', '61465972.212*1478441926', '16968168.1792*1470564684', '26932751.0698*1466794692', '30332640.7714*1483280629', '99378203.4409*1473273269', '75302434.5666*1477705740', '90116931.6941*1457837762', '52282332.0926*1486494234', '34833690.0043*1454436760', '69015044.4015*1459466962', '72256339.8837*1456977037', '80436663.4721*1463231510', '87414169.4691*1452578263', '11023086.4017*1459578491', '17580442.2343*1474132289', '68751079.005*1457863269', '67084384.2351*1453177484', '71458551.8971*1483198389', '77087119.2934*1476770151', '60508515.5983*1460107953', '65334810.3039*1460297005', '46961417.8475*1451940407', '70827992.3573*1467541141', '81827133.4667*1453033841', '74407773.1226*1483051161', '82563890.5329*1457813906', '83822477.489*1453022075', '25039762.2951*1459714071', '92398290.9308*1475244707', '20008243.7137*1469686278', '89437525.3482*1466879965', '92417483.1657*1483085803', '5633176.64812*1478881215', '77707798.2391*1454253916', '52884633.339*1487181702', '18269120.2916*1466775364', '66493444.2541*1462401003', '87800569.4366*1470247222', '9222261.18832*1457504351', '11989995.6049*1472291782', '57901930.4301*1476346187', '484600.335422*1470839853', '90091829.0528*1480613661', '7514041.8878*1451849291', '84165711.5678*1454925128', '63989683.906*1460315268', '90291973.3306*1482428479', '23467117.8*1483943235', '61001142.8914*1460896173', '14887860.244*1465534076', '23515180.6997*1472264510', '69217241.7775*1456418962', '24077158.1806*1468943966', '67228184.2725*1481905788', '80207352.3513*1483508470', '67061144.0941*1468582835', '37175052.9481*1458906241', '94574214.4774*1471930726', '25529943.3081*1483320859', '92340451.4679*1467165688', '65005457.3163*1459752182', '13502143.6546*1453397681', '15195554.4821*1467322448', '98395679.4213*1463236431', '99388214.1475*1463709859', '85471758.5931*1476597522', '97422094.7586*1481645621', '90847665.0588*1459634287', '73509244.756*1465686239', '65651403.4909*1472983704', '74608779.7366*1472134804', '57063232.4157*1488100331', '39662987.621*1472390407', '30492662.4652*1451969401', '5296187.84934*1485343124', '4360803.0256*1469529922', '48189641.9904*1487653008', '93241041.6738*1480711609', '69237645.8673*1464593209', '37543647.0047*1466518183', '20134472.6088*1456356935', '48248608.3891*1487437955', '51160137.0932*1463951459', '12368704.3*1487427072', '47718115.247*1452653775', '48013289.3591*1479663924', '62651134.1497*1469679675', '51260931.4799*1470239313', '75472143.0923*1465478038', '31272354.5319*1479553311', '71897782.8768*1473207817', '61672654.114*1453475784', '26013120.2858*1471382893', '64770782.6278*1474074638', '60861109.8892*1488295605', '45118919.3002*1478202980', '32663415.4184*1484064550', '11100036.1124*1451642296', '43310615.576*1464827550', '53539942.4504*1457416827', '14808858.6973*1485594597', '5988321.79349*1465449276', '89283436.2502*1452794792', '68391229.1294*1465419075', '89856483.4455*1487662683', '29764667.2719*1474840664', '16387742.8805*1456247047', '46676488.7627*1475004617', '41545883.3225*1460842041', '58335563.4463*1456853934', '534105.792209*1467480272', '14111064.1248*1452766808', '24937822.6175*1453708867', '3120553.19323*1453109334', '19097001.5314*1454442500', '53401632.7357*1481962075', '82045523.6945*1470725539', '49830424.6478*1463977294', '81418786.867*1463424789', '39239258.7234*1484919061', '49662250.5575*1485502937', '86081813.7281*1463631463', '55209791.864*1485128752', '37949571.2983*1463172567', '79820927.6497*1467465626', '74739544.5503*1464379353', '62400754.0545*1479427103', '15964175.389*1463770557', '89545239.7112*1475174877', '73732214.6236*1452921598', '86267250.3797*1476587473', '8794931.00874*1486534487', '90437980.801*1480183336', '93850796.4966*1459446726', '4283596.12162*1480816664', '49796278.7784*1465590195', '5530632.80542*1469463928', '65285664.8537*1479648048', '46029026.3002*1475513648', '77087629.3271*1452965865', '76909902.8936*1471468136', '71116325.357*1463240347', '63127170.9214*1462935809', '50397642.5175*1462345726', '22592654.105*1465708340', '87287955.296*1480405261', '63486859.6117*1457815336', '56274660.1362*1475708832', '87079401.2689*1481624122', '36866475.4917*1454133166', '58145176.0326*1452128987', '25230590.8817*1457697219', '55312521.5971*1470744941', '20506511.2528*1453212741', '65986134.8072*1483999357', '55527814.5308*1481195513', '74934575.3129*1459557827', '13999232.3014*1459795085', '74574919.6039*1454628770', '74611358.0075*1453801809', '96657154.1142*1472739110', '43826744.2946*1462850667', '75873280.8573*1461864636', '42260141.3993*1457575905', '31386948.8619*1479273324', '78282411.8389*1478055299', '72751981.362*1458015127', '1845524.07814*1472865599', '98469676.6659*1487350889', '44256392.3655*1473107521', '22768314.8078*1475425901', '17223807.1069*1474710722', '72047548.8802*1472061220', '73545693.6262*1466627385', '70750234.6723*1473316510', '45461537.0677*1486842856', '78546489.1204*1461604252', '86080549.4597*1472989654', '46667207.7752*1479433223', '31462229.2474*1482716886', '42504986.1162*1487353203', '43760985.1473*1458165031', '10904618.762*1485306756', '33360745.3988*1467847757', '2602058.17393*1462931382', '95331859.3014*1465518865', '8867226.92902*1476111242', '28562067.7756*1457224472', '43481935.8239*1472564614', '40170480.2915*1457370855', '93838980.2433*1472385965', '75637866.5281*1486356856', '16738073.6999*1475389646', '76767332.7421*1488299995', '75760412.6484*1482163658', '86592008.7166*1462305858', '47130021.3044*1468906794', '71699727.6657*1464072950', '58608228.7683*1475695941', '69374552.3588*1467151751', '8577798.82841*1453278609', '25924125.5284*1478993462', '97429732.1402*1465986443', '77583638.0756*1480679178', '58713781.4408*1457714920', '64001940.4459*1462996371', '44663440.4598*1473766712', '15525766.3778*1456530231', '24096496.3395*1475062797', '84793006.214*1457065528', '50176325.6081*1475253827', '89773854.6409*1462806860', '10001398.846*1487236019', '7449249.30625*1462583101', '97926405.7388*1457013670', '51800992.7581*1464575049', '88710984.3155*1468086059', '86701204.5967*1467691249', '94019194.8247*1484555386', '41342739.5394*1473301889', '46912709.4378*1461344593', '11401741.1709*1458417780', '12412851.9355*1480677908', '61695486.9391*1467296318', '90116931.6941*1457837762', '95020612.8207*1461110093', '69476036.1621*1465468862', '39716932.2075*1452559315', '42884088.692*1487208431', '92269920.3535*1468306571', '13411395.6287*1472871180', '22491864.4601*1472839731', '40326173.731*1477502384', '53925943.6232*1484865345', '46160350.1171*1483077120', '10723226.2804*1475429256', '83377750.2116*1484012550', '82111216.1147*1468111749', '58848236.907*1460887307', '95234184.983*1452856501', '47604392.0694*1467667562', '33178415.5064*1467643383', '97630212.5162*1460667729', '68182814.0721*1477187875', '67393232.7199*1473010629', '56430790.6382*1483123984', '24741168.4639*1480494802', '37735535.659*1472554597', '36104080.8776*1451866109', '47132028.9321*1481549381', '45729008.1193*1473062706', '78400707.6959*1455364167', '46281152.7862*1485031171', '84517869.1604*1487437414', '37370189.4407*1468435324', '26249082.515*1469174490', '34461855.572*1458124061', '72281156.753*1453185203', '8089556.21754*1482097169', '65478088.4096*1466760845', '51876473.3114*1483969235', '86907911.6148*1457699479', '34843250.7576*1476916676', '66179064.5325*1478430725', '27298622.259*1475452527', '36717160.092*1453900665', '63581293.1331*1478227392', '77724100.8989*1471837505', '1226001.9517*1485149042', '35217790.1779*1459715329', '81573072.5087*1474305259', '31464019.0245*1459932043', '49127612.402*1477335944', '87676234.1149*1480182133', '82042296.7467*1479341025', '75838728.6047*1460976527', '8647265.38586*1475711956', '51331052.4451*1475248005', '82385170.7268*1473924244', '55840677.8939*1466278895', '566581.206043*1482562775', '52576403.7663*1466825018', '11689907.2027*1467381078', '4744097.81192*1474503740', '74392762.6349*1472610448', '24684854.9766*1474675300', '68200114.8008*1466503129', '36081319.462*1472689199', '99490634.5773*1476545182', '27735127.6162*1460303726', '86325425.8893*1473527218', '70859227.0158*1458990609', '10499817.7975*1486697713', '32882663.3812*1487496057', '11740477.9383*1475222113', '43884565.2814*1455690819', '30532020.1656*1476346886', '85136633.5103*1462872571', '53019319.6119*1486161089', '34221429.8949*1469208281', '41834841.3219*1464004286', '87262060.5073*1453816126', '56258407.141*1477060798', '22747551.6654*1475819131', '224188.800447*1483302281', '54499118.6*1464044849', '51446024.3658*1476092722', '55818055.4975*1471636115', '16104338.0131*1463721299', '9968815.7605*1470231043', '44224783.3849*1470711621', '23053404.3841*1456180387', '5273803.73778*1453547182', '46991060.7183*1470632623', '46375739.3738*1461166422', '81247105.9767*1469659465', '88957002.5774*1475124450', '18573600.8108*1484868100', '94685025.4924*1459064337', '81913489.5186*1479253117', '12053347.6408*1458947329', '59240625.5853*1483182264', '81116613.6332*1484216773', '27048275.2235*1472631049', '81700202.4377*1482335200', '68428738.6529*1479937899', '55084550.9971*1465572790', '80491056.3243*1456193229', '94412803.754*1465533840', '47419197.6319*1462780120', '30736088.8419*1453688183', '66691695.8089*1463125313', '68064373.5855*1487044041', '36735437.5507*1481885121', '16221140.5458*1468996470', '56481326.8926*1469388993', '86888720.6768*1470709933', '4007140.47498*1453789032', '65449545.9902*1476861196', '16582015.4309*1476138972', '99542371.8847*1458479561', '47345790.412*1477617645', '13734912.2493*1466084483', '61377774.6128*1467817788', '44238075.5234*1465770805', '14578478.5438*1479096456', '44574636.4358*1456507272', '83503818.3001*1465173985', '88890447.4947*1469025060', '46554143.0351*1471913897', '21649177.0002*1472140697', '125561.724059*1468097172', '1561778.40291*1456288436', '40579437.3889*1452566959', '21465897.001*1454520979', '8206255.63261*1452975753', '75467238.1779*1462373810', '47651756.8542*1469625789', '46730310.8658*1486083885', '14985745.5404*1485567817', '27536321.1583*1461192340', '77583638.0756*1480679178', '29276669.0458*1474093279', '5053786.9476*1475984852', '90583478.031*1478513778', '47510521.6426*1462734014', '73235175.5704*1467804760', '5400190.93326*1467093514', '47129528.4264*1465451360', '63690161.312*1466712092', '99146994.7899*1468744139', '34402892.0023*1475377760', '51212570.0694*1466724230', '43296258.0717*1454710290', '28718936.9737*1455459294', '51770970.7047*1472231192', '28937361.3952*1482819012', '37132887.2501*1484546797', '55277691.4203*1459321557', '47476932.3108*1475785779', '78832584.1605*1467995205', '36735437.5507*1481885121', '97894745.9853*1465121376', '6831330.45945*1465127931', '16089025.3039*1470234999', '46209625.4203*1451844588', '28838734.942*1471215593', '97894745.9853*1465121376', '33125077.1307*1486341583', '11122014.6647*1464476496', '14351061.9747*1472123477', '17059538.1314*1462843455', '53780139.6435*1462243897', '70970851.0346*1467960348', '2201035.82649*1482101449', '15513983.4855*1476966452', '81730559.5598*1472710128', '73191665.4191*1468049263', '39355929.8361*1457694012', '15821883.28*1470844872', '62260850.3796*1473790435', '56437035.4498*1487364725', '20501451.1764*1478158215', '63046294.6518*1467227713', '39416005.1733*1460576438', '23646385.1569*1456588753', '82221640.0793*1472172747', '15069167.8473*1481068517', '43107194.5882*1466832707', '28689257.7628*1455755842', '29128711.7196*1485113412', '87292777.4928*1478578796', '35200172.9318*1480850955', '61023525.5129*1479075772', '94704687.2151*1457357840', '31926314.2812*1462872288', '97582591.9106*1477664883', '60212761.1423*1475595684', '87204741.3238*1481161428', '80440300.7111*1457832779', '50199760.8647*1464613506', '53459107.431*1469248698', '33094541.8672*1461859835', '50335199.1181*1453700380', '85442660.4974*1483700101', '73093799.2926*1486374440', '99055017.969*1483869538', '55426674.2597*1462262940', '8889348.44643*1486780010', '16429861.426*1465786328', '71215465.9904*1463795217', '6539433.96425*1485774035', '75006557.4132*1480114729', '92560579.8954*1486274327', '54562377.5587*1471526206', '17038265.4798*1478504455', '93718314.8823*1468968537', '68829039.885*1460972939', '36607534.8729*1469607556', '38608398.5604*1483933539', '13603855.9017*1464607498', '44467313.616*1461843055', '14140817.382*1458760420', '98474892.4081*1466935405', '62064802.5793*1478177971', '25039762.2951*1459714071', '20784026.468*1455023121', '22200686.0753*1470096015', '5804353.32448*1477790832', '30297710.6162*1484577607', '37842284.6586*1457831748', '81032254.2042*1479543640', '28317297.4944*1459878681', '51525646.6639*1480605385', '81473291.4891*1459100651', '66804161.4602*1463289862', '68244732.748*1452216360', '24553644.8587*1471265968', '63214704.6862*1464297084', '72193510.1749*1480835854', '69329897.9573*1480657958', '3837050.79357*1479100880', '1147887.89183*1456867979', '53202574.9787*1480719578', '40414006.3895*1461663548', '55908447.0452*1464805379', '14834543.4846*1467692082', '41898692.1253*1467049292', '40752545.0836*1482285190', '52570022.0278*1459779761', '86023233.4527*1475797134', '75247896.5889*1476328295', '18523621.7603*1483473147', '2230611.60901*1485826983', '10965350.4311*1472768096', '67594653.9894*1457733955', '70510576.7282*1481772131', '20354762.0873*1458878665', '20951204.1548*1458727899', '47128541.4199*1465412118', '94222229.1437*1466756821', '89340694.7315*1468890786', '22194778.1715*1483841652', '53213198.9244*1456116912', '52590411.9437*1473292100', '4274879.33236*1475098794', '75326546.8344*1458664013', '24437694.7603*1474976547', '23881288.0693*1459024273', '13675413.3762*1454113546', '94487214.1507*1480798690', '99096670.6952*1480824195', '90701040.1403*1487694175', '17471602.4286*1470196229', '87953469.0476*1482320511', '30155072.5774*1465570252', '64721322.1984*1470494344', '57115838.0363*1486017830', '19943056.1715*1459742834', '37622110.9655*1487658331', '76453396.9596*1470598239', '6042259.57904*1460139735', '15262005.0404*1471910784', '82035070.1733*1479918695', '30969399.6606*1480976906', '57915269.8238*1456209859', '5670599.37304*1455601031', '30031630.2231*1486405211', '65037792.4352*1459968159', '90502299.4618*1460809214', '48476254.0778*1473518577', '96598476.2082*1457037310', '14871479.8535*1479578760', '57317432.7962*1469318659', '30476571.3183*1455436764', '5325292.30404*1473437387', '86906043.4997*1479094259', '46274533.7902*1488062828', '38653469.5284*1471426928', '14852018.501*1465698086', '69053545.3331*1467503413', '2478177.56437*1466117397', '60227213.7905*1467766007', '83072347.1762*1467514159', '65956129.9643*1469516222', '7591495.19756*1462677105', '27049920.4035*1466593343', '66783869.9895*1475555239', '48751540.5673*1458492797', '61660657.7218*1485095680', '18964803.7307*1485071639', '59179994.6317*1461805615', '36820929.1434*1467766201', '66643684.8769*1473096725', '96080828.2026*1472431196', '42859657.4404*1472298379', '29967125.5006*1455554279', '14491832.4731*1454070027', '25831045.4214*1479696532', '2004110.29356*1479939874', '47863933.3317*1462195304', '70958029.4151*1480403018', '46611061.9256*1455948503', '20110923.8167*1458106638', '53474514.0208*1474658277', '80830511.9624*1451737630', '7149703.13025*1454083118', '59806220.8632*1464678881', '66708710.6566*1468036739', '19954960.5789*1476261070', '51603380.5636*1472174390', '344963.291615*1480462120', '50386084.6094*1455777606', '25311409.255*1486697368', '22139003.6145*1483113273', '40149180.6846*1471424429', '32430943.9918*1481013706', '10540072.2824*1456109496', '12701565.7484*1454912455', '64521093.5878*1451816310', '31664089.6591*1482341096', '97127619.9674*1478203799', '67599242.5093*1485308489', '52699693.1364*1470816854', '9049547.14952*1466192199', '79460208.2563*1458885262', '54364780.5195*1458165617', '11440260.2093*1456421920', '37702764.927*1462429168', '11878993.7819*1476224878', '12036981.4661*1482676569', '53481792.4888*1451940275', '99243099.8357*1462815784', '49565302.3185*1474928443', '66179064.5325*1478430725', '69543795.2998*1464783149', '98798073.2091*1453680882', '90113875.2264*1473714998', '48108214.1135*1454505804', '51291226.1971*1485398218', '97630212.5162*1460667729', '13973497.4551*1486302472', '97245858.184*1457750519', '63602492.592*1453187208', '31199220.6085*1474614756', '82062706.6109*1457162383', '9839341.30192*1462775199', '99528021.2764*1483150748', '92784707.1199*1461756187', '17634720.7934*1459188221', '81741865.0156*1452080674', '13646528.7087*1478459378', '24288077.0225*1455163576', '33883948.2631*1475998210', '52178396.3042*1470161072', '14527738.5797*1460081347', '96819156.8798*1460686648', '74706403.0339*1488146027', '47708290.6799*1463195840', '17702290.064*1480502534', '69703531.5298*1469362258', '31076179.9488*1480180836', '82584653.109*1486225852', '10725812.1404*1465276374', '29454663.2061*1462550422', '64208060.3072*1482134688', '83626210.2134*1475278079', '60205278.9488*1463872513', '77554862.2235*1468290124', '15969943.5201*1485015144', '16052662.3237*1475082600', '32693833.4949*1483538932', '72040882.0529*1481341272', '44326888.4408*1459365480', '50784994.6158*1456440893', '11307447.0447*1472523561', '74567173.6904*1484052112', '4283596.12162*1480816664', '11989995.6049*1472291782', '4399248.64382*1473839311', '83901697.5486*1456807495', '10182780.0168*1468807114', '82337316.6476*1471272023', '97894745.9853*1465121376', '8578850.03048*1457758290', '46738580.3683*1478610639', '49401558.8201*1471356413', '3565981.41163*1455116091', '77805481.1221*1461381635', '5415252.8005*1464014483', '27739061.8828*1453424792', '6311117.19992*1456614442', '27927194.7981*1468053449', '86169068.4546*1484222223', '80745754.8249*1463484544', '11698260.3393*1472355578', '68345798.7178*1467146097', '84647509.2111*1456251757', '97946161.3748*1487223382', '94222229.1437*1466756821', '30444160.4148*1477918033', '55789174.6814*1476915300', '83919354.626*1478481644', '15200965.9249*1472742985', '51876473.3114*1483969235', '20288144.8645*1485052357', '36219277.4401*1475564383', '21222056.2938*1468721049', '30792395.4671*1466984643', '34843250.7576*1476916676', '39344643.7533*1453868103', '24710362.651*1471921370', '42605973.4433*1470326622', '59367347.5088*1474689388', '5212303.97962*1465710871', '28073989.4932*1460258550', '90779874.4561*1481350606', '46841223.4739*1467658286', '62400260.6377*1459341832', '52822301.0393*1464548406', '48296433.1116*1477969038', '77348061.1588*1470987062', '14887860.244*1465534076', '6616023.79425*1459925478', '58597863.9176*1461519033', '43655741.6602*1463248783', '41338575.5376*1455960887', '38182675.1104*1475355050', '15617.2762814*1462715815', '33838528.9378*1476580612', '87262060.5073*1453816126', '10742436.148*1472323839', '85956117.5666*1482753756', '3716251.46164*1473466690', '51625598.4179*1455070314', '73889182.5907*1458961505', '99425008.0265*1451982108', '44182759.1962*1461323545', '27686036.2304*1454323619', '46069926.4037*1472272935', '93060917.1359*1468498408', '43322232.565*1473042571', '99384160.8585*1477768468', '76057556.559*1464616225', '65678878.1085*1469067541', '28676208.1431*1477620544', '71042695.4408*1487915573', '33881960.686*1477260182', '60241255.7244*1466358268', '56827532.216*1454864257', '10955769.7095*1455702216', '7968489.16805*1458371850', '19888487.5126*1470969794', '24857286.7119*1452848232', '60160960.4837*1471942737', '71556086.1509*1472395516', '99490634.5773*1476545182', '36760606.8587*1473179288', '84268924.2586*1481008727', '50340895.8812*1479070630', '75533384.1735*1467270962', '82394689.6963*1453920355', '40801008.2569*1472952230', '21858367.1994*1472780692', '4054985.98327*1463810846', '10946186.3793*1473037389', '4841856.36532*1480579366', '92232929.8711*1486700138', '52784198.2757*1485429453', '74297659.7397*1465708857', '93838980.2433*1472385965', '12046354.7347*1484296391', '78971818.4165*1481257591', '15838468.6881*1458794400', '54469302.8403*1461206402', '29373713.7856*1464442076', '26314005.3432*1486788917', '38481174.8486*1457382562', '46999500.029*1458608623', '29892442.5205*1480425403', '44977399.8241*1483684868', '46867544.8194*1485505588', '44955464.7388*1456189648', '35186980.5998*1463471190', '68297136.8591*1456188385', '65332395.2933*1479326886', '27596965.4942*1482284385', '54402696.764*1458780732', '61570975.1709*1470980612', '11698260.3393*1472355578', '37288322.8006*1456907775', '16863019.4698*1473505758', '45845221.84*1457902945', '59123678.5767*1468522511', '52708105.0144*1454335269', '32928203.3214*1452058953', '56218377.3754*1456008997', '90404041.3902*1456146009', '41177819.4387*1487343334', '99177737.0464*1474632127', '23627864.6354*1457453427', '16671267.8857*1453008418', '22725794.8048*1463376194', '37406639.7165*1462993430', '79106523.5147*1480628716', '64883735.2425*1481895579', '52256078.7473*1462677749', '17530099.3238*1486390677', '60671392.1766*1469788511', '36208054.3498*1458452523', '45034149.3775*1481535169', '65332395.2933*1479326886', '19369630.304*1477312660', '43720762.8785*1460596579', '838589.981327*1473815937', '70545646.2764*1486935220', '60193961.9658*1454548375', '71664767.0583*1480254275', '39753760.9853*1457985179', '59871538.8513*1468116160', '83478342.2389*1469054271', '44781534.3371*1460029775', '49904715.9065*1459178076', '33953738.3672*1454435643', '20786863.4839*1464590990', '62407856.1822*1485301993', '8572735.12207*1453227295', '69652983.7107*1476172765', '62740356.7586*1457405839', '60932991.4759*1467624350', '57009753.3955*1483183042', '40798016.3622*1476867696', '86555110.6933*1465400207', '98515884.6422*1460021205', '81827133.4667*1453033841', '75281192.7845*1456862784', '79289188.2268*1455091782', '76304755.7187*1473742665', '11576581.3379*1479266514', '7593659.58172*1480300104', '94094686.2281*1460694180', '13532903.8859*1465479522', '15010904.4169*1479292033', '41078036.9646*1458901000', '234249.092451*1478244648', '92110972.0863*1484831578', '38487181.0401*1475434519', '42884088.692*1487208431', '71466982.0215*1461679226', '85002965.3316*1487140210', '13679196.3794*1475286153', '69912009.1272*1458965370', '83768345.1801*1479383376', '84217059.132*1470521017', '9671856.36863*1461397871', '54502767.6129*1462579693', '19705330.7537*1478685848', '38788034.3778*1465659914', '33779288.8434*1487078319', '40704568.5432*1479688478', '99757383.5801*1470945080', '17530099.3238*1486390677', '83558064.9957*1459590149', '28279424.0642*1488017847', '95156917.6397*1469763920', '10312310.7752*1478203714', '83882455.63*1484229982', '33230845.4717*1479182828', '64883735.2425*1481895579', '28040286.4161*1485342474', '65400354.8033*1462106212', '96956113.0642*1458855814', '42453581.1596*1470828917', '12552335.4951*1464581022', '48793155.4402*1477776660', '47097353.0395*1454410566', '13386913.5847*1467872513', '39991761.5801*1475691034', '45917872.7009*1480561019', '1431420.34572*1453585041', '99183522.0522*1478048800', '94686414.4139*1471008516', '15947762.6099*1481087894', '36637974.4844*1475197880', '47154381.092*1470973030', '64677273.8319*1484316641', '79278801.3745*1470987518', '91216861.7876*1483048387', '3409259.81981*1457356315', '13212192.3259*1467194767', '60175400.2741*1458722089', '52073993.2187*1474859071', '41455035.9049*1473766683', '28571541.6915*1460440296', '70715503.6038*1488062362', '66300754.304*1455584371', '24939471.4783*1470155264', '803422.480069*1476821052', '28962079.1133*1484211505', '42137556.6165*1478010438', '24965068.833*1487736761', '35863909.8869*1482621469', '63629021.1417*1469433641', '92835268.5*1477421526', '31856593.1105*1468368981', '17475022.2567*1476158446', '2395600.22251*1484343211', '37030879.8365*1452655684', '18712370.0444*1458561300', '41002971.5768*1477304547', '66364770.7543*1458086796', '16456325.2749*1470966262', '3155739.3684*1484163794', '95770435.1249*1467964170', '41153182.3173*1453134142', '42088500.9666*1484370088', '37068145.1557*1469796800', '44238075.5234*1465770805', '91548685.6278*1470454806', '89260518.1627*1478038272', '82062706.6109*1457162383', '49796278.7784*1465590195', '47886454.0488*1461092905', '64074371.7338*1467375041', '66804838.1681*1483538677', '73556593.6006*1468684843', '42859657.4404*1472298379', '85657162.8738*1471964984', '65903924.6045*1459477311', '86164518.0319*1473112721', '18634389.0116*1468885152', '12127757.8484*1457602368', '62145254.022*1483616470', '75517739.6104*1488293852', '25956733.0979*1456504414', '63447653.6655*1478092551', '28122892.4373*1478568563', '33803923.7592*1454557491', '69724784.9675*1468978806', '73941557.0571*1461631200', '91306168.3485*1471013001', '992387.345545*1463098826', '7417813.14504*1471135271', '67720283.3687*1454587535', '32506216.2746*1468232872', '10069573.1546*1472152533', '61807890.3136*1453925637', '82168230.0448*1452939253', '92041899.4701*1461273469', '1237620.57927*1472591183', '11351422.8239*1479725023', '53242047.2181*1466215401', '34301984.5229*1460193322', '61565663.1882*1455025518', '80820934.1406*1484840397', '92408710.4905*1452665946', '23998559.3133*1488029395', '30801647.3032*1475589258', '44879587.9879*1464768990', '90275912.8709*1483553720', '54499118.6*1464044849', '10141434.791*1484720867', '94658871.4374*1462562531', '68090674.8251*1465467169', '37054885.0889*1484663458', '51954441.4373*1467130709', '44407764.504*1468793612', '11543685.3198*1466753519', '60620608.5511*1453820622', '31655548.0077*1460349200', '96439245.6947*1465823013', '72821006.4913*1488056835', '88083495.4253*1468126059', '29652577.9264*1481707161', '44524327.2014*1483147340', '69453719.0795*1456089984', '60250398.8476*1456297023', '74003643.3481*1480660251', '12155972.9687*1458678223', '77682964.4192*1474388912', '84134770.1931*1458028342', '44189324.4472*1457265420', '12551707.4399*1461938679', '84611888.1176*1469914449', '21082337.6191*1462925012', '47546549.889*1475348880', '41748311.0843*1467798298', '90425804.3373*1455261256', '81369826.4163*1486420347', '35433858.3154*1465201706', '96602725.2951*1482165395', '42591545.6313*1453390947', '82149426.2409*1480024461', '98419997.4652*1452253151', '20515227.9729*1468883686', '55216182.2172*1464398460', '98580697.5861*1485428471', '63863578.6649*1462063975', '14688919.367*1477917906', '82752837.6116*1478979896', '30155072.5774*1465570252', '85227232.5957*1480675497', '70572409.7618*1463616646', '77309299.0071*1484966674', '94609413.3913*1481998156', '53481792.4888*1451940275', '56547457.5362*1466244182', '55974982.1024*1452054902', '32496800.7883*1475883402', '59534679.4851*1470537888', '85525066.5906*1458277134', '20790865.8326*1463300876', '65926700.548*1481429636', '15819326.9882*1469584712', '22060076.6379*1486640319', '42824991.1833*1468840535', '77977132.6647*1463604858', '70378769.4636*1454959270', '78261267.5326*1482607609', '34923788.3982*1480330225', '1311082.98624*1453224654', '42875793.4428*1464776819', '73910687.1881*1470552236', '58576915.6706*1457505556', '96714678.2599*1464413990', '40574438.6029*1487848669', '79305178.2179*1476680831', '59435233.0988*1480198875', '30687203.3082*1475803015', '38893001.4227*1481467631', '56264336.2883*1486328538', '33779288.8434*1487078319', '27541154.5337*1482437151', '14243273.5766*1470512483', '74297659.7397*1465708857', '99778409.8616*1466492591', '14457503.4808*1482959404', '88759519.9992*1453466342', '20076491.7533*1465220792', '74186684.4424*1479896072', '10114243.4765*1481573609', '44661950.1107*1487960824', '75746075.0221*1483601817', '93178805.4317*1487731853', '5014674.18474*1484755301', '31413528.8884*1460811252', '57417903.4187*1476441671', '33892136.0889*1468513288', '34499061.5869*1485263422', '49419451.2307*1452599920', '11804128.9427*1468888480', '54117510.5794*1462729163', '24834119.49*1463568017', '88203108.0285*1484203016', '97972645.2423*1460948548', '86910652.9409*1452727180', '48244827.145*1455540640', '3378230.53796*1475090917', '42881979.554*1472457984', '29178823.9082*1465732568', '66171203.7524*1463261678', '79332709.9743*1452555711', '48214730.0172*1461079234', '45720434.4438*1451982880', '18180685.8351*1466431645', '66813200.8152*1485498360', '79810098.7262*1452208898', '15164763.0293*1479086933', '41831087.1826*1478611757', '46714134.0492*1467180321', '88496896.1238*1479233299', '60546724.5242*1485248401', '91559197.9748*1464418633', '60752139.2396*1472330926', '51400154.1044*1456308665', '18294269.8967*1473322569', '7968489.16805*1458371850', '21199225.0874*1483354478', '55461897.0825*1474953257', '40479485.7978*1470089165', '93334.1293409*1485414827', '59614492.8773*1485562885', '32908841.1525*1483286264', '1045342.16572*1478584127', '9049578.74553*1467991587', '23545068.8407*1456986556', '46999500.029*1458608623', '42742693.2753*1464309159', '45766964.1523*1469596015', '44914761.1394*1470219653', '14874776.3605*1473743159', '68524598.7634*1475631812', '60160960.4837*1471942737', '28846465.0098*1462486276', '38277739.261*1456395729', '26844528.0282*1459134195', '60105353.2425*1484815997', '98165205.4938*1458699184', '76264652.3326*1453558558', '36132849.1228*1478446966', '56678279.993*1460902548', '14243273.5766*1470512483', '77377732.9492*1475268455', '99026024.6626*1481462670', '11214968.5581*1484182748', '57448768.2479*1468961676', '54524701.0006*1467711624', '57875127.0589*1468242572', '95352502.761*1468056791', '83565649.7469*1474289378', '25817339.9452*1461646085', '39569150.4377*1462184742', '67042326.2333*1484402335', '57269327.1262*1465258672', '52724585.3188*1479736899', '55893518.7027*1465861067', '93334.1293409*1485414827', '77941460.2172*1478251826', '78546489.1204*1461604252', '39088990.7467*1460325884', '92168362.6095*1472974950', '38223285.4873*1470729646', '88024288.6456*1464218481', '85773696.31*1484403756', '21720017.0049*1486387956', '25422742.4275*1477082991', '88115826.0282*1460303860', '98060328.2067*1473925079', '58370909.086*1460617269', '22383358.1074*1465579185', '28484703.7289*1466690116', '31094566.9718*1469009767', '34056087.6601*1467657326', '35326345.7218*1463047209', '97822423.8366*1465042273', '28935670.5085*1457790592', '75608108.1949*1465207963', '58106041.1079*1486600054', '99738940.946*1480780106', '94943476.6081*1463690303', '11104671.9594*1451997801', '27065213.2075*1488310882', '85530025.9188*1487851132', '50548265.6707*1459148859', '58608228.7683*1475695941', '64882709.7277*1467044431', '90030953.6853*1481958211', '10990741.7806*1472909726', '20895760.0153*1454226350', '27724796.6631*1475736321', '26512454.4294*1483408534', '11788025.9074*1468435909', '62866485.7666*1455739985', '73619546.0829*1472110936', '21752957.3912*1466112620', '81077243.9756*1453968432', '98924794.0537*1486078018', '81296949.8766*1476931897', '1311082.98624*1453224654', '90509758.8625*1477604257', '99180488.6306*1457753862', '97836652.836*1467042301', '51098504.8423*1454618548', '40026356.6197*1459207091', '53491701.8775*1453615656', '65651403.4909*1472983704', '79650806.7796*1465008499', '12293636.2649*1466927795', '56172930.3727*1463539269', '57936097.0239*1471951187', '60261065.5513*1472041655', '34189730.1296*1482543175', '6431538.45225*1462917917', '87721710.0998*1459876146', '60099199.1194*1476030783', '58465319.9214*1455994733', '67474018.6174*1483947780', '42629352.6874*1468039325', '73129907.289*1484490925', '92898032.7655*1472472376', '88712934.2329*1468251621', '86635106.673*1476154352', '3443316.24798*1460123918', '23840078.3162*1452438952', '43271969.0127*1463677753', '16013125.3418*1461686168', '9041072.42141*1481684635', '18807106.98*1484517730', '70835075.1864*1458220650', '60484898.0906*1473778084', '22491864.4601*1472839731', '96287246.7587*1475177777', '48708732.1638*1483825169', '64976836.7705*1486944402', '16068366.7774*1482230672', '39728236.1165*1469650528', '29562817.523*1463372958', '13348292.3602*1452206051', '65393820.0205*1487073215', '49996670.4414*1479497295', '73000463.882*1475877035', '72451993.7311*1474827364', '93259774.6114*1453256965', '90711950.4307*1466870854', '47446924.1239*1482405233', '77298211.9284*1457397504', '1932091.20437*1477719554', '87794012.8539*1482757364', '85815226.8604*1488122110', '87887868.783*1459610870', '4919977.26238*1473113331', '27401989.7064*1455964958', '17895864.3853*1468340148', '23800893.1403*1457151062', '34371388.6558*1451803879', '15680900.2015*1471385858', '50699992.8023*1482306040', '5661716.29983*1460612549', '86942220.6194*1475125319', '68585340.084*1474249007', '21177856.4307*1486630350', '92041899.4701*1461273469', '45106044.4076*1480877578', '51795715.1239*1451697649', '13007227.6218*1459875124', '25946433.295*1473020118', '42044262.8489*1486598989', '44040504.4309*1468950936', '31472543.347*1461017696', '2320591.46707*1467845311', '58279398.2226*1452854090', '48889759.9515*1487171107', '99777541.5359*1470421977', '99175901.639*1486872260', '55365562.5196*1454966454', '23649468.298*1467851147', '5341380.00427*1488034140', '20775605.7234*1452399973', '65664460.435*1475918849', '89731570.8255*1480475199', '65756845.8513*1462458360', '23668089.9853*1469055418', '21338864.2679*1484150578', '16678982.0657*1461859735', '62863164.9271*1458138671', '52090681.809*1477036772', '3120553.19323*1453109334', '51103804.922*1464382943', '81757439.9818*1460955735', '25247755.0266*1456542382', '51929332.5738*1454463929', '54277344.6803*1459917887', '99026844.1406*1462477876', '54069914.6409*1473144227', '19135350.0295*1452672902', '76594627.3866*1473520625', '37973136.6478*1475312218', '18796415.7935*1471793924', '91306168.3485*1471013001', '13999978.661*1474014172', '48875622.5846*1473913617', '23841156.0603*1472996675', '35253585.4196*1453023061', '71214400.0605*1487045888', '99886607.9317*1474523662', '38889044.8759*1463356807', '50428858.8785*1487139411', '92130167.2575*1474836135', '38620166.0709*1459679738', '96016074.0495*1485498424', '68070316.5058*1473597112', '70445791.9261*1462111958', '4543805.18693*1454016298', '98091239.298*1459713634', '94412803.754*1465533840'], u_time_all=[1462383970, 1478136108, 1465695904, 1476955551, 1470647917, 1468048947, 1452816239, 1468547747, 1472411160, 1458539976, 1455522621, 1463771383, 1462344575, 1483302924, 1464335852, 1462083228, 1467527448, 1474273158, 1455745939, 1464452091, 1482822662, 1477208853, 1459915257, 1476343777, 1476662094, 1462173032, 1453290881, 1477200953, 1453689175, 1455751592, 1470651624, 1460266258, 1456249594, 1466904670, 1459201442, 1452119971, 1459259673, 1474879917, 1468711626, 1463929366, 1454002554, 1478584127, 1470628379, 1479980717, 1481557857, 1474658277, 1468954483, 1452405394, 1481209893, 1487022197, 1470537888, 1462149244, 1476364017, 1487734413, 1472006245, 1487965733, 1472426879, 1459198975, 1478562172, 1472411160, 1460750221, 1482730117, 1464057941, 1470684073, 1470406523, 1477079202, 1477563699, 1452125095, 1468773873, 1486264947, 1481595789, 1483211638, 1469955065, 1477504480, 1462607979, 1472174390, 1463312173, 1474999838, 1486363777, 1461249959, 1453098110, 1453709033, 1472444422, 1486154663, 1475378684, 1482295243, 1466408899, 1482465613, 1455765648, 1452968455, 1472280453, 1479492587, 1469295631, 1456880895, 1462111884, 1479560259, 1461990798, 1459968159, 1472385965, 1456986556, 1474812541, 1482056186, 1481535169, 1485200541, 1456472516, 1483926341, 1483307602, 1462908723, 1463980128, 1469055418, 1484472887, 1474702980, 1474940736, 1464563772, 1456548289, 1472987703, 1483791040, 1453845989, 1483999357, 1470958687, 1473803238, 1457140502, 1455318216, 1458271141, 1455171344, 1475047002, 1458025711, 1459854002, 1467003579, 1472645803, 1479433223, 1454159939, 1463998790, 1468229492, 1464301288, 1458906241, 1483199984, 1480462336, 1471065449, 1483966148, 1483544160, 1452405394, 1465390354, 1455023121, 1487759915, 1474737666, 1480002856, 1452413435, 1475234127, 1464742244, 1475286661, 1483903896, 1459862094, 1461519905, 1467613149, 1484893770, 1461313763, 1470337438, 1468120460, 1477752060, 1483593350, 1481796763, 1483317858, 1458362067, 1483662118, 1484394873, 1474240582, 1464208480, 1456193229, 1486953312, 1486478769, 1478561078, 1456103885, 1472651110, 1471886048, 1462258520, 1456274073, 1457744397, 1473144227, 1467610671, 1453621894, 1472659350, 1484414747, 1455881593, 1465029725, 1484624891, 1484082868, 1482904557, 1485334410, 1467896655, 1486697570, 1474544535, 1468522511, 1460200360, 1478830092, 1476829288, 1473963389, 1453890445, 1481728579, 1461145974, 1456510217, 1473397746, 1483767063, 1478831086, 1470390268, 1485643255, 1464520726, 1451881008, 1453457120, 1464182203, 1472489099, 1455376451, 1472050715, 1481597023, 1459517925, 1472029624, 1481036592, 1453968432, 1488077301, 1488146854, 1468251028, 1471228354, 1479672155, 1476172029, 1459173798, 1487405204, 1483602117, 1470275318, 1465449790, 1470025692, 1452604122, 1451704165, 1478674633, 1482044336, 1455047319, 1463493959, 1478640603, 1479031178, 1470402332, 1478794927, 1464699702, 1481112283, 1474676527, 1473130248, 1463958332, 1483749508, 1463651530, 1472264628, 1457273482, 1460107953, 1464072950, 1475051931, 1478992492, 1466215521, 1457712502, 1482521867, 1475204260, 1475300352, 1486380807, 1482337598, 1457453427, 1485403125, 1487402938, 1471385858, 1469532439, 1464494966, 1473783403, 1453912917, 1476154144, 1475764983, 1465377155, 1486845540, 1467265828, 1457736654, 1467261920, 1471218839, 1467546952, 1483586745, 1479560259, 1460756076, 1460305469, 1466345863, 1470296530, 1453513833, 1472552067, 1483794305, 1464931071, 1472006245, 1466590713, 1461206402, 1485162481, 1468086059, 1453343936, 1453098110, 1453103934, 1465683117, 1452578263, 1480201150, 1474411864, 1466283123, 1467318996, 1472858414, 1469044780, 1485985018, 1486457504, 1467237713, 1469463781, 1464611701, 1475684092, 1480065724, 1455444657, 1472867294, 1488293852, 1456164248, 1484143884, 1484595730, 1487233955, 1476187553, 1474689388, 1484562265, 1479182828, 1477468057, 1483435555, 1458093093, 1458080011, 1469522184, 1465886531, 1479939874, 1453299753, 1456921458, 1461079234, 1459959727, 1468422952, 1474587624, 1483310665, 1452333329, 1486202283, 1462665453, 1454551879, 1471586467, 1453523098, 1467283468, 1456535235, 1472712923, 1468055758, 1485095913, 1482013737, 1468808003, 1486889022, 1487353203, 1453201571, 1452288867, 1456359902, 1460728454, 1482274753, 1481876143, 1452581752, 1474692794, 1487668202, 1479149677, 1454917435, 1459953327, 1481896082, 1486687096, 1454650397, 1483462180, 1466674779, 1452008988, 1464753577, 1488021081, 1472041655, 1487910366, 1480659475, 1453496513, 1455007512, 1460327808, 1481291978, 1487662579, 1484666960, 1477549371, 1455808865, 1478091117, 1455921753, 1459222319, 1485078416, 1463746160, 1480983820, 1463923947, 1456986556, 1471231013, 1474586612, 1473676997, 1466886648, 1473556146, 1453372768, 1469883062, 1462436536, 1470486524, 1481510794, 1479738009, 1474368068, 1460730540, 1478754471, 1463838931, 1481025861, 1477684899, 1466234067, 1467780176, 1465621384, 1472232133, 1458987833, 1475491203, 1487877064, 1477604257, 1470265753, 1474310976, 1465050879, 1468380414, 1467039912, 1471552539, 1454577825, 1485836882, 1471155157, 1458159718, 1468442606, 1467658286, 1488294907, 1484088064, 1487550522, 1454946135, 1477327809, 1469174490, 1461895726, 1461198205, 1463356807, 1484562265, 1457294484, 1472029624, 1487431318, 1456252683, 1453944332, 1452655684, 1467629313, 1461773549, 1468720741, 1484364602, 1453771251, 1453945396, 1469063080, 1484077876, 1481053585, 1486974712, 1467144343, 1461464892, 1476931897, 1475687538, 1456885760, 1454456460, 1467236668, 1474733576, 1473923367, 1472016535, 1455241006, 1464957168, 1481800321, 1454627808, 1475348880, 1484736473, 1456608749, 1457199486, 1454479886, 1462979862, 1459896522, 1484182748, 1469405937, 1452113567, 1452327342, 1473432942, 1478842685, 1472299272, 1471248360, 1452975707, 1452831852, 1459025780, 1462526398, 1467988992, 1465251598, 1486925563, 1486817398, 1452346423, 1470974157, 1460796736, 1475487386, 1470830277, 1466124316, 1462246953, 1464687961, 1454904515, 1464782630, 1476158446, 1464900508, 1455376717, 1481240542, 1454902806, 1470821942, 1478339900, 1484071939, 1457732124, 1461129070, 1481965506, 1468251877, 1473624237, 1455697338, 1480877578, 1473676997, 1467560493, 1467310155, 1473117622, 1470723659, 1473769762, 1471962101, 1478287047, 1474388912, 1464320331, 1484291921, 1483606251, 1471431857, 1463180841, 1475819131, 1481036592, 1476916676, 1485403942, 1465957423, 1474553993, 1462469782, 1471805603, 1467907854, 1466636956, 1468580839, 1462885248, 1451744398, 1468325414, 1460338235, 1487510444, 1475846389, 1462627441, 1473803238, 1457275266, 1468686693, 1465230292, 1474147915, 1477064596, 1455856975, 1485425870, 1481334081, 1480705117, 1463309367, 1477776660, 1469040783, 1477327645, 1462720927, 1487788525, 1473030221, 1472554597, 1452672902, 1461644791, 1454331975, 1476802829, 1459270738, 1458415505, 1469111040, 1455161152, 1479850639, 1462524869, 1458478120, 1479836450, 1469017087, 1475687538, 1472378164, 1480337951, 1467527703, 1461735974, 1478121095, 1454252796, 1469964065, 1470717408, 1476659308, 1484887812, 1469170262, 1461945735, 1455028846, 1465353493, 1471633395, 1459417091, 1473962513, 1475947766, 1486924490, 1466852996, 1455496864, 1472665022, 1481349785, 1475612679, 1452990876, 1457302840, 1469831449, 1457375605, 1482700847, 1474485651, 1458958989, 1485267186, 1485610113, 1480271031, 1465848897, 1467375041, 1461019168, 1464337755, 1467929039, 1465518865, 1484242512, 1460069670, 1461265529, 1486931588, 1471365913, 1478970880, 1457784165, 1469529922, 1471555412, 1458992428, 1477350296, 1473216006, 1482055414, 1484327382, 1466469303, 1456074324, 1473511477, 1474791425, 1454863159, 1468341568, 1473705521, 1470439810, 1459705100, 1486470340, 1487448705, 1460138611, 1471248360, 1473045439, 1480665171, 1477119995, 1485169111, 1473742453, 1465654668, 1454323619, 1478685848, 1459730023, 1467301884, 1464590990, 1482943079, 1480660251, 1452174336, 1469714839, 1457068593, 1480877490, 1479970552, 1458628866, 1465713640, 1472530594, 1462723885, 1456547588, 1462908723, 1474218104, 1454409671, 1458295554, 1460436482, 1457392699, 1478970880, 1461698419, 1469506346, 1469213474, 1487110425, 1462383970, 1473996898, 1486553274, 1452356249, 1477041119, 1481728579, 1464580461, 1483059504, 1457791328, 1454029420, 1463647086, 1455974855, 1472000961, 1456303924, 1482706455, 1461180807, 1472111541, 1479956926, 1461822377, 1464214374, 1474676805, 1488304300, 1483656720, 1476711830, 1470360410, 1456851509, 1467987090, 1461863665, 1479497295, 1466552489, 1477700433, 1454664482, 1471812860, 1471885404, 1481804226, 1457902945, 1459957054, 1486584484, 1459326817, 1465876215, 1472712923, 1478674087, 1456686177, 1452768022, 1476613838, 1474758670, 1471445443, 1457668470, 1466400231, 1459386771, 1477633906, 1452341096, 1453062175, 1464321870, 1486623148, 1453698824, 1465212852, 1455171614, 1455974855, 1468809304, 1461531377, 1479064931, 1460842041, 1458001110, 1487851146, 1478065412, 1471749640, 1461641937, 1465257194, 1458177046, 1471850906, 1454927567, 1474059510, 1461195000, 1466302737, 1484444914, 1463240347, 1471172968, 1453931943, 1470573781, 1472529656, 1476901382, 1459756566, 1457313615, 1485278432, 1453932043, 1456080555, 1476026075, 1457507542, 1461263414, 1484546797, 1459616539, 1474932788, 1454323619, 1484149008, 1457120256, 1487940164, 1479964789, 1475090917, 1484627238, 1454234791, 1457331811, 1472012412, 1477031974, 1482353102, 1460566109, 1474898079, 1474728050, 1484109302, 1464015844, 1461515316, 1472259903, 1476662094, 1455815009, 1485484873, 1481467631, 1463594975, 1466918112, 1477224971, 1478295834, 1482163658, 1476595150, 1461063265, 1470060954, 1456405136, 1454399632, 1487884387, 1460469174, 1465365864, 1487612106, 1475390338, 1488195719, 1457068593, 1469357843, 1485790278, 1477442797, 1475390338, 1464577672, 1460126155, 1477555341, 1474694573, 1471925648, 1478748976, 1454930246, 1478466464, 1463690516, 1455349984, 1455568641, 1465986582, 1479616945, 1486343234, 1467241182, 1483295487, 1456123988, 1483147494, 1469075370, 1461669026, 1465897572, 1471426928, 1454975992, 1468612967, 1483696071, 1477776120, 1479253117, 1456468100, 1487352159, 1458053458, 1474261815, 1465407137, 1452714322, 1483431557, 1477743182, 1455475665, 1480532726, 1468916983, 1459066998, 1478954251, 1487841885, 1462830407, 1455475665, 1473050157, 1483518120, 1469794798, 1454930090, 1466936553, 1452648070, 1481241860, 1457867513, 1460918494, 1468720741, 1473189371, 1456364297, 1459930249, 1465486158, 1468644483, 1468765411, 1474323603, 1454506025, 1459876146, 1463564700, 1472163178, 1482659850, 1481321700, 1484840397, 1481800198, 1482442799, 1451996117, 1486732074, 1478997024, 1470280524, 1480658632, 1486166613, 1455388310, 1487230867, 1452652212, 1459550471, 1483648659, 1485018370, 1455099366, 1465835684, 1458611875, 1476770151, 1463200192, 1455237667, 1462186275, 1467808323, 1463895517, 1478668217, 1474288205, 1466907845, 1485220778, 1480562505, 1460353341, 1476767279, 1452611559, 1480468252, 1465123048, 1477374676, 1475139007, 1479683895, 1481041422, 1466239651, 1487548766, 1486297640, 1473656406, 1466666691, 1473414876, 1480462336, 1455299230, 1480241858, 1465086932, 1451866109, 1475276782, 1468060836, 1483167464, 1458976500, 1483307417, 1462775721, 1478032243, 1471917573, 1459132726, 1466639260, 1485128005, 1479465592, 1457714920, 1488186396, 1452879203, 1474706236, 1487997212, 1456836602, 1452650689, 1471331774, 1456680773, 1467879372, 1487199104, 1453758649, 1456882567, 1456685834, 1451990236, 1467199181, 1481124440, 1460302455, 1451739437, 1486092109, 1454779263, 1473395411, 1481643167, 1456143816, 1476901325, 1462267954, 1453332112, 1468614918, 1470155264, 1479407549, 1477024568, 1475605932, 1471329387, 1488315093, 1464738437, 1457675085, 1453420059, 1453701543, 1466270950, 1456811901, 1479203847, 1471781801, 1457984559, 1476343777, 1460145199, 1452150718, 1458750578, 1467208813, 1456976059, 1460194342, 1454426778, 1453522238, 1481560952, 1463269853, 1469191167, 1472004874, 1479743465, 1464560100, 1466034349, 1453860255, 1471473377, 1486561877, 1469044780, 1466760845, 1473067590, 1472881426, 1466691986, 1487661706, 1465743242, 1476584144, 1464332609, 1454140970, 1486381413, 1474261815, 1480364782, 1466825018, 1454164927, 1468156206, 1475182171, 1454925128, 1461189132, 1487836481, 1454337334, 1477320523, 1480809608, 1470693697, 1480217559, 1454346239, 1473017955, 1483097630, 1451754280, 1488062362, 1461577921, 1477469262, 1465827219, 1455378963, 1459449075, 1454014755, 1475773137, 1456985398, 1469777097, 1467007344, 1472149357, 1467218820, 1476295065, 1465616515, 1470074388, 1476387894, 1459193502, 1473241025, 1487974617, 1476883859, 1468126059, 1453587326, 1473506286, 1466239281, 1464301967, 1481257591, 1453398547, 1456003191, 1452334132, 1475282117, 1473753803, 1458547049, 1453661080, 1465455754, 1484414747, 1456089984, 1471539718, 1484316641, 1479298914, 1464002139, 1468174720, 1461221108, 1463667095, 1451656253, 1478661741, 1469269255, 1473437387, 1455905915, 1471730203, 1459932043, 1458858064, 1485350387, 1479097701, 1455568641, 1454234791, 1461452536, 1462106212, 1480850955, 1481362743, 1458356486, 1485879471, 1472355578, 1474871092, 1454774360, 1460315268, 1470246741, 1476295065, 1462770053, 1487199223, 1476792498, 1464506639, 1472508021, 1458245776, 1470298861, 1467760761, 1465505988, 1472349962, 1474460213, 1463334054, 1462769221, 1464004286, 1468255288, 1486317191, 1460389559, 1460966037, 1487670782, 1469345557, 1487114895, 1473105204, 1482274753, 1484079692, 1460925128, 1472532804, 1477720255, 1455019422, 1465087911, 1467987090, 1472591222, 1468766751, 1467443136, 1486033074, 1454229565, 1459327391, 1461986187, 1463540745, 1462338390, 1467896655, 1479176697, 1462276744, 1474459921, 1467799415, 1453486849, 1466332912, 1479844593, 1453848928, 1472878317, 1460851111, 1456413285, 1455852426, 1454270358, 1465723274, 1478992492, 1455832036, 1457412365, 1465718342, 1470360943, 1487939074, 1479004749, 1471771346, 1458295437, 1471016537, 1462294273, 1461541886, 1467595107, 1466892793, 1478138994, 1456535235, 1481074236, 1459932043, 1453895459, 1465315212, 1466027738, 1484205548, 1484258313, 1463068973, 1483874687, 1470747662, 1474637712, 1467536479, 1452635710, 1462458360, 1457197692, 1461581835, 1453605272, 1471333041, 1475043285, 1484860562, 1466860466, 1477512614, 1464332609, 1461294559, 1455776452, 1468110372, 1458405589, 1484547516, 1487365740, 1458727899, 1483430299, 1460796420, 1462943699, 1474431780, 1480053656, 1486623148, 1479098448, 1478561078, 1465921496, 1487125638, 1471021662, 1473299427, 1453080014, 1466301333, 1458254212, 1477893073, 1466878400, 1467179948, 1464219144, 1455738843, 1478362297, 1458167732, 1462303496, 1454728612, 1473061673, 1468584001, 1481857884, 1471852533, 1486957106, 1466366160, 1472225346, 1479007929, 1473950752, 1469363628, 1469738793, 1464698367, 1466569099, 1468448794, 1467842258, 1475425901, 1485071639, 1464705876, 1452968117, 1458903709, 1471942265, 1458119550, 1480951462, 1459264890, 1477752060, 1476087676, 1478428834, 1485215515, 1482329788, 1480315999, 1453022075, 1452937982, 1485923174, 1465909695, 1467244784, 1478197467, 1479292764, 1485370767, 1452536984, 1468030250, 1460864711, 1475776898, 1468378950, 1487122962, 1472034401, 1457956050, 1475529729, 1465423958, 1458154904, 1485310854, 1472906706, 1477201748, 1461860066, 1482567187, 1466412159, 1487441646, 1457525051, 1476422520, 1465614206, 1472110936, 1480719578, 1452539800, 1478591291, 1484332344, 1481484187, 1485487496, 1454824673, 1454083118, 1467415031, 1470074388, 1469157612, 1476680831, 1469889307, 1473681219, 1457348733, 1475790764, 1462266365, 1482409439, 1487527668, 1461552428, 1456991938, 1478133262, 1473959224, 1474162606, 1452779418, 1475708832, 1456754811, 1457089218, 1487236019, 1459467502, 1482830973, 1485377691, 1453782354, 1467401969, 1486050359, 1478831086, 1477406016, 1472029839, 1454547014, 1486534436, 1465686239, 1476974214, 1454307713, 1466954696, 1468228401, 1471545829, 1468863158, 1469838746, 1476659308, 1458086796, 1484253457, 1484414591, 1453848928, 1477207043, 1467408338, 1478178972, 1472639441, 1462634565, 1456817576, 1455091782, 1487976639, 1469363659, 1459972787, 1485328621, 1485031171, 1470406523, 1471166606, 1478900236, 1478570492, 1486700138, 1452528419, 1461105104, 1471492142, 1459207091, 1460940707, 1464506639, 1482757364, 1455473336, 1473974358, 1473260875, 1482404482, 1472640946, 1466966210, 1454946135, 1484805216, 1461386614, 1474638958, 1483933539, 1462508946, 1467570445, 1472291782, 1479425416, 1484387383, 1461826961, 1452807355, 1456148146, 1473037616, 1461986917, 1469537154, 1485643310, 1456709554, 1451860532, 1452353837, 1478678842, 1462925012, 1483273017, 1473571235, 1478441481, 1483090219, 1465659914, 1454360848, 1457012287, 1460692516, 1476359917, 1465631420, 1488279350, 1487650200, 1466536991, 1459730023, 1457584969, 1472578636, 1458333715, 1452167835, 1471818290, 1485910750, 1463067300, 1470266996, 1461530870, 1478386938, 1483955727, 1475528811, 1479128305, 1486842856, 1470763368, 1476093110, 1483791354, 1460363487, 1475886827, 1451995791, 1455292421, 1478098317, 1484724222, 1471087427, 1455768519, 1481561918, 1461729029, 1462048714, 1475934588, 1467049089, 1466270247, 1452960650, 1472829855, 1468119841, 1460267142, 1479075772, 1456607527, 1467851627, 1470337438, 1483685771, 1486420347, 1461356627, 1480581133, 1453932661, 1469423465, 1463492998, 1465480589, 1458093093, 1486153663, 1481681342, 1459193502, 1461225587, 1474202301, 1471872151, 1455937730, 1468409114, 1460353341, 1470718154, 1451754280, 1458271141, 1453876393, 1459056940, 1461632921, 1469764471, 1471775205, 1462424576, 1471536483, 1462750728, 1459609059, 1488031693, 1479126434, 1457313892, 1462908723, 1486630847, 1453098407, 1458246166, 1482210470, 1462896748, 1468614918, 1459176457, 1479176236, 1481759297, 1458565064, 1460954706, 1464232257, 1469023605, 1454316695, 1454308703, 1475886884, 1479956926, 1485343124, 1479964789, 1478888218, 1481696067, 1462930618, 1469438275, 1452356280, 1456126595, 1486094949, 1471725152, 1472376802, 1483183042, 1484860932, 1453789740, 1463788343, 1461280113, 1465479931, 1456658943, 1454007716, 1474961540, 1481535169, 1456422910, 1486979018, 1455369581, 1465708857, 1467287499, 1455082313, 1455735272, 1483074134, 1456968823, 1483116199, 1484368300, 1462987107, 1485893379, 1458739337, 1464773046, 1454178197, 1475367226, 1483962560, 1466569099, 1473520051, 1458183851, 1453278609, 1463485782, 1454444150, 1481804226, 1458295554, 1483199238, 1469476107, 1479731545, 1465141344, 1476344014, 1486736608, 1486111540, 1487837423, 1473275925, 1455277245, 1464186923, 1456902769, 1462093134, 1483088016, 1461426005, 1459830891, 1477424812, 1461456397, 1473514790, 1467770969, 1487100226, 1465916333, 1480734606, 1462497767, 1484503600, 1452921598, 1487352159, 1472968298, 1482997531, 1485099835, 1476115826, 1471886048, 1483277921, 1477269834, 1455357736, 1485128752, 1462815518, 1462602522, 1461083748, 1478893766, 1464099685, 1464643565, 1487539117, 1482174872, 1457712502, 1454244594, 1461881529, 1465549270, 1462041061, 1454918800, 1474604198, 1469234459, 1482983563, 1478897191, 1470644537, 1458836939, 1466181868, 1470809179, 1461235999, 1460922116, 1467278017, 1452865700, 1466124316, 1482804587, 1469796800, 1470497477, 1473921386, 1482655853, 1461559689, 1479795956, 1467329584, 1475979761, 1459507985, 1468574675, 1476664006, 1460430595, 1472172747, 1481320989, 1483690926, 1484094640, 1459422386, 1484345865, 1469507347, 1457432565, 1484299508, 1472732054, 1481209893, 1454130410, 1476526588, 1473952196, 1482206548, 1475010101, 1463339472, 1473778084, 1466181736, 1452668562, 1477153835, 1485389662, 1470958687, 1487138107, 1480835911, 1476295065, 1476052789, 1477015693, 1478514041, 1478749685, 1460140610, 1487471205, 1461019168, 1459377896, 1486550089, 1459509883, 1456539510, 1478272515, 1467395133, 1455303231, 1452732126, 1469040783, 1469839942, 1457455848, 1487893453, 1485332353, 1477788331, 1453425660, 1456884796, 1458760420, 1462617888, 1469191167, 1471160839, 1472252821, 1453558558, 1475518882, 1456104541, 1466235461, 1451948169, 1467832352, 1459957054, 1476567111, 1452379508, 1454216853, 1459515893, 1466886639, 1467699949, 1476655837, 1467643383, 1461969360, 1461283377, 1484205875, 1457302840, 1457080501, 1462317529, 1487328431, 1471461074, 1456189018, 1458326321, 1465945073, 1481104645, 1457882295, 1486363777, 1473182148, 1476030783, 1466398587, 1460503168, 1471476882, 1481871110, 1478748976, 1456907775, 1474043245, 1479915356, 1479099409, 1463367411, 1458670547, 1455784536, 1486022269, 1470868847, 1486903320, 1458859746, 1473831707, 1470817648, 1479075545, 1463963368, 1452650689, 1472574723, 1465479522, 1464268076, 1464888282, 1471441015, 1470868989, 1487223382, 1470655978, 1488104805, 1476095630, 1456608749, 1471473775, 1470148718, 1453559946, 1451687279, 1484345865, 1484380110, 1466174885, 1456450203, 1485117525, 1458282622, 1480527782, 1454447518, 1454253916, 1467723817, 1483273017, 1487395937, 1485398022, 1476179137, 1472482217, 1456730665, 1457930957, 1475809824, 1481440909, 1477756260, 1478660350, 1484147419, 1452757700, 1464957443, 1475845026, 1461864636, 1469883820, 1461955488, 1467614920, 1478290849, 1479182317, 1456672387, 1465220792, 1461128473, 1479925704, 1468174720, 1485461790, 1454159939, 1461452865, 1469362258, 1457094160, 1484258313, 1460324653, 1472089377, 1469986624, 1474940736, 1462128933, 1456089156, 1482402046, 1452218280, 1483018139, 1466694059, 1478272515, 1469914449, 1453534129, 1458187487, 1487330687, 1467766007, 1461617708, 1475263897, 1480093079, 1465248810, 1485351985, 1476356027, 1484836565, 1478002644, 1454643414, 1474831215, 1465480589, 1468155888, 1471298424, 1484992764, 1486033074, 1465221086, 1471670989, 1487874625, 1451949746, 1469243596, 1486347903, 1475429256, 1463498939, 1452903894, 1484660868, 1479734139, 1462317529, 1464301288, 1459361883, 1460348989, 1485232490, 1479097701, 1455786985, 1480670841, 1474805438, 1484134890, 1479905531, 1476477456, 1468487480, 1485450857, 1466480743, 1468686693, 1476670552, 1472329739, 1468845716, 1480659475, 1483362949, 1464186923, 1454463929, 1453053092, 1456539510, 1455204975, 1469212867, 1473500874, 1465732568, 1482177480, 1470611685, 1487595488, 1461541886, 1472806883, 1464604014, 1480284809, 1473055115, 1457965722, 1474800592, 1482723552, 1487649036, 1473027213, 1478685848, 1480914093, 1459419476, 1472298379, 1456075049, 1479684322, 1479253511, 1482889629, 1482146934, 1481087894, 1466949406, 1472012412, 1452531371, 1472710128, 1487465120, 1480024461, 1484624920, 1468544171, 1479684322, 1468793128, 1487794550, 1487848490, 1454053805, 1456706558, 1483969235, 1478897191, 1487226052, 1463771383, 1469362258, 1459530014, 1467708359, 1458780732, 1454927400, 1482789027, 1478136108, 1487871526, 1465491845, 1454519860, 1458081844, 1487490041, 1457347854, 1461462576, 1462305196, 1481220288, 1463225983, 1477504480, 1461564862, 1472943217, 1452280846, 1471889806, 1458220650, 1454309437, 1470014427, 1455634429, 1480363317, 1474110631, 1465484059, 1487595935, 1481556993, 1460021205, 1456446767, 1471244459, 1479427490, 1475314173, 1465757742, 1455730097, 1458990609, 1468251621, 1464413990, 1483113273, 1453502539, 1477001630, 1482732848, 1474224523, 1487003839, 1473575380, 1453770702, 1468684395, 1470924750, 1475481366, 1467265045, 1467699949, 1455400140, 1465516606, 1474173392, 1457357840, 1463334054, 1455905915, 1471117663, 1487032827, 1470151211, 1486106261, 1457511120, 1464162696, 1474047085, 1454282198, 1453106140, 1474702980, 1458326321, 1472905888, 1472489099, 1476007562, 1455210973, 1482238749, 1469541912, 1474533788, 1460496018, 1457526215, 1482274753, 1483558510, 1480670841, 1485984906, 1470987518, 1455862076, 1486810617, 1463573925, 1463671986, 1456252683, 1461344593, 1478440209, 1462822684, 1477001630, 1464382943, 1451996117, 1454296396, 1466491136, 1469790518, 1482100215, 1479086887, 1458026078, 1469067541, 1452420153, 1465062924, 1461462941, 1474586612, 1453224654, 1466954989, 1472089377, 1452056349, 1458205908, 1454930246, 1483742146, 1487020342, 1483954045, 1463334054, 1486548023, 1465718696, 1465992022, 1462930618, 1468900202, 1472682691, 1471793924, 1451910519, 1459289045, 1476375260, 1469200422, 1484106767, 1459609059, 1455862076, 1454133166, 1483090886, 1456847109, 1467136294, 1485278432, 1468128169, 1474173098, 1462766811, 1466751331, 1458277912, 1479986056, 1452959901, 1484399374, 1484460505, 1457831748, 1473110984, 1461057913, 1485450857, 1463690516, 1463951887, 1475692040, 1482607609, 1481851997, 1485974856, 1454651265, 1472541934, 1473346941, 1485241416, 1486083885, 1473963389, 1479233299, 1464380430, 1471295099, 1460980860, 1487457638, 1464225850, 1475872178, 1479737281, 1483472562, 1461740019, 1467076670, 1473542592, 1469213474, 1473081961, 1469174490, 1464406959, 1485866634, 1458732780, 1456185892, 1468888480, 1476041800, 1462122974, 1467270962, 1475498899, 1476254037, 1475066418, 1468547747, 1473816335, 1455717073, 1484848259, 1476928390, 1458171227, 1461564862, 1470608278, 1469313933, 1480977436, 1462524869, 1486262568, 1470713621, 1461008724, 1460864711, 1472323839, 1453615656, 1470193247, 1468906794, 1477456253, 1457044104, 1462995356, 1460051329, 1479780984, 1459161153, 1481678013, 1485186803, 1452174023, 1486017830, 1465288957, 1465146307, 1470808377, 1471795249, 1465761699, 1479938216, 1469522184, 1460749565, 1465730240, 1478631477, 1478312591, 1466794692, 1488224292, 1486599838, 1481932993, 1461424531, 1455298313, 1463315638, 1456019812, 1454523984, 1466681406, 1463520565, 1460081347, 1487728355, 1470107660, 1477460886, 1473159829, 1482409181, 1486106261, 1460694180, 1481220288, 1469670429, 1461581220, 1467301884, 1457782817, 1486418193, 1463690303, 1461661691, 1475319816, 1455169524, 1475528811, 1459850388, 1481388056, 1460508515, 1462085778, 1452908307, 1475062797, 1479892182, 1466234067, 1452076829, 1486931588, 1465105342, 1462649574, 1486420690, 1460851111, 1485076246, 1465479931, 1485985018, 1454270358, 1472174390, 1478441926, 1481535092, 1468460513, 1466674779, 1456247725, 1456373404, 1460999664, 1460254622, 1478155657, 1482719681, 1471253184, 1458295554, 1480030204, 1460274366, 1468888480, 1487395937, 1485494697, 1484200087, 1469025060, 1459130015, 1467410827, 1468973229, 1482791532, 1466431645, 1468479109, 1464397334, 1470057435, 1473030771, 1475776898, 1464720462, 1487708133, 1454793536, 1481507568, 1473027213, 1472508232, 1487242306, 1475978739, 1464404644, 1465493018, 1460107953, 1453146650, 1469788511, 1476825866, 1453679448, 1469362970, 1469162550, 1460131809, 1460798849, 1456731099, 1470740235, 1460655865, 1475712072, 1455240825, 1475456620, 1472141310, 1469374501, 1456608749, 1454518723, 1479064809, 1481796763, 1454966454, 1458465335, 1474697295, 1451940275, 1480474495, 1457384524, 1465986582, 1487353203, 1471333910, 1462614711, 1461646085, 1487484256, 1477348152, 1471067687, 1462626187, 1464962331, 1471400650, 1461418648, 1488077237, 1471629386, 1474103560, 1467755429, 1487668068, 1483906388, 1472832641, 1473414876, 1455510595, 1467907258, 1453965348, 1469084072, 1462329812, 1479676058, 1474638207, 1481395026, 1453465523, 1463195840, 1470598239, 1456729774, 1467401969, 1465534156, 1474926477, 1471365913, 1488310882, 1482367075, 1486387308, 1468740968, 1460310409, 1462544052, 1469548175, 1472325742, 1467755429, 1486226049, 1462367103, 1487588543, 1460437660, 1482778011, 1465020548, 1483742448, 1465548677, 1483792275, 1473942706, 1458052666, 1485485325, 1473913617, 1473050157, 1467843555, 1461581220, 1484765740, 1466850404, 1463591804, 1486094949, 1474421352, 1482013737, 1487736761, 1456669739, 1469417431, 1452125623, 1453899702, 1464548406, 1486586442, 1481894637, 1472279708, 1454591444, 1455180106, 1452144735, 1487595935, 1471160839, 1457818565, 1480139027, 1485039954, 1467278017, 1468039325, 1476356027, 1479866485, 1465828323, 1483902395, 1482770468, 1458599033, 1455428264, 1456731099, 1471155157, 1474130843, 1463939659, 1484170061, 1465724824, 1484131680, 1455443280, 1460599904, 1456041293, 1455628630, 1466363717, 1452099865, 1474419553, 1478010438, 1485747626, 1467160550, 1487033733, 1473876784, 1487073712, 1479970552, 1475268455, 1475459910, 1483415949, 1471333161, 1451783625, 1473963389, 1482867859, 1459795085, 1472041655, 1484225562, 1469313933, 1486206672, 1486764228, 1482409181, 1474675300, 1484652622, 1475821231, 1478242655, 1461140170, 1463884577, 1452531371, 1462473378, 1474491307, 1473935098, 1482606347, 1482854470, 1487849816, 1467129558, 1482320511, 1470744941, 1469810362, 1480616093, 1454707744, 1462768700, 1462064570, 1487469419, 1456998924, 1486717521, 1472360507, 1464607294, 1459002087, 1478047346, 1470202138, 1461118595, 1456204236, 1474305259, 1470088190, 1487496057, 1466301333, 1459163721, 1469200422, 1479836450, 1471232617, 1467611946, 1460887245, 1486953312, 1460349200, 1453066114, 1481008727, 1461536951, 1469838746, 1462568556, 1476384923, 1468968537, 1466258497, 1457782817, 1477793564, 1478449422, 1461164437, 1465770805, 1473884628, 1461519905, 1481778551, 1464613506, 1479134768, 1470497477, 1456413285, 1463266795, 1453227295, 1478397725, 1483559726, 1463135868, 1459768328, 1483279463, 1479286568, 1469357843, 1484387383, 1464945142, 1456848431, 1453335819, 1476485876, 1459249246, 1456179693, 1455496864, 1462864455, 1484459834, 1463139242, 1473023788, 1453486849, 1478884407, 1480039622, 1466466544, 1452615816, 1457506357, 1477841138, 1466000617, 1472546298, 1456715365, 1481641632, 1454090337, 1483743149, 1475312218, 1459715329, 1483773577, 1457511120, 1458827572, 1477549371, 1462676593, 1483295487, 1482688625, 1483976968, 1475531931, 1454018760, 1454706140, 1457427533, 1462217082, 1452782503, 1473322569, 1481010261, 1487962485, 1465908232, 1464176629, 1470421977, 1461617708, 1472951005, 1467658286, 1484660868, 1463036487, 1481560952, 1453047910, 1481755730, 1468722351, 1454216853, 1486050359, 1456610511, 1467115887, 1451982880, 1456290271, 1461056746, 1479425416, 1471150778, 1473887962, 1477244386, 1481913382, 1456495496, 1483457939, 1464141290, 1470763368, 1486799053, 1477064596, 1468692367, 1465625350, 1475853267, 1470815802, 1471966273, 1465531163, 1467226839, 1464744879, 1453996456, 1479225434, 1466070618, 1465141344, 1474693058, 1468218769, 1484116638, 1473922064, 1452909693, 1474588080, 1476904047, 1454426778, 1473273269, 1452119398, 1456531422, 1472349289, 1476496162, 1461770562, 1452008988, 1455894400, 1471313561, 1465603853, 1481503407, 1467629328, 1483951020, 1468592156, 1465487172, 1465786328, 1473680142, 1468060836, 1458991244, 1456508222, 1487933267, 1463386161, 1451614183, 1459610208, 1483116199, 1472698013, 1485472522, 1483232867, 1464232257, 1480033002, 1484490925, 1484691231, 1453353213, 1473131133, 1471775205, 1457674441, 1478586864, 1487287296, 1477519058, 1469214671, 1456789816, 1453025083, 1474049418, 1460623735, 1473742453, 1466279075, 1464378903, 1471195280, 1470312134, 1462554557, 1466883479, 1480823081, 1464450373, 1461348377, 1455898140, 1458360720, 1461191450, 1457666502, 1452505965, 1459024273, 1454529324, 1459779761, 1476904047, 1452174336, 1485823067, 1464377667, 1484472696, 1474586612, 1455954995, 1468793021, 1454601221, 1468764182, 1484344849, 1466140425, 1485950894, 1470074388, 1472806883, 1476939820, 1461009531, 1481954643, 1470725539, 1459674313, 1459453516, 1468974940, 1472062498, 1479013966, 1460864711, 1481844506, 1457294484, 1455696589, 1488213267, 1460698353, 1467049089, 1481905788, 1475366665, 1481494075, 1485501803, 1477797862, 1456475407, 1461236377, 1479004749, 1483681601, 1474290322, 1452324397, 1479368891, 1475426734, 1482503104, 1455922358, 1479161467, 1477148147, 1462935809, 1467179948, 1457956050, 1486700138, 1470718154, 1487011118, 1473689258, 1467005506, 1483310665, 1456548289, 1465111452, 1485026134, 1485108841, 1486418193, 1483874687, 1478028237, 1469277324, 1470726619, 1464720462, 1458710304, 1477424364, 1456450030, 1476138972, 1480594653, 1487663110, 1480064490, 1473414876, 1486979018, 1458308175, 1452054902, 1481996556, 1467691249, 1486260979, 1474029799, 1487816688, 1458770552, 1475568365, 1460578418, 1453098110, 1452067003, 1474220655, 1464965095, 1473130248, 1479957041, 1458752337, 1457545541, 1476187058, 1464225850, 1465355977, 1475774471, 1468010270, 1484665770, 1479358881, 1472670872, 1468525781, 1454014755, 1479539996, 1484229982, 1480885334, 1473624237, 1478660350, 1453894016, 1453332005, 1469618846, 1467527448, 1458783171, 1452337219, 1479937899, 1468951769, 1466398587, 1462357123, 1454063448, 1483716932, 1477178866, 1461986187, 1484645183, 1485920279, 1460313931, 1464105310, 1486455540, 1457316671, 1476387933, 1457452741, 1478454523, 1451803874, 1460046738, 1457313946, 1458135603, 1483606251, 1480645408, 1474434530, 1484368300, 1488191778, 1476192675, 1486457504, 1485643310, 1477111612, 1472405189, 1471427864, 1456677100, 1476154144, 1475830358, 1470343699, 1482688625, 1477146254, 1481487910, 1477656371, 1453274408, 1475819131, 1472665022, 1485826983, 1459218006, 1473225322, 1459138392, 1476841879, 1464957168, 1451714043, 1479448593, 1462025349, 1457975156, 1479168238, 1469510886, 1479570140, 1463768912, 1480271031, 1468105565, 1463434328, 1486072421, 1482523656, 1477782362, 1456864957, 1464627852, 1486788917, 1455086878, 1479133517, 1454467269, 1468648745, 1453023061, 1475868644, 1480156204, 1473754904, 1479265488, 1464141833, 1483508149, 1457569913, 1456531422, 1466984643, 1465809987, 1466877143, 1458628866, 1453752395, 1457147809, 1458608623, 1462783774, 1472528626, 1461180807, 1476069830, 1478036525, 1461895726, 1482958826, 1463970024, 1465809987, 1474290553, 1472011806, 1468526672, 1485342474, 1485339204, 1451970969, 1470699057, 1456225890, 1468766751, 1466588750, 1481533335, 1483814052, 1485731467, 1462930748, 1473269183, 1481863215, 1477035880, 1467382183, 1463817777, 1479012195, 1482037167, 1479767061, 1458444845, 1470718154, 1473137080, 1458121995, 1457732124, 1463799339, 1475830358, 1480649043, 1476600536, 1453064412, 1471025980, 1478973572, 1464862498, 1456730665, 1458631016, 1469345557, 1452634324, 1481483829, 1483838940, 1485847265, 1478245803, 1463046480, 1452174023, 1467921969, 1485195012, 1474554935, 1487441646, 1464149745, 1466638115, 1468194198, 1487267005, 1480889112, 1483743149, 1461896518, 1476847928, 1474988445, 1475498899, 1452566959, 1484812665, 1486752990, 1476976337, 1463249168, 1487543037, 1455241006, 1470400025, 1487661706, 1481788597, 1483706962, 1464057941, 1462244120, 1453134142, 1481661548, 1479881016, 1467040701, 1466754823, 1478848078, 1455019389, 1478941079, 1471753149, 1479710579, 1471504989, 1471701643, 1470921242, 1472306731, 1458898099, 1457541566, 1476973394, 1464168815, 1488324413, 1485290854, 1460177099, 1480906358, 1461326112, 1472389759, 1485567817, 1485300062, 1487413184, 1461688585, 1452438952, 1479182317, 1478615438, 1484859946, 1483648227, 1461369451, 1458049270, 1481483763, 1485959066, 1452618393, 1469442492, 1476121429, 1472097977, 1477212115, 1459676512, 1474913077, 1468913113, 1477705407, 1461236377, 1468598043, 1477064596, 1487922432, 1451893789, 1461516477, 1462824726, 1487567794, 1460464582, 1482556639, 1484116638, 1484497032, 1480212333, 1466536991, 1457140502, 1463196663, 1453539206, 1462847337, 1486469301, 1460310409, 1485858690, 1460628879, 1478338616, 1474044948, 1458855814, 1471640949, 1455152563, 1476658818, 1465945073, 1464606992, 1472386394, 1484868100, 1478310663, 1479872717, 1461321727, 1478586617, 1468809304, 1483666591, 1465657265, 1463139242, 1480489668, 1454159939, 1467205953, 1481533335, 1487476702, 1478397318, 1461294559, 1475007140, 1478668217, 1483616470, 1476122333, 1469196956, 1468539516, 1486338503, 1481257591, 1463036487, 1470474509, 1455878574, 1479774613, 1457187172, 1485812245, 1476087676, 1457313946, 1467536479, 1462923231, 1484385450, 1483462180, 1454743217, 1473385120, 1480914093, 1452451397, 1484427757, 1477060238, 1485219612, 1451800725, 1478322518, 1486455275, 1453425660, 1480391619, 1466819026, 1453466660, 1477804679, 1475712185, 1465823914, 1463519243, 1458466293, 1483792918, 1462740361, 1467800837, 1478472189, 1473925079, 1478573972, 1464945142, 1480372228, 1462276549, 1487806079, 1470613958, 1482770888, 1452525036, 1485594597, 1462438357, 1481684635, 1468720152, 1480693228, 1464391526, 1486400140, 1465390354, 1469295631, 1463910293, 1468973229, 1464105310, 1469547432, 1473058837, 1485910750, 1465451603, 1474144919, 1471697833, 1454551879, 1474400644, 1478586028, 1467144343, 1459748022, 1474261815, 1452660679, 1460668165, 1464107365, 1483012879, 1487990857, 1459830891, 1476546185, 1482687678, 1479605011, 1475809824, 1459590149, 1457872106, 1481694187, 1470868307, 1454955349, 1475369898, 1452581752, 1483362949, 1483835663, 1456520059, 1451874436, 1475846389, 1454454649, 1466617345, 1485254285, 1476549412, 1476664006, 1467257908, 1472761952, 1481790426, 1483773577, 1487645551, 1475558321, 1486717521, 1467888068, 1471917573, 1481388056, 1482742588, 1462832393, 1455989104, 1477232637, 1485601049, 1469564089, 1487695972, 1487070185, 1454255517, 1479086887, 1460746045, 1459001263, 1469625789, 1485064244, 1476028603, 1453745718, 1460235288, 1465824617, 1472983704, 1466574574, 1458699184, 1482305492, 1461515444, 1475763799, 1485389662, 1472508232, 1457075383, 1463436025, 1483990167, 1469777097, 1454777253, 1462668333, 1471750379, 1487841447, 1456968823, 1471828669, 1465724824, 1488134169, 1456303924, 1471569908, 1461215940, 1476103846, 1470713621, 1462402759, 1462889313, 1469364936, 1471418985, 1474871413, 1474638831, 1479069308, 1481448496, 1455938890, 1462579693, 1453912891, 1488069600, 1476006192, 1466665566, 1478573972, 1470828917, 1487397357, 1460988013, 1465201706, 1463118212, 1463463180, 1456288436, 1483230759, 1462316929, 1458366638, 1462083228, 1466665566, 1454888909, 1467716594, 1463900541, 1461663548, 1451735657, 1473468473, 1482539191, 1468230199, 1483700101, 1457975195, 1486649895, 1468120668, 1470440858, 1467755429, 1460976527, 1459444108, 1455745939, 1487357829, 1466054384, 1458527483, 1462937136, 1480472652, 1469580816, 1473822106, 1482523026, 1458664283, 1453782354, 1481965506, 1477025632, 1482606347, 1472682691, 1461293353, 1484717867, 1476915300, 1480548587, 1464462524, 1461280113, 1467236668, 1486168809, 1486474655, 1477574218, 1487308841, 1463111804, 1483209771, 1485667155, 1488199042, 1472531241, 1486213428, 1486819922, 1458366638, 1456104541, 1463769945, 1468402935, 1457162563, 1484595567, 1481065419, 1467747443, 1455964958, 1463594975, 1459633073, 1463553054, 1468677163, 1462925012, 1481053585, 1478548413, 1456387692, 1469882471, 1486481907, 1451910519, 1468685011, 1475438545, 1454160043, 1453576375, 1487100226, 1466870854, 1451842707, 1454520979, 1474082914, 1457472289, 1481489311, 1482469284, 1459221090, 1466431645, 1476612282, 1470994137, 1487362371, 1477963871, 1465856667, 1474587624, 1459550471, 1466112620, 1477072856, 1464783859, 1454308703, 1459558573, 1483435555, 1453687587, 1469984605, 1472986135, 1480675318, 1473198293, 1451905921, 1477641464, 1483471781, 1460798849, 1483767063, 1457755035, 1473127905, 1456112756, 1459018458, 1473095166, 1487851146, 1458983580, 1488183475, 1472736532, 1456846084, 1452741984, 1457815336, 1456892618, 1452287217, 1456153760, 1457446853, 1470908639, 1479366260, 1473725544, 1474848672, 1475886827, 1485228569, 1484077297, 1461630737, 1476121429, 1476395285, 1462784866, 1477639868, 1470435965, 1459259673, 1459515893, 1460937470, 1463261678, 1452945101, 1469278271, 1465798762, 1484524702, 1483592994, 1458674135, 1474714536, 1484115841, 1454059035, 1459451772, 1460937470, 1481950366, 1472454861, 1463068973, 1477186673, 1478735326, 1463498939, 1474100847, 1476968773, 1464085772, 1476063173, 1468715550, 1457760321, 1472820449, 1470195597, 1462847179, 1470552236, 1468877045, 1453260668, 1483666591, 1478900236, 1483283112, 1459463363, 1481483763, 1475883524, 1478281513, 1487774199, 1454805916, 1454787953, 1466793084, 1476540306, 1484660868, 1474003112, 1452280846, 1460893998, 1477604257, 1454518723, 1478056851, 1484660064, 1471951187, 1476901382, 1464850969, 1469451065, 1452507859, 1460325884, 1452113567, 1457278468, 1478275253, 1458878665, 1470414049, 1465472691, 1483553720, 1461837730, 1455077902, 1476328295, 1454116165, 1479533685, 1455608978, 1476258070, 1462993430, 1483462180, 1460197501, 1488225550, 1463731487, 1452757700, 1453917717, 1465160327, 1487270433, 1483694077, 1473231598, 1479368891, 1478242046, 1470270544, 1459298982, 1467749667, 1475535991, 1458417780, 1477638493, 1484756668, 1460088088, 1462598541, 1483280629, 1460449728, 1453560825, 1459049036, 1475860032, 1469200422, 1467871746, 1474939345, 1472897154, 1463751606, 1452741984, 1465822242, 1454875861, 1468928139, 1476950986, 1468954112, 1485381356, 1468036345, 1481258310, 1479346611, 1454706673, 1475444036, 1473139082, 1453945396, 1465297971, 1471386487, 1455712092, 1454955816, 1481122971, 1479245744, 1461272340, 1484705919, 1463392107, 1478396539, 1458382239, 1453507345, 1464679990, 1468144898, 1457675085, 1487567795, 1466485105, 1487890514, 1465333929, 1458764822, 1471154090, 1481081215, 1472742490, 1478979896, 1452804298, 1464087228, 1472906706, 1462531839, 1484091996, 1477995168, 1461707118, 1456252402, 1472019055, 1468487480, 1486972113, 1470155264, 1476538054, 1472457984, 1476095630, 1487668949, 1487271551, 1457562150, 1467329584, 1486842856, 1462834297, 1472329739, 1467804760, 1472504020, 1463169313, 1459585568, 1477305295, 1468232872, 1460235288, 1458277912, 1477733736, 1465190897, 1465110753, 1455068143, 1467573961, 1482062953, 1463712632, 1487568097, 1475797134, 1488146854, 1485192466, 1468614918, 1488134169, 1464257722, 1452768022, 1461221108, 1468223932, 1451762811, 1476187553, 1462847337, 1466184151, 1465536516, 1466580876, 1470607571, 1475708899, 1470293286, 1468040427, 1452929465, 1464616225, 1483458864, 1456272370, 1463625875, 1456569587, 1458579599, 1468721049, 1481127920, 1474024282, 1467480272, 1482010826, 1472172747, 1476237351, 1461960789, 1478842663, 1473337341, 1473974358, 1471629386, 1470691069, 1455045975, 1465533840, 1470673729, 1480574561, 1472142361, 1488142338, 1469463735, 1461110843, 1474672445, 1482716886, 1462164203, 1453690171, 1453676312, 1474420426, 1484891021, 1462780120, 1483983032, 1487236019, 1484636963, 1462573466, 1451858456, 1468631478, 1473876784, 1467121831, 1453517484, 1455685882, 1460430595, 1455659233, 1486317003, 1464623470, 1487073712, 1484945175, 1472817387, 1478147432, 1487599699, 1466812943, 1479304008, 1459613912, 1456510080, 1473273269, 1467527703, 1472837555, 1485720392, 1455099366, 1455384453, 1486469301, 1456196338, 1474791853, 1466053432, 1485370228, 1472504020, 1467692082, 1481478070, 1472335071, 1465173985, 1453084155, 1475322717, 1467401969, 1460756076, 1455357736, 1460440296, 1454189142, 1484150578, 1472817387, 1465890197, 1457131177, 1452273534, 1480988378, 1451997801, 1476175856, 1465548677, 1462896748, 1456468100, 1452753019, 1453441705, 1465233237, 1460382868, 1471273805, 1474879917, 1456081366, 1455276167, 1472161357, 1456347059, 1483015754, 1482244068, 1479708247, 1474614756, 1483841652, 1469055418, 1456486196, 1475206295, 1463851768, 1484935659, 1468168005, 1455210973, 1473717103, 1480284809, 1485688176, 1452732126, 1464953573, 1485661134, 1465346303, 1464168815, 1465967414, 1466653759, 1483307602, 1452830255, 1454140557, 1473845147, 1461960956, 1466579245, 1487818374, 1483641070, 1459409801, 1454836268, 1486599838, 1457140502, 1454603259, 1463958097, 1487709102, 1481567877, 1464757153, 1464288667, 1454967030, 1487668949, 1468677163, 1459557175, 1480824195, 1482570962, 1484252915, 1463231993, 1474425409, 1465455754, 1459090944, 1462573393, 1453738708, 1454594623, 1454164927, 1483395846, 1453822041, 1458221033, 1469858062, 1475498899, 1456786777, 1474438429, 1484754101, 1458484304, 1483233028, 1459451772, 1484216773, 1473089822, 1478613090, 1465842707, 1479358881, 1475369025, 1482472677, 1453610104, 1470591744, 1487079590, 1483750295, 1478834027, 1457044104, 1484150578, 1468036345, 1461221489, 1462563432, 1481469737, 1485667426, 1471855341, 1452531371, 1478279161, 1485296699, 1465136534, 1464344767, 1480203763, 1472232329, 1464934763, 1470231906, 1457166548, 1488195323, 1455886292, 1473425727, 1470195597, 1472426879, 1460749565, 1463168740, 1476231115, 1483183042, 1455758327, 1484397073, 1481869012, 1464277982, 1479378400, 1463825009, 1463851768, 1470868307, 1470530088, 1456559381, 1466513720, 1483384043, 1460053950, 1486285764, 1456305443, 1475841735, 1485393084, 1470239313, 1476380636, 1456684415, 1462917020, 1466750935, 1472528626, 1457808504, 1479420626, 1460350385, 1476678185, 1463914113, 1465171671, 1478275253, 1454131563, 1475877035, 1458053458, 1470684073, 1465293941, 1459399708, 1463463180, 1473652085, 1472829855, 1475746064, 1466332912, 1480045530, 1485291631, 1469734319, 1451924664, 1468763523, 1478504455, 1455825453, 1477826210, 1477389423, 1472595128, 1470234999, 1476787498, 1470723659, 1466112214, 1463930382, 1484775074, 1462469782, 1461452536, 1481642413, 1454063448, 1478132573, 1456057928, 1484451054, 1459416969, 1459361883, 1482245788, 1474818816, 1480940841, 1461198205, 1482823977, 1456685834, 1486759164, 1458904883, 1456789816, 1473117614, 1468596472, 1481469737, 1458246166, 1487450336, 1483313037, 1480986308, 1485790278, 1471737456, 1454194033, 1478002644, 1480723382, 1479258101, 1462888686, 1479637413, 1454707721, 1464357264, 1453482876, 1471295099, 1481336532, 1464844011, 1463417701, 1454307056, 1470030793, 1484344849, 1478568563, 1470799684, 1452714322, 1485877274, 1461308718, 1470817648, 1453709300, 1480857510, 1456093211, 1460813044, 1464506671, 1482876227, 1462607979, 1487972829, 1472457984, 1460948113, 1457313892, 1482110701, 1481569469, 1472689910, 1470645299, 1468231242, 1472829855, 1475050191, 1472906706, 1452525036, 1474403822, 1475692040, 1453899785, 1483357755, 1487059955, 1485099835, 1451871549, 1463757050, 1474570798, 1457012287, 1461313763, 1462093134, 1468376643, 1468049263, 1465902329, 1468054396, 1480527352, 1484830064, 1479416187, 1458080011, 1457386756, 1486796862, 1477153134, 1453008152, 1467279034, 1484787732, 1473020118, 1476344014, 1471750379, 1458328457, 1474737585, 1455473023, 1459507836, 1484156958, 1462697943, 1475247509, 1461280113, 1473782346, 1461119237, 1455954519, 1476074447, 1465308903, 1483538932, 1486267283, 1472232133, 1454253293, 1473437387, 1469438275, 1472591222, 1470719916, 1486380807, 1456713211, 1486338503, 1454029730, 1481718567, 1456704274, 1469838746, 1471624305, 1471016537, 1459501059, 1481299079, 1461056062, 1453935267, 1471512419, 1487764170, 1464398460, 1473974358, 1465268587, 1470973030, 1479379036, 1478825015, 1485139760, 1454896006, 1478970530, 1484666506, 1480851082, 1481722944, 1458611875, 1471320556, 1456272370, 1481034516, 1465674298, 1460327808, 1455945430, 1485501803, 1484335927, 1458297813, 1457866259, 1488057763, 1484431565, 1452202758, 1485021850, 1466012588, 1471628037, 1483838940, 1486535133, 1454194033, 1456496268, 1453372768, 1470711975, 1481614607, 1478386938, 1480666059, 1458101547, 1462266365, 1477064596, 1469374501, 1460698066, 1453945396, 1458463034, 1454447518, 1480241858, 1479130129, 1477435190, 1482402510, 1458666607, 1460522647, 1475331741, 1479995668, 1464643565, 1459125454, 1479687734, 1475584589, 1485014311, 1476372193, 1477788135, 1460996884, 1477502384, 1486414770, 1459132726, 1476986933, 1485251334, 1459867139, 1465735775, 1475631812, 1488070752, 1468448794, 1488155202, 1466467820, 1481539497, 1477079136, 1473360715, 1459556763, 1459890269, 1462270770, 1456754811, 1461194102, 1469980972, 1486780010, 1462542960, 1458699184, 1454980051, 1485108841, 1457390686, 1468539516, 1456403249, 1477627246, 1465035053, 1487022197, 1458595326, 1455458503, 1472028589, 1459779776, 1458938574, 1465241988, 1457956584, 1474288100, 1473060700, 1480396421, 1453056246, 1486244729, 1475580821, 1477499632, 1456297436, 1460253323, 1465713640, 1480548587, 1460267142, 1457755659, 1468797530, 1460389055, 1471356039, 1454918800, 1466015085, 1461649645, 1485773530, 1478363323, 1452960650, 1486380807, 1470276920, 1454927567, 1463150244, 1451906236, 1487480761, 1475264050, 1480114729, 1472111541, 1457506357, 1469990335, 1453482876, 1475455035, 1482282603, 1463165993, 1479185862, 1454901281, 1464037966, 1481483763, 1487328431, 1476802368, 1479808616, 1483199984, 1461312458, 1453563835, 1486341583, 1472923272, 1486413684, 1487816751, 1482784364, 1486347717, 1481161428, 1482904557, 1462563432, 1461227126, 1462184742, 1486978079, 1481407912, 1478961978, 1455389623, 1482744908, 1455162773, 1479807719, 1460315268, 1475686709, 1465713640, 1468763211, 1473276222, 1458579599, 1452425880, 1462794732, 1454686405, 1484165866, 1482889629, 1476642234, 1487467153, 1455023074, 1472665606, 1456800228, 1459309912, 1468251877, 1459418462, 1469845449, 1475595684, 1483866387, 1470573781, 1471567235, 1463493959, 1474481563, 1464502578, 1470275677, 1476384391, 1478631477, 1453236054, 1477592990, 1484868100, 1473590470, 1456855183, 1464965835, 1486766298, 1453033290, 1454724059, 1463795217, 1460657625, 1473985724, 1479578760, 1485021850, 1460576438, 1486274327, 1472034401, 1465861067, 1462174702, 1474111642, 1463755986, 1456203896, 1480279596, 1454975992, 1486398683, 1486908153, 1478347652, 1455894400, 1452341096, 1454271200, 1470270544, 1459878681, 1467691490, 1472041655, 1475412378, 1455879439, 1457583482, 1487239540, 1473107471, 1457286431, 1461228085, 1478533697, 1478660350, 1488315093, 1461121135, 1472018269, 1466430471, 1468136765, 1453684671, 1472061894, 1452056349, 1454369701, 1471885404, 1452214123, 1480291351, 1457379724, 1474692794, 1476580612, 1459751488, 1480486581, 1486710201, 1469831449, 1458539976, 1472341708, 1468525293, 1460016726, 1454603259, 1465795825, 1473647611, 1465640673, 1478504455, 1479689821, 1475451843, 1463097005, 1466236147, 1479323131, 1473884505, 1476254133, 1452909057, 1468418437, 1481624122, 1479869612, 1481585673, 1483129201, 1464688808, 1485092355, 1457675085, 1455760440, 1468144898, 1488034140, 1479743465, 1460831397, 1476611724, 1478493217, 1456148988, 1454520818, 1463037592, 1469479559, 1463664305, 1465757742, 1480110826, 1463963368, 1466159902, 1486022269, 1456397856, 1467680355, 1487885193, 1476258862, 1463914113, 1461235999, 1453798480, 1455382735, 1461462941, 1487809573, 1482420649, 1481510794, 1488213421, 1484860562, 1486017830, 1473042251, 1474591701, 1479902363, 1486641047, 1465795825, 1457270809, 1459089710, 1479170600, 1455375933, 1483460047, 1474207246, 1462822684, 1468380256, 1474608246, 1477200953, 1476023782, 1484110503, 1483354292, 1476640353, 1455060771, 1484169512, 1452371127, 1462255132, 1482982472, 1473831707, 1477456253, 1462815518, 1485858690, 1464667310, 1464162696, 1483183042, 1461221489, 1475845026, 1473790435, 1461060775, 1456701689, 1452816239, 1462520751, 1464485697, 1467594620, 1486226049, 1486411825, 1477120469, 1477490798, 1480183336, 1459409801, 1485913028, 1470583781, 1468378950, 1486111540, 1462473378, 1483027802, 1455735573, 1456132712, 1462888686, 1455065233, 1452425880, 1452438361, 1478791482, 1462253767, 1457454385, 1476119462, 1452692188, 1464238448, 1486736608, 1483824528, 1453601341, 1459457952, 1456242225, 1462340013, 1455881593, 1457612920, 1484370088, 1474128879, 1482329788, 1461727509, 1464638879, 1476841879, 1471424875, 1455960887, 1465076240, 1473952196, 1461415710, 1487965559, 1465008499, 1486925563, 1460197501, 1465720083, 1474128879, 1457405839, 1475433038, 1458884024, 1481583223, 1471120354, 1479524402, 1460988013, 1468890786, 1466398587, 1482165395, 1466803142, 1485255898, 1477370494, 1470006970, 1456364297, 1454520979, 1454142069, 1456124267, 1487328753, 1459473239, 1462253767, 1461941291, 1470102373, 1487212319, 1479605005, 1472138806, 1453553770, 1452528419, 1458637096, 1455808865, 1484867871, 1459143162, 1481536478, 1463963368, 1475444036, 1480180836, 1482341096, 1482539191, 1482467270, 1473207817, 1478121095, 1468010718, 1452333329, 1473836500, 1457153808, 1457064565, 1461203057, 1475605932, 1476671498, 1487309954, 1460918494, 1455169524, 1469464766, 1453496513, 1480493557, 1479007929, 1477364457, 1473748830, 1454646644, 1459405354, 1453103934, 1478896645, 1481580115, 1480053656, 1465018226, 1475111063, 1465128238, 1455609672, 1459392381, 1481958211, 1457507542, 1461628512, 1480693228, 1458983663, 1484576700, 1456449595, 1481187027, 1459018458, 1487816751, 1468048947, 1454009796, 1480856511, 1487604020, 1464791108, 1463006957, 1487140210, 1453294928, 1469084072, 1454925128, 1464785291, 1468692367, 1469116954, 1455007512, 1461838350, 1454029420, 1469159535, 1468040427, 1478833704, 1453822041, 1464057461, 1458484651, 1454870450, 1467667562, 1481580115, 1473200307, 1477435190, 1453920355, 1467680355, 1487328431, 1455776452, 1452141507, 1454482656, 1473153365, 1483068021, 1462770053, 1485166278, 1484221767, 1451744337, 1475329699, 1456271288, 1459836976, 1457832779, 1468891591, 1476904047, 1458743001, 1461577668, 1473902251, 1463100685, 1459385987, 1467055202, 1462305196, 1466803142, 1458531196, 1465053447, 1475937481, 1470136068, 1480649043, 1484377113, 1478001578, 1480679026, 1467929039, 1469738793, 1463607385, 1469107211, 1476052789, 1460578418, 1466258198, 1456754811, 1452528419, 1468924910, 1484332585, 1481027512, 1467555347, 1476072894, 1485391258, 1485204516, 1480241858, 1467889982, 1473520625, 1457151062, 1462183024, 1484291921, 1486759164, 1470148718, 1470121486, 1480313917, 1487822169, 1480395017, 1473859005, 1458708645, 1477136771, 1454128720, 1480024461, 1487437414, 1459713634, 1478310663, 1465341119, 1456547343, 1458456102, 1457525051, 1476927392, 1457791328, 1474944730, 1460016726, 1485492231, 1463625173, 1477212115, 1453227295, 1452383896, 1462932035, 1482186580, 1461971752, 1485923174, 1457714920, 1453860255, 1485220498, 1465585702, 1481838125, 1455161152, 1462554557, 1460698353, 1475389646, 1487966528, 1476799369, 1457808504, 1483558510, 1458507479, 1470966514, 1464613506, 1483926341, 1452659005, 1462479999, 1463471606, 1462253767, 1484706978, 1453680882, 1457228199, 1485138228, 1464476496, 1482908661, 1465816892, 1465505988, 1488012521, 1468967489, 1477108456, 1451816310, 1484855712, 1451863272, 1479905531, 1479905531, 1484410589, 1466297501, 1465121376, 1452010591, 1464232257, 1486998789, 1468954483, 1475684092, 1478678871, 1477911144, 1486317003, 1457022973, 1459330864, 1475164673, 1483965834, 1472549228, 1483346964, 1464992533, 1480403018, 1454889017, 1457513470, 1452665649, 1468522511, 1482859523, 1487765777, 1474642563, 1453482148, 1453265382, 1476486022, 1481334081, 1478512813, 1480745986, 1470557720, 1453672945, 1455825453, 1470566260, 1481467631, 1457370855, 1479813586, 1472519446, 1466570370, 1469738793, 1457129165, 1453918507, 1481862316, 1463616646, 1482335200, 1477291496, 1483921069, 1463951887, 1474973566, 1485501803, 1476372193, 1453944332, 1457331301, 1483646778, 1478402633, 1465861067, 1459535669, 1485962144, 1469200695, 1486175590, 1485913028, 1467035631, 1475612155, 1454398140, 1468863158, 1460258550, 1487632452, 1465220792, 1485300656, 1485318177, 1485037650, 1485525753, 1459556763, 1458927017, 1481899694, 1485273781, 1459590149, 1452690354, 1484106767, 1464110344, 1486945042, 1454373576, 1454517590, 1459696369, 1469077451, 1477041119, 1452546192, 1473278585, 1468721554, 1474224523, 1466751331, 1473717103, 1477872838, 1456124267, 1460586997, 1483832780, 1468630970, 1483944219, 1454605135, 1473088653, 1458933198, 1476258070, 1482286786, 1456520305, 1460232511, 1454694213, 1458268178, 1467438272, 1469084072, 1468965367, 1465931088, 1458152117, 1475004112, 1457938254, 1481057052, 1487705685, 1476254133, 1486106261, 1463183731, 1475075470, 1469516222, 1479012511, 1479684322, 1463968490, 1478419802, 1456296943, 1484877603, 1461637924, 1472512901, 1474654355, 1456507272, 1465967414, 1458794400, 1471725152, 1454632631, 1478533697, 1475438878, 1477515946, 1466757409, 1454565609, 1486094949, 1467298131, 1481590094, 1475009896, 1461686946, 1457689257, 1487229534, 1471573760, 1482546993, 1454930090, 1475582135, 1459229398, 1485877274, 1456704274, 1473795277, 1480945619, 1451616403, 1486953073, 1474106069, 1485652407, 1454843655, 1473459715, 1455096542, 1478896645, 1476563436, 1477041119, 1478761358, 1468547747, 1487006262, 1482791977, 1468968537, 1485562885, 1484110503, 1479778867, 1473029774, 1467847757, 1481370664, 1467129052, 1458196268, 1462917020, 1485037650, 1476698091, 1461741292, 1482550800, 1452067003, 1466071243, 1467346110, 1474411864, 1481561918, 1463351602, 1458466293, 1455898140, 1485941326, 1461161818, 1461228085, 1486673920, 1463540745, 1455077902, 1481350606, 1469703201, 1454482656, 1458517683, 1484454672, 1466279075, 1454467269, 1472640946, 1455186495, 1470266996, 1476486022, 1463895930, 1461244582, 1477437329, 1457749015, 1455586074, 1483921069, 1453194521, 1468003801, 1453920355, 1475405514, 1458624241, 1479233299, 1460132211, 1476093110, 1461385652, 1461225587, 1483289369, 1458564745, 1472544959, 1482716886, 1459878665, 1456867842, 1470096015, 1454519860, 1478163114, 1479970552, 1458564745, 1465472691, 1463600634, 1483113273, 1471094379, 1479176236, 1474110631, 1457652779, 1485109216, 1470151211, 1486550089, 1475272838, 1479368891, 1471635422, 1469417431, 1462310046, 1456180387, 1460596579, 1479528715, 1478625744, 1454965703, 1477954559, 1475066418, 1466219810, 1483027802, 1456269040, 1472756654, 1473097187, 1475616399, 1473930327, 1479676474, 1475514221, 1463963800, 1465708813, 1485552220, 1464969440, 1478428834, 1463566591, 1476193741, 1477244386, 1458112248, 1485722833, 1476970348, 1462677105, 1478090690, 1472440877, 1474421746, 1478165663, 1475941106, 1484230814, 1480203357, 1471640949, 1475732858, 1474531823, 1487828972, 1458177046, 1454706673, 1452718442, 1481730250, 1485344728, 1470280524, 1457089218, 1480252283, 1470231484, 1458243835, 1459344359, 1470530088, 1488261866, 1457171686, 1483051161, 1470787429, 1468818428, 1481233819, 1473114680, 1474290553, 1457509152, 1484091996, 1478805173, 1461318124, 1452387967, 1485747626, 1477232637, 1487110425, 1476649684, 1470928754, 1478358243, 1463446635, 1451966880, 1475300352, 1465304436, 1471492142, 1478924651, 1473861890, 1459659469, 1477406016, 1487851016, 1471476882, 1470541877, 1476836470, 1487908413, 1467596452, 1464643137, 1456513761, 1474288100, 1475229412, 1486534463, 1475438545, 1481573474, 1453294928, 1475897545, 1453918507, 1457330292, 1477771397, 1485535033, 1463168740, 1456684415, 1482245788, 1459859709, 1459917887, 1483694361, 1483869538, 1471356039, 1453895459, 1471670989, 1467375041, 1462734014, 1481209893, 1485962144, 1466812943, 1453007746, 1461161818, 1484368300, 1463685542, 1463278510, 1455261256, 1477499632, 1480857510, 1477244386, 1462864455, 1466612287, 1463488980, 1484934410, 1477027811, 1458425740, 1476593550, 1464268076, 1464379353, 1463240248, 1474696066, 1458430291, 1461194102, 1479448593, 1453519353, 1468808807, 1476950986, 1487608188, 1467761894, 1462111958, 1453379610, 1481982265, 1472790670, 1470492011, 1478256538, 1456303924, 1477954559, 1472502559, 1469517287, 1486133938, 1473164291, 1482608153, 1458035784, 1453236124, 1483524934, 1474385047, 1461434777, 1475355050, 1468375584, 1470387302, 1459740711, 1479134768, 1465292812, 1466680080, 1479938216, 1478728206, 1477082991, 1455058460, 1464639901, 1454706140, 1469236994, 1478387923, 1481148755, 1484845132, 1456997074, 1462829855, 1485838837, 1488213421, 1461129070, 1467438290, 1488020609, 1465981242, 1466622705, 1476005244, 1457158951, 1487871526, 1477051689, 1472617449, 1468235816, 1470345793, 1484431565, 1463143424, 1486433351, 1484507581, 1456588753, 1453290881, 1461164437, 1470088190, 1474264284, 1485346430, 1471166830, 1455586074, 1477296283, 1452379508, 1473082848, 1485484873, 1467713068, 1474758670, 1479465450, 1462479999, 1452909693, 1475088290, 1465451603, 1473124166, 1460138611, 1486269468, 1479004749, 1484830064, 1484937982, 1476540977, 1479504306, 1463714610, 1480536134, 1468713614, 1478067840, 1459558505, 1462129302, 1462505345, 1485301993, 1471979887, 1468916983, 1485796285, 1476546775, 1463150244, 1480391713, 1469938054, 1452073276, 1456440893, 1454811064, 1455165246, 1462241128, 1484182748, 1456846869, 1469170262, 1487662579, 1465449790, 1461969360, 1487998416, 1456028278, 1459151022, 1454793536, 1470740287, 1479735380, 1455898140, 1478428834, 1461121135, 1479807719, 1480877578, 1483876875, 1487539191, 1470718154, 1477796187, 1461056062, 1463309367, 1479316267, 1452356280, 1463348123, 1476646213, 1486809600, 1452458152, 1463672854, 1472236510, 1472867294, 1483337555, 1451966880, 1452821166, 1452532683, 1459826717, 1487943834, 1475488313, 1453621894, 1481200652, 1487788525, 1469520908, 1467310155, 1455285940, 1487595935, 1458185961, 1455745719, 1453801809, 1485050726, 1485659102, 1481732023, 1480758732, 1487816751, 1485493046, 1467807964, 1462039515, 1472929429, 1473689258, 1484608583, 1461215940, 1483193298, 1484645183, 1452736674, 1470228900, 1467236730, 1462044144, 1458119550, 1478534160, 1468015079, 1479990617, 1456153760, 1453466342, 1463037390, 1458987833, 1477121720, 1470326622, 1468418437, 1466638717, 1477705740, 1478279161, 1460998935, 1452480396, 1474310976, 1453343936, 1485838837, 1475258614, 1472643211, 1464279022, 1464021651, 1481032514, 1473397746, 1470236522, 1487662579, 1460887245, 1452141507, 1460197501, 1453441705, 1466152258, 1484949597, 1452119971, 1471301221, 1480474495, 1482348101, 1481573609, 1486347347, 1461740019, 1467594620, 1465086517, 1472440877, 1456247725, 1483269854, 1469374501, 1466302737, 1454494659, 1479888076, 1461211648, 1461276740, 1466426832, 1475513729, 1471843981, 1455739985, 1462598541, 1462639826, 1488293852, 1455375933, 1471781801, 1472431196, 1476565559, 1460257727, 1464771403, 1487189322, 1464814809, 1480184542, 1467570445, 1458680665, 1470811068, 1478611816, 1452763603, 1458211674, 1486591054, 1483570528, 1471601601, 1478149175, 1471431857, 1473224753, 1451816738, 1461740019, 1478848078, 1478552636, 1470868989, 1464969486, 1463037390, 1486067985, 1461081903, 1460840850, 1482706455, 1478056851, 1454957526, 1467222370, 1485963476, 1488162434, 1459751488, 1480947885, 1471812457, 1464214374, 1478658431, 1458903372, 1471473377, 1454063311, 1467931486, 1455099366, 1470711336, 1483164533, 1475706825, 1473097187, 1459056940, 1464039617, 1462726677, 1460348989, 1467401969, 1466576989, 1458847692, 1468878988, 1481784424, 1478858355, 1455607149, 1468592242, 1468808807, 1483936276, 1474158991, 1464297084, 1456274073, 1469116954, 1473211379, 1479509416, 1485717124, 1487212319, 1464482588, 1451955575, 1461931681, 1474989093, 1478402633, 1480160797, 1465534076, 1467982083, 1481124440, 1467597322, 1474672445, 1453758649, 1477035880, 1475978780, 1474744511, 1470314550, 1487869596, 1469960952, 1483286264, 1461564862, 1483811839, 1465695267, 1471979887, 1464431852, 1476512007, 1467761894, 1468588006, 1472155449, 1463625173, 1453029252, 1477450908, 1459775304, 1465621384, 1485923174, 1473269183, 1476340708, 1460258679, 1484696663, 1452796793, 1476434768, 1472651110, 1477001077, 1458599033, 1456395729, 1453260668, 1457817411, 1482111604, 1469537154, 1462766811, 1464877077, 1487800363, 1487751382, 1477905530, 1475795717, 1476121429, 1475763799, 1458149525, 1456120658, 1487125638, 1474044948, 1468939174, 1480840485, 1453890445, 1471329238, 1487124402, 1486700437, 1470360410, 1476231115, 1461490874, 1487110075, 1470169004, 1483476481, 1470296530, 1473137080, 1482958826, 1482420265, 1461608246, 1459270081, 1466622705, 1473060700, 1486998789, 1480717405, 1484887029, 1461473853, 1478178972, 1474432557, 1453590343, 1465718342, 1466222415, 1452753785, 1466696903, 1466359853, 1469789894, 1479847079, 1462413880, 1455180106, 1452231411, 1455393975, 1485796285, 1476574272, 1474518263, 1458525246, 1462751110, 1485393127, 1471555412, 1475098794, 1477421526, 1476470616, 1457089218, 1470030793, 1468037109, 1484706978, 1459415239, 1455745179, 1484253457, 1470461529, 1476384923, 1466306977, 1464782630, 1452917783, 1471375318, 1461132657, 1468787750, 1475691034, 1466385810, 1471473377, 1486629509, 1456732913, 1470636126, 1478065412, 1475050380, 1469482085, 1458471098, 1469956457, 1453465523, 1471870712, 1471855341, 1478735326, 1463715663, 1467451083, 1452539800, 1454163022, 1457273482, 1469559259, 1487022197, 1457147675, 1485667155, 1462589092, 1452909693, 1452458152, 1468423999, 1456987665, 1481722944, 1487520614, 1461958485, 1473624237, 1460592484, 1455960887, 1485486929, 1459471192, 1483123984, 1482863660, 1454706673, 1476593550, 1467446567, 1478888218, 1453553770, 1454618548, 1487053978, 1453687587, 1463634424, 1459270738, 1477860631, 1454896006, 1456274073, 1476441041, 1469054271, 1460286271, 1476182556, 1485086031, 1473479639, 1470056208, 1468464454, 1484775074, 1482925961, 1481839659, 1460106655, 1480526842, 1465640673, 1482420649, 1464844011, 1460811252, 1477708328, 1458491540, 1470270544, 1462630358, 1464404644, 1487127625, 1475631836, 1471320556, 1467995205, 1468570097, 1473948922, 1465377155, 1456297436, 1477348152, 1453382695, 1468464454, 1463973604, 1452458152, 1480547687, 1480945619, 1485398218, 1462118289, 1459111910, 1468169953, 1455964958, 1471979887, 1469558592, 1473676997, 1472665296, 1464915423, 1467595107, 1479170600, 1459471185, 1455584371, 1467657326, 1454523030, 1464309159, 1452707474, 1455068032, 1454255517, 1464219144, 1470076623, 1482766112, 1477111298, 1466235461, 1487841885, 1464969440, 1468933377, 1474781026, 1487641471, 1485162481, 1487675569, 1469477901, 1462184742, 1460037450, 1458314037, 1466984819, 1485665134, 1458847215, 1486110660, 1487505178, 1466699291, 1479433223, 1466912530, 1454817075, 1465834843, 1466846112, 1473626957, 1455745939, 1476138972, 1485913028, 1452837255, 1462316929, 1462664917, 1451700694, 1467691490, 1457167098, 1461679226, 1474856814, 1452195076, 1478125349, 1479682670, 1469499135, 1452841216, 1467491963, 1453000153, 1485224317, 1472958374, 1484233421, 1481953760, 1477162450, 1486936926, 1466850404, 1480527352, 1461195000, 1457944584, 1456051300, 1452405394, 1481190864, 1451849291, 1483742146, 1466071243, 1452853320, 1471410637, 1463826925, 1476927392, 1466302737, 1459052977, 1461364883, 1471925648, 1472097977, 1453576992, 1451954552, 1452501428, 1472232133, 1486052064, 1487287296, 1463949299, 1484201923, 1457815336, 1462056694, 1476187553, 1465201015, 1485071639, 1484335927, 1469860370, 1468871484, 1483794305, 1457951708, 1456565956, 1486896886, 1454376407, 1473395411, 1482257507, 1459661799, 1473103298, 1452732126, 1476407389, 1478466464, 1478734158, 1475882421, 1488238959, 1471782750, 1474637712, 1487567795, 1456290271, 1459890269, 1484781711, 1467088075, 1455021748, 1480311859, 1461531377, 1469327045, 1487863655, 1478789621, 1455125230, 1471272023, 1461930554, 1475614162, 1483067824, 1461988604, 1454902806, 1470711621, 1455203368, 1452567830, 1462843455, 1464534028, 1487922432, 1453798480, 1454036981, 1477041119, 1473705521, 1474637712, 1474024282, 1470727243, 1468251877, 1464099685, 1483933539, 1483384043, 1487792633, 1487889371, 1464301288, 1472987703, 1473749391, 1481009427, 1463900541, 1463654391, 1454402474, 1469463781, 1476799235, 1469777097, 1464301288, 1467093514, 1487006262, 1465171671, 1466070416, 1466531690, 1455878574, 1466473272, 1458165617, 1461692075, 1452694779, 1475790181, 1464642869, 1452156026, 1462872288, 1475516454, 1459661799, 1457811006, 1458655546, 1487520614, 1463958332, 1467596452, 1486202283, 1463291330, 1475516454, 1471166606, 1474528207, 1458135603, 1453390947, 1479341294, 1478078169, 1462627383, 1468592242, 1488107117, 1480835854, 1471340943, 1456225890, 1459130015, 1459453516, 1477719554, 1453585041, 1463308927, 1466541114, 1468677163, 1458789930, 1471018794, 1486799053, 1468868348, 1475501647, 1487990857, 1453411981, 1475599318, 1477660978, 1487003549, 1458546378, 1482389874, 1462986228, 1466860466, 1464965038, 1485990532, 1475300352, 1462843455, 1480705609, 1452360188, 1467613149, 1474593366, 1483173641, 1473225322, 1474598635, 1455327637, 1473948922, 1487290568, 1480344418, 1472330926, 1458963183, 1464123771, 1461624399, 1476422520, 1466161005, 1470626814, 1471837585, 1487806079, 1487004524, 1482449696, 1486939537, 1469388993, 1465493018, 1476212865, 1463748030, 1484324149, 1484325248, 1458819366, 1457064565, 1457382562, 1466055479, 1464586818, 1474284781, 1456581054, 1456985398, 1468797530, 1476499562, 1474284781, 1465160327, 1487629195, 1471543142, 1481429636, 1474497803, 1459108205, 1484777702, 1478677211, 1479767061, 1479339382, 1475270215, 1462310046, 1454426778, 1476043298, 1472989654, 1466558353, 1459729631, 1452336906, 1471218352, 1480655997, 1462993430, 1479515354, 1480493557, 1477413640, 1465889433, 1467655254, 1461986917, 1454208886, 1475994622, 1458925292, 1462114650, 1467450808, 1476441671, 1485350387, 1476567111, 1484060540, 1459417091, 1474632127, 1456403249, 1458601233, 1466690116, 1473996827, 1478344463, 1454463929, 1467232120, 1475580821, 1460325884, 1476384391, 1486589395, 1473383446, 1468878988, 1473222016, 1472820251, 1472574723, 1479733692, 1482057663, 1480658632, 1481531001, 1477733893, 1456109496, 1481700002, 1461859835, 1461788187, 1464699702, 1460428156, 1457634565, 1476907500, 1463169313, 1472508021, 1476241454, 1462263164, 1478429696, 1454360848, 1466137640, 1454371001, 1468744139, 1479339382, 1480976906, 1471868208, 1459570625, 1464219144, 1456885760, 1471116401, 1461221108, 1465447351, 1464560100, 1463271681, 1472909726, 1456508222, 1479099409, 1473031363, 1478661741, 1480677908, 1469295631, 1470566260, 1483851321, 1476846883, 1456869262, 1484945175, 1463262636, 1487326543, 1476134834, 1453455358, 1481467632, 1478693180, 1468038655, 1474665914, 1463352785, 1456976059, 1462369262, 1453002236, 1465849725, 1477297468, 1473511477, 1453683097, 1475886827, 1472814889, 1480805551, 1471374875, 1470717408, 1487743916, 1483233191, 1475247509, 1466936553, 1453236124, 1463801183, 1482744908, 1452763603, 1479776262, 1480383971, 1466244182, 1474867915, 1456164248, 1459102358, 1486604593, 1460668165, 1488012633, 1473524627, 1459291712, 1480713090, 1472163178, 1475513729, 1470641321, 1459894383, 1484106767, 1486347903, 1478256538, 1477782793, 1480868474, 1484325248, 1469322219, 1473042571, 1474708238, 1471636115, 1476429624, 1480605385, 1485839802, 1465478038, 1474431780, 1457726663, 1465946023, 1462986228, 1469119047, 1474605856, 1484478543, 1475639499, 1481863215, 1466718775, 1456493772, 1470572004, 1469948456, 1452288890, 1460690889, 1468111749, 1471778439, 1463762284, 1452606334, 1467156992, 1484026040, 1470049483, 1465355977, 1475528811, 1480223872, 1477291496, 1487376769, 1478979896, 1466454370, 1476433083, 1477782793, 1476836470, 1477159288, 1484090218, 1467756158, 1487079590, 1473922064, 1462479999, 1453951117, 1464969486, 1462177576, 1457416827, 1473749391, 1477203103, 1484378707, 1472623623, 1458683935, 1452837255, 1486420690, 1461051878, 1466050266, 1485056015, 1468483537, 1468284308, 1482177480, 1452853235, 1451906285, 1465718696, 1477881447, 1454131152, 1486629509, 1461227126, 1460088088, 1469956457, 1453002236, 1478625744, 1487685234, 1461452865, 1455061213, 1486067985, 1467609609, 1456533353, 1484031625, 1483471781, 1459025940, 1474961540, 1485204516, 1469246439, 1466850127, 1473029774, 1479553147, 1466718775, 1471333041, 1476005244, 1479879394, 1475481366, 1480964691, 1480780471, 1477411388, 1473590470, 1481502401, 1479170028, 1468136765, 1468010718, 1463207368, 1485050184, 1459878681, 1459953327, 1476094970, 1485871297, 1477269834, 1483384043, 1465987452, 1467055202, 1486535478, 1460670046, 1481221163, 1463181398, 1476182472, 1464580461, 1479185862, 1485164700, 1479524402, 1484907457, 1463714610, 1476404894, 1482777237, 1478001714, 1463801183, 1455171455, 1478723870, 1474043245, 1454016298, 1465788655, 1466576738, 1483922856, 1462665453, 1452008988, 1469649533, 1469580247, 1464368083, 1485124025, 1467280400, 1487343334, 1467761541, 1481399430, 1456153415, 1466814754, 1468513288, 1486317003, 1469580247, 1484936589, 1466610816, 1471657370, 1472712186, 1453215470, 1478337309, 1475054438, 1472523561, 1481305708, 1454946135, 1472016535, 1458854066, 1459557827, 1477153134, 1470026319, 1472330447, 1464047091, 1471154090, 1465686239, 1463858754, 1479929818, 1470231906, 1477937376, 1479155057, 1452449327, 1462770053, 1452420153, 1455738843, 1468229011, 1473946158, 1455165246, 1458517692, 1474625975, 1466968473, 1479565308, 1453700935, 1462850728, 1480454724, 1462314666, 1453573999, 1458167732, 1454198086, 1473942706, 1456713286, 1482601220, 1452586837, 1467682053, 1481448883, 1473778084, 1486342561, 1453000153, 1478397725, 1477562953, 1453945396, 1478402633, 1456410631, 1472659350, 1473616313, 1466712092, 1487942154, 1453304242, 1458872690, 1467226839, 1480015271, 1474195043, 1470673729, 1467749667, 1458026078, 1462734014, 1479902363, 1488008597, 1465732568, 1472163178, 1457758290, 1463770557, 1481592312, 1465985516, 1460079901, 1468290124, 1483521030, 1472117190, 1471737456, 1457273482, 1467380086, 1457602368, 1485086031, 1474928443, 1469287006, 1467589743, 1485141450, 1486106261, 1459515879, 1468796505, 1454505804, 1475943151, 1475456620, 1488155202, 1487431318, 1464203426, 1465088686, 1462114650, 1467657326, 1477167047, 1476453778, 1474706236, 1487496057, 1480780106, 1465416045, 1470641834, 1478147432, 1460916890, 1486692921, 1473823162, 1464806469, 1458792211, 1460114409, 1478001714, 1465019770, 1455852426, 1480155325, 1463939659, 1466258198, 1463108993, 1476883859, 1471431857, 1487110075, 1463581062, 1479098448, 1487427072, 1463577233, 1456700762, 1456676710, 1454724059, 1466401646, 1479714519, 1461198205, 1482760525, 1469940189, 1452337219, 1466320214, 1487965559, 1472000961, 1465809987, 1480705609, 1476935974, 1470868847, 1477256023, 1474323603, 1456846126, 1476846883, 1460362291, 1487188201, 1467397899, 1459405354, 1457952375, 1482335200, 1480907963, 1475635485, 1486815018, 1477111612, 1464186923, 1472231192, 1460143893, 1459644830, 1472141310, 1471901714, 1467725770, 1480659475, 1461583775, 1469394641, 1475066418, 1478976360, 1478645164, 1467283468, 1456680773, 1466883479, 1470004346, 1472323220, 1453397681, 1475446595, 1470831718, 1481944496, 1453848928, 1464444579, 1476154144, 1462621846, 1473383446, 1470278187, 1458859746, 1462246953, 1487816688, 1473488788, 1458957020, 1482230672, 1487759915, 1463851726, 1482983563, 1466050266, 1467146097, 1488171614, 1460348989, 1483357089, 1470361215, 1470764082, 1476687057, 1456178265, 1477795932, 1476804954, 1460088088, 1467401481, 1462722056, 1462579693, 1488056604, 1487817593, 1473360307, 1486102683, 1453307354, 1487958755, 1484636963, 1475994622, 1464407285, 1470161072, 1472225346, 1483802642, 1487240469, 1475129801, 1474547633, 1466065296, 1452819141, 1465646059, 1464283684, 1484477840, 1457791328, 1487695972, 1479453346, 1475635485, 1473809092, 1471832856, 1477553481, 1466520500, 1483742448, 1475081707, 1471669561, 1471840130, 1469433453, 1485351985, 1457194169, 1456569587, 1466636956, 1463386161, 1466400231, 1472595128, 1467222370, 1478966501, 1454063448, 1479185862, 1483159493, 1457208464, 1483004848, 1481247727, 1462306541, 1458991429, 1482609802, 1456520305, 1474635681, 1472986195, 1484187219, 1487862065, 1482687678, 1476297236, 1467332183, 1463404971, 1486387956, 1459542366, 1469789894, 1461613469, 1478092129, 1469438275, 1452260276, 1474808515, 1456325461, 1470869224, 1488171614, 1457831573, 1473809092, 1481053123, 1470787429, 1480203357, 1456791998, 1480633811, 1462573466, 1459071808, 1470440858, 1472701619, 1486175590, 1477841138, 1460955969, 1461519033, 1481573474, 1463634424, 1461955488, 1471782750, 1486322072, 1475488313, 1467778802, 1478339900, 1451950245, 1477333863, 1456449595, 1478241964, 1470280524, 1455739985, 1462383572, 1454991381, 1481700303, 1462573393, 1473479639, 1464623470, 1458710304, 1485910750, 1477120469, 1458531196, 1453594313, 1484666506, 1458394097, 1488183475, 1455748894, 1487208431, 1482420265, 1467755429, 1457080501, 1455707008, 1480396421, 1456653600, 1467629328, 1484950393, 1485080461, 1460350385, 1469882471, 1456450203, 1475740862, 1462123823, 1483895948, 1452125623, 1473873639, 1456379135, 1482110701, 1474044948, 1485263422, 1483684868, 1467907258, 1484992764, 1485447971, 1453547526, 1479265488, 1451657744, 1475507466, 1463356807, 1481176933, 1452845969, 1480313917, 1472909726, 1471166606, 1472551975, 1464298350, 1480748380, 1477386816, 1485398022, 1476340708, 1461530870, 1476600536, 1453789032, 1487071005, 1460254622, 1464992533, 1460194342, 1463951459, 1482791532, 1481200652, 1475088315, 1468994757, 1481448883, 1482647254, 1463757050, 1457153808, 1478097905, 1481560952, 1462310139, 1474889695, 1482437151, 1468435324, 1472531241, 1456251757, 1461118420, 1476111977, 1456686177, 1465986443, 1476023782, 1483716932, 1482732848, 1468039325, 1466234067, 1463125313, 1476412872, 1460954706, 1463651530, 1480184662, 1453701652, 1485220349, 1479233177, 1480252283, 1463779071, 1470256393, 1452067003, 1473360884, 1482183284, 1456782576, 1479663924, 1456397849, 1464072950, 1461544958, 1484936478, 1466781718, 1477207580, 1468772906, 1483866387, 1464962331, 1465850078, 1486894760, 1470518612, 1476767279, 1456099742, 1464341059, 1471365913, 1465625350, 1459992879, 1473096725, 1465549270, 1452142954, 1467450808, 1458950731, 1469975651, 1462381279, 1455142281, 1478665076, 1468793612, 1467670121, 1463178130, 1451955575, 1455081992, 1462294273, 1483951242, 1481425255, 1456252402, 1482057663, 1478693180, 1464616225, 1488323462, 1455775162, 1466603936, 1484775074, 1479251031, 1464783149, 1464057941, 1462955185, 1476590353, 1475397103, 1471254138, 1488171301, 1459471185, 1472626770, 1461971752, 1481905788, 1485469991, 1462377381, 1480589617, 1473101824, 1475286153, 1467693671, 1484242512, 1483992010, 1475807191, 1460918494, 1456684415, 1484397073, 1486945042, 1465626316, 1463461833, 1469157612, 1483906388, 1480386090, 1452028034, 1485086031, 1474395138, 1459478937, 1486210501, 1470660796, 1482732848, 1457171686, 1481494075, 1457384524, 1483833860, 1472000961, 1471425942, 1454889017, 1474570798, 1462056694, 1459888414, 1480868474, 1468479485, 1468503308, 1462932435, 1454930090, 1463838931, 1454040583, 1453701607, 1472177399, 1458246490, 1486455540, 1457404416, 1487057733, 1456648121, 1454710290, 1470497477, 1462885248, 1452804298, 1482284385, 1469559259, 1472932169, 1461293353, 1469688357, 1475918849, 1451844588, 1487471205, 1478613090, 1466877143, 1482822662, 1480536134, 1488007092, 1481622876, 1482523026, 1460714566, 1465945073, 1466012588, 1478718238, 1472502559, 1484733007, 1455026840, 1454646644, 1452013132, 1454376407, 1456605827, 1477711399, 1463196691, 1478265450, 1456681014, 1471665052, 1484316641, 1455459294, 1462492012, 1483855729, 1473585990, 1455938890, 1480951462, 1473113331, 1466359853, 1461750749, 1478666824, 1459808162, 1454194737, 1481132549, 1463786047, 1481998156, 1455928065, 1485796285, 1487171107, 1478429696, 1471219031, 1470006970, 1484799927, 1456505011, 1485376608, 1469030662, 1483538677, 1487889371, 1487604020, 1476927392, 1485195012, 1468101724, 1474074638, 1455862076, 1470684073, 1461273469, 1461880187, 1457793913, 1480614256, 1485426796, 1464900508, 1475774471, 1467611946, 1454307056, 1477304455, 1460962339, 1456605827, 1454208886, 1475820702, 1462524869, 1465761699, 1487681453, 1457817411, 1453284558, 1471155157, 1472171430, 1456505011, 1482916477, 1478833704, 1474202301, 1475686178, 1473816335, 1480824195, 1452536032, 1464723600, 1476871808, 1453539206, 1464879813, 1485155682, 1465479172, 1478826553, 1462965918, 1462751110, 1483491558, 1475106915, 1477732308, 1472531241, 1477860631, 1459446726, 1464442076, 1460310409, 1466561029, 1470767315, 1451897630, 1478253589, 1456025353, 1476867895, 1481684635, 1465317468, 1479176789, 1458678223, 1487605614, 1474431780, 1485370971, 1454941641, 1485296699, 1479303671, 1462469782, 1469168643, 1463887612, 1472365954, 1468053449, 1482146934, 1488072749, 1468341568, 1460908428, 1460980860, 1457504351, 1468522511, 1460834613, 1482550800, 1452054902, 1454927056, 1470821942, 1465280701, 1473432942, 1486460437, 1458927017, 1461840772, 1478493217, 1487709102, 1472317457, 1478481644, 1464163660, 1480271031, 1456719327, 1454352689, 1472046389, 1482608153, 1484010588, 1486637387, 1467296318, 1488315513, 1464723600, 1480472652, 1483841652, 1485747626, 1468437660, 1478988568, 1478483780, 1466065296, 1463404971, 1457035780, 1453331761, 1457388348, 1470615575, 1459946451, 1451783625, 1466468349, 1479969451, 1476187058, 1452067003, 1451937070, 1471065449, 1484801445, 1479870420, 1473904506, 1460090385, 1467989657, 1473301889, 1458015127, 1456366084, 1484291921, 1464213827, 1484205875, 1485695042, 1470821942, 1473208162, 1485717540, 1455775162, 1464263098, 1483648659, 1486127798, 1457798277, 1475314173, 1454975992, 1452140012, 1458452523, 1456509414, 1477964708, 1477639868, 1479629063, 1471344918, 1476375260, 1466825018, 1457555348, 1465957423, 1469580247, 1477985534, 1453233195, 1484527166, 1456011274, 1455068032, 1467960464, 1456011274, 1483136324, 1466546424, 1464953573, 1481899534, 1466753950, 1460988013, 1486631176, 1461189132, 1482437151, 1472806883, 1482472677, 1474029799, 1456310033, 1474347822, 1457553896, 1478748976, 1473845147, 1481233819, 1472061894, 1471390238, 1465229444, 1486599838, 1455208784, 1466323308, 1455479540, 1456421920, 1480383971, 1483067824, 1456507272, 1484030988, 1468229011, 1471445654, 1485458082, 1488261866, 1481622876, 1465105342, 1480212333, 1456189648, 1452635199, 1471018794, 1478078169, 1454264344, 1464107365, 1465035053, 1453008152, 1465353493, 1473626948, 1465986582, 1474553993, 1454902631, 1485717623, 1481539497, 1473996827, 1482337598, 1486366916, 1457931355, 1486347903, 1454399632, 1486441248, 1477700433, 1471313561, 1477905530, 1486036523, 1483716932, 1467780176, 1482110701, 1453782354, 1452420153, 1474284781, 1479676058, 1472232329, 1479590085, 1465860113, 1454831361, 1486072473, 1468588807, 1451654044, 1452595587, 1466513720, 1465516699, 1473171133, 1462030231, 1473527218, 1463994609, 1482867817, 1464901352, 1453999266, 1461386614, 1466679066, 1478683718, 1470608278, 1483786875, 1477985534, 1484377113, 1467893360, 1484754101, 1481024609, 1452581508, 1485128005, 1478634406, 1472170929, 1481728579, 1474458345, 1479234164, 1471160839, 1456075049, 1455894400, 1456719327, 1484877603, 1451996117, 1480608659, 1474202301, 1463196663, 1479782158, 1470355663, 1473655601, 1485659102, 1462044144, 1483234339, 1472781417, 1483537792, 1466027738, 1469278271, 1477148559, 1453709033, 1467008442, 1452879203, 1479346611, 1472570853, 1473017955, 1460847350, 1481506293, 1451778692, 1476987326, 1465449276, 1454189142, 1454264344, 1487175601, 1467408338, 1463488980, 1478634406, 1453080014, 1480967940, 1487662683, 1459018458, 1454656442, 1475312323, 1463667095, 1483059504, 1473050078, 1464965095, 1471678032, 1484181511, 1467247719, 1487226052, 1487161948, 1474523662, 1458873282, 1453463349, 1456684415, 1483923040, 1476034988, 1453894016, 1465032833, 1458302673, 1457584969, 1475348880, 1478681652, 1472689910, 1461277210, 1463143424, 1479629063, 1471964984, 1469284557, 1451965797, 1461152456, 1481051483, 1473159829, 1474791425, 1479918695, 1478514041, 1452175467, 1453206175, 1465416045, 1486673920, 1487127625, 1483042546, 1458666607, 1459708155, 1480193422, 1475004112, 1459613912, 1452921598, 1480162967, 1472306731, 1459329538, 1462164203, 1464416839, 1468182616, 1456676710, 1479561722, 1473514790, 1458828641, 1469212867, 1456963654, 1452794792, 1475444036, 1459374470, 1473110223, 1470234999, 1478678871, 1456387692, 1478294588, 1455751592, 1475773137, 1457506357, 1464793236, 1469743919, 1485166235, 1484665770, 1470639632, 1452301670, 1461649645, 1456610511, 1464094071, 1460464582, 1468692367, 1485508286, 1474899318, 1457375605, 1484064607, 1479126434, 1468525293, 1455626153, 1483553720, 1469584712, 1474446745, 1475033693, 1472868286, 1459145455, 1460085681, 1479818801, 1486324101, 1460439844, 1481583223, 1471533810, 1479238018, 1485500376, 1487219415, 1480952789, 1481258310, 1451739437, 1456296943, 1466877143, 1453631436, 1474421352, 1467595107, 1458165031, 1466050966, 1483410876, 1472717024, 1455771738, 1458371850, 1477148147, 1457446930, 1477499076, 1461616667, 1475492217, 1452528419, 1465952443, 1475740862, 1452955306, 1453016507, 1476625337, 1461517870, 1474803041, 1470884033, 1452495175, 1464085772, 1469150441, 1458670847, 1461096926, 1468168005, 1454299175, 1487365740, 1470523142, 1485498360, 1464067601, 1483105769, 1466229638, 1477136236, 1466257500, 1452076829, 1465308903, 1452054902, 1473045439, 1479595970, 1466003047, 1460881925, 1473748610, 1477421456, 1481267100, 1462792975, 1468913113, 1478356003, 1472627836, 1485498424, 1486262568, 1472958374, 1485529957, 1479433223, 1471930975, 1486817904, 1483491558, 1459890269, 1458365301, 1456179726, 1455620175, 1479567425, 1463649108, 1460019039, 1475492217, 1477648528, 1474972546, 1464684937, 1470974157, 1467300775, 1480141589, 1457012287, 1461862978, 1456359902, 1455632296, 1465695904, 1462182029, 1473754904, 1461580669, 1465087911, 1484614091, 1472123477, 1486732184, 1481645388, 1454370320, 1452195076, 1466534660, 1454063311, 1458456102, 1482380648, 1469679775, 1469017087, 1472837555, 1478866558, 1476538184, 1456529310, 1464773046, 1466886648, 1487733734, 1465241988, 1455575812, 1483706962, 1479849263, 1461152456, 1471730203, 1462797082, 1475088315, 1478830077, 1460437660, 1475740862, 1474172102, 1468568480, 1475966864, 1476353255, 1477437780, 1467401481, 1468290124, 1475196297, 1477720255, 1487885193, 1485304508, 1465062924, 1478906970, 1453042834, 1455852426, 1470501176, 1469745895, 1474727582, 1477288093, 1484935659, 1455635498, 1486297640, 1487661706, 1481274453, 1473996827, 1468376643, 1467507927, 1472349289, 1474643154, 1471986269, 1478674633, 1476899774, 1466546424, 1455065233, 1479007929, 1462779102, 1453953288, 1465273758, 1456223773, 1472386394, 1456756967, 1480184542, 1465654334, 1483211638, 1475640018, 1466297501, 1462381279, 1478705433, 1460303860, 1455080942, 1469342830, 1452141507, 1471480174, 1480527782, 1467635508, 1475531931, 1454029420, 1455011208, 1466487602, 1455611975, 1483742448, 1471029509, 1470813734, 1477341785, 1483362949, 1476434768, 1461589171, 1452492711, 1470713621, 1467001731, 1476230469, 1482816355, 1482936025, 1457187172, 1452923510, 1483105769, 1478578796, 1474808515, 1458860918, 1487114692, 1452718442, 1486052064, 1475501647, 1455679068, 1483484926, 1464868998, 1454549075, 1460670046, 1455290987, 1458579599, 1483052846, 1479171682, 1483074134, 1465312576, 1458080011, 1462727976, 1462469031, 1474692794, 1453672945, 1475416352, 1472071201, 1465166019, 1469220902, 1471606212, 1475276820, 1471868208, 1466850404, 1471303970, 1470679262, 1482235774, 1465050908, 1474602919, 1457553896, 1484272794, 1466918112, 1485592474, 1464044953, 1458567682, 1481317148, 1478219245, 1457187172, 1481399430, 1465329895, 1483484926, 1464122694, 1455653691, 1486405211, 1485428471, 1477910013, 1453521824, 1465647999, 1470552236, 1455938890, 1462111884, 1483163953, 1468077395, 1477957353, 1469743919, 1461128664, 1454443781, 1480427548, 1459218006, 1479767061, 1479999731, 1463047209, 1477861943, 1473584003, 1475841735, 1476343022, 1462329812, 1465097444, 1479069308, 1479648048, 1476935042, 1475790181, 1466886639, 1483954045, 1467215637, 1484459834, 1476134834, 1466681406, 1455021748, 1459329538, 1477033093, 1482227442, 1484877558, 1459658436, 1460051329, 1470012338, 1487933267, 1465057205, 1467149436, 1458565064, 1458532652, 1455945213, 1467851147, 1487733265, 1474570798, 1470298861, 1478652828, 1483658318, 1461081903, 1459955613, 1477771397, 1457323764, 1456605827, 1469268298, 1459861690, 1462634565, 1451844588, 1486163854, 1467541141, 1482257507, 1471218352, 1469111040, 1460200359, 1488235718, 1456495496, 1463874747, 1477140014, 1463625875, 1461726416, 1487071005, 1479182828, 1464747496, 1469859783, 1470230851, 1469777413, 1476486022, 1466070618, 1462639826, 1461034780, 1485151447, 1466235461, 1462041061, 1452425880, 1470764886, 1470256393, 1481257591, 1468098533, 1470400025, 1455063908, 1468409114, 1455221730, 1475090917, 1483285594, 1463888131, 1474444277, 1454791437, 1468168005, 1472643398, 1461465251, 1473031363, 1471539718, 1477060798, 1478441481, 1456794960, 1479819729, 1468340148, 1478092551, 1470945080, 1486257097, 1456716664, 1482812553, 1465359192, 1483794668, 1459259673, 1487873701, 1483558510, 1478654287, 1487181430, 1462210325, 1453411981, 1458796114, 1453332005, 1454577825, 1473045439, 1460592484, 1467264447, 1455171344, 1461364883, 1478032243, 1456193229, 1481024609, 1481796763, 1488134169, 1461057913, 1469776948, 1473411852, 1488306067, 1474605856, 1481896176, 1459875124, 1485318177, 1476973394, 1480145612, 1471366486, 1482204854, 1470725646, 1484090218, 1457806389, 1483450158, 1451949746, 1469317311, 1457951708, 1461616667, 1465611937, 1460590908, 1453865218, 1483012879, 1457386756, 1456124267, 1470171697, 1480705117, 1470699057, 1466926640, 1452441177, 1485858690, 1487332484, 1455154618, 1483232867, 1487350889, 1465071140, 1473171133, 1474651070, 1463716705, 1487352159, 1462426982, 1481905700, 1481585673, 1483934461, 1458187487, 1485092355, 1485128077, 1460940707, 1487675569, 1459776343, 1454531087, 1485108841, 1472875589, 1461344593, 1470785443, 1479316267, 1461641937, 1481707161, 1457228199, 1487740810, 1459201442, 1464377667, 1484472068, 1469572151, 1467851147, 1455707008, 1467172699, 1474045474, 1468473766, 1482409181, 1485507315, 1452807355, 1457612920, 1477435190, 1453561734, 1484696663, 1487837423, 1479703237, 1478281513, 1468361049, 1452463430, 1475438878, 1485469991, 1476829288, 1460910220, 1477232637, 1487581696, 1455910201, 1478702623, 1478754471, 1459934967, 1469794798, 1482550800, 1451893789, 1459875124, 1480868474, 1479217649, 1487100107, 1465407137, 1486153211, 1486171952, 1479687734, 1461251785, 1480160797, 1472814889, 1452345021, 1469990335, 1454308486, 1486642093, 1484645183, 1480289842, 1476068210, 1462840610, 1451860532, 1478178183, 1473315398, 1470966514, 1468721554, 1485259402, 1483811839, 1453331761, 1471194511, 1479094259, 1455180106, 1454370320, 1476243208, 1458180319, 1460475838, 1487343906, 1465419075, 1476375260, 1460999664, 1460718388, 1453236054, 1483987206, 1472694661, 1455060771, 1459950627, 1463181398, 1455063908, 1457863269, 1482072074, 1477053068, 1467708359, 1462576618, 1484651043, 1475229412, 1455436764, 1458983663, 1454530986, 1481024609, 1472867294, 1458214196, 1471640420, 1487960824, 1481890568, 1453175254, 1488213267, 1467914568, 1469893396, 1456729774, 1452946274, 1485829485, 1452659005, 1485778001, 1483811634, 1464997374, 1466353797, 1469670356, 1484821041, 1453860255, 1463859525, 1472141310, 1466260849, 1459403517, 1470525446, 1460834613, 1464642869, 1452801801, 1472090281, 1459705100, 1465834382, 1480064490, 1468151560, 1459207032, 1476981819, 1483616470, 1458486364, 1461887466, 1452693028, 1460728584, 1452782503, 1485386064, 1473968722, 1481996033, 1462509571, 1485113412, 1461187550, 1459974594, 1455021748, 1483565251, 1486931588, 1486126701, 1467527448, 1470598239, 1479370894, 1481590094, 1467153028, 1468200913, 1474879575, 1485267186, 1483346964, 1459222993, 1487044041, 1487567795, 1482709512, 1460128805, 1479835293, 1462446757, 1468951769, 1472390407, 1485115941, 1456846869, 1461164437, 1468615461, 1475459910, 1460463590, 1453056246, 1473681260, 1483129201, 1462339312, 1457975195, 1457689085, 1485243537, 1476407389, 1454454649, 1475010101, 1464099685, 1468615461, 1459025940, 1452635710, 1484329445, 1462665453, 1481187027, 1473520625, 1476316890, 1459713634, 1486022269, 1476299959, 1468796505, 1471492142, 1473339304, 1460952109, 1479671230, 1470525446, 1481982416, 1472440877, 1458543585, 1458249037, 1480025182, 1466112214, 1487650200, 1468734655, 1458495391, 1488012521, 1459093907, 1483472562, 1472142361, 1474708238, 1460139735, 1484454672, 1474819687, 1458884024, 1480811690, 1477928033, 1459151022, 1453122069, 1457698520, 1454419884, 1486096172, 1469208281, 1481895579, 1479401753, 1470400025, 1461591469, 1458794400, 1469831449, 1478166202, 1473952196, 1460975253, 1485290437, 1480403018, 1465559814, 1469950543, 1464105310, 1485361372, 1461538441, 1481838125, 1469686278, 1484253484, 1472987703, 1466075974, 1484433617, 1485074093, 1486969601, 1463431525, 1473747696, 1466347846, 1483621568, 1467043289, 1488319813, 1459485137, 1468871484, 1455693764, 1460674661, 1474714536, 1469697234, 1466562365, 1479676058, 1468720152, 1471966273, 1454594623, 1479818801, 1483559726, 1476092722, 1464288667, 1453386744, 1470102373, 1470819182, 1458167732, 1457765861, 1481395026, 1482173925, 1454360848, 1476043976, 1473585990, 1474848672, 1480428575, 1466093864, 1454791437, 1460636111, 1487223382, 1471870712, 1465050908, 1486260979, 1465400207, 1476158920, 1458929008, 1471273805, 1473506952, 1487816688, 1481806246, 1487500062, 1483269589, 1477648528, 1459658436, 1488021081, 1473241391, 1485092355, 1484740867, 1459384011, 1484563639, 1487812740, 1476255624, 1461843055, 1453361398, 1468722351, 1477512614, 1465146265, 1482982241, 1474288100, 1472778773, 1476210899, 1459250533, 1470420940, 1467629313, 1476158446, 1470402205, 1463202675, 1483538677, 1455864599, 1452287620, 1461118595, 1466206275, 1470026319, 1488222010, 1485637529, 1479283005, 1461169513, 1479879394, 1473337341, 1475837084, 1484812665, 1461607890, 1482655853, 1457453427, 1463214331, 1485562885, 1468803396, 1472277075, 1484684377, 1470194613, 1463910293, 1485341337, 1481000161, 1466193825, 1461500043, 1462747521, 1479310813, 1471471745, 1453353213, 1475420105, 1476572881, 1454888909, 1472546298, 1478618704, 1469200695, 1456269238, 1486506037, 1461464892, 1479292764, 1471573760, 1451816738, 1467807964, 1472127565, 1477771397, 1455237667, 1451752723, 1467878220, 1480655997, 1477207043, 1460774926, 1488225550, 1463291330, 1482230672, 1473272789, 1476224878, 1484077297, 1463826925, 1458326204, 1471213973, 1473248832, 1485259402, 1462932035, 1481642413, 1478321187, 1452468868, 1452918657, 1470340240, 1453424792, 1466579245, 1454864257, 1467265045, 1486455540, 1484687389, 1455019389, 1473272789, 1476412872, 1467276548, 1459198975, 1459151022, 1455881593, 1461122065, 1460557653, 1481857884, 1470767268, 1487841447, 1457412365, 1469411837, 1455047051, 1480914093, 1456189648, 1459224722, 1476611724, 1465842707, 1463998790, 1480568552, 1481224529, 1457359735, 1470121486, 1455512405, 1473799084, 1472986135, 1477346578, 1484777702, 1475276820, 1484815997, 1486162158, 1481187027, 1472814889, 1465671524, 1487156974, 1483941970, 1457674441, 1487733265, 1481091632, 1467778802, 1480658632, 1482225555, 1481896308, 1485439095, 1452501428, 1486631176, 1481891459, 1485801525, 1455601031, 1486827063, 1479388024, 1482418666, 1481847867, 1455964958, 1452134995, 1458841635, 1459142317, 1456639374, 1475487386, 1457750519, 1470361031, 1462440719, 1467044431, 1462383970, 1487548766, 1463157418, 1465531163, 1453513833, 1488299995, 1469318659, 1468862006, 1480462336, 1483234339, 1488324413, 1481182888, 1456450030, 1481469932, 1486732074, 1456812678, 1471331774, 1485236479, 1487958755, 1455819831, 1465480589, 1476711830, 1480598836, 1464239137, 1470247804, 1455428264, 1475416352, 1484143884, 1464642869, 1470904721, 1470293632, 1468993430, 1475841735, 1477861943, 1475010812, 1459309032, 1456334038, 1484233421, 1473027213, 1487199104, 1482056736, 1454413032, 1479407133, 1482402046, 1467780174, 1469764471, 1477836558, 1483048387, 1468119498, 1468311941, 1483658318, 1463337679, 1461352506, 1480428575, 1471793924, 1457509152, 1482967176, 1455068032, 1459259390, 1488070752, 1470889240, 1452336993, 1458415790, 1475125319, 1455299230, 1475322717, 1479570522, 1478287047, 1483307602, 1455717073, 1474533788, 1474728050, 1461740019, 1483164533, 1485473614, 1475137192, 1484531362, 1470679703, 1472802889, 1467736192, 1467829609, 1451905921, 1471730203, 1467215637, 1471669561, 1481320989, 1486953073, 1461641602, 1470172937, 1481166968, 1457800140, 1453683097, 1463970024, 1486809600, 1465672569, 1483394208, 1471856824, 1485439095, 1487597165, 1482757364, 1485695042, 1483700101, 1486528418, 1468232872, 1456366084, 1476077154, 1461014012, 1477887330, 1452337219, 1456370992, 1465837350, 1460502363, 1474225832, 1455091667, 1454438240, 1474675300, 1473286352, 1484043244, 1484111886, 1484936478, 1486998789, 1487539117, 1454292029, 1479012511, 1454385701, 1464748965, 1462910206, 1458901000, 1456997074, 1463354901, 1451607860, 1466473272, 1487124250, 1477307527, 1474969260, 1477641464, 1484887029, 1481728579, 1470294589, 1484094640, 1467276548, 1472067351, 1479078418, 1468793612, 1488062828, 1466681406, 1477001630, 1462847179, 1481958211, 1452348638, 1453249205, 1486383847, 1481641632, 1479176697, 1475817668, 1483112731, 1476095630, 1459334181, 1482726380, 1478248924, 1470246741, 1456548289, 1477499076, 1459867139, 1451874436, 1487874190, 1475289657, 1459113523, 1487519801, 1467003579, 1476340708, 1478106825, 1456556262, 1478032243, 1476590353, 1485036424, 1471552539, 1474444277, 1461945735, 1486535478, 1459268827, 1461929325, 1456817576, 1481932993, 1486092109, 1474347203, 1478584127, 1482918980, 1485220349, 1454683445, 1478892578, 1455436764, 1455019422, 1455712092, 1455443280, 1457744397, 1459320441, 1467463553, 1476063173, 1466206275, 1458417780, 1462021189, 1469284557, 1484576700, 1468637073, 1480918299, 1483544160, 1486604593, 1472670872, 1460026288, 1462195074, 1480180836, 1462989872, 1463474390, 1475580821, 1478931653, 1452939253, 1467111099, 1458185526, 1466257500, 1478674633, 1474261121, 1475708899, 1486939537, 1468722351, 1479682670, 1455148304, 1479795956, 1457689257, 1474245760, 1461748617, 1487939074, 1487608894, 1468195072, 1482498588, 1466775364, 1461091814, 1472741733, 1466454739, 1464337755, 1486199789, 1457224525, 1467397899, 1478834027, 1463269853, 1457227828, 1459114165, 1484436275, 1487075326, 1455011479, 1482876227, 1460764269, 1460883378, 1481536478, 1483972350, 1478342313, 1475589274, 1486826980, 1478357746, 1466852996, 1472921840, 1472370502, 1471933236, 1484720867, 1477951817, 1466258198, 1484624920, 1453382695, 1454520818, 1455389623, 1452174023, 1458218979, 1452845969, 1464015844, 1468614918, 1466381495, 1484012550, 1471856824, 1482257507, 1478441926, 1466588750, 1457755659, 1466366160, 1470679703, 1472976974, 1468580839, 1479524402, 1456800228, 1478660350, 1454607964, 1476172765, 1469529922, 1462816163, 1456921458, 1463594975, 1472631049, 1484800332, 1463795217, 1478574106, 1470074785, 1463150244, 1469980972, 1483357089, 1471526206, 1476175856, 1485720392, 1464386012, 1466760845, 1476299959, 1469750859, 1481053585, 1467380086, 1483681601, 1474195043, 1475081707, 1457989802, 1479491878, 1464639901, 1458827572, 1475346075, 1453082412, 1467804760, 1457334689, 1470647917, 1483313037, 1488217898, 1459134195, 1461770562, 1474553993, 1457518045, 1472117190, 1470581623, 1458983580, 1472643398, 1453883399, 1479069308, 1468201731, 1483067824, 1456476630, 1452960650, 1455222928, 1475270215, 1463608559, 1480677908, 1480279596, 1461132657, 1453299440, 1469247377, 1474260325, 1468255288, 1460145199, 1487161948, 1466887139, 1472365161, 1478804442, 1485928235, 1477374547, 1468416721, 1485473614, 1462003045, 1451710949, 1474186327, 1467115887, 1453701607, 1461544927, 1476507451, 1455022964, 1468507731, 1460321599, 1473023788, 1466219810, 1474287812, 1479850639, 1487685234, 1463851768, 1481820941, 1475054438, 1479504306, 1473237412, 1476901382, 1469009767, 1455702216, 1486202283, 1481462670, 1480041139, 1461486147, 1456741560, 1475023015, 1477027811, 1481954643, 1482954598, 1476939820, 1482639854, 1486616823, 1483621568, 1455168260, 1453284558, 1457197692, 1478481644, 1465486158, 1453268867, 1484200087, 1472736532, 1451700694, 1487851132, 1482284703, 1452214123, 1470298861, 1485318177, 1479929991, 1459250533, 1478611757, 1468313518, 1472780692, 1463900541, 1485461790, 1466305406, 1477635642, 1475481616, 1486952358, 1464268076, 1484241922, 1477519058, 1471088026, 1463685208, 1457357840, 1479806662, 1462044144, 1475683713, 1464604014, 1488077301, 1456243234, 1474211922, 1452753019, 1469331324, 1475073509, 1484740867, 1460384437, 1460797185, 1480468252, 1475803015, 1475555239, 1479224683, 1476500452, 1484091996, 1487670782, 1473097922, 1463262636, 1452398529, 1475736321, 1469883062, 1474245760, 1478107705, 1469938054, 1453908840, 1463520999, 1466803142, 1471837505, 1483581909, 1478165663, 1486126266, 1467573356, 1467987090, 1461770562, 1458546378, 1475397378, 1466666337, 1465484059, 1487939074, 1456653600, 1461145974, 1457576443, 1472989654, 1462241475, 1454343810, 1456126595, 1479645046, 1465876215, 1461688585, 1472004874, 1455384453, 1482936025, 1472216669, 1469914449, 1469956457, 1479646396, 1485484873, 1472330926, 1472016535, 1457849154, 1476151447, 1459547401, 1462413880, 1464506276, 1461969114, 1483171159, 1461512761, 1451897630, 1468293875, 1483037476, 1478882378, 1481535092, 1471421594, 1453631592, 1474229929, 1452747674, 1474803041, 1469925795, 1481494075, 1472961569, 1473009096, 1484402335, 1454221896, 1468808003, 1482038156, 1455784536, 1484696663, 1451657744, 1452823522, 1484332585, 1457302840, 1452888817, 1474240582, 1452351161, 1478900236, 1470138225, 1485296699, 1461820557, 1469516222, 1455473336, 1458805679, 1469096465, 1470903523, 1484862598, 1485688176, 1476407389, 1471551543, 1474614461, 1466140425, 1451656253, 1452687642, 1482235774, 1480454724, 1461227126, 1457726663, 1474523233, 1468905991, 1480659475, 1463654391, 1485290854, 1471840130, 1473847925, 1461518432, 1462339312, 1469364936, 1465582716, 1468464524, 1469231310, 1467410742, 1476391589, 1459576981, 1481720490, 1465985955, 1461960956, 1467395133, 1481010706, 1457885869, 1462305196, 1485391258, 1465398458, 1473050078, 1481036592, 1469507347, 1456599986, 1485469991, 1457187172, 1458984517, 1485990532, 1466538268, 1485507315, 1488070752, 1469150441, 1485021850, 1458406724, 1487550522, 1459875603, 1461500043, 1469315738, 1462232505, 1464019431, 1470642767, 1487033737, 1476190897, 1479553311, 1486346153, 1451969401, 1468442606, 1478847591, 1455860802, 1451874436, 1475978739, 1465486158, 1484595567, 1456916919, 1456681014, 1483153840, 1471418985, 1453787516, 1485553702, 1479737281, 1487015107, 1459853692, 1472520772, 1459198975, 1470193247, 1456811901, 1465455754, 1454902631, 1483855729, 1474737585, 1452080674, 1487147736, 1482622775, 1472627582, 1455635498, 1454063311, 1460109789, 1485990532, 1462693370, 1476799235, 1481274453, 1474403822, 1475568365, 1487143203, 1467883843, 1471010145, 1460069714, 1454194737, 1453844453, 1472692457, 1459309912, 1484771671, 1458271141, 1454581911, 1453045271, 1477887823, 1462244120, 1464497189, 1483943235, 1461665673, 1453249205, 1453631592, 1462052228, 1475941106, 1451616403, 1477160606, 1454343810, 1482029622, 1472222532, 1467682053, 1465518865, 1487143203, 1456209859, 1475790181, 1459556763, 1456921458, 1486739613, 1474898079, 1462017352, 1482867695, 1485400073, 1468612967, 1451910519, 1481836171, 1452525036, 1474503693, 1470152106, 1452975753, 1456343442, 1460119211, 1472951005, 1485507315, 1480071845, 1478310663, 1467580385, 1462244120, 1478668217, 1463791369, 1486570213, 1451921618, 1486954823, 1458157813, 1479869612, 1479253117, 1483457246, 1486978079, 1469287006, 1457778563, 1475584075, 1462822684, 1466794692, 1467618767, 1470440858, 1485803845, 1484897390, 1474718536, 1454650427, 1487748501, 1468727201, 1487851146, 1483785312, 1485224317, 1472802889, 1477638493, 1453534129, 1488193847, 1467787430, 1480952789, 1485470316, 1484340376, 1479969451, 1469886051, 1479182317, 1481838125, 1471398462, 1472375680, 1485833584, 1487437414, 1482988793, 1461051878, 1458362067, 1462634565, 1459830891, 1483395846, 1483012879, 1486342561, 1454461342, 1464965095, 1481186324, 1468793128, 1478068955, 1479438751, 1472170929, 1455071711, 1485720392, 1453842931, 1454396814, 1483565251, 1456286971, 1474242440, 1459224722, 1485918475, 1461710950, 1474132289, 1451947798, 1474110948, 1467761541, 1454024678, 1453629550, 1484517730, 1464341059, 1480427548, 1467359365, 1478133262, 1459028555, 1476955551, 1458033159, 1485088010, 1451798070, 1469315738, 1456238833, 1478467686, 1478665076, 1469119047, 1461274066, 1465251598, 1482635936, 1460617269, 1469055418, 1457655373, 1454140557, 1461385606, 1481368397, 1465130277, 1460940707, 1479149677, 1481860350, 1468098533, 1479902363, 1466147348, 1466617345, 1483832780, 1465941286, 1466960468, 1483450158, 1487270433, 1467480272, 1485391258, 1475010101, 1452058953, 1484527166, 1483394208, 1458682987, 1455162773, 1465992022, 1468645484, 1464504647, 1475977995, 1481723140, 1463788343, 1461459138, 1487267005, 1485050184, 1467699949, 1478514041, 1473999329, 1462424576, 1485149042, 1486168809, 1486785116, 1465985955, 1455479540, 1472617449, 1461644791, 1480201150, 1461581835, 1455285711, 1477778155, 1457946056, 1458176678, 1452164189, 1487044041, 1482766112, 1451754280, 1483234339, 1472152533, 1463998015, 1474172102, 1471484227, 1464916946, 1471257654, 1460866578, 1463600634, 1479630183, 1465086517, 1486936926, 1461612083, 1484555386, 1467918347, 1463374327, 1458479561, 1480053656, 1485389662, 1469278271, 1481424220, 1473329581, 1463826925, 1454352689, 1479292033, 1468054118, 1472317457, 1464169904, 1453876393, 1484572816, 1479130129, 1478126286, 1476204977, 1472831721, 1474082914, 1451940407, 1469423465, 1484847491, 1487933267, 1483004273, 1457377052, 1482402510, 1461017459, 1468939217, 1454627808, 1459896428, 1461368605, 1484644105, 1467761541, 1468877045, 1482354538, 1467001731, 1481595789, 1466181976, 1478537498, 1473247816, 1469281735, 1488135261, 1475367226, 1484800332, 1467215637, 1456880895, 1458415505, 1463225983, 1463910293, 1487270433, 1468423853, 1482240083, 1465304436, 1468249418, 1461607890, 1484636963, 1476829288, 1456364297, 1471944919, 1462177576, 1487071312, 1472682691, 1475397378, 1473081961, 1480391564, 1452231411, 1459775304, 1452430786, 1471247811, 1474593366, 1481091632, 1469881217, 1460152703, 1452659005, 1469319129, 1482199372, 1464502578, 1475075470, 1487982842, 1477826210, 1473171133, 1486322072, 1464455530, 1453949206, 1477173709, 1486739613, 1484527166, 1463751606, 1476584144, 1471424875, 1483906388, 1452909057, 1484052112, 1484380110, 1478912655, 1479731545, 1481182888, 1453035181, 1472151466, 1483269589, 1485812245, 1476946268, 1458001110, 1475367357, 1482472677, 1478234591, 1480398681, 1474518718, 1461644791, 1480468252, 1453501265, 1473652085, 1486065155, 1484936478, 1458154904, 1452958646, 1472556968, 1466791321, 1465217609, 1481980143, 1463189920, 1457637849, 1485064244, 1488191778, 1460393734, 1477203103, 1466775364, 1481231970, 1481009427, 1483090011, 1463356807, 1457699479, 1482306040, 1488193847, 1468156331, 1483601565, 1458849935, 1459409036, 1478731971, 1467964170, 1454438746, 1474889695, 1478407078, 1467393368, 1478446966, 1474972546, 1463581062, 1488279177, 1482362326, 1468913113, 1468579894, 1482676569, 1454525754, 1482834564, 1462955185, 1486182940, 1456607527, 1451910519, 1452567830, 1461268143, 1463607766, 1478373191, 1459049036, 1477060798, 1454508604, 1462369262, 1475790764, 1486195088, 1460941272, 1456373893, 1471930726, 1477778116, 1456017027, 1460074150, 1465390354, 1456856988, 1485119692, 1457956584, 1453447625, 1465570252, 1463668413, 1483561078, 1471629386, 1454636170, 1462897353, 1465454270, 1455241006, 1453402630, 1475463748, 1469063080, 1465491845, 1471939629, 1471855092, 1454913596, 1482365664, 1476847329, 1477186673, 1464379353, 1452703682, 1474385047, 1476600536, 1486570213, 1478200158, 1452240664, 1484592819, 1453016507, 1480961108, 1471247811, 1467007714, 1476395285, 1463196663, 1471410637, 1471265968, 1476698091, 1488213606, 1473294012, 1460681485, 1463067300, 1452680665, 1468433077, 1473604704, 1474173392, 1463368105, 1457330292, 1468473766, 1483004273, 1462782607, 1460044058, 1470651624, 1456203896, 1464806469, 1466012588, 1476950986, 1469322219, 1472645803, 1478762136, 1488162434, 1485237508, 1486363750, 1455730097, 1464344767, 1479012195, 1478664775, 1459415736, 1452794792, 1457985996, 1456592331, 1455697338, 1473030221, 1485042915, 1487540505, 1454059035, 1479865440, 1453505090, 1461311365, 1486687096, 1483999357, 1483773577, 1454456460, 1474906926, 1456864957, 1462973305, 1460774926, 1465798762, 1457912143, 1477212101, 1472253643, 1473153365, 1456500081, 1484288549, 1476387894, 1455784536, 1463801183, 1462372141, 1487734413, 1455228531, 1479813586, 1466339030, 1467918347, 1469008550, 1484662929, 1463225983, 1453902404, 1474173098, 1464777994, 1456356935, 1460617269, 1477866483, 1477120469, 1479185862, 1483149902, 1454443781, 1462469031, 1452707474, 1483199984, 1476387933, 1479642513, 1461630737, 1456846869, 1484948272, 1486952358, 1464916946, 1457689257, 1481536478, 1451762811, 1471418985, 1482891028, 1459259390, 1464015844, 1454216853, 1472439515, 1478050148, 1457637849, 1482724671, 1480493557, 1461843055, 1472472376, 1483923040, 1473353578, 1485801525, 1483036668, 1456408037, 1452848232, 1458149525, 1478132573, 1474512902, 1462520751, 1476540977, 1484860932, 1457022868, 1477120469, 1461504281, 1457786518, 1454160043, 1451798752, 1470009401, 1486086317, 1476248684, 1481019725, 1461227126, 1487527668, 1454252796, 1461114030, 1463540745, 1455650525, 1477291496, 1486634234, 1479888076, 1462810156, 1476603362, 1486550089, 1477460886, 1456472516, 1481804226, 1456471085, 1458218979, 1482723552, 1485877274, 1482284703, 1464837393, 1462723885, 1485119692, 1460132211, 1475567758, 1461274066, 1458815844, 1461096926, 1451700694, 1469734319, 1453023061, 1454965703, 1485124965, 1471464304, 1465972388, 1457755035, 1476986933, 1456104541, 1461002487, 1476931897, 1459130015, 1467044431, 1457944584, 1483866387, 1456782576, 1457404416, 1465468862, 1458406724, 1470821232, 1464548406, 1474762838, 1451955575, 1483954045, 1476472938, 1482153232, 1477818115, 1479832836, 1483097630, 1479388024, 1482101449, 1484689768, 1462519758, 1463351602, 1478578796, 1462502256, 1479671230, 1454140557, 1475612679, 1475054438, 1479224683, 1472299272, 1486036523, 1475248005, 1481957841, 1457375605, 1458611875, 1463268521, 1472155065, 1482572494, 1458425740, 1481388056, 1462162002, 1473024542, 1483992010, 1479736899, 1456789816, 1459018458, 1452546192, 1475845026, 1475706825, 1486908153, 1453652357, 1465086932, 1462402759, 1453996241, 1451937986, 1484079692, 1470817648, 1487100107, 1473743159, 1463135868, 1475304347, 1474643349, 1483313037, 1467480441, 1483048387, 1454450413, 1473892509, 1472868286, 1453770702, 1477173709, 1484720867, 1477504480, 1457755659, 1463315638, 1467049089, 1470767315, 1473532595, 1482199656, 1473211379, 1463929366, 1460310156, 1484050924, 1485940642, 1452546192, 1480976906, 1460037450, 1473485460, 1487511155, 1459409801, 1483951020, 1464442076, 1465341119, 1465011945, 1484030988, 1458983580, 1487257164, 1479292033, 1457985996, 1486302472, 1482444172, 1463649108, 1487199223, 1452538235, 1472010232, 1453424996, 1468299180, 1473421779, 1464283684, 1473809325, 1469408240, 1472016535, 1476429624, 1458243835, 1470156867, 1486086586, 1480123338, 1487519801, 1477429021, 1467573961, 1462917917, 1485486929, 1470889240, 1481374365, 1469994470, 1473493246, 1470916768, 1485109216, 1453064064, 1471140707, 1468156206, 1453424792, 1484208985, 1486695404, 1478195272, 1462106212, 1487100107, 1464417250, 1484711284, 1468662447, 1453298606, 1463685542, 1452134995, 1474323603, 1470868307, 1454646644, 1488257607, 1482362326, 1477346578, 1462369262, 1487731859, 1461679226, 1478685848, 1456121138, 1452694779, 1474147915, 1483394208, 1466696903, 1460811252, 1461490009, 1482791977, 1485095913, 1476499562, 1468325414, 1478970880, 1480984052, 1478839517, 1468793021, 1476575878, 1468437640, 1482183284, 1483990167, 1476204977, 1484459834, 1470102373, 1467028530, 1469724080, 1471659335, 1487405204, 1478178972, 1485201743, 1460590162, 1452208898, 1458518143, 1467226839, 1459419476, 1468690977, 1459392886, 1484887029, 1470921242, 1487981619, 1458777812, 1453908840, 1475270215, 1481992312, 1479298660, 1483551180, 1484356660, 1472967024, 1480114729, 1471926279, 1456867979, 1465637699, 1457800140, 1488190955, 1470973030, 1484595730, 1481258310, 1465763961, 1481350606, 1486535133, 1459950627, 1465400207, 1453194521, 1454188631, 1483004273, 1461885482, 1459825417, 1476350423, 1462441787, 1456709554, 1475451843, 1474762838, 1475274548, 1457005981, 1479515354, 1479492587, 1480475199, 1487885193, 1477740219, 1473766683, 1462780120, 1456089156, 1482227442, 1463887434, 1460954913, 1469322219, 1481305708, 1476946268, 1480474495, 1475835472, 1482889629, 1463586311, 1469796800, 1466027738, 1478134375, 1457348733, 1468693801, 1454664482, 1451866109, 1460910220, 1459888414, 1475088315, 1476939820, 1472225333, 1465569561, 1481654430, 1460976527, 1486809600, 1461119237, 1472595128, 1463748030, 1471102838, 1456519900, 1483458864, 1465029725, 1476646213, 1477733736, 1479331967, 1468229917, 1458752337, 1452543678, 1482791532, 1466918112, 1477010364, 1487543037, 1476244453, 1479273324, 1481469737, 1476545182, 1468278403, 1475314173, 1474658277, 1454805916, 1474434530, 1460469708, 1474289147, 1454618861, 1474082914, 1466546424, 1482526329, 1484200087, 1452543678, 1459740711, 1485700498, 1452303377, 1458008687, 1458356486, 1457307473, 1485665134, 1488012521, 1458789930, 1458495391, 1461410549, 1479063454, 1463168740, 1479366260, 1452336993, 1472090731, 1477498838, 1458430291, 1462106212, 1471425942, 1467931672, 1464720462, 1476841879, 1470014427, 1485573477, 1463751606, 1458121995, 1473807209, 1475411525, 1468437660, 1465227224, 1482153232, 1476646213, 1475106915, 1457504351, 1474368068, 1481459490, 1473088762, 1479986056, 1487568097, 1454392588, 1486903320, 1486365265, 1484143884, 1469243596, 1470493409, 1460074150, 1487140210, 1468955277, 1462766811, 1480615067, 1475687538, 1456607527, 1461641937, 1471029509, 1481765992, 1488077237, 1456093808, 1472389759, 1452703682, 1461628512, 1481590094, 1456648121, 1481148755, 1488191778, 1460152703, 1457273482, 1477664883, 1475263897, 1478586864, 1473370854, 1464939125, 1477928033, 1460081395, 1485055465, 1456084286, 1462889313, 1486175590, 1459145455, 1469329592, 1465977967, 1476328295, 1474727582, 1487016525, 1475004112, 1468168585, 1459222993, 1457701573, 1475538902, 1457818565, 1469080309, 1462769221, 1474702980, 1458827572, 1485186803, 1476464887, 1468727201, 1464446202, 1481726644, 1480196725, 1452303377, 1484149008, 1485039954, 1488122110, 1474147915, 1452501428, 1461686168, 1483648659, 1479782378, 1481342026, 1474340104, 1487764370, 1486059405, 1455740267, 1473961379, 1463746160, 1452958646, 1454435643, 1475886884, 1454972021, 1486796862, 1478403373, 1453022075, 1461198205, 1454292029, 1461938679, 1475797134, 1474737585, 1454991125, 1486952358, 1478290849, 1456099742, 1465221086, 1456204236, 1478356003, 1457733955, 1474481563, 1454628770, 1466696903, 1457316671, 1461117910, 1461008724, 1468653925, 1457424474, 1462373810, 1464036484, 1485450857, 1458517692, 1487794550, 1456112756, 1459050851, 1465770805, 1482013737, 1474339654, 1463440328, 1482572494, 1486949606, 1457334689, 1459447294, 1486949606, 1473225322, 1462446261, 1475584589, 1451869294, 1453521824, 1483027976, 1487790718, 1467614920, 1453563835, 1483347894, 1469342830, 1454902631, 1487004524, 1452865700, 1476946268, 1464443405, 1452223485, 1478084307, 1466819026, 1463685208, 1464995913, 1466083353, 1485278432, 1479515537, 1482210470, 1478355520, 1465969458, 1457634862, 1466751331, 1457068593, 1470725646, 1462954797, 1471942452, 1478826553, 1479731545, 1487668949, 1454427713, 1467907854, 1459576981, 1458720091, 1467001731, 1458718615, 1467851147, 1485659102, 1463498939, 1460605060, 1475337637, 1461244582, 1464002139, 1463639500, 1471512419, 1478429696, 1452917783, 1479601646, 1464038622, 1456422910, 1471427864, 1461889582, 1463135868, 1481759630, 1456450030, 1474727582, 1455659233, 1463794107, 1454194033, 1464014376, 1467567992, 1468644483, 1480428263, 1478693180, 1460893540, 1455769501, 1472627836, 1477304547, 1481567877, 1468685011, 1471973173, 1480618639, 1467328374, 1472355578, 1465683117, 1467397899, 1483987206, 1454500577, 1474762014, 1457182935, 1467761617, 1467383462, 1456286971, 1453299731, 1486872565, 1474300962, 1475631812, 1475880721, 1487078319, 1475501647, 1456569587, 1487033737, 1473624237, 1457446853, 1473061673, 1455608978, 1461544515, 1484979399, 1460145199, 1463195413, 1479703393, 1455542569, 1474347203, 1453789032, 1458518143, 1459740711, 1465033246, 1462317529, 1459188221, 1484129214, 1481842323, 1478295247, 1461249060, 1487510444, 1466886639, 1459287627, 1460324653, 1476802829, 1465649384, 1464398460, 1454291835, 1467121344, 1457885869, 1454941641, 1468040505, 1468259753, 1459084434, 1479995668, 1487550522, 1463037390, 1466665566, 1463600024, 1459148090, 1475516454, 1473948922, 1471552539, 1473464940, 1472253643, 1462262940, 1460236160, 1454981874, 1461838350, 1452975707, 1455092972, 1470642767, 1483051161, 1482572220, 1462424576, 1454703257, 1482590307, 1487812677, 1458110481, 1483261347, 1479990617, 1483814052, 1468906460, 1459756566, 1461088810, 1461121135, 1485170291, 1478065412, 1465468862, 1476915300, 1480455767, 1455169524, 1483193298, 1463660299, 1485790278, 1461864636, 1467761541, 1453186277, 1479258101, 1464783149, 1479879394, 1478839517, 1454643414, 1455961709, 1467552611, 1459234811, 1481869012, 1459303488, 1468943966, 1463948617, 1479644469, 1473747696, 1452208898, 1453169901, 1463575710, 1470697032, 1476568422, 1454435643, 1459795085, 1465141344, 1473342484, 1474944730, 1464995913, 1465529263, 1451684342, 1469679775, 1476804597, 1481876143, 1467247719, 1453165799, 1473912584, 1458567682, 1463631463, 1482199656, 1471238864, 1479957041, 1484241929, 1487385662, 1451965797, 1476172765, 1460040844, 1455765648, 1468579894, 1488256441, 1471730203, 1474932788, 1462531839, 1462699158, 1473294012, 1468878988, 1476574272, 1465333929, 1462573393, 1453865218, 1468721554, 1479533685, 1484237254, 1477079136, 1478307038, 1465057205, 1487755781, 1457733955, 1457167098, 1464014039, 1468464524, 1472180522, 1477758772, 1462381923, 1477441153, 1488213606, 1473055115, 1465345653, 1475324712, 1483199984, 1457956050, 1484109302, 1468422952, 1486072473, 1487782094, 1479660451, 1466916711, 1453206175, 1478459378, 1463289862, 1469950543, 1469329592, 1463440328, 1465845888, 1483822821, 1476595150, 1476677847, 1462955656, 1459397440, 1482546993, 1455148066, 1482072074, 1456189018, 1483159493, 1472696548, 1481923550, 1477060238, 1483621568, 1473337341, 1470702326, 1453466660, 1475640018, 1483499781, 1454904515, 1474158991, 1465946051, 1467296318, 1452190462, 1472929429, 1479686359, 1475810753, 1475367357, 1470921242, 1469394641, 1467728921, 1455786136, 1484840397, 1479515354, 1458333715, 1464053021, 1465233237, 1463377894, 1483827184, 1471215929, 1473061673, 1476657577, 1467382183, 1484193383, 1476092722, 1454336252, 1453700935, 1462829855, 1467088075, 1479300991, 1472627582, 1477305295, 1473371206, 1458847215, 1464336822, 1463779071, 1457307473, 1463351602, 1469157612, 1453379610, 1480885334, 1455937730, 1462955185, 1471579623, 1452180614, 1483593350, 1469408240, 1480935704, 1481057052, 1463623343, 1477844230, 1464021651, 1460140610, 1473892509, 1480110594, 1452763603, 1484288549, 1463493959, 1479323131, 1465449790, 1462995356, 1465225661, 1472037677, 1452747674, 1458460270, 1479323131, 1475612679, 1488190209, 1481846439, 1488029395, 1465757742, 1471400650, 1477122241, 1483152282, 1483510199, 1484483746, 1477128329, 1463712632, 1480098400, 1466070618, 1467501319, 1464506829, 1464613402, 1471919757, 1464481357, 1456231595, 1454316695, 1483114299, 1463210302, 1458663545, 1474694573, 1480391564, 1460883378, 1461889582, 1468251621, 1477897061, 1465914736, 1463374327, 1466003047, 1459276227, 1470557720, 1472012625, 1456685834, 1479534600, 1478136108, 1468994757, 1455801023, 1461426005, 1459069264, 1487364725, 1455082313, 1451994509, 1472142361, 1482270067, 1485273781, 1477404999, 1462995356, 1460266258, 1482067052, 1483458864, 1478253589, 1465309456, 1477441153, 1488153831, 1476007562, 1462702259, 1480071237, 1460139735, 1487502960, 1453789740, 1475355050, 1470185855, 1472411160, 1464606992, 1461462576, 1483484926, 1480259083, 1464844011, 1456846869, 1479224683, 1461418648, 1478892578, 1455417919, 1486413684, 1485074093, 1455168260, 1468416721, 1457637849, 1479536435, 1458138671, 1458938574, 1456709083, 1482558086, 1478573972, 1467363654, 1476396602, 1456673272, 1465672569, 1484707058, 1472820251, 1475319816, 1486077881, 1462583101, 1477442797, 1454627808, 1452336906, 1459873358, 1462816163, 1474192493, 1460040844, 1462439363, 1462889313, 1465407137, 1478281211, 1462840610, 1469734319, 1480284809, 1475837084, 1472397477, 1470345793, 1458019989, 1453343936, 1457843191, 1472011052, 1457359893, 1487483572, 1477024568, 1483750216, 1465304436, 1464036484, 1457214410, 1479560259, 1487413184, 1482888299, 1454354848, 1488262567, 1453165799, 1458507479, 1453425660, 1472665609, 1484199365, 1457853068, 1463998015, 1464789777, 1474660904, 1466454739, 1458308175, 1461698419, 1472520772, 1456614442, 1469559259, 1458297813, 1463063248, 1454479886, 1470060954, 1467329584, 1460121610, 1451948169, 1486548023, 1487107995, 1470598239, 1482335200, 1464036484, 1458165617, 1470576284, 1482523656, 1482245788, 1477224971, 1481494075, 1481821682, 1484995791, 1478111425, 1477757845, 1478586617, 1475268455, 1476680831, 1477279945, 1456308665, 1460389559, 1465363844, 1486469301, 1458546762, 1468787750, 1479793265, 1463127617, 1486182940, 1486978079, 1451762811, 1459307565, 1467629328, 1486365265, 1462699158, 1483471781, 1475438878, 1465384724, 1462855376, 1469364936, 1461917734, 1483276899, 1487869596, 1459417799, 1456408037, 1462767474, 1470152106, 1460087243, 1453460459, 1465886531, 1475590579, 1468251028, 1464443405, 1479370894, 1462531839, 1458521031, 1456794960, 1483213211, 1473884505, 1460636111, 1458434775, 1455154618, 1468645484, 1458333930, 1471398462, 1463687230, 1465659914, 1473571235, 1487450336, 1461356140, 1479104485, 1461128473, 1453290881, 1455786136, 1467889982, 1454097399, 1459890269, 1452334132, 1472493755, 1454360848, 1484754101, 1480717110, 1477131992, 1462016512, 1462173032, 1461176895, 1457863946, 1453246994, 1464969486, 1479999731, 1458477070, 1467279034, 1466258497, 1474819687, 1454529324, 1470245638, 1484777702, 1485248401, 1481440909, 1451954918, 1452438952, 1461896003, 1459250533, 1469463928, 1481622876, 1468464454, 1474695758, 1458383038, 1471087427, 1454594623, 1463434328, 1487201308, 1470066219, 1484736473, 1468928939, 1478307038, 1472902690, 1466639260, 1479514944, 1475846389, 1472743619, 1481305708, 1456508222, 1473276409, 1484691231, 1471117663, 1481727623, 1481224573, 1478068955, 1483421217, 1477626085, 1459197191, 1483976663, 1458570263, 1478611757, 1454808224, 1482556846, 1470717408, 1470530088, 1462893269, 1476034988, 1463642550, 1472405189, 1482695076, 1476254037, 1471669561, 1462369262, 1484327382, 1476997890, 1456581054, 1471635422, 1458061761, 1482867859, 1479098448, 1454357108, 1453084155, 1479284857, 1465138645, 1472152533, 1455543976, 1473565140, 1454009509, 1466093500, 1469071986, 1458991429, 1468284308, 1464668906, 1456752051, 1475369025, 1472250069, 1456786777, 1482177480, 1453398592, 1482770468, 1478068955, 1457545541, 1464182203, 1466541114, 1473220985, 1465696691, 1477336682, 1475938746, 1457798277, 1487455431, 1476384391, 1468229492, 1459576981, 1455082313, 1454213960, 1471513716, 1465477760, 1464704731, 1474395138, 1486496149, 1462316929, 1474459921, 1471102838, 1454414086, 1461206402, 1482959404, 1485201743, 1460145720, 1472952230, 1486387956, 1476845374, 1452968455, 1453064064, 1467629313, 1477469262, 1469482085, 1471329387, 1455601031, 1484949597, 1478677211, 1485306756, 1473346941, 1478997024, 1464186351, 1468053449, 1486826980, 1477788331, 1453677354, 1460464725, 1465080690, 1464444579, 1479605005, 1483561078, 1469317664, 1458828641, 1459374470, 1453822041, 1452804298, 1452539800, 1457863946, 1483251800, 1480633811, 1478997024, 1474518718, 1454153688, 1458406724, 1482556846, 1468885152, 1475938746, 1467713068, 1483112731, 1478212106, 1486682218, 1462627383, 1461806783, 1461277210, 1478848078, 1472029624, 1485291631, 1482402335, 1453482148, 1482286619, 1454650427, 1478179698, 1483478683, 1454270358, 1476804954, 1484370088, 1467725770, 1452341096, 1457502458, 1466053432, 1459745673, 1467780176, 1485847265, 1470062246, 1455163576, 1452175467, 1488195323, 1487181702, 1476364017, 1452781794, 1470641121, 1470817648, 1475324712, 1480894774, 1469463781, 1480201150, 1465128238, 1452076829, 1468797530, 1480129051, 1478202980, 1483976663, 1468598043, 1464502578, 1478126286, 1483152282, 1479905531, 1476507451, 1464239137, 1456588753, 1475412214, 1475979664, 1468883686, 1485789546, 1458314037, 1485525753, 1472807386, 1454371001, 1462413880, 1455822710, 1456510217, 1476241454, 1486688572, 1463458117, 1475106915, 1483164533, 1464862498, 1463610146, 1476224878, 1482608153, 1484329445, 1484332344, 1457270809, 1452492151, 1452327342, 1457504351, 1456704274, 1472469250, 1481732023, 1461250783, 1473516770, 1467680355, 1472454736, 1459287627, 1480947190, 1463172567, 1467909650, 1464297084, 1461410549, 1476193741, 1477629062, 1454344105, 1467238650, 1462526398, 1461314490, 1474587624, 1485088010, 1455697338, 1484116638, 1481025861, 1465686239, 1481190864, 1454973312, 1467039912, 1476212865, 1484216773, 1475400641, 1472712186, 1484378707, 1463822303, 1470030793, 1456976059, 1459974594, 1487146781, 1455208784, 1464768990, 1467473257, 1461085274, 1474806579, 1453605592, 1459656656, 1452356249, 1484893770, 1452383896, 1480666341, 1456233402, 1471261666, 1482980090, 1485166278, 1485306756, 1455473336, 1464321870, 1459955613, 1458290707, 1474859071, 1481965506, 1465435488, 1460121610, 1471805603, 1458033159, 1475222113, 1455695958, 1481732023, 1486746610, 1475488313, 1455142281, 1487819551, 1487751573, 1462943699, 1473809325, 1453047910, 1472832392, 1483036668, 1456274073, 1477178866, 1479238686, 1484977719, 1459215478, 1485220778, 1474840664, 1475740862, 1460612549, 1458415505, 1472675098, 1454547014, 1487499866, 1471765911, 1475433545, 1478235619, 1460718848, 1467658286, 1463202675, 1474606661, 1472712923, 1455962879, 1468423853, 1476682943, 1477969038, 1478625744, 1478419802, 1472651365, 1454565042, 1481425255, 1455635498, 1451881008, 1460389055, 1474028292, 1484297659, 1457813906, 1486347347, 1482687678, 1473719687, 1488295605, 1476116162, 1466215521, 1461692075, 1460087243, 1478251826, 1480660251, 1466683703, 1457899637, 1475807191, 1459056940, 1479076852, 1487520614, 1479567425, 1456189018, 1454684450, 1464455933, 1462222169, 1487709102, 1475797134, 1464125545, 1482011366, 1470493409, 1476254133, 1464163660, 1481057052, 1484431565, 1454410566, 1483478801, 1478906970, 1454375737, 1453441705, 1456836602, 1477620544, 1464341059, 1487539117, 1481681342, 1473189371, 1476204977, 1484531362, 1471917573, 1477745796, 1468230199, 1465297971, 1467244784, 1456997263, 1459776343, 1453922682, 1462550422, 1453657764, 1457818565, 1456928572, 1484811344, 1475157590, 1463577233, 1487581696, 1477732308, 1461268775, 1465309456, 1477305295, 1478396539, 1458165031, 1467209211, 1463245366, 1455954519, 1460578418, 1462784866, 1485717124, 1465097444, 1477450368, 1456846126, 1468906794, 1477995168, 1463037592, 1468391673, 1481233819, 1472011806, 1468306571, 1469363311, 1468186982, 1487696660, 1457944584, 1456752051, 1470171697, 1458198293, 1460051261, 1485505588, 1468228401, 1452214123, 1483791841, 1458478120, 1453465523, 1470844872, 1486307058, 1471527523, 1486616823, 1454912304, 1461369451, 1459932328, 1465082482, 1481665386, 1479888076, 1465444017, 1487201308, 1457940440, 1478513778, 1474512902, 1481567877, 1470089165, 1476056659, 1453486849, 1466487602, 1487382267, 1461478153, 1453745718, 1453534129, 1462847337, 1480212333, 1480343474, 1462855376, 1451739437, 1462639826, 1487318377, 1477429021, 1472264379, 1458666607, 1470316482, 1487708618, 1457386437, 1475322717, 1478107705, 1465246239, 1460764269, 1455395665, 1470312134, 1456269742, 1472387967, 1483792510, 1463649108, 1467270962, 1457965722, 1465080690, 1455142281, 1456062402, 1466989230, 1472966553, 1456744814, 1463600634, 1486732074, 1483690926, 1457386138, 1454917435, 1476531097, 1486810617, 1467904070, 1457832779, 1464039617, 1470408450, 1485770304, 1465590923, 1474036168, 1474290322, 1461616667, 1487812677, 1474481563, 1477186673, 1467917283, 1480660524, 1456853934, 1458815844, 1453008418, 1477036772, 1469955065, 1478586617, 1462122974, 1459341832, 1465146265, 1459820984, 1463631463, 1479539996, 1465876215, 1453798480, 1482751982, 1467466736, 1469580816, 1458965370, 1474638831, 1469950586, 1476466147, 1486343234, 1461674031, 1487096714, 1453679448, 1466445364, 1473861890, 1473466690, 1470074388, 1482367075, 1466593343, 1468003801, 1485351985, 1456147936, 1476189644, 1487326543, 1466555419, 1476343022, 1469390872, 1486478769, 1482822662, 1482843802, 1457386756, 1460389055, 1465800087, 1473189371, 1477212115, 1471872498, 1466259281, 1485220498, 1485555987, 1452741984, 1461456397, 1458030184, 1483313037, 1481535169, 1470012664, 1460436482, 1486939537, 1484821041, 1486093644, 1480395017, 1484718852, 1485879471, 1468772906, 1470390062, 1466084483, 1481788597, 1467007714, 1462715815, 1465056190, 1452069224, 1481176933, 1455928065, 1468699196, 1486697570, 1461552428, 1485959066, 1461462941, 1473055115, 1478833143, 1454791106, 1485601875, 1481224529, 1461366495, 1460300122, 1472155449, 1464479817, 1461515316, 1468827770, 1452848232, 1472787089, 1474721033, 1454947258, 1473231598, 1476253102, 1480703742, 1478036525, 1460881925, 1482539191, 1468046867, 1472858414, 1459888414, 1468278403, 1475124450, 1457313892, 1456649712, 1461008724, 1466093500, 1487502516, 1472522802, 1459763762, 1465277326, 1462276744, 1487199223, 1470250437, 1477136944, 1458025711, 1474015189, 1462041061, 1478945537, 1486347717, 1483209771, 1452575051, 1457458817, 1479064931, 1486245497, 1482590307, 1464213827, 1471921160, 1458009727, 1462589092, 1485342474, 1461971752, 1486630638, 1476466147, 1464446202, 1461531377, 1486724355, 1455650525, 1464776819, 1453874541, 1473030771, 1476432906, 1474827364, 1463135868, 1482784364, 1461215940, 1485351985, 1458030184, 1480155325, 1467463553, 1469541958, 1460542734, 1462520751, 1474111642, 1451816738, 1478508651, 1487915573, 1480398681, 1467180321, 1481949226, 1468036739, 1467380086, 1483833860, 1458670847, 1470439810, 1485501803, 1462793261, 1457608995, 1481624122, 1476247660, 1459451772, 1482523656, 1468433715, 1484483746, 1485026134, 1457308695, 1481483829, 1465447137, 1486171952, 1473111226, 1466000617, 1473767837, 1460037450, 1488107117, 1462847179, 1457037310, 1461180807, 1468229492, 1458770552, 1469558592, 1477969038, 1469231310, 1465384724, 1457975156, 1456625715, 1455026228, 1476986933, 1486673920, 1464372594, 1462769221, 1481681014, 1468144350, 1486269468, 1467670121, 1477159288, 1472878317, 1454189142, 1472863760, 1469075370, 1480030204, 1458792211, 1461485067, 1461754400, 1477346578, 1466571988, 1457800140, 1475704466, 1458775853, 1458187487, 1465590195, 1478358512, 1486383871, 1456121138, 1480679026, 1458525246, 1476955551, 1460586997, 1463485782, 1465647999, 1473489093, 1454622154, 1451763169, 1462227428, 1453427992, 1467931672, 1454918800, 1452576922, 1457386265, 1475196297, 1479366953, 1461674031, 1467765831, 1481187984, 1452648070, 1460730037, 1455016538, 1476200400, 1475152972, 1477500356, 1465086517, 1474082914, 1478084307, 1453082412, 1480465472, 1458287524, 1467365994, 1458405589, 1461686168, 1465221631, 1463937233, 1480877490, 1478248924, 1462989872, 1465479522, 1467749667, 1475307022, 1471737982, 1478554215, 1461515316, 1453147559, 1477130976, 1481535092, 1472454861, 1485592474, 1486804158, 1460115601, 1487976106, 1470403405, 1483926407, 1481067427, 1473182148, 1474226908, 1462554988, 1454014755, 1486274327, 1456297436, 1482724671, 1479225434, 1455163576, 1483077120, 1475079205, 1485643255, 1475456620, 1484237254, 1474881666, 1473307070, 1468863158, 1466796501, 1474696063, 1475304347, 1456827838, 1455922050, 1459742834, 1484307989, 1453236054, 1481707005, 1485498360, 1487696660, 1452564626, 1487033737, 1465160327, 1476231115, 1474459921, 1454690832, 1484563268, 1486283481, 1452413435, 1460522647, 1464213827, 1479708247, 1455011208, 1463559935, 1472651365, 1460843898, 1480574597, 1483787207, 1466467820, 1463082870, 1477567691, 1483601565, 1452552002, 1462540025, 1486917225, 1471265968, 1475555239, 1458929008, 1452756579, 1455026840, 1480129051, 1474503693, 1466616259, 1482699821, 1479176236, 1455063908, 1475429256, 1480203763, 1455240825, 1454444150, 1471185482, 1474366403, 1464452298, 1481469932, 1461571072, 1461195000, 1455148066, 1483538932, 1458862298, 1466954696, 1467257908, 1456581054, 1453779361, 1468038655, 1456455369, 1471271934, 1474523233, 1465909695, 1466680080, 1452438361, 1460729422, 1460016726, 1464285685, 1455026228, 1483307417, 1480963238, 1483147494, 1461091495, 1459334181, 1454337334, 1480045530, 1460714566, 1476625337, 1482523026, 1481723140, 1456672387, 1469286406, 1454344105, 1458026078, 1486496149, 1486573832, 1481681014, 1480823606, 1477938803, 1469035953, 1484638319, 1475942181, 1467252705, 1463108993, 1473251106, 1460796736, 1487588543, 1484662929, 1483143967, 1477788331, 1482567187, 1484005797, 1453610104, 1470089165, 1460028125, 1473754904, 1478847070, 1455543976, 1458991244, 1473017955, 1457129165, 1479176789, 1460667614, 1487852484, 1483147340, 1456143816, 1467355600, 1471390238, 1486827063, 1475616399, 1462818631, 1474881666, 1476101659, 1486974712, 1461741292, 1466948404, 1460200359, 1468740968, 1471732630, 1462184852, 1476538054, 1455882097, 1461800639, 1455400140, 1459896522, 1475425901, 1459930249, 1483374764, 1461365427, 1479816966, 1476210899, 1464003101, 1479358881, 1453455358, 1479125058, 1462492877, 1455856975, 1475174070, 1458484651, 1487286760, 1477629062, 1456093808, 1469236994, 1470641321, 1479238686, 1483786875, 1475639499, 1469625789, 1452287620, 1463998015, 1470819182, 1471837585, 1452451397, 1469417431, 1487976639, 1453423004, 1456505011, 1484865345, 1479465450, 1482238749, 1453145614, 1460728454, 1467842425, 1486206672, 1482567187, 1459742924, 1452537896, 1474859071, 1473970256, 1487455431, 1468568480, 1480275931, 1468954483, 1467817788, 1477790786, 1484005797, 1475286153, 1476172065, 1475137192, 1485950560, 1460316218, 1463169313, 1455632296, 1484110503, 1454941641, 1466728110, 1467817788, 1461515606, 1487147664, 1464320331, 1485074093, 1486072473, 1452595587, 1481729948, 1487662683, 1487088478, 1476785278, 1475367357, 1483934461, 1472011806, 1471066329, 1462013653, 1459836976, 1475835472, 1458591340, 1468447776, 1464218481, 1471029509, 1468590369, 1463273756, 1465451360, 1482607609, 1469626930, 1482172514, 1473144227, 1479957041, 1484821041, 1484162983, 1462784866, 1483682746, 1472696548, 1484283567, 1457022868, 1479776262, 1468939217, 1465647999, 1456673272, 1479160302, 1482398660, 1475732858, 1477598205, 1453594313, 1453561734, 1476353255, 1475196297, 1458061761, 1459143162, 1477776660, 1472117190, 1487806742, 1482863660, 1469476107, 1475872178, 1467995205, 1464886586, 1480468660, 1474762838, 1474791425, 1483381178, 1460881975, 1469643780, 1481658839, 1472634988, 1473088762, 1465902329, 1461837730, 1453299440, 1466826703, 1461132657, 1474730337, 1461250783, 1484241922, 1476797913, 1464014039, 1456123988, 1470904721, 1485080461, 1483121663, 1485750124, 1456601363, 1463127617, 1451950245, 1461735898, 1453964999, 1475842940, 1475558321, 1473839457, 1479769566, 1471572802, 1468433715, 1466604188, 1477563699, 1453900665, 1472493755, 1468954112, 1458172637, 1473680142, 1461473853, 1467508478, 1469537154, 1484940818, 1486510390, 1485132810, 1460496018, 1469670429, 1486341583, 1475075470, 1478286700, 1477626511, 1482988793, 1461580669, 1468734655, 1481896176, 1458463034, 1457384524, 1485722833, 1452125623, 1484497032, 1466242212, 1455124310, 1454229565, 1469711244, 1477502384, 1476463758, 1469694108, 1463896082, 1477341513, 1463600024, 1469329592, 1463642550, 1477732308, 1479292764, 1482353102, 1465941286, 1484233421, 1470984582, 1485667426, 1455428264, 1486456894, 1456189648, 1486023602, 1480705609, 1470062992, 1456057928, 1476372489, 1459001263, 1486083849, 1481631744, 1477917906, 1460436482, 1461580669, 1482784364, 1486586442, 1479497295, 1477119995, 1466103108, 1458425740, 1483399285, 1457867513, 1464570677, 1463851726, 1466454370, 1460542734, 1470564684, 1487746475, 1458008022, 1484322056, 1470767311, 1478611816, 1473681219, 1455861580, 1470152106, 1459068695, 1477341513, 1478702623, 1475314173, 1453621894, 1478272515, 1453955830, 1474810042, 1474290322, 1472232329, 1464945142, 1483976663, 1470312134, 1456421920, 1454636170, 1480971999, 1483367805, 1462441787, 1473657550, 1465472691, 1473017319, 1472961569, 1451727555, 1471108375, 1472489099, 1479533838, 1459176457, 1474264284, 1455019389, 1487597165, 1487201308, 1482679889, 1481019725, 1463994609, 1461192340, 1454866548, 1468765411, 1482037167, 1474376694, 1457478285, 1457316671, 1476446060, 1471064572, 1461571749, 1471886048, 1461536951, 1456271288, 1456713211, 1468448794, 1483915549, 1478688025, 1483152282, 1462044144, 1472802046, 1479130129, 1478615438, 1475222113, 1467601350, 1470552236, 1480984630, 1469369389, 1477587213, 1466258497, 1456402979, 1467507927, 1481459490, 1465230292, 1461628512, 1461969360, 1480780106, 1453466342, 1485076246, 1471416905, 1473884628, 1473766712, 1460052178, 1476792498, 1466904670, 1457793913, 1466310396, 1488279177, 1453553770, 1458487878, 1478312591, 1472659350, 1486133938, 1475129801, 1466407466, 1467382183, 1479341025, 1484848259, 1460316218, 1468479485, 1480093079, 1468110372, 1458678223, 1488009881, 1477604320, 1460384437, 1479434506, 1475124450, 1484681430, 1464586818, 1458149525, 1479332402, 1473577557, 1477972912, 1467328374, 1479078418, 1451783625, 1452677155, 1473117622, 1476388461, 1482863660, 1483685771, 1467880453, 1456500980, 1456074324, 1457294484, 1458052666, 1470974157, 1453331761, 1476966452, 1469662273, 1452888904, 1488135261, 1480612184, 1478666824, 1480486581, 1470664513, 1480076120, 1476972871, 1480952031, 1470655978, 1462492012, 1477708328, 1478031712, 1461917734, 1481103640, 1460538294, 1452244765, 1487737150, 1455148304, 1467723817, 1463651530, 1484436789, 1453900307, 1471592285, 1476190897, 1478719759, 1477244386, 1478355520, 1480131541, 1452858359, 1456882567, 1456997263, 1474411864, 1472426619, 1480820508, 1478092551, 1467655254, 1459850388, 1470855443, 1463998790, 1465936705, 1486343234, 1477905530, 1467632537, 1462130130, 1472012625, 1486570213, 1484546797, 1487733265, 1452616504, 1477519058, 1471393585, 1478165663, 1459160760, 1480506027, 1484012550, 1464391526, 1466206275, 1461025434, 1478056851, 1479671230, 1451786023, 1457157868, 1468586221, 1478734158, 1455066168, 1462173032, 1456084286, 1452458152, 1454046775, 1471108375, 1454781463, 1462258520, 1487553331, 1453876393, 1469873551, 1461990798, 1470262411, 1479203847, 1478166202, 1459184170, 1482744908, 1486584484, 1487480731, 1458015127, 1477260182, 1454609413, 1479244225, 1460999664, 1462314666, 1454053805, 1475174070, 1469727643, 1463471606, 1457612920, 1477936594, 1475377760, 1472029839, 1474973566, 1481220288, 1459264890, 1481186324, 1479098448, 1479393877, 1474596388, 1468996852, 1467245904, 1485370228, 1456437330, 1471504989, 1464939125, 1470315276, 1457652779, 1461674031, 1472554597, 1458898099, 1488017847, 1456305443, 1482430899, 1469950543, 1486347347, 1481665386, 1477778264, 1460200672, 1488195323, 1457446930, 1480071237, 1483203421, 1481483829, 1473954938, 1479284857, 1456508915, 1469655405, 1483895948, 1475129801, 1453752395, 1461274066, 1477010364, 1453332005, 1484691231, 1454343810, 1468911597, 1465076240, 1480395017, 1488315492, 1457505556, 1484666960, 1460642155, 1471427864, 1487502516, 1473081961, 1452707474, 1473417849, 1480454787, 1471554105, 1468229492, 1483295487, 1463851726, 1474090048, 1467280400, 1464301235, 1456551086, 1472742490, 1484436610, 1457785663, 1484380110, 1474245760, 1470994137, 1487965733, 1483391984, 1477500356, 1485080461, 1476087676, 1473064721, 1463711375, 1465002170, 1469075370, 1466700832, 1458844518, 1484867871, 1467517032, 1453771251, 1462876082, 1467907258, 1459024273, 1476406691, 1469047121, 1453513492, 1466197610, 1468299180, 1471231013, 1467812243, 1452692188, 1465444017, 1468110372, 1462677105, 1471555412, 1480203357, 1460545207, 1459386771, 1485132810, 1471195280, 1480945619, 1455922050, 1480289842, 1472781417, 1457747972, 1464336822, 1453912917, 1461958485, 1471818290, 1469075370, 1467397899, 1477656371, 1479629063, 1468808807, 1474014172, 1457370855, 1462085778, 1454118870, 1476642234, 1487502516, 1466503129, 1481727623, 1456075049, 1469044780, 1470799684, 1460297005, 1477918033, 1467787430, 1472177399, 1460613862, 1474632127, 1464485697, 1478091117, 1462668333, 1461002487, 1464723600, 1465987726, 1462276549, 1483211638, 1467237713, 1483289369, 1456358599, 1467337806, 1454577825, 1460409168, 1465786054, 1455009405, 1455240825, 1484890230, 1480657958, 1478613090, 1454287732, 1470512483, 1461641602, 1480039622, 1464444579, 1452394703, 1484297659, 1456084286, 1476237351, 1487035301, 1463501494, 1478047346, 1475389646, 1455422074, 1465742926, 1487940605, 1458992428, 1483542946, 1461335207, 1456742361, 1459249246, 1467747443, 1467165688, 1480294867, 1460937470, 1458185526, 1472010601, 1471062600, 1484325805, 1455208784, 1472774990, 1452175467, 1463018388, 1467657326, 1462910579, 1482988793, 1487755781, 1468994757, 1484273502, 1486676799, 1485665134, 1463262636, 1465616515, 1463512313, 1458664283, 1471120354, 1483341900, 1480713090, 1479070630, 1474867915, 1468055758, 1459100651, 1472604281, 1475616618, 1458894412, 1480840485, 1461887466, 1472411160, 1474460213, 1452727180, 1454843655, 1477211588, 1477397664, 1487670782, 1462401003, 1466562365, 1465093811, 1463900565, 1464791108, 1475438545, 1464791108, 1459676512, 1458053458, 1458196268, 1456976778, 1477374547, 1478387923, 1456034247, 1467494356, 1455191444, 1451947798, 1465086932, 1459408481, 1477841138, 1469342830, 1461641602, 1463825009, 1472980006, 1460204758, 1474938968, 1475304347, 1461223310, 1477446575, 1482037167, 1462928949, 1482825673, 1485377691, 1478822883, 1469040783, 1455989104, 1472761952, 1480612184, 1466181736, 1477818115, 1453631592, 1487604020, 1459419493, 1462011086, 1481944496, 1468653925, 1475776677, 1478010438, 1485361372, 1464110344, 1452968455, 1461176895, 1454482936, 1483085803, 1457786518, 1488294907, 1459069264, 1458624241, 1482111604, 1460140610, 1487922432, 1487728355, 1483233191, 1476063173, 1485967422, 1480733421, 1455142281, 1479086933, 1461401870, 1463559935, 1474697295, 1487694175, 1466933721, 1459507985, 1465308903, 1458631016, 1452648070, 1454194737, 1468796505, 1480453831, 1461938679, 1477404899, 1460349200, 1463757050, 1480135468, 1471480174, 1458183851, 1451656320, 1487818374, 1478101007, 1486630350, 1481332470, 1476933990, 1460698066, 1465080690, 1458549363, 1452604122, 1487543037, 1487659583, 1482627083, 1464232623, 1453025083, 1462111958, 1456952139, 1469810113, 1460698353, 1461145974, 1480271665, 1472651365, 1472208688, 1471425942, 1468644483, 1458826916, 1455665402, 1452917783, 1468460809, 1460138611, 1456846084, 1483644426, 1463471190, 1464150151, 1471153964, 1472519446, 1454694213, 1471018794, 1467104105, 1482428479, 1460520460, 1487365740, 1484781711, 1467829609, 1470247804, 1478265450, 1483286160, 1453103934, 1465795825, 1453629550, 1457359893, 1475937481, 1463564700, 1453427992, 1474420426, 1475589258, 1456794960, 1487681453, 1465849725, 1457352186, 1456366084, 1468719495, 1467473257, 1476711830, 1456168944, 1468136765, 1472549963, 1454193444, 1455475665, 1466647226, 1476677847, 1481176933, 1453860780, 1454930090, 1465936705, 1455053009, 1480462336, 1481233819, 1485962144, 1458339046, 1460196933, 1465273758, 1451687279, 1463214331, 1465146307, 1454518723, 1453479488, 1484555386, 1470556661, 1473169342, 1459653585, 1452282201, 1451714043, 1478945537, 1454193444, 1459894383, 1461698419, 1459449075, 1464604014, 1486660994, 1481317148, 1453679448, 1470347962, 1457013670, 1470378213, 1464150149, 1476022204, 1464606992, 1457386437, 1460353341, 1470679262, 1477928033, 1479427490, 1472018269, 1460187328, 1484599672, 1476802829, 1483835663, 1476005244, 1485831844, 1462170302, 1478295247, 1475204260, 1483276899, 1467143976, 1485377691, 1477024568, 1452755328, 1475949489, 1488146738, 1476364709, 1463165993, 1455819862, 1467115887, 1486121709, 1475125319, 1473425727, 1463977294, 1480981817, 1456414509, 1482265166, 1459657620, 1479287795, 1472155074, 1452140012, 1483750216, 1463963800, 1461862978, 1469122248, 1482418183, 1461990798, 1468883686, 1456196338, 1460928483, 1455168260, 1464753577, 1455165246, 1483221382, 1469794798, 1466234067, 1464397334, 1453977919, 1457569913, 1472549228, 1487670782, 1468974940, 1474273337, 1470236522, 1486455275, 1470763368, 1454140185, 1467151751, 1456932019, 1455376717, 1454272886, 1463269853, 1466332912, 1472591183, 1482982241, 1477189363, 1457753862, 1482083608, 1457842306, 1466335431, 1477189363, 1474528207, 1451714043, 1484991777, 1467713068, 1478203714, 1466954989, 1458460270, 1452010591, 1451763638, 1453390947, 1456729774, 1473183959, 1481661548, 1485398218, 1469156433, 1471846695, 1484221767, 1473153365, 1486598989, 1467463553, 1456185892, 1462041061, 1470626814, 1453420059, 1485124072, 1469051591, 1483182264, 1465339603, 1471913897, 1473941822, 1472216669, 1460543490, 1482331073, 1482959404, 1480502534, 1477682484, 1478279161, 1482562775, 1454059035, 1479690036, 1479401954, 1475062797, 1475937481, 1479149677, 1474132289, 1487271551, 1475580754, 1467829609, 1452909057, 1477374676, 1455601472, 1455568641, 1477938803, 1467981874, 1468097172, 1488315513, 1484362925, 1455560659, 1458065987, 1485128824, 1477684899, 1465856667, 1476917684, 1475174877, 1452865700, 1453103020, 1482072074, 1478463401, 1484792554, 1472062498, 1462156032, 1481063214, 1472426619, 1481932993, 1481250242, 1473456449, 1451697649, 1460995145, 1480675497, 1478107705, 1458543585, 1454557491, 1480337951, 1470679262, 1476517885, 1486307058, 1474800592, 1456669739, 1469427392, 1456179693, 1467480272, 1480982472, 1465637699, 1467346110, 1460324653, 1455676185, 1457151062, 1457359735, 1481287810, 1465447137, 1457412365, 1479969451, 1472155074, 1461465251, 1455393975, 1472426619, 1482285190, 1465626316, 1456231595, 1482749229, 1483138205, 1455882097, 1460019039, 1479203847, 1473020118, 1480612184, 1462727976, 1456397849, 1458239598, 1462930618, 1456835097, 1454142069, 1467211156, 1471567235, 1459001263, 1464431852, 1484263075, 1466968473, 1453332112, 1451623350, 1465835684, 1481139265, 1464684937, 1458235126, 1464386012, 1468049263, 1481730250, 1469325649, 1475507466, 1456290271, 1459756566, 1485348917, 1468315293, 1462306541, 1480670841, 1461386614, 1467093514, 1460431834, 1464263098, 1483965834, 1455148304, 1452219471, 1476463758, 1486598989, 1486697368, 1458434775, 1487599699, 1458660491, 1454232840, 1465146307, 1470133803, 1486476520, 1466889394, 1464687961, 1477604320, 1481707161, 1478190567, 1487326543, 1465572790, 1477348152, 1485440375, 1463051693, 1470049483, 1464179294, 1468423999, 1454083118, 1458537387, 1466076729, 1481489311, 1454253916, 1485492231, 1468765411, 1467981874, 1476568422, 1468311941, 1467674568, 1457167098, 1461756187, 1459163721, 1473634263, 1478134375, 1483624824, 1469548175, 1457405839, 1471496234, 1483951242, 1475356985, 1484790228, 1461314490, 1454253293, 1484026837, 1484169969, 1478561880, 1464169904, 1460441625, 1473159829, 1462667403, 1478831086, 1463721299, 1458816405, 1485688176, 1454815321, 1470009401, 1477111612, 1488183475, 1483662118, 1476044709, 1483586745, 1483150748, 1478718238, 1456271288, 1473745314, 1481278603, 1465221086, 1452837255, 1478473945, 1471795249, 1451625058, 1455026840, 1467807964, 1478179698, 1483028967, 1460253323, 1483171159, 1463690303, 1486231167, 1472523561, 1453273281, 1484181511, 1460362291, 1463055446, 1456787433, 1487658331, 1451986832, 1465420537, 1465936705, 1461770562, 1470977070, 1473902251, 1486307058, 1463970024, 1455432070, 1470537888, 1477964708, 1460228903, 1457347854, 1454229565, 1463099440, 1468951769, 1480985415, 1471512419, 1460612549, 1453794435, 1456551086, 1462111884, 1487079762, 1480064490, 1461397871, 1488031693, 1455061213, 1463051693, 1458904883, 1469021303, 1483381178, 1483015754, 1456616765, 1484283567, 1483948730, 1463420788, 1476714513, 1458510507, 1461486147, 1468278403, 1466473272, 1478091117, 1463036487, 1460152703, 1482240083, 1471628037, 1470009634, 1472542451, 1465479522, 1460087243, 1454131152, 1464296678, 1474998403, 1469487289, 1473841236, 1455632296, 1462993430, 1464833181, 1455531621, 1453701652, 1463805405, 1472831721, 1475059924, 1478516250, 1473322569, 1460130671, 1470062246, 1477079136, 1476664006, 1467847757, 1468060836, 1479366260, 1479937899, 1452618393, 1486427867, 1487849816, 1472591222, 1477664883, 1487605614, 1459363878, 1479868080, 1487114692, 1452567830, 1485309068, 1463051693, 1484662929, 1453507345, 1465999176, 1486410287, 1481643167, 1457075383, 1459714071, 1473237412, 1461318124, 1462025349, 1478519794, 1462696179, 1474898079, 1476453778, 1484101503, 1479022012, 1462489035, 1464211384, 1468460513, 1465764191, 1451614183, 1483791040, 1471402615, 1455237667, 1479616945, 1459307565, 1461843623, 1459616539, 1482210470, 1470423797, 1463553054, 1476572881, 1479245744, 1463722533, 1458019989, 1479515537, 1485951456, 1470785443, 1455116091, 1456269742, 1485124025, 1455765648, 1485951456, 1467438290, 1474710237, 1452142949, 1456672387, 1479939874, 1456264069, 1480618639, 1455575812, 1459862094, 1463654391, 1480169582, 1470025692, 1481025861, 1472307251, 1466358268, 1487734413, 1453212741, 1472277075, 1451813728, 1460253323, 1454781463, 1486733141, 1464679990, 1472584971, 1463098826, 1474984866, 1483827184, 1471398462, 1454007716, 1459178076, 1456486196, 1455063908, 1487884387, 1454082444, 1464203455, 1460948113, 1467657922, 1488315513, 1480489668, 1464570677, 1461837730, 1474528207, 1486561877, 1473831707, 1454016298, 1466850127, 1462693370, 1483965834, 1483472562, 1476829288, 1482188645, 1457131177, 1481765992, 1487871526, 1477404899, 1478092129, 1453507345, 1478430725, 1476396710, 1486166613, 1478403373, 1462617888, 1484472696, 1481312848, 1471554105, 1474944730, 1469576718, 1460002480, 1463172567, 1486494234, 1480615067, 1488118281, 1459826717, 1479244225, 1470693697, 1479150644, 1467880453, 1459145455, 1472117190, 1459213833, 1487100226, 1474029032, 1470928754, 1481068517, 1460650230, 1469122248, 1470219653, 1462128933, 1487940164, 1485095680, 1483648659, 1487581696, 1485189172, 1454708827, 1458295554, 1474697295, 1462338390, 1483147340, 1482997531, 1477733736, 1466638115, 1487694175, 1469510173, 1465729649, 1485162481, 1463363680, 1456599986, 1459124234, 1451646892, 1457037310, 1457569913, 1457866259, 1477478427, 1454529324, 1478992492, 1476234267, 1477957353, 1476405687, 1485220778, 1483990167, 1466499737, 1470238946, 1453517484, 1465967414, 1478357746, 1480131541, 1477035880, 1469363311, 1476574272, 1464888282, 1486896886, 1467605644, 1484104147, 1482226048, 1452343556, 1486586867, 1461124077, 1478941079, 1478368608, 1480618639, 1460698066, 1459419476, 1464837393, 1485838837, 1456610511, 1455376717, 1486602269, 1477918033, 1480421322, 1465695904, 1482404482, 1456509414, 1472742490, 1482206548, 1485425870, 1456288436, 1485324800, 1468360026, 1463303122, 1463625173, 1458611875, 1480455767, 1457208464, 1461063265, 1460636111, 1451657744, 1456437330, 1486688572, 1467723817, 1453298606, 1487740810, 1473269183, 1468890786, 1466065296, 1479078418, 1474014172, 1474173098, 1470529849, 1459930118, 1479076145, 1473526705, 1467536479, 1463098826, 1472963864, 1482056186, 1484657062, 1473681219, 1478327486, 1460267142, 1483295223, 1482306040, 1463512313, 1487427072, 1465936332, 1470564684, 1485717540, 1473861890, 1477913894, 1466093500, 1469165897, 1451684342, 1467766007, 1487828972, 1463895517, 1487974102, 1486885865, 1470642767, 1462479999, 1474924757, 1451972489, 1457377288, 1472405189, 1457863269, 1467298131, 1484821724, 1481790426, 1468105565, 1473568251, 1463115990, 1465248810, 1458728175, 1453899785, 1468156206, 1461872958, 1467573961, 1485310854, 1461056062, 1472612462, 1455436764, 1464744879, 1466774039, 1462713368, 1463519243, 1481161428, 1471016537, 1464413990, 1476128457, 1478583939, 1455634429, 1474489432, 1466718775, 1486425650, 1459143162, 1474638831, 1476297236, 1461079346, 1471410637, 1477781532, 1454808224, 1485643310, 1465225661, 1453918507, 1453297000, 1461877639, 1477437329, 1457760321, 1471319174, 1466445364, 1479515537, 1456228646, 1464540588, 1456093211, 1470270544, 1451897630, 1474528264, 1468105565, 1464593209, 1485818908, 1467608845, 1472701619, 1478147432, 1473096725, 1469903839, 1458510507, 1483042546, 1485667155, 1471375318, 1479990040, 1467236730, 1479143874, 1468168585, 1466730208, 1467692082, 1459535669, 1476153002, 1466666337, 1468200310, 1462070113, 1470508096, 1466499737, 1477424812, 1483999708, 1485717124, 1475111063, 1466451361, 1470493409, 1474758670, 1478586028, 1479605005, 1464377667, 1464400677, 1479012195, 1456440893, 1472067351, 1475207693, 1454591444, 1472767056, 1465221631, 1482439322, 1460902548, 1482043202, 1469074102, 1473179288, 1482530694, 1462595594, 1454710290, 1454808224, 1474596388, 1485425870, 1470566260, 1453305248, 1474347203, 1470275318, 1451837092, 1459533105, 1472253643, 1455453508, 1465842707, 1471398462, 1472111541, 1465985955, 1481652222, 1454751380, 1480688398, 1461504780, 1454179603, 1478092957, 1465436073, 1486133865, 1454603259, 1476855810, 1459775304, 1470868847, 1484074633, 1474659550, 1470353331, 1452041765, 1462699158, 1468544171, 1474528264, 1477208853, 1487966528, 1485201743, 1454275613, 1471886048, 1461750749, 1463755986, 1487499866, 1453353213, 1468081706, 1454617449, 1481010748, 1473892509, 1474460213, 1465969167, 1467800837, 1468416721, 1454966454, 1457455848, 1474018734, 1484329887, 1475708832, 1475761391, 1472089377, 1456287900, 1479867547, 1487588543, 1473182148, 1453727410, 1460151695, 1468818428, 1471818290, 1458295437, 1468402935, 1456887144, 1464833181, 1476799235, 1472322032, 1470645188, 1478356003, 1456223773, 1465384724, 1462255132, 1477562953, 1465850078, 1488146854, 1467962861, 1482639854, 1462446261, 1464094076, 1461831815, 1460282414, 1453895459, 1462955656, 1456356935, 1452665946, 1487912948, 1464208480, 1484867840, 1461910420, 1463939659, 1483402427, 1474989384, 1475054731, 1470111782, 1477299557, 1466622705, 1462617888, 1452965865, 1470421977, 1475865470, 1475444036, 1486231167, 1468151560, 1456884796, 1471782750, 1478357227, 1463354901, 1467483480, 1466948404, 1478752055, 1484595730, 1466221684, 1455665402, 1475774471, 1461786443, 1472389759, 1483629383, 1484245117, 1463971628, 1461117910, 1485119692, 1478321187, 1457424725, 1461016480, 1472034401, 1461581220, 1454024678, 1480212333, 1457838188, 1473454468, 1479849263, 1485355964, 1456408037, 1479543640, 1468036345, 1467044098, 1480195103, 1456835097, 1469858062, 1477220516, 1482819012, 1485109294, 1452939253, 1482295243, 1467232120, 1476518634, 1463278510, 1470908639, 1467883843, 1460842041, 1482285190, 1462896748, 1486953369, 1481546863, 1467570445, 1485667426, 1463771383, 1487230867, 1454356942, 1459287627, 1478839517, 1477404899, 1471474677, 1477341785, 1457957803, 1480025182, 1487560166, 1454376407, 1486341583, 1461896003, 1484859946, 1483090011, 1463377894, 1486864833, 1483159616, 1482409439, 1466629490, 1466384720, 1477629062, 1473766712, 1471640949, 1470326622, 1452677206, 1463404971, 1482676335, 1463339472, 1475177777, 1479863578, 1478166202, 1460599904, 1479995668, 1465579185, 1487003549, 1459222319, 1479730465, 1477844230, 1487969349, 1463858754, 1469948456, 1481980143, 1458159718, 1464485697, 1460440296, 1484253457, 1458277134, 1470012664, 1455322560, 1473081961, 1487343334, 1464965095, 1481911514, 1454889017, 1476567111, 1476433083, 1465273758, 1462473378, 1451910908, 1452142954, 1486583266, 1469035953, 1467006274, 1475459511, 1486752990, 1475050191, 1459417180, 1456842380, 1459309912, 1478884407, 1476277247, 1453912891, 1477260710, 1485803845, 1463672592, 1476081337, 1462332975, 1451710949, 1470319346, 1461516477, 1478440209, 1466221684, 1465256763, 1473604704, 1463148023, 1476041800, 1454751380, 1478019639, 1461203057, 1460605060, 1462937136, 1462441787, 1459207091, 1475276820, 1486226049, 1466886648, 1488214194, 1466153017, 1485943843, 1457607792, 1470347623, 1455446227, 1475459910, 1477337091, 1486979981, 1464732316, 1454791437, 1457065528, 1487863655, 1475820702, 1456876285, 1456472516, 1471972850, 1478275253, 1468423853, 1482188645, 1472250069, 1486476520, 1487022034, 1486206672, 1465834382, 1455171344, 1479986056, 1461962804, 1459825417, 1453482876, 1485204516, 1473454468, 1459419493, 1467283468, 1479412764, 1482040247, 1476357211, 1486324101, 1476122333, 1471261666, 1453689175, 1466639260, 1454831361, 1481595789, 1475416352, 1476017034, 1458558092, 1474589939, 1471087427, 1476932646, 1474489432, 1470056208, 1475639499, 1473753969, 1463440328, 1476935974, 1475312218, 1470921242, 1480345416, 1463923947, 1471265968, 1472472376, 1475004112, 1455241006, 1468268706, 1454234791, 1479161467, 1481862316, 1456271288, 1475877035, 1460940956, 1476779810, 1460305469, 1457105355, 1455948503, 1457946056, 1486469301, 1459876146, 1474017427, 1453671943, 1454500577, 1467943151, 1472400376, 1486302472, 1475337637, 1483214021, 1474195043, 1463553054, 1466826213, 1460520460, 1480454787, 1473249521, 1477708328, 1451798070, 1482917547, 1471247811, 1486083849, 1454373491, 1483692105, 1472512901, 1479104192, 1456496268, 1487476702, 1452799626, 1480665171, 1473050078, 1483033720, 1487940605, 1454343810, 1476153002, 1472225333, 1484792554, 1478385162, 1471021662, 1483354292, 1469327045, 1458022715, 1468574675, 1466353797, 1463885590, 1460121610, 1483015754, 1464443405, 1466690116, 1487736761, 1463268521, 1486050359, 1483251800, 1456101368, 1453774470, 1479595970, 1458785663, 1472264379, 1475568365, 1461198218, 1454450413, 1479076852, 1470785443, 1457634862, 1468916983, 1483457246, 1462049436, 1458460270, 1485717540, 1472780692, 1470566519, 1465097105, 1454118870, 1461104848, 1487083690, 1451948169, 1452687642, 1460590908, 1465649384, 1471193383, 1464038622, 1470831718, 1487350889, 1455712092, 1477220516, 1470869224, 1467931486, 1455586074, 1482699821, 1487685234, 1477149279, 1467536119, 1481722944, 1456148146, 1482982241, 1482780596, 1485220498, 1484009720, 1456286971, 1455023074, 1480594653, 1483601817, 1460726392, 1475973112, 1454318265, 1457712502, 1479961203, 1461960789, 1473586125, 1479133517, 1475073509, 1452167835, 1472307251, 1470868989, 1483602117, 1483944984, 1481899694, 1472341708, 1464888282, 1471856621, 1477733893, 1455878574, 1473030221, 1470645188, 1476845374, 1474988425, 1481765992, 1479273324, 1482270067, 1477776120, 1453218526, 1464678900, 1479062203, 1457813906, 1454614969, 1452000769, 1473584003, 1465341162, 1482876227, 1480494802, 1460026288, 1456655197, 1453266315, 1480160797, 1462184852, 1465866755, 1480502534, 1485747626, 1473301889, 1478584127, 1477954559, 1475695941, 1456008997, 1484503600, 1478611816, 1475207693, 1463998042, 1481215943, 1485356689, 1474116999, 1452794520, 1483811839, 1466094387, 1470572004, 1471548962, 1469051591, 1478718238, 1456379135, 1475706825, 1478382978, 1463980128, 1454084975, 1486033074, 1487226052, 1485370971, 1455453508, 1454198086, 1483876875, 1456296943, 1466279075, 1456310033, 1486974712, 1466106340, 1468010718, 1477524999, 1488153831, 1479616945, 1467383462, 1483401708, 1453042488, 1472529656, 1453876393, 1458631016, 1478036525, 1480238513, 1483346964, 1486714056, 1463498939, 1481258143, 1471066329, 1484288549, 1469327045, 1480605385, 1459121386, 1465288957, 1470673729, 1458119550, 1473446786, 1472865842, 1453455358, 1461323545, 1467972111, 1455184640, 1456884796, 1458124061, 1470663762, 1458518659, 1475438545, 1468200310, 1456450030, 1465640673, 1454153576, 1453845989, 1459125454, 1452076829, 1477790786, 1463636151, 1456713211, 1458941797, 1487945362, 1474840664, 1469023605, 1485798801, 1482062953, 1467624688, 1486338503, 1459100651, 1470626814, 1454811064, 1475405514, 1462489035, 1482295243, 1488294907, 1460132211, 1482825673, 1479528715, 1456099742, 1462052125, 1469362970, 1463968490, 1457972463, 1465921496, 1462621846, 1470190398, 1468645484, 1480835854, 1471393585, 1457747972, 1474144440, 1452718442, 1476664006, 1484935659, 1486267631, 1454283589, 1482227442, 1461604252, 1474635681, 1452356280, 1476550341, 1469873551, 1460892993, 1481595789, 1473249521, 1488310882, 1460085681, 1457359893, 1487580268, 1452665946, 1475266522, 1462507021, 1468284308, 1478036525, 1476343777, 1468766751, 1487476702, 1461649645, 1484454672, 1486671383, 1466984819, 1466489747, 1487073215, 1452794520, 1473069986, 1473807549, 1482276785, 1475197880, 1455388310, 1463884577, 1485661134, 1462779102, 1481980143, 1473095166, 1455898140, 1478591291, 1470265753, 1480850955, 1478368608, 1474284947, 1468238016, 1465657265, 1453908840, 1475174877, 1478339900, 1459775304, 1478138994, 1474300962, 1459950627, 1460046738, 1456252402, 1456769911, 1460743261, 1470572004, 1474172102, 1455289338, 1468539516, 1478493539, 1465156280, 1452387967, 1455638582, 1479225434, 1455612649, 1465999176, 1481556993, 1453432440, 1458022715, 1456003191, 1484272794, 1463583325, 1473088762, 1486083849, 1485651368, 1484893770, 1464230258, 1485951456, 1482173925, 1476481240, 1459779761, 1463712632, 1479561722, 1465897402, 1475711956, 1478665076, 1465268587, 1457985697, 1463264315, 1454632043, 1487171107, 1478102796, 1451803874, 1482532065, 1487090468, 1467088075, 1472012412, 1485166278, 1477478427, 1486733141, 1470441262, 1486826980, 1460021205, 1477911144, 1470660796, 1475111063, 1483606251, 1474596388, 1452010591, 1469980972, 1475397103, 1464687961, 1468848163, 1458198293, 1474708238, 1464743139, 1462661383, 1457931355, 1487588543, 1479425416, 1454142069, 1476563436, 1481279740, 1473241391, 1479250073, 1484205548, 1473719687, 1484129214, 1481900836, 1452741984, 1485705477, 1465886531, 1480343474, 1460995142, 1453180858, 1477740219, 1466968473, 1482380631, 1463100685, 1485070684, 1478177971, 1464072950, 1460037450, 1478047346, 1470238946, 1482056186, 1457798277, 1478177971, 1453053218, 1462783774, 1454686405, 1466031045, 1460081347, 1466636956, 1470275318, 1483397521, 1451871549, 1481068517, 1474421746, 1453524851, 1458246166, 1465308903, 1466610816, 1470958687, 1452747674, 1484059482, 1469789894, 1481059935, 1486653446, 1460412697, 1468641501, 1478834027, 1469757601, 1479728610, 1459405354, 1480628716, 1464057941, 1457753862, 1457131177, 1487745738, 1457386437, 1487500062, 1453931943, 1481224529, 1475538902, 1478791482, 1455685882, 1465225661, 1475287032, 1484343211, 1477335944, 1471340943, 1467618895, 1456556262, 1457506357, 1478373009, 1484269234, 1456486196, 1462439363, 1469657123, 1463642311, 1452196091, 1470296530, 1458847215, 1472135635, 1464868998, 1455378963, 1486889022, 1488213267, 1479266689, 1487958755, 1463431525, 1477036772, 1483588627, 1452560864, 1475164926, 1477120469, 1466513720, 1458398931, 1473446786, 1480041139, 1476434768, 1467146097, 1467888938, 1456180387, 1467049292, 1465946023, 1478265321, 1468905991, 1461719422, 1485967422, 1478196685, 1486257097, 1461905268, 1466279075, 1481932993, 1462855376, 1462262940, 1461129070, 1485688156, 1468734655, 1462003627, 1471135271, 1454704001, 1452618393, 1486365265, 1480917354, 1468690977, 1475688775, 1475977995, 1482246824, 1471837585, 1478321187, 1480373441, 1485428471, 1458333930, 1471753149, 1453496513, 1451937070, 1452180614, 1462497767, 1480066595, 1473506952, 1463625875, 1452766808, 1485717623, 1482007804, 1460872119, 1476677847, 1479388024, 1473417849, 1480123338, 1487819551, 1481122893, 1485342474, 1473241025, 1483352455, 1483685771, 1474758670, 1471471745, 1461820557, 1472882290, 1459377896, 1485705477, 1462492602, 1475978739, 1453299731, 1483815357, 1454029730, 1456561415, 1468911597, 1452692188, 1487553331, 1467395133, 1471750379, 1453944332, 1472299272, 1463276347, 1464150149, 1470026319, 1476994936, 1468568676, 1478038202, 1486134184, 1464751169, 1483792510, 1474762838, 1459025780, 1464476496, 1485310854, 1451921545, 1475282117, 1480688634, 1462806860, 1473344064, 1470919516, 1471725152, 1480110594, 1453246994, 1454927400, 1456284096, 1458683935, 1457697219, 1475449625, 1467337806, 1456520059, 1459125454, 1484517730, 1466698089, 1451754280, 1452462825, 1475764983, 1458154904, 1468119498, 1481091632, 1474111642, 1461763661, 1471253184, 1455609672, 1475263897, 1470819182, 1482830467, 1485476940, 1458478120, 1473062706, 1485128824, 1470693697, 1465696691, 1478753151, 1465093811, 1456856988, 1481696067, 1459861690, 1466193825, 1473807542, 1474728050, 1457608995, 1461214190, 1479253117, 1466034228, 1465233237, 1452379508, 1481332470, 1468943966, 1456147065, 1452260276, 1487062787, 1464560100, 1474464531, 1482010826, 1469408240, 1474720021, 1453486849, 1463377894, 1472482469, 1479266689, 1462664917, 1481531001, 1466928119, 1473952196, 1455047051, 1479816966, 1471656259, 1476184586, 1464744879, 1458824831, 1473526705, 1480398681, 1453758649, 1466683703, 1456612839, 1461314490, 1482749336, 1485567817, 1479238018, 1462439363, 1475873169, 1459374470, 1456558254, 1481900836, 1487966528, 1458601233, 1465621384, 1455226510, 1459714935, 1451905921, 1452287620, 1481341660, 1488193847, 1459270081, 1473114069, 1454097399, 1453420059, 1457698520, 1476318595, 1469714839, 1463616646, 1478591291, 1469275995, 1484689768, 1471021662, 1460592484, 1479013828, 1471554105, 1477790786, 1483976968, 1469351841, 1477153134, 1483068021, 1461276740, 1467269005, 1473131600, 1485228569, 1461862978, 1478182220, 1488183475, 1479125058, 1482011366, 1480373441, 1468863158, 1474523662, 1455077902, 1459679738, 1469984605, 1475854235, 1475612155, 1480594653, 1458518143, 1473165719, 1459878681, 1486535133, 1481302721, 1487751382, 1474848672, 1483978014, 1464308596, 1463788343, 1486681276, 1458664013, 1488157698, 1463988643, 1462661383, 1460437660, 1451730738, 1482753756, 1480455767, 1482864358, 1463485782, 1477975734, 1454864257, 1478098108, 1483750216, 1453771003, 1479735269, 1467871746, 1473500264, 1463363680, 1465377155, 1477818115, 1451611951, 1480004235, 1473121822, 1465713640, 1462409606, 1463354901, 1460187328, 1461091814, 1462118289, 1482935594, 1460380080, 1458543585, 1475262855, 1479769566, 1456928572, 1470333172, 1451735657, 1472244357, 1470704803, 1467245904, 1459093491, 1477745796, 1472469250, 1465856667, 1480713090, 1464957168, 1455769501, 1454505804, 1457937533, 1459856402, 1456936901, 1457634862, 1462508946, 1482284385, 1473892509, 1475691034, 1460193322, 1486816001, 1468435324, 1471253184, 1475616618, 1487759844, 1469614134, 1452010149, 1485227780, 1472219496, 1478477418, 1468194198, 1463651945, 1484648418, 1472386394, 1479844593, 1457843191, 1481336532, 1470664766, 1452142954, 1484920508, 1483085803, 1487502960, 1488070574, 1458015980, 1453559946, 1488026426, 1455193502, 1466050266, 1454116165, 1461727509, 1474614756, 1476755813, 1456131995, 1480098400, 1476587473, 1485476940, 1453661080, 1455700246, 1477587213, 1467346110, 1487431577, 1463363680, 1481241860, 1482590307, 1483048387, 1462429937, 1470167024, 1477803771, 1486700437, 1467960464, 1455034816, 1460463590, 1452412119, 1456124267, 1453880837, 1467061800, 1461756187, 1455432070, 1470993810, 1481407912, 1481320989, 1472349289, 1454569239, 1482572220, 1485643255, 1463751606, 1478814086, 1471812860, 1485770304, 1469788511, 1458081844, 1479233299, 1486819922, 1478312591, 1487146510, 1484724222, 1466889394, 1452219471, 1465355977, 1461091814, 1479176697, 1479682670, 1469388993, 1473399767, 1477116377, 1453461809, 1473815937, 1470820549, 1457313615, 1484624920, 1479492587, 1476580612, 1461709646, 1471828669, 1466791321, 1484503600, 1483923040, 1469643780, 1453689175, 1470855443, 1464570677, 1487681419, 1467546747, 1453771251, 1461719422, 1485430751, 1467232120, 1484801445, 1487090468, 1471241304, 1479171682, 1464321870, 1488162845, 1480705117, 1460655865, 1463278510, 1484368300, 1451947798, 1479528715, 1484948272, 1476187058, 1488213611, 1474173392, 1473942706, 1458463034, 1470230851, 1458532652, 1478422992, 1458042302, 1468513288, 1483601565, 1486573832, 1476258862, 1457732124, 1468574675, 1461023788, 1461323545, 1452633689, 1483827184, 1467759656, 1453745718, 1476649684, 1457518045, 1468046867, 1461859735, 1487731859, 1469580816, 1464406959, 1464014483, 1461228085, 1453260668, 1457037310, 1476671498, 1469318659, 1458314037, 1473766683, 1470111782, 1464288667, 1464575049, 1454965703, 1470869224, 1471944919, 1487728355, 1457228199, 1471064572, 1462184852, 1468996470, 1458333930, 1473656406, 1476056659, 1477781532, 1475597077, 1461984565, 1465451360, 1483681601, 1457849154, 1462659007, 1456238833, 1478358243, 1453251516, 1481900836, 1472466662, 1478493217, 1458901000, 1476405687, 1454866548, 1456807495, 1455158174, 1470566519, 1455395665, 1453186277, 1480914258, 1486623148, 1483644426, 1456089156, 1462973305, 1470403405, 1455801023, 1468101724, 1478102796, 1472306731, 1473556146, 1455974855, 1483943235, 1465313059, 1476968773, 1485164700, 1473111226, 1487016525, 1468685011, 1483588627, 1481124440, 1456509414, 1459238822, 1469281735, 1461583775, 1484237254, 1467237713, 1458841635, 1471628037, 1458792211, 1473505758, 1473959224, 1477406016, 1480486581, 1478882378, 1459743335, 1465171671, 1461128664, 1481161428, 1456119639, 1484657062, 1451646892, 1485219612, 1452196091, 1452714322, 1467443136, 1470855443, 1451647352, 1486036523, 1477437329, 1459142317, 1475692040, 1458843696, 1475897545, 1476641043, 1471749640, 1470111782, 1469507347, 1483346000, 1471065449, 1457866259, 1484205875, 1466574574, 1469955065, 1461088810, 1463233750, 1485031171, 1473417849, 1469345557, 1459888776, 1481132549, 1478888218, 1471356413, 1456510217, 1484867871, 1456902769, 1459203750, 1463214331, 1479703393, 1468773873, 1469362442, 1468306571, 1483658318, 1487965733, 1486153663, 1462003045, 1480633811, 1471329387, 1483286160, 1452216360, 1480616093, 1472882290, 1483478801, 1461223310, 1473131133, 1482225555, 1472788616, 1474512902, 1457689085, 1483478801, 1455154618, 1477678397, 1466837249, 1472061220, 1472323839, 1452757700, 1452195076, 1457747972, 1473251106, 1470112763, 1473131600, 1476845374, 1479487212, 1454082444, 1471925648, 1473669546, 1458356486, 1456535235, 1460762304, 1466167215, 1461366495, 1452570571, 1468376643, 1473590470, 1480002856, 1485360871, 1479465450, 1463721299, 1452227880, 1465341119, 1484663458, 1466656505, 1453610104, 1460409168, 1455124310, 1451798752, 1459006200, 1483055142, 1471524661, 1460194342, 1480252283, 1472229123, 1479868080, 1480666341, 1478657335, 1476525123, 1471193383, 1466850127, 1457153808, 1482784364, 1483150748, 1482867859, 1465614206, 1474093170, 1466451361, 1459447294, 1457334689, 1452287620, 1459177715, 1479125058, 1480816664, 1482645069, 1460303860, 1475446595, 1480472652, 1460872119, 1459341832, 1488299995, 1484507581, 1462834297, 1459193502, 1472004874, 1477136236, 1467279034, 1484378707, 1469363311, 1475580754, 1476005244, 1457927949, 1458220650, 1465146265, 1483700101, 1476172029, 1484836565, 1483114299, 1464931071, 1454423951, 1461110093, 1454628770, 1478213922, 1486111540, 1460303726, 1470095663, 1481459490, 1458245776, 1453816126, 1483926407, 1471474677, 1476030783, 1484877603, 1474712999, 1481320989, 1461418648, 1477279763, 1484222223, 1455496864, 1466542837, 1463431525, 1487672538, 1478044742, 1483010083, 1483435555, 1458484304, 1482428479, 1455664942, 1454352689, 1473095166, 1473425727, 1478302065, 1454774360, 1474855875, 1470265753, 1453701652, 1485228569, 1473714998, 1461696297, 1469284557, 1469427392, 1481104645, 1478842482, 1462441021, 1485941326, 1464263098, 1484230026, 1468237272, 1452383896, 1474831215, 1474976547, 1473211379, 1461579221, 1456033117, 1453268867, 1463293191, 1474988425, 1473489093, 1486928663, 1462340013, 1486889022, 1480542415, 1467103884, 1487675569, 1465708340, 1468255288, 1477739837, 1466814754, 1458770552, 1478213922, 1458010329, 1483629383, 1469886051, 1461426005, 1463581062, 1486631176, 1453473378, 1469236994, 1471624305, 1469010532, 1482457453, 1483431557, 1482724671, 1482294090, 1459861690, 1453996456, 1453560825, 1484948272, 1455608978, 1481694187, 1477517909, 1475152972, 1463705956, 1454497331, 1475977995, 1479378400, 1475776898, 1457162563, 1470931454, 1461909753, 1461515316, 1479696532, 1472482217, 1452854090, 1454682947, 1480548587, 1473110223, 1461646085, 1488315492, 1484478543, 1465447137, 1480734606, 1483147494, 1485128752, 1465371851, 1468230199, 1463968490, 1477470537, 1479586239, 1466935405, 1456786777, 1483163953, 1452552002, 1487733734, 1474836043, 1458243835, 1460151695, 1468251877, 1462797082, 1475366665, 1464684937, 1468315646, 1483391984, 1458513952, 1477348152, 1473042571, 1481405160, 1485836882, 1474689388, 1467931486, 1466181976, 1467003579, 1472440877, 1472098408, 1460302455, 1466594938, 1472307251, 1468206636, 1463202675, 1486633947, 1465479931, 1476922117, 1469277324, 1473516956, 1462766811, 1452316951, 1464688808, 1484030988, 1473941822, 1480717110, 1479925704, 1461889582, 1451713915, 1458904883, 1476787498, 1470311730, 1468914467, 1477189363, 1471959809, 1456936901, 1462815518, 1468965367, 1479105566, 1481287810, 1468871484, 1485592474, 1470535453, 1466445364, 1459933353, 1455023074, 1454198440, 1473880092, 1477116377, 1456269040, 1485201743, 1471356413, 1474776219, 1471320556, 1456308665, 1477768468, 1464239137, 1480734606, 1453008418, 1465258672, 1475282117, 1473179288, 1456936901, 1462003627, 1478752055, 1483927903, 1476546185, 1467611827, 1457094160, 1478815175, 1473831133, 1473532595, 1468827770, 1458797975, 1482918980, 1473489093, 1459392886, 1483214021, 1487062787, 1477523779, 1466454370, 1457947990, 1469216170, 1464757153, 1478997024, 1486896886, 1473674076, 1484919061, 1454482936, 1464186923, 1485248401, 1456700762, 1473952985, 1477335944, 1453912891, 1461250783, 1454506025, 1454287732, 1457849154, 1485204516, 1456673272, 1482240083, 1483036668, 1456649712, 1483045616, 1453585041, 1485654624, 1485529957, 1461318124, 1474635681, 1457843191, 1471856824, 1460051329, 1481008727, 1451714043, 1470219653, 1479128305, 1486317191, 1452820570, 1467298131, 1483412446, 1483537792, 1456455369, 1479644469, 1462806860, 1469327045, 1466863694, 1467541141, 1478111425, 1471275650, 1469078769, 1466385810, 1463634424, 1458484304, 1465201706, 1455962879, 1474638968, 1459309032, 1480353238, 1456893554, 1483586745, 1470655978, 1468583835, 1474694573, 1461091495, 1455369687, 1467076670, 1456025353, 1473839457, 1483074134, 1465358385, 1486806633, 1478038202, 1486134184, 1455905428, 1467121831, 1456251757, 1458026078, 1455091782, 1480017227, 1479094259, 1474395138, 1460210745, 1471855341, 1481533335, 1467208813, 1468950936, 1479595970, 1455650525, 1473082848, 1487114692, 1484143884, 1470811068, 1485039954, 1466887139, 1466536991, 1470060954, 1474706236, 1476380636, 1453894016, 1487745738, 1462169189, 1487341251, 1452129494, 1481010748, 1474873842, 1469351841, 1481579074, 1464039617, 1478155657, 1470333728, 1464002139, 1466819026, 1480030204, 1474602919, 1460975253, 1468584001, 1482305492, 1481722944, 1485259402, 1466569099, 1471329633, 1486746610, 1468840535, 1469021303, 1472508232, 1476799369, 1458093093, 1462932435, 1478196685, 1462458360, 1453815049, 1481876143, 1456352931, 1458052666, 1458728602, 1486496149, 1466731706, 1475182171, 1474043245, 1457714920, 1479304008, 1455603545, 1459478937, 1474110948, 1474812892, 1462715815, 1476463758, 1463181398, 1471441015, 1481148755, 1477635642, 1458022715, 1488261866, 1461366495, 1466382180, 1486845540, 1485688156, 1467129558, 1465686783, 1466819824, 1454610523, 1473360884, 1451816310, 1487395937, 1464462524, 1480335965, 1476517885, 1482367075, 1487782094, 1481467632, 1470767268, 1470403405, 1469703201, 1464793236, 1467962861, 1463755986, 1466454739, 1464785291, 1474703191, 1487812677, 1452578263, 1476662094, 1481891773, 1488036164, 1487070185, 1474495132, 1480666341, 1467115474, 1478031712, 1453163186, 1476804954, 1474712999, 1479097701, 1470485675, 1456807495, 1468900195, 1481696067, 1485871297, 1461719422, 1454690832, 1487806079, 1459444108, 1453033290, 1457307473, 1480977436, 1486338843, 1476523072, 1454871449, 1472902690, 1454113546, 1480723382, 1461517006, 1482138241, 1470566260, 1457384524, 1464014039, 1463471606, 1454805916, 1472493755, 1468097464, 1484451054, 1476924835, 1465696691, 1467981874, 1461283377, 1471913897, 1470518612, 1486602269, 1459826717, 1470109871, 1473968722, 1464192943, 1470231906, 1465229444, 1487641471, 1487083690, 1479696532, 1483616470, 1461140170, 1458934467, 1469623267, 1451994509, 1461692075, 1471421594, 1480653099, 1466381495, 1464391526, 1453432440, 1474270077, 1472966939, 1476935974, 1473062706, 1458849935, 1487502960, 1463586311, 1464068895, 1479378400, 1454424539, 1469363547, 1457330292, 1467121344, 1479896072, 1461283377, 1475263897, 1464344767, 1474103560, 1461352506, 1483280629, 1479419815, 1487073215, 1453676312, 1466552489, 1487032827, 1483113273, 1476122333, 1477312660, 1479157145, 1485461790, 1456310033, 1467276092, 1486366916, 1464296678, 1452633689, 1482149702, 1472127565, 1466005374, 1455204975, 1472696548, 1457359893, 1452798084, 1460300122, 1486332399, 1453605272, 1466288109, 1452687642, 1470767311, 1455584371, 1452793336, 1464506639, 1474421352, 1451642296, 1487595935, 1461424531, 1457476803, 1453218526, 1487548766, 1456402979, 1462840610, 1463196663, 1466698089, 1474812892, 1465478038, 1460196279, 1464771270, 1474684805, 1467156992, 1484666506, 1483767063, 1482284300, 1472943217, 1484345865, 1452570571, 1464431015, 1482149702, 1470062246, 1457744397, 1480527782, 1473807209, 1458990609, 1459270738, 1484026040, 1461880187, 1484705919, 1460566109, 1484283567, 1476404894, 1469727643, 1467130709, 1461410549, 1464957443, 1469743919, 1485894819, 1480977436, 1452581752, 1488224292, 1475580754, 1459056940, 1475289657, 1465050879, 1464553949, 1482286786, 1456126595, 1478666824, 1476847928, 1467332183, 1461608246, 1485300062, 1484242512, 1456709083, 1464044953, 1476121429, 1453870486, 1454053805, 1468437640, 1481505937, 1460496018, 1458601233, 1460441625, 1456450203, 1471002881, 1474375645, 1477768468, 1485132810, 1467918347, 1474110948, 1466093864, 1466260849, 1478092129, 1464504221, 1480217559, 1482100476, 1479943814, 1484660064, 1452424028, 1486785116, 1482100215, 1460992972, 1463591804, 1476101659, 1465088686, 1483565251, 1452879203, 1465230292, 1464748965, 1458042302, 1478586028, 1472664766, 1466338585, 1465088686, 1457022868, 1469025060, 1455695958, 1453007746, 1479076145, 1453432440, 1459077300, 1487743916, 1471228354, 1461516477, 1456422910, 1464668906, 1483391984, 1483814052, 1461385606, 1456325461, 1471920770, 1469363547, 1477913894, 1470329879, 1480894688, 1462627441, 1456746316, 1453700935, 1480259083, 1455751592, 1482790176, 1454776876, 1457975195, 1475492217, 1471172968, 1459392381, 1483682746, 1462932435, 1467597322, 1484397073, 1456769911, 1462258520, 1455891256, 1452815816, 1471117663, 1454438746, 1461104848, 1473884505, 1475513729, 1479578760, 1475412214, 1467799415, 1461002487, 1459418462, 1476899716, 1465531163, 1466353797, 1474808515, 1478242655, 1478467686, 1486363777, 1486623148, 1457584969, 1461741292, 1468351732, 1477122241, 1483059504, 1465614206, 1476698091, 1459888776, 1476350423, 1473272789, 1467415031, 1453273281, 1466613744, 1474503693, 1467049292, 1468097172, 1461911466, 1453522238, 1473968722, 1483286160, 1453801809, 1471823078, 1473200307, 1466582118, 1476101659, 1467259002, 1457553896, 1484860932, 1481732023, 1484800332, 1469268298, 1468036739, 1452007063, 1478608805, 1453955830, 1476344014, 1457441256, 1456392874, 1459885089, 1483780515, 1480475220, 1487645551, 1467451083, 1465698086, 1470194613, 1475420105, 1460672579, 1476297236, 1459974594, 1486086586, 1452047038, 1454413032, 1454500577, 1460913122, 1466366160, 1453212741, 1468403389, 1454153688, 1474879917, 1464678900, 1485529957, 1483749508, 1452168758, 1472728769, 1479938216, 1483874687, 1463652756, 1480155325, 1488319813, 1472279708, 1468929555, 1477051689, 1485095680, 1483182264, 1484848259, 1462006190, 1484111886, 1454861709, 1479086933, 1474485651, 1488254769, 1480325502, 1483951242, 1484048061, 1477913894, 1470725646, 1461820557, 1461805615, 1472837555, 1463791369, 1458010329, 1463540745, 1455389623, 1472604281, 1484920508, 1466221684, 1475412214, 1460109789, 1466860466, 1463973604, 1484831578, 1484847491, 1458870498, 1473454468, 1458652912, 1478386938, 1481556993, 1482688625, 1468021301, 1459809099, 1477291496, 1480509028, 1468954112, 1480624493, 1487884387, 1465648715, 1484275257, 1458005643, 1459953327, 1460642155, 1455532555, 1465468862, 1465856667, 1459823390, 1476247660, 1475853267, 1452823522, 1453165799, 1476372489, 1483692105, 1470501176, 1481982416, 1470121486, 1456892618, 1452125095, 1458196268, 1475491203, 1456503886, 1467621277, 1469363547, 1478179698, 1461917734, 1485208165, 1458624241, 1478078169, 1473912584, 1472736532, 1486245497, 1467180321, 1475378684, 1474158991, 1456081366, 1454208886, 1465631420, 1480952031, 1471835445, 1462179459, 1454991381, 1473866754, 1464481357, 1484048061, 1465897402, 1468568480, 1457533850, 1481032514, 1466870854, 1454318265, 1481336532, 1459873358, 1486746610, 1455575812, 1467536119, 1473818160, 1463211979, 1469080309, 1461696297, 1465136534, 1464481357, 1486908153, 1486260979, 1481148755, 1465484059, 1488070752, 1463663972, 1476212865, 1456402979, 1482812553, 1459888776, 1459696369, 1473547457, 1461805615, 1455378859, 1467632537, 1468120668, 1471340943, 1458859746, 1464745850, 1471687894, 1460254622, 1462401003, 1484936589, 1476023782, 1462573466, 1459776343, 1464540588, 1463122189, 1465654334, 1466487602, 1458789930, 1477571985, 1485344728, 1477344285, 1466580876, 1479304008, 1480665171, 1488029395, 1472802889, 1469122248, 1471586467, 1480475199, 1476395285, 1477279580, 1456709083, 1480454787, 1471575259, 1485205548, 1483233191, 1474485651, 1478098317, 1478357746, 1468692367, 1473400507, 1464044849, 1467567992, 1465761699, 1479872717, 1470639642, 1454335269, 1460053950, 1457265420, 1485879471, 1483592994, 1483599265, 1458884024, 1487075326, 1478956693, 1480952789, 1466270950, 1458081844, 1481539497, 1485263422, 1472544959, 1485395279, 1463249637, 1479427490, 1462661383, 1462003627, 1482804587, 1480183336, 1463715663, 1456854665, 1464789777, 1462815784, 1456813556, 1461960956, 1468883686, 1465986443, 1458974874, 1469776948, 1452968455, 1478112268, 1452196091, 1458564745, 1460670046, 1468968537, 1458785663, 1486183100, 1479234164, 1457666502, 1485439095, 1483948730, 1464141833, 1453513492, 1474446745, 1454957526, 1456109496, 1487851132, 1470966262, 1464753577, 1459177715, 1458531196, 1485080461, 1476785604, 1458595878, 1483259653, 1457075383, 1457652779, 1473037389, 1469463928, 1482206548, 1459867139, 1459229398, 1470812899, 1476344014, 1478396539, 1466181736, 1467443136, 1459363878, 1456251757, 1478234591, 1466574574, 1452753019, 1457377052, 1456446767, 1453294928, 1486629509, 1475312323, 1463089812, 1460200672, 1472952230, 1458456102, 1462077436, 1459392886, 1460952109, 1483947780, 1475860032, 1477562953, 1472170929, 1482682067, 1474525751, 1473105204, 1468407061, 1487818374, 1457094160, 1469400943, 1478586028, 1460090385, 1471194511, 1474803041, 1468003801, 1468409114, 1485293729, 1457634565, 1478295247, 1482199372, 1466476765, 1476526588, 1460028125, 1464598249, 1484991777, 1475982160, 1455473892, 1480561019, 1455769501, 1453890445, 1473339304, 1469317311, 1470719916, 1476901325, 1481911514, 1457676654, 1478678871, 1456535235, 1457755035, 1453064064, 1467438272, 1469044780, 1468479109, 1464080759, 1479692073, 1463520565, 1487817593, 1471402615, 1476103846, 1483286264, 1462872571, 1487428187, 1471049218, 1467541141, 1462955185, 1487465120, 1480494802, 1481132549, 1481704241, 1475009896, 1457109459, 1474961540, 1452798084, 1466485105, 1486788917, 1462594050, 1463111804, 1453256965, 1478893766, 1487450650, 1482111604, 1459730023, 1453206175, 1458595878, 1462872288, 1465921496, 1487073712, 1460681485, 1460948113, 1455285711, 1475809824, 1471921160, 1476836470, 1452073276, 1483786875, 1462123795, 1483122267, 1475807191, 1466984819, 1470004346, 1452287606, 1481955209, 1474932788, 1472012045, 1473638473, 1485224317, 1455886292, 1488190209, 1463031204, 1484821724, 1482100215, 1456247725, 1486225852, 1454751380, 1485702152, 1464320942, 1465449338, 1476453778, 1481862316, 1459616539, 1468487480, 1458297813, 1453361398, 1486972113, 1459259390, 1468433077, 1468473766, 1465097444, 1465931088, 1480699008, 1456397849, 1470655978, 1463348123, 1469477901, 1470966514, 1471856824, 1461657206, 1456404959, 1469212867, 1478245803, 1474993259, 1479897167, 1474503693, 1465043539, 1480373441, 1487841447, 1469994470, 1480289842, 1466306977, 1473930591, 1486903320, 1460605060, 1477333863, 1474988425, 1476845374, 1460300122, 1464549288, 1459885089, 1458792211, 1486356856, 1469363628, 1475612155, 1483684868, 1487100107, 1486717521, 1454901947, 1483815357, 1471823078, 1487136179, 1479854206, 1452635710, 1463195840, 1476153002, 1474940736, 1461152456, 1458158480, 1480959292, 1463770557, 1479644469, 1471013001, 1469007730, 1453631116, 1463501494, 1459184170, 1455395665, 1452495313, 1452821166, 1485894819, 1455453508, 1458983663, 1472102023, 1461114030, 1472419499, 1455822710, 1456325461, 1476230469, 1468097464, 1463977294, 1452855899, 1457386138, 1472171430, 1453671680, 1482804587, 1487848669, 1468803396, 1485301421, 1477208853, 1473571235, 1463036487, 1486506037, 1456154434, 1488138834, 1456084286, 1484977679, 1471837505, 1482418666, 1482774563, 1480889112, 1465977967, 1479134290, 1477411388, 1473719687, 1480952031, 1472067351, 1484064406, 1456997263, 1456639374, 1481224573, 1469116954, 1476871808, 1476526588, 1451921618, 1459175940, 1482665100, 1479795956, 1457930957, 1480169582, 1463100685, 1485138228, 1464748965, 1486063842, 1481087894, 1470723659, 1454774360, 1455786136, 1487270433, 1478377784, 1456698124, 1483099344, 1470390062, 1453049807, 1453251516, 1467883843, 1469150441, 1462775199, 1455011818, 1453597829, 1471329238, 1467211156, 1467713068, 1452947627, 1452372730, 1477752696, 1459715329, 1464952201, 1459850388, 1488307814, 1453563835, 1474369793, 1471447711, 1466832707, 1458826916, 1478441926, 1470564684, 1466794692, 1483280629, 1473273269, 1477705740, 1457837762, 1486494234, 1454436760, 1459466962, 1456977037, 1463231510, 1452578263, 1459578491, 1474132289, 1457863269, 1453177484, 1483198389, 1476770151, 1460107953, 1460297005, 1451940407, 1467541141, 1453033841, 1483051161, 1457813906, 1453022075, 1459714071, 1475244707, 1469686278, 1466879965, 1483085803, 1478881215, 1454253916, 1487181702, 1466775364, 1462401003, 1470247222, 1457504351, 1472291782, 1476346187, 1470839853, 1480613661, 1451849291, 1454925128, 1460315268, 1482428479, 1483943235, 1460896173, 1465534076, 1472264510, 1456418962, 1468943966, 1481905788, 1483508470, 1468582835, 1458906241, 1471930726, 1483320859, 1467165688, 1459752182, 1453397681, 1467322448, 1463236431, 1463709859, 1476597522, 1481645621, 1459634287, 1465686239, 1472983704, 1472134804, 1488100331, 1472390407, 1451969401, 1485343124, 1469529922, 1487653008, 1480711609, 1464593209, 1466518183, 1456356935, 1487437955, 1463951459, 1487427072, 1452653775, 1479663924, 1469679675, 1470239313, 1465478038, 1479553311, 1473207817, 1453475784, 1471382893, 1474074638, 1488295605, 1478202980, 1484064550, 1451642296, 1464827550, 1457416827, 1485594597, 1465449276, 1452794792, 1465419075, 1487662683, 1474840664, 1456247047, 1475004617, 1460842041, 1456853934, 1467480272, 1452766808, 1453708867, 1453109334, 1454442500, 1481962075, 1470725539, 1463977294, 1463424789, 1484919061, 1485502937, 1463631463, 1485128752, 1463172567, 1467465626, 1464379353, 1479427103, 1463770557, 1475174877, 1452921598, 1476587473, 1486534487, 1480183336, 1459446726, 1480816664, 1465590195, 1469463928, 1479648048, 1475513648, 1452965865, 1471468136, 1463240347, 1462935809, 1462345726, 1465708340, 1480405261, 1457815336, 1475708832, 1481624122, 1454133166, 1452128987, 1457697219, 1470744941, 1453212741, 1483999357, 1481195513, 1459557827, 1459795085, 1454628770, 1453801809, 1472739110, 1462850667, 1461864636, 1457575905, 1479273324, 1478055299, 1458015127, 1472865599, 1487350889, 1473107521, 1475425901, 1474710722, 1472061220, 1466627385, 1473316510, 1486842856, 1461604252, 1472989654, 1479433223, 1482716886, 1487353203, 1458165031, 1485306756, 1467847757, 1462931382, 1465518865, 1476111242, 1457224472, 1472564614, 1457370855, 1472385965, 1486356856, 1475389646, 1488299995, 1482163658, 1462305858, 1468906794, 1464072950, 1475695941, 1467151751, 1453278609, 1478993462, 1465986443, 1480679178, 1457714920, 1462996371, 1473766712, 1456530231, 1475062797, 1457065528, 1475253827, 1462806860, 1487236019, 1462583101, 1457013670, 1464575049, 1468086059, 1467691249, 1484555386, 1473301889, 1461344593, 1458417780, 1480677908, 1467296318, 1457837762, 1461110093, 1465468862, 1452559315, 1487208431, 1468306571, 1472871180, 1472839731, 1477502384, 1484865345, 1483077120, 1475429256, 1484012550, 1468111749, 1460887307, 1452856501, 1467667562, 1467643383, 1460667729, 1477187875, 1473010629, 1483123984, 1480494802, 1472554597, 1451866109, 1481549381, 1473062706, 1455364167, 1485031171, 1487437414, 1468435324, 1469174490, 1458124061, 1453185203, 1482097169, 1466760845, 1483969235, 1457699479, 1476916676, 1478430725, 1475452527, 1453900665, 1478227392, 1471837505, 1485149042, 1459715329, 1474305259, 1459932043, 1477335944, 1480182133, 1479341025, 1460976527, 1475711956, 1475248005, 1473924244, 1466278895, 1482562775, 1466825018, 1467381078, 1474503740, 1472610448, 1474675300, 1466503129, 1472689199, 1476545182, 1460303726, 1473527218, 1458990609, 1486697713, 1487496057, 1475222113, 1455690819, 1476346886, 1462872571, 1486161089, 1469208281, 1464004286, 1453816126, 1477060798, 1475819131, 1483302281, 1464044849, 1476092722, 1471636115, 1463721299, 1470231043, 1470711621, 1456180387, 1453547182, 1470632623, 1461166422, 1469659465, 1475124450, 1484868100, 1459064337, 1479253117, 1458947329, 1483182264, 1484216773, 1472631049, 1482335200, 1479937899, 1465572790, 1456193229, 1465533840, 1462780120, 1453688183, 1463125313, 1487044041, 1481885121, 1468996470, 1469388993, 1470709933, 1453789032, 1476861196, 1476138972, 1458479561, 1477617645, 1466084483, 1467817788, 1465770805, 1479096456, 1456507272, 1465173985, 1469025060, 1471913897, 1472140697, 1468097172, 1456288436, 1452566959, 1454520979, 1452975753, 1462373810, 1469625789, 1486083885, 1485567817, 1461192340, 1480679178, 1474093279, 1475984852, 1478513778, 1462734014, 1467804760, 1467093514, 1465451360, 1466712092, 1468744139, 1475377760, 1466724230, 1454710290, 1455459294, 1472231192, 1482819012, 1484546797, 1459321557, 1475785779, 1467995205, 1481885121, 1465121376, 1465127931, 1470234999, 1451844588, 1471215593, 1465121376, 1486341583, 1464476496, 1472123477, 1462843455, 1462243897, 1467960348, 1482101449, 1476966452, 1472710128, 1468049263, 1457694012, 1470844872, 1473790435, 1487364725, 1478158215, 1467227713, 1460576438, 1456588753, 1472172747, 1481068517, 1466832707, 1455755842, 1485113412, 1478578796, 1480850955, 1479075772, 1457357840, 1462872288, 1477664883, 1475595684, 1481161428, 1457832779, 1464613506, 1469248698, 1461859835, 1453700380, 1483700101, 1486374440, 1483869538, 1462262940, 1486780010, 1465786328, 1463795217, 1485774035, 1480114729, 1486274327, 1471526206, 1478504455, 1468968537, 1460972939, 1469607556, 1483933539, 1464607498, 1461843055, 1458760420, 1466935405, 1478177971, 1459714071, 1455023121, 1470096015, 1477790832, 1484577607, 1457831748, 1479543640, 1459878681, 1480605385, 1459100651, 1463289862, 1452216360, 1471265968, 1464297084, 1480835854, 1480657958, 1479100880, 1456867979, 1480719578, 1461663548, 1464805379, 1467692082, 1467049292, 1482285190, 1459779761, 1475797134, 1476328295, 1483473147, 1485826983, 1472768096, 1457733955, 1481772131, 1458878665, 1458727899, 1465412118, 1466756821, 1468890786, 1483841652, 1456116912, 1473292100, 1475098794, 1458664013, 1474976547, 1459024273, 1454113546, 1480798690, 1480824195, 1487694175, 1470196229, 1482320511, 1465570252, 1470494344, 1486017830, 1459742834, 1487658331, 1470598239, 1460139735, 1471910784, 1479918695, 1480976906, 1456209859, 1455601031, 1486405211, 1459968159, 1460809214, 1473518577, 1457037310, 1479578760, 1469318659, 1455436764, 1473437387, 1479094259, 1488062828, 1471426928, 1465698086, 1467503413, 1466117397, 1467766007, 1467514159, 1469516222, 1462677105, 1466593343, 1475555239, 1458492797, 1485095680, 1485071639, 1461805615, 1467766201, 1473096725, 1472431196, 1472298379, 1455554279, 1454070027, 1479696532, 1479939874, 1462195304, 1480403018, 1455948503, 1458106638, 1474658277, 1451737630, 1454083118, 1464678881, 1468036739, 1476261070, 1472174390, 1480462120, 1455777606, 1486697368, 1483113273, 1471424429, 1481013706, 1456109496, 1454912455, 1451816310, 1482341096, 1478203799, 1485308489, 1470816854, 1466192199, 1458885262, 1458165617, 1456421920, 1462429168, 1476224878, 1482676569, 1451940275, 1462815784, 1474928443, 1478430725, 1464783149, 1453680882, 1473714998, 1454505804, 1485398218, 1460667729, 1486302472, 1457750519, 1453187208, 1474614756, 1457162383, 1462775199, 1483150748, 1461756187, 1459188221, 1452080674, 1478459378, 1455163576, 1475998210, 1470161072, 1460081347, 1460686648, 1488146027, 1463195840, 1480502534, 1469362258, 1480180836, 1486225852, 1465276374, 1462550422, 1482134688, 1475278079, 1463872513, 1468290124, 1485015144, 1475082600, 1483538932, 1481341272, 1459365480, 1456440893, 1472523561, 1484052112, 1480816664, 1472291782, 1473839311, 1456807495, 1468807114, 1471272023, 1465121376, 1457758290, 1478610639, 1471356413, 1455116091, 1461381635, 1464014483, 1453424792, 1456614442, 1468053449, 1484222223, 1463484544, 1472355578, 1467146097, 1456251757, 1487223382, 1466756821, 1477918033, 1476915300, 1478481644, 1472742985, 1483969235, 1485052357, 1475564383, 1468721049, 1466984643, 1476916676, 1453868103, 1471921370, 1470326622, 1474689388, 1465710871, 1460258550, 1481350606, 1467658286, 1459341832, 1464548406, 1477969038, 1470987062, 1465534076, 1459925478, 1461519033, 1463248783, 1455960887, 1475355050, 1462715815, 1476580612, 1453816126, 1472323839, 1482753756, 1473466690, 1455070314, 1458961505, 1451982108, 1461323545, 1454323619, 1472272935, 1468498408, 1473042571, 1477768468, 1464616225, 1469067541, 1477620544, 1487915573, 1477260182, 1466358268, 1454864257, 1455702216, 1458371850, 1470969794, 1452848232, 1471942737, 1472395516, 1476545182, 1473179288, 1481008727, 1479070630, 1467270962, 1453920355, 1472952230, 1472780692, 1463810846, 1473037389, 1480579366, 1486700138, 1485429453, 1465708857, 1472385965, 1484296391, 1481257591, 1458794400, 1461206402, 1464442076, 1486788917, 1457382562, 1458608623, 1480425403, 1483684868, 1485505588, 1456189648, 1463471190, 1456188385, 1479326886, 1482284385, 1458780732, 1470980612, 1472355578, 1456907775, 1473505758, 1457902945, 1468522511, 1454335269, 1452058953, 1456008997, 1456146009, 1487343334, 1474632127, 1457453427, 1453008418, 1463376194, 1462993430, 1480628716, 1481895579, 1462677749, 1486390677, 1469788511, 1458452523, 1481535169, 1479326886, 1477312660, 1460596579, 1473815937, 1486935220, 1454548375, 1480254275, 1457985179, 1468116160, 1469054271, 1460029775, 1459178076, 1454435643, 1464590990, 1485301993, 1453227295, 1476172765, 1457405839, 1467624350, 1483183042, 1476867696, 1465400207, 1460021205, 1453033841, 1456862784, 1455091782, 1473742665, 1479266514, 1480300104, 1460694180, 1465479522, 1479292033, 1458901000, 1478244648, 1484831578, 1475434519, 1487208431, 1461679226, 1487140210, 1475286153, 1458965370, 1479383376, 1470521017, 1461397871, 1462579693, 1478685848, 1465659914, 1487078319, 1479688478, 1470945080, 1486390677, 1459590149, 1488017847, 1469763920, 1478203714, 1484229982, 1479182828, 1481895579, 1485342474, 1462106212, 1458855814, 1470828917, 1464581022, 1477776660, 1454410566, 1467872513, 1475691034, 1480561019, 1453585041, 1478048800, 1471008516, 1481087894, 1475197880, 1470973030, 1484316641, 1470987518, 1483048387, 1457356315, 1467194767, 1458722089, 1474859071, 1473766683, 1460440296, 1488062362, 1455584371, 1470155264, 1476821052, 1484211505, 1478010438, 1487736761, 1482621469, 1469433641, 1477421526, 1468368981, 1476158446, 1484343211, 1452655684, 1458561300, 1477304547, 1458086796, 1470966262, 1484163794, 1467964170, 1453134142, 1484370088, 1469796800, 1465770805, 1470454806, 1478038272, 1457162383, 1465590195, 1461092905, 1467375041, 1483538677, 1468684843, 1472298379, 1471964984, 1459477311, 1473112721, 1468885152, 1457602368, 1483616470, 1488293852, 1456504414, 1478092551, 1478568563, 1454557491, 1468978806, 1461631200, 1471013001, 1463098826, 1471135271, 1454587535, 1468232872, 1472152533, 1453925637, 1452939253, 1461273469, 1472591183, 1479725023, 1466215401, 1460193322, 1455025518, 1484840397, 1452665946, 1488029395, 1475589258, 1464768990, 1483553720, 1464044849, 1484720867, 1462562531, 1465467169, 1484663458, 1467130709, 1468793612, 1466753519, 1453820622, 1460349200, 1465823013, 1488056835, 1468126059, 1481707161, 1483147340, 1456089984, 1456297023, 1480660251, 1458678223, 1474388912, 1458028342, 1457265420, 1461938679, 1469914449, 1462925012, 1475348880, 1467798298, 1455261256, 1486420347, 1465201706, 1482165395, 1453390947, 1480024461, 1452253151, 1468883686, 1464398460, 1485428471, 1462063975, 1477917906, 1478979896, 1465570252, 1480675497, 1463616646, 1484966674, 1481998156, 1451940275, 1466244182, 1452054902, 1475883402, 1470537888, 1458277134, 1463300876, 1481429636, 1469584712, 1486640319, 1468840535, 1463604858, 1454959270, 1482607609, 1480330225, 1453224654, 1464776819, 1470552236, 1457505556, 1464413990, 1487848669, 1476680831, 1480198875, 1475803015, 1481467631, 1486328538, 1487078319, 1482437151, 1470512483, 1465708857, 1466492591, 1482959404, 1453466342, 1465220792, 1479896072, 1481573609, 1487960824, 1483601817, 1487731853, 1484755301, 1460811252, 1476441671, 1468513288, 1485263422, 1452599920, 1468888480, 1462729163, 1463568017, 1484203016, 1460948548, 1452727180, 1455540640, 1475090917, 1472457984, 1465732568, 1463261678, 1452555711, 1461079234, 1451982880, 1466431645, 1485498360, 1452208898, 1479086933, 1478611757, 1467180321, 1479233299, 1485248401, 1464418633, 1472330926, 1456308665, 1473322569, 1458371850, 1483354478, 1474953257, 1470089165, 1485414827, 1485562885, 1483286264, 1478584127, 1467991587, 1456986556, 1458608623, 1464309159, 1469596015, 1470219653, 1473743159, 1475631812, 1471942737, 1462486276, 1456395729, 1459134195, 1484815997, 1458699184, 1453558558, 1478446966, 1460902548, 1470512483, 1475268455, 1481462670, 1484182748, 1468961676, 1467711624, 1468242572, 1468056791, 1474289378, 1461646085, 1462184742, 1484402335, 1465258672, 1479736899, 1465861067, 1485414827, 1478251826, 1461604252, 1460325884, 1472974950, 1470729646, 1464218481, 1484403756, 1486387956, 1477082991, 1460303860, 1473925079, 1460617269, 1465579185, 1466690116, 1469009767, 1467657326, 1463047209, 1465042273, 1457790592, 1465207963, 1486600054, 1480780106, 1463690303, 1451997801, 1488310882, 1487851132, 1459148859, 1475695941, 1467044431, 1481958211, 1472909726, 1454226350, 1475736321, 1483408534, 1468435909, 1455739985, 1472110936, 1466112620, 1453968432, 1486078018, 1476931897, 1453224654, 1477604257, 1457753862, 1467042301, 1454618548, 1459207091, 1453615656, 1472983704, 1465008499, 1466927795, 1463539269, 1471951187, 1472041655, 1482543175, 1462917917, 1459876146, 1476030783, 1455994733, 1483947780, 1468039325, 1484490925, 1472472376, 1468251621, 1476154352, 1460123918, 1452438952, 1463677753, 1461686168, 1481684635, 1484517730, 1458220650, 1473778084, 1472839731, 1475177777, 1483825169, 1486944402, 1482230672, 1469650528, 1463372958, 1452206051, 1487073215, 1479497295, 1475877035, 1474827364, 1453256965, 1466870854, 1482405233, 1457397504, 1477719554, 1482757364, 1488122110, 1459610870, 1473113331, 1455964958, 1468340148, 1457151062, 1451803879, 1471385858, 1482306040, 1460612549, 1475125319, 1474249007, 1486630350, 1461273469, 1480877578, 1451697649, 1459875124, 1473020118, 1486598989, 1468950936, 1461017696, 1467845311, 1452854090, 1487171107, 1470421977, 1486872260, 1454966454, 1467851147, 1488034140, 1452399973, 1475918849, 1480475199, 1462458360, 1469055418, 1484150578, 1461859735, 1458138671, 1477036772, 1453109334, 1464382943, 1460955735, 1456542382, 1454463929, 1459917887, 1462477876, 1473144227, 1452672902, 1473520625, 1475312218, 1471793924, 1471013001, 1474014172, 1473913617, 1472996675, 1453023061, 1487045888, 1474523662, 1463356807, 1487139411, 1474836135, 1459679738, 1485498424, 1473597112, 1462111958, 1454016298, 1459713634, 1465533840])]




In [11]:
#少量数据示例
bx = [Row(cust_id='cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272',
         amt_all = ['20*5','10*15','70*1','30*3','20*2','60*8','90*6','10*7','50*9'],
         balance_amt_all = [],
         u_time_all = [5,15,1,3,2,8,6,7,9]
         )]

#### rdd mapPartitions处理函数

In [12]:
#rdd mapPartitions处理函数
def history_mp_fun(x_list):
    for x in x_list:
        amt_list = [
            ['amt_all',x.amt_all],
            ['balance_amt_all', x.balance_amt_all],
        ]
        
        time_list = [
            ['u_time_all', x.u_time_all],
        ]
        
        res_dict = {}
        a_dict = {}
        # 1.金额
        for col in amt_list:
            col_name = col[0]
            
            # 如果col[1]为空时执行
            if not col[1]:
                #a_dict[col_name] = []
                a_dict[col_name + '_avg'] = float(np.nan)
                a_dict[col_name + '_max'] = float(np.nan)
                a_dict[col_name + '_min'] = float(np.nan)
                a_dict[col_name + '_q1'] = float(np.nan)
                a_dict[col_name + '_q3'] = float(np.nan)
                a_dict[col_name + '_softNoise'] = float(np.nan)
                a_dict[col_name + '_hardNoise'] = float(np.nan)
                a_dict[col_name + '_lastAmt'] = float(np.nan) #最后一次交易金额
                continue
            
            sort_list = []
            for item in col[1]:
                each = item.split('*')
                amt = each[0] #金额
                u_trx_time = each[1] #时间戳
                #print(amt," * ", u_trx_time)
                sort_list.append([amt, u_trx_time])
            
            # 按照时间戳timestemp排序
            sort_list = sorted(sort_list, key=lambda x: int(x[1]))
            tmp_list = []  # 临时list
            for ele in sort_list:
                tmp_list.append(float(ele[0]))
                
            #a_dict[col_name] = tmp_list
            a_dict[col_name + '_avg'] = float(np.mean(tmp_list))
            a_dict[col_name + '_max'] = float(np.max(tmp_list))
            a_dict[col_name + '_min'] = float(np.min(tmp_list))
            a_dict[col_name + '_q1'] = float(np.percentile(tmp_list,25))
            a_dict[col_name + '_q3'] = float(np.percentile(tmp_list,75))
            a_dict[col_name + '_softNoise'] = float(2.5 * a_dict[col_name + '_q3'] - 1.5 * a_dict[col_name + '_q1'])
            a_dict[col_name + '_hardNoise'] = float(4 * a_dict[col_name + '_q3'] - 3 * a_dict[col_name + '_q1'])
            a_dict[col_name + '_lastAmt'] = tmp_list[-1] #最后一次交易金额
         
        # 2.时间
        for col in time_list:
            col_name = col[0]
            
             # 如果col[1]为空时执行
            if not col[1]:
                #a_dict[col_name] = []
                a_dict[col_name + '_lastTime'] = float(np.nan) #最后一次交易时间
                a_dict[col_name + '_avg'] = float(np.nan)
                a_dict[col_name + '_max'] = float(np.nan)
                a_dict[col_name + '_min'] = float(np.nan)
                a_dict[col_name + '_q1'] = float(np.nan)
                a_dict[col_name + '_q3'] = float(np.nan)
                a_dict[col_name + '_softNoise'] = float(np.nan)
                a_dict[col_name + '_hardNoise'] = float(np.nan)
                continue
            
            # col[1]非空时执行
            sort_list = []
            for timestamp in col[1]:
                each = int(timestamp)
                sort_list.append(each)
            
            sort_list = sorted(sort_list)
            
            diff_list = [] #交易时间差分list
            for t in range(0, len(sort_list)-1):
                diff = sort_list[t+1] - sort_list[t]
                diff_list.append(diff)
            
            a_dict[col_name + '_lastTime'] = sort_list[-1] #最后一次交易时间
            a_dict[col_name + '_avg'] = float(np.mean(diff_list))
            a_dict[col_name + '_max'] = float(np.max(diff_list))
            a_dict[col_name + '_min'] = float(np.min(diff_list))
            a_dict[col_name + '_q1'] = float(np.percentile(diff_list,25))
            a_dict[col_name + '_q3'] = float(np.percentile(diff_list,75))
            a_dict[col_name + '_softNoise'] = float(2.5 * a_dict[col_name + '_q3'] - 1.5 * a_dict[col_name + '_q1'])
            a_dict[col_name + '_hardNoise'] = float(4 * a_dict[col_name + '_q3'] - 3 * a_dict[col_name + '_q1'])

        
        b_dict = {"cust_id":x.cust_id}
        
        res_dict = {**b_dict, **a_dict}
        
        keys = list(res_dict.keys())
        #print(keys)

        yield [res_dict[k] for k in keys]
        
    #return res_dict

list(history_mp_fun(ax))
#history_mp_fun(bx)

[['cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272',
  50136866.52888159,
  99997313.9013,
  13658.6609269,
  25796796.7907,
  74648719.5469,
  147926603.68120003,
  221204487.81550002,
  86556196.8779,
  49543170.957238674,
  99978607.4561,
  13517.9560216,
  24831926.517075,
  74035816.1059,
  147841650.4891375,
  221647484.872375,
  33280122.6462,
  1488324413,
  2193.735615701739,
  43695.0,
  0.0,
  0.0,
  3208.0,
  8020.0,
  12832.0]]

In [13]:
#结果示例
{'cust_id': 'cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272',
 'amt_all_avg': 50136866.52888159,
 'amt_all_max': 99997313.9013,
 'amt_all_min': 13658.6609269,
 'amt_all_q1': 25796796.7907,
 'amt_all_q3': 74648719.5469,
 'amt_all_softNoise': 147926603.68120003,
 'amt_all_hardNoise': 221204487.81550002,
 'amt_all_lastAmt': 86556196.8779,
 'balance_amt_all_avg': 49543170.957238674,
 'balance_amt_all_max': 99978607.4561,
 'balance_amt_all_min': 13517.9560216,
 'balance_amt_all_q1': 24831926.517075,
 'balance_amt_all_q3': 74035816.1059,
 'balance_amt_all_softNoise': 147841650.4891375,
 'balance_amt_all_hardNoise': 221647484.872375,
 'balance_amt_all_lastAmt': 33280122.6462,
 'u_time_all_lastTime': 1488324413,
 'u_time_all_avg': 2193.735615701739,
 'u_time_all_max': 43695.0,
 'u_time_all_min': 0.0,
 'u_time_all_q1': 0.0,
 'u_time_all_q3': 3208.0,
 'u_time_all_softNoise': 8020.0,
 'u_time_all_hardNoise': 12832.0}

{'cust_id': 'cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272',
 'amt_all_avg': 50136866.52888159,
 'amt_all_max': 99997313.9013,
 'amt_all_min': 13658.6609269,
 'amt_all_q1': 25796796.7907,
 'amt_all_q3': 74648719.5469,
 'amt_all_softNoise': 147926603.68120003,
 'amt_all_hardNoise': 221204487.81550002,
 'amt_all_lastAmt': 86556196.8779,
 'balance_amt_all_avg': 49543170.957238674,
 'balance_amt_all_max': 99978607.4561,
 'balance_amt_all_min': 13517.9560216,
 'balance_amt_all_q1': 24831926.517075,
 'balance_amt_all_q3': 74035816.1059,
 'balance_amt_all_softNoise': 147841650.4891375,
 'balance_amt_all_hardNoise': 221647484.872375,
 'balance_amt_all_lastAmt': 33280122.6462,
 'u_time_all_lastTime': 1488324413,
 'u_time_all_avg': 2193.735615701739,
 'u_time_all_max': 43695.0,
 'u_time_all_min': 0.0,
 'u_time_all_q1': 0.0,
 'u_time_all_q3': 3208.0,
 'u_time_all_softNoise': 8020.0,
 'u_time_all_hardNoise': 12832.0}

In [14]:
#列名
col_list = [
    'cust_id', 
    'amt_all_avg', 
    'amt_all_max', 
    'amt_all_min', 
    'amt_all_q1', 
    'amt_all_q3', 
    'amt_all_softNoise', 
    'amt_all_hardNoise', 
    'amt_all_lastAmt', 
    'balance_amt_all_avg', 
    'balance_amt_all_max', 
    'balance_amt_all_min', 
    'balance_amt_all_q1', 
    'balance_amt_all_q3', 
    'balance_amt_all_softNoise', 
    'balance_amt_all_hardNoise', 
    'balance_amt_all_lastAmt', 
    'u_time_all_lastTime', 
    'u_time_all_avg', 
    'u_time_all_max', 
    'u_time_all_min', 
    'u_time_all_q1', 
    'u_time_all_q3', 
    'u_time_all_softNoise', 
    'u_time_all_hardNoise']

In [15]:
#rdd执行自定义函数
rdd_history = ori_trx_new_history.where("cust_id='cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272'").limit(2).rdd.mapPartitions(lambda x: history_mp_fun(x))


#### 生成新的Dataframe

In [16]:
#生成新的df
userAct_history = spark.createDataFrame(rdd_history,col_list)

In [17]:
userAct_history.collect()

[Row(cust_id='cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272', amt_all_avg=50136866.52888159, amt_all_max=99997313.9013, amt_all_min=13658.6609269, amt_all_q1=25796796.7907, amt_all_q3=74648719.5469, amt_all_softNoise=147926603.68120003, amt_all_hardNoise=221204487.81550002, amt_all_lastAmt=86556196.8779, balance_amt_all_avg=49543170.957238674, balance_amt_all_max=99978607.4561, balance_amt_all_min=13517.9560216, balance_amt_all_q1=24831926.517075, balance_amt_all_q3=74035816.1059, balance_amt_all_softNoise=147841650.4891375, balance_amt_all_hardNoise=221647484.872375, balance_amt_all_lastAmt=33280122.6462, u_time_all_lastTime=1488324413, u_time_all_avg=2193.735615701739, u_time_all_max=43695.0, u_time_all_min=0.0, u_time_all_q1=0.0, u_time_all_q3=3208.0, u_time_all_softNoise=8020.0, u_time_all_hardNoise=12832.0)]

In [18]:
len(userAct_history.columns)

25